# Deep Learning Analyses - Motor Intention Decoding from 2D EEG Data Formulation

## Exploratory Data Analysis

### Dataset Infos

Esatto, quello che stai guardando – EEGMMIDB_Curated.mat – è il “contenitore” principale già pronto all’uso:

all’interno del file .mat non troverai tante sottocartelle sul disco, ma una struttura MATLAB chiamata EEGMMIDB con questi campi:

matlab

EEGMMIDB
├─ Frequency          % 160 Hz
├─ Subjects           % mappa vecchi→nuovi ID (103 soggetti)
├─ Signal             % 103 celle: Subject_1_Signal … Subject_103_Signal
└─ Annotations        % 103 celle: Subject_1_Annotations … Subject_103_Annotations


Ogni cella Subject_x_Signal contiene 12 matrici (una per run); lo stesso vale per Annotations. 
Quindi i dati di tutti i soggetti e di tutte le run sono già organizzati lì dentro, non in singole cartelle. 


Il paper spiega che, oltre al .mat, esiste anche la versione CSV compressa in un archivio WinRAR: eegmmidb.rar, con 2 472 file chiamati
SUB_[sss]_SIG_[rr].csv e SUB_[sss]_ANN_[rr].csv (uno per segnale, uno per annotazioni, 12×103 soggetti). 
PMC

Nel tuo zip hai effettivamente:

vbnet

Physionet EEGMMIDB in MATLAB structure and CSV files to leverage accessibility and exploitation/
├─ MATLAB structure/EEGMMIDB_Curated.mat
└─ CSV files/eegmmidb.rar          ← qui dentro ci sono tutti i CSV



Cosa fare adesso
Se ti basta lavorare in Python o MATLAB direttamente sul .mat:
In Jupyter puoi caricarlo così:

python
Copia
Modifica
import scipy.io as sio
data = sio.loadmat('EEGMMIDB/Physionet ... /MATLAB structure/EEGMMIDB_Curated.mat',
                   simplify_cells=True)   # opzionale, per avere dict “puliti”

print(data.keys())        # ['EEGMMIDB']
sig   = data['EEGMMIDB']['Signal']        # lista di 103 soggetti
ann   = data['EEGMMIDB']['Annotations']   # idem
Da lì estrai run, epoche, ecc. senza bisogno di altri file.

Se vuoi anche i CSV (utile per pipeline Pandas, MNE, ecc.):

bash

#Crea una cartella d’uscita
mkdir -p EEGMMIDB/csv

#Estrai con 7‑Zip (più robusto degli spazi): metti il percorso tra virgolette
7z x "EEGMMIDB/Physionet EEGMMIDB in MATLAB structure and CSV files to leverage accessibility and exploitation/CSV files/eegmmidb.rar" \
   -oEEGMMIDB/csv
   
In alternativa, dopo aver installato unrar:

bash
Copia
Modifica
unrar x "…/eegmmidb.rar"  EEGMMIDB/csv/
Finirai con tutti i file SUB_001_SIG_01.csv, SUB_001_ANN_01.csv, … fino a SUB_103_ANN_12.csv.

Riassumendo
.mat → unica struttura già indicizzata per soggetto/run: subito pronta per la decodifica.

.rar → stessa informazione ma in tanti CSV; estraila solo se ti serve un formato tabellare.

Scegli la via più comoda per la tua pipeline (Python + scipy/io per il .mat, oppure Pandas/MNE sui CSV).

### EEGMMIDB Dataset EEG (link for papers e dataset)

Link Paper Dataset: https://pmc.ncbi.nlm.nih.gov/articles/PMC10998040/pdf/main.pdf

Link Curated Dataset: https://data.mendeley.com/datasets/dpmtgrn8d8

Link Original Dataset: https://physionet.org/content/eegmmidb/1.0.0/

Link EEG Cap Distribution: https://physionet.org/content/eegmmidb/1.0.0/64_channel_sharbrough.pdf

In [ ]:
import mne 

In [ ]:
import mne
raw = mne.io.read_raw_edf('/home/stefano/EEGMMIDB_Analyses/S001R01.edf', preload=False)
print(raw.ch_names)

In [ ]:
EEG_channels_names = raw.ch_names

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm

import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la isualizzazione dei dati
from sklearn.model_selection import train_test_split

#import torch
#import torch.nn as nn

#import torch.optim as optim
#from torch.utils.data import TensorDataset, DataLoader

In [ ]:
EEG_path = '/home/stefano/EEGMMIDB/Physionet EEGMMIDB in MATLAB structure and CSV files to leverage accessibility and exploitation'
matlab_files = 'MATLAB structure/EEGMMIDB_Curated.mat'

mat_path = os.path.join(EEG_path, matlab_files)

In [ ]:
'''
Il risultato (sig, ann, ecc.) è esattamente quello che avresti avuto con simplify_cells=True:
liste native Python / array NumPy, niente mat_struct o object arrays.
'''

import mat73

data = mat73.loadmat(mat_path)   # ← già “semplificato" dict principale


In [ ]:
type(data)

In [ ]:
print(data.keys())
print(data['EEGMMIDB'].keys())

In [ ]:
print(data['EEGMMIDB'].keys())

In [ ]:
print(type(data['EEGMMIDB']['Annotations']))
print(type(data['EEGMMIDB']['Annotations']))
print(type(data['EEGMMIDB']['Signal']))
print(type(data['EEGMMIDB']['Subjects']))

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_data.pkl', 'rb') as f:
    EEG_data = pickle.load(f)

In [ ]:
type(EEG_data)

In [ ]:
EEG_data.keys()

In [ ]:
EEG_data['EEGMMIDB'].keys()

In [ ]:
signal  = EEG_data['EEGMMIDB']['Signal']                 # lista (len=103) di soggetti
labels  = EEG_data['EEGMMIDB']['Annotations']            # idem
fs      = EEG_data['EEGMMIDB']['Frequency']              # 160
subject_ids = EEG_data['EEGMMIDB']['Subjects']           # mapping vecchi→nuovi ID

### Exploratory Data Analysis: Inspection EEG Dataset Organization 

In [ ]:
type(signal)

In [ ]:
len(signal)

In [ ]:
print(len(signal[0]))
print(len(signal[1]))
print()
print('... e così via per 12 indici')


In [ ]:
type(signal[0])

In [ ]:
len(signal[0])

In [ ]:
signal[0].keys()

In [ ]:
type(signal[0]['Subject_100_Signal'])

In [ ]:
len(signal[0]['Subject_100_Signal'])

In [ ]:
signal[0]['Subject_100_Signal'][0].shape

In [ ]:
'''
Le strutture di Signal è una struttura così composta

Signal -> lista di 12 elementi, ciascuna riferita ai dati di una certa run.
        
        -> Ogni elemento (ogni run) contiene i dati di una certa run di tutti e 103 i soggetti
        
            -> Per accedere ai dati di una specifica run di uno specifico soggetto (contenuti dentro una lista -> len(signal[0]) = 103)
                bisogna indicare una specifica chiave relativa ad un dizionario dello specifico soggetto (i.e., type(signal[0]) --> dict)
            
                --> ESEMPIO: signal[0]['Subject_100_Signal']
                
                    --> Dentro al dizionario di una run di uno specifico soggetto, c'è una lista 
                        (i.e., type(signal[0]['Subject_100_Signal']) = list), 
                        
                        Composta di un elemento solo 
                        (i.e., len(signal[0]['Subject_100_Signal']) = 1)
                        
                        --> Per accedere proprio all'array dei dati, devi indicizzare ulteriormente 
                            
                            signal[0]['Subject_100_Signal'][0].shape = (19680, 64)
                        

'''



print(type(signal))
print()
print(type(signal[0]))
print()
print(signal[0].keys())

In [ ]:
# Questo contiene la lista dei valori credo della run del relativo soggetto...
print(type(signal[0]['Subject_100_Signal']))
print()
print(type(labels[0]['Subject_100_Annotations']))

In [ ]:
#Questo valore dovrebbe corrispondere al numero di campioni dell'intera run di 2 minuti descritta dal paper...

#https://pmc.ncbi.nlm.nih.gov/articles/PMC10998040/pdf/main.pdf


print(len(signal[0]['Subject_100_Signal'][0]))
print()
print(len(labels[0]['Subject_100_Annotations'][0]))

Hai quasi centrato il punto, ma non è esattamente un “errore” – è solo che il file che stai caricando contiene qualche secondo in più 
oltre ai 120 s previsti dal protocollo.

<br>

**Cosa dice il paper (e la documentazione originale)**


| Fonte | Campionamento | Durata (nominale/effettiva) | # campioni |
|-------|---------------|-----------------------------|------------|
| Paper di curation, Tab. 1 (PMC) | 160 Hz | 2 min (= 120 s) | 160 × 120 = 19 200 |
| Scheda del dataset su PhysioNet (physionet.org) | 160 Hz | “two‑minute runs” | 19 200 |
| Quello che vedi tu | 160 Hz | 123 s (≈ 2 min + 3 s) | 19 680 |

<br>

**Da dove arrivano quei 3 secondi?**

Nei file EDF originali il sistema BCI2000 lascia un piccolo buffer prima che appaia il primo target e dopo la scomparsa dell’ultimo 
(tipicamente 1–2 s all’inizio + 1 s alla fine).

Gli autori della curation non li hanno tagliati: in Fig. 5 del paper si vede infatti che la “Total length of each run” spesso 
supera la durata indicata nelle annotazioni .

<br>

**Quindi la risposta alla tua domanda**
«…dovrebbe corrispondere al numero di campioni dell’intera run di 2 minuti descritta dal paper.. giusto?»

In teoria la run “utile” è di 19 200 campioni.

In pratica il file che hai in RAM contiene anche il buffer, perciò trovi 19 680 campioni.


<br>

**Come gestirlo**

fs = 160          # Hz
run_nominale = 120
n_teorico = fs * run_nominale            # 19200
n_osservato = 19680
extra = n_osservato - n_teorico          # 480 campioni ≈ 3 s

#Se vuoi esattamente 2 minuti:
trim = extra // 2                        # 240 campioni da tagliare all’inizio e alla fine
clean_run = raw_signal[trim:-trim]       # shape: (19200, 64)
Usa le annotazioni (T0/T1/T2) se vuoi essere sicuro di includere solo le prove effettive; taglia pure di conseguenza invece di assumere sempre 240 campioni.

Così il tuo array avrà davvero **19 200 campioni**, coerenti con quanto scritto nel paper.

### Info delle Runs del Dataset EEG

Di seguito trovi la mappa “ufficiale” che il paper (Tabella 2) assegna ai 12 run curati.

Nel tuo array signal[subj][run_idx] (e in labels), l’indice Python run_idx va da 0 a 11 e corrisponde esattamente
ai numeri di run 1 – 12 indicati nella colonna più a destra della tabella.


| run_idx (Python) | # run curato | TASK                                                                       | Modalità | Tipo mov.   | Classi presenti nel run (T0, T1, T2)                   |
|------------------|--------------|----------------------------------------------------------------------------|----------|-------------|--------------------------------------------------------|
| 0                | 1            | Motor **Execution** – “open/close *unilateral* fist”                        | ME       | Unilaterale | Relax · **Left fist** · **Right fist**                 |
| 1                | 2            | idem (2ᵃ ripetizione)                                                      | ME       | Unil.       | Relax · Left fist · Right fist                         |
| 2                | 3            | idem (3ᵃ ripetizione)                                                      | ME       | Unil.       | Relax · Left fist · Right fist                         |
| 3                | 4            | Motor **Imagery** – “imagine open/close *unilateral* fist”                  | MI       | Unil.       | Relax · Left fist · Right fist                         |
| 4                | 5            | idem (2ᵃ)                                                                  | MI       | Unil.       | Relax · Left fist · Right fist                         |
| 5                | 6            | idem (3ᵃ)                                                                  | MI       | Unil.       | Relax · Left fist · Right fist                         |
| 6                | 7            | Motor **Execution** – “open/close *bilateral* fists & feet”                 | ME       | Bilat.      | Relax · **Both fists** · **Both feet**                |
| 7                | 8            | idem (2ᵃ)                                                                  | ME       | Bilat.      | Relax · Both fists · Both feet                         |
| 8                | 9            | idem (3ᵃ)                                                                  | ME       | Bilat.      | Relax · Both fists · Both feet                         |
| 9                | 10           | Motor **Imagery** – “imagine open/close *bilateral* fists & feet”           | MI       | Bilat.      | Relax · Both fists · Both feet                         |
| 10               | 11           | idem (2ᵃ)                                                                  | MI       | Bilat.      | Relax · Both fists · Both feet                         |
| 11               | 12           | idem (3ᵃ)                                                                  | MI       | Bilat.      | Relax · Both fists · Both feet                         |


## EEG Preprocessing

### Creazione dei Vari Datasets: Notch 60 Hz, Filtro Passa-Banda 1-45 Hz + PyPrep + Custom ICA + Epoching + Autoreject

- Task 1: Motor Execution (Open and close right or left fist) - Unilateral - Runs 3, 7, 11 
- Task 2: Motor Imagery (Imagine opening and closing right and left fists) - Unilateral - Runs 4, 8, 12 
- Task 3: Motor Execution (Open and close both fists and feet) - Bilateral - Runs 5, 9, 13 
- Task 4: Motor Imagery (Imagine opening and closing both fists and feet) - Bilateral - Runs 6, 10, 14 

#### **Canali Physionet Dataset == Canali dataset EEGMMIDB !** 

#### **(N.B. Canali dataset EEGMMIDB = gli stessi ma sistemati con capitalizzazione giusta)**

In [ ]:
import mne

# Percorso del file EDF scaricato dal dataset originale (modifica se necessario)
raw = mne.io.read_raw_edf('/home/stefano/EEGMMIDB_Analyses/S001R01.edf', preload=False, verbose=False)

# Estrae i nomi dei canali (elettrodi EEG)
channel_names = raw.info['ch_names']

# Stampa ordinata
print("Lista dei canali trovati ORIGINALI nel file:")
for ch in channel_names:
    print(ch, end = " ")    

In [ ]:
''' 

✅ Soluzione: normalizza il nome dei canali EEG prima di metterli dentro la configurazione standard 10_20
secondo la configurazione voluta da MNE

Serve una funzione di normalizzazione intelligente, che:

rimuove i punti (.),

converte tutto in maiuscolo eccetto la lettera z finale,
che deve rimanere minuscola se il canale è uno di quelli "centrali" con z (Fz, Cz, Pz, Oz, Fpz, Poz, Afz, ecc.).


In particolare,la tua funzione normalize_ch_names(...) deve preservare correttamente la capitalizzazione ufficiale MISTA, ovvero:

MNE usa nomi capitalizzati solo parzialmente, ad esempio:

Tuo input	Nome giusto in standard_1020
'Fcz'	❌ non esiste ('FCz' sì!)
'Cpz'	❌ non esiste ('CPz' sì!)
'Afz'	❌ non esiste ('AFz' sì!)
'FP1'	✅ OK
'FP2'	✅ OK
'Poz'	✅ OK


'''


def normalize_ch_names(ch_list):
    
    # Lista dei nomi dei canali standard (inclusi nel 10-20 ufficiale di MNE)
    montage_ch_names = set(mne.channels.make_standard_montage('standard_1020').ch_names)

    corrected = []
    for ch in ch_list:
        ch_clean = ch.replace('.', '')  # rimuovi i punti
        ch_upper = ch_clean.upper()

        # Trova il match nel montage standard (case-insensitive)
        match = next((std_ch for std_ch in montage_ch_names if std_ch.upper() == ch_upper), None)

        if match:
            corrected.append(match)
        else:
            print(f"[WARN] Canale non trovato nel montage: {ch_clean}")
            corrected.append(ch_clean)  # fallback
    return corrected


EEG_channels = normalize_ch_names(channel_names)

In [ ]:
print(EEG_channels)

#### **Sintesi - Struttura dei Dati EEG & Labels Summary**

In [ ]:
'''
La stuttura dati di partenza per dati e labels è 

#signal  = EEG_data['EEGMMIDB']['Signal']                 # lista (len=103) di soggetti
#labels  = EEG_data['EEGMMIDB']['Annotations']            # idem


Signal -> lista di 12 elementi, ciascuna riferita ai dati di una certa run.
        
        -> Ogni elemento (ogni run) contiene i dati di una certa run di tutti e 103 i soggetti
        
            -> Per accedere ai dati di una specifica run di uno specifico soggetto (contenuti dentro una lista -> len(signal[0]) = 103)
                bisogna indicare una specifica chiave relativa ad un dizionario dello specifico soggetto (i.e., type(signal[0]) --> dict)
            
                --> ESEMPIO: signal[0]['Subject_100_Signal']
                
                    --> Dentro al dizionario di una run di uno specifico soggetto, c'è una lista 
                        (i.e., type(signal[0]['Subject_100_Signal']) = list), 
                        
                        Composta di un elemento solo 
                        (i.e., len(signal[0]['Subject_100_Signal']) = 1)
                        
                        --> Per accedere proprio all'array dei dati, devi indicizzare ulteriormente 
                            
                            signal[0]['Subject_100_Signal'][0].shape = (19680, 64)
                            
#dict_keys(['Subject_100_Signal', 'Subject_101_Signal', 'Subject_102_Signal', 'Subject_103_Signal', 'Subject_10_Signal', 'Subject_11_Signal', 
            #'Subject_12_Signal', 'Subject_13_Signal', 'Subject_14_Signal', 'Subject_15_Signal', 'Subject_16_Signal', 'Subject_17_Signal', 
            #'Subject_18_Signal', 'Subject_19_Signal', 'Subject_1_Signal', 'Subject_20_Signal', 'Subject_21_Signal', 'Subject_22_Signal', 
            #'Subject_23_Signal', 'Subject_24_Signal', 'Subject_25_Signal', 'Subject_26_Signal', 'Subject_27_Signal', 'Subject_28_Signal',
            #'Subject_29_Signal', 'Subject_2_Signal', 'Subject_30_Signal', 'Subject_31_Signal', 'Subject_32_Signal', 'Subject_33_Signal', 
            #'Subject_34_Signal', 'Subject_35_Signal', 'Subject_36_Signal', 'Subject_37_Signal', 'Subject_38_Signal', 'Subject_39_Signal', 
            #'Subject_3_Signal', 'Subject_40_Signal', 'Subject_41_Signal', 'Subject_42_Signal', 'Subject_43_Signal', 'Subject_44_Signal', 
            #'Subject_45_Signal', 'Subject_46_Signal', 'Subject_47_Signal', 'Subject_48_Signal', 'Subject_49_Signal', 'Subject_4_Signal', 
            #'Subject_50_Signal', 'Subject_51_Signal', 'Subject_52_Signal', 'Subject_53_Signal', 'Subject_54_Signal', 'Subject_55_Signal', 
            #'Subject_56_Signal', 'Subject_57_Signal', 'Subject_58_Signal', 'Subject_59_Signal', 'Subject_5_Signal', 'Subject_60_Signal', 
            #'Subject_61_Signal', 'Subject_62_Signal', 'Subject_63_Signal', 'Subject_64_Signal', 'Subject_65_Signal', 'Subject_66_Signal',
            #'Subject_67_Signal', 'Subject_68_Signal', 'Subject_69_Signal', 'Subject_6_Signal', 'Subject_70_Signal', 'Subject_71_Signal', 
            #'Subject_72_Signal', 'Subject_73_Signal', 'Subject_74_Signal', 'Subject_75_Signal', 'Subject_76_Signal', 'Subject_77_Signal',
            #'Subject_78_Signal', 'Subject_79_Signal', 'Subject_7_Signal', 'Subject_80_Signal', 'Subject_81_Signal', 'Subject_82_Signal', 
            #'Subject_83_Signal', 'Subject_84_Signal', 'Subject_85_Signal', 'Subject_86_Signal', 'Subject_87_Signal', 'Subject_88_Signal', 
            #'Subject_89_Signal', 'Subject_8_Signal', 'Subject_90_Signal', 'Subject_91_Signal', 'Subject_92_Signal', 'Subject_93_Signal', 
            #'Subject_94_Signal', 'Subject_95_Signal', 'Subject_96_Signal', 'Subject_97_Signal', 'Subject_98_Signal', 'Subject_99_Signal',
            #'Subject_9_Signal'])
            
            
Labels -> lista di 12 elementi, ciascuna riferita ai dati di una certa run.
        
        -> Ogni elemento (ogni run) contiene le labels di una certa run di tutti e 103 i soggetti
        
            -> Per accedere alle labels di una specifica run di uno specifico soggetto (contenuti dentro una lista -> len(labels[0]) = 103)
                bisogna indicare una specifica chiave relativa ad un dizionario dello specifico soggetto (i.e., type(labels[0]) --> dict)
            
                --> ESEMPIO: labels[0]['Subject_100_Annotations']
                
                    --> Dentro al dizionario di una run di uno specifico soggetto, c'è una lista 
                        (i.e., type(labels[0]['Subject_100_Annotations']) = list), 
                        
                        Composta di un elemento solo 
                        (i.e., len(labels[0]['Subject_100_Annotations']) = 1)
                        
                        --> Per accedere proprio all'array della labels della SPECIFICA RUN dello SPECIFICO SOGGETTO,
                            devi indicizzare ulteriormente!
                            
                            In questo modo, dovresti avere tutti e 30 i trial della specifica run, ciascuno composto da 5 campi
                            
                            
                            
                            type(labels[0]['Subject_100_Annotations'][0]) -> list
                            
                            Output:
                            
                            [[array(1.), array(4.1), array(656.), array(1.), array(656.)],
                             [array(3.), array(4.1), array(656.), array(657.), array(1312.)],
                             [array(1.), array(4.1), array(656.), array(1313.), array(1968.)],
                             [array(2.), array(4.1), array(656.), array(1969.), array(2624.)],
                             [array(1.), array(4.1), array(656.), array(2625.), array(3280.)],
                             [array(3.), array(4.1), array(656.), array(3281.), array(3936.)],
                             [array(1.), array(4.1), array(656.), array(3937.), array(4592.)],
                             [array(2.), array(4.1), array(656.), array(4593.), array(5248.)],
                             [array(1.), array(4.1), array(656.), array(5249.), array(5904.)],
                             [array(2.), array(4.1), array(656.), array(5905.), array(6560.)],
                             [array(1.), array(4.1), array(656.), array(6561.), array(7216.)],
                             [array(3.), array(4.1), array(656.), array(7217.), array(7872.)],
                             [array(1.), array(4.1), array(656.), array(7873.), array(8528.)],
                             [array(2.), array(4.1), array(656.), array(8529.), array(9184.)],
                             [array(1.), array(4.1), array(656.), array(9185.), array(9840.)],
                             [array(3.), array(4.1), array(656.), array(9841.), array(10496.)],
                             [array(1.), array(4.1), array(656.), array(10497.), array(11152.)],
                             [array(2.), array(4.1), array(656.), array(11153.), array(11808.)],
                             [array(1.), array(4.1), array(656.), array(11809.), array(12464.)],
                             [array(3.), array(4.1), array(656.), array(12465.), array(13120.)],
                             [array(1.), array(4.1), array(656.), array(13121.), array(13776.)],
                             [array(2.), array(4.1), array(656.), array(13777.), array(14432.)],
                             [array(1.), array(4.1), array(656.), array(14433.), array(15088.)],
                             [array(3.), array(4.1), array(656.), array(15089.), array(15744.)],
                             [array(1.), array(4.1), array(656.), array(15745.), array(16400.)],
                             [array(3.), array(4.1), array(656.), array(16401.), array(17056.)],
                             [array(1.), array(4.1), array(656.), array(17057.), array(17712.)],
                             [array(2.), array(4.1), array(656.), array(17713.), array(18368.)],
                             [array(1.), array(4.1), array(656.), array(18369.), array(19024.)],
                             [array(3.), array(4.1), array(656.), array(19025.), array(19680.)]]
                            
                        
                        --> Per accedere proprio all'array della labels di uno SPECIFICO TRIAL di quella SPECIFICA RUN, devi indicizzare ulteriormente 
                            (se ne vuoi prendere proprio uno specifico!) 
                            
                            type(labels[0]['Subject_100_Annotations'][0][0]) -> list
                            
                            Output:
                            
                            labels[0]['Subject_100_Annotations'][0][0] ->  [array(1.), array(4.1), array(656.), array(1.), array(656.)]
                            
                            
'''

In [ ]:
type(signal[0]['Subject_100_Signal'])


# Questo contiene la lista dei valori della run del relativo soggetto...

#Questa per i dati

#A questo livello prendo la run del soggetto 100 ed è una lista

print(type(signal[0]['Subject_100_Signal']))

#Una lista di un elemento solo (ossia, tutto il reading di una run intera)

#Questo valore dovrebbe corrispondere al numero di campioni dell'intera run di 2 minuti descritta dal paper...

#--> https://pmc.ncbi.nlm.nih.gov/articles/PMC10998040/pdf/main.pdf

print(len(signal[0]['Subject_100_Signal']))


#Vedendo il singolo elemento della lista, ossia tutto il reading di una run intera
#So che questo sia un'array!
print(type(signal[0]['Subject_100_Signal'][0]))

#Se voglio prendere solo la prima dimensione dell'array, e sapere la sua lunghezza so che è pari al numero tot campioni EEG dell'intera run!
print(len(signal[0]['Subject_100_Signal'][0]))

#Se voglio prendere l'intera shape dell'array, vedo che l'intero EEG reading ha come dimensioni: 
print(signal[0]['Subject_100_Signal'][0].shape)
    #dim 1: length totale della run
    #dim 2: 64 (elettrodi usati durante la rec) 

print()


#Questa per le labels

print(type(labels[0]['Subject_100_Annotations']))

#Se voglio sapere la lunghezza delle annotazioni so che la sua lunghezza è pari al numero di trials componibili in base alla lunghezza dell'intera run
print(len(labels[0]['Subject_100_Annotations'][0]))

#Per ogni trial invece della run del relativo soggetto, so che ogni elemento è composto da 5 campi, che si riferiscono a:

#label usata
#durata in secondi
#duranta in termini di sample
#indice di inizio del sample
#indice di fine del sample

print(len(labels[0]['Subject_100_Annotations'][0][0]))

#Per accedere al valore di ognuno dei 5 campi, vedo prima il suo tipo
print(type(labels[0]['Subject_100_Annotations'][0][0][0]))
      
#Poi, accedo al suo valore
print(labels[0]['Subject_100_Annotations'][0][0][0])

#### **EEG Proprocessing - Filtering Notch 60 Hz and IIR 1-45 Hz**

In [ ]:
import mne 

In [ ]:
import mne
raw = mne.io.read_raw_edf('/home/stefano/EEGMMIDB_Analyses/S001R01.edf', preload=False)
print(raw.ch_names)

channel_names = raw.ch_names
print()
print(type(channel_names))
print()
print(channel_names)

In [ ]:
''' 

✅ Soluzione: normalizza il nome dei canali EEG prima di metterli dentro la configurazione standard 10_20
secondo la configurazione voluta da MNE

Serve una funzione di normalizzazione intelligente, che:

rimuove i punti (.),

converte tutto in maiuscolo eccetto la lettera z finale,
che deve rimanere minuscola se il canale è uno di quelli "centrali" con z (Fz, Cz, Pz, Oz, Fpz, Poz, Afz, ecc.).


In particolare,la tua funzione normalize_ch_names(...) deve preservare correttamente la capitalizzazione ufficiale MISTA, ovvero:

MNE usa nomi capitalizzati solo parzialmente, ad esempio:

Tuo input	Nome giusto in standard_1020
'Fcz'	❌ non esiste ('FCz' sì!)
'Cpz'	❌ non esiste ('CPz' sì!)
'Afz'	❌ non esiste ('AFz' sì!)
'FP1'	✅ OK
'FP2'	✅ OK
'Poz'	✅ OK


'''


def normalize_ch_names(ch_list):
    
    # Lista dei nomi dei canali standard (inclusi nel 10-20 ufficiale di MNE)
    montage_ch_names = set(mne.channels.make_standard_montage('standard_1020').ch_names)

    corrected = []
    for ch in ch_list:
        ch_clean = ch.replace('.', '')  # rimuovi i punti
        ch_upper = ch_clean.upper()

        # Trova il match nel montage standard (case-insensitive)
        match = next((std_ch for std_ch in montage_ch_names if std_ch.upper() == ch_upper), None)

        if match:
            corrected.append(match)
        else:
            print(f"[WARN] Canale non trovato nel montage: {ch_clean}")
            corrected.append(ch_clean)  # fallback
    return corrected


EEG_channels = normalize_ch_names(channel_names)

In [ ]:
import numpy as np
import pickle
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Tuple
import mne
import time 


from tqdm import tqdm  # Assicurati di avere tqdm installato


# --- Configurazioni globali ---

#ICA_MODELS_PATH  = Path("/home/stefano/EEGMMIDB_Analyses/ica_models.pkl")

CHECKPOINT_PATH = Path("/home/stefano/EEGMMIDB_Analyses/task_data_ICA.pkl")

CHECKPOINT_PATH.parent.mkdir(exist_ok=True, parents=True)

# === Lista dei 64 canali compatibili con il dataset Mendeley ===

'''NORMALIZZO I CANALI EEG (GIA' FATTO SOPRA)
EEG_channels = normalize_ch_names(channel_names)
'''

# Mappatura Task → indice run (0-based)
TASK_RUNS = {
    "task_1": [0, 4, 8],
    "task_2": [1, 5, 9],
    "task_3": [2, 6, 10],
    "task_4": [3, 7, 11],
}

def label_to_Tx(label_num: int) -> str:
    return ["T0", "T1", "T2"][(label_num - 1) % 3]

def to_plain_dict(d):
    if isinstance(d, dict) or isinstance(d, defaultdict):
        return {k: to_plain_dict(v) for k, v in d.items()}
    return d

'''
Dove cercare ESATTAMENTE questa cosa che il dato vada convertito in Volt?


vai qui --> https://mne.tools/stable/generated/mne.io.RawArray.html?utm_source=chatgpt.com 

al metodo 'get_data', vai nella sua sezione del parametro "units", dove dice 

unitsstr | dict | None
Specify the unit(s) that the data should be returned in. 
If None (default), the data is returned in the channel-type-specific default units, 
which are SI units (see Internal representation (units) and data channels).

A quel punto clicca sul link di "internal units"
(https://mne.tools/stable/documentation/implementation.html#units) 

E poi andando nella sezione che si apre cercando la tabella "Supported channel types" e vedendo che, su riga "eeg" compare 'Volts"
'''



# 1) Estrae RawArray filtrato e con solo i 64 canali Mendeley
def get_raw_for_run(
    sig_arr: np.ndarray,
    fs: int,
    channel_names: List[str]
) -> mne.io.Raw:
    
    # Filtra solo i canali ufficiali
    keep_idx = [i for i, ch in enumerate(channel_names) if ch in channel_names]
    
    sig_arr = sig_arr * 1e-6  # Converti da µV a Volt
    
    
    sig_arr = sig_arr[:, keep_idx]
    chs = [channel_names[i] for i in keep_idx]

    # Crea RawArray
    info = mne.create_info(ch_names=chs, sfreq=fs, ch_types='eeg')
    raw = mne.io.RawArray(sig_arr.T, info)
    
    # Costruisci montage ridotto
    
    '''
    Carichi il montaggio standard (standard_1020) che contiene le posizioni 3D per tutti i canali noti.
    Filtri queste posizioni con --> if ch in EEG_channels
    
    quindi mantieni solo le posizioni dei canali effettivamente presenti nei tuoi dati .mat, confrontando i nomi.
    
    Crei un nuovo montaggio personalizzato con solo quei canali tramite make_dig_montage.
    Lo assegni al tuo oggetto raw.

    Quindi sì, questo approccio:
    Si assicura che i nomi dei tuoi canali matchino esattamente con quelli nel montaggio standard.
    Evita errori nel caso tu abbia un sottoinsieme (come i 64 canali della Mendeley EEG dataset).
    È robusto per dati che non vengono direttamente da MNE (come i tuoi .mat) perché ti consente un controllo esplicito sulla selezione dei canali.

    '''
    
    montage = mne.channels.make_standard_montage('standard_1020')
    pos = {ch: pos for ch, pos in montage.get_positions()['ch_pos'].items() if ch in EEG_channels}
    montage_eegmidb = mne.channels.make_dig_montage(ch_pos=pos, coord_frame='head')
    raw.set_montage(montage_eegmidb)
    
    iir_params = dict(order = 2, ftype='butter', output = 'sos')
    
    raw.notch_filter([60], method='iir', iir_params = iir_params)
    
    raw.filter(1.0, 45.0, method ='iir', iir_params = iir_params)

    return raw


'''
VERSIONE GIA' IN FORMATO DICT DIRETTO

✅ Cosa fa:
Estrae i dati da signal e labels (entrambi strutturati come liste di dizionari).

Converte ogni array di segnali (sig_arr) in un oggetto mne.io.RawArray usando get_raw_for_run(...).

Salva:

raw: il segnale EEG PREPROCESATO già in formato MNE.
ann_arr: le annotazioni in formato np.ndarray (30 x 5).

Organizza tutto in una struttura gerarchica raw_collection[task][subject][run] = (raw, ann_arr).


N.B. il metodo get_data() restituisce i dati EEG in formato NumPy array direttamente dal tuo oggetto mne.io.RawArray


'''

# 2) Costruisce struttura RAW (non epocata) per tutti i task/run/soggetto
def build_raw_collection(
    signal: List[Dict[str, list]],
    labels: List[Dict[str, list]],
    fs: int,
    channel_names: List[str]
) -> Dict[str, Dict[str, Dict[int, Tuple[mne.io.Raw, np.ndarray]]]]:
    
    """
    Restituisce raw_collection[task][subject][run] = (raw, annotation_array)
    """
    #raw_collection = defaultdict(lambda: defaultdict(dict))
    
    raw_collection = dict()
    
    # Avvia il timer
    start_time = time.time()
    
    #Il numero totale di soggetto×run è semplicemente la somma del numero di soggetti per ogni run
    
    # Stima del numero totale di soggetti × run → per la progress bar
    total_jobs = sum(len(signal[r]) for runs in TASK_RUNS.values() for r in runs)
    pbar = tqdm(total=total_jobs, desc="Costruzione RAW filtrati", unit="run")
    

    for task, runs in TASK_RUNS.items():
        for r in runs:
            run_sig   = signal[r]
            run_label = labels[r]
            for subj_key, sig_list in run_sig.items():
                subj_id = subj_key.split('_')[1]
                sig_arr = sig_list[0]  # (n_samples, n_channels)
                ann_arr = np.asarray(run_label[subj_key.replace('Signal', 'Annotations')][0])

                raw = get_raw_for_run(sig_arr, fs, channel_names)
                
                if task not in raw_collection:
                    raw_collection[task] = dict()
                    
                if subj_id not in raw_collection[task]:
                    raw_collection[task][subj_id] = dict()
                
                raw_collection[task][subj_id][r] = (raw, ann_arr)
                
                pbar.update(1)
    
    pbar.close()
    
    # Calcola il tempo totale
    end_time = time.time()
    
    # Calcola il tempo totale
    total_time = end_time - start_time
    hours = int(total_time // 3600)
    minutes = int((total_time % 3600) // 60)
    seconds = int(total_time % 60)

    # Stampa il tempo totale in formato leggibile
    print(f"\nTempo totale impiegato: \033[1m{hours} ore, {minutes} minuti e {seconds} secondi\033[0m.\n")

    return raw_collection



# Versione "non filtrata" di get_raw_for_run

def get_raw_for_run_unfiltered(
    sig_arr: np.ndarray,
    fs: int,
    channel_names: List[str]
) -> mne.io.Raw:
    
    # Filtra solo i canali ufficiali
    keep_idx = [i for i, ch in enumerate(channel_names) if ch in channel_names]
    
    sig_arr = sig_arr * 1e-6  # Converti da µV a Volt
    
    
    sig_arr = sig_arr[:, keep_idx]
    chs = [channel_names[i] for i in keep_idx]

    # Crea RawArray
    info = mne.create_info(ch_names=chs, sfreq=fs, ch_types='eeg')
    raw = mne.io.RawArray(sig_arr.T, info)
    
    montage = mne.channels.make_standard_montage('standard_1020')
    pos = {ch: pos for ch, pos in montage.get_positions()['ch_pos'].items() if ch in EEG_channels}
    montage_eegmidb = mne.channels.make_dig_montage(ch_pos=pos, coord_frame='head')
    raw.set_montage(montage_eegmidb)

    return raw






'''VERSIONE GIA' IN FORMATO DICT DIRETTO

✅ Cosa fa:
Estrae i dati da signal e labels (entrambi strutturati come liste di dizionari).

Converte ogni array di segnali (sig_arr) in un oggetto mne.io.RawArray usando get_raw_for_run(...).

Salva:

raw: il segnale EEG NON PROCESSATO già in formato MNE.
ann_arr: le annotazioni in formato np.ndarray (30 x 5).

Organizza tutto in una struttura gerarchica raw_collection[task][subject][run] = (raw, ann_arr).


N.B. il metodo .get_data() restituisce i dati EEG in formato NumPy array direttamente dal tuo oggetto mne.io.RawArray

Ossia raw.get_data() restituisce i dati EEG in formato NumPy array direttamente dal tuo oggetto mne.io.RawArray.

🔍 Cosa ottieni:

data = raw.get_data()
print(data.shape)

→ Restituisce una matrice di shape: (n_channels, n_times)

Ad esempio: (64, 19680)  # 64 canali EEG, 19680 campioni (≈ 123 secondi a 160 Hz)


🧠 Se vuoi l'array trasposto (per n_times × n_channels):

Puoi fare:

data = raw.get_data().T
print(data.shape)  # (n_times, n_channels)

💡 Ricorda:
Il metodo raw.get_data() non restituisce segmenti o epoche — è l'intero segnale continuo.

Se vuoi i singoli trial, devi usare le colonne 3 e 4 di ann_arr per tagliare i segmenti desiderati.

Se vuoi ti aiuto a costruire quella parte 👇 (cioè extract_trials_from_raw(...)).


'''
# 3bis) Costruisce struttura RAW (non epocata) NON filtrata
def build_raw_collection_unfiltered(
    signal: List[Dict[str, list]],
    labels: List[Dict[str, list]],
    fs: int,
    channel_names: List[str]
) -> Dict[str, Dict[str, Dict[int, Tuple[mne.io.Raw, np.ndarray]]]]:
    
    """
    Restituisce raw_collection_unfiltered[task][subject][run] = (raw, annotation_array)
    """
    #raw_collection_unfiltered = defaultdict(lambda: defaultdict(dict))
    
    raw_collection_unfiltered = dict()
    
    # Avvia il timer
    start_time = time.time()
    
    #Il numero totale di soggetto×run è semplicemente la somma del numero di soggetti per ogni run
    
    # Stima del numero totale di soggetti × run → per la progress bar
    total_jobs = sum(len(signal[r]) for runs in TASK_RUNS.values() for r in runs)
    pbar = tqdm(total=total_jobs, desc="Costruzione RAW non filtrati", unit="run")
    
    
    for task, runs in TASK_RUNS.items():
        for r in runs:
            run_sig   = signal[r]
            run_label = labels[r]
            
            for subj_key, sig_list in run_sig.items():
                subj_id = subj_key.split('_')[1]
                sig_arr = sig_list[0]  # (n_samples, n_channels)
                ann_arr = np.asarray(run_label[subj_key.replace('Signal', 'Annotations')][0])

                raw = get_raw_for_run_unfiltered(sig_arr, fs, channel_names)
                
                if task not in raw_collection_unfiltered:
                    raw_collection_unfiltered[task] = dict()
                    
                if subj_id not in raw_collection_unfiltered[task]:
                    raw_collection_unfiltered[task][subj_id] = dict()
                    
                raw_collection_unfiltered[task][subj_id][r] = (raw, ann_arr)
                
                pbar.update(1)
    
    pbar.close()
    
    # Calcola il tempo totale
    end_time = time.time()
    
    # Calcola il tempo totale
    total_time = end_time - start_time
    hours = int(total_time // 3600)
    minutes = int((total_time % 3600) // 60)
    seconds = int(total_time % 60)

    # Stampa il tempo totale in formato leggibile
    print(f"\nTempo totale impiegato: \033[1m{hours} ore, {minutes} minuti e {seconds} secondi\033[0m.\n")
    
    return raw_collection_unfiltered



In [ ]:
'''
RUN PULITE
'''
raw_collection = build_raw_collection(
    signal=signal,
    labels=labels,
    fs=160,
    channel_names=EEG_channels  # <-- la lista normalizzata
)

In [ ]:
'''
RUN NON PULITE
'''
raw_unfiltered_collection = build_raw_collection_unfiltered(
    signal=signal,
    labels=labels,
    fs=160,
    channel_names = EEG_channels  # <-- la lista normalizzata
)

In [ ]:
print('finito')

In [ ]:
'''Salvo strutture dati per 

1) Dati Filtrati
2) Dati Non Filtrati (puri raw)

'''

import pickle

# Salvare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/'

with open(f'{path}EEG_only_filtered_data.pkl', 'wb') as f:
    pickle.dump(raw_collection, f) 
    
with open(f'{path}EEG_unfiltered_data.pkl', 'wb') as f:
    pickle.dump(raw_unfiltered_collection, f) 
    

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_only_filtered_data.pkl', 'rb') as f:
    EEG_data_filtered = pickle.load(f)
    
    
# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_unfiltered_data.pkl', 'rb') as f:
    EEG_data_unfiltered = pickle.load(f)

In [ ]:
# import numpy as np
import matplotlib.pyplot as plt
import mne
from IPython.display import clear_output
import numpy as np

def plot_comparison(raw_filt: mne.io.Raw, raw_unfilt: mne.io.Raw,
                    start_sec: float = 0.0, duration_sec: float = 5.0,
                    ch_start: int = 0, ch_end: int = 0):
    """
    Estrae un segmento di durata duration_sec a partire da start_sec,
    per i canali dal ch_start a ch_end (inclusi), e ne plotta il confronto filtrato vs non filtrato
    usando due subplot (non filtrato sopra, filtrato sotto) e legenda centrale sotto.
    """
    sfreq = raw_filt.info['sfreq']
    start_samp = int(start_sec * sfreq)
    stop_samp  = int((start_sec + duration_sec) * sfreq)
    times = np.arange(start_samp, stop_samp) / sfreq
    
    microvolt_scale = 1e6
    
    # Configura figura con 2 righe, asse x condiviso
    fig, (ax_unfilt, ax_filt) = plt.subplots(2, 1, sharex=True, figsize=(12, 8))
    
    # Plot dati non filtrati
    for ch_idx in range(ch_start, ch_end + 1):
        data_u = raw_unfilt.get_data(picks=[ch_idx]).squeeze()[start_samp:stop_samp] * microvolt_scale
        ch_name = raw_filt.ch_names[ch_idx]
        ax_unfilt.plot(times, data_u, label=ch_name, alpha=0.7)
    ax_unfilt.set_title(f"Non filtrato (Task={task}, Sbj={sbj}, Run={run})")
    ax_unfilt.set_ylabel("Ampiezza (µV)")

    # Plot dati filtrati
    for ch_idx in range(ch_start, ch_end + 1):
        data_f = raw_filt.get_data(picks=[ch_idx]).squeeze()[start_samp:stop_samp] * microvolt_scale
        ch_name = raw_filt.ch_names[ch_idx]
        ax_filt.plot(times, data_f, label=ch_name, alpha=0.7)
    ax_filt.set_title("Filtrato (1-45Hz + notch 50Hz)")
    ax_filt.set_xlabel("Tempo (s)")
    ax_filt.set_ylabel("Ampiezza (µV)")

    # Legenda unica sotto i subplot
    handles, labels = ax_unfilt.get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=min(len(labels), 6), bbox_to_anchor=(0.5, -0.05))

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)  # spazio per legenda
    plt.show()


# ------ MENU INTERATTIVO ------
while True:
    clear_output(wait=True)
    cmd = input("Digita 'ok' per plottare o andare al soggetto successivo, 'stop' per uscire: ").strip().lower()
    if cmd == 'stop':
        confirm = input("Sei sicuro di voler uscire? ('ok' per confermare, continua per proseguire): ").strip().lower()
        if confirm == 'ok':
            print("Uscita confermata. Bye!")
            clear_output(wait=True)
            break
        else:
            clear_output(wait=True)
            continue
    elif cmd != 'ok':
        print("Comando non riconosciuto, digita 'ok' o 'stop'.")
        continue

    # Ora procedi con scelta task, soggetto e run e plotting
    task = input(f"\nScegli task tra {list(EEG_data_filtered.keys())}: ").strip()
    if task not in EEG_data_filtered:
        print("Task non valido.")
        continue

    sbj = input(f"\nScegli soggetto tra {list(EEG_data_filtered[task].keys())}: ").strip()
    if sbj not in EEG_data_filtered[task]:
        print("Soggetto non valido.")
        continue

    available_runs = list(EEG_data_filtered[task][sbj].keys())
    run = int(input(f"\nScegli run tra {available_runs}: ").strip())
    if run not in available_runs:
        print("Run non valido.")
        continue

    start_sec    = float(input("\nStart time (s) [default 0]: ") or 0.0)
    duration_sec = float(input("Duration (s) [default 5]: ") or 5.0)

    raw_filt    = EEG_data_filtered[task][sbj][run][0]
    raw_unfilt  = EEG_data_unfiltered[task][sbj][run][0]

    # Stampa canali orizzontalmente
    ch_names = raw_filt.ch_names
    print("\nElenco canali:", "  ".join(f"\033[1m{i}\033[0m: \033[1m{ch}\033[0m" for i,ch in enumerate(ch_names)))
    max_idx = len(ch_names) - 1
    ch_start = int(input(f"Inizio canale [0–{max_idx}] [default 0]: ") or 0)
    ch_end   = int(input(f"Fine canale [0–{max_idx}] [default {ch_start}]: ") or ch_start)

    plot_comparison(raw_filt, raw_unfilt,
                    start_sec, duration_sec,
                    ch_start, ch_end)


#### **EEG Preprocessing - Data Quality Check Canali Post Filtering Stage Notch 60 and IIR 1-45Hz**

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_only_filtered_data.pkl', 'rb') as f:
    EEG_data_filtered = pickle.load(f)
    
    
# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_unfiltered_data.pkl', 'rb') as f:
    EEG_data_unfiltered = pickle.load(f)

In [ ]:
'''

Dopo che io ho filtrato i dati, vado a fare una esplorazione del valore di ogni canale per ogni task, soggetto e run. 

Ossia, vado a vedere per ogni canale quante volte, ossia quanti campioni superano la sogia di meno o più 100 microvolt, in valore assoluto.

Questa cosa la faccio per ogni canale, soggetto, task e run. 

Vorrei vedere se c'è una certa consistenza o pattern nel modo in cui determinati canali, anche dopo il filtraggio, sono con dei valori sopra la soia consentita.
Vorrei poi definire per ogni soggetto quali sono i canali che effettivamente creano problemi. 

Questo perché vorrei capire se c'è un problema comune per gli stessi canali tra i soggetti a parità di task e run, punto. 

A quel punto, se effettivamente c'è una certa consistenza, potremmo pensare di togliere questi canali. Può avere senso?

'''
print(EEG_data_filtered.keys())
print(EEG_data_filtered['task_1'].keys())
print(EEG_data_filtered['task_1']['100'].keys())
print(type(EEG_data_filtered['task_1']['100'][0][0]))
print(type(EEG_data_filtered['task_1']['100'][0][1]))

In [ ]:
# import numpy as np
import matplotlib.pyplot as plt
import mne
from IPython.display import clear_output
import numpy as np

def plot_comparison(raw_filt: mne.io.Raw, raw_unfilt: mne.io.Raw,
                    start_sec: float = 0.0, duration_sec: float = 5.0,
                    ch_start: int = 0, ch_end: int = 0):
    """
    Estrae un segmento di durata duration_sec a partire da start_sec,
    per i canali dal ch_start a ch_end (inclusi), e ne plotta il confronto filtrato vs non filtrato
    usando due subplot (non filtrato sopra, filtrato sotto) e legenda centrale sotto.
    """
    sfreq = raw_filt.info['sfreq']
    start_samp = int(start_sec * sfreq)
    stop_samp  = int((start_sec + duration_sec) * sfreq)
    times = np.arange(start_samp, stop_samp) / sfreq
    
    microvolt_scale = 1e6
    
    # Configura figura con 2 righe, asse x condiviso
    fig, (ax_unfilt, ax_filt) = plt.subplots(2, 1, sharex=True, figsize=(12, 8))
    
    # Plot dati non filtrati
    for ch_idx in range(ch_start, ch_end + 1):
        data_u = raw_unfilt.get_data(picks=[ch_idx]).squeeze()[start_samp:stop_samp] * microvolt_scale
        ch_name = raw_filt.ch_names[ch_idx]
        ax_unfilt.plot(times, data_u, label=ch_name, alpha=0.7)
    ax_unfilt.set_title(f"Non filtrato (Task={task}, Sbj={sbj}, Run={run})")
    ax_unfilt.set_ylabel("Ampiezza (µV)")

    # Plot dati filtrati
    for ch_idx in range(ch_start, ch_end + 1):
        data_f = raw_filt.get_data(picks=[ch_idx]).squeeze()[start_samp:stop_samp] * microvolt_scale
        ch_name = raw_filt.ch_names[ch_idx]
        ax_filt.plot(times, data_f, label=ch_name, alpha=0.7)
    ax_filt.set_title("Filtrato (1-45Hz + notch 60Hz)")
    ax_filt.set_xlabel("Tempo (s)")
    ax_filt.set_ylabel("Ampiezza (µV)")

    # Legenda unica sotto i subplot
    handles, labels = ax_unfilt.get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=min(len(labels), 6), bbox_to_anchor=(0.5, -0.05))

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)  # spazio per legenda
    plt.show()


# ------ MENU INTERATTIVO ------
while True:
    clear_output(wait=True)
    cmd = input("Digita 'ok' per plottare o andare al soggetto successivo, 'stop' per uscire: ").strip().lower()
    if cmd == 'stop':
        confirm = input("Sei sicuro di voler uscire? ('ok' per confermare, continua per proseguire): ").strip().lower()
        if confirm == 'ok':
            print("Uscita confermata. Bye!")
            clear_output(wait=True)
            break
        else:
            clear_output(wait=True)
            continue
    elif cmd != 'ok':
        print("Comando non riconosciuto, digita 'ok' o 'stop'.")
        continue

    # Ora procedi con scelta task, soggetto e run e plotting
    task = input(f"\nScegli task tra {list(EEG_data_filtered.keys())}: ").strip()
    if task not in EEG_data_filtered:
        print("Task non valido.")
        continue

    sbj = input(f"\nScegli soggetto tra {list(EEG_data_filtered[task].keys())}: ").strip()
    if sbj not in EEG_data_filtered[task]:
        print("Soggetto non valido.")
        continue

    available_runs = list(EEG_data_filtered[task][sbj].keys())
    run = int(input(f"\nScegli run tra {available_runs}: ").strip())
    if run not in available_runs:
        print("Run non valido.")
        continue

    start_sec    = float(input("\nStart time (s) [default 0]: ") or 0.0)
    duration_sec = float(input("Duration (s) [default 5]: ") or 5.0)

    raw_filt    = EEG_data_filtered[task][sbj][run][0]
    raw_unfilt  = EEG_data_unfiltered[task][sbj][run][0]

    # Stampa canali orizzontalmente
    ch_names = raw_filt.ch_names
    print("\nElenco canali:", "  ".join(f"\033[1m{i}\033[0m: \033[1m{ch}\033[0m" for i,ch in enumerate(ch_names)))
    max_idx = len(ch_names) - 1
    ch_start = int(input(f"Inizio canale [0–{max_idx}] [default 0]: ") or 0)
    ch_end   = int(input(f"Fine canale [0–{max_idx}] [default {ch_start}]: ") or ch_start)

    plot_comparison(raw_filt, raw_unfilt,
                    start_sec, duration_sec,
                    ch_start, ch_end)


#### **EEG Preprocessing - Cleaning via PyPREP Pipeline**

#### **PyPrep con oggetto PrepPipeline**

In [ ]:
'''

Ottimo — prendo esattamente la tua funzione e ci innesto (senza stravolgerla):

un checkpoint opzionale per-run (salvo il Raw pulito in .fif, salvo le labels in file a parte, e salvo il mini log della run in CSV);

la ripartenza che salta le run già fatte e le ricarica dal disco ricostruendo la stessa struttura (Raw pulito, labels);

i metadati ref_max_iterations e converged (True se n_iter < ref_max_iterations, False se uguale, None se l’informazione non è disponibile);

nessuna modifica alla forma dell’output in RAM: sempre dict nidificato; ogni run: Raw oppure (Raw, labels).

Di seguito trovi:

due piccoli helper per checkpoint;

la tua funzione aggiornata (solo aggiunte, il resto resta come l’hai scritto);

un esempio d’uso.


'''


#1) Helper per checkpoint (salvataggio/ricarica per-run)

import os, json
from pathlib import Path
import numpy as np
import pandas as pd
import mne
import pickle
from typing import Any, Tuple

'''
1) _paths(base): crea (se non ci sono) le cartelle:

raw_prep_fif/ (per i .fif)
labels/ (per le labels in vari formati)
logs_per_run/ (csv per-run)
completed_runs.json (indice delle run già fatte)


2) _key(task, subj, run): genera una chiave hashable per indicizzare la run. (Vedi nota sopra: meglio tornare una tupla).

3) _load_done / _save_done: caricano/salvano l’insieme di chiavi “già fatte”.

4) _save_labels / _load_labels: serializzano labels in vari formati (npz, csv, json, annotations come CSV, pickle fallback) e le ricaricano.

    Nota: se vuoi ricostruire davvero un mne.Annotations (e non un DataFrame), in _load_labels quando trovi annotations_csv puoi fare:
    
    df = pd.read_csv(p)
    return mne.Annotations(onset=df['onset'], duration=df['duration'], description=df['description'])
    
    … adegua i nomi colonnari a quelli che hai salvato.

5) _save_run_checkpoint: salva il raw .fif, le labels (via _save_labels) e il mini-log per-run (csv) nelle loro cartelle dedicate, e restituisce i path/meta utili.

6) _load_run_checkpoint: ricarica il raw .fif, cerca labels nei formati supportati e ritorna anche il mini-log csv se esiste.


'''
def _paths(base: str) -> Tuple[Path, Path, Path, Path]:
    base = Path(base)
    raw_dir = base / "raw_prep_fif"
    lbl_dir = base / "labels"
    log_dir = base / "logs_per_run"
    idx_json = base / "completed_runs.json"
    raw_dir.mkdir(parents=True, exist_ok=True)
    lbl_dir.mkdir(parents=True, exist_ok=True)
    log_dir.mkdir(parents=True, exist_ok=True)
    return raw_dir, lbl_dir, log_dir, idx_json

def _key(task, subj, run):
    #return [str(task), str(subj), int(run)]
    
    return (str(task), str(subj), int(run))

def _load_done(idx_json: Path):
    if idx_json.exists():
        return {tuple(x) for x in json.loads(idx_json.read_text())}
    return set()

def _save_done(idx_json: Path, done_set):
    idx_json.write_text(json.dumps([list(x) for x in done_set]))

def _save_labels(lbl_dir: Path, task, subj, run, labels: Any) -> dict:
    """Salva labels in vari formati comuni; ritorna un dict meta {kind, path}."""
    if labels is None:
        return {"kind": "none", "path": None}
    out_dir = lbl_dir / str(task) / str(subj)
    out_dir.mkdir(parents=True, exist_ok=True)
    base = out_dir / f"run-{run}"
    if isinstance(labels, np.ndarray):
        path = base.with_suffix(".npz")
        np.savez_compressed(path, labels=labels)
        return {"kind": "npz", "path": str(path)}
    if isinstance(labels, pd.DataFrame):
        path = base.with_suffix(".csv")
        labels.to_csv(path, index=False)
        return {"kind": "csv", "path": str(path)}
    if isinstance(labels, dict):
        path = base.with_suffix(".json")
        path.write_text(json.dumps(labels))
        return {"kind": "json", "path": str(path)}
    if isinstance(labels, mne.Annotations):
        # serializza come tabella semplice
        df = labels.to_data_frame()
        path = base.with_suffix(".annotations.csv")
        df.to_csv(path, index=False)
        return {"kind": "annotations_csv", "path": str(path)}
    # fallback: pickle
    path = base.with_suffix(".pkl")
    with open(path, "wb") as f:
        pickle.dump(labels, f, protocol=pickle.HIGHEST_PROTOCOL)
    return {"kind": "pickle", "path": str(path)}

def _load_labels(meta: dict):
    kind = meta.get("kind"); path = meta.get("path")
    if kind in (None, "none") or not path:
        return None
    p = Path(path)
    if kind == "npz":
        return np.load(p)["labels"]
    if kind == "csv":
        return pd.read_csv(p)
    if kind == "json":
        return json.loads(p.read_text())
    if kind == "annotations_csv":
        return pd.read_csv(p)  # ricostruisci mne.Annotations qui se ti serve
    if kind == "pickle":
        with open(p, "rb") as f:
            return pickle.load(f)
    return None

def _save_run_checkpoint(base_dir: str, task, subj, run, raw, labels, run_log_df: pd.DataFrame):
    raw_dir, lbl_dir, log_dir, idx_json = _paths(base_dir)
    # Raw -> FIF
    out_dir = raw_dir / str(task) / str(subj)
    out_dir.mkdir(parents=True, exist_ok=True)
    fif_path = out_dir / f"run-{run}_prep-raw.fif"
    raw.save(fif_path.as_posix(), overwrite=True)
    # labels
    lbl_meta = _save_labels(lbl_dir, task, subj, run, labels)
    # mini-log per-run
    log_out_dir = log_dir / str(task) / str(subj)
    log_out_dir.mkdir(parents=True, exist_ok=True)
    
    #1) Scrittura del mini-log per-run in _save_run_checkpoint
    
    #Perché: nel tuo codice fai (...).write_text(run_log_df.to_csv(index=False)).
    #to_csv restituisce già una stringa, quindi write_text(...) va ma è superfluo. 
    #Meglio lasciare che sia pandas a scrivere il file direttamente: è più pulito e gestisce anche l’encoding.

    #(log_out_dir / f"run-{run}.csv").write_text(run_log_df.to_csv(index=False))
    
    #Niente cambia nel comportamento, solo più solido/idiomatico.
    run_log_path = log_out_dir / f"run-{run}.csv"
    run_log_df.to_csv(run_log_path, index=False)

    return str(fif_path), lbl_meta

def _load_run_checkpoint(base_dir: str, task, subj, run):
    raw_dir, lbl_dir, log_dir, idx_json = _paths(base_dir)
    fif_path = raw_dir / str(task) / str(subj) / f"run-{run}_prep-raw.fif"
    raw = mne.io.read_raw_fif(fif_path.as_posix(), preload=True)
    # labels: cerca nei formati salvati
    base = lbl_dir / str(task) / str(subj) / f"run-{run}"
    labels = None
    for cand in [base.with_suffix(s) for s in (".npz",".csv",".json",".annotations.csv",".pkl")]:
        if cand.exists():
            # ricava kind
            if cand.suffix == ".npz":
                meta = {"kind":"npz","path":str(cand)}
            elif cand.name.endswith(".annotations.csv"):
                meta = {"kind":"annotations_csv","path":str(cand)}
            elif cand.suffix == ".csv":
                meta = {"kind":"csv","path":str(cand)}
            elif cand.suffix == ".json":
                meta = {"kind":"json","path":str(cand)}
            else:
                meta = {"kind":"pickle","path":str(cand)}
            labels = _load_labels(meta)
            break
    # mini-log
    run_log = None
    run_log_path = log_dir / str(task) / str(subj) / f"run-{run}.csv"
    if run_log_path.exists():
        run_log = pd.read_csv(run_log_path)
    return raw, labels, run_log

In [ ]:
'''
Codice: pipeline end-to-end (riusa i tuoi helper di checkpoint)
'''

from typing import Dict, Tuple, List, Any, Optional
import numpy as np
import pandas as pd
import mne

# NEW: importa la pipeline
from pyprep.prep_pipeline import PrepPipeline

# --- riusa i TUOI helper già definiti ---
# _paths, _key, _load_done, _save_done, _save_labels, _load_labels,
# _save_run_checkpoint, _load_run_checkpoint
# (li lascio invariati: incollarli così come li hai già)

def _safe_list(x):
    """Rende lista in modo robusto (anche se None o np.ndarray scalare)."""
    if x is None:
        return []
    if isinstance(x, (list, tuple, set)):
        return list(x)
    try:
        return list(x)
    except Exception:
        return [x]

def _get_attr_any(obj, *names):
    """Ritorna il primo attributo esistente tra quelli elencati (altrimenti None)."""
    for n in names:
        if hasattr(obj, n):
            return getattr(obj, n)
    return None

def _extract_pipeline_lists(prep: Any) -> Tuple[List[str], List[str], List[str], Optional[int]]:
    """
    Estrae in modo cross-version:
      - pre_ref_bad_channels         (bad_before_interpolation)
      - interpolated_channels
      - post_interp_residual_channels (still_noisy_channels)
      - n_iter (se esposto)
    """
    pre_ref_bad = _safe_list(_get_attr_any(
        prep,
        "bad_before_interpolation",      # nome comune
        "excluded_before_interpolation", # in alcune versioni
    ))
    interpolated = _safe_list(_get_attr_any(
        prep, "interpolated_channels", "interpolated"
    ))
    still_noisy = _safe_list(_get_attr_any(
        prep, "still_noisy_channels", "final_noisy_channels", "ref_still_noisy_channels"
    ))
    # iterazioni effettive (se disponibile)
    n_iter = _get_attr_any(prep, "n_iterations_", "n_iterations", "iterations")
    try:
        n_iter = int(n_iter) if n_iter is not None else None
    except Exception:
        n_iter = None

    # ordina e de-duplica
    pre_ref_bad = sorted(set(pre_ref_bad))
    interpolated = sorted(set(interpolated))
    still_noisy = sorted(set(still_noisy))
    return pre_ref_bad, interpolated, still_noisy, n_iter

In [ ]:
'''

Cosa fa la run_preppipeline_on_nested step-by-step

Riassumo il flusso della versione “end-to-end” che ti ho passato:

Checkpoint setup
Crea/simula raw_prep_fif/, labels/, logs_per_run/, legge completed_runs.json.

Loop su task → subject → run
Mantiene la stessa struttura in output.

Skip run già completate
Se la run è in done, carica da disco (_load_run_checkpoint) e aggiunge il mini-log.

Normalizza input
Supporta Raw o (Raw, labels). Copia il Raw e fa .load_data().

Controllo montage
Se non c’è un montage/dig, solleva un errore chiaro (l’interpolazione ne ha bisogno).



Costruisce i parametri della pipeline


prep_params = {
    "ref_chs": "eeg",       # usa tutti gli EEG per stimare la ref
    "reref_chs": "eeg",     # re-referenzia tutti gli EEG
    "max_iterations": 4,
    # "line_freqs": None  # (nel tuo caso)
}


Esegue la pipeline

prep = PrepPipeline(raw_in, prep_params, montage=montage)
prep.fit()
raw_out = prep.raw  # Raw pulito

Estrae le liste chiave (cross-version)
Con un helper robusto prende:

pre_ref_bad_channels ← bad_before_interpolation

interpolated_channels

post_interp_residual_channels ← still_noisy_channels

n_iter (se disponibile)

Deriva i bad after-ref (pre-interp)
post_ref_bad_channels = interpolated − pre_ref (vedi punto 2).

Convergenza
converged = (n_iter is not None) and (n_iter < max_iterations).

Log per-canale (mini-log)
Per ogni canale salva: liste pre/post, flag was_interpolated, bad_any_post, e meta (iterazioni, convergenza).
Scrive il CSV per-run in logs_per_run/task/subject/run-*.csv.

Checkpoint salvataggio
Salva il .fif pulito, le labels (se presenti) e aggiorna completed_runs.json.

Output in RAM
Rimonta la stessa struttura: Raw oppure (Raw, labels).

'''

In [ ]:
def run_preppipeline_on_nested(
    EEG_data_filtered: Dict[str, Dict[str, Dict[int, Any]]],
    checkpoint_dir: Optional[str] = None,
    skip_completed: bool = True,
    
    # parametri PREP/REF
    max_iterations: int = 4,
    line_freqs: Optional[List[float]] = None,   # es. [50] o [60]; se None, lascia alla pipeline
) -> Tuple[Dict[str, Dict[str, Dict[int, Any]]], pd.DataFrame]:
    
    """
    Versione 'end-to-end' con PrepPipeline:
    - in:  dict nidificato task -> subject -> run -> Raw oppure (Raw, labels)
    - out: stessa forma (Raw pulito o (Raw pulito, labels)) + prep_log per-canale
    - checkpoint: identico al tuo
    """
    
    if checkpoint_dir:
        raw_dir, lbl_dir, log_dir, idx_json = _paths(checkpoint_dir)
        done = _load_done(idx_json)
    else:
        done = set()

    EEG_data_prep: Dict[str, Dict[str, Dict[int, Any]]] = {}
    all_log_rows: List[Dict[str, Any]] = []

    for task, subjs in EEG_data_filtered.items():
        EEG_data_prep.setdefault(task, {})
        for subj, runs in subjs.items():
            EEG_data_prep[task].setdefault(subj, {})

            for run, value in runs.items():
                key = _key(task, subj, run)

                # checkpoint: salta se già fatto
                if checkpoint_dir and skip_completed and key in done:
                    raw_loaded, labels_loaded, run_log_df = _load_run_checkpoint(checkpoint_dir, task, subj, run)
                    EEG_data_prep[task][subj][run] = (raw_loaded, labels_loaded) if labels_loaded is not None else raw_loaded
                    if run_log_df is not None:
                        all_log_rows.extend(run_log_df.to_dict("records"))
                    continue

                # normalizza input (Raw oppure (Raw, labels))
                if isinstance(value, tuple) and len(value) >= 1:
                    raw_in = value[0]
                    labels = value[1] if len(value) > 1 else None
                else:
                    raw_in = value
                    labels = None

                if not isinstance(raw_in, mne.io.BaseRaw):
                    raise TypeError(
                        f"Atteso mne.io.Raw o (Raw, labels), trovato {type(raw_in)} in {task}/{subj}/{run}"
                    )

                raw_in = raw_in.copy().load_data()  # dati già filtrati (come prima)

                # --- MONTAGE: PrepPipeline richiede posizioni sensori coerenti per interpolare ---
                # Se non hai già un montage su raw_in, impostalo PRIMA (altrimenti l’interpolazione fallisce)
                montage = raw_in.get_montage()
                if montage is None and (raw_in.info.get("dig") is None):
                    
                    # Se NON hai un montage definito, è meglio fermarsi esplicitamente:
                    raise RuntimeError(
                        f"Nessun montage trovato in {task}/{subj}/{run}. "
                        "Imposta un montage (es. standard_1020) prima di chiamare la pipeline."
                    )

                # --- Parametri PrepPipeline ---
                # ref_chs / reref_chs: usa tutti gli EEG; la pipeline escluderà i bad nelle iterazioni.
                
                #Note:

                #Non serve convertirla in np.array; una lista va benissimo (la pipeline fa solo len(...)).
                #Se hai già fatto il notch a 60 Hz, tieni line_freqs = [].
                #Se vuoi far rimuovere a PREP il line-noise, passa ad es. [60] o [50] a seconda della rete.

                # 1) Normalizza line_freqs
                if line_freqs is None:
                    line_freqs = []                 # già notchato: niente rimozione linea
                else:
                    line_freqs = [float(f) for f in line_freqs]  # es. [50], [60], [50, 100]
    
                # 2) Costruisci i params includendo SEMPRE la chiave
                prep_params = {
                    "ref_chs": "eeg",
                    "reref_chs": "eeg",
                    "max_iterations": int(max_iterations),
                    "line_freqs": line_freqs,
                }


                # --- Esegui PrepPipeline ---
                
                #https://pyprep.readthedocs.io/en/latest/generated/pyprep.PrepPipeline.html
                
                prep = PrepPipeline(raw_in, prep_params, montage=montage)
                prep.fit()

                # Raw pulito prodotto dalla pipeline
                raw_out = _get_attr_any(prep, "raw", "raw_out")
                if raw_out is None:
                    # fallback conservativo: se per qualche ragione non esposto, usa raw_in
                    raw_out = raw_in.copy()

                # Estrai liste chiave (cross-version)
                pre_ref_bad_channels, interpolated_channels, post_interp_residual_channels, n_iter = \
                    _extract_pipeline_lists(prep)

                # Proxy dei bad AFTER-REF (pre-interp): nuovi bad emersi durante la robust ref
                post_ref_bad_channels = sorted(
                    set(interpolated_channels) - set(pre_ref_bad_channels)
                )

                # convergenza (se disponibile)
                ref_max_iterations = max_iterations
                converged = (n_iter is not None) and (n_iter < ref_max_iterations)

                # --- LOG per-canale (più snello: niente A1/A2/B, niente amp-gate) ---
                run_rows = []
                for ch in raw_out.ch_names:
                    run_rows.append({
                        "task": task,
                        "subject": subj,
                        "run": run,
                        "channel": ch,

                        # fasi chiare
                        "pre_ref_bad_channels": pre_ref_bad_channels,
                        "post_ref_bad_channels": post_ref_bad_channels,
                        "interpolated_channels": interpolated_channels,
                        "post_interp_residual_channels": post_interp_residual_channels,

                        # indicatori per-canale
                        "was_interpolated": int(ch in interpolated_channels),
                        "bad_any_post": int(ch in post_interp_residual_channels),

                        # meta ref
                        "ref_n_iterations": n_iter,
                        "ref_max_iterations": ref_max_iterations,
                        "ref_converged": converged,
                    })
                all_log_rows.extend(run_rows)

                # --- checkpoint per-run (identico al tuo)
                if checkpoint_dir:
                    run_log_df = pd.DataFrame(run_rows)
                    _save_run_checkpoint(checkpoint_dir, task, subj, run, raw_out, labels, run_log_df)
                    done.add(key)
                    _save_done(idx_json, done)

                # --- salva in RAM (stessa forma dell'input)
                if labels is None:
                    EEG_data_prep[task][subj][run] = raw_out
                else:
                    EEG_data_prep[task][subj][run] = (raw_out, labels)

    prep_log = pd.DataFrame(all_log_rows).sort_values(["task", "subject", "run", "channel"])
    return EEG_data_prep, prep_log

In [ ]:
EEG_data_prep, prep_log = run_preppipeline_on_nested(
    EEG_data_filtered,
    checkpoint_dir="/home/stefano/EEGMMIDB_Analyses/pyprep_outputs",  # oppure None
    skip_completed=True,
    max_iterations=4,
    line_freqs=None  # già fatto il notch a 60 Hz
)

#### **Salvataggio strutture dati DOPO pipeline PyPrep**

In [ ]:
type(EEG_data_prep)

In [ ]:
'''

IO:

Ma scusa tanto comunque la mia struttura EEG_data_prep sta iterativamente salvando al suo interno i dati di tutti i soggetti, task e runs.. giusto?

per cui alla fine il salvataggio lo sta già facendo gradualmente.. e per cui se faccio 

# 1) salva i Dati Ricostruiti

import pickle

base = '/home/stefano/EEGMMIDB_Analyses/'  # cartella di destinazione

with open(f"{base}EEG_post_pyprep_data.pkl", "wb") as f:
    pickle.dump(EEG_data_prep, f, protocol=pickle.HIGHEST_PROTOCOL)

dovrebbe essere abbastanza ok come cosa no?

e la stessa cosa per 

# 2) salva i log/tabelloni

prep_log.to_csv(f"{path}prep_log.csv", index=False)

se volessi salvarmi il file completo di tutti i log dei dati di tutti i soggetti, task e runs?



CHATGPT:
Esatto, hai centrato il punto 👍

Durante l’esecuzione, la pipeline sta già popolando EEG_data_prep con tutti i (Raw_pulito, labels) per ogni task → subject → run

Alla fine, fare il tuo pickle.dump(...) è sufficiente per salvare tutta la struttura completa in un colpo solo

Stessa cosa per prep_log: alla fine della pipeline, se prep_log contiene tutte le righe accumulate di tutti i soggetti/task/run, un semplice

python
Copia
Modifica
prep_log.to_csv(f"{base}prep_log.csv", index=False)
ti salva il file unico e completo

📌 In pratica:

EEG_data_prep → salvi l’oggetto Python (dati Raw + labels)

prep_log → salvi il tabellone completo in CSV

💡 Ti segnalo solo che, se i tuoi dati sono grandi, il .pkl potrebbe essere pesante e lento da ricaricare; 
se in futuro ti servisse, potresti anche salvare i .fif separati nei checkpoint e poi ricostruire la struttura a runtime.

Vuoi che ti scriva anche il loader “ufficiale” per ricaricare EEG_data_prep e prep_log insieme e ripristinare tutto in un colpo?


Salvo strutture dati per 

1) Dati Ripulti dopo PyPREP 
2) Log dei Dati Ripulti dopo PyPREP 


Se salvi prep_log con pickle (o joblib), il DataFrame manterrà esattamente:

Tutti i tipi originali di ogni colonna (liste, dict, array ecc.)
L’indice originale (index)

Eventuali tipi complessi che nel CSV diventerebbero stringhe

'''

import pickle

# Salvare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/pyprep_outputs/'

# 1) salva i dati preppati (Raw o (Raw, labels))
with open(f"{path}EEG_post_pyprep_data.pkl", "wb") as f:
    pickle.dump(EEG_data_prep, f, protocol=pickle.HIGHEST_PROTOCOL)
    
# 2) salva i log/tabelloni
#prep_log.to_csv(f"{path}prep_log.csv", index=False)


In [ ]:
import pickle

# Salvare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/pyprep_outputs/'

# 1) salva i dati preppati (Raw o (Raw, labels))
with open(f"{path}prep_log.pkl", "wb") as f:
    pickle.dump(prep_log, f, protocol=pickle.HIGHEST_PROTOCOL)


#### **EEG Preprocessing - Data Quality Check Canali Post PyPrep Stage**
##### **Visual Inspection - Data Comparison between files 1) EEG_prep_data and 2) EEG_data_filtered**

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_only_filtered_data.pkl', 'rb') as f:
    EEG_data_filtered = pickle.load(f)
    

path_post_prep = '/home/stefano/EEGMMIDB_Analyses/pyprep_outputs/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path_post_prep}EEG_post_pyprep_data.pkl', 'rb') as f:
    EEG_data_prep = pickle.load(f)

In [ ]:
'''
Ora, posso ri-usare questo codice per vedere se i dati son stati ripuliti correttamente, prima di passarli alla mia ICA custom,
confrontano 

1) i dati solo pre-filtrati con 
2) i dati post pipeline prep, ossia quelli dopo esser passati e ripuliti con la pipeline pyprep?

'''

# import numpy as np
import matplotlib.pyplot as plt
import mne
from IPython.display import clear_output
    
    
def plot_comparison(raw_filt: mne.io.Raw, raw_prep: mne.io.Raw,
                    start_sec: float = 0.0, duration_sec: float = 5.0,
                    ch_start: int = 0, ch_end: int = 0):
    """
    Estrae un segmento di durata duration_sec a partire da start_sec,
    per i canali dal ch_start a ch_end (inclusi), e ne plotta il confronto filtrato vs non filtrato
    usando due subplot (non filtrato sopra, filtrato sotto) e legenda centrale sotto.
    """
    sfreq = raw_filt.info['sfreq']
    start_samp = int(start_sec * sfreq)
    stop_samp  = int((start_sec + duration_sec) * sfreq)
    times = np.arange(start_samp, stop_samp) / sfreq
    
    microvolt_scale = 1e6
    
    # Configura figura con 2 righe, asse x condiviso
    fig, (ax_filt, ax_prep) = plt.subplots(2, 1, sharex=True, figsize=(12, 8))

    # PRE (filtrato prima di PyPREP)
    for ch_idx in range(ch_start, ch_end + 1):
        data_filtered = raw_filt.get_data(picks=[ch_idx]).squeeze()[start_samp:stop_samp] * microvolt_scale
        ch_name = raw_filt.ch_names[ch_idx]
        ax_filt.plot(times, data_filtered, label=ch_name, alpha=0.7)
    ax_filt.set_title("PRE PyPREP")
    ax_filt.set_xlabel("Tempo (s)")
    ax_filt.set_ylabel("Ampiezza (µV)")
    
    # POST (dopo PyPREP)
    
    for ch_idx in range(ch_start, ch_end + 1):
        data_prep = raw_prep.get_data(picks=[ch_idx]).squeeze()[start_samp:stop_samp] * microvolt_scale
        ch_name = raw_prep.ch_names[ch_idx]
        ax_prep.plot(times, data_prep, label=ch_name, alpha=0.7)
    ax_prep.set_title(f"POST PyPREP")
    ax_prep.set_ylabel("Ampiezza (µV)")
    
    
    # Legenda unica sotto i subplot
    handles, labels = ax_prep.get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=min(len(labels), 6), bbox_to_anchor=(0.5, -0.05))

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)  # spazio per legenda
    plt.show()


# ------ MENU INTERATTIVO ------
while True:
    clear_output(wait=True)
    cmd = input("Digita 'ok' per plottare o andare al soggetto successivo, 'stop' per uscire: ").strip().lower()
    if cmd == 'stop':
        confirm = input("Sei sicuro di voler uscire? ('ok' per confermare, continua per proseguire): ").strip().lower()
        if confirm == 'ok':
            print("Uscita confermata. Bye!")
            clear_output(wait=True)
            break
        else:
            clear_output(wait=True)
            continue
    elif cmd != 'ok':
        print("Comando non riconosciuto, digita 'ok' o 'stop'.")
        continue

    # Ora procedi con scelta task, soggetto e run e plotting
    task = input(f"\nScegli task tra {list(EEG_data_filtered.keys())}: ").strip()
    if task not in EEG_data_filtered:
        print("Task non valido.")
        continue

    sbj = input(f"\nScegli soggetto tra {list(EEG_data_prep[task].keys())}: ").strip()
    if sbj not in EEG_data_prep[task]:
        print("Soggetto non valido.")
        continue

    available_runs = list(EEG_data_filtered[task][sbj].keys())
    run = int(input(f"\nScegli run tra {available_runs}: ").strip())
    if run not in available_runs:
        print("Run non valido.")
        continue

    start_sec    = float(input("\nStart time (s) [default 0]: ") or 0.0)
    duration_sec = float(input("End Time (s) [default 5]: ") or 5.0)

    raw_filt = EEG_data_filtered[task][sbj][run][0]
    raw_prep = EEG_data_prep[task][sbj][run][0]

    # Stampa canali orizzontalmente
    ch_names = raw_filt.ch_names
    print("\nElenco canali:", "  ".join(f"\033[1m{i}\033[0m: {ch}" for i,ch in enumerate(ch_names)))
    max_idx = len(ch_names) - 1
    ch_start = int(input(f"Inizio canale [0–{max_idx}] [default 0]: ") or 0)
    ch_end   = int(input(f"Fine canale [0–{max_idx}] [default {ch_start}]: ") or ch_start)

    plot_comparison(raw_filt, raw_prep,
                    start_sec, duration_sec,
                    ch_start, ch_end)
 

#### **Re-Loading dei files 1) EEG_prep_data e 2) Prep_Log direttamente come file .pkl**

#### **Per PyPrep con oggetto Prep Pipeline**

**Pipeline eseguita da qui in avanti**

1) Import/filtri iniziali (per ogni run)

- Conversione da µV a V.

- Montage: posizioni 10–20 ridotte ai canali del dataset.

- Notch 60 Hz (IIR Butterworth, SOS, fase zero).

- Band-pass 1–45 Hz (IIR Butterworth, SOS, fase zero).

    - Nota: con order=2 il band-pass ha ~8° ordine effettivo.

        Nota su ordine effettivo dei filtri IIR

        Con butter(N, ...) un band-pass ha ordine 2·N nel passaggio forward.

        MNE usa filtro a fase zero (forward + backward), che raddoppia l’ordine in ampiezza.

        Nel tuo caso order = 2 → 4° ordine forward (perché 2·N) → ~8° ordine effettivo con fase zero.

        Idem per il notch a 60 Hz: order=2 → ~8° effettivo.
        ➡️ Il settaggio attuale è equilibrato per EEG; impostare order=8 porterebbe a ~32° effettivo (troppo ripido).



2) Costruzione del contenitore

- Funzione build_raw_collection(...) → struttura:

        raw_collection[task][subject][run] = (Raw, annotations_array)


    I Raw sono già filtrati; le annotazioni ( = labels) - sono caricate.


3) Pulizia automatica con PyPREP (end-to-end)

- Funzione: run_preppipeline_on_nested(...) (checkpoint per-run attivo).

- Parametri principali:

    - ref_chs='eeg', reref_chs='eeg', max_iterations=4
    - line_freqs=[] (nessuna rimozione linea: notch già applicato)

- Per ogni run, PrepPipeline.fit() esegue:

    - Robust average reference iterativa
    - Identificazione canali “bad”
    - Interpolazione canali interi (spherical splines)
    - Re-reference finale

- Log per-canale salvato a ogni run, con:

    - pre_ref_bad_channels
    - post_ref_bad_channels (nuovi bad emersi durante la robust-ref)
    - interpolated_channels (canali interpolati integralmente)
    - post_interp_residual_channels (= still_noisy_channels post-PREP)
    - meta: ref_n_iterations, ref_converged, ecc.
    
- Output:

    - EEG_data_prep (stessa struttura dell’input, Raw puliti)
    - prep_log (tabella per-canale)


4) QC finale “solo residui”

- Funzione: summarize_run_quality_residuals(prep_log, max_resid_warning_n=3, max_resid_warning_frac=0.125).

- Per ogni run calcola:

    - post_interp_residual_channels_n
    - residual_frac = post_interp_residual_channels_n / n_chan

- Regola decisionale (solo residui):

    - OK se n_residui == 0
    - WARNING se 1–3 e residual_frac ≤ 0.125
    - DISCARD se n_residui > 3 oppure residual_frac > 0.125

- Tabelle prodotte:

    - df_runs (una riga per run, con liste/contatori e decision_final)
    - df_subjects (conteggio OK/WARNING/DISCARD per soggetto e task)



Risultati ottenuti

- OK: 682 run 
- WARNING: 480 run 
- DISCARD: 74 run


Approfondimento WARNING

- con ≤ 8 canali interpolati: 307 run (warn_keep)
- con > 8 canali interpolati: 173 run (warn_drop)

Interpretazione

Stai usando i residui post-PREP come metrica principale di qualità. 
In aggiunta, un criterio prudente sul numero di canali interpolati separa le WARNING “accettabili” (da includere per ICA/analisi estese) da quelle da scartare.


<br>

Infine:

Contiamo le runs OK/WARNING/DISCARD senza errori (len(...) o mask .sum());

una funzione riutilizzabile che:

seleziona OK + WARNING con ≤ 8 canali interpolati
ricostruisce il nuovo contenitore EEG_for_ICA a partire da EEG_data_prep
per le sole WARNING sovrascrive raw.info['bads'] con i soli residui finali (post_interp_residual_channels)
restituisce anche keep_df (la tabella delle run tenute) e un bads_log (audit)

In [ ]:
import pandas as pd

import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/pyprep_outputs/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_post_pyprep_data.pkl', 'rb') as f:
    EEG_data_prep = pickle.load(f)
    

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}prep_log.pkl', 'rb') as f:
    prep_log = pickle.load(f)
    

In [ ]:
'''

1) Che cosa fa summarize_run_quality_residuals

Input: il prep_log prodotto da run_preppipeline_on_nested, ovvero un DataFrame per-canale in cui ogni riga ha, tra le altre cose, le liste per-run replicate:
interpolated_channels
post_interp_residual_channels ⟵ questa è la lista chiave
pre_ref_bad_channels
post_ref_bad_channels

Idea: sintetizza il log per ottenere una riga per run e decide lo stato qualità solo in base ai residui post-PREP con le regole:

OK se n_residui == 0
WARNING se 1–3 e residual_frac ≤ 0.125
DISCARD se n_residui > 3 oppure residual_frac > 0.125


Output:

df_runs (una riga per run) con:
numero canali (n_chan)
liste una sola volta per run (interpolated / residual / pre_ref / post_ref)
contatori (es. post_interp_residual_channels_n)
residual_frac (= residui / n_chan)
decision_final (OK / WARNING / DISCARD)
flag pratici: keep_for_analysis, keep_for_ica
df_subjects: pivot per soggetto (quante run OK/WARNING/DISCARD e totale)

'''

# ============================================
# 2) QC basato SOLO sui residui post-PREP
#    (regole: OK / WARNING / DISCARD come richiesto)
# ============================================

'''
Accetta il log per-canale; fissa le soglie per classificare i residui (1–3 e ≤12.5% → WARNING)
'''

import pandas as pd
from typing import Tuple

def summarize_run_quality_residuals(
    prep_log: pd.DataFrame,
    max_resid_warning_n: int = 3,          # 1–3 → WARNING (se frazione ≤ soglia)
    max_resid_warning_frac: float = 0.125, # 12.5% → WARNING se entro questa frazione
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    
    
    """
    Crea un riepilogo per-run con decisione basata SOLO sui residui post-PREP:

      OK       se n_residui == 0
      WARNING  se 1–3 e residual_frac ≤ 0.125
      DISCARD  se n_residui > 3 oppure residual_frac > 0.125
    """
    
    '''
    Setup base

    import ast : servirà a trasformare stringhe come "['F3','F4']" in vere liste.
    
    keys = ["task","subject","run"] : le tre colonne usate per raggruppare per run.
    '''

    import ast
    keys = ["task","subject","run"]
    
    
    '''
    Helper di normalizzazione

    def _ensure_list(x): ...
    se x è lista → la restituisce;
    se x è stringa → ast.literal_eval per convertirla in lista (se possibile);
    altrimenti → [].

    Obiettivo: qualunque valore “diventa lista”, evitando crash quando contiamo o facciamo set(...).
    '''

    def _ensure_list(x):
        if isinstance(x, list): return x
        if isinstance(x, str):
            try:
                v = ast.literal_eval(x)
                return v if isinstance(v, list) else []
            except Exception:
                return []
        return []
    
    
    '''
    Numero di canali per run

    base = (prep_log.groupby(keys, as_index=False).agg(n_chan=('channel','count'))): conta quante righe (canali) ha ogni run → produce n_chan per run.
    '''
        

    # n_chan per run
    base = (prep_log.groupby(keys, as_index=False)
            .agg(n_chan=('channel','count')))
    
    
    '''
    Utility per estrarre una lista “una volta per run”

    def _pull_first(col): ...
    prende, per la colonna col (che contiene una lista replicata su ogni riga della run), il primo valore per ciascuna run.
    se la colonna non esiste in prep_log, ritorna un DataFrame vuoto con quella colonna.
    '''
    
    # estrai (una sola volta per run) le liste prodotte da PREP
    def _pull_first(col):
        if col not in prep_log.columns:
            return pd.DataFrame({**{k:[] for k in keys}, col: []})
        return (prep_log.groupby(keys, as_index=False)
                .agg(**{col: (col, 'first')}))
    
    '''
    Recupero delle 4 liste principali

    inter_meta = _pull_first("interpolated_channels")
    resid_meta = _pull_first("post_interp_residual_channels")
    pre_meta = _pull_first("pre_ref_bad_channels")
    after_meta = _pull_first("post_ref_bad_channels")
    per ciascuna lista, ottieni “una lista per run”.
    '''
    
    inter_meta = _pull_first("interpolated_channels")
    resid_meta = _pull_first("post_interp_residual_channels")
    pre_meta   = _pull_first("pre_ref_bad_channels")
    after_meta = _pull_first("post_ref_bad_channels")
    
    '''
    Merge in un unico DataFrame per-run

    df_runs = base.merge(inter_meta, ...).merge(resid_meta, ...).merge(pre_meta, ...).merge(after_meta, ...)

    unisce n_chan e tutte le liste in un’unica riga per run.
    '''
    
    df_runs = base.merge(inter_meta, on=keys, how='left') \
                  .merge(resid_meta, on=keys, how='left') \
                  .merge(pre_meta,   on=keys, how='left') \
                  .merge(after_meta, on=keys, how='left')
    
    '''
    Sanificazione delle liste

    for col in [...]: df_runs[col] = df_runs[col].apply(_ensure_list)
    garantisce che ognuna di quelle colonne sia davvero una lista (mai NaN/stringhe).
    '''
    for col in ["interpolated_channels","post_interp_residual_channels",
                "pre_ref_bad_channels","post_ref_bad_channels"]:
        df_runs[col] = df_runs[col].apply(_ensure_list)
        
    '''
    Contatori e frazioni

    df_runs["interpolated_channels_n"] = df_runs["interpolated_channels"].apply(len)
    quanti canali sono stati interpolati in quella run.
    df_runs["post_interp_residual_channels_n"] = df_runs["post_interp_residual_channels"].apply(len)
    quanti residui post-PREP (metrica chiave).
    df_runs["residual_frac"] = df_runs["post_interp_residual_channels_n"] / df_runs["n_chan"]

    frazione di residui rispetto ai canali totali.
    
    '''
    
    # contatori e frazioni
    df_runs["interpolated_channels_n"]        = df_runs["interpolated_channels"].apply(len)
    df_runs["post_interp_residual_channels_n"] = df_runs["post_interp_residual_channels"].apply(len)
    df_runs["residual_frac"] = df_runs["post_interp_residual_channels_n"] / df_runs["n_chan"]
    
    '''
    Regola decisionale (solo residui)

    def decide_resid(n_resid, frac, n_warn=..., frac_warn=...): ...
        implementa le tre condizioni:
        n_resid == 0 → "OK"
        1 ≤ n_resid ≤ n_warn e frac ≤ frac_warn → "WARNING"
        altrimenti → "DISCARD".
        
    df_runs["decision_final"] = df_runs.apply(lambda r: decide_resid(...), axis=1)
        calcola la classe per ogni run.
    '''
    

    # decisione (solo residui) – come richiesto
    def decide_resid(n_resid, frac, n_warn=max_resid_warning_n, frac_warn=max_resid_warning_frac):
        if n_resid == 0:
            return "OK"
        if (1 <= n_resid <= n_warn) and (frac <= frac_warn):
            return "WARNING"
        return "DISCARD"

    df_runs["decision_final"] = df_runs.apply(
        lambda r: decide_resid(r["post_interp_residual_channels_n"], r["residual_frac"]),
        axis=1
    )
    
    '''
    Flag pratici

    df_runs["keep_for_analysis"] = (df_runs["post_interp_residual_channels_n"] == 0)
    vero solo se OK (nessun residuo) → perfetta per analisi “pulite”.
    df_runs["keep_for_ica"] = ( (resid==0) | (1–3 & frac≤0.125) )
    consenti OK e WARNING (pochi residui e quota bassa) per includere più dati nell’ICA.
    
    '''
    # flag utili
    df_runs["keep_for_analysis"] = (df_runs["post_interp_residual_channels_n"] == 0)
    df_runs["keep_for_ica"] = (
        (df_runs["post_interp_residual_channels_n"] == 0) |
        (
            (df_runs["post_interp_residual_channels_n"] >= 1) &
            (df_runs["post_interp_residual_channels_n"] <= max_resid_warning_n) &
            (df_runs["residual_frac"] <= max_resid_warning_frac)
        )
    )
    
    '''
    Riepilogo per soggetto

    df_subjects = (df_runs.pivot_table(..., columns='decision_final', ...))
    conteggi OK/WARNING/DISCARD per (task, subject).
    for loop che assicura le tre colonne sempre presenti (anche se zero).
    df_subjects['runs_total'] = ...sum(axis=1)
    totale run per soggetto.
    '''
    
    # riepilogo per soggetto
    df_subjects = (df_runs
                   .pivot_table(index=['task','subject'],
                                columns='decision_final',
                                values='run',
                                aggfunc='count',
                                fill_value=0)
                   .reset_index())
    for col in ['OK','WARNING','DISCARD']:
        if col not in df_subjects.columns:
            df_subjects[col] = 0
    df_subjects['runs_total'] = df_subjects[['OK','WARNING','DISCARD']].sum(axis=1)
    
    '''
    Ordina e restituisci

    return df_runs.sort_values(keys), df_subjects.sort_values(['task','subject'])

    tabelle finali ordinate.
    '''
    return df_runs.sort_values(keys), df_subjects.sort_values(['task','subject'])

In [ ]:
# 2) QC solo-residui con le regole richieste
df_runs, df_subjects = summarize_run_quality_residuals(
    prep_log,
    max_resid_warning_n=3,
    max_resid_warning_frac=0.125
)


In [ ]:
# Esempi filtri:
runs_ok        = df_runs[df_runs.decision_final == "OK"]
runs_warning   = df_runs[df_runs.decision_final == "WARNING"]
runs_discard   = df_runs[df_runs.decision_final == "DISCARD"]

runs_for_ica       = df_runs[df_runs.keep_for_ica]
runs_for_analysis  = df_runs[df_runs.keep_for_analysis]

In [ ]:
runs_ok 

In [ ]:
runs_warning

In [ ]:
runs_discard

In [ ]:
'''
Sì, ha senso 👍
Con 64 canali, 8/64 = 12.5%: è coerente con la soglia che stai già usando per i residui. Quindi:

ICA: tieni le run WARNING solo se interpolated_channels_n ≤ 8 (oppure, meglio: interpolated_frac ≤ 0.125).

Escludi (DISCARD) le WARNING con più di 8 canali interpolati.

Ecco due modi pratici.

A) Filtro “al volo” sulle WARNING già calcolate

Se hai df_runs da summarize_run_quality_residuals:

# quante WARNING tenere (≤ 8 canali interpolati)?
warn_keep = df_runs[(df_runs.decision_final == 'WARNING') &
                    (df_runs.interpolated_channels_n <= 8)]
n_warn_keep = len(warn_keep)

# quante WARNING escludere (> 8 canali interpolati)?
warn_drop = df_runs[(df_runs.decision_final == 'WARNING') &
                    (df_runs.interpolated_channels_n > 8)]
n_warn_drop = len(warn_drop)


Perché è una buona idea

Eviti run con troppe ricostruzioni (interpolazioni) che possono biasare ICA e analisi successive.

La soglia “8 su 64” è la stessa del 12.5%: se un domani cambi numero di canali, la versione con frazione resta valida.

'''


In [ ]:
warn_keep = df_runs[(df_runs.decision_final == 'WARNING') &
                    (df_runs.interpolated_channels_n <= 8)]
n_warn_keep = len(warn_keep)

In [ ]:
n_warn_keep

In [ ]:
# quante WARNING escludere (> 8 canali interpolati)?
warn_drop = df_runs[(df_runs.decision_final == 'WARNING') &
                    (df_runs.interpolated_channels_n > 8)]
n_warn_drop = len(warn_drop)

In [ ]:
n_warn_drop

##### **Creazione del nuovo dataset filtrato a partire da EEG_prep_data**

##### A partire da df_runs (ossia il totale delle runs) vado a 

##### 1) **escludere** le runs che son con **'DISCARD'** (i.e., mask_discard = (df_runs['decision_post_interp'] == 'DISCARD'))


##### Dopodiché, da df_runs (SENZA le runs che son con 'DISCARD') vado a 

##### 2) **tenermi** le runs che son con **'OK'** 

##### 3) **tenermi** le runs che son con **'WARNING'**
    
   ##### 3.a) Per le runs con 'WARNING' -> ossia, per ogni runs, task e soggetto, ossia nel file raw corrispondente, andarmi a segnare i canali EEG come 'bad' (ossia di quelli che si trovano nella colonna 'post_interp_residual_channels'), in modo da poi eventualmente escluderli dalla ICA custom che ho creato (che comunque esegue anche una ulteriore interpolazione dei canali sporchi anche post-ICA, per cui anche quei canali sporchi che son dentro a 'post_interp_residual_channels' per le runs che son 'WARNING' dovrebbero essere poi ripuliti comunque anche loro, perché verrebbero esclusi dall'ICA ma al tempo stesso riceverebbero una ulteriore pulizia)
 

In [ ]:
# --- 0) Maschere coerenti con summarize_run_quality_residuals(decision_final) ---

#1) Conteggi senza errori

# Conteggi corretti
n_ok       = int((df_runs['decision_final'] == 'OK').sum())
n_warning  = int((df_runs['decision_final'] == 'WARNING').sum())
n_discard  = int((df_runs['decision_final'] == 'DISCARD').sum())

print(f"OK: {n_ok} | WARNING: {n_warning} | DISCARD: {n_discard}")


In [ ]:
n_warn_keep = int(((df_runs['decision_final'] == 'WARNING') &
                   (df_runs['interpolated_channels_n'] <= 8)).sum())
n_warn_drop = int(((df_runs['decision_final'] == 'WARNING') &
                   (df_runs['interpolated_channels_n'] > 8)).sum())

print(f"WARNING ≤8 interpolati: {n_warn_keep} | WARNING >8 interpolati: {n_warn_drop}")


In [ ]:
#2) Selezione + ricostruzione dataset (OK ∪ WARNING≤8) con bads settati ai residui

'''

Questa funzione:

seleziona OK + WARNING con ≤ 8 canali interpolati,
ricostruisce il nuovo contenitore EEG_for_ICA a partire da EEG_data_prep,
per le sole WARNING sovrascrive raw.info['bads'] con i soli residui finali (post_interp_residual_channels),
restituisce anche keep_df (la tabella delle run tenute) e un bads_log (audit).

Perché overwrite dei bads solo coi residui?

Dopo PyPREP hai già interpolato i canali pre/after-ref e, con reset_bads=True, raw.info['bads'] viene svuotato.
Per le run WARNING vuoi che l’ICA ignori solo i canali ancora residui: quindi è corretto sovrascrivere raw.info['bads'] con post_interp_residual_channels (filtrati ai nomi validi).
Le run OK non hanno residui → raw.info['bads'] resta vuoto.

'''

import pandas as pd
import ast

def _ensure_list(x):
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        try:
            v = ast.literal_eval(x)
            return v if isinstance(v, list) else []
        except Exception:
            return []
    return []

def select_and_build_dataset(df_runs: pd.DataFrame,
                             EEG_data_prep: dict,
                             max_interp_warn: int = 8):
    """
    Tieni:
      - tutte le run OK
      - le WARNING con interpolated_channels_n ≤ max_interp_warn

    Ricostruisce EEG_for_ICA con la stessa struttura annidata di EEG_data_prep.
    Per le sole WARNING, sovrascrive raw.info['bads'] con i canali in
    post_interp_residual_channels (filtrati ai nomi validi presenti in raw).

    Ritorna:
      - EEG_for_ICA  : dict annidato task->subject->run -> Raw oppure (Raw, labels)
      - keep_df      : dataframe delle run tenute (task, subject, run, decision_final, residui)
      - bads_log     : lista di dict (audit: per WARNING, cosa è stato marcato bad)
    """
    # --- maschere di selezione ---
    ok_mask = (df_runs['decision_final'] == 'OK')
    warn_keep_mask = ((df_runs['decision_final'] == 'WARNING') &
                      (df_runs['interpolated_channels_n'] <= max_interp_warn))

    # --- subset delle run da tenere ---
    cols_needed = ['task', 'subject', 'run', 'decision_final', 'post_interp_residual_channels']
    keep_df = df_runs.loc[ok_mask | warn_keep_mask, cols_needed].copy()

    # normalizza tipi e liste
    keep_df['subject'] = keep_df['subject'].astype(str)
    keep_df['run'] = keep_df['run'].astype(int)
    keep_df['post_interp_residual_channels'] = keep_df['post_interp_residual_channels'].apply(_ensure_list)

    EEG_for_ICA = {}
    bads_log = []

    # utility per inserimento nel contenitore
    def _put(container, t, s, r, val):
        container.setdefault(t, {}).setdefault(s, {})[r] = val

    # costruzione nuovo dataset
    for row in keep_df.itertuples(index=False):
        t, s, r = row.task, row.subject, row.run
        status = row.decision_final
        resid_list = row.post_interp_residual_channels or []

        # recupera dall'originale
        val = EEG_data_prep[t][s][r]
        if isinstance(val, tuple):
            raw_in, labels = val
        else:
            raw_in, labels = val, None

        raw = raw_in.copy()

        # WARNING: sovrascrivi i bads con i soli RESIDUI FINALI (validi)
        if status == 'WARNING':
            resid_valid = [ch for ch in resid_list if ch in raw.ch_names]
            raw.info['bads'] = sorted(set(resid_valid))
            # audit
            bads_log.append({
                'task': t, 'subject': s, 'run': r,
                'n_residual': len(resid_valid),
                'bads_set': raw.info['bads']
            })
        # OK: non tocchiamo raw.info['bads'] (dovrebbe essere vuoto dopo PREP)

        # inserisci nel nuovo contenitore
        _put(EEG_for_ICA, t, s, r, (raw, labels) if labels is not None else raw)

    # report rapido
    n_kept = sum(len(runs) for subj in EEG_for_ICA.values() for runs in subj.values())
    print(f"Run mantenute nel nuovo dataset: {n_kept}")

    return EEG_for_ICA, keep_df, bads_log

# ---- ESECUZIONE ----
EEG_for_ICA, keep_df, bads_log = select_and_build_dataset(
    df_runs=df_runs,
    EEG_data_prep=EEG_data_prep,
    max_interp_warn=8
)

# conteggi finali
print("OK tenute:", int((keep_df['decision_final'] == 'OK').sum()))
print("WARNING≤8 tenute:", int((keep_df['decision_final'] == 'WARNING').sum()))

In [ ]:
bads_log

In [ ]:
'''
4) Salvataggi su disco (opzionale)
'''

# --- 4) (Opzionale) salvataggi su disco ---

'''
Imposta la cartella di output e la crea se non esiste.
'''
import pickle, os
outdir = '/home/stefano/EEGMMIDB_Analyses/pyprep_outputs'
os.makedirs(outdir, exist_ok=True)

'''
Salva la nuova struttura annidata (solo OK+WARNING, con i WARNING già marcati) in un .pkl per riuso.
'''

with open(os.path.join(outdir, 'EEG_for_ICA.pkl'), 'wb') as f:
    pickle.dump(EEG_for_ICA, f, protocol=pickle.HIGHEST_PROTOCOL)
    
    
with open(os.path.join(outdir, 'keep_df.pkl'), 'wb') as f:
    pickle.dump(keep_df, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(os.path.join(outdir, 'bads_log.pkl'), 'wb') as f:
    pickle.dump(bads_log, f, protocol=pickle.HIGHEST_PROTOCOL)

#### **EEG Post PyPREP: Ispezione delle Strutture Dati**

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/pyprep_outputs/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_for_ICA.pkl', 'rb') as f:
    EEG_for_ICA = pickle.load(f)

In [ ]:
print(EEG_for_ICA.keys())
print(EEG_for_ICA['task_1'].keys())
print(EEG_for_ICA['task_1']['100'].keys())
print(type(EEG_for_ICA['task_1']['100'][0][0]))
print(type(EEG_for_ICA['task_1']['100'][0][1]))

In [ ]:
'''
Per verificare che la struttura del dizionario sia corretta e uguale per entrambe le collezioni (raw_collection e raw_collection_unfiltered), 
possiamo ispezionarla con uno script che:

Controlla i livelli del dizionario.
Stampa il numero di soggetti per ogni task.
Conta il numero di run per ciascun soggetto.
Mostra la shape dei dati EEG e delle annotazioni.
Verifica che i tipi siano coerenti: mne.io.Raw e np.ndarray.
Ecco un codice che puoi lanciare dopo aver caricato i due dizionari da file o averli creati:


Possiamo estendere la funzione inspect_raw_collection per:

raccogliere tutti i soggetti (subj) e tutte le run (run) disponibili nel collection,
mostrarli alla fine con np.unique per avere una lista completa e verificare quali soggetti e run siano presenti 
in ciascuna struttura (filtrata e non filtrata),
eventualmente restituire le liste uniche per confrontarle.


Nel tuo caso, gli annotations non sono un oggetto mne.Annotations, ma un array numpy.ndarray con forma (N, 5), dove:

N = 30 è il numero di eventi o segmenti annotati (tipicamente i 30 trial della run),
e le 5 colonne rappresentano probabilmente attributi di un array fatto così, ad esempio:
[  1.    4.1 656.    1.  656. ]


Vediamolo passo per passo e confrontiamolo con le informazioni dal paper PMC10998040, 
in particolare Tabella 2 e le descrizioni relative all’organizzazione temporale dei trial.

| Colonna | Significato                        | Esempio                            |
| ------- | ---------------------------------- | ---------------------------------- |
| `[0]`   | **Etichetta originale** del trial  | `1` (T0), `2` (T1), `3` (T2)       |
| `[1]`   | **Durata del trial (secondi)**     | `4.1`                              |
| `[2]`   | **Durata in campioni** (`samples`) | `656`                              |
| `[3]`   | **Indice di inizio trial**         | `657`                              |
| `[4]`   | **Indice di fine trial**           | `1312`                             |


'''

import numpy as np

def inspect_raw_collection(collection, name=""):
    print(f"\nInspecting structure: \033[1m{name}\033[0m\n" + "-"*50)
    
    all_subjects = []
    all_runs = []

    for task, subjects in collection.items():
        print(f"📁 Task: \033[94m{task}\033[0m → {len(subjects)} subjects")
        
        # Mappa etichette in base al task
        if task in ["task_1", "task_2"]:
            label_map = {1: "T0 (Rest)", 2: "T1 (Left Fist)", 3: "T2 (Right Fist)"}
        else:
            label_map = {1: "T0 (Rest)", 2: "T1 (Both Feet)", 3: "T2 (Both Fists)"}

        for subj, runs in subjects.items():
            all_subjects.append(subj)
            print(f"  └─ 👤 Subject {subj} → {len(runs)} runs")
            for run, (raw, annotations) in runs.items():
                all_runs.append(run)
                print(f"      └─ 🏃‍♂️ Run {run}:")
                print(f"         ├─ Raw type:      {type(raw)}")
                print(f"         ├─ Data shape:    {raw.get_data().shape}  (channels x time)")
                print(f"         ├─ Annotations:   {type(annotations)}, shape={annotations.shape}")
                if isinstance(annotations, np.ndarray):
                    print(f"           ├─ First annotation row: {annotations[0]}")
                    print()
                    print(f"             ├─ Unique stimolo_id [0]: {np.unique(annotations[:, 0])}")
                    print(f"             ├─ Durata trial secondi [1]: {np.unique(annotations[:, 1])}")
                    print(f"             ├─ Durata trial in samples [2]: {np.unique(annotations[:, 2])}\n")
                    print(f"             ├─ Start sample  [3]: {np.unique(annotations[:, 3])}\n")
                    print(f"             └─ End sample [4]: {np.unique(annotations[:, 4])}\n")
                print(f"         └─ Duration (s):  {raw.times[-1]:.2f}s")
                
            break  # Solo il primo soggetto per ogni task
        print()

    # Mostra soggetti e run unici
    unique_subjects = np.unique(all_subjects)
    unique_runs = np.unique(all_runs)

    print(f"\n📊 \033[1mRiepilogo {name}\033[0m")
    print(f"→ Soggetti unici: {len(unique_subjects)} → {unique_subjects}")
    print(f"→ Run uniche:     {len(unique_runs)} → {unique_runs}")
    print("✓ Done.\n")

    return unique_subjects, unique_runs

In [ ]:
#inspect_raw_collection(EEG_data_filtered, "Filtered")
#inspect_raw_collection(EEG_data_unfiltered, "Unfiltered (raw original)")
inspect_raw_collection(EEG_for_ICA, "Post PyPREP (ICA-ready)")

In [ ]:
print(EEG_for_ICA['task_1'].keys())
print()
print(EEG_for_ICA['task_1']['100'].keys())
print()
print(type(EEG_for_ICA['task_1']['100'][0]))
print()
print(len(EEG_for_ICA['task_1']['100'][0]))
print()
print(type(EEG_for_ICA['task_1']['100'][0][0]))
print(type(EEG_for_ICA['task_1']['100'][0][1]))
print(EEG_for_ICA['task_1']['100'][0][1].shape)

In [ ]:
EEG_for_ICA['task_1'].keys()

#### **EEG Preprocessing - Data Quality Check tra i files tra 1) EEG_post_pyprep_data e 2) EEG_for_ICA**

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/pyprep_outputs/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_post_pyprep_data.pkl', 'rb') as f:
    EEG_data_prep = pickle.load(f)
    
    
# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_for_ICA.pkl', 'rb') as f:
    EEG_for_ICA = pickle.load(f)

    
# Caricare l'intero dizionario annidato con pickle
with open(f'{path}keep_df.pkl', 'rb') as f:
    keep_df = pickle.load(f)


In [ ]:
print(EEG_data_prep.keys())
print(EEG_data_prep['task_1'].keys())
print(EEG_data_prep['task_1']['100'].keys())
print(type(EEG_data_prep['task_1']['100'][0][0]))
print(type(EEG_data_prep['task_1']['100'][0][1]))
print()
print()
print(EEG_for_ICA.keys())
print(EEG_for_ICA['task_1'].keys())
print(EEG_for_ICA['task_1']['100'].keys())
print(type(EEG_for_ICA['task_1']['100'][0][0]))
print(type(EEG_for_ICA['task_1']['100'][0][1]))

In [ ]:
'''
Ok e potresti includere anche una parte che 


1) se non trova i canali sopra soglia, di quelli residui post-PREP, mi scrive invece
i canali sopra soglia che trova, 

e plotta SOLO sia il canale residuo post post-PREP (in quel caso non sopra soglia)
ed i canali trovati sopra soglia? 


2) se trova che i canali residui post-PREP son sopra soglia, 
e poi mi trova anche altri canali sopra soglia (che non fanno parte di quelli residui post-PREP),
mi scrive sia i canali sopra soglia che trova in post-PREP che quelli sopra soglia che non fanno parte di quelli residui post-PREP,
e plotta SOLO sia i canale residui post post-PREP (in questo caso sopra soglia) sia i canali trovati sopra soglia
(che che non fanno parte di quelli residui post-PREP)?


perfetto — ecco la versione con while True che:

prende i residui post-PREP (da raw.info['bads'] o keep_df),
trova i canali sopra soglia (es. 100 µV) nella finestra scelta,

A) Stampa un riepilogo chiaro dei casi:

residui sopra soglia,
canali sopra soglia non residui,

B) Plotta SOLO:

se nessun residuo è sopra soglia: tutti i residui + i canali sopra soglia non residui;
se ci sono residui sopra soglia: residui sopra soglia + canali sopra soglia non residui;
se non c’è nulla sopra soglia e non ci sono residui: chiede un range di canali da plottare.


La soglia è interattiva (default 100 µV).

Nella traccia sopra:
residui = linea più spessa,
canali “> soglia” non-residui = spessore medio.

Nessuna modifica viene fatta ai Raw: qui stiamo solo ispezionando.
Se vuoi una modalità che aggiunge i canali > soglia a raw.info['bads'] (per un’ICA “augmented bads”), dillo e te la integro come toggle.

'''



import numpy as np
import matplotlib.pyplot as plt
import mne
from IPython.display import clear_output
import ast

# =============== Helpers ===============

def _ensure_list(x):
    """Rende 'x' sempre una lista (gestisce anche stringhe tipo "['F3','F4']")."""
    if isinstance(x, list): 
        return x
    if isinstance(x, str):
        try:
            v = ast.literal_eval(x)
            return v if isinstance(v, list) else []
        except Exception:
            return []
    return []

def get_raw(d, task, subj, run):
    """Recupera Raw o (Raw, labels) dalla struttura annidata e ritorna il Raw."""
    v = d[task][str(subj)][int(run)]
    return v[0] if isinstance(v, tuple) else v

def get_residual_channels_from_keep(keep_df, task, subject, run, raw=None):
    """
    Ritorna i canali residui post-PREP per (task, subject, run).
      1) Se 'raw' ha bads -> usa quelli
      2) Altrimenti fallback da keep_df['post_interp_residual_channels']
    """
    if raw is not None:
        bads = raw.info.get('bads') or []
        if isinstance(bads, list) and len(bads) > 0:
            return sorted([ch for ch in bads if ch in raw.ch_names])

    row = keep_df[(keep_df['task'] == task) &
                  (keep_df['subject'].astype(str) == str(subject)) &
                  (keep_df['run'].astype(int) == int(run))]
    if len(row) == 0:
        return []
    resid = _ensure_list(row.iloc[0]['post_interp_residual_channels'])
    if raw is not None:
        resid = [ch for ch in resid if ch in raw.ch_names]
    return sorted(resid)

def channels_above_threshold(raw: mne.io.Raw,
                             start_sec: float,
                             duration_sec: float,
                             threshold_uv: float) -> list:
    """
    Lista di canali con |amp| > threshold_uv in [start_sec, start_sec+duration_sec] (µV).
    """
    sfreq = raw.info['sfreq']
    start_samp = int(start_sec * sfreq)
    stop_samp  = int((start_sec + duration_sec) * sfreq)
    stop_samp  = min(stop_samp, raw.n_times)
    if stop_samp <= start_samp:
        return []
    data = raw.get_data()[:, start_samp:stop_samp] * 1e6  # µV
    mask = np.max(np.abs(data), axis=1) > float(threshold_uv)
    return [ch for ch, is_hi in zip(raw.ch_names, mask) if is_hi]

def plot_comparison(raw_for_ica: mne.io.Raw,
                    raw_full: mne.io.Raw,
                    start_sec: float = 0.0,
                    duration_sec: float = 5.0,
                    picks_by_name=None,
                    ch_start: int = 0,
                    ch_end: int = 0,
                    hi_set: set = None):
    """
    Confronto 2 pannelli: sopra EEG_for_ICA (residui = linewidth maggiore; canali > soglia = medio),
    sotto EEG_data_prep. 'hi_set' serve per evidenziare canali sopra soglia (non-residui).
    """
    sfreq = raw_for_ica.info['sfreq']
    start_samp = int(start_sec * sfreq)
    stop_samp  = int((start_sec + duration_sec) * sfreq)
    stop_samp  = min(stop_samp, raw_for_ica.n_times, raw_full.n_times)  # guardrail
    times = np.arange(start_samp, stop_samp) / sfreq
    microvolt_scale = 1e6

    # Scelta canali
    if picks_by_name and len(picks_by_name) > 0:
        picks_names = [ch for ch in picks_by_name if ch in raw_for_ica.ch_names]
        picks_idx   = [raw_for_ica.ch_names.index(ch) for ch in picks_names]
    else:
        ch_end = max(ch_end, ch_start)
        picks_idx   = list(range(ch_start, ch_end + 1))
        picks_names = [raw_for_ica.ch_names[i] for i in picks_idx]

    fig, (ax_top, ax_bot) = plt.subplots(2, 1, sharex=True, figsize=(12, 8))

    # TOP: EEG_for_ICA — linewidth:
    #   residui (in bads)      -> 2.8
    #   sopra soglia (hi_set)  -> 2.0
    #   altri                   -> 1.0
    bads = set(raw_for_ica.info.get('bads') or [])
    hi_set = set(hi_set or [])
    for i, ch_idx in enumerate(picks_idx):
        ch_name = picks_names[i]
        data = raw_for_ica.get_data(picks=[ch_idx]).squeeze()[start_samp:stop_samp] * microvolt_scale
        if ch_name in bads:
            lw = 2.8
        elif ch_name in hi_set:
            lw = 2.0
        else:
            lw = 1.0
        ax_top.plot(times, data, label=ch_name, linewidth=lw, alpha=0.95)
    ax_top.set_title("EEG_for_ICA (post-PyPREP) — residui (spesso) & > soglia (medio)")
    ax_top.set_ylabel("µV")

    # BOTTOM: EEG_data_prep
    for ch_idx, ch_name in zip(picks_idx, picks_names):
        data = raw_full.get_data(picks=[ch_idx]).squeeze()[start_samp:stop_samp] * microvolt_scale
        ax_bot.plot(times, data, label=ch_name, alpha=0.9)
    ax_bot.set_title("EEG_data_prep (post-PyPREP)")
    ax_bot.set_xlabel("Tempo (s)")
    ax_bot.set_ylabel("µV")

    # legenda unica
    handles, labels = ax_bot.get_legend_handles_labels()
    if labels:
        fig.legend(handles, labels, loc='lower center',
                   ncol=min(len(labels), 6), bbox_to_anchor=(0.5, -0.05))
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)
    plt.show()


# =============== MENU INTERATTIVO ===============
while True:
    clear_output(wait=True)
    cmd = input("Digita 'ok' per plottare, 'stop' per uscire: ").strip().lower()
    if cmd == 'stop':
        print("Bye!")
        break
    if cmd != 'ok':
        print("Comando non riconosciuto.")
        continue

    # Selezione task
    tasks = list(EEG_for_ICA.keys())
    print(f"\nTask disponibili: {tasks}")
    task = input("Scegli task: ").strip()
    if task not in EEG_for_ICA:
        print("Task non valido.")
        continue

    # Selezione soggetto
    subjects = list(EEG_for_ICA[task].keys())
    print(f"\nSoggetti disponibili: {subjects}")
    sbj = input("Scegli soggetto: ").strip()
    if sbj not in EEG_for_ICA[task]:
        print("Soggetto non valido.")
        continue

    # Selezione run
    available_runs = sorted(list(EEG_for_ICA[task][sbj].keys()))
    print(f"\nRun disponibili: {available_runs}")
    try:
        run = int(input("Scegli run: ").strip())
    except Exception:
        print("Run non valida.")
        continue
    if run not in available_runs:
        print("Run non valida.")
        continue

    # Parametri analisi
    try:
        start_sec    = float(input("\nStart time (s) [default 0]: ") or 0.0)
        duration_sec = float(input("Durata (s) [default 5]: ") or 5.0)
        thr_uv       = float(input("Soglia (µV) [default 100]: ") or 100.0)
    except Exception:
        print("Valori inseriti non validi.")
        continue

    # Raw e insiemi di canali
    raw_for_ica = get_raw(EEG_for_ICA,   task, sbj, run)
    raw_full    = get_raw(EEG_data_prep, task, sbj, run)

    R = set(get_residual_channels_from_keep(keep_df, task, sbj, run, raw_for_ica))  # residui post-PREP
    T = set(channels_above_threshold(raw_for_ica, start_sec, duration_sec, thr_uv)) # > soglia nella finestra

    T_R     = sorted(R & T)   # residui sopra soglia
    T_notR  = sorted(T - R)   # sopra soglia non-residui
    R_only  = sorted(R)       # tutti i residui (anche non sopra soglia)

    # ---- Report testuale ----
    print("\n=== RIEPILOGO CANALI ===")
    print(f"Residui post-PREP (R): {R_only if R_only else 'nessuno'}")
    if T:
        print(f"Canali > {thr_uv:.1f} µV (T): {sorted(T)}")
    else:
        print(f"Nessun canale > {thr_uv:.1f} µV nella finestra selezionata.")

    if T_R:
        print(f"Residui > {thr_uv:.1f} µV (R ∩ T): {T_R}")
    else:
        print(f"Nessun residuo > {thr_uv:.1f} µV.")

    if T_notR:
        print(f"> {thr_uv:.1f} µV NON-residui (T \\ R): {T_notR}")
    else:
        print(f"Nessun canale > {thr_uv:.1f} µV al di fuori dei residui.")

    # ---- Scelta cosa plottare (logica richiesta) ----
    picks = []
    hi_set = set()  # per evidenziare canali > soglia

    if T:  # esistono canali sopra soglia
        if T_R:  # residui sopra soglia presenti
            # Plotta SOLO residui sopra soglia + non-residui sopra soglia
            picks = sorted(set(T_R) | set(T_notR))
            hi_set = set(picks)  # tutti i picks qui sono > soglia
            print("\nPlot: residui sopra soglia + non-residui sopra soglia.")
        else:
            # Nessun residuo sopra soglia -> plotta T_notR + TUTTI i residui (anche non > soglia)
            picks = sorted(set(T_notR) | set(R_only))
            hi_set = set(T_notR)  # evidenzia solo i > soglia non-residui
            print("\nPlot: tutti i residui (non > soglia) + canali > soglia NON-residui.")
    else:
        if R:
            # Nessun canale > soglia, ma ci sono residui: plotta solo i residui
            picks = R_only
            hi_set = set()  # nessuno > soglia
            print("\nPlot: nessun canale > soglia, mostro solo i residui.")
        else:
            # Nessun residuo e nessun > soglia: fallback manuale
            ch_names = raw_for_ica.ch_names
            print("\nNessun residuo e nessun canale > soglia.")
            print("Elenco canali:", "  ".join(f"\033[1m{i}\033[0m:{ch}" for i, ch in enumerate(ch_names)))
            max_idx = len(ch_names) - 1
            try:
                ch_start = int(input(f"Inizio canale [0–{max_idx}] [default 0]: ") or 0)
                ch_end   = int(input(f"Fine canale [0–{max_idx}] [default {ch_start}]: ") or ch_start)
            except Exception:
                print("Intervallo canali non valido.")
                continue
            plot_comparison(raw_for_ica, raw_full, start_sec, duration_sec,
                            ch_start=ch_start, ch_end=ch_end, hi_set=set())
            continue

    # Esegui il plot sui canali selezionati
    if len(picks) == 0:
        print("\nNulla da plottare con i criteri scelti.")
        continue

    plot_comparison(raw_for_ica, raw_full, start_sec, duration_sec,
                    picks_by_name=picks, hi_set=hi_set)


#### **EEG Post PyPREP - Custom ICA Pipeline**

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/pyprep_outputs/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_for_ICA.pkl', 'rb') as f:
    EEG_for_ICA = pickle.load(f)

In [ ]:
print(EEG_for_ICA.keys())
print(EEG_for_ICA['task_1'].keys())
print(EEG_for_ICA['task_1']['100'].keys())
print(type(EEG_for_ICA['task_1']['100'][0][0]))
print(type(EEG_for_ICA['task_1']['100'][0][1]))

In [ ]:
print(EEG_for_ICA.keys())

In [ ]:
''' 
                                                            ATTENZIONE: UFFICIALE SENZA TRASCRIZIONI
                                                                        PER USO CON SPYDER

                                                                        CON PLOT STATICI
                                                                          
                                                                        OLD 3 AGOSTO FINO A 14:20
                                                                        DOPO CANCELLAZIONI 
                                                                          FUNZIONI VECCHIE

                                                                        


                                                                        
📁 File locali
Questi sono i file che salvi sul tuo computer locale, dentro base_path:

base_path = "/home/stefano/Downloads/"s

post_ica_path = f"{base_path}post_ica_data.pkl"
ica_models_path = f"{base_path}ica_models.pkl"
bad_components_path = f"{base_path}ica_bad_components.pkl"

completed_runs_file = f"{base_path}processed_runs_file.pkl"


Ti danno come output:

/home/stefano/Downloads/post_ica_data.pkl

/home/stefano/Downloads/ica_models.pkl

/home/stefano/Downloads/ica_bad_components.pkl

/home/stefano/Downloads/processed_runs_file.pkl


🌐 File sul server remoto


Quando chiami:


send_to_server(
    [post_ica_path, ica_models_path, bad_components_path],
    remote_user, remote_host, remote_path
)
viene generato un comando scp simile a questo:


scp /home/stefano/Downloads/post_ica_data.pkl  stefano@160.80.53.119:/home/stefano/EEGMMIDB_Analyses/
Il comportamento è:

il file viene copiato da /home/stefano/Downloads/...
fino a /home/stefano/EEGMMIDB_Analyses/ sul server remoto

Lo stesso succede anche per ica_models.pkl e ica_bad_components.pkl.

Il file processed_runs_file.pkl non viene inviato al server, serve solo per salvare localmente le run completate.

🧠 Riepilogo

| File                      | Percorso locale                                                 | Percorso nel server (`scp`)                              |
| ------------------------- | --------------------------------------------------------------- | -------------------------------------------------------- |
| post\_ica\_data.pkl       | `/home/stefano/Downloads/post_ica_data.pkl`                     | `/home/stefano/EEGMMIDB_Analyses/post_ica_data.pkl`      |
| ica\_models.pkl           | `/home/stefano/Downloads/ica_models.pkl`                        | `/home/stefano/EEGMMIDB_Analyses/ica_models.pkl`         |
| ica\_bad\_components.pkl  | `/home/stefano/Downloads/ica_bad_components.pkl`                | `/home/stefano/EEGMMIDB_Analyses/ica_bad_components.pkl` |
| processed\_runs\_file.pkl | `/home/stefano/Downloads/processed_runs_file.pkl` (solo locale) | —                                                        |


✅ In uso
Il codice salva i file locali, li invia al server, e poi rimuove le run già processate dal file locale.

Il server mantiene sempre una copia completa di tutti i file inviati, anche se li elimini dal locale.

Il file processed_runs_file.pkl serve solo come tracking locale, e non viene copiato sulla macchina remota.


Perfetto vorrei solo aggiungere delle chiarificazioni ai plots.. ossia...

1) nella prima colonna, mi devo chiedo se l'asse x conviene metterlo come "time (s) " mentre l'asse y non so se tenere il range minimo e massimo

2) nella seconda e terza colonna, mi devo chiedo se l'asse x conviene metterlo come "time (s) " mentre l'asse y direi che sia una indicazione 
dei microvolts ossia mv (con la m greca magari).. 
per verificare però con certezza il range di voltaggio del potenziale, allora conviene che esploro dai dati originali il range, giusto? 
così che, se son sicuro che sia in microvolt (magari identificabile dall'unità di misura quanticabile ossia a quanti microvolts corrisponde un volt e
vedere se effettivamente nei miei dati c'è quel range là?) 

E poi, verificato questo, metterò sull'asse y di 2° e 3° colonna una indicazione in range microvolt su asse y... giusto?


| colonna                             | cosa stai tracciando                                 | unità “fisica”                                                                               | che etichetta mettere?                                                                 |
| ----------------------------------- | ---------------------------------------------------- | -------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------- |
| **ICA Sources**                     | coefficienti delle sorgenti (mix lineare dei canali) | stesso ordine di grandezza dell’EEG (Volt → µV) ma non hanno *un significato fisico diretto* | `µV (ICA)` oppure semplicemente `a.u.` se vuoi ricordare che non è un segnale misurato |
| **Segnale originale / ricostruito** | canali EEG reali                                     | Volt ⇒ tipicamente 10-100 µV                                                                 | `µV`                                                                                   |


Regola spiccia: se stai già moltiplicando i tracciati per 1e6 (oppure MNE lo fa per te) puoi etichettare l’asse Y con µV.
Gli assi X vanno invece sempre in secondi (Time (s)).


'''






'''
Di seguito trovi tutto il necessario per:

Controllare che il segnale ricostruito non superi ±100 µV.

Interpolare (facoltativamente o in automatico) i canali che sforano, prima di passare al prompt ICA_confirm.

Gli snippet sono pensati per essere incollati nella tua stessa cella di definizioni;
basta poi integrare poche righe nel loop principale subito dopo raw_clean = ica.apply(raw_copy).







Come funziona l’interpolazione in MNE
Quando chiami:

    raw_out.info["bads"] = chan_names
    raw_out = raw_out.interpolate_bads(reset_bads=reset_bads)
    
    
MNE:

Identifica le posizioni degli elettrodi (serve un montage valido in raw_out.info).

Per i canali marcati come bads calcola i valori per ogni istante di tempo usando una interpolazione spaziale basata sulle potenzialità dei canali buoni circostanti.

Per EEG usa tipicamente spherical spline interpolation (Perrin et al.), cioè una soluzione liscia sullo scalpo costruita dai vicini.

Sostituisce nel dato i campioni dei canali “bad” con i valori interpolati.

Se reset_bads=True, svuota info['bads'] alla fine (i canali non risultano più “bad”).

In pratica: non stima un filtro nel tempo, ma ricostruisce spazialmente i canali “guasti” a partire dagli altri elettrodi a ogni time point.
Le unità restano quelle di partenza (nel tuo caso µV). 

L’ICA non viene modificata: stai operando sul segnale ricostruito (raw_clean) dopo ica.apply().


    def interpolate_selected_channels(raw_in, channels, by="index", reset_bads=True, verbose=True):
        raw_out = raw_in.copy()
        # 1) Risolve i nomi canale (da indici o da nomi)
        if by == "index":
            chan_names = [raw_out.ch_names[i] for i in channels]
        else:
            chan_names = list(channels)

        # 2) Valida che i nomi esistano
        missing = [ch for ch in chan_names if ch not in raw_out.ch_names]
        if missing:
            raise ValueError(...)

        # 3) Marca quei canali come "bads"
        raw_out.info["bads"] = chan_names
        if verbose:
            print("Interpolo i canali:", raw_out.info["bads"])

        # 4) Interpola i "bads" usando gli altri canali e le posizioni elettrodiche
        raw_out = raw_out.interpolate_bads(reset_bads=reset_bads)

        # 5) Restituisce una *copia* con i canali rimpiazzati dalla versione interpolata
        return raw_out
        
        

Nel tuo flusso succede questo:

Applichi l’ICA → raw_clean = ica.apply(raw_copy).

Controlli le ampiezze ±100 µV → ottieni bad_idx / bad_names.

Se decidi di interpolare, chiami:

    raw_clean = interpolate_selected_channels(
        raw_in=raw_clean, channels=bad_idx, by="index", reset_bads=True
    )
e sovrascrivi raw_clean con la versione interpolata.



Riplotti con plot_sources_vs_original_vs_reconstructed(...) per vedere l’effetto.

Requisiti/attenzioni pratiche
Montage obbligatorio: senza posizioni 3D degli elettrodi l’interpolazione non può funzionare correttamente. 
Assicurati di avere già raw.set_montage(...) (o che le posizioni siano nel file).

Interpolare pochi canali: l’interpolazione è affidabile se i “bads” sono pochi e ben circondati da canali buoni. 
                            Interpolare molti canali contigui degrada la qualità.

Tipi di canale: MNE interpola i canali EEG. Se nel tuo raw ci sono EOG/EMG/etc., non passarli alla funzione. (Se vuoi, si può limitare ai soli EEG).

reset_bads:

    True → dopo l’interpolazione info['bads'] viene svuotato (di solito ciò che si vuole).
    False → tieni traccia di quali canali sono stati interpolati.



(Opzionale) Versione più robusta della funzione
Se vuoi: 1) preservare eventuali “bads” già presenti; 2) limitarti ai soli EEG; 3) restituire anche quali canali sono stati interpolati:


    def interpolate_selected_channels(raw_in, channels, by="index", reset_bads=True, verbose=True):
        raw_out = raw_in.copy()

        # Limita ai soli EEG
        eeg_names = set([raw_out.ch_names[i] for i in mne.pick_types(raw_out.info, eeg=True, meg=False)])
        if by == "index":
            chan_names_req = [raw_out.ch_names[i] for i in channels]
        else:
            chan_names_req = list(channels)

        # Tieni solo canali EEG e presenti
        chan_names = [ch for ch in chan_names_req if ch in eeg_names]
        missing = [ch for ch in chan_names_req if ch not in raw_out.ch_names]
        if missing:
            raise ValueError(f"Canali inesistenti: {missing}")

        # Unione con bads già presenti
        already_bads = set(raw_out.info.get("bads", []))
        raw_out.info["bads"] = list(already_bads.union(chan_names))

        if verbose:
            print("🛠️ Interpolo i canali (EEG):", raw_out.info["bads"])

        raw_out = raw_out.interpolate_bads(reset_bads=reset_bads)

        return raw_out
Messaggio chiave
Dopo l’ICA, prima della conferma: marchi i canali oltre soglia come “bad”, MNE li ricostruisce spazialmente dai vicini, ottieni un raw_clean interpolato, riplotti e poi salvi.

Non tocca l’ICA già calcolata; agisce solo sul segnale pulito nel dominio dei canali.








IO: volevo solo capire se con 'auto' effettivamente interpola quelli che stampa già come quelli fuori range appunto 





CHATGPT:

1 – Le posizioni elettrodiche (“montage”)
Topomap: la chiamata plot_topomap(...) funziona anche senza un vero montage, perché se i canali non hanno coordinate 3-D MNE li disegna comunque in cerchio.

interpolate_bads: invece ha davvero bisogno delle coordinate.

Se mancano, MNE solleva un’eccezione del tipo


RuntimeError: Cannot interpolate bad channels. No location available.
Quindi sì: se nel tuo raw non ci fosse già raw.info['chs'][i]['loc'] valorizzato, l’interpolazione fallirebbe.

Il fatto che non dia errori indica che il montage c’è (o che lo hai impostato con raw.set_montage(...) in fase di import).

Check rapido


# se non sei sicuro:
print(raw.get_montage())    # None se non c’è nulla
Se stampa qualcosa (es. DigitizedMontage | 64 channels), sei a posto.

2 – Cosa succede quando scegli “auto”
check_recon_amplitude_uV ritorna bad_idx, cioè gli indici dei canali che superano ±100 µV.

Col comando auto fai, letteralmente:


raw_clean = interpolate_selected_channels(
    raw_in=raw_clean,
    channels=bad_idx,     # <-- proprio quelli segnalati
    by="index",
    reset_bads=True
)
Dentro interpolate_selected_channels:

Converte quegli indici in nomi (chan_names).

Li mette in raw_out.info["bads"].

Chiama raw_out.interpolate_bads(...).

Restituisce la copia con i canali ricostruiti.

Quindi “auto” interpolerà esattamente i canali che il check ha stampato.



IO:


ok ma quindi se io metto false, i dati di quei canali bad mi vengono interpolati, il plot DOPO l'interpolazione avrà i dati correttamente interpolati 
MA comunque questa info io la posso 'rintracciare nei metadati di quella run specifica di quel task e soggetto successivamente, 
SE volessi ricordarmi quali canali erano stati interpolati?

cioè nella mia pipeline in caso successivamente, dopo potrei ri-prelevare dai metadati di quella  di quella run specifica di quel task e soggetto
che QUEI CANALI erano bads ed erano stati interpolati?




CHATGPT:

Esattamente — con reset_bads=False ottieni entrambe le cose:

| Cosa rimane nel file `.raw` salvato                                                          | Dove lo trovi      |
| -------------------------------------------------------------------------------------------- | ------------------ |
| **Segnale** dei canali “guasti” ricostruito (quindi il plot dopo l’interpolazione è a posto) | `raw.get_data()`   |
| **Etichetta** che quei canali erano “bad” al momento dell’interpolazione                     | `raw.info['bads']` |


Come recuperarlo in un secondo momento

# Carichi la run dal pickle
raw_clean, ann = post_ica_master[task_id][subject_id][run_id]

# I canali che hai interpolato all’epoca
interpolated_channels = raw_clean.info['bads']
print("Canali marcati come bad/interpolati:", interpolated_channels)


Se all’epoca avevi usato reset_bads=True, la lista sarebbe vuota; con
False resta popolata e puoi sempre risalire a quali elettrodi sono stati
ritoccati.


Suggerimento per la tua pipeline

Se vuoi essere sicuro di non perdere l’informazione anche nel caso in cui
domani decida di fare reset_bads=True, puoi salvarla a parte, subito dopo
l’interpolazione:

# dopo interpolate_selected_channels(...)
interpolated_now = list(raw_clean.info['bads'])
append_pickle(
    {task: {subject: {run: interpolated_now}}},
    interpolated_chans_path,   # un nuovo pkl dedicato, se vuoi
    return_master=False
)
Ma se per te va bene che resti dentro raw.info['bads'], basta portare avanti
reset_bads=False: MNE conserverà quel promemoria insieme ai dati.



'''


import mne
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
import subprocess
import os


import numpy as np
from mne.viz import plot_topomap

from IPython.display import clear_output  

import io


import collections  # serve per il merge ricorsivo


#1 · Funzione di check ampiezza ±100 µV

#1.1) Riconversione dinamica in MICROVOLT (i dati di partenza son in VOLT!)

def get_data_uV(raw, picks=None):
    return raw.get_data(picks=picks) * 1e6  # V → µV

def check_recon_amplitude_uV(raw_obj, threshold_uV=100.0, picks=None, verbose=True):
    """
    Ritorna:
        bad_idx  – lista indici canali > soglia
        bad_names – nomi canali corrispondenti
        peak_abs_uV – array picco |µV| per ciascun canale controllato
    """
    
    data = get_data_uV(raw_obj, picks=picks)   # <-- ora in µV (lo confermi con il tuo test)
    peak_abs_uV = np.max(np.abs(data), axis=1)

    if picks is None:
        idx = np.arange(len(raw_obj.ch_names))
        names = raw_obj.ch_names
    else:
        idx = np.array(picks, dtype=int)
        names = [raw_obj.ch_names[i] for i in idx]

    bad_mask = peak_abs_uV > threshold_uV
    bad_idx = idx[bad_mask].tolist()
    bad_names = [names[i] for i, m in enumerate(bad_mask) if m]

    if verbose:
        if bad_names:
            print(f"⚠️  Canali oltre ±{threshold_uV} µV:", bad_names)
        else:
            print(f"✅ Tutti i canali entro ±{threshold_uV} µV")

    return bad_idx, bad_names, peak_abs_uV


#2 · Funzione di interpolazione selettiva

def interpolate_selected_channels(raw_in, channels, by="index", reset_bads=False, verbose=True, final_reref=False):  # + final_reref
    """
    Interpola i canali indicati (indici o nomi).
    Ritorna una *copia* di raw con i canali interpolati.
    """
    raw_out = raw_in.copy()
    if by == "index":
        chan_names = [raw_out.ch_names[i] for i in channels]
    else:
        chan_names = list(channels)

    # Verifica che esistano
    missing = [ch for ch in chan_names if ch not in raw_out.ch_names]
    if missing:
        raise ValueError(f"Canali inesistenti: {missing}")

    raw_out.info["bads"] = chan_names
    if verbose:
        print("🛠️  Interpolo i canali:", raw_out.info["bads"])
    raw_out = raw_out.interpolate_bads(reset_bads=reset_bads)
    
    
    '''
    …e dove la chiami (auto o “idx”), puoi passare final_reref=True per farla lì dentro e togliere la chiamata separata.
    '''
    
    if final_reref:                                  # + opzionale: reref finale qui
        raw_out.set_eeg_reference('average', projection=False)
    
    return raw_out



# Helper per accodare (deep-merge) i dati nel pickle

def append_pickle(obj_to_add: dict, pkl_path: str, return_master = False):

    """Accoda obj_to_add al pickle (merge task→subject→run)."""
    if os.path.exists(pkl_path):
        with open(pkl_path, "rb") as f:
            master = pickle.load(f)
    else:
        master = {}
        # merge ricorsivo
        
    for task, subj_dict in obj_to_add.items():
        master.setdefault(task, {})
        for subj, run_dict in subj_dict.items():
            master[task].setdefault(subj, {})
            master[task][subj].update(run_dict)
            
    with open(pkl_path, "wb") as f:
        pickle.dump(master, f)
        f.flush(); os.fsync(f.fileno())
        
    if return_master:
        return master

# --- FUNZIONE locale per plottare tutte le ICA components in una sola figura ---
def plot_all_components_single_figure(ica, n_cols=8, title="ICA Components"):

    '''
    components --> restituisce la matrice (n_canali × n_componenti)
    '''
    components = ica.get_components()
    n_components = components.shape[1]
    n_rows = int(np.ceil(n_components / n_cols))

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 2, n_rows * 2))
    axes = axes.flatten()

    for idx, ax in enumerate(axes):
        if idx < n_components:
            plot_topomap(components[:, idx], ica.info, axes=ax, show=False)
            ax.set_title(f"ICA {idx}", fontsize=8)
        else:
            ax.axis("off")

    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

    
#✅ 1. Plot statico delle sorgenti ICA su un range temporale limitato

def plot_static_ica_sources(ica, raw, picks, start_time=0, end_time=15, task=None, subject=None, run=None):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time * sfreq)
    stop_sample = int((start_time + end_time) * sfreq)

    # Estrai le sorgenti ICA (componenti selezionate)
    sources = ica.get_sources(raw).get_data()[picks, start_sample:stop_sample]

    # 🛡️ Verifica se ci sono sorgenti da plottare
    if sources.shape[0] == 0:
        print("⚠️ Nessuna sorgente ICA da plottare (picks vuoto o indici non validi)")

    ymin, ymax = np.min(sources), np.max(sources)  # Y-range comune
    
    # Plot
    fig, axes = plt.subplots(len(picks), 1, figsize=(12, len(picks) * 1.5), sharex=True)
    for idx, ax in enumerate(axes):
        ax.plot(np.linspace(start_time, start_time + end_time, sources.shape[1]), sources[idx])
        ax.set_ylabel(f"ICA {picks[idx]}")
        ax.set_xlim(start_time, start_time + end_time)
        ax.set_ylim(ymin, ymax)

    plt.suptitle(f"ICA Sources — Task {task}, Subject {subject}, Run {run}", y=1.02)
    fig.subplots_adjust(top=0.93)
    plt.xlabel("Time (s)")
    plt.tight_layout()
    plt.show()

#✅ 2. Plot statico del segnale ricostruito su canali EEG selezionati


def plot_static_reconstructed_signal(raw_clean, channel_indices, start_time=0, end_time=15, task=None, subject=None, run=None):
    sfreq = raw_clean.info['sfreq']
    start_sample = int(start_time * sfreq)
    stop_sample = int((start_time + end_time) * sfreq)

    data = raw_clean.get_data(picks=channel_indices)[:, start_sample:stop_sample] * 1e6  # V → µV
    ymin, ymax = np.min(data), np.max(data)

    fig, axes = plt.subplots(len(channel_indices), 1, figsize=(12, len(channel_indices) * 1.5), sharex=True)
    for idx, ax in enumerate(axes):
        ax.plot(np.linspace(start_time, start_time + end_time, data.shape[1]), data[idx])
        ax.set_ylabel(raw_clean.ch_names[channel_indices[idx]])
        ax.set_xlim(start_time, start_time + end_time)
        ax.set_ylim(ymin, ymax)
        

    plt.suptitle(f"Segnale Ricostruito — Task {task}, Subject {subject}, Run {run}", y=1.02)
    fig.subplots_adjust(top=0.93)
    plt.xlabel("Time (s)")
    plt.tight_layout()
    plt.show()



#Ecco la versione “adattiva” della funzione, con
#asse X «Time (s)» sempre mostrato sull’ultima riga effettivamente popolata di ciascuna colonna;
#etichette Y già come le avevi definite (µV per EEG);
#nessuna modifica alla logica del toggle same_scale_eeg

#Come funziona

#| Colonna | Riga dove si mette l’etichetta “Time (s)” |
#| ------- | ----------------------------------------- |
#| ICA     | `last_row_ic = len(picks_sources) - 1`    |
#| Orig.   | `len(picks_channels) - 1`                 |
#| Ricost. | idem Orig.                                |

#Se n_rows > len(picks_channels) (succede se mostri più IC che canali),
#l’etichetta rimane comunque sull’ultima riga che contiene dati EEG; 
#le righe vuote ereditano l’asse ma non visualizzano label duplicati.
#Così i plot restano ordinati indipendentemente da quanti componenti o canali scegli di visualizzare


def plot_sources_vs_original_vs_reconstructed(
    ica,
    raw_for_sources,          # raw usato nel fit ICA (es. raw_copy)
    raw_original,             # raw originale PRIMA dell’ICA
    raw_clean,                # raw dopo ica.apply(...)
    picks_sources, picks_channels,
    start_time, end_time,
    task=None, subject=None, run=None,
    same_scale_eeg=True,
    return_bytes = True
):
    # -------------------------- estrazione --------------------------
    sfreq = raw_for_sources.info['sfreq']
    start_sample = int(start_time * sfreq)
    stop_sample  = int((start_time + end_time) * sfreq)

    sources  = ica.get_sources(raw_for_sources).get_data()[
                 picks_sources, start_sample:stop_sample]
    original = raw_original.get_data(picks=picks_channels)[:, start_sample:stop_sample]* 1e6  # V→µV
    recon    = raw_clean.get_data(picks=picks_channels)[:, start_sample:stop_sample]* 1e6  # V→µV

    # -------------------------- limiti Y --------------------------
    y_min_src, y_max_src = sources.min(), sources.max()

    if same_scale_eeg:
        y_min_eeg = min(original.min(), recon.min())
        y_max_eeg = max(original.max(), recon.max())
        y_min_ori = y_min_rec = y_min_eeg
        y_max_ori = y_max_rec = y_max_eeg
    else:
        y_min_ori, y_max_ori = original.min(), original.max()
        y_min_rec, y_max_rec = recon.min(),    recon.max()

    # -------------------------- plotting --------------------------
    n_rows = max(len(picks_sources), len(picks_channels))
    fig, axes = plt.subplots(n_rows, 3, figsize=(18, n_rows * 1.8), sharex=True)

    # --- col. 0 – ICA sources
    for i in range(len(picks_sources)):
        ax = axes[i, 0] if n_rows > 1 else axes[0]
        ax.plot(np.linspace(start_time, start_time + end_time, sources.shape[1]), sources[i])
        ax.set_ylabel(f"ICA {picks_sources[i]}")
        ax.set_ylim(y_min_src, y_max_src)
        
    # etichetta X per la colonna 0 (sempre ultima riga effettiva delle IC)
    last_row_ic = len(picks_sources) - 1
    axes[last_row_ic, 0].set_xlabel("Time (s)")

    # --- col. 1 – originale
    for i in range(len(picks_channels)):
        ax = axes[i, 1] if n_rows > 1 else axes[1]
        ax.plot(np.linspace(start_time, start_time + end_time, original.shape[1]), original[i])
        ax.set_ylabel(f"{raw_original.ch_names[picks_channels[i]]} (µV)")
        ax.set_ylim(y_min_ori, y_max_ori)
    # etichetta X ultima riga effettiva degli EEG
    axes[len(picks_channels) - 1, 1].set_xlabel("Time (s)")

    # --- col. 2 – ricostruito
    for i in range(len(picks_channels)):
        ax = axes[i, 2] if n_rows > 1 else axes[2]
        ax.plot(np.linspace(start_time, start_time + end_time, recon.shape[1]), recon[i])
        ax.set_ylabel(f"{raw_original.ch_names[picks_channels[i]]} (µV)")
        ax.set_ylim(y_min_rec, y_max_rec)
    axes[len(picks_channels) - 1, 2].set_xlabel("Time (s)")

    # -------------------------- titoli & layout --------------------------
    axes[0, 0].set_title("ICA Sources")
    axes[0, 1].set_title("Segnale Originale")
    axes[0, 2].set_title("Segnale Ricostruito")

    fig.suptitle(
        f"Confronto Completo — Task {task}, Subject {subject}, Run {run}",
        fontsize=14, y=1.02
    )
    fig.subplots_adjust(top=0.93)
    plt.tight_layout()
    plt.show()

    # …  tutta la parte di plotting rimane invariata  …

    plt.tight_layout()
    if not return_bytes:                    # ❶  caso “vecchio”: mostra e basta
        plt.show()
        return None

    # ❷  nuovo: salva in buffer e restituisci bytes
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=150)
    buf.seek(0)
    img_bytes = buf.getvalue()
    plt.close(fig)
    buf.close()
    return img_bytes                        # <-- eccoli



'''
Ecco un modo per rendere il range Y completamente dinamico in base ai canali che, dopo interpolazione, restano sopra soglia, aggiungendo un margine delta:

Come funziona
Estrai il segnale ricostruito in µV e prendi solo i canali ancora “bad” (post_bad_idx).

Calcoli max_bad = max(abs(recon[bad_channels])).

Applichi un range Y uniforme per tutti i subplot ricostruiti:

[ -(max_bad + delta_uV), +(max_bad + delta_uV) ]
Tracci le righe tratteggiate a ± threshold_uV.

In questo modo:

Il plot si “adatta” automaticamente all’ampiezza residua degli artefatti.

Vedi esattamente se e quanto i canali post-interpolazione superano ±100 µV.

Il margine delta_uV evita che le tracce tocchino i bordi del grafico.

Puoi naturalmente incorporare questa funzione al posto di quella precedente, passando post_bad_idx preso da check_recon_amplitude_uV dopo l’interpolazione.




perfetto — ti lascio un diff puntuale (stile unified) + spiegazione chiara di cosa cambia e perché.
L’obiettivo è: 

1) evitare l’IndexError quando passi indici “globali” dei canali a un plot che usa un sottoinsieme (picks_channels), 
2) forzare il plot di TUTTI i canali nel check post-ICA,
3) darti una funzione di plot “robusta” che rimappa automaticamente gli indici globali in indici locali (e accetta anche i nomi dei canali).


'''

def plot_sources_vs_original_vs_reconstructed_post_interp(
    ica,
    raw_for_sources,
    raw_original,
    raw_clean,
    picks_sources,
    picks_channels,
    start_time,
    end_time,
    
    #post_bad_idx,
    
    post_bad_idx = None,          # lista di indici canali ancora > threshold
    post_bad_names = None,   # oppure, in alternativa, i NOMI dei canali
    
    threshold_uV=100.0,
    delta_uV=10.0,         # margine extra da aggiungere al range dinamico
    same_scale_eeg=True,
    task=None,        # ← aggiungi
    subject=None,     # ← aggiungi
    run=None,         # ← aggiungi
    return_bytes=True
):
    sfreq = raw_for_sources.info['sfreq']
    start_sample = int(start_time * sfreq)
    stop_sample  = int((start_time + end_time) * sfreq)

    # Estrai dati in µV
    sources  = ica.get_sources(raw_for_sources).get_data()[picks_sources, start_sample:stop_sample]
    original = raw_original.get_data(picks=picks_channels)[:, start_sample:stop_sample] * 1e6
    recon    = raw_clean.get_data(picks=picks_channels)[:, start_sample:stop_sample] * 1e6

    
    # Calcola il valore assoluto massimo tra i canali "bad" post-interpolazione
    #if post_bad_idx:
        #max_bad = np.max(np.abs(recon[post_bad_idx, :]))
    #else:
        #max_bad = np.max(np.abs(recon))
    
    
    # --- Rimappo gli INDICI "globali" (o i Nomi) -> indici LOCALI rispetto a picks_channels ---
    local_bad_idx = []
    if post_bad_names is not None:
        picks_names    = [raw_clean.ch_names[i] for i in picks_channels]
        name_to_local  = {nm: i for i, nm in enumerate(picks_names)}
        local_bad_idx  = [name_to_local[nm] for nm in post_bad_names if nm in name_to_local]
    elif post_bad_idx is not None:
        global_set     = set(post_bad_idx)
        local_bad_idx  = [i for i, ch_idx in enumerate(picks_channels) if ch_idx in global_set]

    # Calcola il valore assoluto massimo tra i canali "bad" (locali) post-interpolazione
    if local_bad_idx:
        max_bad = np.max(np.abs(recon[local_bad_idx, :]))
    else:
        max_bad = np.max(np.abs(recon))
    
    
    # Imposta il range dinamico attorno a ±(max_bad + delta_uV)
    y_lim_low  = - (max_bad + delta_uV)
    y_lim_high =   (max_bad + delta_uV)

    n_rows = max(len(picks_sources), len(picks_channels))
    fig, axes = plt.subplots(n_rows, 3, figsize=(18, n_rows*1.8), sharex=True)
    axes = np.atleast_2d(axes)
    
    

    # colonna 0: ICA
    t = np.linspace(start_time, start_time + end_time, sources.shape[1])
    for i, comp in enumerate(picks_sources):
        ax = axes[i,0]
        ax.plot(t, sources[i])
        ax.set_ylabel(f"ICA {comp}")
    axes[picks_sources[-1],0].set_xlabel("Time (s)")

    # colonna 1: originale
    for i, ch in enumerate(picks_channels):
        ax = axes[i,1]
        ax.plot(t, original[i])
        ax.set_ylabel(f"{raw_original.ch_names[ch]} (µV)")
    axes[picks_channels[-1],1].set_xlabel("Time (s)")

    # colonna 2: ricostruito + righe soglia
    for i, ch in enumerate(picks_channels):
        ax = axes[i,2]
        ax.plot(t, recon[i])
        
        # righe di soglia ±threshold
        
        #ax.axhline( threshold_uV, linestyle='--', linewidth=1)
        #ax.axhline(-threshold_uV, linestyle='--', linewidth=1)
        
        # range dinamico
        
        #ax.set_ylim(y_lim_low, y_lim_high)
        #ax.set_ylabel(f"{raw_original.ch_names[ch]} (µV)")
    #axes[picks_channels[-1],2].set_xlabel("Time (s)")
        
        # righe di soglia ±threshold
        ax.axhline( threshold_uV, linestyle='--', linewidth=1)
        ax.axhline(-threshold_uV, linestyle='--', linewidth=1)
        
        # range dinamico
        ax.set_ylim(y_lim_low, y_lim_high)
        ax.set_ylabel(f"{raw_original.ch_names[ch]} (µV)")
    axes[picks_channels[-1],2].set_xlabel("Time (s)")
    
    
    axes[0,2].set_title(f"Ricostruito (+/–{threshold_uV} µV)\n(dyn±{max_bad:.1f}+{delta_uV} µV)")
    fig.suptitle(f"{task} | {subject} | {run}", y=1.02)
    plt.tight_layout()

    if not return_bytes:
        plt.show()
        return

    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi=150)
    buf.seek(0)
    img = buf.getvalue()
    plt.close(fig)
    return img



def save_completed_runs(completed_runs, completed_runs_file):
    with open(completed_runs_file, "wb") as f:
        pickle.dump(completed_runs, f)
    print(f"\n📌 Set di run completate salvato in: {completed_runs_file}")
    return completed_runs          # ← AGGIUNGI QUESTO

def load_completed_runs(completed_runs_file):
    if os.path.exists(completed_runs_file):
        with open(completed_runs_file, "rb") as f:
            return pickle.load(f)
    return set()  # Se il file non esiste, è la prima esecuzione



'''

Strategia consigliata (robusta e “standard-friendly”)

Selezione run
Tieni le OK + le WARNING con ≤3 residui e ≤8 canali interpolati (come hai già fatto).
Non mi preoccuperei se in alcune finestre compaiono molti canali “sopra 100 µV”: gran parte di quei picchi è transient 
e verrà gestita meglio dopo l’ICA e soprattutto a livello di epoca.

ICA sui dati “quasi grezzi” post-PyPREP

Escludi dal fit ICA i canali segnati in raw.info['bads'] (lo fai già con picks=..., exclude='bads')
Non interpolare altri canali prima dell’ICA: più canali “veri” (anche un po’ sporchi) → mixing più realistico → componenti ICA migliori.
Apply ICA → reintegro bad pre-ICA → reref unico
raw_clean = ica.apply(raw_copy)
Interpola solo i “pre-bads” (quelli marchiati prima del fit), come hai già inserito.
Average reference una sola volta qui (niente re-ref successivi): raw_clean.set_eeg_reference('average', projection=False).
Check ampiezza post-ICA (solo guardrail)
Il tuo check ±100 µV va bene come safety: se trovi canali persistentemente sopra soglia (non picchi sporadici), puoi interpolarli ora.
Bene anche non fare un secondo re-ref qui (evita “drift” della baseline).
Epocaggio + Autoreject/local
Epoca in base al tuo paradigma.
Usa autoreject (local) per stimare per canale e per epoca le soglie di picco-picco e interpolare solo nelle epoche dove serve.
Questo è il punto in cui elimini davvero i residui “fastidiosi” senza buttare via run intere.


Perché questa sequenza ha senso
ICA prima delle decisioni dure per-epoca: separa blink/EMG dalla sorgente “neurale” senza distruggere la struttura statistica con troppe interpolazioni.
Interpolazioni mirate: prima solo i “pre-bads” (strutturali), poi autoreject si occupa del transient a granularità di epoca.
Un solo re-ref stabile: evita di rimuovere e riaggiungere offset più volte.
Riduci i falsi positivi dei picchi occasionali >100 µV: sono il classico caso che l’autoreject gestisce meglio dell’hard-threshold globale.
Micro-check sulla tua implementazione
Nel tuo codice già fai: picks con exclude='bads', ica.apply, interpolazione dei pre-bads, re-ref unico → ✅
Il check ±100 µV lo usi dopo l’ICA → ✅ (mantienilo come guardrail; se vuoi, passa al criterio “persistente” vs “sporadico” che ti ho dato prima).
Poi passa a epocare e autoreject: lì farai la pulizia fine.
In breve: sì, proseguirei subito con la tua pipeline ICA (come l’hai scritta) e demanderei le scelte “severe” sulla qualità al livello di epoca con autoreject/local.


quindi tu in questo step quando dicevi 

"Check ampiezza post-ICA (solo guardrail) 
Il tuo check ±100 µV va bene come safety:
se trovi canali persistentemente sopra soglia (non picchi sporadici), puoi interpolarli ora."

quale di queste strategie adotteresti, se fossi un neuroscienziato? 

o 1) non ti preoccuperesti di interpolare eventualmente anche altri canali persistentemente sopra soglia, 
(ma definiti come persistentemente sopra soglia in caso?) 

o 2) ti preoccuperesti di interpolare eventualmente anche altri canali persistentemente sopra soglia,
(ma definiti come persistentemente sopra soglia in caso?) 

o 3) lasciare la pipeline così com'è, senza interpolare null'altro, tranne solo i “pre-bads” 
(quelli marchiati prima del fit), come hai già inserito e occuparsi eventualmente del tutto in fase di epoca (con autoreject)?


Breve e chiaro: io sceglierei la 3 di default — niente altre interpolazioni subito dopo l’ICA (oltre ai “pre-bads” che già reintegri),
e delegare il resto a autoreject/local a livello di epoca.
È la strategia più stabile e riproducibile: eviti di “levigare” troppo il segnale globale 
e lasci che l’algoritmo decida dove e quando intervenire, solo nelle epoche realmente contaminate.

'''
def interactive_ica_pipeline(
    raw_collection: dict,
    
    post_ica_path: str,
    ica_models_path: str,
    bad_components_path: str,
    
    #recon_figs_path: str,  #<<<<< <<<<< <<<<<
    
    post_interp_bad_chan_path : str, #<<<<< <<<<< <<<<< #<<<<< <<<<< <<<<<
    
    
    show_sources: bool, #= True,
    n_components: int, #= 63,
    
    resume_from_last: bool, #= False,

    #--> CI DEVI INSERIRE LA PATH DI SALVATAGGIO IN .PKL DEL SET RICHIAMO
    completed_runs_file: str, 

    ICA_confirm: bool, #= True,
):
    
    cleaned_data = dict()
    ica_models = dict()
    bad_components_data = dict()
    #recon_figs_data = dict() #<<<<< <<<<< <<<<<
    post_interp_bad_chan_data = dict() #<<<<< <<<<< <<<<< #<<<<< <<<<< <<<<<

    
    # all'inizio
    completed_runs = set()

    # Carica il set delle run completate
    completed_runs = load_completed_runs(completed_runs_file)
   
    # Calcolare solo le run totali 
    #total_runs = sum(len(runs) for subjects in raw_collection.values() for runs in subjects.values())

    # Calcolare total_runs 
    total_runs = 0

    # Calcolare le run completate dentro al set di run già completate
    processed_runs = 0

    # All'inizio della funzione, inizializza bad_components_official fuori dai cicli
    bad_components_official = set()  # Inizializza un set vuoto per tenere traccia dei componenti da escludere

    bad_components = {}
    
    for task, subjects in raw_collection.items():
        for subject, runs in subjects.items():
            for run in runs:
                total_runs += 1
                if (task, subject, run) in completed_runs:
                    processed_runs += 1  # Conta le run già completate


    missing_runs = total_runs - processed_runs  # Calcola le run mancanti

    print(f"Trovate già in \033[1mcompleted_runs\033[0m n° \033[1m{processed_runs} runs\033[0m, ne \033[1mmancano\033[0m n° \033[1m{missing_runs}\033[0m!")

    run_counter = 0

    for task, subjects in raw_collection.items():
        cleaned_data[task] = {}
        ica_models[task] = {}
        bad_components_data[task] = {}
        
        #recon_figs_data[task] = {} #<<<<< <<<<< <<<<<
        
        
        post_interp_bad_chan_data[task] = {} #<<<<< <<<<< <<<<< #<<<<< <<<<< <<<<<


        for subject, runs in tqdm(subjects.items(), desc=f"Processing Task {task}", total=len(subjects)):

            cleaned_data[task][subject] = {}
            ica_models[task][subject] = {}
            bad_components_data[task][subject] = {}
            
            #recon_figs_data[task][subject] = {} #<<<<< <<<<< <<<<<
            
            post_interp_bad_chan_data[task][subject] = {} #<<<<< <<<<< <<<<< #<<<<< <<<<< <<<<<

            
            for run in sorted(runs):
                                
                if resume_from_last and (task, subject, run) in completed_runs:
                    print(f"→ Task \033[1m{task}\033[0m, Subject \033[1m{subject}\033[0m, run \033[1m{run}\033[0m è già completata, ripartiamo dalla successiva...")
                    continue  # Salta la run già completata

                run_counter += 1
                print(f"\n🧪 Processinng ICA per Task: \033[1m{task}\033[0m | Subject: \033[1m{subject}\033[0m | Run: \033[1m{run}\033[0m")

                '''ADESSO QUI TI CARICHI I DATI (RUN) DEL TASK, SOGGETTO (E QUINDI RUN) CORRENTE (O CHE MANCA RISPETTO A COMPLETED_RUNS!)'''
                raw, annotations = runs[run]  # 🧠 AGGIUNGI QUESTA RIGA
                
                '''
                Mini check-list (riassunto operativo)

                Fit ICA: usa picks = mne.pick_types(..., exclude='bads') e n_comp <= len(picks).
                Dopo ica.apply: interpola sempre i pre_bads (quelli marcati in EEG_for_ICA) e fai set_eeg_reference('average', ...).
                Poi il tuo check ±100 µV e (se serve) ulteriore interpolazione dei canali rimasti sopra soglia (con eventuale reref finale).
                Così la tua pipeline è allineata alle buone pratiche: i canali “rotti” non inficiano il fit ICA, ma rientrano puliti nel segnale finale.
                '''
               
                while True:
                    try:
                        # --- Fit ICA -----------------------------------------------------------

                        #✔️ OK — Crei una copia di raw su cui eseguire il fitting ICA, così non tocchi i dati originali

                        raw_copy = raw.copy().load_data()
                        
                        
                        '''NEW LAST ADDITION 19 AGOSTO
                        
                        Sì 👍, quello che hai scritto va già molto bene:

                        Calcoli i picks escludendo i bad → ✅ così i canali da interpolare non entrano nel fit ICA.
                        Limiti n_components al numero di canali buoni (min(n_components, n_good)) → ✅ evita crash se ci sono pochi canali.
                        Fit ICA con picks e reject_by_annotation=True → ✅ best practice, esclude anche eventuali segmenti annotati come “BAD”.
                        Ricavi picks_sources da ica.n_components_ → ✅ hai il numero effettivo di componenti.
                        
                        '''
                        
                        # + calcola i picks escludendo i canali bad
                        # + (usa solo EEG; niente MEG/EOG nel fit)
                        picks = mne.pick_types(raw_copy.info, eeg=True, meg=False, eog=False, exclude='bads')
                        
                        # + per sicurezza imponi n_components ≤ #canali buoni
                        n_good = len(picks)
                        n_comp = min(n_components, n_good)  # (se vuoi: usa il rank reale)


                        '''OLD APPROACH PRE 19 AGOSTO'''
                        #ica = mne.preprocessing.ICA(n_components=n_components, random_state=97, max_iter='auto')
                        #ica.fit(raw_copy)
                        
                        '''NEW LAST ADDITION 19 AGOSTO'''
                        
                        ica = mne.preprocessing.ICA(n_components=n_components, random_state=97, max_iter='auto')
                        
                        # + IMPORTANT: passa i picks e rispetta le annotazioni (reject_by_annotation)
                        ica.fit(raw_copy, picks=picks, reject_by_annotation=True)


                        #✔️ OK — Se hai fatto ICA con n_components=0.99 o simili, ica.n_components_ 
                        #ti dà il numero effettivo di componenti spieganti il 99% della varianza.

                        picks_sources = list(range(ica.n_components_))  # tutte le comp. estratte

                        #  (se vuoi un massimo di 20: picks_sources = picks_sources[:20])
                        # ---------------------------------------------


                        # --- 1. Topomap di tutte le IC in figura unica matplotlib (come in Jupyter) ---

                        #✔️ OK — La funzione plottando ica.get_components() ti mostra le topografie (ossia “dove” ogni componente ha peso nel cervello).
                        plot_all_components_single_figure(ica, title=f"ICA Components — Task {task}, Subject {subject}, Run {run}")

                        # --- 2. PLOT SOURCES (GUI interattiva non bloccante) ---
                        # ---------- LOOP interattivo sources → bads → ricostruzione ------

                        if show_sources:

                            # 1. Definisco intervallo temporale
                            while True:
                                # ↳ start-time scelto dall’utente
                                try:
                                    user_input_start = float(input("→ Inserisci Start-time in secondi: "))
                                    user_input_end = float(input("→ Inserisci End-time in secondi: "))
                                    if user_input_end <= user_input_start:
                                        print("ATTENZIONE ⚠️! End-time deve essere maggiore di start-time.")
                                        continue
                                    break

                                except ValueError:
                                    print("❌ Input non valido, inserisci solo numeri interi")
                                    #start_time = 0



                            # --- 2. Plot statico delle SOURCES (prime 10 IC)
                            plot_static_ica_sources(
                                ica=ica,
                                raw=raw_copy,

                                picks=picks_sources,                 ### <-- QUI

                                start_time=user_input_start,
                                end_time=user_input_end,
                                task=task, subject=subject, run=run
                            )

                            plt.pause(0.1)


                            # --- 2.1 Se vuoi cambiare intervallo SOURCES
                            #change_interval = input("→ Cambiare Intervallo delle Sources? (yes/no/stop): ").lower()

                            # --- 2.1 Vuoi cambiare intervallo SOURCES?
                            while True:
                                change_interval = input(
                                    "→ Cambiare Intervallo delle Sources? (yes/no/stop): "
                                ).strip().lower()

                                #Fa sparire le immagini già disegnate dall’area di output di Jupyter, così il notebook rimane “pulito”.
                                #clear_output(wait=True)      # <— pulisci l’area di output


                                # valori ammessi ----------------------------------------------------------
                                if change_interval in {"yes", "no", "stop", "no stop"}:
                                    break  # OK, esci dal loop e gestisci la scelta qui sotto

                                # qualunque altra cosa = errore ------------------------------------------
                                print("❌ Input non valido. Digita 'yes', 'no' oppure 'stop'.")
                                # e il while ricomincia
                            # ---------------------------------------------------------------------------


                            if change_interval == 'yes':
                                plt.close('all')  # Chiudi il plot
                                continue  # Torna al loop per scegliere un nuovo intervallo

                                #Dopo aver chiesto se vuoi cambiare intervallo (ramo change_interval == 'yes')
                                #– lì chiudi già le figure per mostrarne di nuove.

                                #Fa sparire le immagini già disegnate dall’area di output di Jupyter, così il notebook rimane “pulito”.
                                clear_output(wait=True)      # <— pulisci l’area di output


                            elif change_interval == 'no':
                                plt.close('all')  # Chiudi il plot
                                # Definisci raw_clean qui, **prima di applicare l'ICA**
                                raw_clean = raw_copy.copy()
                                # Esci dal loop per continuare con la selezione delle componenti

                            elif change_interval in ['stop', 'no stop']:
                                print("🛑 Interruzione manuale richiesta. Salvataggio parziale e uscita immediata.")
                                raise KeyboardInterrupt

                                #plt.close('all')
                                #gc.collect()

                                #Fa sparire le immagini già disegnate dall’area di output di Jupyter, così il notebook rimane “pulito”.
                                #clear_output(wait=True)      # <— pulisci l’area di output
                                #return cleaned_data, ica_models, bad_components_data, recon_figs_data, post_interp_bad_chan_data # 🧠 STOP TOTALE


                            bad_components = {}

                            bad_components_official = set()  # Set vuoto per ogni run


                            bads = input("→ Scegli le componenti da escludere (es. 0,1,5 o digita 'stop' per uscire): ")

                            if bads.strip().lower() == 'stop':
                                print("🛑 Interruzione richiesta manualmente. Chiudo tutto e salvo stato corrente.")
                                plt.close('all')
                                gc.collect()

                                #Fa sparire le immagini già disegnate dall’area di output di Jupyter, così il notebook rimane “pulito”.
                                clear_output(wait=True)      # <— pulisci l’area di output
                                #return cleaned_data, ica_models, bad_components_data, recon_figs_data, post_interp_bad_chan_data # 🧠 STOP TOTALE
                                raise KeyboardInterrupt


                            bad_components = {int(idx) for idx in bads.split(",") if idx.strip().isdigit()}  # Converte l'input in un set

                            # Crea una copia indipendente di bad_components
                            # Se sono state selezionate delle componenti da escludere, aggiorna `bad_components_official`

                            bad_components_official.update(bad_components)  # Aggiunge i componenti esclusi al set


                            ica.exclude = list(bad_components_official)  # Converte il set in lista per ICA

                            # --- 3. APPLY ICA & PLOT RECONSTRUCTED SIGNAL (GUI interattiva non bloccante) ---
                            

                            raw_clean = ica.apply(raw_copy)
                            
                            '''
                            NEW LAST ADDITION 19 AGOSTO
                            
                            Qui L’unica cosa che rimane da fare è il passo dopo ica.apply(...):
                            👉 interpolare i pre-bads + fare una reref.

                            Se non aggiungi quel passaggio, i canali segnati come bad nel fit restano vuoti/NaN nel raw_clean, quindi rischi:
                            di avere meno canali del previsto per i plot/check successivi,
                            e di non rimettere dentro i canali esclusi.

                            
                            Perché: i “bad” sono stati esclusi dal fit; ora li reintegri via interpolazione e stabilizzi la baseline con la average re-ref. 
                            Così i plot e il check ±100 µV guardano già il segnale con i pre-bads sistemati.
                            
                            👉 Se aggiungi queste 6 righe dopo ica.apply(...), ossia
                            
                            pre_bads = list(raw_copy.info.get("bads", []))
                            if pre_bads:
                                raw_clean = interpolate_selected_channels(
                                    raw_in=raw_clean,
                                    channels=pre_bads,
                                    by="name",          # passiamo i NOMI dei canali
                                    reset_bads=True,
                                    final_reref=False,  # niente reref qui, lo facciamo subito sotto
                                    verbose=True
                                )

                            # >>> REREF finale (una sola volta qui)
                            raw_clean.set_eeg_reference('average', projection=False)
                            
                            Allora sei a posto:

                            fit ICA pulito (solo canali buoni),
                            reintegri i bad con l’interpolazione,
                            reref finale fatto una sola volta.

                            In sintesi

                            ✔️ Fit ICA con picks che escludono i bad → OK.
                            ✔️ n_comp ≤ #canali buoni → OK.
                            ✅ Manca solo: interpolare i pre-bads + una reref subito dopo ica.apply(...).
                            🔁 Poi prosegui con il tuo check ±100 µV e, se serve, altra interpolazione (senza ulteriore reref).
                            Fatta questa micro-patch, la pipeline è perfettamente allineata alle buone pratiche.
                            Buon lavoro e… a domani! 👋

                            '''
                            
                            # >>> PRE-BADS: interpola SEMPRE i canali marcati prima del fit
                            pre_bads = list(raw_copy.info.get("bads", []))
                            if pre_bads:
                                raw_clean = interpolate_selected_channels(
                                    raw_in=raw_clean,
                                    channels=pre_bads,
                                    by="name",          # passiamo i NOMI dei canali
                                    reset_bads=True,
                                    final_reref=False,  # niente reref qui, lo facciamo subito sotto
                                    verbose=True
                                )

                            # >>> REREF finale (una sola volta qui)
                            raw_clean.set_eeg_reference('average', projection=False)

                            
                            '''QUI FORNISCO UN PICK_CHANNELS DINAMICO ANCHE DURANTE LA SOLA VISUALIZZAZIONE DELLE RICOSTRUZIONI'''
                            while True:
                                try:
                                    max_chan = raw_clean.info["nchan"]  # numero canali totali
                                    user_n = input(
                                        f"   → Quanti canali EEG vuoi visualizzare? (1–{max_chan} oppure 'stop' o 'quit' per interrompere): "
                                    ).strip()
                                    if user_n.lower() in {"stop", "quit"}:
                                        raise KeyboardInterrupt
                                    n_plot = int(user_n)
                                    if 1 <= n_plot <= max_chan:
                                        break
                                    print(f"   ❌ Inserisci un intero compreso fra 1 e {max_chan}.")
                                except ValueError:
                                    print("ATTENZIONE ⚠️ Inserisci un numero intero valido.")
                    
                            
                            picks_channels = list(range(n_plot))  # es. primi N canali

                            # --- 4. Plot statico del segnale ricostruito
                            plot_static_reconstructed_signal(
                                raw_clean=raw_clean,

                                #channel_indices=list(range(15)),
                                #channel_indices=picks_sources,       
                                
                                channel_indices=picks_channels,       ### <-- QUI

                                start_time=user_input_start,
                                end_time=user_input_end,
                                task=task,
                                subject=subject,
                                run=run
                            )
                            plt.pause(0.1)


                            ''' SE NON VOLESSI SALVARE MAI LA FIGURA'''
                            #img_bytes = None     # <-- prima di chiedere compare_view


                            # --- 4.1 Vuoi confrontare SOURCES + RICOSTRUZIONE in un'unica figura?
                            compare_view = input(f"→ Vuoi confrontare le SOURCES, SEGNALE GREZZO e la RICOSTRUZIONE in un'unica figura? (ok/no/stop/restart): ").strip().lower()
                            if compare_view == 'ok':


                                #RICORDATI LA FUNZIONE SOPRA SAREBBE:

                                #def plot_sources_vs_original_vs_reconstructed(
                                        #ica,
                                        #raw_for_sources,          # raw usato nel fit ICA (p.e. raw_copy)
                                        #raw_original,             # raw originale, PRIMA dell’ICA
                                        #raw_clean,                # raw dopo ica.apply(...)
                                        #picks_sources, picks_channels,
                                        #start_time, end_time,
                                        #task=None, subject=None, run=None,
                                        #same_scale_eeg=True       # <<<<<<<<<<<<<<  NUOVO TOGGLE
                                    #):

                                # -----------------------------------------------------------------
                                #  chiedi quanti canali EEG mostrare
                                while True:
                                # vista “comparativa” (stesso range per EEG)
                                    try:
                                        max_chan = raw.info["nchan"]          # numero canali totali
                                        user_n = input(
                                            f"   → Quanti canali EEG vuoi visualizzare? (1–{max_chan} oppure 'stop' o 'quit' per interrompere): "
                                        ).strip()
                                        if user_n.lower() in {"stop", "quit"}:
                                            raise KeyboardInterrupt
                                        n_plot = int(user_n)
                                        if 1 <= n_plot <= max_chan:
                                            break
                                        print(f"   ❌ Inserisci un intero compreso fra 1 e {max_chan}.")
                                    except ValueError:
                                        print("ATTENZIONE ⚠️ Inserisci un numero intero valido.")
                                        
                                
                                '''QUI FORNISCO UN PICK_CHANNELS CHE SI ADATTA ALLA SCELTA DEL MIO UTENTE'''
                                picks_channels = list(range(n_plot))          # es. primi N canali

                                #Cosa succede ora
                                #PRIMA figura: vedi sources, originale, ricostruito appena applicata l’ICA.

                                #Il check ±100 µV segnala eventuali canali “esplosivi”.

                                #Puoi interpolarli in automatico (auto), a mano (idx) o saltare (skip).

                                #DOPO l’interpolazione (se fatta) hai la SECONDA figura con il segnale aggiornato.

                                #Il resto della tua pipeline (scelta compare_view, prompt ICA_confirm, salvataggi)
                                #segue identico ma ora lavora sulla versione definitiva di raw_clean.

                                # -----------------------------------------------------------------
                                #  chiamata al plot (puoi lasciare same_scale_eeg=True o False a piacere)

                                # === A) FIGURA PRIMA DEL CHECK / INTERPOLAZIONE ==================
                                print("\n📊 Segnale ricostruito PRIMA del check/interpolazione:")

                                img_bytes = plot_sources_vs_original_vs_reconstructed(
                                    ica = ica,
                                    raw_for_sources = raw_copy, # raw usato nel fit ICA (p.e. raw_copy)
                                    raw_original = raw, # raw originale, PRIMA dell’ICA
                                    raw_clean = raw_clean, # raw dopo ica.apply(...)
                                    picks_sources = picks_sources,
                                    picks_channels = picks_channels,
                                    start_time = user_input_start, 
                                    end_time = user_input_end,
                                    task=task, 
                                    subject=subject,
                                    run=run,
                                    same_scale_eeg=True,
                                    return_bytes=False          # <<<<<<<<<<
                                )

                                # === B) CHECK ampiezze ±100 µV ===================================
                                bad_idx, bad_names, _ = check_recon_amplitude_uV(
                                    raw_clean, threshold_uV=100.0, picks=None, verbose=True
                                )

                                # === C) EVENTUALE interpolazione =================================
                                if bad_idx:
                                    while True:
                                        choice = input(
                                            f"\n→ Canali fuori soglia {bad_names}. Interpolazione?\n"
                                            "   'auto'  : interpola quelli fuori soglia\n"
                                            "   'idx'   : inserisci manualmente indici\n"
                                            "   'skip'  : non interpolare\n"
                                            "   'restart' : rifai l'ICA da capo per questa run\n"
                                            "> "
                                        ).strip().lower()

                                        if choice == "skip":
                                            break

                                        elif choice == "auto":
                                            raw_clean = interpolate_selected_channels(
                                                raw_in=raw_clean,
                                                channels=bad_idx,
                                                by="index",
                                                reset_bads=True,
                                                final_reref=False
                                            )
                                            break

                                        elif choice == "idx":
                                            try:
                                                idx_list = [
                                                    int(x) for x in input("   Indici (es. 0,4,7): ").split(",")
                                                    if x.strip().isdigit()
                                                ]
                                                if not idx_list:
                                                    print("   ❌ Nessun indice valido.")
                                                    continue
                                                raw_clean = interpolate_selected_channels(
                                                    raw_in=raw_clean,
                                                    channels=idx_list,
                                                    by="index",
                                                    reset_bads=True,
                                                    final_reref=False
                                                )
                                                break
                                            except Exception as e:
                                                print("   Errore:", e)
                                                continue
                                        
                                        elif choice == "restart":
                                            print("🔄 Rilancio ICA per questa run…")
                                            plt.close('all')
                                            clear_output(wait=True)
                                            # solleviamo un’eccezione per saltare al try/except esterno
                                            raise RuntimeError("restart_ica")
                                            
                                        else:
                                            print("   Comando non valido (‘auto’, ‘idx’, ‘skip’).")
                                
                                
                                #QUESTA PARTE DI RICERCA DELLE BAD CHANNELS LA SPOSTO FUORI DALL' IF_BADX
                                
                                
                                # (facoltativo) ricontrollo
                                # 🆕 POST-INTERPOLAZIONE: log canali residui sopra soglia (senza toccare i dati)
                                bad_idx_final, bad_names_final, peak_abs_uV = check_recon_amplitude_uV(
                                    raw_clean, threshold_uV=100.0, picks=None, verbose=True
                                )

                                #'''ATTENZIONE QUI MI SALVO I NUOVI CANALI SOPRA SOGLIA ±100 µV ANCHE DOPO INTERPOLAZIONE!!!'''

                                # 1) salva sul dict in memoria

                                #post_interp_bad_chan_data[task][subject][run] = {
                                #    "bad_idx": bad_idx_final,
                                #    "bad_names": bad_names_final,
                                #    "peak_abs_uV": peak_abs_uV.tolist()  # se è np.array
                                #}


                                # 2) salva subito su pickle (accoda/merge)
                                #append_pickle(
                                #    {task: {subject: {run: {
                                #        "bad_idx": bad_idx_final,
                                #        "bad_names": bad_names_final,
                                #        "peak_abs_uV": peak_abs_uV.tolist()
                                #    }}}},
                                #    post_interp_bad_chan_path,
                                #    return_master=False
                                #)

                                # ✅ PRINT DI CONTROLLO
                                #print(f"📥 Canali post-interpolazione salvati per {task} | {subject} | {run}:")
                                #print(f"   → bad_names: {bad_names_final}")
                                #print(f"   → bad_idx  : {bad_idx_final}")
                                #print(f"   → peak_abs_uV: {np.round(peak_abs_uV[bad_idx_final], 2)} µV")

                                #Cosa succede
                                #Calcola max_bad = max(abs(recon[post_bad_idx, :])) per prendere il picco più alto tra i canali “cattivi”.

                                #Imposta l’asse Y da –(max_bad + delta_uV) a +(max_bad + delta_uV).

                                #Traccia le linee tratteggiate a ±100 µV.

                                #In questo modo il grafico “si allarga” esattamente quanto serve a mostrare tutti i picchi residui,
                                #con un piccolo margine delta_uV. 

                                #Se vuoi, puoi anche re-chiamare 
                                #la variabile end_time = user_input_end – user_input_start per rendere ancora più chiaro che quel parametro è una durata,
                                #ma l’alias non impedisce il funzionamento.
                                
                                
                                picks_channels_all = list(range(raw_clean.info['nchan']))
                                
                                # === D) FIGURA DOPO DEL CHECK / INTERPOLAZIONE ==================
                                print("\n📊 Segnale ricostruito DOPO il check/interpolazione:")
                                img_bytes = plot_sources_vs_original_vs_reconstructed_post_interp(
                                    ica              = ica,
                                    raw_for_sources  = raw_copy,              # raw usato per fit ICA
                                    raw_original     = raw,                   # raw prima di ICA
                                    raw_clean        = raw_clean,             # raw dopo ica.apply()
                                    picks_sources    = picks_sources,
                                    
                                    #picks_channels   = picks_channels,
                                    
                                    picks_channels   = picks_channels_all,   # <-- qui TUTTI i canali
                                    
                                    start_time       = user_input_start,
                                    end_time         = user_input_end,        # o come lo chiami tu
                                    
                                    post_bad_idx     = bad_idx_final,         # <— lista degli indici ancora fuori soglia # indici GLOBALI (ok!)
                                    post_bad_names   = bad_names_final,     # mapping per nome (ancora più robusto)
                                    
                                    threshold_uV     = 100.0,                 # soglia di riferimento
                                    delta_uV         = 10.0,                  # margine extra (puoi cambiarlo)
                                    same_scale_eeg   = True,
                                    task = task,        # ← aggiungi
                                    subject =subject,     # ← aggiungi
                                    run = run,         # ← aggiungi
                                    return_bytes     = True # <<<<<<<<<<
                                )


                            elif compare_view == 'no':
                                print("Ok passiamo al salvataggio ICA per questa run")
                                plt.close('all')
                                gc.collect()
                                # Esci dal loop della run corrente → si salva e continua con la prossima
                            
                            elif compare_view == "restart":
                                print("🔄 Rilancio ICA per questa run…")
                                plt.close('all')
                                clear_output(wait=True)
                                # solleviamo un’eccezione per saltare al try/except esterno
                                raise RuntimeError("restart_ica")
                                            
                            else:
                                print("🛑 Interruzione manuale richiesta. Salvataggio parziale e uscita immediata.")
                                plt.close('all')
                                gc.collect()

                                #Fa sparire le immagini già disegnate dall’area di output di Jupyter, così il notebook rimane “pulito”.
                                clear_output(wait=True)      # <— pulisci l’area di output
                                #return cleaned_data, ica_models, bad_components_data, recon_figs_data, post_interp_bad_chan_data # 🧠 STOP TOTALE
                                raise KeyboardInterrupt


                            if ICA_confirm:
                                next_run = input(f"Posso salvare l'ICA di questa run appena eseguita?\n"
                                                 f"Rispondi con:\n\n"
                                                 f"'ok' per continuare\n"
                                                 f"'no' per ri-eseguire la run\n"
                                                 f"'stop' o 'no stop' per interrompere tutto:").strip().lower()

                                if next_run == 'ok':
                                    print(f"\n✅ Ok salvo ICA per questa run")
                                    plt.close('all')
                                    gc.collect()
                                    
                                    # Salvo i canali post-interpolazione **solo se ce n’è almeno uno**
                                    if bad_idx_final:                       # ← qui il filtro
                                        post_interp_bad_chan_data[task][subject][run] = {
                                            "bad_idx": bad_idx_final,
                                            "bad_names": bad_names_final,
                                            "peak_abs_uV": peak_abs_uV.tolist()
                                        }
                                        append_pickle(
                                            {task: {subject: {run: {
                                                "bad_idx": bad_idx_final,
                                                "bad_names": bad_names_final,
                                                "peak_abs_uV": peak_abs_uV.tolist()
                                            }}}},
                                            post_interp_bad_chan_path,
                                            return_master=False
                                        )
                                        # ✅ PRINT DI CONTROLLO
                                        print(f"📥 Salvati {len(bad_idx_final)} canali post-interpolazione fuori soglia "
                                              f"per {task} | {subject} | {run}")
                                        print(f"   → bad_names: {bad_names_final}")
                                        print(f"   → bad_idx  : {bad_idx_final}")
                                        print(f"   → peak_abs_uV: {np.round(peak_abs_uV[bad_idx_final], 2)} µV")
                                    else:
                                        print("✅ Nessun canale oltre soglia dopo l’interpolazione — nulla da salvare")

                                    break  # Esci dal loop della run corrente → si salva e continua con la prossima

                                elif next_run in ['stop', 'no stop']:
                                    print("🛑 Interruzione manuale richiesta. Salvataggio parziale e uscita immediata.")
                                    plt.close('all')
                                    gc.collect()

                                    #Fa sparire le immagini già disegnate dall’area di output di Jupyter, così il notebook rimane “pulito”.
                                    clear_output(wait=True)      # <— pulisci l’area di output
                                    #return cleaned_data, ica_models, bad_components_data, recon_figs_data, post_interp_bad_chan_data# 🧠 STOP TOTALE
                                    raise KeyboardInterrupt

                                else:
                                    print("🔁 Riprovo ICA per questa run...")
                                    plt.close('all')
                                    gc.collect()

                                    #Fa sparire le immagini già disegnate dall’area di output di Jupyter, così il notebook rimane “pulito”.
                                    clear_output(wait=True)      # <— pulisci l’area di output
                        
                    except RuntimeError as e:
                        if str(e) == "restart_ica":

                            #Fa sparire le immagini già disegnate dall’area di output di Jupyter, così il notebook rimane “pulito”.
                            clear_output(wait=True)      # <— pulisci l’area di output

                            # semplicemente ricomincia il while True dall'inizio
                            continue
                        else:
                            # rilancia ogni altro RuntimeError
                            raise

                # ---------- salvataggio cumulativo ----------
                #cleaned_data = append_pickle({task: {subject: {run: (raw_clean, annotations)}}},
                #              post_ica_path, return_master=True)
                #ica_models = append_pickle({task: {subject: {run: ica}}}, ica_models_path, return_master=True)
                
                #bad_components_data = append_pickle({task: {subject: {run: list(bad_components_official)}}},
                #              bad_components_path, return_master=True)
                #recon_figs_data = append_pickle({task: {subject: {run: img_bytes}}}, recon_figs_path, return_master=True)
                
                append_pickle({task: {subject: {run: (raw_clean, annotations)}}},
                              post_ica_path, return_master=False)
                append_pickle({task: {subject: {run: ica}}}, ica_models_path, return_master=False)
                
                append_pickle({task: {subject: {run: list(bad_components_official)}}},
                              bad_components_path, return_master=False)
                #append_pickle({task: {subject: {run: img_bytes}}}, recon_figs_path, return_master=False)
                
                
                #'''INDENTATE INDIETRO!!!'''

                completed_runs.add((task, subject, run))  # ✅ aggiorni il set

                # Salva l'ultima run completata nel set
                completed_runs = save_completed_runs(completed_runs, completed_runs_file)

                print(f"\n💻 Task {task}, Subject {subject}, Run {run} salvata localmente nel set delle run completate!")
                

                print(f"\n📡 Checkpoint salvato per Task: \033[1m{task}\033[0m | Subject: \033[1m{subject}\033[0m | Run: \033[1m{run}\033[0m")

                next_run = input(f"\n\n→ Posso eseguire la prossima run ? Scrivi:\n"
                                 f"'ok' per continuare\n"
                                 f"'no' se vuoi stoppare qui e proseguire dopo :)")
                
                if next_run.lower() == 'no':
                    break
                    
                else:
                    plt.close('all')
                    gc.collect()

                    #Fa sparire le immagini già disegnate dall’area di output di Jupyter, così il notebook rimane “pulito”.
                    clear_output(wait=True)      # <— pulisci l’area di output
                    
            print(f"\n\n✅ Subject \033[1m{subject}\033[0m completato ({run_counter}/{total_runs})")

        print(f"\n\n📊 Task \033[1m{task}\033[0m completato.")

    print("\n\n✅ Tutti i dati ICA sono stati elaborati, salvati e inviati al server.")
    plt.close('all')
    gc.collect()

    #Fa sparire le immagini già disegnate dall’area di output di Jupyter, così il notebook rimane “pulito”.
    clear_output(wait=True)      # <— pulisci l’area di output 
    
    #return cleaned_data, ica_models, bad_components_data, recon_figs_data, post_interp_bad_chan_data

    return cleaned_data, ica_models, bad_components_data, post_interp_bad_chan_data

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/pyprep_outputs/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_for_ICA.pkl', 'rb') as f:
    EEG_for_ICA = pickle.load(f)
    
# Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_post_pyprep_data.pkl', 'rb') as f:
    EEG_prep_data = pickle.load(f)

In [ ]:
print(EEG_for_ICA.keys())
print(EEG_for_ICA['task_1'].keys())
print(EEG_for_ICA['task_1']['100'].keys())
print(type(EEG_for_ICA['task_1']['100'][0][0]))
print(type(EEG_for_ICA['task_1']['100'][0][1]))

In [ ]:
print(EEG_for_ICA.keys())
print()
print(EEG_for_ICA['task_1'].keys())
print()
print(EEG_for_ICA['task_1']['100'].keys())
print(EEG_for_ICA['task_2']['100'].keys())
print(EEG_for_ICA['task_3']['100'].keys())
print(EEG_for_ICA['task_4']['100'].keys())
print()
print()
print(type(EEG_for_ICA['task_1']['100']))
print(EEG_for_ICA['task_1']['100'].keys())
print()
print(type(EEG_for_ICA['task_1']['100'][0]))
print(len(EEG_for_ICA['task_1']['100'][0]))
print(type(EEG_for_ICA['task_1']['100'][0][0]))
print(type(EEG_for_ICA['task_1']['100'][0][1]))

In [ ]:
print(EEG_prep_data.keys())
print()
print(EEG_prep_data['task_1'].keys())
print()
print(EEG_prep_data['task_1']['100'].keys())
print(EEG_prep_data['task_2']['100'].keys())
print(EEG_prep_data['task_3']['100'].keys())
print(EEG_prep_data['task_4']['100'].keys())
print()
print()
print(type(EEG_prep_data['task_1']['100']))
print(EEG_prep_data['task_1']['100'].keys())
print()
print(type(EEG_prep_data['task_1']['100'][0]))
print(len(EEG_prep_data['task_1']['100'][0]))
print(type(EEG_prep_data['task_1']['100'][0][0]))
print(type(EEG_prep_data['task_1']['100'][0][1]))

In [ ]:
'''
Aggiungiamo un dizionario dedicato e un nuovo pickle per salvare, per task → subject → run, 
i canali che restano sopra soglia anche dopo l’interpolazione. 


Così:

tracci sempre i canali residui in un file dedicato,

mantieni la shape identica tra soggetti/run,

a valle potrai usare np.nanmean / np.nan* oppure scegliere un sottoinsieme comune di canali solo al termine dell’analisi.

'''


base_path = "/home/stefano/EEGMMIDB_Analyses/post_ica_data/"
processed_runs_file = 'processed_ICA_runs_file.pkl'

# Definisci le directory per il salvataggio dei vari file
post_ica_path = f"{base_path}post_ica_data.pkl"
ica_models_path = f"{base_path}ica_models.pkl"
bad_components_path = f"{base_path}ica_bad_components.pkl"


#recon_figs_path = f"{base_path}recon_figs.pkl"

# ✅ nuovo pickle per i canali residui post-interpolazione
post_interp_bad_chan_path = f"{base_path}post_interp_bad_chan_data.pkl"


completed_runs_file = base_path + processed_runs_file



In [ ]:
## 
'''

✅ 1. Uso di n_components=0.99
Sì, è corretto e consigliato: quando passi n_components=0.99 a mne.preprocessing.ICA, stai dicendo:

«Estrai automaticamente il numero minimo di componenti che spiegano il 99% della varianza del segnale».

Di solito, con 63 canali, potresti ottenere qualcosa come 25–40 componenti (dipende dal dataset), 
quindi ti evita di dover esplorare sempre tutte le 63, mantenendo l’analisi più focalizzata e interpretabile.
'''

#cleaned, models, bads, recon_figs,  post_interp_bad_chan_data = interactive_ica_pipeline(

cleaned, models, bads, post_interp_bad_chan_data = interactive_ica_pipeline(
    #raw_collection = EEG_data_filtered,
    #raw_collection = mini_collection,
    
    raw_collection = EEG_for_ICA,
    
    post_ica_path = post_ica_path,
    ica_models_path = ica_models_path,
    bad_components_path = bad_components_path,

    #recon_figs_path = recon_figs_path,
    
    post_interp_bad_chan_path = post_interp_bad_chan_path,
    
    show_sources=True,
    n_components=0.99, 
    
    resume_from_last= True,
    
    completed_runs_file = completed_runs_file,
    
    ICA_confirm = True
)


#0,1,2,3,8,12,13,15

#### **POST ESECUZIONE ICA: CANCELLARE RUNS PER EVENTUALI RI-ESECUZIONI**

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/post_ica_data/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}post_ica_data.pkl', 'rb') as f:
    EEG_post_ICA = pickle.load(f)


In [ ]:
EEG_post_ICA.keys()

In [ ]:
EEG_post_ICA['task_3']['101'].keys()

In [ ]:
#✅ Codice per rimuovere il soggetto '9' da 'task_2'

#EEG_post_ICA['task_3'].pop('101', None)

EEG_post_ICA['task_3']['101'].pop(10, None)

In [ ]:
#type(EEG_for_ICA['task_2']['9'][1][1])

In [ ]:
#print(EEG_post_ICA['task_2'].keys()) 

In [ ]:
'''

Sì — ti basta rimuovere da completed_runs_file (il tuo processed_ICA_runs_file) le tuple corrispondenti, così la pipeline non le salterà con resume_from_last=True.

Ecco uno snippet pronto:

'''



# runs da rieseguire
'''
to_remove = {
    ("task_2", "9", 1),
    ("task_2", "9", 5),
    ("task_2", "9", 9),
}
'''


import os, pickle

processed_ICA_runs_file = "/home/stefano/EEGMMIDB_Analyses/post_ica_data/processed_ICA_runs_file.pkl"  # <— metti il path giusto

to_remove = {
    ("task_4", "20", 7)
}


def remove_runs_from_completed(file_path, items):
    if not os.path.exists(file_path):
        print(f"❌ File non trovato: {file_path}")
        return
    with open(file_path, "rb") as f:
        completed = pickle.load(f)

    # di norma è già un set; forziamo comunque
    completed = set(completed)
    before = len(completed)
    completed.difference_update(items)

    with open(file_path, "wb") as f:
        pickle.dump(completed, f)
        f.flush(); os.fsync(f.fileno())

    print(f"🧹 Rimosse {before - len(completed)} voci. Rimaste: {len(completed)}")

In [ ]:
remove_runs_from_completed(processed_ICA_runs_file, to_remove)

#### **Post Custom ICA: Data Structure Inspection**

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/post_ica_data/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}post_ica_data.pkl', 'rb') as f:
    EEG_post_ICA = pickle.load(f)

In [ ]:
print(EEG_post_ICA.keys())
print(EEG_post_ICA['task_1'].keys())
print(EEG_post_ICA['task_1']['100'].keys())
print()
print()
print(type(EEG_post_ICA['task_1']['100'][0][0]))
print(type(EEG_post_ICA['task_1']['100'][0][1]))


In [ ]:
'''
Per verificare che la struttura del dizionario sia corretta e uguale per entrambe le collezioni (raw_collection e raw_collection_unfiltered), 
possiamo ispezionarla con uno script che:

Controlla i livelli del dizionario.
Stampa il numero di soggetti per ogni task.
Conta il numero di run per ciascun soggetto.
Mostra la shape dei dati EEG e delle annotazioni.
Verifica che i tipi siano coerenti: mne.io.Raw e np.ndarray.
Ecco un codice che puoi lanciare dopo aver caricato i due dizionari da file o averli creati:


Possiamo estendere la funzione inspect_raw_collection per:

raccogliere tutti i soggetti (subj) e tutte le run (run) disponibili nel collection,
mostrarli alla fine con np.unique per avere una lista completa e verificare quali soggetti e run siano presenti 
in ciascuna struttura (filtrata e non filtrata),
eventualmente restituire le liste uniche per confrontarle.


Nel tuo caso, gli annotations non sono un oggetto mne.Annotations, ma un array numpy.ndarray con forma (N, 5), dove:

N = 30 è il numero di eventi o segmenti annotati (tipicamente i 30 trial della run),
e le 5 colonne rappresentano probabilmente attributi di un array fatto così, ad esempio:
[  1.    4.1 656.    1.  656. ]


Vediamolo passo per passo e confrontiamolo con le informazioni dal paper PMC10998040, 
in particolare Tabella 2 e le descrizioni relative all’organizzazione temporale dei trial.

| Colonna | Significato                        | Esempio                            |
| ------- | ---------------------------------- | ---------------------------------- |
| `[0]`   | **Etichetta originale** del trial  | `1` (T0), `2` (T1), `3` (T2)       |
| `[1]`   | **Durata del trial (secondi)**     | `4.1`                              |
| `[2]`   | **Durata in campioni** (`samples`) | `656`                              |
| `[3]`   | **Indice di inizio trial**         | `657`                              |
| `[4]`   | **Indice di fine trial**           | `1312`                             |


'''

import numpy as np

def inspect_raw_collection(collection, name=""):
    print(f"\nInspecting structure: \033[1m{name}\033[0m\n" + "-"*50)
    
    all_subjects = []
    all_runs = []

    for task, subjects in collection.items():
        print(f"📁 Task: \033[94m{task}\033[0m → {len(subjects)} subjects")
        
        # Mappa etichette in base al task
        if task in ["task_1", "task_2"]:
            label_map = {1: "T0 (Rest)", 2: "T1 (Left Fist)", 3: "T2 (Right Fist)"}
        else:
            label_map = {1: "T0 (Rest)", 2: "T1 (Both Feet)", 3: "T2 (Both Fists)"}

        for subj, runs in subjects.items():
            all_subjects.append(subj)
            print(f"  └─ 👤 Subject {subj} → {len(runs)} runs")
            for run, (raw, annotations) in runs.items():
                all_runs.append(run)
                print(f"      └─ 🏃‍♂️ Run {run}:")
                print(f"         ├─ Raw type:      {type(raw)}")
                print(f"         ├─ Data shape:    {raw.get_data().shape}  (channels x time)")
                print(f"         ├─ Annotations:   {type(annotations)}, shape={annotations.shape}")
                if isinstance(annotations, np.ndarray):
                    print(f"           ├─ First annotation row: {annotations[0]}")
                    print()
                    print(f"             ├─ Unique stimolo_id [0]: {np.unique(annotations[:, 0])}")
                    print(f"             ├─ Durata trial secondi [1]: {np.unique(annotations[:, 1])}")
                    print(f"             ├─ Durata trial in samples [2]: {np.unique(annotations[:, 2])}\n")
                    print(f"             ├─ Start sample  [3]: {np.unique(annotations[:, 3])}\n")
                    print(f"             └─ End sample [4]: {np.unique(annotations[:, 4])}\n")
                print(f"         └─ Duration (s):  {raw.times[-1]:.2f}s")
                
            break  # Solo il primo soggetto per ogni task
        print()

    # Mostra soggetti e run unici
    unique_subjects = np.unique(all_subjects)
    unique_runs = np.unique(all_runs)

    print(f"\n📊 \033[1mRiepilogo {name}\033[0m")
    print(f"→ Soggetti unici: {len(unique_subjects)} → {unique_subjects}")
    print(f"→ Run uniche:     {len(unique_runs)} → {unique_runs}")
    print("✓ Done.\n")

    return unique_subjects, unique_runs

In [ ]:
#inspect_raw_collection(EEG_data_filtered, "Filtered")
#inspect_raw_collection(EEG_data_unfiltered, "Unfiltered (raw original)")
inspect_raw_collection(EEG_post_ICA, "Post ICA")

In [ ]:
print(EEG_post_ICA['task_1'].keys())
print()
print(EEG_post_ICA['task_1']['100'].keys())
print()
print(type(EEG_post_ICA['task_1']['100'][0]))
print()
print(len(EEG_post_ICA['task_1']['100'][0]))
print()
print(type(EEG_post_ICA['task_1']['100'][0][0]))
print(type(EEG_post_ICA['task_1']['100'][0][1]))
print(EEG_post_ICA['task_1']['100'][0][1].shape)

In [ ]:
print(type(EEG_post_ICA['task_1']['100'][0][1][0]))

In [ ]:
print((EEG_post_ICA['task_1']['100'][0][1][0]))

#### **POST Custom ICA: Istructions to create the New Data Structure before Autoreject Step**

Allora aspetta però...


 perché allora devo modificare la funzione che mi epoca i dati, partendo però da 

EEG_post_ICA adesso... perché escludendo la parte di salvataggio degli ICA models ...


Originariamente i miei la mia costruzione del set dei dati era questa


#Percorso del file di checkpoint\CHECKPOINT_PATH = Path("/home/stefano/EEGMMIDB_Analyses/task_data_ICA.pkl")
CHECKPOINT_PATH = Path("/home/stefano/EEGMMIDB_Analyses/task_data_ICA.pkl")

ICA_MODELS_PATH = Path("/home/stefano/EEGMMIDB_Analyses/ica_models.pkl")  # Per salvare gli oggetti ICA

CHECKPOINT_PATH.parent.mkdir(exist_ok=True, parents=True)


#---------------------------------------------------- #
#Tabella 2 – mappatura Task  ↔  indice run (0‑based) #
#---------------------------------------------------- #
TASK_RUNS = {
    "task_1": [0, 4, 8],    # curated 1, 5, 9
    "task_2": [1, 5, 9],    # curated 2, 6, 10
    "task_3": [2, 6, 10],   # curated 3, 7, 11
    "task_4": [3, 7, 11],   # curated 4, 8, 12
}

def label_to_Tx(label_num: int) -> str:
    """Converte la label 1‑12 nei tre gruppi T0/T1/T2."""
    return ["T0", "T1", "T2"][(label_num - 1) % 3]

def to_plain_dict(d):
    """
    Ricorsivamente trasforma defaultdict (e dict) in normali dict,
    così da poterlo serializzare con pickle.
    """
    if isinstance(d, defaultdict) or isinstance(d, dict):
        return {k: to_plain_dict(v) for k, v in d.items()}
    else:
        return d


def build_task_data(
    signal: List[Dict[str, list]],
    labels: List[Dict[str, list]],
    fs: int = 160,
    epoch_len_s: float = 4.0,
    drop_shorter: bool = True,
    #lowcut: float = 1.0,
    #highcut: float = 45.0,
    #order: int = 4,
    channel_names: List[str] = EEG_channels_names,
    checkpoint_every_subject: bool = True
) -> Dict[str, Dict[str, Dict[str, Dict[str, np.ndarray]]]]:
    """
    task_data[task][subject][T0|T1|T2]['data'|'labels']

    * 'data'   → ndarray (n_trial, 64, 640)
    * 'labels' → ndarray (n_trial,)
    """
    epoch_len = int(epoch_len_s * fs)
    task_data = defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(lambda: {"data": [], "labels": []})
        )
    )
    
    ica_models = {}  # Dizionario per salvare gli oggetti ICA per ogni task, soggetto e run
    
    for task, runs in TASK_RUNS.items():
        for r in runs:                               # loop sui run della Table 2 (0‑based)
            run_sig   = signal[r]
            run_label = labels[r]

            for subj_key in run_sig.keys():          # per ogni soggetto
                subj_anno_key = subj_key.replace("Signal", "Annotations")
                
                # 1) estrai raw EEG (n° samples di una intera run ,n° elettrodi)
                #dim 1: length totale della run
                #dim 2: 64 (elettrodi usati durante la rec) 
                
                sig_arr = run_sig[subj_key][0]            # (N, 64)
                
                
                # 2) applica il filtro passa-banda (1-45 Hz) **prima** di fare l’epoching
                #    (qui uso i default lowcut=1, highcut=45, order=4)
                
                #sig_arr = bandpass_filter(sig_arr, fs, lowcut, highcut, order)
                
                # 3) applico l'ICA ai già filtrati **prima** di fare l’epoching
                #    (qui uso i default lowcut=1, highcut=45, order=4)
                
                # 2) Esegui l'ICA
                sig_arr, ica = ICA(sig_arr, fs, channel_names)
                
                ann_arr = np.asarray(run_label[subj_anno_key][0])

                onsets = ann_arr[:, 3].astype(int) - 1    # 0‑based
                durs   = ann_arr[:, 2].astype(int)
                lbls   = ann_arr[:, 0].astype(int)

                subject_id = subj_key.split("_")[1]       # '100', ecc.
                
                sig_arr = np.array(sig_arr)
                                   
                for onset, dur, labnum in zip(onsets, durs, lbls):
                    if dur < epoch_len and drop_shorter:
                        continue
                    if onset + epoch_len > sig_arr.shape[0]:
                        continue

                    # --------- estrazione e TRASPOSTA (64, 640) ---------
                    seg = sig_arr[onset : onset + epoch_len, :].T.astype(np.float32)

                    Tx = label_to_Tx(labnum)
                    td = task_data[task][subject_id][Tx]
                    td["data"].append(seg)
                    td["labels"].append(labnum)
                
                # —––––– CHECKPOINT PARZIALE –––––—
                #Checkpoint parziale: garantisce che, dopo ogni soggetto, il file contenga già tutti i dati processati fino a lì.
                
                '''
                L’idea di “normalizzare” i tuoi defaultdict prima del dump con una funzione come to_plain_dict è perfettamente valida 
                e ti permette di tenerti il resto del codice praticamente uguale.
                
                IO:
                
                Quindi questo dovrebbe riuscire a salvarmi le varie ICA per ogni soggetto ogni volta 
                E alla fine impili il tutto insieme e ti salva la struttura completa.
                L'idea dovrebbe essere che, salvando ogni soggetto, poi quando devi salvarsi la struttura impilata di tutti i soggetti assieme, 
                avendo già creato parzialmente la struttura di ogni singolo soggetto, dovrebbe salvarmi con più "facilità" la struttura finale risultante ?
                
                
                CHATGPT:
                Esatto: l’idea è proprio quella.

                Durante il loop, subito dopo aver processato ciascun soggetto (filtro → ICA → epoching), 
                tu fai un dump parziale di task_data (convertito in dict “plain” per essere serializzabile). 
                
                In questo modo, se il kernel si blocca, hai già su disco il risultato completo fino all’ultimo soggetto “finito”.
                Alla fine, una volta terminati tutti i soggetti e fatto lo stacking definitivo (np.stack), fai un ultimo dump di task_data 
                (sempre convertito in dict), che contiene ora tutta la struttura completa con gli array finali.

                Quindi:

                Checkpoint parziale

                Salvi man mano liste di epoche ("data": […], "labels": […]) soggetto per soggetto.
                Il file cresce soggetto dopo soggetto, ma hai sempre uno stato coerente fino a quel punto.

                Checkpoint finale
                Sovrascrivi con la versione “impilata” (ndarray) di tutta la struttura.

                Questo non “alleggerisce” magicamente il salvataggio finale in termini di complessità algoritmica,
                ma ti garantisce di non perdere i dati già elaborati se il processo si interrompe a metà. 
                
                Alla fine, il file conterrà esattamente la stessa cosa che avresti ottenuto facendo un unico dump a lavoro ultimato.
                Se poi ti interessa riprendere da un checkpoint intermedio, basta caricare quel file, ricostruire (o estendere) task_data in memoria 
                e continuare il loop da dove sei arrivato.

                In breve:

                Sì, checkpoint soggetto-per-soggetto + dump finale ti dà sia sicurezza che la struttura completa, senza perdere nulla.
                No, la serializzazione finale non diventa “più semplice” in sé, 
                ma è garantita dal fatto che usi sempre to_plain_dict al momento del dump, mentre in memoria rimane il defaultdict per un accesso automatico
                senza KeyError.

                '''
                
                # —––––– CHECKPOINT PARZIALE –––––—
                # Aggiungi l'oggetto ICA al dizionario
                ica_models[(task, subj_key, r)] = ica
                
                
                plain = to_plain_dict(task_data)
                
                if checkpoint_every_subject:
                    with open(CHECKPOINT_PATH, "wb") as f:
                        pickle.dump(plain, f)
                        
                    # Salva anche gli oggetti ICA separatamente
                    with open(ICA_MODELS_PATH, "wb") as f:
                        pickle.dump(ica_models, f)
                    print(f"[Checkpoint] salvato dopo soggetto \033[1m{subj_key}\033[0m")   
                    
    # ------ stack finale e conversione a ndarray ------
    for task in task_data:
        for subj in task_data[task]:
            for Tx in task_data[task][subj]:
                td = task_data[task][subj][Tx]
                if td["data"]:
                    td["data"]   = np.stack(td["data"], axis=0)           # (n, 64, 640)
                    td["labels"] = np.asarray(td["labels"], dtype=int)    # (n,)
                else:  # nessun trial valido
                    td["data"]   = np.empty((0, 64, epoch_len), np.float32)
                    td["labels"] = np.empty((0,), int)
    
    plain = to_plain_dict(task_data)
    
    # —––––– SALVATAGGIO FINALE –––––—
    with open(CHECKPOINT_PATH, "wb") as f:
        pickle.dump(plain, f)
        
    with open(ICA_MODELS_PATH, "wb") as f:
        pickle.dump(ica_models, f)
        
    print("[Salvataggio finale] task_data_ICA completo.")
    
    return task_data



ma il punto è che la costruzione del dataset partiva da 'Signal' e 'Annotations'....

Mentre ORA, dobbiamo partire da un dizionario che è EEG_post_ICA, che è dizionario annidato che ha alla fine una tupla formata dai dati (post ICA, ma come ancora unica serie temporale di circa 2 minuti) come primo elemento e le labels come secondo elemento (che contiene tutti i vari metadati per fare la costruzione del dataset)...

invece prima  'Signal' e 'Annotations' erano due strutture separate!

la difficoltà è questa in sostanza!


La mia idea è, partendo questa volta da EEG_post_ICA, vorrei ri-adattare questa filosofia di approccio alla costruzione del dataset usata prima, dove

label_to_Tx era stata già fatto quando avevo creato il dataset con il solo filtraggio, per cui quella parte dovrebbe esser ok, nel senso che ho T0, T1 e T2 fatto in questo modo per ogni task


La corrispondenza era infatti:

Task 1: {1=T0 Rest, 2=T1 Left Fist, 3=T2 Right Fist}

Task 2: {4=T0 Rest, 5=T1 Both Feet, 6=T2 Both Fists}

Task 3: {7=T0 Rest, 8=T1 Both Fists, 9=T2 Both Feet}

Task 4: {10=T0 Rest, 11=T1 Right Fist, 12=T2 Left Fist}


Per sicurezza, però, vorrei ri-eseguire lo stesso approccio, ri-usando anche nuovamente label_to_Tx (anche se è stato già usato)







Infatti, mi sono reso conto che, quando visualizzavo la struttura dati di EEG_post_ICA, con questa


import numpy as np

def inspect_raw_collection(collection, name=""):
    print(f"\nInspecting structure: \033[1m{name}\033[0m\n" + "-"*50)
    
    all_subjects = []
    all_runs = []

    for task, subjects in collection.items():
        print(f"📁 Task: \033[94m{task}\033[0m → {len(subjects)} subjects")
        
        # Mappa etichette in base al task
        if task in ["task_1", "task_2"]:
            label_map = {1: "T0 (Rest)", 2: "T1 (Left Fist)", 3: "T2 (Right Fist)"}
        else:
            label_map = {1: "T0 (Rest)", 2: "T1 (Both Feet)", 3: "T2 (Both Fists)"}

        for subj, runs in subjects.items():
            all_subjects.append(subj)
            print(f"  └─ 👤 Subject {subj} → {len(runs)} runs")
            for run, (raw, annotations) in runs.items():
                all_runs.append(run)
                print(f"      └─ 🏃‍♂️ Run {run}:")
                print(f"         ├─ Raw type:      {type(raw)}")
                print(f"         ├─ Data shape:    {raw.get_data().shape}  (channels x time)")
                print(f"         ├─ Annotations:   {type(annotations)}, shape={annotations.shape}")
                if isinstance(annotations, np.ndarray):
                    print(f"           ├─ First annotation row: {annotations[0]}")
                    print()
                    print(f"             ├─ Unique stimolo_id [0]: {np.unique(annotations[:, 0])}")
                    print(f"             ├─ Durata trial secondi [1]: {np.unique(annotations[:, 1])}")
                    print(f"             ├─ Durata trial in samples [2]: {np.unique(annotations[:, 2])}\n")
                    print(f"             ├─ Start sample  [3]: {np.unique(annotations[:, 3])}\n")
                    print(f"             └─ End sample [4]: {np.unique(annotations[:, 4])}\n")
                print(f"         └─ Duration (s):  {raw.times[-1]:.2f}s")
                
            break  # Solo il primo soggetto per ogni task
        print()

    # Mostra soggetti e run unici
    unique_subjects = np.unique(all_subjects)
    unique_runs = np.unique(all_runs)

    print(f"\n📊 \033[1mRiepilogo {name}\033[0m")
    print(f"→ Soggetti unici: {len(unique_subjects)} → {unique_subjects}")
    print(f"→ Run uniche:     {len(unique_runs)} → {unique_runs}")
    print("✓ Done.\n")

    return unique_subjects, unique_runs


in realtà se VEDI BENE, si usava un MAPPING, che non mi faceva vedere che IN REALTA' le labels erano ANCORA T0,T1 e T2 per ogni task !

Quindi la differenza è che qui, dovrei integrare l'approccio di prima, ma partendo da EEG_post_ICA e costruendo di nuovo un dizionario annidato (MA NON default dict! ed escludendo la parte della ICA e dei checkpoint associati all'ICA, che son stati già fatti) a 4 livelli, dove i livelli erano


task, soggetto, condizione sperimentale e poi dentro la suddivisione in due sottochiavi di dati e labels di quella condizione sperimentale lì ...

e se possibile anche integrare queste ALTRE funzioni , tra cui:


1) 'rename_conditions' che serviva per 

partire dal mio nuovo EEG_data (il dizionario annidato ottenuto a 4 livelli post-fase di epocatura, ossia con già i dati suddivisi in epoche di 4 secondi per ogni trial), per i dati ogni task, ossia a task_1, task_2 etc.
mi andasse a cambiare il nome della chiave del 3° livello, a seconda del task di cui ci si riferisce, per ogni soggetto...

ossia, vorrei che le chiavi T0, T1, e T2 prendessero il nome della condizione sperimentale corrente del task. Ad esempio

task 1 : 

T0: rest
T1: left_fist
T2: right_fist

task 2 : 

T0: rest
T1: left_fist
T2: right_fist


task 3 : 

T0: rest
T1: both_fists
T2: both_feet

task 4 : 

T0: rest
T1: both_fists
T2: both_feet


l'esempio di codice era questo:

'''
Ecco una funzione semplice e rapida che fa esattamente ciò che chiedi: rinomina le chiavi T0, T1 e T2 
con il nome esplicito della condizione sperimentale in base al task:
'''

def rename_conditions(task_data):
    # Definisci la mappatura per ciascun task
    condition_mapping = {
        'task_1': {'T0': 'rest', 'T1': 'left_fist', 'T2': 'right_fist'},
        'task_2': {'T0': 'rest', 'T1': 'left_fist', 'T2': 'right_fist'},
        'task_3': {'T0': 'rest', 'T1': 'both_fists', 'T2': 'both_feet'},
        'task_4': {'T0': 'rest', 'T1': 'both_fists', 'T2': 'both_feet'},
    }
    
    # Nuovo dizionario da restituire con nomi modificati
    renamed_data = {}
    
    # Loop sui task e soggetti per cambiare i nomi delle condizioni
    for task, subjects in task_data.items():
        renamed_data[task] = {}
        mapping = condition_mapping[task]
        
        for subj, conditions in subjects.items():
            renamed_data[task][subj] = {}
            for old_cond, data_labels in conditions.items():
                new_cond = mapping[old_cond]
                renamed_data[task][subj][new_cond] = data_labels
    
    return renamed_data


dove in questo caso 'task_data' sarebbe appunto il nuovo dizionario annidato costruito a partire da EEG_post_ICA

2) 'reformat_for_task' che sarebbe funzione che prende in ingresso il tuo EEG_data (post_epocatura -ossia il dizionario a 4 livelli task → subject → condition → {data,labels} ossia sempre il nuovo dizionario annidato costruito a partire da EEG_post_ICA) 

e lo riformatta in un dizionario di liste di dict, uno per ciascun task:

'''

def reformat_for_task(task_data):
    """
    Input:
      task_data[task][subj][condition] = {"data": ndarray, "labels": ndarray}
    Output:
      out[task] = [
        { "<cond>_<subj>"        : data_array,
          "<cond>_<subj>_labels": labels_array,
          ... per ogni condition e per ogni subj ...
        },
        ... uno dict per ogni subject ...
      ]
    """
    reformatted = {}
    for task, subjects in task_data.items():
        lst = []
        for subj, conds in subjects.items():
            rec = {}
            for cond, dl in conds.items():
                # costruisci le due chiavi
                key_data   = f"{cond}_{subj}"
                key_labels = f"{cond}_{subj}_labels"
                rec[key_data]   = dl["data"]
                rec[key_labels] = dl["labels"]
            lst.append(rec)
        reformatted[task] = lst
    return reformatted

dove quindi sempre qui anche "task_data" sarebbe appunto il nuovo dizionario annidato costruito a partire da EEG_post_ICA


3) una ultima funzione "preprocess_EEG_labels" che 

mi creava una funzione che a partire da questa struttura (dizionario di liste di dizionari, ossia l'output di "reformat_for_task" ossia --> reformatted in sostanza) ed iterava su ogni soggetto (ogni indice che qua nell'esempio è lo 0 per indicare il primo soggetto della lista di dizionari) e verificava se nelle chiavi ci fosse '_labels' all'interno della chiave del relativo dizionario ed inoltre doveva vedere se comparissero diverse stringhe al suo interno.


Nello specifico: 

se è 'task_1' o 'task_2' come dizionario di partenza allora dovrebbe vedere se poi all'interno di ogni soggetto sulle chiavi delle labels  (quelle che hanno anche '_labels' al suo interno) ci siano 3 possibili stringhe che sono 'rest' 'left_fist' 'right_fist'


se è rest allora le sue labels le rendi dal loro valore corrente a 0
se è left_fist allora le sue labels le rendi dal loro valore corrente a 1
se è right_fist allora le sue labels le rendi dal loro valore corrente a 2


se è 'task_3' o 'task_4' come dizionario di partenza allora dovrebbe vedere se poi all'interno di ogni soggetto sulle chiavi delle labels  (quelle che hanno anche '_labels' al suo interno) ci siano 3 possibili stringhe che sono 'rest' 'both_fists' 'both_feet'


se è rest allora le sue labels le rendi dal loro valore corrente a 0
se è both_fists allora le sue labels le rendi dal loro valore corrente a 1
se è both_feet allora le sue labels le rendi dal loro valore corrente a 2



def preprocess_EEG_labels(eeg_data):
    """
    Come prima, ma con filtro Butterworth di ordine 8.
    """
    mapping_12 = {'rest': 0, 'left_fist': 1, 'right_fist': 2}
    mapping_34 = {'rest': 0, 'both_fists': 1, 'both_feet': 2}

    for task, subjects in eeg_data.items():
        if task in ('task_1', 'task_2'):
            mapping = mapping_12
        elif task in ('task_3', 'task_4'):
            mapping = mapping_34
        else:
            continue

        for subj in subjects:
           
            # 1) rimappo le etichette
            for key, labels in list(subj.items()):
                if key.endswith('_labels'):
                    prefix = key.replace('_labels', '').rsplit('_', 1)[0]
                    if prefix in mapping:
                        subj[key] = np.full(len(labels), mapping[prefix], dtype=int)

    return eeg_data



ora, dimmi tu che cosa convenga fare tra

A) integrare l'approccio di prima, di costruzione del dataset annidato a 4 livelli, ed integrare tutti gli step solo fino a 'reformat_for_task' o se fare 'preprocess_EEG_labels' separatamente.

B) fare tutto assieme, mantenendo la logica di salvataggio del dataset già annidato a 4 livelli con i vari checkpoint e l'integrazione di "rename_conditions", "reformat_for_task" e preprocess_EEG_labels" tutto in un unico blocco.

Propenderei per la B, ma dimmi se sia fattibile, tenendo conto della complessità del task che ti chiedo.

Anche perché in realtà poi, alla fine di tutto, io vorrei usare la funzione Autoreject (con libreria ancora da installare) per fare l'ulteriore verifica di artifact rejection dei trial con valori di voltaggio sopra i 100 microvolt, 

ok per sicurezza, nella table 2 del file pdf https://pmc.ncbi.nlm.nih.gov/articles/PMC10998040/pdf/main.pdf dove si legge a destra 


"Curated dataset" e sotto "Signal and annotation reference" dovrebbe riferirsi al numero della run corrispondente, che io già correttamente riformattato durante la creazione del mio dataset già solo filtrato giusto? 



Sì, il ragionamento è corretto 👍

In annotations la prima colonna che vedi (1 nel tuo esempio [1, 4.1, 656., 1., 656.]) è l’ID della classe locale al task (1, 2, 3), non la stringa “T0/T1/T2”. 

La mappatura label_to_Tx = ["T0","T1","T2"][(label-1)%3] serve proprio a trasformare 1→T0, 2→T1, 3→T2 per qualunque task.

Le colonne “4.1” e “656” confermano la durata trial e i campioni (a 160 Hz: 4.1 s ↔ 656 campioni; 4.2 s ↔ 672), e gli indici [3]=start, [4]=end sono MATLAB 1-based (da portare a 0-based in Python).

In Tabella 2 del paper, la colonna “Signal and annotation reference” indica i run originali da cui sono stati presi segnali e annotazioni per il “curated dataset”. La tua mappatura Python 0-based:

task_1 → [0, 4, 8] (corrisponde a 1,5,9 in 1-based)

task_2 → [1, 5, 9] (2,6,10)

task_3 → [2, 6, 10] (3,7,11)

task_4 → [3, 7, 11] (4,8,12)
è esattamente l’interpretazione giusta.

Se vuoi una mini-verifica runtime (consiglio rapido):

raw, ann = EEG_post_ICA['task_1']['100'][0]   # esempio
fs = int(raw.info['sfreq'])
assert fs == 160

labs = ann[:,0].astype(int)
durs = ann[:,1]
nsmp = ann[:,2].astype(int)
on   = ann[:,3].astype(int)
off  = ann[:,4].astype(int)

#Check coerenza (MATLAB→Python: start dovrebbe essere 1-based)
assert np.all((nsmp == (off - on + 1)))
assert set(labs) <= {1,2,3}
assert set(np.unique(durs)) <= {4.1, 4.2}
assert set(np.unique(nsmp)) <= {656, 672}


In sintesi: puoi usare tranquillamente label_to_Tx per tornare a T0/T1/T2, e la tua mappatura dei run è allineata a quanto indicato nella tabella “Signal and annotation reference”. Procedi sereno 😉



#### **POST Custom ICA: Implementation to create the New Data Structure before Autoreject Step**

In [ ]:
'''

Grande — andiamo con la B: è fattibilissima e ti fa arrivare a 

un pacchetto coerente “dalla serie continua post-ICA → epoche → dizionari riformattati → label 0/1/2 pronte per Autoreject / ML”.

Qui sotto ti lascio tutto il blocco (compatto ma curato) per:

epocare da EEG_post_ICA in un dict annidato a 4 livelli
task → subject → T0/T1/T2 → {"data": (n, ch, t), "labels": (n,)}

rinominare T0/T1/T2 con i nomi espliciti delle condizioni (per task)

riformattare per task in un “dict di liste di dict” (come avevi)

rimappare le etichette in 0/1/2 in base ai nomi delle condizioni

Include il checkpoint opzionale “per soggetto”.


'''


import os, pickle
from pathlib import Path
from typing import Dict, Tuple
import numpy as np
import mne

# ---------- util ----------

'''

                                                                    Perché tenere --> label_to_Tx???


Mi serve per far 'ritornare' i valori numerici in T0, T1,T2 per ogni task
(che è fondamentale poi per fare le modifiche successive con l'aggiunta delle funzioni di integrazione), è corretto il ragionamento?

Sì, il ragionamento è giusto ✅

Quella riga che hai stampato
[1, 4.1, 656., 1., 656.]
significa: stimolo_id=1 → in questo dataset 1=T0, 2=T1, 3=T2 (coerente con ciò che avevi visto: np.unique(annotations[:,0]) = [1.,2.,3.]).

La funzione

def label_to_Tx(label_num: int) -> str:
    return ["T0","T1","T2"][(int(label_num)-1) % 3]


è esattamente ciò che ti serve per ricondurre i numeri a T0/T1/T2.

È anche “robusta”: se mai avessi etichette 1..12 (aggregando i 4 task),
il modulo % 3 le mappa comunque correttamente:

1,4,7,10 → T0

2,5,8,11 → T1

3,6,9,12 → T2

Poi, per task, usi rename_conditions per trasformare T0/T1/T2 nei nomi espliciti (rest/left_fist/right_fist o rest/both_fists/both_feet). Questo è lo step che usa il task.

Infine, se vuoi etichette numeriche 0/1/2 per ML/Autoreject, puoi:

o usare la tua preprocess_EEG_labels (mappa dai nomi condizione → 0/1/2),

oppure, più diretto, salvare subito y = ((label_num-1) % 3) quando epocheggi (ma mantenere prima T0/T1/T2 rende tutto più leggibile).

Quindi:
Sì, tieni label_to_Tx nello step di epocatura da EEG_post_ICA → T0/T1/T2, poi rename_conditions per i nomi per-task, e opzionalmente preprocess_EEG_labels per ottenere 0/1/2.

'''


def label_to_Tx(label_num: int) -> str:
    """Robusta: gestisce 1..12 o 1..3 → T0/T1/T2."""
    idx = (int(label_num) - 1) % 3
    return ["T0", "T1", "T2"][idx]

def _save_checkpoint(obj, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "wb") as f:
        pickle.dump(obj, f)

# ---------- 1) build_task_data: epoche da EEG_post_ICA ----------
def build_task_data_from_postICA(
    EEG_post_ICA: Dict[str, Dict[str, Dict[int, Tuple[mne.io.BaseRaw, np.ndarray]]]],
    epoch_len_s: float = 4.0,
    drop_shorter: bool = True,
    convert_to_uV: bool = True,
    checkpoint_path: Path = None,
    checkpoint_every_subject: bool = False,
) -> Dict[str, Dict[str, Dict[str, Dict[str, np.ndarray]]]]:
    """
    Output:
      task_data[task][subject][T0|T1|T2] = {
          "data":   ndarray (n_trials, n_chan, epoch_len),
          "labels": ndarray (n_trials,)  # etichette originali ann_arr[:,0]
      }
    """
    task_data: Dict[str, Dict[str, Dict[str, Dict[str, np.ndarray]]]] = {}

    for task, subjects in EEG_post_ICA.items():
        task_data.setdefault(task, {})
        for subj, runs in subjects.items():
            # collezioniamo su 3 contenitori (uno per T0/T1/T2)
            bins = {k: {"data": [], "labels": []} for k in ("T0", "T1", "T2")}

            # usa un info “template” dal primo run (serve se poi farai EpochsArray)
            # (facoltativo; utile in futuro)
            first_raw = None

            for run_idx, (raw, ann) in sorted(runs.items()):
                if first_raw is None:
                    first_raw = raw

                fs = float(raw.info["sfreq"])
                ep_len = int(round(epoch_len_s * fs))

                Xv = raw.get_data().T  # (n_times, n_chan), in Volt
                n_times = Xv.shape[0]

                # ann: (30, 5) = [label, dur_s, dur_samples, start_sample, end_sample] (start 1-based)
                onsets = ann[:, 3].astype(int) - 1  # 0-based
                durs   = ann[:, 2].astype(int)
                labs   = ann[:, 0].astype(int)

                for onset, dur, lab in zip(onsets, durs, labs):
                    if dur < ep_len and drop_shorter:
                        continue
                    start = onset
                    stop  = onset + ep_len
                    if stop > n_times:
                        continue

                    seg = Xv[start:stop, :]  # (ep_len, n_chan) in Volt
                    if convert_to_uV:
                        seg = seg * 1e6      # → µV
                    seg = seg.T.astype(np.float32)  # (n_chan, ep_len)

                    Tx = label_to_Tx(lab)
                    bins[Tx]["data"].append(seg)
                    bins[Tx]["labels"].append(int(lab))

            # stack per ogni Tx
            task_data[task].setdefault(subj, {})
            for Tx in ("T0", "T1", "T2"):
                if bins[Tx]["data"]:
                    data = np.stack(bins[Tx]["data"], axis=0)              # (n, ch, t)
                    labels = np.asarray(bins[Tx]["labels"], dtype=int)     # (n,)
                else:
                    # nessun trial valido di quel tipo
                    # per coerenza, usa dimensioni nulle con ep_len dell’ultimo run visto (o default 0)
                    # Proviamo a inferire ep_len dal primo run visto
                    if "raw" in locals() and raw is not None:
                        fs = float(raw.info["sfreq"])
                        ep_len = int(round(epoch_len_s * fs))
                    else:
                        ep_len = 0
                    data = np.empty((0, raw.info["nchan"], ep_len), dtype=np.float32)
                    labels = np.empty((0,), dtype=int)

                task_data[task][subj][Tx] = {"data": data, "labels": labels}

            # checkpoint per soggetto (salva *tutto* task_data corrente: semplice e robusto)
            if checkpoint_every_subject and checkpoint_path is not None:
                _save_checkpoint(task_data, checkpoint_path)

    # salvataggio finale (se richiesto)
    if checkpoint_path is not None and not checkpoint_every_subject:
        _save_checkpoint(task_data, checkpoint_path)

    return task_data

# ---------- 2) rename_conditions: T0/T1/T2 → nomi espliciti ----------
CONDITION_MAPPING = {
    # Usa ESATTAMENTE la mappa che hai richiesto tu
    'task_1': {'T0': 'rest', 'T1': 'left_fist',  'T2': 'right_fist'},
    'task_2': {'T0': 'rest', 'T1': 'left_fist',  'T2': 'right_fist'},
    'task_3': {'T0': 'rest', 'T1': 'both_feet', 'T2': 'both_fists'},
    'task_4': {'T0': 'rest', 'T1': 'both_feet', 'T2': 'both_fists'},
}

def rename_conditions(task_data, mapping=CONDITION_MAPPING):
    renamed = {}
    for task, subjects in task_data.items():
        renamed[task] = {}
        mp = mapping.get(task, {'T0':'T0','T1':'T1','T2':'T2'})
        for subj, conds in subjects.items():
            renamed[task][subj] = {}
            for old_cond, dl in conds.items():
                new_cond = mp.get(old_cond, old_cond)
                renamed[task][subj][new_cond] = dl
    return renamed

# ---------- 3) reformat_for_task ----------
def reformat_for_task(task_data):
    """
    Input:  task_data[task][subj][condition] = {"data": ndarray, "labels": ndarray}
    Output: out[task] = [ { "cond_subj": data, "cond_subj_labels": labels }, ... per subject ... ]
    """
    reformatted = {}
    for task, subjects in task_data.items():
        lst = []
        for subj, conds in subjects.items():
            rec = {}
            for cond, dl in conds.items():
                rec[f"{cond}_{subj}"] = dl["data"]
                rec[f"{cond}_{subj}_labels"] = dl["labels"]
            lst.append(rec)
        reformatted[task] = lst
    return reformatted

# ---------- 4) preprocess_EEG_labels: porta a 0/1/2 (per task) ----------
def preprocess_EEG_labels(eeg_data_reformatted):
    """
    Rimappa *_labels a {0,1,2} in base al nome della condizione nella chiave.
    Segue la tua convenzione richiesta:
      - task_1 & task_2: rest=0, left_fist=1, right_fist=2
      - task_3 & task_4: rest=0, both_feet=1, both_fists=2
    """
    mapping_12 = {'rest': 0, 'left_fist': 1, 'right_fist': 2}
    mapping_34 = {'rest': 0, 'both_feet': 1, 'both_fists': 2}

    out = {}
    for task, subjects in eeg_data_reformatted.items():
        out_list = []
        if task in ('task_1', 'task_2'):
            mp = mapping_12
        elif task in ('task_3', 'task_4'):
            mp = mapping_34
        else:
            mp = {}

        for subj_dict in subjects:
            rec = {}
            for key, val in subj_dict.items():
                if key.endswith('_labels'):
                    # es: "left_fist_12_labels" → condition = "left_fist"
                    prefix = key[:-7]  # rimuove "_labels"
                    cond = prefix.rsplit('_', 1)[0]  # rimuove _<subj>
                    if cond in mp:
                        rec[key] = np.full(len(val), mp[cond], dtype=int)
                    else:
                        # fallback: lascia intatti se non riconosciuti
                        rec[key] = val.astype(int)
                else:
                    rec[key] = val
            out_list.append(rec)
        out[task] = out_list

    return out

# ---------- 5) wrapper "tutto in uno" ----------
def build_and_format_all(
    EEG_post_ICA,
    epoch_len_s=4.0,
    drop_shorter=True,
    convert_to_uV=True,
    checkpoint_path: Path = None,
    checkpoint_every_subject: bool = False,
):
    task_data = build_task_data_from_postICA(
        EEG_post_ICA=EEG_post_ICA,
        epoch_len_s=epoch_len_s,
        drop_shorter=drop_shorter,
        convert_to_uV=convert_to_uV,
        checkpoint_path=checkpoint_path,
        checkpoint_every_subject=checkpoint_every_subject,
    )
    task_data_named = rename_conditions(task_data)
    reformatted     = reformat_for_task(task_data_named)
    reformatted_y   = preprocess_EEG_labels(reformatted)
    return task_data, task_data_named, reformatted, reformatted_y


#### **POST Custom ICA: Implementation for the New Data Structure before Autoreject Step**

#### **Example with One TASK, SUBJECT and RUN**

In [ ]:
'''

ok se volessi quindi provare a fare un esempio su un solo task di un solo soggetto e run, 
E vedere davvero la sola struttura finale che sarebbe "reformatted_y" dal tuo esempio


"build_and_format_all", 

che cosa dovrei fare? perché il punto è che potrei alla fine ri-sovrascrivere a 'task_data' quello che esce fuori da 'reformatted_y' giusto?



Perfetto — facciamo un “mini-giro” su un solo task, un solo soggetto, una sola run e guardiamo solo la struttura finale reformatted_y.
Qui sotto trovi uno snippet pronto che:

crea un sotto-insieme di EEG_post_ICA (p.es. task_1, soggetto "1", run 0),

chiama la tua build_and_format_all(...) (quella che abbiamo definito poco fa),

stampa esattamente cosa c’è in reformatted_y per quel caso.

Nota: lo snippet assume che tu abbia già definite le funzioni:
build_and_format_all, build_task_data_postICA, rename_conditions, reformat_for_task, preprocess_EEG_labels.
'''


In [ ]:
'''

Cosa vedrai

In reformatted_y["task_1"][0] troverai coppie di chiavi del tipo:

rest_1 e rest_1_labels

left_fist_1 e left_fist_1_labels

right_fist_1 e right_fist_1_labels

Le labels saranno già mappate a 0/1/2 in base al task (per task_1: rest→0, left_fist→1, right_fist→2).

Le forme dei dati saranno (n_epoche, n_canali, n_samples) (con n_samples = 4 s × fs), e le labels avranno forma (n_epoche,).
'''

# === 1) Scegli cosa testare ===
test_task    = "task_1"
test_subject = "1"     # stringa, come in EEG_post_ICA
test_runs    = [0]     # prova con la run 0

# === 2) Crea un sotto-dizionario con SOLO quel caso ===
def subset_post_ICA(src, task, subject, runs):
    out = {task: {subject: {}}}
    for r in runs:
        out[task][subject][r] = src[task][subject][r]  # (raw, ann)
    return out

EEG_subset = subset_post_ICA(EEG_post_ICA, test_task, test_subject, test_runs)

# === 3) Esegui la pipeline di epocatura + rinomina condizioni + rimappa labels ===
from pathlib import Path
# evita di sovrascrivere il checkpoint "globale" durante il test
CHECKPOINT_TEST = None  # oppure Path("/tmp/task_data_postICA_TEST.pkl")

task_data, task_data_named, reformatted, reformatted_y = build_and_format_all(
    EEG_subset,
    epoch_len_s=4.0,
    drop_shorter=True,
    convert_to_uV=False,
    checkpoint_path=CHECKPOINT_TEST,
    checkpoint_every_subject=False,
)

print("\n[OK] Build & format completati per il subset.\n")

# === 4) Ispeziona SOLO reformatted_y per quel task/soggetto ===
# reformatted_y[task] = [ { "<cond>_<subj>": X, "<cond>_<subj>_labels": y, ... }, ... ]
rec_list = reformatted_y[test_task]
assert len(rec_list) == 1, f"Mi aspettavo 1 soggetto nel subset, trovato {len(rec_list)}"
rec = rec_list[0]

print(f"Struttura reformatted_y[{test_task}] per il soggetto {test_subject}:")
for k, v in rec.items():
    # v è un np.ndarray; stampa forma e, se sono labels, le uniche
    if k.endswith("_labels"):
        import numpy as np
        uniq, cnt = np.unique(v, return_counts=True)
        print(f"  {k:>20}  shape={v.shape}  values={dict(zip(uniq.tolist(), cnt.tolist()))}")
    else:
        print(f"  {k:>20}  shape={v.shape}")

# (opzionale) mini sanity check: numero totale epoche attese ~ 29-30 (120s / 4s)
n_epochs = sum(v.shape[0] for k,v in rec.items() if not k.endswith("_labels"))
print(f"\nTotale epoche (tutte le condizioni): {n_epochs}")


#### **POST Custom ICA: Implementation for the New Data Structure before Autoreject Step, ALL TASKS, SUBJECTS and RUNS**

In [ ]:
'''
Posso “sovrascrivere” task_data con reformatted_y?

Meglio di no 🙂. Sono strutture diverse e servono a scopi diversi:

task_data (4 livelli: task→soggetto→T0/T1/T2→{data,labels}) è perfetto per ispezioni “a piramide” e controlli qualitativi.

reformatted_y (task→lista di dict per soggetto, con chiavi <cond>_<subj> e <cond>_<subj>_labels) è più comodo per ML e per passaggi successivi 
(p.es. Autoreject/feature extraction).

Tieni entrambi: usa reformatted_y per addestramento/valutazioni e task_data/task_data_named per QC e tracciabilità. Se vuoi, salva reformatted_y in un file a parte 
(p.es. task_data_postICA_reformatted_y.pkl).

Se vuoi, nel prossimo step ti mostro come passare da reformatted_y a un mne.Epochs (per singolo soggetto/task) e applicare Autoreject a livello di epoca.
'''

In [ ]:
import os, pickle
from pathlib import Path
from typing import Dict, Tuple
import numpy as np
import mne

# ---------- util ----------
def label_to_Tx(label_num: int) -> str:
    """Robusta: gestisce 1..12 o 1..3 → T0/T1/T2."""
    idx = (int(label_num) - 1) % 3
    return ["T0", "T1", "T2"][idx]

def _save_checkpoint(obj, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "wb") as f:
        pickle.dump(obj, f)

# ---------- 1) build_task_data: epoche da EEG_post_ICA ----------
def build_task_data_from_postICA(
    EEG_post_ICA: Dict[str, Dict[str, Dict[int, Tuple[mne.io.BaseRaw, np.ndarray]]]],
    epoch_len_s: float = 4.0,
    drop_shorter: bool = True,
    convert_to_uV: bool = True,
    checkpoint_path: Path = None,
    checkpoint_every_subject: bool = False,
) -> Dict[str, Dict[str, Dict[str, Dict[str, np.ndarray]]]]:
    """
    Output:
      task_data[task][subject][T0|T1|T2] = {
          "data":   ndarray (n_trials, n_chan, epoch_len),
          "labels": ndarray (n_trials,)  # etichette originali ann_arr[:,0]
      }
    """
    task_data: Dict[str, Dict[str, Dict[str, Dict[str, np.ndarray]]]] = {}

    for task, subjects in EEG_post_ICA.items():
        task_data.setdefault(task, {})
        for subj, runs in subjects.items():
            
            # collezioniamo su 3 contenitori (uno per T0/T1/T2)
            bins = {k: {"data": [], "labels": []} for k in ("T0", "T1", "T2")}

            # usa un info “template” dal primo run (serve se poi farai EpochsArray)
            # (facoltativo; utile in futuro)
            first_raw = None

            for run_idx, (raw, ann) in sorted(runs.items()):
                
                # contatori drop per questo run
                n_dropped_short = 0
                n_dropped_edge = 0
                
                if first_raw is None:
                    first_raw = raw

                fs = float(raw.info["sfreq"])
                ep_len = int(round(epoch_len_s * fs))

                Xv = raw.get_data().T  # (n_times, n_chan), in Volt
                n_times = Xv.shape[0]

                # ann: (30, 5) = [label, dur_s, dur_samples, start_sample, end_sample] (start 1-based)
                onsets = ann[:, 3].astype(int) - 1  # 0-based
                durs   = ann[:, 2].astype(int)
                labs   = ann[:, 0].astype(int)

                for onset, dur, lab in zip(onsets, durs, labs):
                    
                    # 1) evento più corto della finestra richiesta → scarta
                    
                    if dur < ep_len and drop_shorter:
                        n_dropped_short += 1
                        continue
                    start = onset
                    stop  = onset + ep_len
                    
                    # 2) finestra che esce dalla traccia → scarta
                    
                    if stop > n_times:
                        n_dropped_edge += 1
                        continue

                    seg = Xv[start:stop, :]  # (ep_len, n_chan) in Volt
                    if convert_to_uV:
                        seg = seg * 1e6      # → µV
                    seg = seg.T.astype(np.float32)  # (n_chan, ep_len)

                    Tx = label_to_Tx(lab)
                    bins[Tx]["data"].append(seg)
                    bins[Tx]["labels"].append(int(lab))
                    
                # (facoltativo) log per run
                #if (n_dropped_short + n_dropped_edge) > 0:
                    #print(f"[{task} | subj {subj} | run {run_idx}] dropped short={n_dropped_short}, edge={n_dropped_edge}")

            # stack per ogni Tx
            task_data[task].setdefault(subj, {})
            for Tx in ("T0", "T1", "T2"):
                if bins[Tx]["data"]:
                    data = np.stack(bins[Tx]["data"], axis=0)              # (n, ch, t)
                    labels = np.asarray(bins[Tx]["labels"], dtype=int)     # (n,)
                else:
                    # nessun trial valido di quel tipo
                    # per coerenza, usa dimensioni nulle con ep_len dell’ultimo run visto (o default 0)
                    # Proviamo a inferire ep_len dal primo run visto
                    
                    '''
                    Ottima domanda — non sto insinuando che “mancano i dati” del soggetto/run 🙂. 
                    
                    Il mini-fix serve per robustezza in un caso molto specifico: 
                    quando per una certa condizione (T0/T1/T2) non rimane nessun trial valido dopo i filtri (p.es. drop_shorter=True,
                    ultimi eventi troppo vicini alla fine, ecc.). 
                    
                    In quel ramo “nessun trial valido” dobbiamo comunque creare un array vuoto con shape coerente (0, nchan, ep_len).

                    Perché cambiare?

                    Nel tuo codice “prima”, quel ramo usava la variabile raw (l’ultima vista nel loop). Questo è fragile perché:

                    se per qualche motivo il loop non entra mai (soggetto senza run o run vuoti), raw non esiste → UnboundLocalError;

                    anche se esiste, non è chiaro da dove provengano nchan ed ep_len (dipendono “per caso” dall’ultimo run iterato).

                    Il fix propone di usare first_raw, che memorizziamo appena incontriamo il primo run del soggetto, 
                    e che rappresenta chiaramente “l’info di riferimento” (n. canali, fs) per costruire le shape anche quando una condizione non ha trial.
                    
                    Cosa cambia “sul campo”?

                    Caso tipico (il tuo): tutte le condizioni hanno >0 trial → non cambia nulla nei risultati; questo ramo non viene nemmeno eseguito.
                    Caso limite: una condizione resta con 0 trial (es. hai impostato epoch_len_s più lungo della durata degli eventi,
                    oppure gli ultimi eventi non entrano per onset + ep_len > n_times).

                    Prima: rischio di errore (o shape incoerenti).
                    Dopo: ottieni in modo sicuro data.shape == (0, nchan, ep_len) e labels.shape == (0,), mantenendo la struttura uniforme e senza crash.

                    In breve: il fix non cambia i risultati quando i trial ci sono, ma rende la funzione a prova di bomba nel raro caso di condizione vuota. 
                    Costa zero tenerlo e ti evita sorprese in futuro.

                    '''
                    
                    #if "raw" in locals() and raw is not None:
                        #fs = float(raw.info["sfreq"])
                        #ep_len = int(round(epoch_len_s * fs))
                    #else:
                        #ep_len = 0
                    #data = np.empty((0, raw.info["nchan"], ep_len), dtype=np.float32)
                    #labels = np.empty((0,), dtype=int)
                    
                    if first_raw is not None:
                        fs = float(first_raw.info["sfreq"])
                        ep_len = int(round(epoch_len_s * fs))
                        nchan = first_raw.info["nchan"]
                    else:
                        ep_len, nchan = 0, 0
                    data = np.empty((0, nchan, ep_len), dtype=np.float32)
                    labels = np.empty((0,), dtype=int)

                task_data[task][subj][Tx] = {"data": data, "labels": labels}

            # checkpoint per soggetto (salva *tutto* task_data corrente: semplice e robusto)
            if checkpoint_every_subject and checkpoint_path is not None:
                _save_checkpoint(task_data, checkpoint_path)

    # salvataggio finale (se richiesto)
    if checkpoint_path is not None and not checkpoint_every_subject:
        _save_checkpoint(task_data, checkpoint_path)

    return task_data

# ---------- 2) rename_conditions: T0/T1/T2 → nomi espliciti ----------
CONDITION_MAPPING = {
    # Usa ESATTAMENTE la mappa che hai richiesto tu
    
    'task_1': {'T0': 'rest', 'T1': 'left_fist',  'T2': 'right_fist'},
    'task_2': {'T0': 'rest', 'T1': 'left_fist',  'T2': 'right_fist'},
    'task_3': {'T0': 'rest', 'T1': 'both_fists', 'T2': 'both_feet'},
    'task_4': {'T0': 'rest', 'T1': 'both_fists', 'T2': 'both_feet'},
}

def rename_conditions(task_data, mapping=CONDITION_MAPPING):
    renamed = {}
    for task, subjects in task_data.items():
        renamed[task] = {}
        mp = mapping.get(task, {'T0':'T0','T1':'T1','T2':'T2'})
        for subj, conds in subjects.items():
            renamed[task][subj] = {}
            for old_cond, dl in conds.items():
                new_cond = mp.get(old_cond, old_cond)
                renamed[task][subj][new_cond] = dl
    return renamed

# ---------- 3) reformat_for_task ----------
def reformat_for_task(task_data):
    """
    Input:  task_data[task][subj][condition] = {"data": ndarray, "labels": ndarray}
    Output: out[task] = [ { "cond_subj": data, "cond_subj_labels": labels }, ... per subject ... ]
    """
    reformatted = {}
    for task, subjects in task_data.items():
        lst = []
        for subj, conds in subjects.items():
            rec = {}
            for cond, dl in conds.items():
                rec[f"{cond}_{subj}"] = dl["data"]
                rec[f"{cond}_{subj}_labels"] = dl["labels"]
            lst.append(rec)
        reformatted[task] = lst
    return reformatted

# ---------- 4) preprocess_EEG_labels: porta a 0/1/2 (per task) ----------
def preprocess_EEG_labels(eeg_data_reformatted):
    """
    Rimappa *_labels a {0,1,2} in base al nome della condizione nella chiave.
    Segue la tua convenzione richiesta:
      - task_1 & task_2: rest=0, left_fist=1, right_fist=2
      - task_3 & task_4: rest=0, both_fists = 1, both_feet =2
    """
    mapping_12 = {'rest': 0, 'left_fist': 1, 'right_fist': 2}
    mapping_34 = {'rest': 0, 'both_fists': 1, 'both_feet': 2}

    out = {}
    for task, subjects in eeg_data_reformatted.items():
        out_list = []
        if task in ('task_1', 'task_2'):
            mp = mapping_12
        elif task in ('task_3', 'task_4'):
            mp = mapping_34
        else:
            mp = {}

        for subj_dict in subjects:
            rec = {}
            for key, val in subj_dict.items():
                if key.endswith('_labels'):
                    # es: "left_fist_12_labels" → condition = "left_fist"
                    prefix = key[:-7]  # rimuove "_labels"
                    cond = prefix.rsplit('_', 1)[0]  # rimuove _<subj>
                    if cond in mp:
                        rec[key] = np.full(len(val), mp[cond], dtype=int)
                    else:
                        # fallback: lascia intatti se non riconosciuti
                        rec[key] = val.astype(int)
                else:
                    rec[key] = val
            out_list.append(rec)
        out[task] = out_list

    return out

# ---------- 5) wrapper "tutto in uno" ----------
def build_and_format_all(
    EEG_post_ICA,
    epoch_len_s=4.0,
    drop_shorter=True,
    convert_to_uV=True,
    checkpoint_path: Path = None,
    checkpoint_every_subject: bool = False,
):
    task_data = build_task_data_from_postICA(
        EEG_post_ICA=EEG_post_ICA,
        epoch_len_s=epoch_len_s,
        drop_shorter=drop_shorter,
        convert_to_uV=convert_to_uV,
        checkpoint_path=checkpoint_path,
        checkpoint_every_subject=checkpoint_every_subject,
    )
    task_data_named = rename_conditions(task_data)
    reformatted     = reformat_for_task(task_data_named)
    reformatted_y   = preprocess_EEG_labels(reformatted)
    return task_data, task_data_named, reformatted, reformatted_y

In [ ]:
'''ESECUZIONE REALE CON TUTTI I SOGGETTI


Con quel codice ottieni esattamente ciò che descrivi.

build_task_data_from_postICA:

epoca i Raw post-ICA (4 s), 
gestisce l’indicizzazione 1→0, 
scarta 

(i) trial troppo corti e 
(ii) l’eventuale ultimo trial che “sborda” oltre la traccia,

e accumula per soggetto tutti i trial delle 3 condizioni (T0/T1/T2) aggregando le run dello stesso task.

rename_conditions: rinomina T0/T1/T2 in rest/… a seconda del task.
reformat_for_task: produce per ogni task una lista (uno per soggetto) di dict con coppie cond_subj e cond_subj_labels.
preprocess_EEG_labels: rimappa le label in 0/1/2 in base alla condizione (schema task 1–2 vs 3–4).

Salvi infine reformatted_y in task_data_postICA.pkl: è il formato “pronto” per ML/valutazioni e, se vuoi usare autoreject, puoi ricostruire velocemente mne.


'''

from pathlib import Path

CHECKPOINT = Path("/home/stefano/EEGMMIDB_Analyses/task_data_postICA.pkl")

task_data, task_data_named, reformatted, reformatted_y = build_and_format_all(
    EEG_post_ICA,
    epoch_len_s=4.0,
    drop_shorter=True,
    convert_to_uV=False,                 # utile se poi ragioni in µV
    checkpoint_path=CHECKPOINT,         # opzionale
    checkpoint_every_subject=True,      # salva dopo ogni soggetto
)


In [ ]:
import pickle

# Salvare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/'

# 1) salva i dati preppati (Raw o (Raw, labels))
with open(f"{path}task_data_postICA.pkl", "wb") as f:
    pickle.dump(reformatted_y, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
'''

Con il flusso che hai eseguito, il file che salvi/carichi (task_data_postICA.pkl) corrisponde a reformatted_y, cioè:

un dict per task → new_EEG_data['task_1'], new_EEG_data['task_2'], …

in ciascun task hai una lista con un dict per soggetto

dentro il dict del soggetto hai coppie di chiavi per condizione:

"<condizione>_<subjectID>" → l’array dei dati (shape (n_epoche, n_chan, n_times))

"<condizione>_<subjectID>_labels" → il vettore etichette (rimappato in 0/1/2)


Nella struttura che hai ora (quella salvata come reformatted_y), 
tutti i trial di uno stesso soggetto per un dato task e una data condizione sono già impilati insieme, indipendentemente dalla run.
Quindi, ad es., new_EEG_data['task_1'][i]['rest_1'] contiene tutti i trial “rest” del soggetto 1 nel task 1 (sommando le sue run di quel task).

'''

import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}task_data_postICA.pkl', 'rb') as f:
    new_EEG_data = pickle.load(f)
    
path_ica = '/home/stefano/EEGMMIDB_Analyses/pyprep_outputs/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path_ica}EEG_for_ICA.pkl', 'rb') as f:
    EEG_for_ICA = pickle.load(f)

In [ ]:
new_EEG_data.keys()

In [ ]:
print(new_EEG_data['task_1'][0].keys())
print(new_EEG_data['task_2'][0].keys())
print(new_EEG_data['task_3'][0].keys())
print(new_EEG_data['task_4'][0].keys())

#### **AUTOREJECT: Function Application to the New Data Structure Reformatted Post ICA Phase**

#### **All TASKS, Subjects & Runs**

In [ ]:
'''

Autoreject: #https://autoreject.github.io/stable/index.html

Cite
[1] Mainak Jas, Denis Engemann, Federico Raimondo, Yousra Bekhti, and Alexandre Gramfort, “Automated rejection and repair of bad trials in MEG/EEG.”
In 6th International Workshop on Pattern Recognition in Neuroimaging (PRNI), 2016.

[2] Mainak Jas, Denis Engemann, Yousra Bekhti, Federico Raimondo, and Alexandre Gramfort. 2017. “Autoreject: Automated artifact rejection for MEG and EEG data”.
NeuroImage, 159, 417-429.
'''

'''
Cosa misura

Per ogni epoca e per ogni canale calcola l’ampiezza peak-to-peak (p2p). È una misura robusta dell’escursione del segnale nell’epoca.

Soglie “locali” per canale

Impara soglie p2p specifiche per canale (non una unica per tutti) in modo data-driven. Lo fa con cross-validation: prova soglie candidate
e sceglie quelle che meglio separano epoche “plausibili” da “artefattuali”.

Interpolate-o-scarto (ρ, κ)

Due iperparametri chiave:

n_interpolate (ρ): massimo numero di canali che posso interpolare dentro una singola epoca per “ripararla”.
consensus (κ): tolleranza complessiva; se in una epoca i canali che sforano la soglia sono “troppi”, l’epoca viene scartata invece che riparata.

AutoReject fa una griglia sui valori che gli passi (es. [1,2,4,8,16] × 0.7) e sceglie la combinazione migliore via CV.

Cosa esce

Epoche pulite: quelle non scartate; se avevano pochi canali “malati”, quei canali sono interpolati (sferica/3D in base al montage).
Epoche scartate: troppe violazioni secondo la κ scelta.

Un reject_log per sapere quali epoche sono state scartate e, per le epoche tenute, quali canali sono stati interpolati.
Le soglie apprese per canale: ar.threshes_ (in Volt).

Perché non è “= ±100 µV”

AutoReject non usa una soglia fissa globale (tipo ±100 µV). Stima soglie per canale in base ai tuoi dati (più flessibile).
Se vuoi imporre l’equivalente di ±100 µV, usa MNE reject={'eeg': 100e-6} su Epochs. Oppure puoi usare AutoReject ma controllare le soglie apprese (threshes_) 
e verificarne la coerenza.

Parametri che ti interessano davvero

n_interpolate: lista di interi. Con 64 canali, una griglia tipica è [1, 2, 4, 8, 16]. Aumentare il massimo → più aggressivo
(più canali per epoca possono essere “riparati”).

consensus: float in (0,1]. Valori più alti = più tolleranza all’interpolazione (meno scarti, più riparazioni), 
valori più bassi = più rigore negli scarti. 0.7 è un buon compromesso nella pratica.

random_state, n_jobs: riproducibilità e parallelizzazione.

verbose: log interno (noi lo teniamo False e stampiamo il nostro riassunto).



Hai tenuto i dati in Volt (corretto) e converti le soglie in µV solo per il report: perfetto.
Il wrapper costruisce EpochsArray, esegue AutoReject (local), rimpiazza i trial riparati/tenuti e rimuove quelli irrecuperabili, e logga soglie e conteggi. ✅


La tua scelta attuale (AutoReject) è locale: stima soglie per canale, di solito migliore per EEG 64ch.

Ricordati che consensus ∈ [0, 1]: valori più alti ⇒ più facile scartare l’epoca (meno permissivo); più bassi ⇒ più riparazioni (più permissivo). 
Il default 0.7 è un buon punto di partenza.

'''
import numpy as np
import mne
from copy import deepcopy
from autoreject import AutoReject #https://autoreject.github.io/stable/index.html --> https://autoreject.github.io/stable/explanation.html


#_find_info_template(...): prende l’info (sfreq, ch_names, tipo canali, montage) dal primo run disponibile di quel task+soggetto. 
#Serve a costruire Epochs coerenti con i tuoi dati.

def _find_info_template(EEG_post_ICA, task, subj_id):
    """Prende l’info del primo run disponibile per (task, subject)."""
    runs = EEG_post_ICA[task][subj_id]
    first_run_idx = sorted(runs.keys())[0]
    raw, _ = runs[first_run_idx]
    return raw.info.copy()


#Verifica che il numero di canali in X combaci con len(info["ch_names"]). Se ok, crea mne.EpochsArray(X, info, tmin). 
#Nota unità: X è in Volt, che è proprio ciò che vuole AutoReject.

def _build_epochs_from_array(X, info, tmin=0.0):
    """
    X: (n_epochs, n_channels, n_times) in Volt
    info: mne.Info coerente con X (ch_names, sfreq, ch_types='eeg')
    """
    if X.shape[1] != len(info["ch_names"]):
        raise ValueError(f"Mismatch n_channels: X has {X.shape[1]}, info has {len(info['ch_names'])}")
    return mne.EpochsArray(X, info=info, tmin=tmin, verbose=False)



#Se vuoi leggere/stampare le soglie in µV o mV, convertili solo per il report, non per l’input.
#Converte le soglie p-p stimate da AutoReject (salvate in Volt) in unità leggibili per report (µV/mV/V). Non tocca i dati, è solo per logging.
def _convert_threshes_for_report(threshes_dict, unit="uV"):
    """
    Converte il dict {ch_name: threshold_in_volts} in uV/mV solo per reporting.
    unit: "V", "mV", "uV"
    """
    if threshes_dict is None:
        return None
    if unit == "V":
        factor = 1.0
    elif unit == "mV":
        factor = 1e3
    elif unit == "uV":
        factor = 1e6
    else:
        raise ValueError("unit must be 'V', 'mV', or 'uV'")
    return {ch: float(thr * factor) for ch, thr in threshes_dict.items()}


#apply_autoreject_to_reformatted(...):

#copia profonda di new_EEG_data → lavoriamo su una copia e torniamo cleaned.
#per ogni task e per ogni soggetto:
#recupera subj_id dalla chiave (es. 'rest_1' → '1').
#prende l’info (vedi sopra).
#per ogni condizione (tutte le chiavi che non finiscono con _labels):
#prende X (epoche) e y (etichette corrispondenti).
#se vuoto, annota zero e passa oltre.
#costruisce EpochsArray(X, info).
#istanzia AutoReject(...) con la tua griglia ρ/κ, lo fitta e fa transform(..., return_log=True).
#dal reject_log estrae:
#bad_epochs → indici da scartare,
#bad_channels[i] → canali interpolati nella i-esima epoca tenuta.
#aggiorna la struttura copiata (cleaned) con:
#dati riparati X_clean e
#labels corrispondenti filtrate (y[good_idx]).
#salva nel summary i conteggi: epoche in ingresso, tenute, scartate e quante riparate.
#ritorna cleaned (stessa forma di new_EEG_data) e summary.

'''

1) Che cosa significa consensus=0.7?

Nel modello “local” di autoreject ci sono due iper-parametri:

ρ (rho) = n_interpolate: quanti canali al massimo posso interpolare in un’epoca per ripararla.
κ (kappa) = consensus: frazione di sensori “cattivi” oltre la quale l’epoca viene scartata.

La regola è (parafraso la doc ufficiale):

“Le epoche con più di κ·N canali cattivi (N = #canali) vengono scartate. Per le altre epoche, si interpolano i ρ canali peggiori
(se i cattivi sono meno di ρ, si interpolano tutti).” 

LEGGI QUI SOTTO
autoreject.github.io --> https://autoreject.github.io/stable/auto_examples/plot_auto_repair.html




Quindi, con consensus=0.7 e, ad esempio, 64 canali:

scarteresti un’epoca solo se > 44 canali risultano cattivi (0.7·64 ≈ 45).
È una soglia molto permissiva: praticamente non scarterai quasi nulla; la pulizia avverrà quasi tutta via interpolazione (limitata da ρ).

Di solito si lascia scegliere κ via CV passando una griglia (es. np.linspace(0, 0.4, 5));
mettere un singolo valore fisso (0.7) è raro e tendenzialmente troppo “buono”.



2) Cosa definisco con n_interpolate=[1, 2, 4, 8, 16]?

Questa lista è la griglia di valori candidati per ρ. 
Autoreject fa cross-validation interna e sceglie ρ* (quanti canali interpolare al massimo per epoca).

Se in una certa epoca i canali cattivi sono ≤ ρ* → li interpola tutti.
Se i canali cattivi sono > ρ* ma ≤ κ·N → interpola solo i ρ* peggiori e tiene l’epoca.
Se i canali cattivi sono > κ·N → scarta l’epoca. 

autoreject.github.io

Nota: perché avvenga davvero la selezione via CV sia per ρ sia per κ, devi passare array/list a entrambi i parametri.
Se passi un float singolo a consensus, κ resta fisso e non viene ottimizzato.

Consigli pratici per EEG 64 canali

Passa griglie, non valori singoli, così lasci ottimizzare al metodo:

n_interpolate = [1, 2, 4, 8, 16]
consensus     = np.linspace(0.05, 0.4, 8)  # da “severo” a “moderato”


κ = 0.7 è di solito troppo alto: finirai per non scartare quasi nulla.

Se hai HD-EEG (>=128 canali), puoi estendere n_interpolate (es. includere 32).

Controlla i log: se vedi troppe epoche “repaired” con molti canali, prova una griglia di κ un po’ più bassa o limita ρ.
'''


def apply_autoreject_to_reformatted(
    
    #Parametri chiave:
    #new_EEG_data: la lista di dict per task che hai creato con reformat_for_task (chiavi tipo rest_1, rest_1_labels, ecc.).
    #EEG_post_ICA: usato solo per recuperare un info coerente per ogni soggetto/task (non rilegge i dati, prende solo meta-info).
    #n_interpolate_grid = griglia per ρ (quanti canali interpolare max per epoca). Se None, useremo [3,4,6,8,10].
    #consensus_grid = griglia per κ (frazione di canali “cattivi” oltre cui scartare un’epoca). Se None, calcoliamo in funzione di N canali (vedi sotto).
    #n_jobs=-1: sfrutta tutti i core durante la CV interna di AutoReject.
    #verbose: stampa o no i riepiloghi per condizione.
    
    new_EEG_data,
    EEG_post_ICA,
    
    # parametri di default: griglie ρ=3..10 e κ centrato su 12 canali (±2)
    n_interpolate_grid=None,     # se None -> [3,4,6,8,10]
    consensus_grid=None,         # se None -> np.linspace(10/N, 14/N, 5) calcolato per ogni (task, subj)
    cv_max: int = 10,            # ← CV massimo (default AR è 10)
    random_state=97,
    n_jobs=-1,
    verbose=True,
):
    """
    new_EEG_data: dict -> task -> list[dict_per_subject]
    EEG_post_ICA: per ricavare info (N canali, sfreq, ch_names, ecc.)
    Ritorna:
      cleaned: stessa struttura con epoche riparate/scartate
      summary: log per (task, subject, condition)
    """
    
    '''
    Clona la struttura di input: modificare “cleaned” non tocca i tuoi dati originali.
    summary conterrà, per ciascun task/soggetto/condizione, il conteggio epoche in-kept-drop-repaired e le soglie stimate.
    '''
    cleaned = deepcopy(new_EEG_data)
    summary = {}
    
    
    # ---- contatori globali (tutti i task / soggetti / condizioni) ----
    total_in_all = 0            # epoche/trial in ingresso
    total_kept_all = 0          # epoche/trial tenute (ripulite)
    total_dropped_all = 0       # epoche/trial scartate
    total_repaired_all = 0      # epoche/trial tenute ma interpolate (>=1 canale)
    
    '''
    Ciclo per task (task_1 … task_4).
    Inizializza la lista di log per quel task.
    '''
        
    for task, subjects in new_EEG_data.items():
        summary[task] = []
            
        '''
        Ciclo per soggetto (ogni elemento della lista è il dict relativo a un soggetto).
        Prende una chiave dati qualunque (non _labels) per ricavare l’ID soggetto (tutto dopo l’ultimo _).
        '''
        for si, subj_dict in enumerate(subjects):
            # ricava subject id dalla prima chiave dati
            any_data_key = next(k for k in subj_dict if not k.endswith('_labels'))
            subj_id = any_data_key.rsplit('_', 1)[-1]
            
            '''
            Recupera l’info del primo run di quel (task, soggetto).
            N = # canali EEG (serve per trasformare “numero canali cattivi” → frazione per κ).
            '''
            info = _find_info_template(EEG_post_ICA, task, subj_id)
            N = len(info["ch_names"])

            # --- griglie default se non specificate ---
            
            '''
            ρ: se non hai passato nulla, usiamo una griglia ragionevole [3,4,6,8,10].
            AutoReject farà CV interna e sceglierà il ρ* migliore per questo soggetto+condizione.
            '''
            rho_grid = n_interpolate_grid if n_interpolate_grid is not None else [3, 4, 6, 8, 10]
            
            '''
            κ: se non hai passato nulla, costruiamo una griglia che corrisponde a ~10–14 canali cattivi → [10/N, 11/N, 12/N, 13/N, 14/N].
            Questo centra la CV su “~12 canali” ma le lascia un margine di ±2.
            Se vuoi κ fisso (es. 10 canali), passa consensus_grid=[10/N] e stop.
            '''
            # 12 canali come centro, ±2 di margine per la CV → [10..14]/N
            if consensus_grid is None:
                low_ch, high_ch, steps = 10, 14, 5
                kappa_grid = np.linspace(low_ch / N, high_ch / N, steps)
            else:
                kappa_grid = consensus_grid  # già frazione
                
            '''
            Prepara il contenitore di log per questo soggetto (poi aggiungeremo condizione per condizione).
            '''
            
            subj_log = {"subject": subj_id, "conditions": {}}
            
            '''
            Ciclo sulle condizioni di questo soggetto (tutte le chiavi senza _labels).
            Recupera la chiave delle etichette corrispondenti.
            '''
            for key in [k for k in subj_dict if not k.endswith('_labels')]:
                cond = key.rsplit('_', 1)[0]            # 'rest_1' -> 'rest'
                labels_key = f"{cond}_{subj_id}_labels"
                
                '''
                X sono le epoche (già in Volt).
                y le etichette integer (0/1/2) che avevi preprocessato.
                '''
                X = subj_dict[key]                      # (n_epochs, n_chan, n_times) in Volt
                y = subj_dict[labels_key]               # (n_epochs,)
                
                '''
                Condizione senza epoche → logga zeri e passa oltre.
                '''
                if X is None or len(X) == 0:
                    subj_log["conditions"][cond] = {
                        "n_in": 0, "n_kept": 0, "n_dropped": 0, "n_repaired": 0, "drop_idx": []
                    }
                    continue
                    
                '''
                Crea un EpochsArray coerente (stessi canali, sfreq, montaggio dell’info che abbiamo pescato dal run reale).
                '''
                epochs = _build_epochs_from_array(X, info)
                
                '''
                Instanzia AutoReject in modalità local con le griglie che hai scelto.
                fit = CV interna per stimare:
                soglie p-p per canale (threshes_),
                migliore ρ* dalla griglia,
                migliore κ* (se gli hai dato una griglia di frazioni).
                transform applica le decisioni: scarta epoche troppo “cattive” (oltre κ*·N) e interpolazione dei canali problematici (fino a ρ*).
                '''
                
                # --- CV sempre attiva e adattiva al #epoche disponibili ---
                n_ep = len(epochs)
                if n_ep < 2:
                    raise ValueError(
                        f"AutoReject richiede almeno 2 epoche per la CV; trovate {n_ep} epoche in "
                        f"{task} / soggetto {subj_id} / condizione {cond}."
                    )
                
                cv_k = min(cv_max, n_ep)  # es. n_ep=8 → cv=8; n_ep>=10 → cv=10
                
                ar = AutoReject(
                    n_interpolate=rho_grid,   # ρ: lista canali max da interpolare
                    consensus=kappa_grid,     # κ: frazione di canali cattivi oltre cui scartare
                    cv=cv_k,                  # ← usa CV adattiva (sempre attiva)
                    
                    random_state=random_state,
                    n_jobs=n_jobs,
                    verbose=False
                )
                ar.fit(epochs)
                epochs_clean, reject_log = ar.transform(epochs, return_log=True)
                
                '''
                Estrae gli indici epoche scartate / tenute.
                '''
                bad_epochs = reject_log.bad_epochs.astype(bool)
                good_idx = np.where(~bad_epochs)[0]
                drop_idx = np.where(bad_epochs)[0].tolist()
                
                '''
                Conta quante epoche tenute sono state effettivamente riparate (≥1 canale interpolato).
                '''
                # epoche tenute ma riparate (>=1 canale interpolato)
                repaired = 0
                for i in good_idx:
                    try:
                        if len(reject_log.bad_channels[i]) > 0:
                            repaired += 1
                    except Exception:
                        pass
                
                '''
                Prende i dati puliti (solo epoche tenute) e filtra le etichette sugli stessi indici.
                '''
                X_clean = epochs_clean.get_data()
                y_clean = y[good_idx].astype(int)
                
                '''
                Scrive indietro nella struttura clonata cleaned i dati puliti e le labels corrispondenti.
                (Non tocchiamo mai new_EEG_data.)
                '''
                
                cleaned[task][si][key] = X_clean
                cleaned[task][si][labels_key] = y_clean
                
                
                # ---- aggiorna contatori globali ----
                
                total_in_all       += int(len(X))
                total_kept_all     += int(len(X_clean))
                total_dropped_all  += int(len(drop_idx))
                total_repaired_all += int(repaired)
                
                '''
                Recupera dal modello le soglie p-p per canale in Volt e le converte in µV solo per log/report.
                '''
                ppV = getattr(ar, "threshes_", None)            # soglie p-p in Volt
                pp_report = _convert_threshes_for_report(ppV, unit="uV")
                
                
                '''
                Compila il log per questa condizione:

                epoche in ingresso / tenute / scartate,
                quante riparate (interpolate),
                indici scartati,
                soglie p-p per canale (in µV) che AR ha appreso,
                griglie usate e # canali (così puoi rileggere i parametri poi).
                '''
                
                subj_log["conditions"][cond] = {
                    "n_in": int(len(X)),
                    "n_kept": int(len(X_clean)),
                    "n_dropped": int(len(drop_idx)),
                    "n_repaired": int(repaired),
                    "drop_idx": drop_idx,
                    "pp_thresh_unit": "uV",
                    "pp_thresh": pp_report,
                    "rho_grid_used": list(rho_grid),
                    "kappa_grid_used": [float(x) for x in np.atleast_1d(kappa_grid)],
                    "N_channels": int(N)
                }
                
                '''
                Stampa un riassunto leggibile della condizione.
                '''
                if verbose:
                    print(f"[{task} | subj {subj_id} | {cond}] "
                          f"in={len(X):>3d}  kept={len(X_clean):>3d}  "
                          f"dropped={len(drop_idx):>3d}  repaired={repaired:>3d}")
            
            '''
            A fine soggetto → aggiunge il log del soggetto alla lista del task.
            '''

            summary[task].append(subj_log)
    
    # ---- riepilogo globale finale ----
    total_final_all = total_kept_all  # dopo AR, i "rimasti" coincidono con i tenuti
    if verbose:
        pct_kept = 100.0 * total_kept_all / total_in_all if total_in_all else 0.0
        pct_drop = 100.0 * total_dropped_all / total_in_all if total_in_all else 0.0
        pct_rep_kept = 100.0 * total_repaired_all / total_kept_all if total_kept_all else 0.0
        print("\n====== Autoreject — riepilogo globale ======")
        print(f"Epoche totali (iniziali):            {total_in_all}")
        print(f"Epoche tenute (finali ripulite):     {total_kept_all}  ({pct_kept:.1f}%)")
        print(f"Epoche scartate:                     {total_dropped_all}  ({pct_drop:.1f}%)")
        print(f"Epoche rimaste (finali):             {total_final_all}")
        print(f"Epoche riparate tra le tenute:       {total_repaired_all}  ({pct_rep_kept:.1f}% delle tenute)")
    
    
    # (opzionale) salva anche nel summary per uso programmatico
    summary["_global"] = {
        "total_in": int(total_in_all),
        "total_kept": int(total_kept_all),
        "total_dropped": int(total_dropped_all),
        "total_final": int(total_final_all),
        "total_repaired_among_kept": int(total_repaired_all),
    }
    
    '''
    Ritorna:

    cleaned: stessa forma di new_EEG_data, ma con i trial riparati/scartati;
    summary: log completo per task/soggetto/condizione.
    '''
    return cleaned, summary

In [ ]:
'''
Per “mantenere i valori cercati via CV” basta passare liste (griglie) a n_interpolate_grid (ρ) e a consensus_grid (κ). 
Così la CV sceglie ρ ∈ {3,4,6,8,10} e κ ∈ {10/N,…,14/N} (N = #canali del soggetto).
Al termine vedrai il riepilogo globale stampato (grazie alle aggiunte che hai fatto) e puoi anche leggerlo da summary["_global"].


N.B.

Sì — l’adattività della CV lo rende più affidabile nel decidere cosa scartare/tenere rispetto a un 10-fold fisso.

Perché:

N-aware: con poche epoche per (task,soggetto,condizione), un 10-fold è instabile/impossibile; usare cv_k = min(cv_max, n_ep)
evita fold “tirati” e soglie rumorose → meno falsi positivi/negativi.

Stima delle soglie più stabile: la scelta di ρ (canali da interpolare) e κ (frazione di canali cattivi oltre cui scartare) 
via griglie + CV si adatta al “regime” del soggetto/condizione, invece di imporre un’unica severità per tutti.

Coerenza tra soggetti: la griglia di κ in frazione (basata su N canali) rende paragonabili le decisioni anche se N differisce leggermente.

Due note operative:

AutoReject lavora a livello di epoca, non “elimina run” in blocco. Se vuoi prendere decisioni a livello run, 
basta tenere traccia della run originaria di ogni epoca (o lanciare AR separatamente per run) 
e poi definire un criterio tipo: “scarta la run se >30% epoche droppate o media canali interpolati >5”.

Con pochissime epoche (es. 2–3) la CV è comunque rumorosa per natura: l’adattività evita l’errore, 
ma le soglie restano meno precise. In quei casi puoi restringere le griglie (ρ e κ più corte) per regolarizzare ulteriormente.

In sintesi: la tua versione adattiva è un passo avanti in accuratezza e robustezza nella pulizia dei trial; 
se ti serve una decisione “a run”, aggiungiamo un piccolo punteggio qualità per run e una soglia di esclusione.


'''
#1) Lasciare che la funzione usi le griglie di default (già centrate su ciò che volevi)

cleaned, summary = apply_autoreject_to_reformatted(
    new_EEG_data=new_EEG_data,
    EEG_post_ICA = EEG_for_ICA,
    n_interpolate_grid=None,   # -> usa [3, 4, 6, 8, 10]
    consensus_grid=None,       # -> usa np.linspace(10/N, 14/N, 5) per ogni soggetto (≈ 12±2 canali)
    cv_max=10,                 # CV fino a 10, poi min con n_ep
    n_jobs=-1,
    verbose=True,
)

In [ ]:
print('finito')

In [ ]:
'''
====== Autoreject — riepilogo globale ======
Epoche totali (iniziali):            29430
Epoche tenute (finali ripulite):     26093  (88.7%)
Epoche scartate:                     3337  (11.3%)
Epoche rimaste (finali):             26093
Epoche riparate tra le tenute:       0  (0.0% delle tenute)
'''


type(cleaned)


In [ ]:
#cleaned.keys()

In [ ]:
print(cleaned['task_1'][0].keys())
print(cleaned['task_2'][0].keys())
print(cleaned['task_3'][0].keys())
print(cleaned['task_4'][0].keys())


#### **POST AUTOREJECT: Save Post Autoreject Data Structure in compressed format to have the originally desired Data Organization Format**

#### **All TASKS, Subjects & Runs**

In [ ]:
import pickle

# Salvare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/'

# 1) salva i dati preppati (Raw o (Raw, labels))
with open(f"{path}final_cleaned_EEG_data.pkl", "wb") as f:
    pickle.dump(cleaned, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print('finito')

In [ ]:
'''
Cosa fa (in breve):

salva .npz compresso con una voce per ogni foglia (array dati/labels),
ricostruisce la struttura originale (dict → list → dict),
confronta shape, dtype e valori di ogni array,
solo se tutto coincide elimina il .pkl.
Così non cambi versioni di NumPy, non duplichi file grossi, e hai parità garantita tra PKL e NPZ.

'''

import os, pickle, numpy as np

PKL = "/home/stefano/EEGMMIDB_Analyses/final_cleaned_EEG_data.pkl"
NPZ = "/home/stefano/EEGMMIDB_Analyses/final_cleaned_EEG_data.npz"

# ---------- 1) Carica il PKL ----------
with open(PKL, "rb") as f:
    orig = pickle.load(f)  # new_EEG_data: dict -> task -> list[dict]

# ---------- 2) Appiattisci e salva NPZ ----------
def flatten(obj, prefix=""):
    """
    Appiattisce dict/list annidati in un mapping {path: np.ndarray}.
    Liste indicizzate con segmenti numerici (es. task_1/0/rest_1).
    """
    out = {}
    def _rec(x, path):
        if isinstance(x, dict):
            for k, v in x.items():
                _rec(v, f"{path}/{k}" if path else k)
        elif isinstance(x, list):
            for i, v in enumerate(x):
                _rec(v, f"{path}/{i}" if path else str(i))
        else:
            # forza a ndarray (labels/dati sono già array)
            v = np.asarray(x) if not isinstance(x, np.ndarray) else x
            if not isinstance(v, np.ndarray):
                raise TypeError(f"Foglia non convertibile in ndarray: {path} ({type(x)})")
            out[path] = v
    _rec(obj, prefix)
    return out

flat = flatten(orig)
np.savez_compressed(NPZ, **flat)

# ---------- 3) Ricostruisci dal NPZ ----------
def unflatten_npz_simple(npz_obj):
    """
    Ricostruisce dict -> task -> list[dict] da chiavi tipo: task_1/0/rest_1, task_1/0/rest_1_labels, ...
    (esattamente 3 segmenti: task / idx / leaf)
    """
    root = {}
    for full_key in npz_obj.files:
        segs = full_key.split('/', 2)
        if len(segs) != 3:
            raise ValueError(f"Chiave NPZ inattesa (attesi 3 segmenti): {full_key}")
        task, idx_str, leaf = segs
        idx = int(idx_str)
        root.setdefault(task, [])
        while len(root[task]) <= idx:
            root[task].append({})
        root[task][idx][leaf] = npz_obj[full_key]
    return root

npz = np.load(NPZ, allow_pickle=False)
recon = unflatten_npz_simple(npz)

# ---------- 4) Verifica profonda di uguaglianza ----------
ok = True

# a) ogni task/subject/chiave di orig deve combaciare in recon (shape, dtype, valori)
for task in orig:
    if task not in recon:
        print("❌ Manca task in NPZ:", task); ok = False; continue
    if len(orig[task]) != len(recon[task]):
        print(f"❌ Num soggetti diverso per {task}: {len(orig[task])} vs {len(recon[task])}"); ok = False
    for i, rec in enumerate(orig[task]):
        if i >= len(recon[task]):
            print(f"❌ Manca soggetto index {i} in {task}"); ok = False; continue
        for k, arr in rec.items():
            if k not in recon[task][i]:
                print(f"❌ Manca chiave {task}/{i}/{k} nel NPZ"); ok = False; continue
            arr2 = recon[task][i][k]
            if arr.shape != arr2.shape or arr.dtype != arr2.dtype or not np.array_equal(arr, arr2):
                print(f"❌ Mismatch {task}/{i}/{k}: "
                      f"{arr.shape, arr.dtype} vs {arr2.shape, arr2.dtype}")
                ok = False

# b) opzionale: controlla che non ci siano chiavi “extra” nel NPZ
for task in recon:
    if task not in orig:
        print("❌ Task extra nel NPZ:", task); ok = False; continue
    if len(recon[task]) != len(orig[task]):
        print(f"❌ Num soggetti diverso per {task} (NPZ→orig): {len(recon[task])} vs {len(orig[task])}"); ok = False
    for i, rec in enumerate(recon[task]):
        if i >= len(orig[task]):
            print(f"❌ Soggetto extra index {i} in {task}"); ok = False; continue
        for k in rec.keys():
            if k not in orig[task][i]:
                print(f"❌ Chiave extra nel NPZ: {task}/{i}/{k}"); ok = False

print("✅ NPZ identico al PKL" if ok else "❌ Differenze trovate")

# ---------- 5) Elimina il PKL solo se tutto combacia ----------
if ok:
    try:
        os.remove(PKL)
        print(f"🗑️  Rimosso PKL: {PKL}\n💾 Mantieni NPZ: {NPZ}")
    except OSError as e:
        print("⚠️  Non sono riuscito a rimuovere il PKL:", e)
else:
    print("⛔ Non elimino il PKL: NPZ non perfettamente identico.")

In [ ]:
'''
Use new_pd_env environment to open npz formatted dataset!

'''

import numpy as np

NPZ = "/home/stefano/EEGMMIDB_Analyses/final_cleaned_EEG_data.npz"
npz = np.load(NPZ, allow_pickle=False)

def unflatten_npz_simple(npz_obj):
    root = {}
    for full_key in npz_obj.files:
        task, idx_str, leaf = full_key.split('/', 2)
        idx = int(idx_str)
        root.setdefault(task, [])
        while len(root[task]) <= idx:
            root[task].append({})
        root[task][idx][leaf] = npz_obj[full_key]
    return root

new_EEG_data = unflatten_npz_simple(npz)

In [ ]:
new_EEG_data.keys()

## EEGMMIDB Dataset: Post EEG Preprocessing Steps

## Modifica della Funzione per la **Rappresentazione Tempo-Frequenza dei miei dati EEG (STFT)** a seconda del Dataset del Task scelto

- Task 1: Motor Execution (Open and close right or left fist) - Unilateral - Runs 3, 7, 11 
- Task 2: Motor Imagery (Imagine opening and closing right and left fists) - Unilateral - Runs 4, 8, 12 
- Task 3: Motor Execution (Open and close both fists and feet) - Bilateral - Runs 5, 9, 13 
- Task 4: Motor Imagery (Imagine opening and closing both fists and feet) - Bilateral - Runs 6, 10, 14 

#### IO (Modifica della Funzione per la Rappresentazione Tempo-Frequenza dei miei dati EEG (STFT) a seconda del Dataset del Task scelto

Vorrei semplicemente ri-adattare questo codice, in modo che aggiunta una clause condizione (if statament),  all'inizio del codice che mi consenta di filtrare in base al dizionario del relativo task (da scegliere fornendo quindi un nuovo argomento ad un nuovo parametro da aggiungere nella funzione, che chiameremo 'data_task')..

e che ri-esegua però poi, una volta scelto la chiave di primo livello del dizionario corrispondente, associato ai dati del task specifico (i.e., 'task_1', 'task_2', 'task_3', 'task_4'), 

questa trasformazione dei dati in formato tempo-frequenza dei miei dati EEG, eseguendo quindi una short time fourier transform su finestre consecutivo del mio dato di 4s dei dati di esecuzione del movimento e che quindi mi consenta di ottenere una rappresentazione dello spettrogramma per ogni condizione sperimentale, aggregando così l'informazione spettrale ottenuta da queste finestra consecutive


        '''CREAZIONE SPETTROGRAMMI FREQUENZE x TEMPO'''

        import numpy as np
        import pywt
        from scipy.signal import stft

        def create_spectograms_from_stft(data, window_size = 50, step_size = 25, fs = 250):

            # Dizionario per memorizzare i risultati separati per condizione sperimentale
            condition_results = {}

            for idx, data_dict in enumerate(data):
                print(f"\n\n\t\t\t\t\t\033[1mProcessing Subject {idx + 1}\033[0m:\n")

                for condition, value in data_dict.items():
                    if '_labels' in condition:
                        base_condition = condition.replace('_labels', '')  # Rimuove '_labels'
                        condition_key = base_condition
                        labels_key = condition
                    else:
                        base_condition = condition
                        condition_key = base_condition
                        labels_key = f'{base_condition}_labels'

                    # Aggiungi chiavi inizializzate per nuove condizioni
                    if '_labels' not in condition and condition_key not in condition_results:
                        condition_results[condition_key] = {'spectrograms': [], 'labels': []}

                    # Applicazione wavelet solo per array np.ndarray
                    if isinstance(value, np.ndarray):
                        print(f"\nApplicazione STFT per '\033[1m{condition_key}\033[0m'\n")
                        print(f"\033[1mShape\033[0m dei dati per \033[1m{condition_key}\033[0m: {value.shape}")


                        # Esegui STFT per ogni canale e trial          

                        #La funzione stft di SciPy restituisce uno spettrogramma con la forma (n_frequencies, n_time_steps),
                        #quindi:38 corrisponde al numero di bin di frequenza calcolati a partire dalla finestra FFT di 75 punti.

                        #Infatti, per una finestra di lunghezza 75, il numero di bin è calcolato (in questo caso, dato che 75 è dispari) come (75 + 1) // 2, ovvero 38.
                        #Il numero di passi temporali (n_time_steps) viene invece calcolato in base alla lunghezza del segnale e ai parametri di finestra e overlap.

                        #Nel tuo caso, estraendo 250 punti (da 50 a 300) con nperseg=75 e noverlap=25 (cioè con un salto di 50 campioni per finestra), 
                        #il numero di segmenti dovrebbe essere:
                        #1 + [250−75/75-25] = 1 + [175/50]  = 1 + 3 = 4

                        #Pertanto, il risultato di stft (cioè Zxx) ha forma (38, 4) e non (4, 38). 
                        #Se ti aspetti una struttura con l'asse del tempo come terza dimensione (ad esempio, (trial, canali, tempo, frequenze))
                        #potresti dover trasporre l'array in modo da scambiare le dimensioni frequenza e tempo.

                        '''
                        1) crearmi una lista per salvarmi poi tutti gli spettrogrammi di tutti i trials
                        2) per il trials corrente mi salvo lo spettrogramma e lo appendo poi ad 1)
                        3) converto la lista di tutti gli spettrogrammi di tutti i trials della condizione corrente in array numpy 
                        4) carico poi l'array numpy dentro il dizionario della condizione corrente dentro la chiave 'spectrograms'
                        '''

                        all_trial_spectrograms = []

                        for trial in range(value.shape[0]):
                            single_trial_spectrograms = []  # Lista per memorizzare gli spettrogrammi per il trial corrente

                            for channel in range(value.shape[1]):

                                single_channel_data = value[trial, channel, 50:300]  # Estrai i dati del singolo canale

                                #print(f"Segment length: {len(single_channel_data)}")

                                '''
                                TIENI A MENTE CHE DUE PARAMETRI GESTISCONO IL PADDING QUI AUTOMATICAMENTE, CHE SONO:

                                boundary='zeros', 
                                padded=True,

                                PER QUESTO MOTIVO LE FINESTRE SARANNO 11 E NON 9.

                                IN QUESTO MODO, SI MIGLIORA LA RISOLUZIONE IN FREQUENZA ALL'INIZIO E FINE DEL SEGNALE (DI 250 PUNTI!)
                                ANDANDO IN QUESTO MODO A RIDURRE GLI EFFETTI DI BORDO
                                '''

                                f, t, Zxx = stft(single_channel_data, fs=fs, window='hann', nperseg = 50, noverlap = 25)

                                single_trial_spectrograms.append(Zxx)  # Aggiungi lo spettrogramma calcolato per il canale corrente

                            all_trial_spectrograms.append(np.array(single_trial_spectrograms))  # Forma: (numero_canali, frequenze, tempo)

                        condition_results[condition_key]['spectrograms'].extend(all_trial_spectrograms)

                        # Gestione etichette
                        if labels_key in data_dict:
                            num_labels = data_dict[labels_key]
                            if condition_key in condition_results:
                                print(f"\nFound labels for condition \033[1m{labels_key}\033[0m, length: \033[1m{len(num_labels)}\033[0m\n")
                                condition_results[condition_key]['labels'].extend(num_labels)
                    else:
                        print(f"\033[1m{labels_key}\033[0m' non è un np.array, non applicare la STFT ")

            # Preparazione risultati concatenati
            all_subjects_condition_results = {}

            for condition_key, results in condition_results.items():
                if "_labels" in condition_key or len(results['spectrograms']) == 0:  # Verifica che ci siano spettrogrammi
                    print(f"\nSkipping \033[1m{labels_key}\033[0m as it is a label or has no valid data.")
                    continue

                wave_condition_name = f'spectrograms_{condition_key}'
                print(f"\n\nResults for \t\t\t\t\t\033[1m{wave_condition_name}\033[0m:\n\n")

                # Converti in un array NumPy 3D finale
                spectrograms_4d = np.array(results['spectrograms'])  # Forma: (numero_di_trial, numero_canali, frequenze, tempo)
                all_subjects_condition_results[wave_condition_name] = {
                    'spectrograms_4d': spectrograms_4d,
                    'labels': results['labels']
                }

                print(f"spectrograms_4d shape: {spectrograms_4d.shape}")

            '''
            Quindi alla fine si avrà una shape del tipo (44, 61, 38, 7) dove indica che avrai
            uno spettrogramma per ciascun trial, per ciascun canale, 
            che contiene informazioni sulle bande di frequenza e sui punti temporali. 

            Questa struttura è perfetta per essere utilizzata come input per una rete neurale convoluzionale (CNN),
            poiché la rete può apprendere dalle relazioni spaziali e temporali nei dati.
            '''

            return all_subjects_condition_results


tieni presente che ora, 

la mia nuova struttura di ingresso è quello che ho appena cambiato, che è molto simile a quella per cui questa funzione era stata adattata...


e che è new_EEG_data, che a differenza di quella che vedi nell'immagine (therapist_data) -che era una lista di dizionari... dove per ogni soggetto avevi una serie di chiavi che si riferivano ai dati e labels di una certa condizione sperimentale..-

new_EEG_data invece è un dizionario, che ha prima una prima chiave che distingue i dati del relativo task, e poi al suo interno ha la stessa struttura di "therapist_data" (vedi immagine), ossia lista di dizionari, che si riferiscono ai dati e labels di ogni condizione sperimentale per ogni soggetto..

 
 ora semplicemente new_EEG_data ha un livello in più.. l'idea è che la funzione debba iterare su quel livello e vedere se la chiave sia presente tra quelle che ci si aspetta (ad esempio creando una lista di stringhe che hanno la stesse stringhe di una delle chiavi del primo livello di new_EEG_data, ossia tra   'task_1', 'task_2', 'task_3', 'task_4'..

in questo modo, si fa in modo che se esiste una di queste chiavi dentro new_EEG_data, allora poi si itererà per le chiavi del dizionario dei dati di ogni soggetto e si fa la stessa identica procedura che si eseguiva prima.. ottimizzando le modifiche al minimo del codice che ho già creato  

#### IO: Domanda su Window Size e Step Size

ok ora è una domanda di ragionamento: dato che ho tagliato a 4 secondi il mio dato EEG per tutti i dati di ogni condizione sperimentale e avendo ora i dati quindi di 640 campioni, ma con frequenza di campionamento a 160 Hz...

che cosa mi consigli di mettere come window_size e step_size qui?

#### CHATGPT: Risposta su Window Size e Step Size

Con 640 sample a 160 Hz (→ 4 s di dati) hai una finestra totale piuttosto corta, per cui dovrai bilanciare risoluzione in frequenza vs risoluzione in tempo:
    
| Scelta  | window_size (campioni) | durata finestra (s) | freq. risoluzione Δf≈1/T (Hz) | step_size (campioni) | overlap (%) |
|---------|------------------------|---------------------|-------------------------------|----------------------|-------------|
| 1 s     | 160                    | 1,0                 | 1 Hz                          | 80                   | 50 %        |
| 0,5 s   | 80                     | 0,5                 | 2 Hz                          | 40                   | 50 %        |
| 0,25 s  | 40                     | 0,25                | 4 Hz                          | 20                   | 50 %        |
| 2 s     | 320                    | 2,0                 | 0,5 Hz                        | 160                  | 50 %        |


Raccomandazioni

Se ti interessa studiare bande strette (α, β, γ…), meglio window_size ≥ 1 s (160 campioni), che ti dà Δf ≃ 1 Hz.
Se invece vuoi seguire variazioni rapide nel tempo (ad es. transienti > 250 ms), puoi scalare fino a 0.5 s (80 campioni), Δf ≃ 2 Hz.
L’overlap al 50 % (step_size = window_size/2) è uno standard che conserva buon compromesso.



Esempio concreto


**#finestra = 1 secondo, overlap = 50%**
window_size = 160
step_size   = 80
fs          = 160

oppure, se ti va di spingerti su tempi più corti:

**#finestra = 0.5 secondi, overlap = 50%**
window_size =  80
step_size   =  40
fs          = 160
Scegli in base a:

1) Δf minima di interesse (ad es. se vuoi separare 8 Hz da 12 Hz serve almeno Δf ≤ 4 Hz, quindi finestra ≥ 0.25 s).
2) Temporal resolution desiderata (finestre più corte catturano transitori migliori).

In generale per EEG MI io partirei con 1 s (160 campioni) e 50 % overlap, poi sperimenterei con 0.5 s se ti servono finestre più ravvicinate.

#### IMPLEMENTAZIONE -Modifica della Funzione per la Rappresentazione Tempo-Frequenza dei miei dati EEG (STFT) a seconda del Dataset del Task scelto

#### **Calcolo Spettrogrammi** All **SINGLE** Subject Data (**TH**) per **Tutte le Condizioni sperimentali**

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la isualizzazione dei dati
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
'''
Con questo workflow puoi poi caricare in ambiente new_pd_env con:

'''

import numpy as np

NPZ = "/home/stefano/EEGMMIDB_Analyses/final_cleaned_EEG_data.npz"
npz = np.load(NPZ, allow_pickle=False)

def unflatten_npz_simple(npz_obj):
    root = {}
    for full_key in npz_obj.files:
        task, idx_str, leaf = full_key.split('/', 2)
        idx = int(idx_str)
        root.setdefault(task, [])
        while len(root[task]) <= idx:
            root[task].append({})
        root[task][idx][leaf] = npz_obj[full_key]
    return root

new_EEG_data = unflatten_npz_simple(npz)


In [ ]:
'''
Una volta che hai new_EEG_data = unflatten_npz_simple(npz), puoi contare i trial totali (e fare un riepilogo per task / soggetto / condizione) così:
'''

def count_trials(new_EEG_data, verbose=True):
    """
    new_EEG_data: dict -> task -> list[dict_per_subject]
    Ogni dict_per_subject ha chiavi tipo:
      - "<cond>_<subj>"           -> array (n_epochs, n_chan, n_times)
      - "<cond>_<subj>_labels"    -> array (n_epochs,)
    Ritorna:
      totals:      {"global": int, "tasks": {task: int}}
      per_subj:    {task: {subj_id: int}}
      per_cond:    {task: {cond_name: int}}  # cond_name senza _<subj>
      problems:    lista di mismatch (dati vs labels mancanti o con len diversa)
    """
    totals = {"global": 0, "tasks": {}}
    per_subj = {}
    per_cond = {}
    problems = []

    for task, subj_list in new_EEG_data.items():
        t_total = 0
        per_subj[task] = {}
        per_cond[task] = {}

        for subj_dict in subj_list:
            # ricava subject id da una qualsiasi chiave dati (non _labels)
            try:
                any_key = next(k for k in subj_dict if not k.endswith('_labels'))
                subj_id = any_key.rsplit('_', 1)[-1]
            except StopIteration:
                # soggetto “vuoto”
                continue

            s_total = 0

            # 1) conteggio “ufficiale” dai *_labels
            for key, arr in subj_dict.items():
                if key.endswith('_labels'):
                    # es: "rest_12_labels" -> cond_base="rest_12", cond_name="rest"
                    cond_base = key[:-7]
                    cond_name = cond_base.rsplit('_', 1)[0]
                    n = int(arr.shape[0])
                    s_total += n
                    t_total += n
                    totals["global"] += n
                    per_cond[task][cond_name] = per_cond[task].get(cond_name, 0) + n

            # 2) check coerenza dati vs labels
            for key, X in subj_dict.items():
                if key.endswith('_labels'):
                    continue
                labels_key = f"{key}_labels"
                if labels_key not in subj_dict:
                    problems.append(f"[{task} | subj {subj_id}] manca {labels_key}")
                    continue
                y = subj_dict[labels_key]
                if getattr(X, "shape", (None,))[0] != len(y):
                    problems.append(
                        f"[{task} | subj {subj_id} | {key}] "
                        f"X.shape[0]={getattr(X,'shape',(None,))[0]} vs len(y)={len(y)}"
                    )

            per_subj[task][subj_id] = s_total

        totals["tasks"][task] = t_total

    if verbose:
        print("\n====== Riepilogo trials ======")
        print(f"Totale globale trial: {totals['global']}")
        for task, n in totals["tasks"].items():
            print(f"  - {task}: {n}")
        print("\nPer task e soggetto:")
        for task, d in per_subj.items():
            counts = ", ".join([f"s{subj}: {n}" for subj, n in sorted(d.items(), key=lambda x: int(x[0]))])
            print(f"  {task}: {counts if counts else '—'}")
        print("\nPer task e condizione (aggregata su soggetti):")
        for task, d in per_cond.items():
            counts = ", ".join([f"{cond}: {n}" for cond, n in d.items()])
            print(f"  {task}: {counts if counts else '—'}")
        if problems:
            print("\n⚠️  Incoerenze trovate:")
            for msg in problems[:10]:
                print("  -", msg)
            if len(problems) > 10:
                print(f"  ... (+{len(problems)-10} altre)")

    return totals, per_subj, per_cond, problems


In [ ]:
'''
Questo:

conta i trial usando le labels (più robuste),

fa un check di coerenza con le epoche dati (stesso n_epochs),

ti stampa il totale globale, il totale per task, il dettaglio per soggetto e per condizione,

riporta eventuali incoerenze (chiavi mancanti o mismatch di lunghezza).

'''

totals, per_subj, per_cond, problems = count_trials(new_EEG_data, verbose=True)

# one-liner "grezza" (solo totale), conteggiando direttamente i *_labels:
total_trials = sum(
    len(subj_dict[k]) 
    for task in new_EEG_data.values()
    for subj_dict in task
    for k in subj_dict.keys()
    if k.endswith('_labels')
)
print("Totale trial (one-liner):", total_trials)

In [ ]:
print(new_EEG_data['task_2'][2].keys())

In [ ]:
print(new_EEG_data['task_1'][0].keys())

In [ ]:
print(new_EEG_data['task_1'][0]['rest_1'].shape)
print(new_EEG_data['task_1'][0]['rest_1_labels'])
print(new_EEG_data['task_1'][0]['right_fist_1_labels'])
print(new_EEG_data['task_1'][0]['left_fist_1_labels'])

In [ ]:
'''

NEW UPDATED VERSION FOR DATASET MOTOR TASK - TIME x FREQUENCY UFFICIALE

Ecco la versione rivista della tua funzione, con in più il parametro data_task che seleziona il sotto-dizionario giusto in base al task
(es. "task_1", "task_2", …) e poi riapplica esattamente la stessa logica di STFT sui soli soggetti di quel task.

'''

import numpy as np
from scipy.signal import stft


def create_spectograms_from_stft(
    new_EEG_data: dict,
    data_task: str,
    window_size: int = 160,
    step_size: int = 80,
    fs: int = 160
):
    """
    new_EEG_data: { task: [ {cond_subj: data, cond_subj_labels: labels,...}, ... ], ... }
    data_task:     "task_1" / "task_2" / "task_3" / "task_4"
    """

    # --- 1) Filtra sul task richiesto ---
    if data_task not in new_EEG_data:
        raise KeyError(f"Task '{data_task}' non trovato in new_EEG_data")
    
    data = new_EEG_data[data_task]   # ora 'data' è la lista di dict (un elemento per soggetto)
    
    # --- 2) stesso codice che avevi prima ---
    condition_results = {}
    
    for idx, data_dict in enumerate(data):
        print(f"\n\n\t\t\t\t\t\033[1mProcessing Subject {idx + 1}\033[0m:\n")

        for condition, value in data_dict.items():
            
            # Calcolo condition_key senza '_labels'
            if condition.endswith('_labels'):
                condition_key = condition[:-7]
            else:
                condition_key = condition
            
            # inizializzo sempre la entry
            if condition_key not in condition_results:

                condition_results[condition_key] = {'spectrograms': [], 'labels': []}
            
            # ——————————————————————————
            # [PATCHED] ramo label
            if condition.endswith('_labels'):
                
                # Converto in lista di interi
                #labels = np.asarray(value, dtype=int).tolist()
                labels = np.asarray(value, dtype=int)
                
                print(f"\nFound labels for condition \033[1m{condition}\033[0m, length: \033[1m{len(labels)}\033[0m\n")
                    
                # Uso condition_key (non labels_key!)
                condition_results[condition_key]['labels'].extend(labels)
                continue   
            
            '''
            Cosa cambia in questo codice
            STFT viene eseguita solo su condition che non finiscono con _labels e che sono array 3‑D.

            Estensione delle etichette (.extend(num_labels)) viene spostata in un blocco if condition.endswith('_labels'),
            dove value è già il np.ndarray mono‑dimensionale delle labels.

            In questo modo:

            Non tenterai più di indicizzare value[trial, channel, …] su un array 1‑D
            Stamperai e accoderai correttamente le etichette nell’array condition_results[...]['labels']
            Con queste due modifiche minime tutto tornerà a girare senza ulteriori errori.
            '''
            
            # 1) processa solo i dati (3‑D) non le etichette 1‑D
            if isinstance(value, np.ndarray) and value.ndim == 3:
    
                print(f"\nApplicazione STFT per '\033[1m{condition_key}\033[0m'\n")
                print(f"\033[1mShape\033[0m dei dati per \033[1m{condition_key}\033[0m: {value.shape}")
                
                '''
                # Esegui STFT per ogni canale e trial          
                
                #La funzione stft di SciPy restituisce uno spettrogramma con la forma (n_frequencies, n_time_steps),
                #quindi:38 corrisponde al numero di bin di frequenza calcolati a partire dalla finestra FFT di 75 punti.
                
                #Infatti, per una finestra di lunghezza 75, il numero di bin è calcolato (in questo caso, dato che 75 è dispari) come (75 + 1) // 2, ovvero 38.
                #Il numero di passi temporali (n_time_steps) viene invece calcolato in base alla lunghezza del segnale e ai parametri di finestra e overlap.
                
                #Nel tuo caso, estraendo 250 punti (da 50 a 300) con nperseg=75 e noverlap=25 (cioè con un salto di 50 campioni per finestra), 
                #il numero di segmenti dovrebbe essere:
                #1 + [250−75/75-25] = 1 + [175/50]  = 1 + 3 = 4

                #Pertanto, il risultato di stft (cioè Zxx) ha forma (38, 4) e non (4, 38). 
                #Se ti aspetti una struttura con l'asse del tempo come terza dimensione (ad esempio, (trial, canali, tempo, frequenze))
                #potresti dover trasporre l'array in modo da scambiare le dimensioni frequenza e tempo.
                
                
                1) crearmi una lista per salvarmi poi tutti gli spettrogrammi di tutti i trials
                2) per il trials corrente mi salvo lo spettrogramma e lo appendo poi ad 1)
                3) converto la lista di tutti gli spettrogrammi di tutti i trials della condizione corrente in array numpy 
                4) carico poi l'array numpy dentro il dizionario della condizione corrente dentro la chiave 'spectrograms'
                '''
                
                all_trial_spectrograms = []
                
                for trial in range(value.shape[0]):
                    single_trial_spectrograms = []
                    
                    for channel in range(value.shape[1]):
                        single_channel_data = value[trial, channel, :640]
                        
                        #print(f"Segment length: {len(single_channel_data)}")
                        
                        '''
                        TIENI A MENTE CHE DUE PARAMETRI GESTISCONO IL PADDING QUI AUTOMATICAMENTE, CHE SONO:
                        
                        boundary='zeros', 
                        padded=True,
                        
                        PER QUESTO MOTIVO LE FINESTRE SARANNO 11 E NON 9.
                        
                        IN QUESTO MODO, SI MIGLIORA LA RISOLUZIONE IN FREQUENZA ALL'INIZIO E FINE DEL SEGNALE (DI 250 PUNTI!)
                        ANDANDO IN QUESTO MODO A RIDURRE GLI EFFETTI DI BORDO
                        '''
                        
                        f, t, Zxx = stft(
                            single_channel_data,
                            fs=fs,
                            window='hann',
                            nperseg=window_size,
                            noverlap=window_size - step_size
                        )
                        single_trial_spectrograms.append(Zxx) # Aggiungi lo spettrogramma calcolato per il canale corrente
                        
                    all_trial_spectrograms.append(np.array(single_trial_spectrograms)) # Forma: (numero_canali, frequenze, tempo)

                condition_results[condition_key]['spectrograms'].extend(all_trial_spectrograms)
                
                
                #if labels_key in data_dict:
                    #num_labels = data_dict[labels_key] #QUI IN REALTA' SAREBBE GIA' ARRAY NUMPY MA PER IL CODICE CONVIENE RIPORTARLO INT COME LISTA!
                    #'''UNICA MODIFICA IMPORTANTE DA FARE --> RIPORTARE A VALORE DI INT COME LISTA DELLE LABELS (IMPORTANTE PER DOPO!)'''
                    #num_labels = np.asarray(value).tolist()   # trasforma in lista di int
                    #if condition_key in condition_results:
                    #    print(f"\nFound labels for condition \033[1m{labels_key}\033[0m, length: \033[1m{len(num_labels)}\033[0m\n")
                    #    condition_results[condition_key]['labels'].extend(num_labels)
                
            
            else:
                print(f"\033[1m{condition_key}\033[0m è np.ndarray di labels, salto STFT")
                        
    # --- 3) prepara l'output concatenato ---
    
    # Preparazione risultati concatenati
    all_subjects_condition_results = {}

    for condition_key, results in condition_results.items():
        if "_labels" in condition_key or len(results['spectrograms']) == 0:  # Verifica che ci siano spettrogrammi
            print(f"\nSkipping \033[1m{condition_key}\033[0m as it is a label or has no valid data.")
            continue

        wave_condition_name = f'spectrograms_{condition_key}'
        print(f"\n\nResults for \t\t\t\t\t\033[1m{wave_condition_name}\033[0m:\n\n")

        # Converti in un array NumPy 3D finale
        spectrograms_4d = np.array(results['spectrograms'])  # Forma: (numero_di_trial, numero_canali, frequenze, tempo)
        labels = results['labels']
        
        all_subjects_condition_results[wave_condition_name] = {
            'spectrograms_4d': spectrograms_4d,
            'labels': labels
        }

        print(f"spectrograms_4d shape: {spectrograms_4d.shape}")
    
    '''
    Quindi alla fine si avrà una shape del tipo (44, 61, 38, 7) dove indica che avrai
    uno spettrogramma per ciascun trial, per ciascun canale, 
    che contiene informazioni sulle bande di frequenza e sui punti temporali. 
    
    Questa struttura è perfetta per essere utilizzata come input per una rete neurale convoluzionale (CNN),
    poiché la rete può apprendere dalle relazioni spaziali e temporali nei dati.
    '''
    
    return all_subjects_condition_results

In [ ]:
# Richiamo della funzione per processare tutti i soggetti e ottenere i risultati per ciascuna condizione

'''

def create_spectograms_from_stft(
    new_EEG_data: dict,
    data_task: str,
    window_size: int = 160,
    step_size: int = 80,
    fs: int = 160
):
'''

new_all_subjects_condition_spectrograms_task_1 = create_spectograms_from_stft(new_EEG_data, 
                                                                             'task_1',
                                                                             window_size  = 160,
                                                                             step_size = 80,
                                                                             fs=160)


In [ ]:
print('finito')

In [ ]:
new_all_subjects_condition_spectrograms_task_1.keys()

In [ ]:
print(new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100']['spectrograms_4d'].shape)
print()
print(len(new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100']['labels']))
print()
print(new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100']['labels'])

In [ ]:
import pickle

base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)


In [ ]:
# Salvare l'intero dizionario annidato con pickle
with open(f'{base_path}new_all_subjects_condition_spectrograms_task_1.pkl', 'wb') as f:
    pickle.dump(new_all_subjects_condition_spectrograms_task_1, f)
    print(f"✅ Dati salvati in: {base_path}")

In [ ]:
new_all_subjects_condition_spectrograms_task_1.keys()

In [ ]:
new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100'].keys()

In [ ]:
print(new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100']['spectrograms_4d'].shape)
print(len(new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100']['labels']))

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}new_all_subjects_condition_spectrograms_task_1.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data.keys()

In [ ]:
type(data['spectrograms_rest_100'])

In [ ]:
data['spectrograms_rest_100'].keys()

In [ ]:
data['spectrograms_rest_100']['spectrograms_4d'].shape

#### **Concatenazione** All **SINGLE** Subject **Spectrograms** per **Tutte le Condizioni Sperimentali**

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la isualizzazione dei dati
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path}new_all_subjects_condition_spectrograms_task_1.pkl', 'rb') as f:
    new_all_subjects_condition_spectrograms_task_1 = pickle.load(f)

In [ ]:
#new_all_subjects_condition_spectrograms_task_1.keys()

#new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100'].keys()

new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100']['spectrograms_4d'].shape

In [ ]:
import numpy as np

# Definizione degli indici dei canali desiderati

'''TOLGO PER SPETTROGRAMMI FREQUENZE x ELETTRODI'''
#selected_channels = [12, 30, 48]  # Indici per Fz, Cz, Pz

# Creazione di un dizionario dei dati di tutti i terapisti singolarmente, per salvare gli spettrogrammi calcolati
new_single_th_all_extracted_spectrograms_2D_task_1 = {}

# Iterazione su tutte le condizioni sperimentali di tutti i soggetti 

for condition, data in new_all_subjects_condition_spectrograms_task_1.items():
    
    # Estrarre le etichette per la condizione corrente
    labels = data['labels']
    
    #  Estrarre gli spettrogrammi per la condizione corrente
    '''TOLGO PER SPETTROGRAMMI FREQUENZE x ELETTRODI'''
    
    spectrograms = data['spectrograms_4d']
    
    #spectrograms = data['spectrograms_3d']
    
    '''TOLGO PER SPETTROGRAMMI FREQUENZE x ELETTRODI'''
    #Selezione dei canali di interesse (Fz, Cz e Pz)
    #spectrograms = spectrograms[:, selected_channels, :, :]
    
    new_single_th_all_extracted_spectrograms_2D_task_1[condition] = {
        'spectrograms': spectrograms,
        #'delta': delta_reconstruction,
        #'coeff_fifth_detail_theta': coeff_fifth_detail_theta, 
        'labels': labels
    }
    
# Verifica delle dimensioni del risultato estratto per una condizione specifica di ogni soggetto
print(f"\t\t\033[1mEstrazione Spettrogrammi di tutti i terapisti per tutte le condizioni sperimentali")
print(f"\n\n\t\t\t\t(i.e., \033[1mnew_single_th_all_extracted_spectrograms_2D_task_1\033[0m):\n")

for condition, extracted_data in new_single_th_all_extracted_spectrograms_2D_task_1.items():
    
    print(f"Condizione: \033[1m{condition}\033[0m, "
          f"Spectrograms shape: \033[1m{extracted_data['spectrograms'].shape}\033[0m, "
          f"Labels: \033[1m{len(extracted_data['labels'])}\033[0m")


In [ ]:
'''Salvataggio di new_single_th_all_extracted_spectrograms_2D_task_1'''

base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/'


# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
#new_single_th_all_extracted_spectrograms_2D_task_1.keys()

#new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100'].keys()

new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100']['spectrograms'].shape

In [ ]:
import pickle

# Salvare l'intero dizionario annidato con pickle
with open(f'{base_path}new_single_th_all_extracted_spectrograms_2D_task_1.pkl', 'wb') as f:
    pickle.dump(new_single_th_all_extracted_spectrograms_2D_task_1, f)

In [ ]:
#print(new_single_th_all_extracted_spectrograms_2D_task_1.keys())
#print()
print(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100'].keys())
print()
print(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100']['spectrograms'].shape)
print()
print(len(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100']['labels']))

In [ ]:
'''
Questa struttura consente di avere i dati degli spettrogrammi correttamente organizzati e concatenati per ogni soggetto,
mantenendo la corrispondenza tra dati e etichette per ogni condizione sperimentale

Da           "single_th_all_extracted_reconstructions"       a           "subject_level_concatenations_th"

Da           "new_single_th_all_extracted_reconstructions"       a       "new_subject_level_concatenations_th"

Da           "new_single_th_all_extracted_spectrograms_2D_task_1"       a       "new_subject_level_concatenations_spectrograms_th_2D_task_1"
'''


# Dizionario per contenere i dati concatenati per ogni soggetto
new_subject_level_concatenations_spectrograms_th_2D_task_1 = {}

# Variabile per tracciare il soggetto precedente
previous_subject_suffix = None

print(f"\t\t\033[1mConcatenazione spettrogrammi 2D per OGNI soggetto di TUTTE le condizioni sperimentali INSIEME\033[0m")

# Iterazione su tutte le chiavi di new_single_th_all_extracted_spectrograms_2D_task_1
for condition, data in new_single_th_all_extracted_spectrograms_2D_task_1.items():
    
    # Estrazione del suffisso numerico del soggetto
    subject_suffix = condition.split('_')[-1]  # Prende solo la parte numerica
    
    # Creazione del nome della chiave del soggetto specifico
    subj_name = f'th_{subject_suffix}'
    
    # Se stiamo per passare a un nuovo soggetto, concatenare e stampare i risultati del soggetto precedente
    if previous_subject_suffix is not None and subject_suffix != previous_subject_suffix:
        
        print(f"\n\n\t\t\t\t\033[1mConcatenazioni per il soggetto corrente ('th_{previous_subject_suffix}'):\033[0m\n")
        prev_subject_name = f'th_{previous_subject_suffix}'
        levels = new_subject_level_concatenations_spectrograms_th_2D_task_1[prev_subject_name]

        # Concatenazione dei dati prima della stampa
        spectrograms_concat = np.concatenate(levels['spectrograms'], axis=0)
        labels_concat = np.concatenate(levels['labels'], axis=0)

        print(f"Soggetto: \033[1m{prev_subject_name}\033[0m, "
              f"Spettrogrammi shape: \033[1m{spectrograms_concat.shape}\033[0m, "
              f"Labels length: \033[1m{len(labels_concat)}\033[0m")
        
        # Salvataggio dei dati concatenati nel dizionario
        new_subject_level_concatenations_spectrograms_th_2D_task_1[prev_subject_name]['spectrograms'] = spectrograms_concat
        new_subject_level_concatenations_spectrograms_th_2D_task_1[prev_subject_name]['labels'] = labels_concat
    
    # Inizializzare il dizionario per il soggetto specifico se non esiste già
    if subj_name not in new_subject_level_concatenations_spectrograms_th_2D_task_1:
        new_subject_level_concatenations_spectrograms_th_2D_task_1[subj_name] = {
            'spectrograms': [],
            'labels': []
        }
    
    # Stampiamo le informazioni per ogni condizione
    print(f"\n\n\nSoggetto: \033[1m{subj_name}\033[0m, Condizione: \033[1m{condition}\033[0m")
    print(f"  - Spettrogrammi shape: \033[1m{data['spectrograms'].shape}\033[0m")
    print(f"  - Labels shape: \033[1m{len(data['labels'])}\033[0m")
    print(f"  - Valore unico delle etichette: \033[1m{np.unique(data['labels'])}\033[0m")
    
    # Concatenazione dei dati
    new_subject_level_concatenations_spectrograms_th_2D_task_1[subj_name]['spectrograms'].append(data['spectrograms'])
    new_subject_level_concatenations_spectrograms_th_2D_task_1[subj_name]['labels'].append(data['labels'])
    
    # Aggiorna il soggetto precedente
    previous_subject_suffix = subject_suffix

# Dopo aver iterato su tutte le condizioni, concatenare e stampare le informazioni dell'ultimo soggetto
if previous_subject_suffix is not None:
    
    last_subject_name = f'th_{previous_subject_suffix}'
    levels = new_subject_level_concatenations_spectrograms_th_2D_task_1[last_subject_name]
    
    print(f"\n\n\t\t\t\t\033[1mConcatenazioni per il soggetto corrente ('{last_subject_name}'):\033[0m\n")
    
    # Concatenazione dei dati dell'ultimo soggetto
    spectrograms_concat = np.concatenate(levels['spectrograms'], axis=0)
    labels_concat = np.concatenate(levels['labels'], axis=0)
    
    # Salvataggio dei dati concatenati dell'ultimo soggetto nel dizionario
    new_subject_level_concatenations_spectrograms_th_2D_task_1[last_subject_name] = {
        'spectrograms': spectrograms_concat,
        'labels': labels_concat
    }
    
    print(f"Soggetto: \033[1m{last_subject_name}\033[0m, "
          f"Spettrogrammi shape: \033[1m{spectrograms_concat.shape}\033[0m, "
          f"Labels length: \033[1m{len(labels_concat)}\033[0m\n\n")


In [ ]:
'''CHIAVI DI TUTTO IL DIZIONARIO '''
#subject_level_concatenations_th.keys()


new_subject_level_concatenations_spectrograms_th_2D_task_1.keys()


#SOGGETTO 1

#CHIAVI
#subject_level_concatenations_th['th_1'].keys()

#DATI
#subject_level_concatenations_th['th_1']['theta'].shape

#new_subject_level_concatenations_th['th_1']['theta'].shape

#subject_level_concatenations_th['th_1']['delta'].shape

#LABELS
#subject_level_concatenations_th['th_1']['labels'].shape)
#type(subject_level_concatenations_th['th_1']['labels'])

# Check della concantenazione delle labels TH_1

#subject_level_concatenations_th['th_1']['labels'][:42]
#subject_level_concatenations_th['th_1']['labels'][41:81]
#subject_level_concatenations_th['th_1']['labels'][81:120]
#subject_level_concatenations_th['th_1']['labels'][121:164]

In [ ]:
#subject_level_concatenations_th['th_15']['labels'].shape

In [ ]:
#subject_level_concatenations_th['th_1'].keys()

new_subject_level_concatenations_spectrograms_th_2D_task_1['th_1'].keys()


In [ ]:
#unique_values, counts = np.unique(subject_level_concatenations_th['th_1']['labels'], return_counts=True)

#unique_values, counts = np.unique(new_subject_level_concatenations_th['th_1']['labels'], return_counts=True)

In [ ]:
#unique_values
#counts
#counts[0]

In [ ]:
#new_subject_level_concatenations_spectrograms_th_2D_task_1['th_1'].keys()
#dict_keys(['spectrograms', 'labels'])

new_subject_level_concatenations_spectrograms_th_2D_task_1['th_1']['spectrograms'].shape

In [ ]:
!pwd

In [ ]:
'''Salvataggio di new_single_th_all_extracted_spectrograms_2D_task_1'''

base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
import pickle

# Salvare l'intero dizionario annidato con pickle
with open(f'{base_path}new_subject_level_concatenations_spectrograms_th_2D_task_1.pkl', 'wb') as f:
    pickle.dump(new_subject_level_concatenations_spectrograms_th_2D_task_1, f)

#### **Concatenazione** All **SINGLE** Subject **Spectrograms (TH)** per **Coppie di Condizioni Sperimentali**

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la isualizzazione dei dati
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pickle

In [ ]:
!pwd

In [ ]:
#cd ..

In [ ]:
#cd New_Plots_Sliding_Estimator_MNE_1_45_NF/

In [ ]:
import pickle
import numpy as np

base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/'
    
# Caricare l'intero dizionario annidato con pickle
with open(f'{base_path}new_single_th_all_extracted_spectrograms_2D_task_1.pkl', 'rb') as f:
    new_single_th_all_extracted_spectrograms_2D_task_1 = pickle.load(f)

In [ ]:
#new_single_th_all_extracted_spectrograms_2D_task_1.keys()

In [ ]:
new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100'].keys()

##### Spiegazione STEP - Concatenazione All Single Subject Spectrograms (TH) across Couples of Experimental Conditions

###### **Concatenazione All Single Subject Data (TH) per Coppie di Condizioni Sperimentali**

<br>

In questo caso, voglio **trasformare le variabili che contengono i miei dati e labels delle 4 condizioni sperimentali assieme**, in modo **da creare sottoinsiemi per ogni possibile coppia di condizioni sperimentali**. 

Dunque quello che si dovrà fare **per ogni coppia**:

1) **Filtrare i dati e le etichette**:

- **Estrarre** i dati associati **SOLO alle DUE condizioni in esame** (ad esempio 0 e 1, oppure 1 e 2).
- **Creare un nuovo array di etichette** e **un array di dati** ***corrispondenti*** **SOLO a quelle DUE condizioni**.

2) **Ripetere questa operazione per OGNI combinazione**:

- **Assicurarsi di considerare tutte le coppie** di condizioni sperimentali **SENZA ripetizioni** (ad esempio, non serve processare sia 0 vs 1 che 1 vs 0, perché sono equivalenti!).

3) **Creare quindi una struttura organizzata**, che contenga i dati e le etichette separatamente per ogni coppia, per ciascun livello di ricostruzione (theta, delta, theta_strict).

<br>

Quindi l'obiettivo è di modificare la procedura in modo da **concatenare SOLO le coppie di condizioni sperimentali per OGNI soggetto**. 

Per fare questo, possiamo utilizzare un **approccio combinatorio** per generare **tutte le possibili coppie di condizioni**.


Per quanto riguarda il calcolo combinatorio, possiamo usare il concetto di combinazioni per selezionare le coppie. 
Se hai 4 condizioni, il numero di combinazioni di coppie di condizioni (senza ripetizioni) si calcola come:

$$
\binom{4}{2} = \frac{4!}{2!(4-2)!} = 6
$$


Quindi, con 4 condizioni sperimentali, le combinazioni di coppie di condizioni saranno 6, e queste coppie sono:

(condizione 1, condizione 2) = 'baseline vs th_resp'
(condizione 1, condizione 3) = 'baseline vs pt_resp'
(condizione 1, condizione 4) = 'baseline vs shared_resp'
(condizione 2, condizione 3) = 'th_resp vs pt_resp'
(condizione 2, condizione 4) = 'th_resp vs shared_resp'
(condizione 3, condizione 4) = 'pt_resp vs shared_resp' 


<br>

Piano di modifiche al codice:

1) **Creazione delle coppie di condizioni**: Per ogni soggetto, genereremo tutte le coppie possibili di condizioni sperimentali. Utilizzeremo itertools.combinations per ottenere queste coppie.

2) **Concatenazione per ciascuna coppia**: Per ogni coppia di condizioni, concatenare i dati e le etichette delle due condizioni selezionate.

3) **Salvataggio delle concatenazioni**: Creeremo un dizionario per ciascuna coppia di condizioni per ogni soggetto.


**N.B.**

Nel codice che sto creando...

Ho fatto in modo che ci sia il modo di capire, **per ogni coppia di condizione sperimentale di dati e labels**, a quali condizione sperimentali di dati e labels ci si riferisca, mi spiego:

Sviluppo un **modo "standardizzato" per il quale per OGNI coppia di condizione sperimentale, la variabile che conterrà i dati e labels associate, sia chiamata in un certo modo**...

E nello specifico **con il nome delle DUE condizioni sperimentali che conterranno quei dati e relative labels**...

Del tipo: 

- se è baseline vs th_resp, la variabile sarà "baseline_vs_th_resp"
- se è baseline vs pt_resp, la variabile sarà "baseline_vs_pt_resp"
- se è baseline vs shared_resp, la variabile sarà "baseline_vs_shared_resp"


Senza contare che, questo **processo di standardizzazione del nome delle variabili**, deve esser fatto **per ogni livello di ricostruzione dei dati (sia 4° e 5° livello a partire dai coefficienti di approssimazione, che dal 5 ° livello dei coefficienti di dettaglio)**..

Questo significherebbe che, **per ogni soggetto**, dovrei (o potrei insomma) creare **un dizionario**, che 

- Contenga delle **sotto-chiavi di secondo ordine** (che son theta, delta e theta_strict) e dentro ognuna di queste
- Ci siano contenute **le relative 6 sotto-sotto-chiavi di terzo ordine**, relative ai dati e alle labels associate (anch'esse concatenate ovviamente) delle diverse combinazioni di condizioni sperimentali...


Ossia, vogliamo creare un dizionario strutturato che rappresenti ogni combinazione di coppie di condizioni sperimentali per ciascun soggetto, separando i dati e le etichette per i vari livelli di ricostruzione (theta, delta, theta_strict). 

Ogni combinazione di condizioni sperimentali sarà memorizzata in una chiave strutturata, come ad esempio "baseline_vs_th_resp", e all'interno di ciascuna chiave avremo le informazioni per i vari livelli di ricostruzione.


<br>


#### Esempio di Struttura proposta per il dizionario:


- Soggetto: ad esempio "th_1".
- Livelli di ricostruzione: theta, delta, theta_strict.
- Combinazioni di condizioni sperimentali: per ciascun livello di ricostruzione, avremo sotto-chiavi per ogni coppia di condizioni.

        new_subject_level_concatenations = {
            'th_1': {
                'theta': {
                    'baseline_vs_th_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    'baseline_vs_pt_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    'baseline_vs_shared_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    # Altre combinazioni...
                },
                'delta': {
                    'baseline_vs_th_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    'baseline_vs_pt_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    'baseline_vs_shared_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    # Altre combinazioni...
                },
                'theta_strict': {
                    'baseline_vs_th_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    'baseline_vs_pt_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    'baseline_vs_shared_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    # Altre combinazioni...
                }
            }
        }

<br>

**SOTTO-STEP 1: IDENTIFICAZIONE DELLE STRINGA NUMERICHE PER LA CREAZIONE DELLE VARIABILI - FUTURE CHIAVI - CHE CONTENGONO LE COPPIE DI DATI E LABELS DELLE RELATIVE COPPIE DI CONDIZIONI SPERIMENTALI CONSIDERATE NEL CICLO CORRENTE DEL LOOP**

Aspetta, ci sono altre cose che dovrebbero esser integrate nel codice...

cominciamo dalla prima, che è relativa all' "inferire" diciamo il nome da fornire alla variabile che conterrà i dati e labels delle due condizioni sperimentali correnti... e deve esser fatto, a partire dal nome delle chiavi di 

"new_single_th_all_extracted_reconstructions"...

ora, le sue chiavi son queste

dict_keys(['wave_baseline_1', 'wave_th_resp_1', 'wave_pt_resp_1', 'wave_shared_resp_1', 'wave_baseline_2', 'wave_th_resp_2', 'wave_pt_resp_2', 'wave_shared_resp_2', 'wave_baseline_3', 'wave_th_resp_3', 'wave_pt_resp_3', 'wave_shared_resp_3', 'wave_baseline_4', 'wave_th_resp_4', 'wave_pt_resp_4', 'wave_shared_resp_4', 'wave_baseline_5', 'wave_th_resp_5', 'wave_pt_resp_5', 'wave_shared_resp_5', 'wave_baseline_6', 'wave_th_resp_6', 'wave_pt_resp_6', 'wave_shared_resp_6', 'wave_baseline_7', 'wave_th_resp_7', 'wave_pt_resp_7', 'wave_shared_resp_7', 'wave_baseline_8', 'wave_th_resp_8', 'wave_pt_resp_8', 'wave_shared_resp_8', 'wave_baseline_9', 'wave_th_resp_9', 'wave_pt_resp_9', 'wave_shared_resp_9', 'wave_baseline_10', 'wave_th_resp_10', 'wave_pt_resp_10', 'wave_shared_resp_10', 'wave_baseline_11', 'wave_th_resp_11', 'wave_pt_resp_11', 'wave_shared_resp_11', 'wave_baseline_12', 'wave_th_resp_12', 'wave_pt_resp_12', 'wave_shared_resp_12'])


ora, il modo in cui si identifica un certo soggetto (ossia la chiavi di "primo ordine" che indentificano il soggetto, ossia ad esempio 'th_1'), dipenderebbe dal numero stringa che c'è alla fine di ogni nome stringa di ogni chiave dentro "new_single_th_all_extracted_reconstructions", mi spiego:

il codice dovrebbe capire che, ad esempio, solo l'ultima carattere stringa di queste prime 4 chiavi:

'wave_baseline_1', 'wave_th_resp_1', 'wave_pt_resp_1', 'wave_shared_resp_1', 

si riferisca al soggetto 1, ossia al futuro "th_1"...


per il soggetto 2, sarebbero: 

'wave_th_resp_2', 'wave_pt_resp_2', 'wave_shared_resp_2', e così via per gli altri soggetti..


Facendo attenzione al fatto che, dopo il soggetto 9, ovviamente, si passerà alle decine (a livello numerico), per cui saranno gli ultime due caratteri da considerare per l'identificazione delle sue rispettive chiavi, che in quel caso saranno per il soggetto 10, ossia il futuro th_10

'wave_baseline_10', 'wave_th_resp_10', 'wave_pt_resp_10', 'wave_shared_resp_10',

Inoltre, poi, per creare le variabili associate alle coppie di dati e labels delle relative condizioni sperimentali, dovrà il codice far in modo che il nome di quella variabile dipenda dalle stringhe sempre che si riferiscono al nome delle due chiavi di "new_single_th_all_extracted_reconstructions" da cui preleva i dati e le labels... 

ad esempio, se deve creare la variabile dei dati 'baseline_vs_th_resp" del primo soggetto, le chiavi rispettive da cui deve andare a prelevare le stringhe saranno appunto "'wave_baseline_1'" e "'wave_th_resp_1'"....

diciamo che si potrebbe creare una lista di stringhe che sarà del tipo:

experimental_conditions = ['baseline', 'th_resp', 'pt_resp', 'shared_resp'], per vedere se una di queste sia dentro la rispettiva chiave di  new_single_th_all_extracted_reconstructions su cui sta iterando, che nel nostro caso di esempio sarebbero sempre

'wave_baseline_1' (e da cui vede che c'è la stinga 'baseline')
'wave_th_resp_1' (e da cui vede che c'è la stinga 'th_resp')

è chiaro?


<br>

In sostanza, desideri un modo per inferire dinamicamente il nome del soggetto e delle variabili che conterranno i dati e le etichette per ogni combinazione di condizioni, a partire dal nome delle chiavi di new_single_th_all_extracted_reconstructions. Ecco un piano per raggiungere questo obiettivo:

Passaggi:

- Identificare il soggetto:

Le chiavi di new_single_th_all_extracted_reconstructions finiscono con un numero che identifica il soggetto (es. wave_baseline_1, wave_th_resp_1, wave_pt_resp_1).
Questo numero può essere estratto dalle ultime cifre della chiave (ad esempio, 1 da wave_baseline_1).
La variabile associata a ciascun soggetto sarà chiamata th_X, dove X è il numero del soggetto.

- Generare dinamicamente il nome delle variabili:

A partire dai nomi delle chiavi, possiamo inferire quale combinazione di condizioni sperimentali stiamo trattando (es. baseline_vs_th_resp).
Creeremo una lista delle condizioni (experimental_conditions = ['baseline', 'th_resp', 'pt_resp', 'shared_resp']), quindi esamineremo le chiavi per determinare quali condizioni sono presenti.

- Creare il dizionario:

A ciascun soggetto verrà associato un dizionario, e per ogni combinazione di condizioni sperimentali, assoceremo i dati e le etichette in una chiave che seguirà la convenzione del tipo baseline_vs_th_resp.


<br>

Spiegazione:

1) Estrazione delle condizioni e soggetto:

La chiave, ad esempio wave_baseline_1, viene separata in parti. La parte baseline viene estratta per determinare la condizione, mentre 1 indica il numero del soggetto.
Queste informazioni vengono utilizzate per creare il nome del soggetto, th_1.

2) Combinazioni di condizioni:

La lista experimental_conditions contiene le 4 condizioni: baseline, th_resp, pt_resp, shared_resp.
Per ogni soggetto, controlliamo quali condizioni sono disponibili nella chiave (ad esempio, se il soggetto 1 ha wave_baseline_1 e wave_th_resp_1, creiamo la combinazione baseline_vs_th_resp).

3) Creazione dinamica delle variabili:

Utilizziamo la combinazione di condizioni per generare la chiave della variabile (baseline_vs_th_resp).
I dati e le etichette per la combinazione di condizioni vengono estratti e concatenati.


4) Risultato finale:

La struttura finale del dizionario new_subject_level_concatenations avrà una chiave per ciascun soggetto (th_1, th_2, ecc.), e sotto ogni soggetto ci saranno le combinazioni di condizioni per ciascun livello di ricostruzione (theta, delta, theta_strict), con i dati e le etichette corrispondenti.

<br>

Esempio di output:

    {
        'th_1': {
            'theta': {
                'baseline_vs_th_resp': {'data': np.array([...]), 'labels': np.array([...])},
                'baseline_vs_pt_resp': {'data': np.array([...]), 'labels': np.array([...])},
                'baseline_vs_shared_resp': {'data': np.array([...]), 'labels': np.array([...])},
                'th_resp_vs_pt_resp': {'data': np.array([...]), 'labels': np.array([...])},
                # Altre combinazioni...
            },
            'delta': {
                # Stessa struttura per 'delta'
            },
            'theta_strict': {
                # Stessa struttura per 'theta_strict'
            }
        }
    }

 
Questa struttura ti permetterà di avere un'organizzazione chiara e dinamica dei dati per ogni soggetto e combinazione di condizioni sperimentali.
 
 
<br>

**SOTTO-STEP 2: IDENTIFICAZIONE DELLE STRINGA ALFABETICHE PER LA CREAZIONE DELLE VARIABILI - FUTURE CHIAVI - CHE CONTENGONO IL NOME DELLE DUE COPPIE DI CONDIZIONI SPERIMENTALI DI CUI VENGONO PRELEVATI I DATI E LABELS CONCATENATI E CHE VENGONON CONSIDERATE NEL CICLO CORRENTE DEL LOOP**


ok, ora c'è una ultima cosa che mi manca da dirti, che è relativa alla ri-assegnazione del codice numerico associato ad ogni coppia di condizioni sperimentali...

nel senso che, originariamente...

new_single_th_all_extracted_reconstructions ha queste labels, che se io le vedo avrò 

per baseline una serie di 0, per th_resp una serie di 1, per pt_resp, una serie di 2 e per shared_resp una serie di 3, in questo modo


print(f"Baseline in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_baseline_1']['labels'], return_counts = True)}\n")
print(f"Th_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_th_resp_1']['labels'], return_counts = True)}\n")
print(f"Pt_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_pt_resp_1']['labels'], return_counts = True)}\n")
print(f"Shared_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_shared_resp_1']['labels'], return_counts = True)}\n")


Baseline in TH_1:, (array(['0'], dtype='<U1'), array([51]))

Th_Resp in TH_1:, (array(['1'], dtype='<U1'), array([40]))

Pt_Resp in TH_1:, (array(['2'], dtype='<U1'), array([40]))

Shared_Resp in TH_1:, (array(['3'], dtype='<U1'), array([56]))

di conseguenza,  nel tuo codice, ho bisogno che 

le labels associate a 'cond_1' (quando crei la relativa variabile delle due condizioni sperimentali di cui vengono concatenati i dati e le labels) siano riconvertite sempre in '0', mentre 

le labels associate a 'cond_2' (quando crei la relativa variabile delle due condizioni sperimentali di cui vengono concatenati i dati e le labels) siano riconvertite sempre in '1'

questo aspetto è importante, per quando crei le variabili che conterranno le coppie di dati condizioni sperimentali...

ora, nel caso in cui tu stia considerando magari baseline vs le altre condizioni sperimentali, è chiaro che in quel caso specifico, ad esempio, baseline sarà sempre la stringa associata a 'cond_1' e quindi in quel caso sarà sempre 0, e quindi sarebbe apposto..

ma per gli altri casi di confronto di baseline rispetto alle altre 3 condizioni sperimentali, è chiaro che se sarà th_resp la condizione di confronto, allora in quel caso di quella specifica variabile, le labels non avranno bisogno di esser diciamo 'ri-convertite' perché già naturalmente diciamo baseline avrà le labels '0' e  e th_resp avrà già '1' (come è già originariamente codificato in 'new_subject_level_concatenations'!)

ma nel caso delle altre variabili da creare le labels andranno ovviamente riconvertite! Perché?
perché ovviamente nel caso sia la th_resp la 'cond_1', ad esempip, in quel caso, le sue labels (che son  originariamente codificate come '1' in 'new_subject_level_concatenations'!) diveteranno '0', mentre le altre condizioni di confronto, una alla volta, diventeranno '1' invece...


è chiara diciamo la logica di ri-assegnazione delle relative labels in sostanza?


<br>

La **logica di ri-assegnazione delle etichette (labels)** è chiara e posso integrarla nel codice. 

La logica che stai descrivendo consiste essenzialmente nel **garantire che, quando crei una variabile che contiene dati e etichette per due condizioni sperimentali, le etichette siano sempre "0" per la prima condizione (cond_1) e "1" per la seconda condizione (cond_2)**, 

                **INDIPENDENTEMENTE** da come sono originariamente codificate in **"new_single_th_all_extracted_reconstructions"!
                
                

Ecco come possiamo procedere:

- Identificazione delle condizioni: Come hai descritto, determineremo le due condizioni sperimentali da confrontare, ad esempio "baseline" contro "th_resp", "baseline" contro "pt_resp", ecc.

- Riconversione delle etichette:

 - Se la condizione di confronto è "baseline" (che è associata a 0), non dobbiamo modificare le etichette.
 - Se la condizione di confronto non è "baseline", dovremo riconvertire le etichette di quella condizione a 0 e le etichette della condizione di confronto a 1.

- Struttura del codice: Durante la creazione delle variabili, dovremo controllare quale condizione è la prima (cond_1) e quale è la seconda (cond_2), e applicare questa logica per ri-assegnare le etichette di conseguenza.


<br>

Inoltre


dentro 

"# Crea tutte le combinazioni uniche di condizioni sperimentali (es. 'baseline vs th_resp')
    condition_pairs = list(itertools.combinations(experimental_conditions, 2))
"

Effettivamente vengono create delle tuple di coppie di stringhe che corrispondono a quelle con cui dovrebbero esser create le sotto-sotto-chiavi delle coppie di condizioni sperimentali..

tuttavia, è giusto che iteri rispetto a 'condition_pairs', ma deve inserire un qualche controllo anche rispetto a questa cosa:

originariamente...

new_single_th_all_extracted_reconstructions ha queste labels, che se io le vedo avrò

per baseline una serie di 0, per th_resp una serie di 1, per pt_resp, una serie di 2 e per shared_resp una serie di 3, in questo modo

print(f"Baseline in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_baseline_1']['labels'], return_counts = True)}\n") print(f"Th_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_th_resp_1']['labels'], return_counts = True)}\n") print(f"Pt_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_pt_resp_1']['labels'], return_counts = True)}\n") print(f"Shared_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_shared_resp_1']['labels'], return_counts = True)}\n")

Baseline in TH_1:, (array(['0'], dtype='<U1'), array([51]))

Th_Resp in TH_1:, (array(['1'], dtype='<U1'), array([40]))

Pt_Resp in TH_1:, (array(['2'], dtype='<U1'), array([40]))

Shared_Resp in TH_1:, (array(['3'], dtype='<U1'), array([56]))

di conseguenza, nel tuo codice, ho bisogno che

le labels associate a 'cond_1' (quando crei la relativa variabile delle due condizioni sperimentali di cui vengono concatenati i dati e le labels) siano riconvertite sempre in '0', mentre

le labels associate a 'cond_2' (quando crei la relativa variabile delle due condizioni sperimentali di cui vengono concatenati i dati e le labels) siano riconvertite sempre in '1'

questo aspetto è importante, per quando crei le variabili che conterranno le coppie di dati condizioni sperimentali...

ora, nel caso in cui tu stia considerando magari baseline vs le altre condizioni sperimentali, che nel caso di "condition_pairs" dovrebbe corrispondere ai primi 3 elementi della sua lista no, perché sarebbero 

[('baseline', 'th_resp'), 
    ('baseline', 'pt_resp'), 
    ('baseline', 'shared_resp'), 

è chiaro che in quel caso specifico, ad esempio, baseline sarà sempre la stringa associata a 'cond_1' e quindi in quel caso sarà sempre 0, e quindi sarebbe apposto..

ma per gli altri casi di confronto di baseline rispetto alle altre 3 condizioni sperimentali, è chiaro che se sarà th_resp la condizione di confronto, allora in quel caso di quella specifica variabile, le labels non avranno bisogno di esser diciamo 'ri-convertite' perché già naturalmente diciamo baseline avrà le labels '0' e e th_resp avrà già '1' (come è già originariamente codificato in 'new_subject_level_concatenations'!)

ma nel caso delle altre variabili da creare, che fa riferimento a questi altri elementi di 
"condition_pairs" che è creato nel loop ossia

('th_resp', 'pt_resp'), 
('th_resp', 'shared_resp'), 
('pt_resp', 'shared_resp')]

le labels andranno ovviamente riconvertite! Perché? 

Perché ovviamente nel caso sia la th_resp la 'cond_1', ad esempio, in quel caso, le sue labels (che son originariamente codificate come '1' in 'new_subject_level_concatenations'!) diventeranno '0',

mentre le altre condizioni di confronto, una alla volta, diventeranno '1' invece...

è chiara diciamo la logica di ri-assegnazione delle relative labels in sostanza?dentro "# Crea tutte le combinazioni uniche di condizioni sperimentali (es. 'baseline vs th_resp')
    condition_pairs = list(itertools.combinations(experimental_conditions, 2))
    "

effettivamente vengono create delle tuple di coppie di stringhe che corrispondono a quelle con cui dovrebbero esser create le sotto-sotto-chiavi delle coppie di condizioni sperimentali..

tuttavia, è giusto che iteri rispetto a 'condition_pairs', ma deve inserire un qualche controllo anche rispetto a questa cosa:

originariamente...

new_single_th_all_extracted_reconstructions ha queste labels, che se io le vedo avrò

per baseline una serie di 0, per th_resp una serie di 1, per pt_resp, una serie di 2 e per shared_resp una serie di 3, in questo modo

print(f"Baseline in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_baseline_1']['labels'], return_counts = True)}\n") print(f"Th_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_th_resp_1']['labels'], return_counts = True)}\n") print(f"Pt_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_pt_resp_1']['labels'], return_counts = True)}\n") print(f"Shared_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_shared_resp_1']['labels'], return_counts = True)}\n")

Baseline in TH_1:, (array(['0'], dtype='<U1'), array([51]))

Th_Resp in TH_1:, (array(['1'], dtype='<U1'), array([40]))

Pt_Resp in TH_1:, (array(['2'], dtype='<U1'), array([40]))

Shared_Resp in TH_1:, (array(['3'], dtype='<U1'), array([56]))

di conseguenza, nel tuo codice, ho bisogno che

le labels associate a 'cond_1' (quando crei la relativa variabile delle due condizioni sperimentali di cui vengono concatenati i dati e le labels) siano riconvertite sempre in '0', mentre

le labels associate a 'cond_2' (quando crei la relativa variabile delle due condizioni sperimentali di cui vengono concatenati i dati e le labels) siano riconvertite sempre in '1'

questo aspetto è importante, per quando crei le variabili che conterranno le coppie di dati condizioni sperimentali...

ora, nel caso in cui tu stia considerando magari baseline vs le altre condizioni sperimentali, che nel caso di "condition_pairs" dovrebbe corrispondere ai primi 3 elementi della sua lista no, perché sarebbero 

[('baseline', 'th_resp'), 
    ('baseline', 'pt_resp'), 
    ('baseline', 'shared_resp'), 

è chiaro che in quel caso specifico, ad esempio, baseline sarà sempre la stringa associata a 'cond_1' e quindi in quel caso sarà sempre 0, e quindi sarebbe apposto..

ma per gli altri casi di confronto di baseline rispetto alle altre 3 condizioni sperimentali, è chiaro che se sarà th_resp la condizione di confronto, allora in quel caso di quella specifica variabile, le labels non avranno bisogno di esser diciamo 'ri-convertite' perché già naturalmente diciamo baseline avrà le labels '0' e e th_resp avrà già '1' (come è già originariamente codificato in 'new_subject_level_concatenations'!)

ma nel caso delle altre variabili da creare, che fa riferimento a questi altri elementi di 
"condition_pairs" che è creato nel loop ossia

('th_resp', 'pt_resp'), 
('th_resp', 'shared_resp'), 
('pt_resp', 'shared_resp')]

le labels andranno ovviamente riconvertite! Perché? 

Perché ovviamente nel caso sia la th_resp la 'cond_1', ad esempio, in quel caso, le sue labels (che son originariamente codificate come '1' in 'new_subject_level_concatenations'!) diventeranno '0',

mentre le altre condizioni di confronto, una alla volta, diventeranno '1' invece...

è chiara diciamo la logica di ri-assegnazione delle relative labels in sostanza?



##### Implementazione STEP - Concatenazione All Single Subject Spectrograms (TH) across Couples of Experimental Conditions

In [ ]:
new_single_th_all_extracted_spectrograms_2D_task_1.keys()

In [ ]:
new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100'].keys()

In [ ]:
type(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100'])

In [ ]:
print(f"Rest in TH_1:, {np.unique(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100']['labels'], return_counts = True)}\n")
print(f"Left_Fist in TH_1:, {np.unique(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_left_fist_100']['labels'], return_counts = True)}\n")
print(f"Right_Fist in TH_1:, {np.unique(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_right_fist_100']['labels'], return_counts = True)}\n")

In [ ]:
new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100'].keys()

In [ ]:
'''DETAILED VERSION WITH COMPLICATED PRINTS

    CORREZIONE UFFICIALE PER SPETTROGRAMS
'''


import itertools
import numpy as np

# Definizione delle condizioni sperimentali
#experimental_conditions = ['baseline', 'th_resp', 'pt_resp', 'shared_resp']

'''UNICA MODIFICA PER DATASET MOTOR TASK'''
experimental_conditions = ['rest', 'left_fist', 'right_fist']

# Variabile per tenere traccia del soggetto corrente (per output facoltativo)
last_subject_key = None

# Dizionario per tenere traccia delle combinazioni già elaborate per ogni soggetto (evitare duplicati)
printed_combinations = {}

# Nuovo dizionario per memorizzare i risultati concatenati per coppia di condizioni per ogni soggetto
new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1 = {}

# Itera su tutte le chiavi del dizionario di partenza
for key in new_single_th_all_extracted_spectrograms_2D_task_1.keys():
    # Esempio di key: "spectrograms_baseline_1", "spectrograms_th_resp_1", etc.
    key_parts = key.split('_')
    subject_number = key_parts[-1] if key_parts[-1].isdigit() else None
    if subject_number is None:
        continue  # Salta eventuali chiavi non valide
    
    # Crea il nome del soggetto (es. "th_1")
    subject_key = f"th_{subject_number}"
    
    # Se cambiamo soggetto, stampiamo (facoltativo)
    if subject_key != last_subject_key:
        if last_subject_key is not None:
            print("\n" + "-" * 50 + f" END OF {last_subject_key.upper()} " + "-" * 50 + "\n")
        print(f"\nProcessing Subject: {subject_key}\n" + "=" * 80)
        printed_combinations[subject_key] = set()  # Inizializza il set per il soggetto corrente
    last_subject_key = subject_key

    # Inizializza la struttura per il soggetto se non esiste già
    if subject_key not in new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1:
        new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1[subject_key] = {}

    # Crea tutte le combinazioni uniche di condizioni sperimentali (es. "baseline_vs_th_resp")
    condition_pairs = list(itertools.combinations(experimental_conditions, 2))
    
    # Loop sulle coppie di condizioni
    for cond_1, cond_2 in condition_pairs:
        condition_pair_key = f"{cond_1}_vs_{cond_2}"
        if condition_pair_key in printed_combinations[subject_key]:
            continue  # Salta se già elaborata per questo soggetto
        
        print(f"\n\tCreation of Coupled Condition: \033[1m{condition_pair_key}\033[0m")
        printed_combinations[subject_key].add(condition_pair_key)
        
        # Estrai i dati e le etichette per la prima condizione
        data_cond_1 = new_single_th_all_extracted_spectrograms_2D_task_1.get(f"spectrograms_{cond_1}_{subject_number}", {}).get('spectrograms')
        labels_cond_1 = new_single_th_all_extracted_spectrograms_2D_task_1.get(f"spectrograms_{cond_1}_{subject_number}", {}).get('labels')
        
        # Estrai i dati e le etichette per la seconda condizione
        data_cond_2 = new_single_th_all_extracted_spectrograms_2D_task_1.get(f"spectrograms_{cond_2}_{subject_number}", {}).get('spectrograms')
        labels_cond_2 = new_single_th_all_extracted_spectrograms_2D_task_1.get(f"spectrograms_{cond_2}_{subject_number}", {}).get('labels')
        
        # Salta la coppia se uno dei dati non è presente
        if data_cond_1 is None or labels_cond_1 is None or data_cond_2 is None or labels_cond_2 is None:
            continue
        
        # Riassegna le etichette: tutti 0 per cond_1, tutti 1 per cond_2
        labels_cond_1 = np.zeros_like(labels_cond_1, dtype=int)
        labels_cond_2 = np.ones_like(labels_cond_2, dtype=int)
        
        # Concatenazione dei dati (verticale) e delle etichette (orizzontale)
        concatenated_data = np.concatenate((data_cond_1, data_cond_2), axis=0)
        concatenated_labels = np.concatenate((labels_cond_1, labels_cond_2), axis=0)
        
        # Salva il risultato per questa coppia nel dizionario del soggetto
        new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1[subject_key][condition_pair_key] = {
            'data': concatenated_data,
            'labels': concatenated_labels
        }
        
        print(f"Condition pair '{condition_pair_key}' for subject {subject_key} => Data shape: {concatenated_data.shape}, Labels length: {len(concatenated_labels)}")


In [ ]:
print('finito')

In [ ]:
print(f"\t\t\t\t\033[1mStructure of new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1\033[0m: \n")
print(f"\033[1mFirst Order Keys\033[0m: {new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1.keys()}\n")
print(f"\033[1mSecond Order Keys\033[0m: {new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_1'].keys()}\n")
print(f"\033[1mThird Order Keys\033[0m: \n{new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_1']['rest_vs_left_fist'].keys()}\n")

In [ ]:
'''VERIFICO CHE LA CONCATENAZIONE SIA AVVENUTA CORRETTAMENTE!

    CORREZIONE UFFICIALE PER SPETTROGRAMS
'''

print("\tNOW, LET US SEE IF THE CONCATENATIONS RESPECT THE ORIGINAL SHAPES FOR EVERY COUPLED EXPERIMENTAL CONDITION!")
print("\tFOR THE 1°ST SUBJECT: CHECK IF THE SUM OF INDIVIDUAL EXP COND SHAPE MATCHES THE COUPLED COND CONCATENATION SHAPE:\n\n")

print("\033[1mINDIVIDUAL EXP COND SHAPE OF FIRST SUBJECT\033[0m:")

# Verifica delle dimensioni originali per le singole condizioni sperimentali

#for cond in ['baseline', 'th_resp', 'pt_resp', 'shared_resp']:
for cond in ['rest', 'left_fist', 'right_fist']:    
    
    key = f'spectrograms_{cond}_1'
    if key in new_single_th_all_extracted_spectrograms_2D_task_1:
        unique_labels, counts = np.unique(new_single_th_all_extracted_spectrograms_2D_task_1[key]['labels'], return_counts=True)
        print(f"{cond.capitalize()} in TH_1: {unique_labels}, Counts: {counts}\n")
    else:
        print(f"Warning: {key} not found in new_single_th_all_extracted_spectrograms_2D_task_1\n")

# Seleziona il primo soggetto (ad esempio, 'th_1')
subject_key = 'th_1'

# Itera per ogni coppia di condizioni sperimentali direttamente (senza livelli)
for condition_pair_key, data_labels in new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1.get(subject_key, {}).items():
    
    # Estrarre dati e labels dalla struttura corretta
    data = data_labels['data']
    labels = data_labels['labels']

    # Stampare il nome della coppia di condizioni e le loro dimensioni
    print(f"Condition Pair: \033[1m{condition_pair_key}\033[0m")
    
    if data is not None and labels is not None:
        print(f"  Data Shape: {data.shape}")
        print(f"  Labels Shape: {labels.shape}")
    else:
        print("  Missing data or labels!")

        

In [ ]:
print(np.unique(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_1']['rest_vs_left_fist']['labels'], return_counts = True))
print(np.unique(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_1']['rest_vs_right_fist']['labels'], return_counts = True))

In [ ]:
print(f"\t\t\t\033[1mREST_VS_LEFT_FIST\033[0m\n")
print(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_1']['rest_vs_left_fist']['labels'][:44])
print(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_1']['rest_vs_left_fist']['labels'][44:])
print()
print()
print(f"\t\t\t\033[1mREST_VS_RIGHT_FIST\033[0m\n")
print(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_1']['rest_vs_right_fist']['labels'][:44])
print(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_1']['rest_vs_right_fist']['labels'][44:])

##### STEP - Salvataggio Dataset di All Single Therapists EEG Data Spectrograms across Couples of Experimental Conditions

In [ ]:
!pwd

In [ ]:
base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
import pickle

# Salvare l'intero dizionario annidato con pickle
with open(f'{base_path}new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1.pkl', 'wb') as f:
    pickle.dump(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1, f)
    print(f"✅ Dati salvati in: {base_path}")

#### **Concatenazione** **ALL** Single Subject **Spectrograms (TH)** per **Coppie di Condizioni Sperimentali INSIEME**

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la isualizzazione dei dati
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pickle

import os 

base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
# Caricare l'intero dizionario annidato con pickle
with open(f'{base_path}/new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1.pkl', 'rb') as f:
    new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1 = pickle.load(f)

In [ ]:
#new_subject_level_concatenations_spectrograms_coupled_exp_th.keys()
#new_subject_level_concatenations_spectrograms_coupled_exp_th['th_1'].keys()
#new_subject_level_concatenations_spectrograms_coupled_exp_th['th_1']['theta'].keys()
#new_subject_level_concatenations_spectrograms_coupled_exp_th['th_1']['theta']['baseline_vs_th_resp'].keys()
#np.unique(new_subject_level_concatenations_spectrograms_coupled_exp_th['th_1']['theta']['baseline_vs_th_resp']['labels'], return_counts = True)

In [ ]:
np.unique(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_1']['rest_vs_left_fist']['labels'], return_counts = True)

In [ ]:
'''CODICE UFFICIALE CON MODIFICHE A SPETTROGRAMMI'''


import numpy as np

def concatenate_all_subjects_coupled_spectrogram_conditions_th(data_structure, conditions):
    
    """
    Concatena globalmente i dati e le etichette per ogni coppia di condizioni sperimentali
    a partire dalla struttura 'new_subject_level_concatenations_spectrograms_coupled_exp_th'.
    I dati di ogni soggetto (già raggruppati per coppia di condizioni) vengono ulteriormente 
    concatenati per ottenere un unico array globale per ogni coppia.
    
    Parameters:
        data_structure (dict): Struttura dati con chiavi per soggetto (es. 'th_1', 'th_2', ...)
                               e, per ciascun soggetto, chiavi per ciascuna coppia di condizioni sperimentali.
        condition_pairs (list): Lista di coppie di condizioni sperimentali (es. 'baseline_vs_th_resp', ...)
        
    Returns:
        dict: Un dizionario globale in cui ogni chiave è una coppia di condizioni e il valore
              è un dizionario con 'data' e 'labels' concatenati globalmente (da tutti i soggetti).
    """
        
    # Dizionario per contenere i dati concatenati di tutti i soggetti
    all_subj_data_by_coupled_cond = {}

    # Itera su ogni coppia di condizioni sperimentali
    for condition_pair in conditions:

        #Dizionari per raccogliere dati e labels ordinati per ogni coppia di condizioni sperimentali
        # che è dinamico, per ogni coppia di condizioni sperimentali 
        #(quindi si ricreeranno passando alla condizione sperimentale successiva!)

        data_by_label = {}
        labels_by_label = {}
        shape_labels_per_subject = {0: [], 1: []}  # Inizializza il dizionario per 0 e 1

        # Itera su tutti i soggetti
        # In questo caso, itera sulle chiavi di ogni soggetto

        # Quindi prenderà, per la relativa coppia di condizioni sperimentali di ogni soggetto,
        # i dati e le labels di quel soggetto, per quella relativa coppia di condizioni sperimentali

        for subject_key in data_structure.keys():

            # Controlla se il livello di ricostruzione esiste per il soggetto
            if condition_pair not in data_structure[subject_key]:
                continue

            # Qui estrae i dati e le labels per questa condizione sperimentale di quel soggettio lì
            subject_data = data_structure[subject_key][condition_pair]['data']
            subject_labels = data_structure[subject_key][condition_pair]['labels']

            # Trova le etichette uniche e gli indici corrispondenti per quella coppia di condizioni sperimentali lì,
            # Ossia, potrà trovare gli 0 e gli 1 

            unique_labels = np.unique(subject_labels)

            #A quel punto, per ognuna delle due labels trovate per quella coppia di condizioni sperimentali lì
            #Che saranno sempre o 0 o 1

            for label in unique_labels:

                # Trova gli indici dei dati corrispondenti a questa etichetta
                # Una volta per lo 0 ed una volta per l' 1

                label_indices = np.where(subject_labels == label)[0]

                # A quel punto, estrae i dati e le labels per questi indici
                # Una volta per lo 0 ed una volta per l' 1

                data_for_label = subject_data[label_indices]
                labels_for_label = subject_labels[label_indices]

                # A quel punto, per ogni soggetto, per ogni condzione sperimentale, 
                # per quelle due labels là (sempre presenti, per ogni coppia di condizioni sperimentali)
                # Una volta per lo 0 ed una volta per l' 1

                #Andrà a creare per ogni etichetta, una chiave che si chiamerà 
                #o 0 od 1 (inizialmente, vuoto -> perché deve esser inizializzato)
                #E poi, dopo, ci appenderà le labels 
                 # Una volta per lo 0 ed una volta per l' 1
                #  Di quel soggetto per quella coppia di condizioni sperimentali iterate a quel momento

                #Aggiunge ai dizionari globali, creando la chiave se non esiste
                if label not in data_by_label:
                    data_by_label[label] = []
                    labels_by_label[label] = []

                data_by_label[label].append(data_for_label)
                labels_by_label[label].append(labels_for_label)

                # **Salva la shape delle etichette per il soggetto corrente**
                shape_labels_per_subject[label].append(labels_for_label.shape[0]) # Aggiungi solo la dimensione (numero di etichette)

                '''PRINT PER CHECK LABELS DI OGNI SOGGETTO PER OGNI COPPIA DI CONDIZIONE CONDIZIONI SPERIMENTALI''' 
                #print(f"Soggetto: \033[1m{subject_key}\033[0m, Livello: \033[1m{reconstruction_level}\033[0m, Condizione: \033[1m{condition_pair}\033[0m, "
                #      f"Etichetta: \033[1m{label}\033[0m, Shape: \033[1m{labels_for_label.shape[0]}\033[0m")

                # **Aggiungi un print di controllo per ogni soggetto**
                #print(f"Soggetto: \033[1m{subject_key}\033[1m, Livello: {reconstruction_level}, Condizione: {condition_pair}, Etichetta: {label}, Shape: {labels_for_label.shape[0]}")

            
        #*** *** *** *** *** *** *** *** *** *** *** *** *** *** ***

        # Dopodiché, vado a creare invece i dizionari che conterranno
        # Le concatenazioni, di dati e labels corrispondenti, di tutti i soggetti per cui l'etichetta era 
        # o 0 od 1 

        # Di conseguenza, qui dentro dovrei avere, per ogni coppia di condizioni sperimentali
        # Tutti gli 0 ed 1 (ed i relativi dati corrispondenti)
        # di tutti i soggetti, ma concatenati

        concatenated_data_by_label = {}
        concatenated_labels_by_label = {}

        #for label in data_by_label.keys():
        #    if len(data_by_label[label]) > 0:  # Evita errori di concatenazione
        #        concatenated_data_by_label[label] = np.vstack(data_by_label[label])
        #        concatenated_labels_by_label[label] = np.hstack(labels_by_label[label])
            
        for label in data_by_label.keys():
            concatenated_data_by_label[label] = np.vstack(data_by_label[label])
            concatenated_labels_by_label[label] = np.hstack(labels_by_label[label])

            # **Calcolare la shape totale delle etichette**
            #total_labels_0 = np.sum(1 for subject_labels in labels_by_label[label] if 0 in subject_labels)
            #total_labels_1 = np.sum(1 for subject_labels in labels_by_label[label] if 1 in subject_labels)
            #total_labels = total_labels_0 + total_labels_1

            # **Determinare gli indici delle etichette 0 e 1 nell'array finale**

            # Gli indici delle etichette 0
            indices_labels_0 = np.where(concatenated_labels_by_label[label] == 0)[0]

            # Gli indici delle etichette 1
            indices_labels_1 = np.where(concatenated_labels_by_label[label] == 1)[0]

            # Indici iniziale e finale per le etichette 0 e 1
            start_idx_0 = indices_labels_0[0] if len(indices_labels_0) > 0 else None
            end_idx_0 = indices_labels_0[-1] if len(indices_labels_0) > 0 else None
            start_idx_1 = indices_labels_1[0] if len(indices_labels_1) > 0 else None
            end_idx_1 = indices_labels_1[-1] if len(indices_labels_1) > 0 else None


            # **Print finale per verificare la concatenazione per ogni label**
            #print(f"\nCondizione: \033[1m{condition_pair}\033[0m, Etichetta: {label}")
            #print(f"  - Shape dei dati concatenati per \033[1m{label}\033[0m: {concatenated_data_by_label[label].shape}")
            #print(f"  - Shape delle etichette concatenate per \033[1m{label}\033[0m: {concatenated_labels_by_label[label].shape}")

            # **Stampa la lista delle shapes delle etichette per ogni soggetto per questa etichetta**
            #print(f"\n  - Shape delle etichette per soggetto (per etichetta {label}): {shape_labels_per_subject[label]}\n")


            # **Stampa il conteggio totale delle etichette**
            #print(f"\n  - Totale delle etichette 0: {total_labels_0}")
            #print(f"  - Totale delle etichette 1: {total_labels_1}")
            #print(f"  - Totale delle etichette per la condizione: {total_labels}\n")

            # **Stampa gli indici per le etichette 0 e 1**
            # **OSSIA --> Stampa gli indici per l'etichetta corrente**

            #if label == 0:
            #    print(f"\n  - Indici per etichetta 0: Inizio = {start_idx_0}, Fine = {end_idx_0}")
            #else:
            #    print(f"  - Indici per etichetta 1: Inizio = {start_idx_1}, Fine = {end_idx_1}\n")


            #print(f"\n  - Indici per etichetta 0: Inizio = {start_idx_0}, Fine = {end_idx_0}")
            #print(f"  - Indici per etichetta 1: Inizio = {start_idx_1}, Fine = {end_idx_1}")


        #ARRIVATI FINO A QUI, abbiamo che siccome stiamo iterando prima per tutti gli 0 e poi per tutti gli 1
        #Significa che, assumendo che siamo dentro 'baseline_vs_th_resp' e che 

        #'baseline' sia rappresentato dalle etichette 0
        #'th_resp' sia rappresentato dalle etichette 1

        #Al PRIMO ciclo avrò che:

        #con concatenated_data_by_label[label] ho solo concatenato tutti i dati (di 'baseline' per 'baseline_vs_th_resp' di tutti i soggetti)
        #con concatenated_labels_by_label[label] ho solo concatenato tutti gli 0 (di 'baseline' per 'baseline_vs_th_resp'di tutti i soggetti)

        #Al SECONDO ciclo avrò che:

        #con concatenated_data_by_label[label] ho solo concatenato tutti i dati (di 'th_resp' per 'baseline_vs_th_resp'  di tutti i soggetti)
        #con concatenated_labels_by_label[label] ho solo concatenato tutti gli 0 (di 'th_resp' per 'baseline_vs_th_resp' di tutti i soggetti)

        #Quindi mi manca ancora 

        #A) CONCATENARE I DATI:

        #1)prima tutti i dati (di 'baseline' per 'baseline_vs_th_resp' di tutti i soggetti) 

        #CON

        #2)tutti i dati (di 'th_resp' per 'baseline_vs_th_resp'  di tutti i soggetti)

        #che verrà fatto dentro all_data (che è anch'esso una variabile dinamica!)

        #B) CONCATENARE LE LABELS:

        #1)prima tutti le labels (di 'baseline' per 'baseline_vs_th_resp' di tutti i soggetti) 

        #CON

        #2)tutti le labels (di 'th_resp' per 'baseline_vs_th_resp' di tutti i soggetti)

        #che verrà fatto dentro all_labels (che è anch'esso una variabile dinamica!)

        # Liste per raccogliere dati e labels di tutte le etichette

        # Qui dentro, invece, dov con

        all_data = []
        all_labels = []

        for label in concatenated_data_by_label.keys():
            all_data.append(concatenated_data_by_label[label])
            all_labels.append(concatenated_labels_by_label[label])

        #Alla fine, qui dentro avrò che, PER OGNI COPPIA DI CONDIZIONI SPERIMENTALI


        #'final_data' dovrebbe avere 
            #- prima tutti i dati di tutti i soggetti associati all'etichetta 0 
            #- e poi tutti i dati di tutti i soggetti associati all'etichetta 1

        #'final_labels' dovrebbe avere 
            #- prima tutte le labels di tutti i soggetti associati all'etichetta 0
            #- e poi tutte tutte le labels di tutti soggetti associati all'etichetta 1...

        if all_data:  # Evita errori di concatenazione se non ci sono dati
            final_data = np.vstack(all_data)
            final_labels = np.hstack(all_labels)

            all_subj_data_by_coupled_cond[condition_pair] = {
                'data': final_data,
                'labels': final_labels
            }
            
    return all_subj_data_by_coupled_cond


In [ ]:
# Parametri


#conditions = ['baseline_vs_th_resp', 'baseline_vs_pt_resp', 'baseline_vs_shared_resp',
#              'th_resp_vs_pt_resp', 'th_resp_vs_shared_resp', 'pt_resp_vs_shared_resp']

    
conditions = ['rest_vs_left_fist', 'rest_vs_right_fist', 'left_fist_vs_right_fist']


# Chiamata alla funzione, qui definisco a mano il nome della variabile (i.e., 
new_all_th_concat_spectrograms_coupled_exp_task_1 = concatenate_all_subjects_coupled_spectrogram_conditions_th(
    data_structure=new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1,
    conditions=conditions
)

#print("\n\033[1mIntervalli di indici per ciascuna etichetta nei sotto-dizionari:\033[0m\n")


for condition_pair, value in new_all_th_concat_spectrograms_coupled_exp_task_1.items():
    labels = value['labels']
    unique_labels = np.unique(labels)
    total_labels = 0

    #print(f"\nCondizione: \033[1m{condition_pair}\033[0m")
    for label in unique_labels:
        label_indices = np.where(labels == label)[0]
        start_idx = label_indices[0] if len(label_indices) > 0 else None
        end_idx = label_indices[-1] if len(label_indices) > 0 else None
        total_labels += len(label_indices)

        #print(f"  Etichetta \033[1m{label}\033[0m: Indici \033[1m{start_idx}-{end_idx}\033[0m "
        #      f"(Totale Etichette: \033[1m{len(label_indices)}\033[0m)")

    #print(f"Totale etichette (0 e 1): \033[1m{total_labels}\033[0m\n")

In [ ]:
print('finito')

In [ ]:
new_all_th_concat_spectrograms_coupled_exp_task_1.keys()

In [ ]:
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['left_fist_vs_right_fist']['labels'], return_counts = True))

In [ ]:
'''
Ecco un piccolo snippet che, per ciascuna coppia di condizioni in new_all_th_concat_spectrograms_coupled_exp_task_1, 
stampa per ogni valore di label (0 e 1) l’indice di partenza e quello di fine nel vettore labels:

Come funziona

np.unique(labels) restituisce i valori distinti (ad es. [0, 1]).

Per ciascun valore val, np.where(labels == val)[0] trova tutti gli indici in cui labels è uguale a val.

Con .min() e .max() ricavi il primo e l’ultimo indice di ciascun blocco.

Se vuoi anche vedere tutte le sott‐sequenze (ad es. se le label non sono concatenate in un unico blocco), 
puoi invece usare [itertools.groupby] o scorrere i idxs alla ricerca di “buchi”. 

Ma nel tuo caso — visto che hai prima aggiunto tutti gli zero e poi tutti gli uno — vedrai sempre un singolo intervallo per ciascuna label.

'''

import numpy as np

for cond_pair, payload in new_all_th_concat_spectrograms_coupled_exp_task_1.items():
    labels = np.array(payload['labels'])
    print(f"\n=== {cond_pair} ===")
    for val in np.unique(labels):
        idxs = np.where(labels == val)[0]
        start, end = idxs.min(), idxs.max()
        print(f"  label {val}: indices {start} → {end} (count = {len(idxs)})")
        
print()
print()
for cond_pair, cond_data in new_all_th_concat_spectrograms_coupled_exp_task_1.items():
    data = cond_data['data']      # shape: (n_trials, ...)
    labels = cond_data['labels']  # shape: (n_trials,)
    print(f"{cond_pair}:")
    print(f"  data shape  = {data.shape}")
    print(f"  labels len  = {len(labels)}\n")

In [ ]:
for cond_pair, cond_data in new_all_th_concat_spectrograms_coupled_exp_task_1.items():
    data = cond_data['data']      # shape: (n_trials, ...)
    labels = cond_data['labels']  # shape: (n_trials,)
    print(f"{cond_pair}:")
    print(f"  data shape  = {data.shape}")
    print(f"  labels len  = {len(labels)}\n")

##### STEP - Salvataggio Dataset di **All Single Therapists** EEG Data Spectrograms across **Couples of Experimental Conditions** INSIEME

In [ ]:
pwd

In [ ]:
cd ..

In [ ]:
base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
import pickle

# Salvare l'intero dizionario annidato con pickle
with open(f'{base_path}new_all_th_concat_spectrograms_coupled_exp_task_1.pkl', 'wb') as f:
    pickle.dump(new_all_th_concat_spectrograms_coupled_exp_task_1, f)

## Modifica della Funzione per la **Rappresentazione Frequenza-Elettrodi dei miei dati EEG** a seconda del Dataset del Task scelto

- Task 1: Motor Execution (Open and close right or left fist) - Unilateral - Runs 3, 7, 11 
- Task 2: Motor Imagery (Imagine opening and closing right and left fists) - Unilateral - Runs 4, 8, 12 
- Task 3: Motor Execution (Open and close both fists and feet) - Bilateral - Runs 5, 9, 13 
- Task 4: Motor Imagery (Imagine opening and closing both fists and feet) - Bilateral - Runs 6, 10, 14 

#### IMPLEMENTAZIONE -Modifica della Funzione per la Rappresentazione Frequenza-Elettrodi dei miei dati EEG a seconda del Dataset del Task scelto


#### **Calcolo Spettrogrammi** All **SINGLE** Subject Data (**TH**) per **Tutte le Condizioni sperimentali**

In [ ]:
'''
Con questo workflow puoi poi caricare in ambiente new_pd_env con:

'''

import numpy as np

NPZ = "/home/stefano/EEGMMIDB_Analyses/final_cleaned_EEG_data.npz"
npz = np.load(NPZ, allow_pickle=False)

def unflatten_npz_simple(npz_obj):
    root = {}
    for full_key in npz_obj.files:
        task, idx_str, leaf = full_key.split('/', 2)
        idx = int(idx_str)
        root.setdefault(task, [])
        while len(root[task]) <= idx:
            root[task].append({})
        root[task][idx][leaf] = npz_obj[full_key]
    return root

new_EEG_data = unflatten_npz_simple(npz)

In [ ]:
'''
CREAZIONE SPETTROGRAMMI ELETTRODI x FREQUENZE - VERSIONE UFFICIALE



NEW UPDATED VERSION FOR DATASET MOTOR TASK - FREQUENCY x CHANNELS UFFICIALE 

Ecco la versione rivista della tua funzione, con in più il parametro data_task che seleziona il sotto-dizionario giusto in base al task
(es. "task_1", "task_2", …) e poi riapplica esattamente la stessa logica di STFT sui soli soggetti di quel task.



Ti propongo una versione modificata della funzione che, per ogni soggetto e condizione, 
calcola la potenza spettrale tramite una FFT (usando np.fft.rfft) per ogni canale su un segmento specifico del segnale,
media i risultati sui trial e costruisce un’unica immagine 2D con:

Asse 0 (righe): vettore delle frequenze (ottenuto dalla FFT)
Asse 1 (colonne): elettrodi

In questo modo, per ogni soggetto e condizione avrai un’unica immagine in cui ogni “colonna” (cioè ogni elettrodo) 
mostra il vettore 1D delle potenze spettrali, come richiesto per poter interpretare successivamente il Grad-CAM.



Spiegazione delle modifiche

FFT anziché STFT:
Al posto di calcolare una STFT e poi aggregare nel tempo, ora per ogni canale si esegue una FFT sul segmento selezionato
(da segment_start a segment_end). La funzione np.fft.rfft restituisce solo i componenti a frequenza non negativa.

Aggregazione sui trial:

Per ciascun soggetto e condizione, 
i risultati della FFT ( = vettori di potenza spettrale), 
ossia il vettore ottenuto da ogni canale, viene impilato 
in modo da ottenere l'immagine di un trial. 
 
Ogni immagine rappresenta la potenza spettrale raccolta dalla FFT di ogni elettrodo di quel trial, e ogni immagine viene memorizzata separatamente.

Formazione dell’immagine:
I vettori di potenza di ciascun elettrodo vengono “impilati” in modo tale che, 
dopo la trasposizione, le righe rappresentino le frequenze e le colonne gli elettrodi. 

Questo formato permette al Grad-CAM di indicare quali frequenze e quali elettrodi risultano più discriminanti.

Questa funzione dovrebbe rispondere alle richieste del tuo supervisore, generando per ogni soggetto e condizione una sola immagine interpretabile con il Grad-CAM.


La funzione è stata progettata per rispettare quanto richiesto da Nicola. 

In particolare:

Calcolo per ogni elettrodo: Per ciascun canale (elettrodo) viene calcolata la FFT sul segmento specificato (da segment_start a segment_end), 
ottenendo un vettore 1D con la potenza spettrale in funzione della frequenza.

Aggregazione sui trial: 

Per ciascun soggetto e condizione, 
i risultati della FFT ( = vettori di potenza spettrale), 
ossia il vettore ottenuto da ogni canale, viene impilato 
in modo da ottenere l'immagine di un trial. 
 
Ogni immagine rappresenta la potenza spettrale raccolta dalla FFT di ogni elettrodo di quel trial, e ogni immagine viene memorizzata separatamente.

 
Creazione dell'immagine: 

Questi vettori, NON concatenati ma organizzati in colonne, vengono trasposti in modo da avere 

    - le righe rappresentanti le frequenze
    - le colonne gli elettrodi 
    
Così si ottiene, per ogni soggetto (e per ogni condizione) una singola immagine, 
che permette al Grad-CAM di evidenziare quali elettrodi (e quali frequenze) siano determinanti nel processo decisionale.

Se questo è esattamente quanto intendeva Nicola ("l'immagine sarebbe una sola per ogni soggetto" ottenuta raggruppando i vettori di ogni elettrodo), 
allora la funzione è adattata correttamente.


---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ----
RISULTATO FINALE:

1) Una immagine per ogni trial:
La variabile trial_power calcola la potenza spettrale per ciascun trial e la salva come una singola immagine (2D). 
    Queste immagini sono salvate nel dizionario condition_results[condition_key]['fft_images'] una per ciascun trial.

2) Shape dell'immagine:
Ogni immagine ha la forma (n_freq, n_channels), dove:

- n_freq è il numero di frequenze calcolato con np.fft.rfft.
- n_channels è il numero di elettrodi.

3) Etichette:
Se sono presenti le etichette per la condizione, vengono associate correttamente e stampate, come nel codice originale.

4) Dizionario finale:
Alla fine, il dizionario all_subjects_condition_results contiene un array 3D per ogni condizione (uno per ogni trial), con la forma 
    (n_trials, n_freq, n_channels).

Questo codice ora mantiene la logica originale di creare una immagine per ogni trial, mentre applica la FFT per calcolare la potenza spettrale. 

---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ----
'''



#Ecco la versione rivista della tua funzione, con in più il parametro data_task che seleziona il sotto-dizionario giusto in base al task
#(es. "task_1", "task_2", …) e poi riapplica esattamente la stessa logica di STFT sui soli soggetti di quel task.



import numpy as np

def create_spectrograms_from_fft_images_of_electrodes(
    new_EEG_data: dict,
    data_task: str,
    segment_start: int = 0,
    segment_end: int = 640,
    fs: int = 160,
    max_freq_bins: int = 45
):
    
    """
    Per ogni soggetto e condizione, calcola le immagini della potenza spettrale per ciascun elettrodo 
    usando la FFT (np.fft.rfft) sul segmento definito del segnale (per esempio l'intero trial).
    
    Per ogni trial, viene calcolata la FFT per ogni canale, ottenendo un vettore 1D di potenza spettrale.
    Viene poi tagliato lo spettro per mantenere solo le prime 'max_freq_bins' frequenze.
    
    Ogni trial genera un'immagine 2D (righe: frequenze, colonne: elettrodi) che viene memorizzata separatamente.
    
    Parameters:
        data (list di dict): Ogni dict corrisponde a un soggetto e contiene le condizioni sperimentali
                              (chiavi con dati in formato np.array di shape (n_trials, n_channels, n_timepoints))
                              e, eventualmente, le relative etichette (con chiave condition_labels).
        segment_start (int): Indice iniziale del segmento da analizzare.
        segment_end (int): Indice finale del segmento da analizzare.
        fs (int): Frequenza di campionamento.
        max_freq_bins (int): Numero massimo di frequenze da mantenere (ad es. le prime 45 frequenze).
    
    Returns:
        dict: Un dizionario in cui per ogni condizione sono memorizzati:
              - 'spectrograms_3d': array 3D contenente le immagini per ogni trial (shape: n_trials x max_freq_bins x n_channels)
              - 'labels': etichette associate (se presenti)
    """
    
    
    """
    new_EEG_data: { 
      task: [ {cond_subj: np.ndarray, cond_subj_labels: np.ndarray, ...},  # subj1
               { ... },  # subj2
               ... ],
      ...
    }
    data_task:     "task_1" / "task_2" / "task_3" / "task_4"
    
    Ritorna:
      { 'cond': {
           'spectrograms_3d': ndarray(n_trials_tot, n_freq_bins, n_channels),
           'labels':         ndarray(n_trials_tot,)
         },
        ...
      }
    """
    # --- 1) Filtra sul task richiesto ---
    if data_task not in new_EEG_data:
        raise KeyError(f"Task '{data_task}' non trovato in new_EEG_data")
    data = new_EEG_data[data_task]  # lista di dict, uno per soggetto

    

    # --- 2) corpo originale, invariato ---

    # Dizionario per memorizzare i risultati per ciascuna condizione sperimentale
    condition_results = {}
    
    for idx, data_dict in enumerate(data):
        

        print(f"\n\n\t\t\t\t\t\033[1mProcessing Subject {idx + 1}\033[0m:\n")

        for condition, value in data_dict.items():
            
            # Calcolo condition_key senza '_labels'
            if condition.endswith('_labels'):
                condition_key = condition[:-7]
            else:
                condition_key = condition
            
            # inizializzo sempre la entry
            if condition_key not in condition_results:

                condition_results[condition_key] = {'fft_images': [], 'labels': []}
    
            
            # ——————————————————————————
            # [PATCHED] ramo label
            if condition.endswith('_labels'):
                
                # Converto in lista di interi
                #labels = np.asarray(value, dtype=int).tolist()
                labels = np.asarray(value, dtype=int)
                
                print(f"\nFound labels for condition \033[1m{condition}\033[0m, length: \033[1m{len(labels)}\033[0m\n")
                    
                # Uso condition_key (non labels_key!)
                condition_results[condition_key]['labels'].extend(labels)
                continue
            
            '''
            In pratica
            Riga 1: if not condition.endswith('_labels') and isinstance(value, np.ndarray):
            → processa solo i dati (n_trials, n_chan, n_time)

            Riga 2: elif condition.endswith('_labels') and isinstance(value, np.ndarray):
            → processa solo le labels (n_trials,)

            Riga 3: else:
            → tutto il resto viene ignorato

            Con questo schema le tue fft‐images useranno solo gli array 3‑D, e le labels verranno raccolte nel punto giusto, 
            senza mai andare a indicizzare una shape non valida.
            '''
            
            #OLD VERSION --> vecchia versione: processi ogni np.ndarray, incluso quello 1‑D delle labels
            #if isinstance(value, np.ndarray):
            
            # 1) FFT SOLO su array 3‑D (i dati veri)
            #if not condition.endswith('_labels') and isinstance(value, np.ndarray):
            
            # 1) processa solo i dati (3‑D) non le etichette 1‑D
            if isinstance(value, np.ndarray) and value.ndim == 3:
                
                # … STFT sul valore 3‑D …
                print(f"\nApplicazione FFT per '\033[1m{condition_key}\033[0m'\n")
                print(f"\033[1mShape\033[0m dei dati per \033[1m{condition_key}\033[0m: {value.shape}")
                
                # Presupponiamo che 'value' abbia forma: (n_trials, n_channels, n_timepoints)
                n_trials, n_channels, _ = value.shape
                
                N = segment_end - segment_start  # lunghezza del segmento analizzato
                
                n_freq = N // 2 + 1  # np.fft.rfft restituisce n_freq = N//2 + 1
                
                # Determina il numero di frequenze da utilizzare (taglia lo spettro)
                effective_freq_bins = min(n_freq, max_freq_bins)
                
                # Itera su ogni trial e ogni canale per calcolare la potenza spettrale
                for trial in range(n_trials):
                    
                    #trial_power = np.zeros((n_channels, n_freq))  # Matrice per la potenza per ogni trial
                    trial_power = np.zeros((n_channels, effective_freq_bins))  # Matrice per la potenza per ogni trial
                    
                    for channel in range(n_channels):
                        
                        # Estrai il segmento del segnale
                        single_channel_data = value[trial, channel, segment_start:segment_end]
                        
                        # Calcola la FFT (solo componenti a frequenza non negativa)
                        # -> https://numpy.org/doc/stable/reference/generated/numpy.fft.rfft.html
                        fft_vals = np.fft.rfft(single_channel_data)
                        
                        # Calcola la potenza spettrale (modulo al quadrato)
                        power = np.abs(fft_vals) ** 2
                        
                        #trial_power[channel] = power
                        
                        trial_power[channel] = power[:effective_freq_bins]
                    
                    # Trasponi in modo che le righe siano le frequenze e le colonne gli elettrodi
                    fft_image = trial_power.T  # shape: (n_freq, n_channels)
                    #print(f"FFT image shape for trial {trial + 1} of condition '{condition_key}': {fft_image.shape}")
                    
                    # Salva l'immagine per il trial corrente
                    #Ogni trial genera una sua immagine della potenza spettrale che viene memorizzata separatamente!
                    condition_results[condition_key]['fft_images'].append(fft_image)
                
                # Gestione delle etichette se presenti                
                #if labels_key in data_dict:
                    #num_labels = data_dict[labels_key] #QUI IN REALTA' SAREBBE GIA' ARRAY NUMPY MA PER IL CODICE CONVIENE RIPORTARLO INT COME LISTA!
                    #'''UNICA MODIFICA IMPORTANTE DA FARE --> RIPORTARE A VALORE DI INT COME LISTA DELLE LABELS (IMPORTANTE PER DOPO!)'''
                    #num_labels = np.asarray(value).tolist()   # trasforma in lista di int
                    #if condition_key in condition_results:
                    #    print(f"\nFound labels for condition \033[1m{labels_key}\033[0m, length: \033[1m{len(num_labels)}\033[0m\n")
                    #    condition_results[condition_key]['labels'].extend(num_labels)
                
            else:
                #print(f"\033[1m{labels_key}\033[0m non è un np.ndarray, non applicare la FFT")
                print(f"\033[1m{condition_key}\033[0m è np.ndarray di labels, salto FFT")    
    
    # Riorganizza i risultati in un dizionario finale
    all_subjects_condition_results = {}
    for condition_key, results in condition_results.items():
        if "_labels" in condition_key or len(results['fft_images']) == 0:
            print(f"\nSkipping \033[1m{condition_key}\033[0m as it is a label or has no valid data.")
            continue
        
        wave_condition_name = f'spectrograms_{condition_key}'
        print(f"\n\nResults for \t\t\t\t\t\033[1m{wave_condition_name}\033[0m:\n\n")
        
        # Ogni trial produce un'immagine 2D di shape (n_freq, n_channels)
        spectrograms_3d = np.array(results['fft_images'])
        all_subjects_condition_results[wave_condition_name] = {
            'spectrograms_3d': spectrograms_3d,
            'labels': results['labels']
        }
        print(f"spectrograms_3d shape for {wave_condition_name}: {spectrograms_3d.shape}")
    
    return all_subjects_condition_results


#new_all_subjects_condition_spectrograms_th = create_spectrograms_from_fft_images_of_electrodes (therapists_data, segment_start = 50, segment_end = 300, fs=250, max_freq_bins=45)

In [ ]:
'''
def create_spectrograms_from_fft_images_of_electrodes(
    new_EEG_data: dict,
    data_task: str,
    segment_start: int = 0,
    segment_end: int = 640,
    fs: int = 160,
    max_freq_bins: int = 45
):
    
'''

new_all_subjects_condition_spectrograms_task_1 = create_spectrograms_from_fft_images_of_electrodes(new_EEG_data,
                                                                                                   'task_1',
                                                                                                   segment_start = 0,
                                                                                                   segment_end = 640,
                                                                                                   fs=160,
                                                                                                   max_freq_bins=45)

In [ ]:
print('finito')

In [ ]:
new_all_subjects_condition_spectrograms_task_1.keys()

In [ ]:
print(new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100']['spectrograms_3d'].shape)
print()
print(len(new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100']['labels']))

In [ ]:
import pickle
import os
base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
# Salvare l'intero dizionario annidato con pickle
with open(f'{base_path}new_all_subjects_condition_spectrograms_task_1.pkl', 'wb') as f:
    pickle.dump(new_all_subjects_condition_spectrograms_task_1, f)

In [ ]:
#new_all_subjects_condition_spectrograms_task_1.keys()

In [ ]:
#new_all_subjects_condition_spectrograms_task_1['spectrograms_baseline_1'].keys()

In [ ]:
#new_all_subjects_condition_spectrograms_task_1['spectrograms_baseline_1']['spectrograms_3d'].shape

#### **Creazione Immagine 2D Grid EEG** per All **SINGLE** Subject **Spectrograms** per **Tutte le Condizioni Sperimentali**

##### **Richiesta Fatta a ChatGPT**

Ok ora per dirti, sto cercando di capire che step devo fare per creare questa immagine 2D...

inizialmente usavo questa funzione per calcolarmi la rappresentazione frequenza-elettrodi dell'immagine

import numpy as np

def create_spectrograms_from_fft_images_of_electrodes(
    new_EEG_data: dict,
    data_task: str,
    segment_start: int = 0,
    segment_end: int = 640,
    fs: int = 160,
    max_freq_bins: int = 45
):
    
    """
    Per ogni soggetto e condizione, calcola le immagini della potenza spettrale per ciascun elettrodo 
    usando la FFT (np.fft.rfft) sul segmento definito del segnale (per esempio l'intero trial).
    
    Per ogni trial, viene calcolata la FFT per ogni canale, ottenendo un vettore 1D di potenza spettrale.
    Viene poi tagliato lo spettro per mantenere solo le prime 'max_freq_bins' frequenze.
    
    Ogni trial genera un'immagine 2D (righe: frequenze, colonne: elettrodi) che viene memorizzata separatamente.
    
    Parameters:
        data (list di dict): Ogni dict corrisponde a un soggetto e contiene le condizioni sperimentali
                              (chiavi con dati in formato np.array di shape (n_trials, n_channels, n_timepoints))
                              e, eventualmente, le relative etichette (con chiave condition_labels).
        segment_start (int): Indice iniziale del segmento da analizzare.
        segment_end (int): Indice finale del segmento da analizzare.
        fs (int): Frequenza di campionamento.
        max_freq_bins (int): Numero massimo di frequenze da mantenere (ad es. le prime 45 frequenze).
    
    Returns:
        dict: Un dizionario in cui per ogni condizione sono memorizzati:
              - 'spectrograms_3d': array 3D contenente le immagini per ogni trial (shape: n_trials x max_freq_bins x n_channels)
              - 'labels': etichette associate (se presenti)
    """
    
    
    """
    new_EEG_data: { 
      task: [ {cond_subj: np.ndarray, cond_subj_labels: np.ndarray, ...},  # subj1
               { ... },  # subj2
               ... ],
      ...
    }
    data_task:     "task_1" / "task_2" / "task_3" / "task_4"
    
    Ritorna:
      { 'cond': {
           'spectrograms_3d': ndarray(n_trials_tot, n_freq_bins, n_channels),
           'labels':         ndarray(n_trials_tot,)
         },
        ...
      }
    """
    # --- 1) Filtra sul task richiesto ---
    if data_task not in new_EEG_data:
        raise KeyError(f"Task '{data_task}' non trovato in new_EEG_data")
    data = new_EEG_data[data_task]  # lista di dict, uno per soggetto

    

    # --- 2) corpo originale, invariato ---

    # Dizionario per memorizzare i risultati per ciascuna condizione sperimentale
    condition_results = {}
    
    for idx, data_dict in enumerate(data):
        

        print(f"\n\n\t\t\t\t\t\033[1mProcessing Subject {idx + 1}\033[0m:\n")

        for condition, value in data_dict.items():
            
            # Calcolo condition_key senza '_labels'
            if condition.endswith('_labels'):
                condition_key = condition[:-7]
            else:
                condition_key = condition
            
            # inizializzo sempre la entry
            if condition_key not in condition_results:

                condition_results[condition_key] = {'fft_images': [], 'labels': []}
    
            
            # ——————————————————————————
            # [PATCHED] ramo label
            if condition.endswith('_labels'):
                
                # Converto in lista di interi
                #labels = np.asarray(value, dtype=int).tolist()
                labels = np.asarray(value, dtype=int)
                
                print(f"\nFound labels for condition \033[1m{condition}\033[0m, length: \033[1m{len(labels)}\033[0m\n")
                    
                # Uso condition_key (non labels_key!)
                condition_results[condition_key]['labels'].extend(labels)
                continue
            
            '''
            In pratica
            Riga 1: if not condition.endswith('_labels') and isinstance(value, np.ndarray):
            → processa solo i dati (n_trials, n_chan, n_time)

            Riga 2: elif condition.endswith('_labels') and isinstance(value, np.ndarray):
            → processa solo le labels (n_trials,)

            Riga 3: else:
            → tutto il resto viene ignorato

            Con questo schema le tue fft‐images useranno solo gli array 3‑D, e le labels verranno raccolte nel punto giusto, 
            senza mai andare a indicizzare una shape non valida.
            '''
            
            #OLD VERSION --> vecchia versione: processi ogni np.ndarray, incluso quello 1‑D delle labels
            #if isinstance(value, np.ndarray):
            
            # 1) FFT SOLO su array 3‑D (i dati veri)
            #if not condition.endswith('_labels') and isinstance(value, np.ndarray):
            
            # 1) processa solo i dati (3‑D) non le etichette 1‑D
            if isinstance(value, np.ndarray) and value.ndim == 3:
                
                # … STFT sul valore 3‑D …
                print(f"\nApplicazione FFT per '\033[1m{condition_key}\033[0m'\n")
                print(f"\033[1mShape\033[0m dei dati per \033[1m{condition_key}\033[0m: {value.shape}")
                
                # Presupponiamo che 'value' abbia forma: (n_trials, n_channels, n_timepoints)
                n_trials, n_channels, _ = value.shape
                
                N = segment_end - segment_start  # lunghezza del segmento analizzato
                
                n_freq = N // 2 + 1  # np.fft.rfft restituisce n_freq = N//2 + 1
                
                # Determina il numero di frequenze da utilizzare (taglia lo spettro)
                effective_freq_bins = min(n_freq, max_freq_bins)
                
                # Itera su ogni trial e ogni canale per calcolare la potenza spettrale
                for trial in range(n_trials):
                    
                    #trial_power = np.zeros((n_channels, n_freq))  # Matrice per la potenza per ogni trial
                    trial_power = np.zeros((n_channels, effective_freq_bins))  # Matrice per la potenza per ogni trial
                    
                    for channel in range(n_channels):
                        
                        # Estrai il segmento del segnale
                        single_channel_data = value[trial, channel, segment_start:segment_end]
                        
                        # Calcola la FFT (solo componenti a frequenza non negativa)
                        # -> https://numpy.org/doc/stable/reference/generated/numpy.fft.rfft.html
                        fft_vals = np.fft.rfft(single_channel_data)
                        
                        # Calcola la potenza spettrale (modulo al quadrato)
                        power = np.abs(fft_vals) ** 2
                        #trial_power[channel] = power
                        trial_power[channel] = power[:effective_freq_bins]
                    
                    # Trasponi in modo che le righe siano le frequenze e le colonne gli elettrodi
                    fft_image = trial_power.T  # shape: (n_freq, n_channels)
                    #print(f"FFT image shape for trial {trial + 1} of condition '{condition_key}': {fft_image.shape}")
                    
                    # Salva l'immagine per il trial corrente
                    #Ogni trial genera una sua immagine della potenza spettrale che viene memorizzata separatamente!
                    condition_results[condition_key]['fft_images'].append(fft_image)
                
                # Gestione delle etichette se presenti                
                #if labels_key in data_dict:
                    #num_labels = data_dict[labels_key] #QUI IN REALTA' SAREBBE GIA' ARRAY NUMPY MA PER IL CODICE CONVIENE RIPORTARLO INT COME LISTA!
                    #'''UNICA MODIFICA IMPORTANTE DA FARE --> RIPORTARE A VALORE DI INT COME LISTA DELLE LABELS (IMPORTANTE PER DOPO!)'''
                    #num_labels = np.asarray(value).tolist()   # trasforma in lista di int
                    #if condition_key in condition_results:
                    #    print(f"\nFound labels for condition \033[1m{labels_key}\033[0m, length: \033[1m{len(num_labels)}\033[0m\n")
                    #    condition_results[condition_key]['labels'].extend(num_labels)
                
            else:
                #print(f"\033[1m{labels_key}\033[0m non è un np.ndarray, non applicare la FFT")
                print(f"\033[1m{condition_key}\033[0m è np.ndarray di labels, salto FFT")    
    
    # Riorganizza i risultati in un dizionario finale
    all_subjects_condition_results = {}
    for condition_key, results in condition_results.items():
        if "_labels" in condition_key or len(results['fft_images']) == 0:
            print(f"\nSkipping \033[1m{condition_key}\033[0m as it is a label or has no valid data.")
            continue
        
        wave_condition_name = f'spectrograms_{condition_key}'
        print(f"\n\nResults for \t\t\t\t\t\033[1m{wave_condition_name}\033[0m:\n\n")
        
        # Ogni trial produce un'immagine 2D di shape (n_freq, n_channels)
        spectrograms_3d = np.array(results['fft_images'])
        all_subjects_condition_results[wave_condition_name] = {
            'spectrograms_3d': spectrograms_3d,
            'labels': results['labels']
        }
        print(f"spectrograms_3d shape for {wave_condition_name}: {spectrograms_3d.shape}")
    
    return all_subjects_condition_results


ora, io mi ero caricato già la lista degli elettrodi nella directory del file da remoto


import pickle

#Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/'

#Caricare l'intero dizionario annidato con pickle
with open(f'{path}EEG_channels_names.pkl', 'rb') as f:
    EEG_channels_names = pickle.load(f)


#Supponendo che EEG_channels_names sia la tua lista originale
EEG_channels_names = [ch.rstrip('.') for ch in EEG_channels_names]
print(EEG_channels_names, end = '')

e veniva fuori questa lista alla fine..

['Fc5', 'Fc3', 'Fc1', 'Fcz', 'Fc2', 'Fc4', 'Fc6', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'Cp5', 'Cp3', 'Cp1', 'Cpz', 'Cp2', 'Cp4', 'Cp6', 'Fp1', 'Fpz', 'Fp2', 'Af7', 'Af3', 'Afz', 'Af4', 'Af8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'Ft7', 'Ft8', 'T7', 'T8', 'T9', 'T10', 'Tp7', 'Tp8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'Po7', 'Po3', 'Poz', 'Po4', 'Po8', 'O1', 'Oz', 'O2', 'Iz']

ora, per costruire la griglia, devo però riprendere questa lista che ho salvato e 
fornirla come input alla funzione che creerà l'immagine della griglia stessa...
questa funzione da creare, dovrà creare la griglia confrontando 

il valore stringa di ogni elettrodo nella lista degli elettrodi del file .xlsx  con il valore stringa degli elettrodi nelle righe del campo 'Electrode' del file .xlsx (forse questo dovrò esser caricato come pandas dataframe alla fine..)

e vedere se c'è corrispondenza. 

Nel caso non ci sia, il nome delle righe del campo 'Electrode' del file .xlsx (ossia le stringhe associate ad ogni elettrodo) devono essere uguali (in caso non lo fossero già) a quelli di ogni elettrodo nella lista degli elettrodi del file .xlsx ..

una volta che son uguali.. da quello che ho capito.. io dovrei praticamente

1) riprendere la struttura dati che ho usato per salvare gli spettrogrammi di ogni ogni soggetto (la funzione di cui sopra) e

2) andare a prelevare rispetto ad ogni elettrodo il valore della sua coordinata sull'asse x ed y (presenti nei campi grid_x e grid_y del file xlsx) ed apporli dentro all'immagine griglia da creare.

3) associare a quell'elettrodo là poi il valore della potenza totale (derivabile semplicemente dal valore che c'è dentro la struttura dati che ho salvato per salvarmi la potenza di ogni elettrodo durante la costruzione dello spettrogramma per ogni canale)...

4) non so si possa al tempo stesso salvare o un'altra griglia od in un semplice dizionario, che invece semplicemente salvi anziché il valore della potenza totale per l'i-esimo elettrodo,  il valore della stringa associata all'elettrodo nella sua relativa combinazione della coordinata x ed y corrispondente, in modo che poi successivamente io sappia per successive analisi capire come ricostruire il nome dell'elettrodo a partire dalle sue coordinate nell'immagine a griglia.

Se vedi, inoltre, nel file .xlsx, ci sono alcune righe del campo 'Electrode' che hanno scritto 'EMPTY'.. ecco, in quei casi lì, non c'è un elettrodo, però in quel set di coordinate corrispondente anziché mettere il valore della potenza (che in quel caso appunto non c'è) ci si dovrà mettere 0 ovviamente (perché appunto non c'è nessun elettrodo)..

tieni presente che questo è stato fatto per creare una griglia "quadrata" con una disposizione in 2D degli elettrodi, dato che poi verrà passata come immagine dentro ad una rete CNN2D_LSTM...

è tutto chiaro più o meno?

##### **CHATGPT**

Perfetto, la struttura del file .xlsx è chiara. Il DataFrame ha:

Colonna 'Electrode' con i nomi dei canali EEG (e anche alcuni placeholder tipo 'EMPTY').

Colonne 'grid_x' e 'grid_y' che definiscono la posizione 2D dell'elettrodo nell’immagine.

✅ Riepilogo chiaro dei passaggi da fare:
1. Allineamento dei nomi
Normalizza i nomi degli elettrodi:

Elimina maiuscole/minuscole (upper() o lower()).

Rimuovi punti o simboli se presenti (rstrip('.') lo fai già).

Verifica che i nomi della tua lista EEG_channels_names combacino con quelli del file .xlsx.

2. Creazione dell'immagine 2D
Per ciascun trial:

Inizializza una griglia 2D vuota di dimensioni coerenti (es. 9x9) → magari come np.zeros((9,9)) o direttamente da grid_x e grid_y normalizzati.

Per ogni riga del .xlsx:

Se 'Electrode' è 'EMPTY': metti 0 in quella posizione (griglia vuota).

Se è un elettrodo valido:

Trova il suo indice nella lista EEG_channels_names per accedere al valore di potenza.

Inserisci quel valore nella posizione (grid_y, grid_x).

🔁 Opzionale: Dizionario per mappare coordinate → nome elettrodo
Per esempio:

python
Copia
Modifica
coordinate_to_electrode = {(y, x): name}
Utile per sapere quale elettrodo si trova in quale posizione della griglia.

🔜 Prossimo passo
Vuoi che ti costruisca io una funzione Python che:

Prende in input:

Il dataframe .xlsx,

La lista EEG_channels_names,

Un vettore di potenze (es. da uno spettrogramma fft_image per un trial),

Restituisce:

L’immagine 2D completa (con 0 dove non ci sono elettrodi),

Un dizionario (y, x) → 'Electrode'?

##### **IO**

Semplicemente in 

1. Allineamento dei nomi

Verifica che i nomi della tua lista EEG_channels_names combacino con quelli del file .xlsx, se nel file .xlsx non combaciano per lo stesso elettrodo allora semplicemente cambiali e rendili come quelli della lista EEG_channels_names 

2. Creazione dell'immagine 2D

Per ciascun trial (? o se può fare prima?) 

Inizializza una griglia 2D vuota, pari al numero di elettrodi della lista EEG_channels_names

Per ogni riga del .xlsx:

Associa ad ogni riga del campo 'Electrode' una posizione dell'elettrodo trovato, in base alle coordinate (ai valori) che si trovano nella stessa riga al campo 'grid_x' e 'grid_y'

N.B. tieni presente che, se ho capito bene, per disporre gli elettrodi nella griglia 2D, l'ordine degli assi è 'asse y' (grid_y) e asse x (grid_x), quindi fai attenzione a prendere prima il valore dell'asse y e poi quello dell'asse x (controllando il nome del campo nel file .xlsx, che sarebbero 'grid_y' e 'grid_x'!).

Ora, fai attenzione che son stati messi anche degli altri valori nel campo 'Electrode' del file .xlsx che non sono degli elettrodi, ma che servivano per costruire l'immagine a griglia 2D come immagine quadrata.
 
Se in 'Electrode' c'è un valore che è uguale ad 'EMPTY': allora metti 0 in quella posizione della griglia 2D (anzichè il valore della potenza del segnale, che ovviamente non c'è perché appunto non c'era nessun elettrodo) 

Altrimenti, se in 'Electrode' c'è un valore che è diverso da  'EMPTY', allora significa che là ci sarà  un elettrodo valido. 
Quindi, allora, a quel punto, trova per quell'elettrodo il suo indice nella lista EEG_channels_names per poter accedere al suo relativo valore di potenza tramite indicizzazione corrispondente all'interno della matrice dei dati, che costituisce le informazioni frequenza x canali del trial EEG corrente iterato e inserisci quel valore di potenza totale in quella posizione (grid_y, grid_x) dell'elettrodo trovato.

in questo modo, potrei avere a partire dalla stessa (o da una copia se è più sensato) struttura dati di partenza (la struttura dati che fornisco in input alla funzione) una griglia 2D quadrata con la disposizione degli elettrodi secondo le sue relative coordinate ed in sua corrispondenza il solo valore della potenza totale per quell'elettrodo nella sua posizione al trial corrente...

In questo modo alla fine, dovrei avere una stessa struttura dati di quella fornita in input, ma dove ogni trial sarebbe una griglia 2D quadrata, con una posizione approssimata degli elettrodi in base alle coordinate fornite e con, a quella posizione corrispondente (tramite il set di coordinate del relativo elettrodo estrapolate dal file xlsx), la potenza totale ottenuta in quel trial corrente per quell'elettrodo...

giusto?

##### **CHATGPT**

Sì, hai perfettamente centrato la logica. Riassumo e confermo i passaggi con massima chiarezza:

✅ OBIETTIVO:
Costruire una nuova rappresentazione dei dati EEG per trial, dove:

Ogni trial → un’immagine 2D quadrata.

Ogni posizione (grid_y, grid_x) nella griglia corrisponde:

a un elettrodo → valore della potenza totale in quel trial.

a un placeholder ‘EMPTY’ → valore 0.

✅ STEP DETTAGLIATI
🔹 1. Allineamento nomi
Carichi il file .xlsx con le colonne Electrode, grid_x, grid_y.

Per ogni riga:

Se Electrode non è in EEG_channels_names, ma è chiaramente lo stesso canale con nomi diversi (es. fc1 vs Fc1), lo sostituisci con il nome giusto.

Se Electrode == 'EMPTY', lo lasci.

🔹 2. Per ogni trial (meglio farlo per trial perché i valori cambiano nel tempo):
Inizializzi una griglia 2D vuota (es. np.zeros((grid_height, grid_width))) → che rappresenta la "mappa" da riempire.

Per ogni riga del .xlsx:

Se Electrode == 'EMPTY':

Aggiungi 0 nella griglia in posizione (grid_y, grid_x).

Se Electrode è valido:

Ottieni l’indice dell’elettrodo nella lista EEG_channels_names.

Preleva il valore corrispondente nella matrice spettrogramma (frequenze x canali).

Calcola la potenza totale per quell’elettrodo: ad esempio con np.sum(fft_image[:, idx]).

Inserisci il valore in (grid_y, grid_x).

✅ OUTPUT
Per ogni condizione sperimentale avrai:

Una struttura del tipo:

python
Copia
Modifica
{
  'condizione_X': {
      'images_2d': [griglia_trial1, griglia_trial2, ...],  # shape es. (n_trials, 9, 9)
      'labels': [...],
  },
  ...
}
✅ BONUS (facoltativo)
Se vuoi mantenere una mappa inversa:

python
Copia
Modifica
coordinate_to_electrode = {(grid_y, grid_x): 'Cz', ...}
così puoi risalire all’elettrodo da una posizione nella griglia.

🔜 Pronto a scriverti la funzione?
Vuoi che ti costruisca subito:

una funzione convert_fft_to_2d_grid() che:

prende uno fft_image, il DataFrame .xlsx, la lista EEG_channels_names,

e restituisce la griglia 2D?

Così possiamo anche usarla in un ciclo per costruire tutte le immagini di tutti i trial. Fammi un cenno e procedo.

##### **IO**

Vorrei che prendesse la struttura dati che gli fornisco in input, il file .xlxs e la lista EEG_channels_names...

tipo così


#Caricare l'intero dizionario annidato con pickle
path_data = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

#Caricare file xlsx con pickle
path_xlsx = '/home/stefano/'

#Caricare file canali EEG con pickle
path_channels = '/home/stefano/EEGMMIDB_Analyses/

#Caricare l'intero dizionario annidato con pickle
with open(f'{path_data}new_all_subjects_condition_spectrograms_task_1.pkl', 'rb') as f:
    new_all_subjects_condition_spectrograms_task_1 = pickle.load(f)
    
#Caricare l'intero dizionario annidato con pickle
with open(f'{path_xlsx}EEG_grid.xlsx.pkl', 'rb') as f:
    EEG_file = pickle.load(f)
    
#Caricare l'intero dizionario annidato con pickle
with open(f'{path_channels}EEG_channels_names.pkl.pkl', 'rb') as f:
    EEG_channels = pickle.load(f)
    

Da quella struttura dati di ingresso, credo convenga che

1) si faccia una copia 

2) e poi costruisca per ogni soggetto e per ogni trial la griglia 2D con le coordinate e la potenza totale in sua corrispondenza come ti ho detto per ogni trial..


Però in modo che semplicemente poi la struttura dati ritornata in output sia praticamente la stessa  di quella di partenza, con la differenza che, ad ogni trial, io abbia la griglia 2D quadrata, con gli elettrodi posti nelle coordinate corrispondenti ed il valore della potenza totale in sua corrispondenza


##### **Implementazione Immagine Griglia 2D - Step Creazione Set Canali Giusto**

In [ ]:
import pickle 
import pandas as pd

# Caricare l'intero dizionario annidato con pickle
path_data = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

# Caricare file canali EEG con pickle
path_channels = '/home/stefano/EEGMMIDB_Analyses/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path_data}new_all_subjects_condition_spectrograms_task_1.pkl', 'rb') as f:
    new_all_subjects_condition_spectrograms_task_1 = pickle.load(f)
    
# Caricare l'intero dizionario annidato con pickle
with open(f'{path_channels}EEG_channels_names.pkl', 'rb') as f:
    old_EEG_channels = pickle.load(f)
    
# Caricare file xlsx con pickle
path_xlsx = '/home/stefano/EEG_grid.xlsx'

# Caricare l'intero dizionario annidato con pickle
#with open(f'{path_xlsx}EEG_grid.xlsx', 'rb') as f:
#    EEG_file = pd.read_excel(f)

# Caricamento del file in un DataFrame
EEG_file = pd.read_excel(path_xlsx)


In [ ]:
#EEG_file

In [ ]:
len(old_EEG_channels)

In [ ]:
'''

Elettrodi del Dataset Originario

https://physionet.org/content/eegmmidb/1.0.0/ - Sezione Montage


Il montaggio specifica chiaramente che:

Sono esclusi: Nz, F9, F10, FT9, FT10, A1, A2, TP9, TP10, P9, P10
'''

print(old_EEG_channels, end = '')

In [ ]:
len(old_EEG_channels)

In [ ]:
# supponendo che EEG_channels_names sia la tua lista originale


old_EEG_channels = [ch.rstrip('.') for ch in EEG_channels]


# supponendo che EEG_channels_names sia la tua lista originale
#EEG_channels_names = [ch.rstrip('.') for ch in EEG_channels_names]
#print(EEG_channels_names, end = '')


print(old_EEG_channels, end = '')


In [ ]:
''' 
Rendo la lista elettrodi trovata nel file brain vision qui sopra come quella del file xlsx
'''


#Questo è l'ordine sul file excel, escludendo al momento i valori EMPTY...

'''
filtered_electrodes_xlsx_file = [
    'AF7', 'Fp1', 'AFz', 'Fpz', 'Fp2', 'AF8',
    'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8',
    'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8',
    'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8',
    'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8',
    'PO7', 'PO3', 'POz', 'PO4', 'PO8',
    'O1', 'Oz', 'O2',
    'Iz']
    
'''

#Ora, invece, vado a ri-ordinare la tua lista originale secondo la stessa sequenza di riferimento del file brain vision ossia questa
#(i.e., old_EEG_channels)

'''
['Fc5', 'Fc3', 'Fc1', 'Fcz', 'Fc2', 'Fc4', 'Fc6', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'Cp5', 'Cp3', 'Cp1', 'Cpz', 'Cp2', 'Cp4', 
'Cp6', 'Fp1', 'Fpz', 'Fp2', 'Af7', 'Af3', 'Afz', 'Af4', 'Af8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'Ft7', 'Ft8', 'T7', 'T8', 'T9',
'T10', 'Tp7', 'Tp8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'Po7', 'Po3', 'Poz', 'Po4', 'Po8', 'O1', 'Oz', 'O2', 'Iz']
'''

#ma mantenendo la capitalizzazione originale della tua lista creata sul file excel

'''
filtered_electrodes_xlsx_file = [
    'AF7', 'Fp1', 'AFz', 'Fpz', 'Fp2', 'AF8',
    'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8',
    'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8',
    'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8',
    'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8',
    'PO7', 'PO3', 'POz', 'PO4', 'PO8',
    'O1', 'Oz', 'O2',
    'Iz']
'''

ordered_electrodes = [
    'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6',
    'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6',
    'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6',
    'Fp1', 'Fpz', 'Fp2',
    'AF7', 'AFz', 'AF8',
    'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FT8',
    'T7', 'T8',
    'TP7', 'TP8',
    'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8',
    'PO7', 'PO3', 'POz', 'PO4', 'PO8',
    'O1', 'Oz', 'O2',
    'Iz'
]

In [ ]:
import pickle

path = '/home/stefano/EEGMMIDB_Analyses/'

# Salvare l'intero dizionario annidato con pickle
with open(f'{path}ordered_electrodes.pkl', 'wb') as f:
    pickle.dump(ordered_electrodes, f)

##### **IO**

In [ ]:
Mi spieghi meglio questo passaggio?

✅ 1) Perché np.sum(fft_image, axis=0) e non un ciclo?
Hai scritto:

“per ogni canale c, la potenza totale è np.sum(fft_image[:, c])”

Sì, questo è vero.
Ma invece di fare:

python
Copia
Modifica
total_power = []
for c in range(n_channels):
    total_power.append(np.sum(fft_image[:, c]))
puoi usare vectorizzazione con NumPy:

python
Copia
Modifica
total_power = np.sum(fft_image, axis=0)  # shape: (n_channels,)
È esattamente equivalente — solo molto più veloce ed efficiente.
Quindi:

fft_image[:, c] → fetta per un solo canale

np.sum(fft_image, axis=0) → somma di tutti i canali in una sola riga

✅ Conclusione: non è una contraddizione, è solo una scrittura più efficiente e direttamente vettoriale.


##### **CHATGPT**

In [ ]:
ChatGPT ha detto:
Sì, te lo spiego con un esempio chiaro per visualizzare il concetto.

✅ CONTEXTO
Hai una matrice fft_image di forma (n_freqs, n_channels), ad esempio:

python
Copia
Modifica
fft_image.shape == (45, 64)  # 45 frequenze, 64 canali
Ogni colonna rappresenta un elettrodo (canale), e ogni riga una frequenza.

🎯 OBIETTIVO
Per ogni canale (cioè per ogni colonna), vuoi calcolare:

La somma dei valori su tutte le frequenze

In altre parole:
Per ogni canale c, vuoi:

python
Copia
Modifica
np.sum(fft_image[:, c])  # somma verticale, lungo le frequenze
🐌 MODO LENTO (con ciclo for)
python
Copia
Modifica
n_channels = fft_image.shape[1]
total_power = []

for c in range(n_channels):
    total_power.append(np.sum(fft_image[:, c]))

# total_power sarà una lista lunga 64 (n_channels)
Questo funziona, ma è lento e meno elegante.

🚀 MODO VELOCE (vettoriale con NumPy)
python
Copia
Modifica
total_power = np.sum(fft_image, axis=0)
Questo fa la stessa cosa, ma:

Lo fa tutto in un colpo solo

È scritto in una sola riga

Usa le ottimizzazioni interne di NumPy (molto più veloce in pratica)

🧠 Come funziona axis=0?
axis=0 significa: somma lungo le righe (cioè sulle frequenze)

Risultato: ottieni un array con una somma per ogni colonna → un valore per ogni canale

📊 Visualizzazione semplice:
Immagina fft_image così:

markdown
Copia
Modifica
      Canale_1  Canale_2  Canale_3
Freq1     2         4         6
Freq2     3         5         7
Freq3     1         2         3
Se fai:

python
Copia
Modifica
np.sum(fft_image, axis=0)
Ottieni:

python
Copia
Modifica
[2+3+1, 4+5+2, 6+7+3] = [6, 11, 16]
Esattamente come se avessi iterato su ogni colonna con un ciclo.

✅ Conclusione
np.sum(fft_image, axis=0) è il modo idiomatico, efficiente e leggibile per ottenere la potenza totale per canale a partire da una matrice frequenza x canali.

Vuoi che ti mostri anche l'equivalenza con codice pratico in Python per verificarlo con np.allclose?

##### **IO**

Sì, mostrami anche l'equivalenza con codice pratico in Python per verificarlo con np.allclose?

##### **CHATGPT**

In [ ]:
import numpy as np

# Simula una matrice FFT: 45 frequenze x 64 canali
np.random.seed(42)
fft_image = np.random.rand(45, 64)

# Metodo 1: con ciclo (manuale)
manual_power = []
for c in range(fft_image.shape[1]):
    manual_power.append(np.sum(fft_image[:, c]))
manual_power = np.array(manual_power)

# Metodo 2: con vectorizzazione
vectorized_power = np.sum(fft_image, axis=0)

# Verifica che i due metodi siano equivalenti
are_equal = np.allclose(manual_power, vectorized_power)

manual_power[:5], vectorized_power[:5], are_equal

Ecco il risultato del confronto tra i due metodi:

🎯 I primi 5 valori (identici):
Metodo manuale (ciclo):

csharp
Copia
Modifica
[23.0585, 25.0655, 20.1955, 20.4349, 21.2330]
Metodo vettoriale (con np.sum(axis=0)):

csharp
Copia
Modifica
[23.0585, 25.0655, 20.1955, 20.4349, 21.2330]
✅ Verifica finale:
python
Copia
Modifica
np.allclose(manual_power, vectorized_power) → True
Quindi i due metodi sono numericamente equivalenti, ma il secondo è:

più elegante

più veloce

più adatto a dati grandi

Se vuoi, posso anche mostrarti quanto tempo impiegano entrambi su matrici più grandi (%timeit). Vuoi vedere anche questo benchmark?

##### **Implementazione Immagine Griglia 2D - Step Creazione Griglia vera e propria**

1) **Creazione Griglia Dati EEG + Griglia Labels Coordinate Elettrodiche**
2) **Salvataggio Griglia Dati EEG + Griglia Labels Coordinate Elettrodiche**

In [ ]:
import pickle 
import pandas as pd

# Caricare l'intero dizionario annidato con pickle
path_data = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

# Caricare file canali EEG con pickle
path_channels = '/home/stefano/EEGMMIDB_Analyses/'

# Caricare l'intero dizionario annidato con pickle
with open(f'{path_data}new_all_subjects_condition_spectrograms_task_1.pkl', 'rb') as f:
    new_all_subjects_condition_spectrograms_task_1 = pickle.load(f)
    
# Caricare l'intero dizionario annidato con pickle
with open(f'{path_channels}ordered_electrodes.pkl', 'rb') as f:
    EEG_channels = pickle.load(f)
    
    
# Caricare file xlsx con pickle
path_xlsx = '/home/stefano/EEGMMIDB_Analyses/EEG_grid.xlsx'

# Caricamento del file in un DataFrame
EEG_file = pd.read_excel(path_xlsx)

In [ ]:
#new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100'].keys()
new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100']['spectrograms_3d'].shape

In [ ]:
import pandas as pd
import numpy as np
import pickle

'''

La funzione aggiornata convert_fft_images_to_2d_grids() ora:

Allinea correttamente i nomi dei canali nel file .xlsx alla lista EEG_channels_names, 
tenendo conto della capitalizzazione.
Segnala eventuali nomi mancanti con un messaggio di warning.
Genera una griglia 2D per ogni trial, con le potenze totali posizionate correttamente.


L’immagine da cui si ispira la generazione dell'immagine 2D è chiaramente una griglia 9x9,
con gli elettrodi disposti in celle regolari e alcune caselle vuote (placeholder).

✅ Sì, la griglia costruita dal codice corrisponde correttamente a questa rappresentazione se:

Usi il file .xlsx che assegna le coordinate (grid_y, grid_x) ad ogni elettrodo (inclusi gli 'EMPTY').
Imposti grid_shape = (9, 9) o lo ricavi automaticamente come ti ho suggerito.


✅ Caso di default:
bandpass=None

👉 La funzione userà:

total_power_per_channel = np.sum(fft_image, axis=0)

Quindi calcolerà la potenza totale sommando su tutte le frequenze disponibili dello spettro.

✅ Caso con filtraggio in banda:
bandpass=(4, 7)

👉 Verranno usati solo i bin di frequenza compresi tra 4 e 7 Hz, estraendo:


band_power = fft_image[freq_mask, :]
total_power_per_channel = np.sum(band_power, axis=0)

In questo modo ottieni la potenza nella banda specificata, per ogni canale.

    ✅ Cosa fa internamente
    Calcola le frequenze corrispondenti ai bin della rfft con:

    freqs = np.fft.rfftfreq(n_fft_points, d=1/fs)
    Applica un filtro logico per tenere solo i bin della banda desiderata.

    Somma solo quelle righe (frequenze) in fft_image[freq_mask, :].


'''


def convert_fft_images_to_2d_grids_all_freqs(
    original_data: dict,
    eeg_grid_df: pd.DataFrame,
    eeg_channels_names: list,
    grid_shape: tuple = (9, 9),
    fs: int = 160,
    n_fft_points: int = 640
    #bandpass: tuple = None  # es. (4, 7)
):
    """
    Converte spettrogrammi frequenza x canali in immagini 2D (griglia quadrata) per banda di frequenza, dove 
    
    1) ogni trial è una griglia 9x9x N_bande con i valori di potenza totale per elettrodo.
    2) la posizione (y, x) viene determinata dalle coordinate nel file .xlsx e il valore
    
    è la potenza totale rispetto alla banda di frequenza per ciascun elettrodo.
    
    Inoltre, restituisce una griglia fissa con le etichette degli elettrodi.
    
    Args:
        original_data (dict): struttura come new_all_subjects_condition_spectrograms_task_1
        eeg_grid_df (pd.DataFrame): contiene colonne ['Electrode', 'grid_x', 'grid_y']
        eeg_channels_names (list): lista dei nomi canali EEG, già in forma compatibile
        grid_shape (tuple): dimensioni della griglia finale (default 9x9)
        fs (int): frequenza di campionamento (Hz)
        n_fft_points (int): numero di punti FFT (lunghezza del segmento analizzato)
        bandpass (tuple): banda di frequenze (min, max), es. (4, 7); se None → totale
        .

    Returns:
        tuple:
            - dict: struttura con immagini 2D per ogni trial
            - np.ndarray: griglia 2D con etichette degli elettrodi (stringhe)
        
    """
    
    # Definizione delle bande EEG
    bands = {
        #'delta': (0.5, 4),
        'delta': (1, 4),
        'theta': (4, 8),
        'alpha': (8, 13),
        'beta':  (13, 30),
        'gamma': (30, 45)
    }
    
    updated_data = {}
                
    # Crea una copia del DataFrame per sicurezza
    df_copy = eeg_grid_df.copy()
    df_copy['Electrode'] = df_copy['Electrode'].astype(str).str.strip()

    # Controllo elettrodi non trovati
    unique_electrodes = set(df_copy['Electrode'].unique())
    unique_electrodes.discard('EMPTY')
    not_found = unique_electrodes - set(eeg_channels_names)
    
    if not_found:
        print("\n🚨 Elettrodi nel file Excel ma non trovati nei dati EEG:")
        print(sorted(not_found))
    
    # Griglia delle etichette
    
    label_grid = np.full(grid_shape, '', dtype=object)
    
    '''
    Dal contenuto del file Excel è evidente che le colonne grid_x e grid_y contengono coordinate normalizzate nell'intervallo da 0.0 a 1.0.
    Queste coordinate sono suddivise con uno step di 1/8 = 0.125, coerente con una griglia 9x9 (quindi 9 posizioni da 0.0 a 1.0).
    
    Cosa significa
    
    Per posizionare correttamente gli elettrodi su una griglia 2D di shape (9, 9):
    
    Devi moltiplicare le coordinate normalizzate per grid_size - 1 e convertirle a interi, ossia così:
    
    x = int(round(row['grid_x'] * (grid_shape[1] - 1)))
    y = int(round(row['grid_y'] * (grid_shape[0] - 1)))
    
    N.B.
    
    Se vuoi costruire una matrice 2D indicizzata (es. grid[y, x]) per usarla come immagine 2D o input per modelli CNN:
    
    → Devi usare indici interi
    
    Quindi sì, devi fare:
    
    x = int(round(row['grid_x'] * (grid_shape[1] - 1)))
    y = int(round(row['grid_y'] * (grid_shape[0] - 1)))
    
    Questo perché una matrice NumPy grid_2d[y, x] non può accettare float come indici    
    '''
    
    # Mappa che associa ad ogni elettrodo la sua coordinate intere nella griglia
    electrode_grid_map = {}  
  
    for _, row in df_copy.iterrows():
        elec = row['Electrode']
        
        '''ATTENZIONE CHE VA MODIFICATO SIA PER CREAZIONE MAPPA DEGLI ELETTRODI'''
        #y, x = int(row['grid_y']), int(row['grid_x'])
        
        x = int(round(row['grid_x'] * (grid_shape[1] - 1)))
        y = int(round(row['grid_y'] * (grid_shape[0] - 1)))
    
        label_grid[y, x] = '' if elec == 'EMPTY' else elec
        print(f"Elettrodo: {elec} → (y = {y}, x = {x})")
        
        '''
        label_grid → serve per etichettare visivamente le posizioni (es. nel plot, mostrare "Fz", "Cz", ecc.).
        electrode_grid_map → mappa solo gli elettrodi reali, e ti dice dove piazzare/recuperare i valori nella griglia.
        grid_2d (Griglia numerica) → in corrispondenza degli EMPTY, si assegnano valori 0 (come già fai con grid_2d[y, x] = 0).
        
        
        Quindi: vanno tenuti gli EMPTY nella mappa?
        
        Risposta breve:
        
        No, non vanno tenuti in electrode_grid_map, 
        perché questa serve solo a mappare i canali effettivi (cioè quelli per cui hai dati).
        
        Sì, vanno mantenuti in grid_2d e label_grid, 
        perché servono a preservare la forma della griglia (9x9) e garantire che:
        
        la CNN o i plot ricevano input a griglia completa (senza buchi),
        le posizioni senza elettrodi siano rappresentate (con valore 0),
        la normalizzazione congiunta consideri anche questi zeri, che sono parte delle immagini.
        
        ✅ Cosa fare (in sintesi):
        
        Mantieni gli EMPTY in label_grid, grid_2d, e nelle heatmap → così la griglia resta coerente.
        Non includerli in electrode_grid_map → perché non rappresentano un elettrodo reale.
        Quando normalizzi o plotti, non ignorare gli zeri: sono significativi, servono a evitare buchi nella griglia.

        '''
        
        if elec != 'EMPTY':
            electrode_grid_map[elec] = (y, x)
    
    # Calcola i bin delle frequenze solo una volta se necessario
    
    # https://numpy.org/doc/2.1/reference/generated/numpy.fft.rfftfreq.html
    
    '''
    Nella tua funzione create_spectrograms_from_fft_images_of_electrodes() tagli già lo spettro FFT
    ai primi max_freq_bins (es. 45), quindi l'output finale ha solo 45 frequenze.
    
    Di conseguenza, la tua funzione non può usare direttamente:
    
    freqs = np.fft.rfftfreq(n_fft_points, d=1/fs)
    
    per costruire le maschere, perché fft_image.shape[0] == 45, mentre np.fft.rfftfreq(640, d=1/160) ti dà 321 frequenze!
    
    Soluzione consigliata: usare solo i primi max_freq_bins del vettore delle frequenze
    
    Passaggi:
    
    1) Genera solo i primi max_freq_bins del vettore delle frequenze:
    
    max_freq_bins = original_data[list(original_data.keys())[0]]['spectrograms_3d'].shape[1]
    
    all_freqs = np.fft.rfftfreq(n_fft_points, d=1/fs)[:max_freq_bins]
    
    
    2) Costruisci le maschere band_masks in base a queste:
    
    band_masks = {band: (all_freqs >= f_range[0]) & (all_freqs <= f_range[1]) for band, f_range in bands.items()}
    '''
    
    max_freq_bins = original_data[list(original_data.keys())[0]]['spectrograms_3d'].shape[1]
    
    '''
    Funzione --> np.fft.rfftfreq   https://numpy.org/doc/2.1/reference/generated/numpy.fft.rfftfreq.html
    
    
    Questa funzione serve per calcolare le frequenze associate ai coefficienti della trasformata di Fourier reale (RFFT), 
    ovvero numpy.fft.rfft
    
    Parametri:
    n (int):
    Il numero di campioni del segnale in input. È la lunghezza del segnale su cui verrà calcolata la trasformata.
    
    d (float, opzionale):
    Il passo temporale tra due campioni (cioè 1 / frequenza di campionamento). Il valore di default è 1.0.
    
    Quindi Serve proprio per calcolare le frequenze corrispondenti ai coefficienti della trasformata di Fourier reale (RFFT), 
    in modo da sapere a quali frequenze corrispondono i valori presenti nello spettrogramma.
    
    In questo caso, 
    la trasformata di Fourier è stata fatta su finestre di lunghezza n_fft_points = 640, a frequenza di campionamento fs = 160 Hz.
    
    Nello specifico, 
    
    produce un array all_freqs contenente le frequenze in Hz associate ai bins FFT
    (cioè agli "indici" lungo l'asse delle frequenze nel tuo spettrogramma)
    Ho poi tagliato questo array per tenerne solo i primi max_freq_bins valori (es. 45), 
    perché il tuo spettrogramma è stato già troncato a 45 frequenze.
    
    Perché è importante?
    
    Il tuo spettrogramma spectros ha shape:
    
    (n_trials, n_freqs, n_channels)
    
    E n_freqs è tipicamente molto più piccolo di n_fft_points // 2 + 1. Quindi hai bisogno di sapere:
    👉 "A quale frequenza reale corrisponde la riga i di questo spettrogramma tagliato?"
    
    La funzione rfftfreq ti dà questa corrispondenza.
    
    
    🧩 Uso nella tua funzione
    
    Hai un dizionario di bande EEG come:
    
    bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    ...
    }
    
    Poi costruisci una maschera per ciascuna banda:
    
    band_masks = {band: (all_freqs >= f_range[0]) & (all_freqs <= f_range[1]) for band, f_range in bands.items()}
    Queste maschere (True/False) ti dicono quali righe del tuo spettrogramma corrispondono a una certa banda di frequenza.
    
    Poi puoi fare:
    
    band_power = fft_image[mask, :]  # somma solo le righe corrispondenti alla banda e 
    ottenere la potenza totale nella banda per ogni canale EEG
    
    Quindi in sostanza
    
    La riga:
    
    all_freqs = np.fft.rfftfreq(n_fft_points, d=1/fs)[:max_freq_bins]
    
    serve a trovare i valori in Hz associati ai coefficienti dello spettro.
    Senza questa riga, non potresti sapere quali righe dello spettrogramma appartengono a quale banda (delta, theta, ecc.),
    perché lo spettrogramma è indicizzato per riga (0, 1, 2, ...), non in Hz.
    '''
    
    #Sintesi: 
    #'all_freqs' serve per trovare i coefficienti della FFT associati allo specifico range di frequenze massime 
    #a cui era già stato filtrato il mio segnale (45Hz) 
    #Dopodiché, poi faccio una sotto-selezione di questi coefficienti rispetto alla specifica banda, 
    #con delle maschere booleane che mi sanno estrarre l'indice dei relativi coefficienti associati a quella banda di frequenza corrente 
    #e da quelle poi si crea le griglie con la potenza relativa alla banda di frequenza estratta
    
    all_freqs = np.fft.rfftfreq(n_fft_points, d=1/fs)[:max_freq_bins]
    
    # Maschere di frequenza per ciascuna banda
    
    #freq_mask = (freqs >= bandpass[0]) & (freqs <= bandpass[1])
    band_masks = {band: (all_freqs >= f_range[0]) & (all_freqs <= f_range[1]) for band, f_range in bands.items()}
    
    for condition, cond_data in original_data.items():
        spectros = cond_data['spectrograms_3d']  # shape: (n_trials, n_freqs, n_channels)
        labels = cond_data['labels']
        n_trials, n_freqs, n_channels = spectros.shape

        images_3d = []
        
        '''
        Nella tua funzione FFT:
        
        fft_image = trial_power.T  # shape: (n_freq, n_channels)
        
        Ogni fft_image rappresenta la potenza spettrale (cioè |FFT|²) per:
        Righe = frequenze (es. 45 bins)
        Colonne = canali EEG (elettrodi)
        
        Quindi, per ogni canale c, la potenza totale è:
        
        total_power_c = np.sum(fft_image[:, c])
        
        Cosa fa la funzione convert_fft_images_to_2d_grids?
        
        total_power_per_channel = np.sum(fft_image, axis=0)  # somma lungo l'asse delle frequenze
        
        Quindi estrae la potenza totale aggregando i valori di frequenza.
        Questo non è un calcolo superfluo, perché:
        
        Il dato originale fft_image ha frequenza x canale
        L’immagine griglia vuole invece 1 solo valore per canale (asse z), da associare alla posizione (x, y)
        
        In sintesi:
        
        ✔ La potenza totale non è esplicitamente salvata, ma è immediatamente disponibile e si ottiene con una semplice np.sum(..., axis=0)
        ✔ Quello che fa la funzione è semplicemente riassumere le righe (frequenze) → potenza totale per elettrodo
        
        ❌ Non c’è duplicazione inutile: questo passaggio è necessario per ridurre 2D (f x ch) → 1D (ch)

        '''
        
        for trial_idx in range(n_trials):
            fft_image = spectros[trial_idx]  # (n_freqs, n_channels)
            
            band_grids = []

            # Banda specifica
            
            for band_name, mask in band_masks.items():
                band_power = fft_image[mask, :]  # (n_freqs_in_band, n_channels)
            
                total_power_per_channel = np.sum(band_power, axis=0)
         
                grid_2d = np.zeros(grid_shape)

                for _, row in df_copy.iterrows():
                    electrode = row['Electrode']

                    '''ATTENZIONE CHE VA MODIFICATO ANCHE QUI, PER L'ASSEGNAZIONE DELLA POTENZA TOTALE AL RELATIVO ELETTRODO!'''
                    #y, x = int(row['grid_y']), int(row['grid_x'])

                    x = int(round(row['grid_x'] * (grid_shape[1] - 1)))
                    y = int(round(row['grid_y'] * (grid_shape[0] - 1)))

                    if electrode == 'EMPTY':
                        grid_2d[y, x] = 0

                    elif electrode in eeg_channels_names:
                        ch_index = eeg_channels_names.index(electrode)
                        grid_2d[y, x] = total_power_per_channel[ch_index]
                    else:
                        grid_2d[y, x] = 0

            
                band_grids.append(grid_2d)
            
            # Stack delle 5 griglie su ultima dimensione → (9, 9, 5)
            trial_tensor = np.stack(band_grids, axis=-1)
            images_3d.append(trial_tensor)

        '''ATTENZIONE CHE QUI CONVIENE TENERE LA STESSA CHIAVE DEL DIZIONARIO - spectrograms_3d (anziché  spectrograms_2d)-
        PER LE SUCCESSIVE ANALISI, ANCHE SE SI TRATTA DELLA GRIGLIA 2D ASSOCIATA AD OGNI TRIAL'''
        
        updated_data[condition] = {
            
            #spectrograms_2d
            'spectrograms_3d': np.array(images_3d), #(n_trials, 9, 9, 5)
            'labels': labels
        }

    return updated_data, label_grid, electrode_grid_map


In [ ]:
'''
new_all_subjects_condition_spectrograms_task_1 = convert_fft_images_to_2d_grids(original_data: dict,
                                                                                eeg_grid_df: pd.DataFrame,
                                                                                eeg_channels_names: list,
                                                                                grid_shape: tuple = (9, 9),
                                                                                fs: int = 160,
                                                                                n_fft_points: int = 640,
                                                                                bandpass: tuple = None  # es. (4, 7)
'''                                                                                

new_all_subjects_condition_spectrograms_task_1, label_grid, electrode_grid_map = convert_fft_images_to_2d_grids_all_freqs(new_all_subjects_condition_spectrograms_task_1,
                                                                                EEG_file,
                                                                                EEG_channels,
                                                                                grid_shape = (9, 9),
                                                                                fs = 160,
                                                                                n_fft_points = 640
                                                                                )  # or band-specific (4, 7)

In [ ]:
new_all_subjects_condition_spectrograms_task_1['spectrograms_rest_100']['spectrograms_3d'].shape

In [ ]:
power_values = new_all_subjects_condition_spectrograms_task_1["spectrograms_left_fist_100"]["spectrograms_3d"][0][:, :, 0]
print("Min:", power_values.min(), "Max:", power_values.max())

#new_all_subjects_condition_spectrograms_task_1.keys()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_trial_grid_multiband(trial_tensor, band_names=None, title_prefix="Trial", cmap="viridis"):
    """
    Visualizza le 5 bande di un trial come 5 heatmap (una per banda),
    usando la stessa scala di colori per tutte le bande.
    
    Args:
        trial_tensor (np.ndarray): (9, 9, 5) → griglia 2D + bande
        band_names (list): nomi delle bande (es. ['delta', 'theta', ...])
        title_prefix (str): prefisso per il titolo
        cmap (str): colormap da usare
    """
    n_bands = trial_tensor.shape[-1]
    
    if band_names is None:
        band_names = [f"Band {i}" for i in range(n_bands)]

    # Calcola min e max globali per normalizzare la scala colori
    vmin = np.min(trial_tensor)
    vmax = np.max(trial_tensor)

    fig, axes = plt.subplots(1, n_bands, figsize=(3*n_bands, 3))
    fig.suptitle(f"{title_prefix}: Griglie di potenza per banda", fontsize=14)

    for i in range(n_bands):
        ax = axes[i]
        im = ax.imshow(trial_tensor[:, :, i], cmap=cmap, origin='upper', vmin=vmin, vmax=vmax)
        ax.set_title(band_names[i])
        ax.axis('off')
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()


In [ ]:
trial = new_all_subjects_condition_spectrograms_task_1["spectrograms_left_fist_100"]["spectrograms_3d"][0]  # (9, 9, 5)
band_names = ['delta', 'theta', 'alpha', 'beta', 'gamma']
plot_trial_grid_multiband(trial, band_names)

In [ ]:
#electrode_grid_map

In [ ]:
import pickle
import os
base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
# Salvare l'intero dizionario annidato con pickle
with open(f'{base_path}new_all_subjects_condition_spectrograms_task_1_2d_grid_multiband.pkl', 'wb') as f:
    pickle.dump(new_all_subjects_condition_spectrograms_task_1, f)

In [ ]:
import pickle

path = '/home/stefano/EEGMMIDB_Analyses/'

# Salvare l'intero dizionario annidato con pickle
with open(f'{path}final_EEG_electrodes_grid.pkl', 'wb') as f:
    pickle.dump(label_grid, f)
    

# Salvare l'intero dizionario annidato con pickle
with open(f'{path}electrode_grid_map.pkl', 'wb') as f:
    pickle.dump(electrode_grid_map, f)

##### **Implementazione Immagine Griglia 2D** 

1) **Check Posizionamento Etichette degli Elettrodi su Griglia 2D**
2) **Check Valore 0 a Etichette degli Elettrodi Vuoti (i.e., gli EMPTY) su Griglia 2D**

In [ ]:
label_grid

In [ ]:
'''

Se vuoi costruire una matrice 2D indicizzata (es. grid[y, x]) per usarla come immagine 2D o input per modelli CNN:

→ Devi usare indici interi

Quindi sì, devi fare:

x = int(round(row['grid_x'] * (grid_shape[1] - 1)))
y = int(round(row['grid_y'] * (grid_shape[0] - 1)))

Questo perché una matrice NumPy grid_2d[y, x] non può accettare float come indici
'''

import matplotlib.pyplot as plt
import numpy as np

def plot_eeg_grid_labels(label_grid: np.ndarray, title: str = "Posizione elettrodi sulla griglia EEG"):
    """
    Visualizza una griglia 2D con le etichette degli elettrodi (senza potenza).
    
    Args:
        label_grid (np.ndarray): Griglia 2D (grid_y x grid_x) con stringhe degli elettrodi.
        title (str): Titolo del grafico.
    """
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])

    # Disegna la griglia e annota le etichette
    for y in range(label_grid.shape[0]):
        for x in range(label_grid.shape[1]):
            label = label_grid[y, x]
            rect = plt.Rectangle((x, y), 1, 1, fill=False, edgecolor='gray', lw=1)
            ax.add_patch(rect)
            if label != '':
                ax.text(x + 0.5, y + 0.5, label, ha='center', va='center', fontsize=8)

    ax.set_xlim(0, label_grid.shape[1])
    ax.set_ylim(label_grid.shape[0], 0)  # Inverti y per avere origine in alto a sinistra
    plt.tight_layout()
    plt.show()


In [ ]:
plot_eeg_grid_labels(label_grid)

In [ ]:
'''

Come accedere a tutte le posizioni EMPTY
Puoi usare la label_grid, che ha lo stesso shape (9, 9) della matrice di potenza grid_2d.
'''

empty_mask = (label_grid == '')
empty_positions = np.argwhere(empty_mask)

In [ ]:
'''
Questo mostrerà tutti gli slot EMPTY e la relativa potenza (che sarà 0 se il tutto è coerente)
'''

for y, x in empty_positions:
    print(f"EMPTY @ (y={y}, x={x}) → Potenza: {power_values[y, x]}")

##### **Implementazione Immagine Griglia 2D** 

1) **Check Posizionamento Etichette degli Elettrodi su Griglia 2D**
2) **Check Valore Potenza Totale ad Etichette degli Elettrodi VERI su Griglia 2D**

In [ ]:
'''

✅ plot_eeg_grid_image(trial_2d_image)
Mostra visivamente una griglia 2D EEG generata per un trial.

Puoi usarla ad esempio così:

plot_eeg_grid_image(new_data_2d['condition_name']['spectrograms_2d'][0])

Oppure, per iterare su più condizioni e trial:

for cond, data in new_data_2d.items():
    plot_eeg_grid_image(data['spectrograms_2d'][0], title=f"Condizione: {cond} - Trial 0")
    break  # Rimuovi il break per vederle tutte

Fammi sapere se vuoi anche:

Annotare gli elettrodi sulla griglia.

Salvare le immagini .png.

Ottenere un confronto tra immagine generata e immagine esempio (EEG_grid_map). 

❗Nota:
Se hai valori negativi o zero nella griglia, la scala logaritmica li ignorerà o li "appiattirà" a un minimo (eps) per evitare errori.

Se vuoi forzare una normalizzazione identica per tutti i trial, puoi impostare vmin e vmax manualmente.

Fammi sapere se vuoi anche:

Annotare i valori numerici in ciascuna cella

Plottare la differenza tra due condizioni

Salvare direttamente i plot come file .png


'''

import matplotlib.pyplot as plt
import numpy as np

def plot_eeg_grid_image(trial_2d_image: np.ndarray, title: str = "EEG Power Grid (Trial)"):
    """
    Visualizza un'immagine 2D (griglia EEG) di un singolo trial.

    Args:
        trial_2d_image (np.ndarray): Array 2D (grid_y x grid_x) con i valori di potenza.
        title (str): Titolo del grafico.
    """
    plt.figure(figsize=(6, 6))
    plt.imshow(trial_2d_image, cmap='viridis', origin='upper')
    plt.colorbar(label="Potenza Totale")
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    

In [ ]:
#▶️ 1. Per plottare il primo trial di una condizione:
plot_eeg_grid_image(new_all_subjects_condition_spectrograms_task_1["spectrograms_left_fist_100"]["spectrograms_3d"][0], title="Condizione: Left Fist - Trial 0")

#▶️ 2. Per plottare il primo trial di tutte le condizioni:
#for cond, data in new_all_subjects_condition_spectrograms_task_1.items():
#    plot_eeg_grid_image(data['spectrograms_2d'][0], title=f"Condizione: {cond} - Trial 0")
#    break  # 🔁 rimuovi questo break per vedere tutte le condizioni
    
#▶️ 3. Per plottare più trial di una condizione:

#for i, trial_image in enumerate(new_all_subjects_condition_spectrograms_task_1["spectrograms_left_fist_100"]["spectrograms_2d"][:5]):  # primi 5 trial
#    plot_eeg_grid_image(trial_image, title=f"Condizione: spectrograms_left_fist_100 - Trial {i}")

In [ ]:
power_values = new_all_subjects_condition_spectrograms_task_1["spectrograms_left_fist_100"]["spectrograms_3d"][1]
print("Min:", power_values.min(), "Max:", power_values.max())

In [ ]:
#🔄 Se vuoi aggiungere le etichette degli elettrodi sulla griglia:

def plot_eeg_grid_with_labels(trial_2d_image: np.ndarray, label_grid: np.ndarray, title: str = "Condizione: Left Fist - Trial 0"):
    plt.figure(figsize=(6, 6))
    plt.imshow(trial_2d_image, cmap='viridis', origin='upper')
    for y in range(label_grid.shape[0]):
        for x in range(label_grid.shape[1]):
            label = label_grid[y, x]
            if label != "":
                plt.text(x, y, label, ha='center', va='center', color='white', fontsize=8)
    plt.colorbar(label="Potenza Totale")
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_eeg_grid_with_labels(new_all_subjects_condition_spectrograms_task_1["spectrograms_left_fist_100"]["spectrograms_3d"][0], label_grid)

#### **Concatenazione** All **SINGLE** Subject **Spectrograms** per **Tutte le Condizioni Sperimentali**

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la isualizzazione dei dati
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

# Caricare l'intero dizionario annidato con pickle

with open(f'{path}new_all_subjects_condition_spectrograms_task_1_2d_grid_multiband.pkl', 'rb') as f:
    new_all_subjects_condition_spectrograms_task_1 = pickle.load(f)

In [ ]:
new_all_subjects_condition_spectrograms_task_1.keys()

In [ ]:
import numpy as np

# Definizione degli indici dei canali desiderati

'''TOLGO PER SPETTROGRAMMI FREQUENZE x ELETTRODI'''
#selected_channels = [12, 30, 48]  # Indici per Fz, Cz, Pz

# Creazione di un dizionario dei dati di tutti i terapisti singolarmente, per salvare gli spettrogrammi calcolati
new_single_th_all_extracted_spectrograms_2D_task_1 = {}

# Iterazione su tutte le condizioni sperimentali di tutti i soggetti 

for condition, data in new_all_subjects_condition_spectrograms_task_1.items():
    
    # Estrarre le etichette per la condizione corrente
    labels = data['labels']
    
    #  Estrarre gli spettrogrammi per la condizione corrente
    '''TOLGO PER SPETTROGRAMMI FREQUENZE x ELETTRODI'''
    #spectrograms = data['spectrograms_4d']
    
    spectrograms = data['spectrograms_3d']
    
    '''TOLGO PER SPETTROGRAMMI FREQUENZE x ELETTRODI'''
    #Selezione dei canali di interesse (Fz, Cz e Pz)
    #spectrograms = spectrograms[:, selected_channels, :, :]
    
    new_single_th_all_extracted_spectrograms_2D_task_1[condition] = {
        'spectrograms': spectrograms,
        #'delta': delta_reconstruction,
        #'coeff_fifth_detail_theta': coeff_fifth_detail_theta, 
        'labels': labels
    }
    
# Verifica delle dimensioni del risultato estratto per una condizione specifica di ogni soggetto
print(f"\t\t\033[1mEstrazione Spettrogrammi di tutti i terapisti per tutte le condizioni sperimentali")
print(f"\n\n\t\t\t\t(i.e., \033[1mnew_single_th_all_extracted_spectrograms_2D_task_1\033[0m):\n")

for condition, extracted_data in new_single_th_all_extracted_spectrograms_2D_task_1.items():
    
    print(f"Condizione: \033[1m{condition}\033[0m, "
          f"Spectrograms shape: \033[1m{extracted_data['spectrograms'].shape}\033[0m, "
          f"Labels: \033[1m{len(extracted_data['labels'])}\033[0m")


In [ ]:
'''Salvataggio di new_single_th_all_extracted_spectrograms_2D_task_1'''

base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'


# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
import pickle

# Salvare l'intero dizionario annidato con pickle
with open(f'{base_path}new_single_th_all_extracted_spectrograms_2D_task_1_2d_grid_multiband.pkl', 'wb') as f:
    pickle.dump(new_single_th_all_extracted_spectrograms_2D_task_1, f)

In [ ]:
print(new_single_th_all_extracted_spectrograms_2D_task_1.keys())
print()
print(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100'].keys())
print()
print(len(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100']['labels']))
print()
#print(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100']['labels'][:10])

In [ ]:
'''
Questa struttura consente di avere i dati degli spettrogrammi correttamente organizzati e concatenati per ogni soggetto,
mantenendo la corrispondenza tra dati e etichette per ogni condizione sperimentale

Da           "single_th_all_extracted_reconstructions"       a           "subject_level_concatenations_th"

Da           "new_single_th_all_extracted_reconstructions"       a       "new_subject_level_concatenations_th"

Da           "new_single_th_all_extracted_spectrograms_2D_task_1"       a       "new_subject_level_concatenations_spectrograms_th_2D_task_1"
'''


# Dizionario per contenere i dati concatenati per ogni soggetto
new_subject_level_concatenations_spectrograms_th_2D_task_1 = {}

# Variabile per tracciare il soggetto precedente
previous_subject_suffix = None

print(f"\t\t\033[1mConcatenazione spettrogrammi 2D per OGNI soggetto di TUTTE le condizioni sperimentali INSIEME\033[0m")

# Iterazione su tutte le chiavi di new_single_th_all_extracted_spectrograms_2D_task_1
for condition, data in new_single_th_all_extracted_spectrograms_2D_task_1.items():
    
    # Estrazione del suffisso numerico del soggetto
    subject_suffix = condition.split('_')[-1]  # Prende solo la parte numerica
    
    # Creazione del nome della chiave del soggetto specifico
    subj_name = f'th_{subject_suffix}'
    
    # Se stiamo per passare a un nuovo soggetto, concatenare e stampare i risultati del soggetto precedente
    if previous_subject_suffix is not None and subject_suffix != previous_subject_suffix:
        
        print(f"\n\n\t\t\t\t\033[1mConcatenazioni per il soggetto corrente ('th_{previous_subject_suffix}'):\033[0m\n")
        prev_subject_name = f'th_{previous_subject_suffix}'
        levels = new_subject_level_concatenations_spectrograms_th_2D_task_1[prev_subject_name]

        # Concatenazione dei dati prima della stampa
        spectrograms_concat = np.concatenate(levels['spectrograms'], axis=0)
        labels_concat = np.concatenate(levels['labels'], axis=0)

        print(f"Soggetto: \033[1m{prev_subject_name}\033[0m, "
              f"Spettrogrammi shape: \033[1m{spectrograms_concat.shape}\033[0m, "
              f"Labels length: \033[1m{len(labels_concat)}\033[0m")
        
        # Salvataggio dei dati concatenati nel dizionario
        new_subject_level_concatenations_spectrograms_th_2D_task_1[prev_subject_name]['spectrograms'] = spectrograms_concat
        new_subject_level_concatenations_spectrograms_th_2D_task_1[prev_subject_name]['labels'] = labels_concat
    
    # Inizializzare il dizionario per il soggetto specifico se non esiste già
    if subj_name not in new_subject_level_concatenations_spectrograms_th_2D_task_1:
        new_subject_level_concatenations_spectrograms_th_2D_task_1[subj_name] = {
            'spectrograms': [],
            'labels': []
        }
    
    # Stampiamo le informazioni per ogni condizione
    print(f"\n\n\nSoggetto: \033[1m{subj_name}\033[0m, Condizione: \033[1m{condition}\033[0m")
    print(f"  - Spettrogrammi shape: \033[1m{data['spectrograms'].shape}\033[0m")
    print(f"  - Labels shape: \033[1m{len(data['labels'])}\033[0m")
    print(f"  - Valore unico delle etichette: \033[1m{np.unique(data['labels'])}\033[0m")
    
    # Concatenazione dei dati
    new_subject_level_concatenations_spectrograms_th_2D_task_1[subj_name]['spectrograms'].append(data['spectrograms'])
    new_subject_level_concatenations_spectrograms_th_2D_task_1[subj_name]['labels'].append(data['labels'])
    
    # Aggiorna il soggetto precedente
    previous_subject_suffix = subject_suffix

# Dopo aver iterato su tutte le condizioni, concatenare e stampare le informazioni dell'ultimo soggetto
if previous_subject_suffix is not None:
    
    last_subject_name = f'th_{previous_subject_suffix}'
    levels = new_subject_level_concatenations_spectrograms_th_2D_task_1[last_subject_name]
    
    print(f"\n\n\t\t\t\t\033[1mConcatenazioni per il soggetto corrente ('{last_subject_name}'):\033[0m\n")
    
    # Concatenazione dei dati dell'ultimo soggetto
    spectrograms_concat = np.concatenate(levels['spectrograms'], axis=0)
    labels_concat = np.concatenate(levels['labels'], axis=0)
    
    # Salvataggio dei dati concatenati dell'ultimo soggetto nel dizionario
    new_subject_level_concatenations_spectrograms_th_2D_task_1[last_subject_name] = {
        'spectrograms': spectrograms_concat,
        'labels': labels_concat
    }
    
    print(f"Soggetto: \033[1m{last_subject_name}\033[0m, "
          f"Spettrogrammi shape: \033[1m{spectrograms_concat.shape}\033[0m, "
          f"Labels length: \033[1m{len(labels_concat)}\033[0m\n\n")


In [ ]:
'''CHIAVI DI TUTTO IL DIZIONARIO '''
#subject_level_concatenations_th.keys()


new_subject_level_concatenations_spectrograms_th_2D_task_1.keys()


#SOGGETTO 1

#CHIAVI
#subject_level_concatenations_th['th_1'].keys()

#DATI
#subject_level_concatenations_th['th_1']['theta'].shape

#new_subject_level_concatenations_th['th_1']['theta'].shape

#subject_level_concatenations_th['th_1']['delta'].shape

#LABELS
#subject_level_concatenations_th['th_1']['labels'].shape)
#type(subject_level_concatenations_th['th_1']['labels'])

# Check della concantenazione delle labels TH_1

#subject_level_concatenations_th['th_1']['labels'][:42]
#subject_level_concatenations_th['th_1']['labels'][41:81]
#subject_level_concatenations_th['th_1']['labels'][81:120]
#subject_level_concatenations_th['th_1']['labels'][121:164]

In [ ]:
#subject_level_concatenations_th['th_15']['labels'].shape

In [ ]:
#subject_level_concatenations_th['th_1'].keys()

new_subject_level_concatenations_spectrograms_th_2D_task_1['th_1'].keys()


In [ ]:
#unique_values, counts = np.unique(subject_level_concatenations_th['th_1']['labels'], return_counts=True)

#unique_values, counts = np.unique(new_subject_level_concatenations_th['th_1']['labels'], return_counts=True)

In [ ]:
#unique_values
#counts
#counts[0]

In [ ]:
!pwd

In [ ]:
'''Salvataggio di new_single_th_all_extracted_spectrograms_2D_task_1'''

base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
import pickle

# Salvare l'intero dizionario annidato con pickle
with open(f'{base_path}new_subject_level_concatenations_spectrograms_th_2D_task_1_2d_grid_multiband.pkl', 'wb') as f:
    pickle.dump(new_subject_level_concatenations_spectrograms_th_2D_task_1, f)

#### **Concatenazione** All **SINGLE** Subject **Spectrograms (TH)** per **Coppie di Condizioni Sperimentali**

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la isualizzazione dei dati
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pickle

In [ ]:
!pwd

In [ ]:
cd ..

In [ ]:
#cd New_Plots_Sliding_Estimator_MNE_1_45_NF/

In [ ]:
import pickle
import numpy as np

base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'
    
# Caricare l'intero dizionario annidato con pickle
with open(f'{base_path}new_single_th_all_extracted_spectrograms_2D_task_1_2d_grid_multiband.pkl', 'rb') as f:
    new_single_th_all_extracted_spectrograms_2D_task_1 = pickle.load(f)

In [ ]:
new_single_th_all_extracted_spectrograms_2D_task_1.keys()

In [ ]:
new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100'].keys()

##### Spiegazione STEP - Concatenazione All Single Subject Spectrograms (TH) across Couples of Experimental Conditions

###### **Concatenazione All Single Subject Data (TH) per Coppie di Condizioni Sperimentali**

<br>

In questo caso, voglio **trasformare le variabili che contengono i miei dati e labels delle 4 condizioni sperimentali assieme**, in modo **da creare sottoinsiemi per ogni possibile coppia di condizioni sperimentali**. 

Dunque quello che si dovrà fare **per ogni coppia**:

1) **Filtrare i dati e le etichette**:

- **Estrarre** i dati associati **SOLO alle DUE condizioni in esame** (ad esempio 0 e 1, oppure 1 e 2).
- **Creare un nuovo array di etichette** e **un array di dati** ***corrispondenti*** **SOLO a quelle DUE condizioni**.

2) **Ripetere questa operazione per OGNI combinazione**:

- **Assicurarsi di considerare tutte le coppie** di condizioni sperimentali **SENZA ripetizioni** (ad esempio, non serve processare sia 0 vs 1 che 1 vs 0, perché sono equivalenti!).

3) **Creare quindi una struttura organizzata**, che contenga i dati e le etichette separatamente per ogni coppia, per ciascun livello di ricostruzione (theta, delta, theta_strict).

<br>

Quindi l'obiettivo è di modificare la procedura in modo da **concatenare SOLO le coppie di condizioni sperimentali per OGNI soggetto**. 

Per fare questo, possiamo utilizzare un **approccio combinatorio** per generare **tutte le possibili coppie di condizioni**.


Per quanto riguarda il calcolo combinatorio, possiamo usare il concetto di combinazioni per selezionare le coppie. 
Se hai 4 condizioni, il numero di combinazioni di coppie di condizioni (senza ripetizioni) si calcola come:

$$
\binom{4}{2} = \frac{4!}{2!(4-2)!} = 6
$$


Quindi, con 4 condizioni sperimentali, le combinazioni di coppie di condizioni saranno 6, e queste coppie sono:

(condizione 1, condizione 2) = 'baseline vs th_resp'
(condizione 1, condizione 3) = 'baseline vs pt_resp'
(condizione 1, condizione 4) = 'baseline vs shared_resp'
(condizione 2, condizione 3) = 'th_resp vs pt_resp'
(condizione 2, condizione 4) = 'th_resp vs shared_resp'
(condizione 3, condizione 4) = 'pt_resp vs shared_resp' 


<br>

Piano di modifiche al codice:

1) **Creazione delle coppie di condizioni**: Per ogni soggetto, genereremo tutte le coppie possibili di condizioni sperimentali. Utilizzeremo itertools.combinations per ottenere queste coppie.

2) **Concatenazione per ciascuna coppia**: Per ogni coppia di condizioni, concatenare i dati e le etichette delle due condizioni selezionate.

3) **Salvataggio delle concatenazioni**: Creeremo un dizionario per ciascuna coppia di condizioni per ogni soggetto.


**N.B.**

Nel codice che sto creando...

Ho fatto in modo che ci sia il modo di capire, **per ogni coppia di condizione sperimentale di dati e labels**, a quali condizione sperimentali di dati e labels ci si riferisca, mi spiego:

Sviluppo un **modo "standardizzato" per il quale per OGNI coppia di condizione sperimentale, la variabile che conterrà i dati e labels associate, sia chiamata in un certo modo**...

E nello specifico **con il nome delle DUE condizioni sperimentali che conterranno quei dati e relative labels**...

Del tipo: 

- se è baseline vs th_resp, la variabile sarà "baseline_vs_th_resp"
- se è baseline vs pt_resp, la variabile sarà "baseline_vs_pt_resp"
- se è baseline vs shared_resp, la variabile sarà "baseline_vs_shared_resp"


Senza contare che, questo **processo di standardizzazione del nome delle variabili**, deve esser fatto **per ogni livello di ricostruzione dei dati (sia 4° e 5° livello a partire dai coefficienti di approssimazione, che dal 5 ° livello dei coefficienti di dettaglio)**..

Questo significherebbe che, **per ogni soggetto**, dovrei (o potrei insomma) creare **un dizionario**, che 

- Contenga delle **sotto-chiavi di secondo ordine** (che son theta, delta e theta_strict) e dentro ognuna di queste
- Ci siano contenute **le relative 6 sotto-sotto-chiavi di terzo ordine**, relative ai dati e alle labels associate (anch'esse concatenate ovviamente) delle diverse combinazioni di condizioni sperimentali...


Ossia, vogliamo creare un dizionario strutturato che rappresenti ogni combinazione di coppie di condizioni sperimentali per ciascun soggetto, separando i dati e le etichette per i vari livelli di ricostruzione (theta, delta, theta_strict). 

Ogni combinazione di condizioni sperimentali sarà memorizzata in una chiave strutturata, come ad esempio "baseline_vs_th_resp", e all'interno di ciascuna chiave avremo le informazioni per i vari livelli di ricostruzione.


<br>


#### Esempio di Struttura proposta per il dizionario:


- Soggetto: ad esempio "th_1".
- Livelli di ricostruzione: theta, delta, theta_strict.
- Combinazioni di condizioni sperimentali: per ciascun livello di ricostruzione, avremo sotto-chiavi per ogni coppia di condizioni.

        new_subject_level_concatenations = {
            'th_1': {
                'theta': {
                    'baseline_vs_th_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    'baseline_vs_pt_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    'baseline_vs_shared_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    # Altre combinazioni...
                },
                'delta': {
                    'baseline_vs_th_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    'baseline_vs_pt_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    'baseline_vs_shared_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    # Altre combinazioni...
                },
                'theta_strict': {
                    'baseline_vs_th_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    'baseline_vs_pt_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    'baseline_vs_shared_resp': {'data': np.array([...]), 'labels': np.array([...])},
                    # Altre combinazioni...
                }
            }
        }

<br>

**SOTTO-STEP 1: IDENTIFICAZIONE DELLE STRINGA NUMERICHE PER LA CREAZIONE DELLE VARIABILI - FUTURE CHIAVI - CHE CONTENGONO LE COPPIE DI DATI E LABELS DELLE RELATIVE COPPIE DI CONDIZIONI SPERIMENTALI CONSIDERATE NEL CICLO CORRENTE DEL LOOP**

Aspetta, ci sono altre cose che dovrebbero esser integrate nel codice...

cominciamo dalla prima, che è relativa all' "inferire" diciamo il nome da fornire alla variabile che conterrà i dati e labels delle due condizioni sperimentali correnti... e deve esser fatto, a partire dal nome delle chiavi di 

"new_single_th_all_extracted_reconstructions"...

ora, le sue chiavi son queste

dict_keys(['wave_baseline_1', 'wave_th_resp_1', 'wave_pt_resp_1', 'wave_shared_resp_1', 'wave_baseline_2', 'wave_th_resp_2', 'wave_pt_resp_2', 'wave_shared_resp_2', 'wave_baseline_3', 'wave_th_resp_3', 'wave_pt_resp_3', 'wave_shared_resp_3', 'wave_baseline_4', 'wave_th_resp_4', 'wave_pt_resp_4', 'wave_shared_resp_4', 'wave_baseline_5', 'wave_th_resp_5', 'wave_pt_resp_5', 'wave_shared_resp_5', 'wave_baseline_6', 'wave_th_resp_6', 'wave_pt_resp_6', 'wave_shared_resp_6', 'wave_baseline_7', 'wave_th_resp_7', 'wave_pt_resp_7', 'wave_shared_resp_7', 'wave_baseline_8', 'wave_th_resp_8', 'wave_pt_resp_8', 'wave_shared_resp_8', 'wave_baseline_9', 'wave_th_resp_9', 'wave_pt_resp_9', 'wave_shared_resp_9', 'wave_baseline_10', 'wave_th_resp_10', 'wave_pt_resp_10', 'wave_shared_resp_10', 'wave_baseline_11', 'wave_th_resp_11', 'wave_pt_resp_11', 'wave_shared_resp_11', 'wave_baseline_12', 'wave_th_resp_12', 'wave_pt_resp_12', 'wave_shared_resp_12'])


ora, il modo in cui si identifica un certo soggetto (ossia la chiavi di "primo ordine" che indentificano il soggetto, ossia ad esempio 'th_1'), dipenderebbe dal numero stringa che c'è alla fine di ogni nome stringa di ogni chiave dentro "new_single_th_all_extracted_reconstructions", mi spiego:

il codice dovrebbe capire che, ad esempio, solo l'ultima carattere stringa di queste prime 4 chiavi:

'wave_baseline_1', 'wave_th_resp_1', 'wave_pt_resp_1', 'wave_shared_resp_1', 

si riferisca al soggetto 1, ossia al futuro "th_1"...


per il soggetto 2, sarebbero: 

'wave_th_resp_2', 'wave_pt_resp_2', 'wave_shared_resp_2', e così via per gli altri soggetti..


Facendo attenzione al fatto che, dopo il soggetto 9, ovviamente, si passerà alle decine (a livello numerico), per cui saranno gli ultime due caratteri da considerare per l'identificazione delle sue rispettive chiavi, che in quel caso saranno per il soggetto 10, ossia il futuro th_10

'wave_baseline_10', 'wave_th_resp_10', 'wave_pt_resp_10', 'wave_shared_resp_10',

Inoltre, poi, per creare le variabili associate alle coppie di dati e labels delle relative condizioni sperimentali, dovrà il codice far in modo che il nome di quella variabile dipenda dalle stringhe sempre che si riferiscono al nome delle due chiavi di "new_single_th_all_extracted_reconstructions" da cui preleva i dati e le labels... 

ad esempio, se deve creare la variabile dei dati 'baseline_vs_th_resp" del primo soggetto, le chiavi rispettive da cui deve andare a prelevare le stringhe saranno appunto "'wave_baseline_1'" e "'wave_th_resp_1'"....

diciamo che si potrebbe creare una lista di stringhe che sarà del tipo:

experimental_conditions = ['baseline', 'th_resp', 'pt_resp', 'shared_resp'], per vedere se una di queste sia dentro la rispettiva chiave di  new_single_th_all_extracted_reconstructions su cui sta iterando, che nel nostro caso di esempio sarebbero sempre

'wave_baseline_1' (e da cui vede che c'è la stinga 'baseline')
'wave_th_resp_1' (e da cui vede che c'è la stinga 'th_resp')

è chiaro?


<br>

In sostanza, desideri un modo per inferire dinamicamente il nome del soggetto e delle variabili che conterranno i dati e le etichette per ogni combinazione di condizioni, a partire dal nome delle chiavi di new_single_th_all_extracted_reconstructions. Ecco un piano per raggiungere questo obiettivo:

Passaggi:

- Identificare il soggetto:

Le chiavi di new_single_th_all_extracted_reconstructions finiscono con un numero che identifica il soggetto (es. wave_baseline_1, wave_th_resp_1, wave_pt_resp_1).
Questo numero può essere estratto dalle ultime cifre della chiave (ad esempio, 1 da wave_baseline_1).
La variabile associata a ciascun soggetto sarà chiamata th_X, dove X è il numero del soggetto.

- Generare dinamicamente il nome delle variabili:

A partire dai nomi delle chiavi, possiamo inferire quale combinazione di condizioni sperimentali stiamo trattando (es. baseline_vs_th_resp).
Creeremo una lista delle condizioni (experimental_conditions = ['baseline', 'th_resp', 'pt_resp', 'shared_resp']), quindi esamineremo le chiavi per determinare quali condizioni sono presenti.

- Creare il dizionario:

A ciascun soggetto verrà associato un dizionario, e per ogni combinazione di condizioni sperimentali, assoceremo i dati e le etichette in una chiave che seguirà la convenzione del tipo baseline_vs_th_resp.


<br>

Spiegazione:

1) Estrazione delle condizioni e soggetto:

La chiave, ad esempio wave_baseline_1, viene separata in parti. La parte baseline viene estratta per determinare la condizione, mentre 1 indica il numero del soggetto.
Queste informazioni vengono utilizzate per creare il nome del soggetto, th_1.

2) Combinazioni di condizioni:

La lista experimental_conditions contiene le 4 condizioni: baseline, th_resp, pt_resp, shared_resp.
Per ogni soggetto, controlliamo quali condizioni sono disponibili nella chiave (ad esempio, se il soggetto 1 ha wave_baseline_1 e wave_th_resp_1, creiamo la combinazione baseline_vs_th_resp).

3) Creazione dinamica delle variabili:

Utilizziamo la combinazione di condizioni per generare la chiave della variabile (baseline_vs_th_resp).
I dati e le etichette per la combinazione di condizioni vengono estratti e concatenati.


4) Risultato finale:

La struttura finale del dizionario new_subject_level_concatenations avrà una chiave per ciascun soggetto (th_1, th_2, ecc.), e sotto ogni soggetto ci saranno le combinazioni di condizioni per ciascun livello di ricostruzione (theta, delta, theta_strict), con i dati e le etichette corrispondenti.

<br>

Esempio di output:

    {
        'th_1': {
            'theta': {
                'baseline_vs_th_resp': {'data': np.array([...]), 'labels': np.array([...])},
                'baseline_vs_pt_resp': {'data': np.array([...]), 'labels': np.array([...])},
                'baseline_vs_shared_resp': {'data': np.array([...]), 'labels': np.array([...])},
                'th_resp_vs_pt_resp': {'data': np.array([...]), 'labels': np.array([...])},
                # Altre combinazioni...
            },
            'delta': {
                # Stessa struttura per 'delta'
            },
            'theta_strict': {
                # Stessa struttura per 'theta_strict'
            }
        }
    }

 
Questa struttura ti permetterà di avere un'organizzazione chiara e dinamica dei dati per ogni soggetto e combinazione di condizioni sperimentali.
 
 
<br>

**SOTTO-STEP 2: IDENTIFICAZIONE DELLE STRINGA ALFABETICHE PER LA CREAZIONE DELLE VARIABILI - FUTURE CHIAVI - CHE CONTENGONO IL NOME DELLE DUE COPPIE DI CONDIZIONI SPERIMENTALI DI CUI VENGONO PRELEVATI I DATI E LABELS CONCATENATI E CHE VENGONON CONSIDERATE NEL CICLO CORRENTE DEL LOOP**


ok, ora c'è una ultima cosa che mi manca da dirti, che è relativa alla ri-assegnazione del codice numerico associato ad ogni coppia di condizioni sperimentali...

nel senso che, originariamente...

new_single_th_all_extracted_reconstructions ha queste labels, che se io le vedo avrò 

per baseline una serie di 0, per th_resp una serie di 1, per pt_resp, una serie di 2 e per shared_resp una serie di 3, in questo modo


print(f"Baseline in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_baseline_1']['labels'], return_counts = True)}\n")
print(f"Th_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_th_resp_1']['labels'], return_counts = True)}\n")
print(f"Pt_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_pt_resp_1']['labels'], return_counts = True)}\n")
print(f"Shared_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_shared_resp_1']['labels'], return_counts = True)}\n")


Baseline in TH_1:, (array(['0'], dtype='<U1'), array([51]))

Th_Resp in TH_1:, (array(['1'], dtype='<U1'), array([40]))

Pt_Resp in TH_1:, (array(['2'], dtype='<U1'), array([40]))

Shared_Resp in TH_1:, (array(['3'], dtype='<U1'), array([56]))

di conseguenza,  nel tuo codice, ho bisogno che 

le labels associate a 'cond_1' (quando crei la relativa variabile delle due condizioni sperimentali di cui vengono concatenati i dati e le labels) siano riconvertite sempre in '0', mentre 

le labels associate a 'cond_2' (quando crei la relativa variabile delle due condizioni sperimentali di cui vengono concatenati i dati e le labels) siano riconvertite sempre in '1'

questo aspetto è importante, per quando crei le variabili che conterranno le coppie di dati condizioni sperimentali...

ora, nel caso in cui tu stia considerando magari baseline vs le altre condizioni sperimentali, è chiaro che in quel caso specifico, ad esempio, baseline sarà sempre la stringa associata a 'cond_1' e quindi in quel caso sarà sempre 0, e quindi sarebbe apposto..

ma per gli altri casi di confronto di baseline rispetto alle altre 3 condizioni sperimentali, è chiaro che se sarà th_resp la condizione di confronto, allora in quel caso di quella specifica variabile, le labels non avranno bisogno di esser diciamo 'ri-convertite' perché già naturalmente diciamo baseline avrà le labels '0' e  e th_resp avrà già '1' (come è già originariamente codificato in 'new_subject_level_concatenations'!)

ma nel caso delle altre variabili da creare le labels andranno ovviamente riconvertite! Perché?
perché ovviamente nel caso sia la th_resp la 'cond_1', ad esempip, in quel caso, le sue labels (che son  originariamente codificate come '1' in 'new_subject_level_concatenations'!) diveteranno '0', mentre le altre condizioni di confronto, una alla volta, diventeranno '1' invece...


è chiara diciamo la logica di ri-assegnazione delle relative labels in sostanza?


<br>

La **logica di ri-assegnazione delle etichette (labels)** è chiara e posso integrarla nel codice. 

La logica che stai descrivendo consiste essenzialmente nel **garantire che, quando crei una variabile che contiene dati e etichette per due condizioni sperimentali, le etichette siano sempre "0" per la prima condizione (cond_1) e "1" per la seconda condizione (cond_2)**, 

                **INDIPENDENTEMENTE** da come sono originariamente codificate in **"new_single_th_all_extracted_reconstructions"!
                
                

Ecco come possiamo procedere:

- Identificazione delle condizioni: Come hai descritto, determineremo le due condizioni sperimentali da confrontare, ad esempio "baseline" contro "th_resp", "baseline" contro "pt_resp", ecc.

- Riconversione delle etichette:

 - Se la condizione di confronto è "baseline" (che è associata a 0), non dobbiamo modificare le etichette.
 - Se la condizione di confronto non è "baseline", dovremo riconvertire le etichette di quella condizione a 0 e le etichette della condizione di confronto a 1.

- Struttura del codice: Durante la creazione delle variabili, dovremo controllare quale condizione è la prima (cond_1) e quale è la seconda (cond_2), e applicare questa logica per ri-assegnare le etichette di conseguenza.


<br>

Inoltre


dentro 

"# Crea tutte le combinazioni uniche di condizioni sperimentali (es. 'baseline vs th_resp')
    condition_pairs = list(itertools.combinations(experimental_conditions, 2))
"

Effettivamente vengono create delle tuple di coppie di stringhe che corrispondono a quelle con cui dovrebbero esser create le sotto-sotto-chiavi delle coppie di condizioni sperimentali..

tuttavia, è giusto che iteri rispetto a 'condition_pairs', ma deve inserire un qualche controllo anche rispetto a questa cosa:

originariamente...

new_single_th_all_extracted_reconstructions ha queste labels, che se io le vedo avrò

per baseline una serie di 0, per th_resp una serie di 1, per pt_resp, una serie di 2 e per shared_resp una serie di 3, in questo modo

print(f"Baseline in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_baseline_1']['labels'], return_counts = True)}\n") print(f"Th_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_th_resp_1']['labels'], return_counts = True)}\n") print(f"Pt_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_pt_resp_1']['labels'], return_counts = True)}\n") print(f"Shared_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_shared_resp_1']['labels'], return_counts = True)}\n")

Baseline in TH_1:, (array(['0'], dtype='<U1'), array([51]))

Th_Resp in TH_1:, (array(['1'], dtype='<U1'), array([40]))

Pt_Resp in TH_1:, (array(['2'], dtype='<U1'), array([40]))

Shared_Resp in TH_1:, (array(['3'], dtype='<U1'), array([56]))

di conseguenza, nel tuo codice, ho bisogno che

le labels associate a 'cond_1' (quando crei la relativa variabile delle due condizioni sperimentali di cui vengono concatenati i dati e le labels) siano riconvertite sempre in '0', mentre

le labels associate a 'cond_2' (quando crei la relativa variabile delle due condizioni sperimentali di cui vengono concatenati i dati e le labels) siano riconvertite sempre in '1'

questo aspetto è importante, per quando crei le variabili che conterranno le coppie di dati condizioni sperimentali...

ora, nel caso in cui tu stia considerando magari baseline vs le altre condizioni sperimentali, che nel caso di "condition_pairs" dovrebbe corrispondere ai primi 3 elementi della sua lista no, perché sarebbero 

[('baseline', 'th_resp'), 
    ('baseline', 'pt_resp'), 
    ('baseline', 'shared_resp'), 

è chiaro che in quel caso specifico, ad esempio, baseline sarà sempre la stringa associata a 'cond_1' e quindi in quel caso sarà sempre 0, e quindi sarebbe apposto..

ma per gli altri casi di confronto di baseline rispetto alle altre 3 condizioni sperimentali, è chiaro che se sarà th_resp la condizione di confronto, allora in quel caso di quella specifica variabile, le labels non avranno bisogno di esser diciamo 'ri-convertite' perché già naturalmente diciamo baseline avrà le labels '0' e e th_resp avrà già '1' (come è già originariamente codificato in 'new_subject_level_concatenations'!)

ma nel caso delle altre variabili da creare, che fa riferimento a questi altri elementi di 
"condition_pairs" che è creato nel loop ossia

('th_resp', 'pt_resp'), 
('th_resp', 'shared_resp'), 
('pt_resp', 'shared_resp')]

le labels andranno ovviamente riconvertite! Perché? 

Perché ovviamente nel caso sia la th_resp la 'cond_1', ad esempio, in quel caso, le sue labels (che son originariamente codificate come '1' in 'new_subject_level_concatenations'!) diventeranno '0',

mentre le altre condizioni di confronto, una alla volta, diventeranno '1' invece...

è chiara diciamo la logica di ri-assegnazione delle relative labels in sostanza?dentro "# Crea tutte le combinazioni uniche di condizioni sperimentali (es. 'baseline vs th_resp')
    condition_pairs = list(itertools.combinations(experimental_conditions, 2))
    "

effettivamente vengono create delle tuple di coppie di stringhe che corrispondono a quelle con cui dovrebbero esser create le sotto-sotto-chiavi delle coppie di condizioni sperimentali..

tuttavia, è giusto che iteri rispetto a 'condition_pairs', ma deve inserire un qualche controllo anche rispetto a questa cosa:

originariamente...

new_single_th_all_extracted_reconstructions ha queste labels, che se io le vedo avrò

per baseline una serie di 0, per th_resp una serie di 1, per pt_resp, una serie di 2 e per shared_resp una serie di 3, in questo modo

print(f"Baseline in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_baseline_1']['labels'], return_counts = True)}\n") print(f"Th_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_th_resp_1']['labels'], return_counts = True)}\n") print(f"Pt_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_pt_resp_1']['labels'], return_counts = True)}\n") print(f"Shared_Resp in TH_1:, {np.unique(new_single_th_all_extracted_reconstructions['wave_shared_resp_1']['labels'], return_counts = True)}\n")

Baseline in TH_1:, (array(['0'], dtype='<U1'), array([51]))

Th_Resp in TH_1:, (array(['1'], dtype='<U1'), array([40]))

Pt_Resp in TH_1:, (array(['2'], dtype='<U1'), array([40]))

Shared_Resp in TH_1:, (array(['3'], dtype='<U1'), array([56]))

di conseguenza, nel tuo codice, ho bisogno che

le labels associate a 'cond_1' (quando crei la relativa variabile delle due condizioni sperimentali di cui vengono concatenati i dati e le labels) siano riconvertite sempre in '0', mentre

le labels associate a 'cond_2' (quando crei la relativa variabile delle due condizioni sperimentali di cui vengono concatenati i dati e le labels) siano riconvertite sempre in '1'

questo aspetto è importante, per quando crei le variabili che conterranno le coppie di dati condizioni sperimentali...

ora, nel caso in cui tu stia considerando magari baseline vs le altre condizioni sperimentali, che nel caso di "condition_pairs" dovrebbe corrispondere ai primi 3 elementi della sua lista no, perché sarebbero 

[('baseline', 'th_resp'), 
    ('baseline', 'pt_resp'), 
    ('baseline', 'shared_resp'), 

è chiaro che in quel caso specifico, ad esempio, baseline sarà sempre la stringa associata a 'cond_1' e quindi in quel caso sarà sempre 0, e quindi sarebbe apposto..

ma per gli altri casi di confronto di baseline rispetto alle altre 3 condizioni sperimentali, è chiaro che se sarà th_resp la condizione di confronto, allora in quel caso di quella specifica variabile, le labels non avranno bisogno di esser diciamo 'ri-convertite' perché già naturalmente diciamo baseline avrà le labels '0' e e th_resp avrà già '1' (come è già originariamente codificato in 'new_subject_level_concatenations'!)

ma nel caso delle altre variabili da creare, che fa riferimento a questi altri elementi di 
"condition_pairs" che è creato nel loop ossia

('th_resp', 'pt_resp'), 
('th_resp', 'shared_resp'), 
('pt_resp', 'shared_resp')]

le labels andranno ovviamente riconvertite! Perché? 

Perché ovviamente nel caso sia la th_resp la 'cond_1', ad esempio, in quel caso, le sue labels (che son originariamente codificate come '1' in 'new_subject_level_concatenations'!) diventeranno '0',

mentre le altre condizioni di confronto, una alla volta, diventeranno '1' invece...

è chiara diciamo la logica di ri-assegnazione delle relative labels in sostanza?



##### Implementazione STEP - Concatenazione All Single Subject Spectrograms (TH) across Couples of Experimental Conditions

In [ ]:
new_single_th_all_extracted_spectrograms_2D_task_1.keys()

In [ ]:
new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100'].keys()

In [ ]:
type(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100'])

In [ ]:
print(f"Rest in TH_1:, {np.unique(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100']['labels'], return_counts = True)}\n")
print(f"Left_Fist in TH_1:, {np.unique(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_left_fist_100']['labels'], return_counts = True)}\n")
print(f"Right_Fist in TH_1:, {np.unique(new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_right_fist_100']['labels'], return_counts = True)}\n")

In [ ]:
new_single_th_all_extracted_spectrograms_2D_task_1['spectrograms_rest_100'].keys()

In [ ]:
'''DETAILED VERSION WITH COMPLICATED PRINTS

    CORREZIONE UFFICIALE PER SPETTROGRAMS
'''


import itertools
import numpy as np

# Definizione delle condizioni sperimentali
#experimental_conditions = ['baseline', 'th_resp', 'pt_resp', 'shared_resp']

'''UNICA MODIFICA PER DATASET MOTOR TASK'''
experimental_conditions = ['rest', 'left_fist', 'right_fist']

# Variabile per tenere traccia del soggetto corrente (per output facoltativo)
last_subject_key = None

# Dizionario per tenere traccia delle combinazioni già elaborate per ogni soggetto (evitare duplicati)
printed_combinations = {}

# Nuovo dizionario per memorizzare i risultati concatenati per coppia di condizioni per ogni soggetto
new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1 = {}

# Itera su tutte le chiavi del dizionario di partenza
for key in new_single_th_all_extracted_spectrograms_2D_task_1.keys():
    # Esempio di key: "spectrograms_baseline_1", "spectrograms_th_resp_1", etc.
    key_parts = key.split('_')
    subject_number = key_parts[-1] if key_parts[-1].isdigit() else None
    if subject_number is None:
        continue  # Salta eventuali chiavi non valide
    
    # Crea il nome del soggetto (es. "th_1")
    subject_key = f"th_{subject_number}"
    
    # Se cambiamo soggetto, stampiamo (facoltativo)
    if subject_key != last_subject_key:
        if last_subject_key is not None:
            print("\n" + "-" * 50 + f" END OF {last_subject_key.upper()} " + "-" * 50 + "\n")
        print(f"\nProcessing Subject: {subject_key}\n" + "=" * 80)
        printed_combinations[subject_key] = set()  # Inizializza il set per il soggetto corrente
    last_subject_key = subject_key

    # Inizializza la struttura per il soggetto se non esiste già
    if subject_key not in new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1:
        new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1[subject_key] = {}

    # Crea tutte le combinazioni uniche di condizioni sperimentali (es. "baseline_vs_th_resp")
    condition_pairs = list(itertools.combinations(experimental_conditions, 2))
    
    # Loop sulle coppie di condizioni
    for cond_1, cond_2 in condition_pairs:
        condition_pair_key = f"{cond_1}_vs_{cond_2}"
        if condition_pair_key in printed_combinations[subject_key]:
            continue  # Salta se già elaborata per questo soggetto
        
        print(f"\n\tCreation of Coupled Condition: \033[1m{condition_pair_key}\033[0m")
        printed_combinations[subject_key].add(condition_pair_key)
        
        # Estrai i dati e le etichette per la prima condizione
        data_cond_1 = new_single_th_all_extracted_spectrograms_2D_task_1.get(f"spectrograms_{cond_1}_{subject_number}", {}).get('spectrograms')
        labels_cond_1 = new_single_th_all_extracted_spectrograms_2D_task_1.get(f"spectrograms_{cond_1}_{subject_number}", {}).get('labels')
        
        # Estrai i dati e le etichette per la seconda condizione
        data_cond_2 = new_single_th_all_extracted_spectrograms_2D_task_1.get(f"spectrograms_{cond_2}_{subject_number}", {}).get('spectrograms')
        labels_cond_2 = new_single_th_all_extracted_spectrograms_2D_task_1.get(f"spectrograms_{cond_2}_{subject_number}", {}).get('labels')
        
        # Salta la coppia se uno dei dati non è presente
        if data_cond_1 is None or labels_cond_1 is None or data_cond_2 is None or labels_cond_2 is None:
            continue
        
        # Riassegna le etichette: tutti 0 per cond_1, tutti 1 per cond_2
        labels_cond_1 = np.zeros_like(labels_cond_1, dtype=int)
        labels_cond_2 = np.ones_like(labels_cond_2, dtype=int)
        
        # Concatenazione dei dati (verticale) e delle etichette (orizzontale)
        concatenated_data = np.concatenate((data_cond_1, data_cond_2), axis=0)
        concatenated_labels = np.concatenate((labels_cond_1, labels_cond_2), axis=0)
        
        # Salva il risultato per questa coppia nel dizionario del soggetto
        new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1[subject_key][condition_pair_key] = {
            'data': concatenated_data,
            'labels': concatenated_labels
        }
        
        print(f"Condition pair '{condition_pair_key}' for subject {subject_key} => Data shape: {concatenated_data.shape}, Labels length: {len(concatenated_labels)}")


In [ ]:
print('finito')

In [ ]:
print(f"\t\t\t\t\033[1mStructure of new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1\033[0m: \n")
print(f"\033[1mFirst Order Keys\033[0m: {new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1.keys()}\n")
print(f"\033[1mSecond Order Keys\033[0m: {new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_100'].keys()}\n")
print(f"\033[1mThird Order Keys\033[0m: \n{new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_100']['rest_vs_left_fist'].keys()}\n")

In [ ]:
'''VERIFICO CHE LA CONCATENAZIONE SIA AVVENUTA CORRETTAMENTE!

    CORREZIONE UFFICIALE PER SPETTROGRAMS
'''

print("\tNOW, LET US SEE IF THE CONCATENATIONS RESPECT THE ORIGINAL SHAPES FOR EVERY COUPLED EXPERIMENTAL CONDITION!")
print("\tFOR THE 1°ST SUBJECT: CHECK IF THE SUM OF INDIVIDUAL EXP COND SHAPE MATCHES THE COUPLED COND CONCATENATION SHAPE:\n\n")

print("\033[1mINDIVIDUAL EXP COND SHAPE OF FIRST SUBJECT\033[0m:")

# Verifica delle dimensioni originali per le singole condizioni sperimentali

#for cond in ['baseline', 'th_resp', 'pt_resp', 'shared_resp']:
for cond in ['rest', 'left_fist', 'right_fist']:    
    
    key = f'spectrograms_{cond}_100'
    if key in new_single_th_all_extracted_spectrograms_2D_task_1:
        unique_labels, counts = np.unique(new_single_th_all_extracted_spectrograms_2D_task_1[key]['labels'], return_counts=True)
        print(f"{cond.capitalize()} in TH_100: {unique_labels}, Counts: {counts}\n")
    else:
        print(f"Warning: {key} not found in new_single_th_all_extracted_spectrograms_2D_task_1\n")

# Seleziona il primo soggetto (ad esempio, 'th_1')
subject_key = 'th_100'

# Itera per ogni coppia di condizioni sperimentali direttamente (senza livelli)
for condition_pair_key, data_labels in new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1.get(subject_key, {}).items():
    
    # Estrarre dati e labels dalla struttura corretta
    data = data_labels['data']
    labels = data_labels['labels']

    # Stampare il nome della coppia di condizioni e le loro dimensioni
    print(f"Condition Pair: \033[1m{condition_pair_key}\033[0m")
    
    if data is not None and labels is not None:
        print(f"  Data Shape: {data.shape}")
        print(f"  Labels Shape: {labels.shape}")
    else:
        print("  Missing data or labels!")

        

In [ ]:
print(np.unique(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_100']['rest_vs_left_fist']['labels'], return_counts = True))
print(np.unique(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_100']['rest_vs_right_fist']['labels'], return_counts = True))

In [ ]:
print(f"\t\t\t\033[1mBASELINE_VS_TH_RESP\033[0m\n")
print(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_100']['rest_vs_left_fist']['labels'][:40])
print(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_100']['rest_vs_left_fist']['labels'][40:])
print()
print()
print(f"\t\t\t\033[1mTH_RESP_VS_SHARED_RESP\033[0m\n")
print(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_100']['rest_vs_right_fist']['labels'][:40])
print(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_100']['rest_vs_right_fist']['labels'][40:])

##### STEP - Salvataggio Dataset di All Single Therapists EEG Data Spectrograms across Couples of Experimental Conditions

In [ ]:
!pwd

In [ ]:
base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
import pickle

# Salvare l'intero dizionario annidato con pickle
with open(f'{base_path}new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1_2d_grid_multiband.pkl', 'wb') as f:
    pickle.dump(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1, f)

#### **Concatenazione** **ALL** Single Subject **Spectrograms (TH)** per **Coppie di Condizioni Sperimentali INSIEME**

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la isualizzazione dei dati
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pickle

import os 

base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
# Caricare l'intero dizionario annidato con pickle
with open(f'{base_path}/new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1_2d_grid_multiband.pkl', 'rb') as f:
    new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1 = pickle.load(f)

In [ ]:
#new_subject_level_concatenations_spectrograms_coupled_exp_th.keys()
#new_subject_level_concatenations_spectrograms_coupled_exp_th['th_1'].keys()
#new_subject_level_concatenations_spectrograms_coupled_exp_th['th_1']['theta'].keys()
#new_subject_level_concatenations_spectrograms_coupled_exp_th['th_1']['theta']['baseline_vs_th_resp'].keys()
#np.unique(new_subject_level_concatenations_spectrograms_coupled_exp_th['th_1']['theta']['baseline_vs_th_resp']['labels'], return_counts = True)

In [ ]:
np.unique(new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1['th_100']['rest_vs_left_fist']['labels'], return_counts = True)

In [ ]:
'''CODICE UFFICIALE CON MODIFICHE A SPETTROGRAMMI'''


import numpy as np

def concatenate_all_subjects_coupled_spectrogram_conditions_th(data_structure, conditions):
    
    """
    Concatena globalmente i dati e le etichette per ogni coppia di condizioni sperimentali
    a partire dalla struttura 'new_subject_level_concatenations_spectrograms_coupled_exp_th'.
    I dati di ogni soggetto (già raggruppati per coppia di condizioni) vengono ulteriormente 
    concatenati per ottenere un unico array globale per ogni coppia.
    
    Parameters:
        data_structure (dict): Struttura dati con chiavi per soggetto (es. 'th_1', 'th_2', ...)
                               e, per ciascun soggetto, chiavi per ciascuna coppia di condizioni sperimentali.
        condition_pairs (list): Lista di coppie di condizioni sperimentali (es. 'baseline_vs_th_resp', ...)
        
    Returns:
        dict: Un dizionario globale in cui ogni chiave è una coppia di condizioni e il valore
              è un dizionario con 'data' e 'labels' concatenati globalmente (da tutti i soggetti).
    """
        
    # Dizionario per contenere i dati concatenati di tutti i soggetti
    all_subj_data_by_coupled_cond = {}

    # Itera su ogni coppia di condizioni sperimentali
    for condition_pair in conditions:

        #Dizionari per raccogliere dati e labels ordinati per ogni coppia di condizioni sperimentali
        # che è dinamico, per ogni coppia di condizioni sperimentali 
        #(quindi si ricreeranno passando alla condizione sperimentale successiva!)

        data_by_label = {}
        labels_by_label = {}
        shape_labels_per_subject = {0: [], 1: []}  # Inizializza il dizionario per 0 e 1

        # Itera su tutti i soggetti
        # In questo caso, itera sulle chiavi di ogni soggetto

        # Quindi prenderà, per la relativa coppia di condizioni sperimentali di ogni soggetto,
        # i dati e le labels di quel soggetto, per quella relativa coppia di condizioni sperimentali

        for subject_key in data_structure.keys():

            # Controlla se il livello di ricostruzione esiste per il soggetto
            if condition_pair not in data_structure[subject_key]:
                continue

            # Qui estrae i dati e le labels per questa condizione sperimentale di quel soggettio lì
            subject_data = data_structure[subject_key][condition_pair]['data']
            subject_labels = data_structure[subject_key][condition_pair]['labels']

            # Trova le etichette uniche e gli indici corrispondenti per quella coppia di condizioni sperimentali lì,
            # Ossia, potrà trovare gli 0 e gli 1 

            unique_labels = np.unique(subject_labels)

            #A quel punto, per ognuna delle due labels trovate per quella coppia di condizioni sperimentali lì
            #Che saranno sempre o 0 o 1

            for label in unique_labels:

                # Trova gli indici dei dati corrispondenti a questa etichetta
                # Una volta per lo 0 ed una volta per l' 1

                label_indices = np.where(subject_labels == label)[0]

                # A quel punto, estrae i dati e le labels per questi indici
                # Una volta per lo 0 ed una volta per l' 1

                data_for_label = subject_data[label_indices]
                labels_for_label = subject_labels[label_indices]

                # A quel punto, per ogni soggetto, per ogni condzione sperimentale, 
                # per quelle due labels là (sempre presenti, per ogni coppia di condizioni sperimentali)
                # Una volta per lo 0 ed una volta per l' 1

                #Andrà a creare per ogni etichetta, una chiave che si chiamerà 
                #o 0 od 1 (inizialmente, vuoto -> perché deve esser inizializzato)
                #E poi, dopo, ci appenderà le labels 
                 # Una volta per lo 0 ed una volta per l' 1
                #  Di quel soggetto per quella coppia di condizioni sperimentali iterate a quel momento

                #Aggiunge ai dizionari globali, creando la chiave se non esiste
                if label not in data_by_label:
                    data_by_label[label] = []
                    labels_by_label[label] = []

                data_by_label[label].append(data_for_label)
                labels_by_label[label].append(labels_for_label)

                # **Salva la shape delle etichette per il soggetto corrente**
                shape_labels_per_subject[label].append(labels_for_label.shape[0]) # Aggiungi solo la dimensione (numero di etichette)

                '''PRINT PER CHECK LABELS DI OGNI SOGGETTO PER OGNI COPPIA DI CONDIZIONE CONDIZIONI SPERIMENTALI''' 
                #print(f"Soggetto: \033[1m{subject_key}\033[0m, Livello: \033[1m{reconstruction_level}\033[0m, Condizione: \033[1m{condition_pair}\033[0m, "
                #      f"Etichetta: \033[1m{label}\033[0m, Shape: \033[1m{labels_for_label.shape[0]}\033[0m")

                # **Aggiungi un print di controllo per ogni soggetto**
                #print(f"Soggetto: \033[1m{subject_key}\033[1m, Livello: {reconstruction_level}, Condizione: {condition_pair}, Etichetta: {label}, Shape: {labels_for_label.shape[0]}")

            
        #*** *** *** *** *** *** *** *** *** *** *** *** *** *** ***

        # Dopodiché, vado a creare invece i dizionari che conterranno
        # Le concatenazioni, di dati e labels corrispondenti, di tutti i soggetti per cui l'etichetta era 
        # o 0 od 1 

        # Di conseguenza, qui dentro dovrei avere, per ogni coppia di condizioni sperimentali
        # Tutti gli 0 ed 1 (ed i relativi dati corrispondenti)
        # di tutti i soggetti, ma concatenati

        concatenated_data_by_label = {}
        concatenated_labels_by_label = {}

        #for label in data_by_label.keys():
        #    if len(data_by_label[label]) > 0:  # Evita errori di concatenazione
        #        concatenated_data_by_label[label] = np.vstack(data_by_label[label])
        #        concatenated_labels_by_label[label] = np.hstack(labels_by_label[label])
            
        for label in data_by_label.keys():
            concatenated_data_by_label[label] = np.vstack(data_by_label[label])
            concatenated_labels_by_label[label] = np.hstack(labels_by_label[label])

            # **Calcolare la shape totale delle etichette**
            #total_labels_0 = np.sum(1 for subject_labels in labels_by_label[label] if 0 in subject_labels)
            #total_labels_1 = np.sum(1 for subject_labels in labels_by_label[label] if 1 in subject_labels)
            #total_labels = total_labels_0 + total_labels_1

            # **Determinare gli indici delle etichette 0 e 1 nell'array finale**

            # Gli indici delle etichette 0
            indices_labels_0 = np.where(concatenated_labels_by_label[label] == 0)[0]

            # Gli indici delle etichette 1
            indices_labels_1 = np.where(concatenated_labels_by_label[label] == 1)[0]

            # Indici iniziale e finale per le etichette 0 e 1
            start_idx_0 = indices_labels_0[0] if len(indices_labels_0) > 0 else None
            end_idx_0 = indices_labels_0[-1] if len(indices_labels_0) > 0 else None
            start_idx_1 = indices_labels_1[0] if len(indices_labels_1) > 0 else None
            end_idx_1 = indices_labels_1[-1] if len(indices_labels_1) > 0 else None


            # **Print finale per verificare la concatenazione per ogni label**
            #print(f"\nCondizione: \033[1m{condition_pair}\033[0m, Etichetta: {label}")
            #print(f"  - Shape dei dati concatenati per \033[1m{label}\033[0m: {concatenated_data_by_label[label].shape}")
            #print(f"  - Shape delle etichette concatenate per \033[1m{label}\033[0m: {concatenated_labels_by_label[label].shape}")

            # **Stampa la lista delle shapes delle etichette per ogni soggetto per questa etichetta**
            #print(f"\n  - Shape delle etichette per soggetto (per etichetta {label}): {shape_labels_per_subject[label]}\n")


            # **Stampa il conteggio totale delle etichette**
            #print(f"\n  - Totale delle etichette 0: {total_labels_0}")
            #print(f"  - Totale delle etichette 1: {total_labels_1}")
            #print(f"  - Totale delle etichette per la condizione: {total_labels}\n")

            # **Stampa gli indici per le etichette 0 e 1**
            # **OSSIA --> Stampa gli indici per l'etichetta corrente**

            #if label == 0:
            #    print(f"\n  - Indici per etichetta 0: Inizio = {start_idx_0}, Fine = {end_idx_0}")
            #else:
            #    print(f"  - Indici per etichetta 1: Inizio = {start_idx_1}, Fine = {end_idx_1}\n")


            #print(f"\n  - Indici per etichetta 0: Inizio = {start_idx_0}, Fine = {end_idx_0}")
            #print(f"  - Indici per etichetta 1: Inizio = {start_idx_1}, Fine = {end_idx_1}")


        #ARRIVATI FINO A QUI, abbiamo che siccome stiamo iterando prima per tutti gli 0 e poi per tutti gli 1
        #Significa che, assumendo che siamo dentro 'baseline_vs_th_resp' e che 

        #'baseline' sia rappresentato dalle etichette 0
        #'th_resp' sia rappresentato dalle etichette 1

        #Al PRIMO ciclo avrò che:

        #con concatenated_data_by_label[label] ho solo concatenato tutti i dati (di 'baseline' per 'baseline_vs_th_resp' di tutti i soggetti)
        #con concatenated_labels_by_label[label] ho solo concatenato tutti gli 0 (di 'baseline' per 'baseline_vs_th_resp'di tutti i soggetti)

        #Al SECONDO ciclo avrò che:

        #con concatenated_data_by_label[label] ho solo concatenato tutti i dati (di 'th_resp' per 'baseline_vs_th_resp'  di tutti i soggetti)
        #con concatenated_labels_by_label[label] ho solo concatenato tutti gli 0 (di 'th_resp' per 'baseline_vs_th_resp' di tutti i soggetti)

        #Quindi mi manca ancora 

        #A) CONCATENARE I DATI:

        #1)prima tutti i dati (di 'baseline' per 'baseline_vs_th_resp' di tutti i soggetti) 

        #CON

        #2)tutti i dati (di 'th_resp' per 'baseline_vs_th_resp'  di tutti i soggetti)

        #che verrà fatto dentro all_data (che è anch'esso una variabile dinamica!)

        #B) CONCATENARE LE LABELS:

        #1)prima tutti le labels (di 'baseline' per 'baseline_vs_th_resp' di tutti i soggetti) 

        #CON

        #2)tutti le labels (di 'th_resp' per 'baseline_vs_th_resp' di tutti i soggetti)

        #che verrà fatto dentro all_labels (che è anch'esso una variabile dinamica!)

        # Liste per raccogliere dati e labels di tutte le etichette

        # Qui dentro, invece, dov con

        all_data = []
        all_labels = []

        for label in concatenated_data_by_label.keys():
            all_data.append(concatenated_data_by_label[label])
            all_labels.append(concatenated_labels_by_label[label])

        #Alla fine, qui dentro avrò che, PER OGNI COPPIA DI CONDIZIONI SPERIMENTALI


        #'final_data' dovrebbe avere 
            #- prima tutti i dati di tutti i soggetti associati all'etichetta 0 
            #- e poi tutti i dati di tutti i soggetti associati all'etichetta 1

        #'final_labels' dovrebbe avere 
            #- prima tutte le labels di tutti i soggetti associati all'etichetta 0
            #- e poi tutte tutte le labels di tutti soggetti associati all'etichetta 1...

        if all_data:  # Evita errori di concatenazione se non ci sono dati
            final_data = np.vstack(all_data)
            final_labels = np.hstack(all_labels)

            all_subj_data_by_coupled_cond[condition_pair] = {
                'data': final_data,
                'labels': final_labels
            }
            
    return all_subj_data_by_coupled_cond


In [ ]:
# Parametri


#conditions = ['baseline_vs_th_resp', 'baseline_vs_pt_resp', 'baseline_vs_shared_resp',
#              'th_resp_vs_pt_resp', 'th_resp_vs_shared_resp', 'pt_resp_vs_shared_resp']

    
conditions = ['rest_vs_left_fist', 'rest_vs_right_fist', 'left_fist_vs_right_fist']


# Chiamata alla funzione, qui definisco a mano il nome della variabile (i.e., 
new_all_th_concat_spectrograms_coupled_exp_task_1 = concatenate_all_subjects_coupled_spectrogram_conditions_th(
    data_structure=new_subject_level_concatenations_spectrograms_coupled_exp_th_task_1,
    conditions=conditions
)

#print("\n\033[1mIntervalli di indici per ciascuna etichetta nei sotto-dizionari:\033[0m\n")


for condition_pair, value in new_all_th_concat_spectrograms_coupled_exp_task_1.items():
    labels = value['labels']
    unique_labels = np.unique(labels)
    total_labels = 0

    #print(f"\nCondizione: \033[1m{condition_pair}\033[0m")
    for label in unique_labels:
        label_indices = np.where(labels == label)[0]
        start_idx = label_indices[0] if len(label_indices) > 0 else None
        end_idx = label_indices[-1] if len(label_indices) > 0 else None
        total_labels += len(label_indices)

        #print(f"  Etichetta \033[1m{label}\033[0m: Indici \033[1m{start_idx}-{end_idx}\033[0m "
        #      f"(Totale Etichette: \033[1m{len(label_indices)}\033[0m)")

    #print(f"Totale etichette (0 e 1): \033[1m{total_labels}\033[0m\n")

In [ ]:
print('finito')

In [ ]:
new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist'].keys()

In [ ]:
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['left_fist_vs_right_fist']['labels'], return_counts = True))

In [ ]:
'''
Ecco un piccolo snippet che, per ciascuna coppia di condizioni in new_all_th_concat_spectrograms_coupled_exp_task_1, 
stampa per ogni valore di label (0 e 1) l’indice di partenza e quello di fine nel vettore labels:

Come funziona

np.unique(labels) restituisce i valori distinti (ad es. [0, 1]).

Per ciascun valore val, np.where(labels == val)[0] trova tutti gli indici in cui labels è uguale a val.

Con .min() e .max() ricavi il primo e l’ultimo indice di ciascun blocco.

Se vuoi anche vedere tutte le sott‐sequenze (ad es. se le label non sono concatenate in un unico blocco), 
puoi invece usare [itertools.groupby] o scorrere i idxs alla ricerca di “buchi”. 

Ma nel tuo caso — visto che hai prima aggiunto tutti gli zero e poi tutti gli uno — vedrai sempre un singolo intervallo per ciascuna label.

'''

import numpy as np

for cond_pair, cond_data in new_all_th_concat_spectrograms_coupled_exp_task_1.items():
    labels = np.array(cond_data['labels'])
    print(f"\n=== {cond_pair} ===")
    for val in np.unique(labels):
        idxs = np.where(labels == val)[0]
        start, end = idxs.min(), idxs.max()
        print(f"  label {val}: indices {start} → {end} (count = {len(idxs)})")
        


In [ ]:
for cond_pair, cond_data in new_all_th_concat_spectrograms_coupled_exp_task_1.items():
    data = cond_data['data']      # shape: (n_trials, ...)
    labels = cond_data['labels']  # shape: (n_trials,)
    print(f"{cond_pair}:")
    print(f"  data shape  = {data.shape}")
    print(f"  labels len  = {len(labels)}\n")

##### STEP - Salvataggio Dataset di **All Single Therapists** EEG Data Spectrograms across **Couples of Experimental Conditions** INSIEME

In [ ]:
pwd

In [ ]:
cd ..

In [ ]:
base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
import pickle

# Salvare l'intero dizionario annidato con pickle
with open(f'{base_path}new_all_th_concat_spectrograms_coupled_exp_task_1_2d_grid_multiband.pkl', 'wb') as f:
    pickle.dump(new_all_th_concat_spectrograms_coupled_exp_task_1, f)

## Impostazione **Weight & Biases DL Training** con **Rappresentazione Tempo-Frequenza dei miei dati EEG** a seconda del Dataset del Task scelto


Notebook **"Optimization Weight and Biases Time Domain EEG Signal"**

- Task 1: Motor Execution (Open and close right or left fist) - Unilateral - Runs 3, 7, 11 
- Task 2: Motor Imagery (Imagine opening and closing right and left fists) - Unilateral - Runs 4, 8, 12 
- Task 3: Motor Execution (Open and close both fists and feet) - Bilateral - Runs 5, 9, 13 
- Task 4: Motor Imagery (Imagine opening and closing both fists and feet) - Bilateral - Runs 6, 10, 14 

#### **Weight & Biases Login & REMOTE CHECKS - EEG Spectrograms - Time x Frequencies**

##### **Weight & Biases Login PRECEDURA CORRETTA ✅ (CANCELLAZIONE RUNS e NON SWEEPS!) - EEG Spectrograms - Time x Frequencies**

In [ ]:
'''

Puoi usare l’API Python di W&B per elencare da remoto tutti i progetti di una certa entity (tuo utente o organizzazione).

Ad esempio:

'''

import wandb

# 1) autenticati (se non l’hai già fatto con wandb.login())
# wandb.login()

# 2) crea un’istanza dell’API
api = wandb.Api()

# 3) scegli la tua entity (nome utente o org). 
#    Se omesso, prende la tua entity di default.

entity = 'stefano-bargione-universit-di-roma-tor-vergata'

# 4) prendi la lista dei progetti
projects = api.projects(entity=entity)

# 5) stampa i nomi
for proj in projects:
    print(proj.name)

In [ ]:
###TUTTE ASSIEME


import wandb

# Nomi dei progetti
project_names = [
    'left_fist_vs_right_fist_spectrograms_new',
    'rest_vs_right_fist_spectrograms_new',
    'rest_vs_left_fist_spectrograms_new'
]

user_name = 'stefano-bargione-universit-di-roma-tor-vergata'

# Connessione all'API di W&B
api = wandb.Api()

# Itera su tutti i progetti
for project_name in project_names:
    print(f"\nSelezione delle run per il progetto: \033[1m{project_name}\033[0m")
    
    # Recupera tutte le run del progetto
    runs = api.runs(f"{user_name}/{project_name}")
    
    # Salva gli ID delle run da eliminare
    run_ids_to_delete = []
    
    for run in runs:
        #print(f"Estrazione run ID: {run.id} - Run Name: {run.name}")
        run_ids_to_delete.append(run.id)  # Aggiungi l'ID della run alla lista
    
    print(f"\nTotale runs da eliminare: \033[1m{len(run_ids_to_delete)}\033[0m")
    
    # Elimina le run
    for run_id in run_ids_to_delete:
        run = api.run(f"{user_name}/{project_name}/{run_id}")
        run.delete()  # Elimina la run
        #print(f"Eliminata la run con ID: {run_id}")
    print(f"Eliminazione runs completata")

        

#### **IO**

Prima avevo avviata una ricerca di iper-parametri con API da jupyter e avevo impostato la creazione di cartelle per ogni combinazione di dati...

tuttavia, ora quando andava ho notato che le cartelle non so state create alle fine.. come mai? è dovuto ad un overload di sovraccarica o è semplicemente un ritardo dovuto all'esecuzione di grande mole di dati da remoto tra l'altro con l'api che ritarda la sincronizzazione delle operazioni del codice su quello che dovrei vedere su jupyter? 

è fondamentale che le cartelle ed i vari files .pkl del mio codice si salvino, perché poi io le richiamo successivamente per estrarmi gli iper-parametri migliori di ogni ottimizzazione fatta.. devo gestire le risorse di calcolo in modo più equilibrato, affinché le operazioni di salvataggio vengano eseguite correttamente?

#### **CHATGPT**

In [ ]:
È molto più probabile che la cartella ci sia già, ma:

Non si trovi sotto la radice che Jupyter ti mostra (di solito /mnt/data), bensì in /home/stefano/..., per cui il file‑browser non la “vede” fino a quando non navighi esplicitamente lì.

O che tu abbia settato WANDB_DIR dopo l’import wandb, quindi wandb abbia già inizializzato il suo percorso di default.

In ogni caso non è un problema di “troppa mole di dati che rallenta” o di W&B che perde file, bensì di:

percorso sbagliato in cui guardi

sincronizzazione asincrona di wandb (che salva localmente in run.dir e lo invia al cloud in background)

Controlla davvero se la cartella esiste
Nella cella del notebook, esegui:

bash
Copia
Modifica
!ls -l /home/stefano/Interrogait/WB_spectrograms_analyses_motor_task_1
o più genericamente

bash
Copia
Modifica
!find /home/stefano/Interrogait -maxdepth 2 -type d
Se la cartella esiste veramente, basta che la navighi nel file‑browser (“…” in alto a sinistra → “Change directory” → incolli quel path) o che tu salvi direttamente in /mnt/data/... così la vedi subito.

Come forzare wandb a usare la cartella che vuoi
Metti prima di ogni import wandb queste due righe:

python
Copia
Modifica
import os
WB_dir = "/mnt/data/WB_spectrograms_analyses_motor_task_1"
os.makedirs(WB_dir, exist_ok=True)
os.environ["WANDB_DIR"] = WB_dir
Poi:

python
Copia
Modifica
import wandb
wandb.login()
run = wandb.init(
    project="spectrograms-analyses-motor-task-1",
    dir=WB_dir,         # <–– anche qui puoi sovrascrivere il path
    resume="allow"      # <–– se fai sweep/runs multipli
)
In questo modo:

Wandb salverà localmente tutti i log e i file di run in WB_dir.

Il file‑browser di Jupyter (che punta su /mnt/data) li vedrà immediatamente.

Wandb poi sincronizzerà tutto in cloud, ma tu puoi lavorare “in locale” senza perdere nulla.

Salvare i .pkl
Quando salvi i tuoi .pkl all’interno della loop di hyperopt fai:

python
Copia
Modifica
out_path = os.path.join(WB_dir, f"{run.id}__{combo_name}.pkl")
with open(out_path, "wb") as f:
    pickle.dump(my_results, f)
wandb.save(out_path)   # <–– forza wandb ad aggiungere questo file allo snapshot del run
In questo modo:

Il file viene scritto subito in WB_dir

Viene collegato al run W&B (utile se poi fai wandb.restore)

Verifica “a freddo”
Dopo aver fatto partire i tuoi esperimenti, in un’altra cella puoi lanciare:

bash
Copia
Modifica
!ls -R /mnt/data/WB_spectrograms_analyses_motor_task_1
oppure – da CLI –

bash
Copia
Modifica
wandb files <ENTITY>/<PROJECT>/<RUN_ID>
per vedere l’elenco dei file che sono stati salvati/sincronizzati.

Se invece proprio non trovi nulla, controlla:

che non ci siano errori silenziosi (un try/except: pass che assorbe i problemi di I/O)

che tu stia eseguendo la cella di creazione delle cartelle prima di qualunque wandb.init

che non stia scrivendo in un container o in un filesystem ephemeral diverso da quello mostrato dal file‑browser

Con questa configurazione dovresti avere:

Cartelle create sotto /mnt/data/WB_spectrograms_analyses_motor_task_1

I tuoi .pkl visibili subito in Jupyter

Wandb che continua a fare upload in background senza perderli

Fammi sapere se così riesci a trovarli ed evitare di perdere i risultati dei tuoi sweep!

### Optimization Weight and Biases - EEG Spectrograms - Frequency x Time  - Con CNN2D_LSTM

#### **Weight & Biases Procedure FINAL SEQUENCE OF STEPS - EEG Spectrograms - Time x Frequencies ONLY HYPER-PARAMS**

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm
from tqdm import tqdm

import random 

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la visualizzazione dei dati

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader

import os
import pickle

import random

import wandb

In [ ]:
base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
import pickle
import numpy as np

base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/'
    
# Caricare l'intero dizionario annidato con pickle
with open(f'{base_path}new_all_th_concat_spectrograms_coupled_exp_task_1.pkl', 'rb') as f:
    new_all_th_concat_spectrograms_coupled_exp_task_1 = pickle.load(f)

In [ ]:
new_all_th_concat_spectrograms_coupled_exp_task_1.keys() 

In [ ]:
new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist'].keys()

In [ ]:
print(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist']['data'].shape)
print()
print(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist']['data'].shape)
print()
print(new_all_th_concat_spectrograms_coupled_exp_task_1['left_fist_vs_right_fist']['data'].shape)
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['left_fist_vs_right_fist']['labels'], return_counts = True))

#### **Utils Functions - EEG Spectrograms - Time x Frequencies**

In [ ]:
'''
QUI DENTRO HO CONFIGURATO 
LE FUNZIONI DI CONTROLLO DELLE STRINGHE 
PER IL SALVATAGGIO DELLE PERFORMANCE DEL MODELLO
NELLE RELATIVE SUBFOLDERS

(I.E., get_subfolder_from_key, get_subfolder_from_key_hyper)

IN MODO CHE SI LEGHINO ALLA CHIAVE 'STANDARDIZATION' DELL'OGGETTO SWEEP_CONFIG

'''


def load_data(data_type, category, subject_type, condition = "th_resp_vs_pt_resp"):
    """
    Carica i dati EEG dalla directory appropriata, già salvati con la finestra temporale (0°-640° punto)

    Parameters:
    - data_type: str, "spectrograms",
    - category: str, "task_1"
    - subject_type: str, "th" (terapisti)
    - condition: str, condizione sperimentale da selezionare
    

    Returns:
    - X: Dati EEG sotto-selezionati (50°-300° punto e canali selezionati se applicabile)
    - y: Etichette corrispondenti
    """

    # Definizione dei percorsi base
    base_paths = {
        "spectrograms": {
            "familiar": "/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/"
            #"unfamiliar": "/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms/"
        },
    }

    # Seleziona il path corretto
    base_path = base_paths[data_type][category]

    # Determina il nome del file corretto
    if data_type in ["spectrograms"]:
        filename = f"new_all_{subject_type}_concat_spectrograms_coupled_exp_task_1.pkl"
    else:
        raise ValueError("data_type non valido!")
        
    # Caricamento del file
    filepath = base_path + filename
    
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    
    '''
    Per i dati spectrogram, la funzione seleziona la condizione desiderata (i.e., condition = "th_resp_vs_pt_resp") 
    e preleva i dati e le etichette associati a quella condizione.
    '''
    
    # Selezione della finestra temporale e delle etichette
    X = data[condition]["data"]
    y = data[condition]["labels"]

    
    return X, y


def select_channels(data, channels=[12, 30, 48]):
    """
    Seleziona i canali EEG specificati SOLO per i dati 1-20 e 1-45.

    Parameters:
    - data: array NumPy, dati EEG con shape (n_trials, n_channels, n_timepoints)
    - channels: list, indici dei canali da selezionare

    Returns:
    - data filtrato sui canali specificati
    """
    return data[:, channels, :]


# Funzione per train-test split
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

def split_data(X, y, test_size=0.2, val_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test

'''ATTENZIONE CAMBIATA'''
# Funzione per standardizzare i dati
def standardize_data(X_train, X_val, X_test, eps = 1e-8):
    
    mean = X_train.mean(axis=0, keepdims=True)
    std = X_train.std(axis=0, keepdims=True)
    
    X_train = (X_train - mean) / (std + eps)
    X_val = (X_val - mean) / (std + eps)
    X_test = (X_test - mean) / (std + eps)
    
    return X_train, X_val, X_test



# Import modelli (definisci le classi CNN1D, ReadMEndYou, ReadMYMind)
#from models import CNN1D, ReadMEndYou, ReadMYMind  # Assicurati di avere i modelli definiti in 'models.py'

# Funzione per inizializzare i modelli
def initialize_models():
    #model = CNN1D(input_channels=3, num_classes=2)
    #model_CNN = CNN2D(input_channels=3, num_classes=2)
    
    model_CNN = CNN2D(input_channels=64, num_classes=2)
    
    #model_LSTM = ReadMEndYou(input_size=3, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    #model_LSTM = ReadMEndYou(input_size=3 * 26, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    
    model_LSTM = ReadMEndYou(input_size=64 * 81, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    
    #model_Transformer = ReadMYMind(num_channels=3, seq_length=250, d_model=16, num_heads=4, num_layers=2, num_classes=2)
    #model_Transformer = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=3, freqs=26)
    
    model_Transformer = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=64, freqs=81)
    
    
    return model_CNN, model_LSTM, model_Transformer


import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight


'''
Questa funzione prende in input i dati di training, validation e test, 
il tipo di modello scelto e la dimensione del batch. Si occupa di:

Calcolare i pesi delle classi.
Convertire i dati in tensori PyTorch, con le opportune trasformazioni per CNN, LSTM o Transformer.
Creare i dataset e i dataloader per il training.
'''


def prepare_data_for_model(X_train, X_val, X_test, y_train, y_val, y_test, model_type, batch_size=48):
    
    # Calcolo dei pesi delle classi
    class_weights = compute_class_weight(class_weight='balanced', 
                                         classes=np.unique(y_train), 
                                         y=y_train)
    
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
    class_weights_tensor = class_weights_tensor.to(dtype=torch.float32, device=device)
    
    # Conversione delle etichette in interi
    y_train = y_train.astype(int)
    y_val = y_val.astype(int)
    y_test = y_test.astype(int)
    
    # Conversione dei dati in tensori PyTorch con permutazione se necessario
    
    '''OCCHIO QUI CAMBIATO'''
    #if model_type == "CNN2D":
    
    if model_type == "CNN2D_LSTM_TF":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #BiLSTM (ReadMEndYou):
    #Ora il modello si aspetta l’input con shape (batch, canali, frequenze, tempo) 
    #e, al suo interno, 
    #esegue la permutazione per avere il tempo come dimensione sequenziale. 
    #Non serve quindi applicare una permutazione anche qui.
    
    elif model_type == "BiLSTM":
            
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #Transformer (ReadMYMind):
    #Analogamente, il modello gestisce internamente la riorganizzazione dell’input, quindi lasciamo i dati nella loro forma originale.
    elif model_type == "Transformer":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    else:
        raise ValueError("Modello non riconosciuto. Scegli tra 'CNN', 'LSTM' o 'Transformer'.")
    
    # Conversione delle etichette in tensori
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    # Creazione dei dataset
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    # Creazione dei dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, class_weights_tensor



# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, model_standardization):
    
    #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
    if '_familiar_th' in key:
        return 'th_fam'
    elif '_unfamiliar_th' in key:
        return 'th_unfam'
    elif '_familiar_pt' in key:
        return 'pt_fam'
    elif '_unfamiliar_pt' in key:
        return 'pt_unfam'
    else:
        return None
     
   
    
# Funzione per salvare i risultati
def save_performance_results(model_name, my_train_results, my_test_results, key, data_type, sweep_config, condition = "th_resp_vs_pt_resp"):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, sweep_config)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
     # Determinazione del tipo di dato direttamente dalla chiave
    if "spectrograms" in key:
        data_type_str = "spectrograms"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    file_name = f"{model_name}_performances_{condition}_{subfolder}_{data_type_str}.pkl"
    folder_path = os.path.join(base_folder, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo ✅ in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"Errore durante il salvataggio dei risultati: {e}")

#### **DL Models - EEG Spectrograms - Time x Frequencies**

In [ ]:
'''
Per far sì che la SOMMA ELEMENT-WISE DEL PERCORSO RESIDUO sia valida, il numero di canali di res deve coincidere con quello dell’output convoluzionale (64).
Nel nostro blocco 2, però:

- Il path principale prende in ingresso 32 canali (x dopo il primo pool) e, attraverso due convoluzioni, esce con 64 canali.
- Il path residuo (res = x) resta a 32 canali: non possiamo sommarlo al main path a 64 canali.

La soluzione “standard” in ResNet è applicare sullo skip una 1×1 convolution che porta i 32 canali di res a 64, esattamente come il main path.

In PyTorch:

1.  Aggiungi un layer di “proiezione” 1×1 nel blocco 2
    
    Nel costruttore (__init__), subito prima di definire il blocco 2, dichiari:

    # in __init__(), subito dopo self.pool o prima di bn2a
    self.res_conv = nn.Conv2d(32, 64, kernel_size=1, stride=1, padding=0, bias=False)
    self.res_bn   = nn.BatchNorm2d(64)   # (opzionale, migliora la stabilità)

2. Applica la proiezione nel forward
   
   Nel forward, nel blocco 2 sostituisci:

    # prima:
    res = x
    
    # ... convoluzioni ...
    x = x + res
    
    
    con invece questa altra cosa 👇


    # prendi il “residuo” su cui fare la somma
    res = x  

    # proietta res da 32→64 canali
    res = self.res_conv(res)  # (B,32,40,4) -> (B,64,40,4)
    res = self.res_bn(res)    # (B,64,40,4)

    # main path
    x = self.bn2a(x)
    x = F.relu(self.conv2a(x))
    x = self.bn2b(x)
    x = self.conv2b(x)        # (B,64,40,4)

    # ora i due tensori hanno entrambi 64 canali
    x = x + res               # (B,64,40,4)
    x = F.relu(x)
    x = self.pool(x)          # (B,64,20,2)


3. Il blocco 2 diventa così

    # --- Block 2 (residual with projection) ---
    # in __init__:
    self.res_conv = nn.Conv2d(32, 64, kernel_size=1, bias=False)
    self.res_bn   = nn.BatchNorm2d(64)
    self.bn2a  = nn.BatchNorm2d(32)
    self.conv2a= nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.bn2b  = nn.BatchNorm2d(64)
    self.conv2b= nn.Conv2d(64, 64, kernel_size=3, padding=1)

    # in forward:
    res = x
    res = self.res_conv(res)
    res = self.res_bn(res)

    x = self.bn2a(x)
    x = F.relu(self.conv2a(x))
    x = self.bn2b(x)
    x = self.conv2b(x)

    x = x + res
    x = F.relu(x)
    x = self.pool(x)
    


Perché funziona

- res_conv con kernel 1×1 ricalcola solo i canali, lasciando inalterata la risoluzione spaziale (40 × 4).
- Ora res ha 64 canali e può essere sommato all’output del main path, che ha a sua volta 64 mappe feature.
- La BatchNorm aggiunta dopo la proiezione aiuta a mantenere la stessa distribuzione statistica di ciò che accade nel main path.

Con questo piccolo accorgimento rendi il tuo blocco 2 conforme alla ResNet originaria e senza errori di shape nella somma residua.

******************* ******************* ******************* ******************* ******************* ******************* ******************* *******************

[...]

IO: Ok, ma che cosa cambia? oppure, rimane uguale all'implementazione descritta nel paper con questa ulteriore modifica?


******************* ******************* ******************* ******************* ******************* ******************* ******************* *******************
[...]

Non cambia nulla in termini di capacità espressiva o di “architettura astratta” rispetto al paper, perché quella 1×1 “di proiezione” 
è semplicemente un dettaglio d’implementazione che serve a far combaciare i canali quando costruisci il blocco residuo in PyTorch.


    Cosa scrive il paper
    Block 1: 1×1 input→32 canali → pool
    Block 2: due convoluzioni 3×3 con 64 filtri ciascuna, somma con l’input del blocco → pool
    Block 3: conv 3×3 con 128 filtri → pool

Nel testo non viene esplicitato, ma implicitamente per fare


    input_block2  (B,32,H,W)
→ conv2a & conv2b (B,64,H,W)
→ add(input_block2, output)  # somma residua
devi “allineare” i 32 canali dell’input ai 64 del main path.



Perché serve la 1×1 conv
In PyTorch, se fai

    res = x              # (B,32,H,W)
    x   = conv2a(x)      # → (B,64,H,W)
    x   = conv2b(x)      # → (B,64,H,W)
    x   = x + res        # ❌ ERROR: 64 vs 32 canali
    

devi prima trasformare res da 32→64 canali, e il modo “canonicamente” corretto è una convoluzione 1×1:


    res = self.res_conv(res)  # Conv2d(32→64, kernel_size=1)
    res = self.res_bn(res)z
    # ora res e x hanno entrambi 64 canali
    x   = x + res             # ✅ OK


In che modo resta fedele al paper:

    Numero di filtri nei tre blocchi (32 → 64 → 128)
    Kernel 3×3 e padding=“same”,
    MaxPool2d(2,2) dopo ogni blocco,
    Dropout 0.2,
    LSTM unidirezionale 64,
    Head finale 3-class softmax,

    Tutto rimane identico-->  La 1×1 conv è solo un’aggiunta necessaria per realizzare “meccanicamente” la somma residua quando l’input
    e l’output del blocco hanno numero diverso di canali.

Quindi:

Architettura logica = quella del paper (3 blocchi, residual sul secondo, LSTM, …)
Dettaglio d’implementazione = aggiunta della 1×1 conv per far combaciare le shape interne in PyTorch
Senza quella 1×1 conv, il tuo codice andrebbe in errore di dimensione, ma dal punto di vista del paper non stai in alcun modo “cambiando” 
le scelte di filtro, pooling o flusso dei dati.

******************* ******************* ******************* ******************* ******************* ******************* ******************* *******************


Ok vorrei che mi facessi 3 cose:

1) mi rispiegassi questo passaggio "somma element-wise del percorso residuo sia valida"
2) nel mio caso, parto da una matrice di dati EEG a 64 canali, e nella tua implementazione manca questa info o mi sbaglio?
3) mi ridai tutta la classe della rete modificata in base a questo discorso che mi facevi rispetto al " Aggiungi un layer di “proiezione” 1×1 nel blocco 2"?


******************* ******************* ******************* ******************* ******************* ******************* ******************* *******************



1) “SOMMA ELEMENT-WISE DEL PERCORSO RESIDUO SIA VALIDA”

Quando fai un blocco residuale (come in ResNet), hai due “rami”:

Main path: applica due (o più) convoluzioni al tensore di input

Skip path: trasporta l’input in parallelo, per sommarlo all’output del main path

Per poter sommare due tensori element‐wise, devono avere esattamente la stessa shape in tutte le dimensioni: batch, canali, altezza, larghezza.

Nel nostro caso:

Input al blocco ha shape (B, C_in, H, W).

Main path lo trasforma in (B, C_out, H, W) tramite due conv 3×3 (qui C_out=64).

Skip path inizialmente era semplicemente res = input, dunque res.shape == (B, C_in, H, W) con C_in=32.

Non puoi quindi fare

    # ❌ non funziona, 32 ≠ 64 canali
    out = main_path(input)      # shape (B,64,H,W)
    res = input                 # shape (B,32,H,W)
    sum = out + res             # ERRORE di broadcast sui canali

Per “allineare” i canali del skip a quelli del main path, inserisci una convoluzione 1×1 che mappa C_in→C_out:


    res = conv1x1(res)  # ora res.shape == (B,64,H,W)
    out = main_path(input)  # (B,64,H,W)
    sum = out + res         # ✅ canali e spatial matchata → somma valida
    
Questa 1×1 conv non cambia altezza/larghezza, ma riallinea il numero di feature‐map (canali).



2) Numero di canali di input nel tuo caso
Dati EEG originali: 61 elettrodi, poi ridotti a 15 per soggetto, quindi 30 canali totali per il dataset paired.

Se tu invece parti da un dataset a 64 canali (magari stai usando tutti i 61 electrodes + derivazioni), 
devi semplicemente passare quel numero di canali al modello:


model = CNNLSTM_TF(n_channels=64, n_classes=2, dropout=0.2)

In questo modo BatchNorm2d e Conv2d del primo blocco sapranno che devono aspettarsi input con 64 canali.


In [ ]:
'''
DEFINIZIONE DEI MODELLI NEW VERSION PER SPETTROGRAMMI
'''


'''CNN2D_LSTM_TF

Spiegazione dettagliata dei passaggi

1) Input

Shape Iniziale: (batch_size, 64, 81, 9)
64 canali EEG, 81 bin di frequenza, 9 finestre temporali

2) Block 1

BatchNorm2d(64) normalizza ciascun canale
Conv2d(64→32, 3×3, padding=1) produce 32 mappe (B,32,81,9)
ReLU mantiene la stessa shape
MaxPool2d(2,2) dimezza: frequenza 81→40, time 9→4 → (B,32,40,4)

3) Block 2 “residuo”

res = x salva skip (B,32,40,4)
res_conv 1×1: Conv2d(32→64,k=1) riallinea i canali → (B,64,40,4)
res_bn normalizza → (B,64,40,4)

    Main path:

        BatchNorm2d(32) → (B,32,40,4)
        Conv2d(32→64,3×3,pad=1) + ReLU → (B,64,40,4)
        BatchNorm2d(64) → (B,64,40,4)
        Conv2d(64→64,3×3,pad=1) → (B,64,40,4)

    Somma residua: (B,64,40,4) + (B,64,40,4) → (B,64,40,4)
    ReLU + Pool: dimezza → (B,64,20,2)

4) Block 3

    Conv2d(64→128,3×3,pad=1) + BatchNorm + ReLU → (B,128,20,2)
    MaxPool2d → frequenza 20→10, time 2→1 → (B,128,10,1)

5) Preparazione per LSTM

    permute(0,2,1,3): porta freq come sequenza → (B,10,128,1)
    reshape(b,10,128*1): collapse canali×time → (B,10,128)

6) LSTM + testata finale

    Dropout 0.2 su (B,10,128)
    LSTM unidirezionale 128→64: produce out = (B,10,64)
    Estrai ultimo timestep out[:, -1, :] → (B,64)
    Linear(64→2) → logits (B,2)

Così hai un modello che parte da 64 canali EEG, mantiene l’architettura a 3 blocchi residui + LSTM del paper, e termina con una classificazione binaria.

'''

import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN2D_LSTM_TF(nn.Module):

    def __init__(self, input_channels=64, num_classes=2, dropout=0.2):
        super().__init__()
        # --- Block 1 ---
        self.bn1   = nn.BatchNorm2d(input_channels)    # normalizza 64 canali
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool2d(2, 2)

        # --- Block 2 (residual) ---
        # Proiezione 1×1 per riallineare i canali di skip (32→64)
        self.res_conv = nn.Conv2d(32, 64, kernel_size=1, bias=False)
        self.res_bn   = nn.BatchNorm2d(64)

        self.bn2a   = nn.BatchNorm2d(32)
        self.conv2a = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2b   = nn.BatchNorm2d(64)
        self.conv2b = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # --- Block 3 ---
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(128)

        # --- Head: Dropout + LSTM + FC finale ---
        self.dropout     = nn.Dropout(dropout)
        self.hidden_size = 64
        
        # dopo 3 pool: freq da 81→10, time da 9→1 → feature per timestep = 128×1
        self.lstm = nn.LSTM(
            input_size=128,
            hidden_size=self.hidden_size,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )
        self.classifier = nn.Linear(self.hidden_size, num_classes)

    def forward(self, x):
        # x: (B,64,81,9)

        # --- Block 1 ---
        x = self.bn1(x)                   # → (B,64,81,9)
        x = F.relu(self.conv1(x))         # → (B,32,81,9)
        x = self.pool(x)                  # → (B,32,40,4)

        # --- Block 2 (residuo) ---
        res = x                           # skip: (B,32,40,4)
        res = self.res_conv(res)          # progetto: → (B,64,40,4)
        res = self.res_bn(res)            # → (B,64,40,4)

        # main path
        x = self.bn2a(x)                  # → (B,32,40,4)
        x = F.relu(self.conv2a(x))        # → (B,64,40,4)
        x = self.bn2b(x)                  # → (B,64,40,4)
        x = self.conv2b(x)                # → (B,64,40,4)

        x = x + res                       # somma residua valida → (B,64,40,4)
        x = F.relu(x)                     
        x = self.pool(x)                  # → (B,64,20,2)

        # --- Block 3 ---
        x = F.relu(self.bn3(self.conv3(x)))  # → (B,128,20,2)
        x = self.pool(x)                     # → (B,128,10,1)

        # --- Prepara per LSTM ---
        x = x.permute(0, 2, 1, 3)         # → (B,10,128,1)
        b, seq, ch, tw = x.size()        
        x = x.reshape(b, seq, ch * tw)    # → (B,10,128)

        # --- LSTM + classificazione ---
        out, _ = self.lstm(self.dropout(x))  # → out: (B,10,64)
        last = out[:, -1, :]                 # prendo l’ultima uscita → (B,64)
        logits = self.classifier(last)       # → (B,2)
        return logits


In [ ]:
'''
DEFINIZIONE DEI MODELLI NEW VERSION PER SPETTROGRAMMI
'''    
    

'''CNN2D

Una CNN2D si aspetta input in forma (batch, canali, altezza, larghezza). 
Nel tuo caso, puoi interpretare l’"altezza" come i bin di frequenza (38)
e la "larghezza" come le finestre temporali (6).

Quindi, la tua CNN2D lavorerebbe direttamente con:
Shape: (batch, canali, frequenze, tempo)




class CNN2D(nn.Module):
    
    def __init__(self, input_channels, num_classes):
        
        super(CNN2D, self).__init__()
        
        # Ipotizziamo kernel 3x3 con padding per mantenere le dimensioni (puoi adattare a tuo piacimento)
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2))
        
        self.conv3 = nn.Conv2d(32, 48, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn3 = nn.BatchNorm2d(48)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2))
        
        # Utilizzo LazyLinear per evitare di calcolare manualmente la dimensione piatta finale
        self.fc1 = nn.LazyLinear(8)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.LazyLinear(num_classes)
        
    def forward(self, x):
        
        # x: (batch, canali, frequenze, tempo)
        
        # Passaggio attraverso il primo strato convoluzionale, BatchNorm e pooling
        x = self.conv1(x)
        x = self.bn1(x)  # Batch Normalization
        x = F.elu(x)
        #x = torch.tanh(x)  # Sostituito ELU con tanh
        x = self.pool1(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv2(x)
        x = self.bn2(x)  # Batch Normalization
        x = F.elu(x)
        #x = torch.tanh(x)  # Sostituito ELU con tanh
        x = self.pool2(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv3(x)
        x = self.bn3(x)  # Batch Normalization
        x = F.elu(x)
        x = torch.tanh(x)  # Sostituito ELU con tanh
        x = self.pool3(x)

        # Flatten per preparare i dati per gli strati fully connected
        x = torch.flatten(x, start_dim=1)

        # Passaggio attraverso il primo strato fully connected
        x = self.fc1(x)
        x = F.elu(x)
        #x = torch.tanh(x)  # Sostituito ELU con tanh

        # Dropout per evitare overfitting
        x = self.dropout(x)

        # Passaggio attraverso il secondo strato fully connected
        x = self.fc2(x)

        return x


'''


'''
Gli LSTM si aspettano un input in forma (batch, lunghezza_sequenza, dimensione_feature). 
Dovrai quindi decidere qual è la dimensione sequenziale.

Opzione comune: usare il tempo come sequenza
Step 1: Trasponi i dati in modo da avere il tempo come dimensione sequenziale.

Dalla forma (batch, canali, frequenze, tempo) puoi fare:


x = x.permute(0, 3, 1, 2)  # Diventa (batch, tempo, canali, frequenze)

Step 2: Unisci le dimensioni dei canali e dei bin di frequenza in un’unica dimensione di feature:


batch, tempo, canali, frequenze = x.shape
x = x.reshape(batch, tempo, canali * frequenze)  # Ora: (batch, tempo, canali*frequenze)

Nel tuo caso, per 3 canali e 38 bin di frequenza: input_size = 3 * 38 = 114 e lunghezza sequenza = 6.

Nota: Se invece preferisci usare i bin di frequenza come sequenza, potresti fare:

x = x.permute(0, 2, 1, 3)  # (batch, frequenze, canali, tempo)
x = x.reshape(batch, frequenze, canali * tempo)  # Sequence length = 38, feature size = 3*6 = 18
La scelta dipende dal tipo di informazione temporale o spettrale che vuoi evidenziare.

'''

class ReadMEndYou(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, output_size, dropout=0.5, bidirectional=False):
        """
        input_size: dimensione delle feature per time-step (dovrà essere canali * frequenze)
        hidden_sizes: lista con le dimensioni degli hidden state, es. [24, 48, 62]
        output_size: numero di classi
        
        """
    
        super(ReadMEndYou, self).__init__()
        
        self.bidirectional = bidirectional # Impostazione della bidirezionalità    
        
        # Adattiamo hidden_size in base alla bidirezionalità
        self.hidden_sizes = [
            hidden_sizes[0] * 2 if bidirectional else hidden_sizes[0],
            hidden_sizes[1] * 2 if bidirectional else hidden_sizes[1],
            hidden_sizes[2] * 2 if bidirectional else hidden_sizes[2]
        ]
        
        self.lstm1 = nn.LSTM(input_size=input_size, 
                             hidden_size=self.hidden_sizes[0], 
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0, 
                             bidirectional=bidirectional)
        self.lstm2 = nn.LSTM(input_size=self.hidden_sizes[0] * 2 if bidirectional else self.hidden_sizes[0],
                             hidden_size=self.hidden_sizes[1], 
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0,
                             bidirectional=bidirectional)
        self.lstm3 = nn.LSTM(input_size=self.hidden_sizes[1] * 2 if bidirectional else self.hidden_sizes[1],
                             hidden_size=self.hidden_sizes[2],
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0,
                             bidirectional=bidirectional)
        
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.hidden_sizes[2] * 2 if bidirectional else self.hidden_sizes[2], output_size)
    
    def forward(self, x):
        
        # x: (batch, canali, frequenze, tempo)
        
        # Trasponi per avere il tempo come dimensione sequenziale:
        x = x.permute(0, 3, 1, 2)  # -> (batch, tempo, canali, frequenze)
        
        batch, time, channels, freqs = x.shape
        
        x = x.reshape(batch, time, channels * freqs)  # -> (batch, tempo, channels*frequencies)
        # Ora input_size deve essere channels * freqs (es. 64 * 81 = 7471)
        
        # LSTM 1
        out, _ = self.lstm1(x)
        out = self.dropout(out)
        
        # LSTM 2
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        
        # LSTM 3
        out, _ = self.lstm3(out)
        out = self.dropout(out)
        
        # Estraiamo l'output dell'ultimo time-step
        out = out[:, -1, :]
        
        # Dropout prima del layer fully connected    
        out = self.dropout(out)
        
        # Passaggio attraverso il layer finale per la previsione
        out = self.fc(out)
        return out
        


'''
Il modulo Transformer in PyTorch lavora tipicamente su input di forma (seq_length, batch, embedding_dim).

Nel codice attuale, si parte da una forma simile a (batch, canali, seq_length), ma dovrai adattarla alla nuova struttura.

Possibili approcci:

1) Approccio A: usare il tempo come sequenza

Se consideri il tempo (6 time windows) come la sequenza, puoi procedere come segue:

A) Unisci canali e frequenze in un’unica dimensione di feature:

# Dati originali: (batch, canali, frequenze, tempo)
x = x.permute(0, 3, 1, 2)  # (batch, tempo, canali, frequenze)
batch, tempo, canali, frequenze = x.shape
x = x.reshape(batch, tempo, canali * frequenze)  # (batch, tempo, 3*38 = 114)

B) Modifica il layer di embedding:

Nel codice attuale, l'embedding è definito come:

self.embedding = nn.Linear(seq_length, d_model)
Dovrai cambiarlo in modo che mappi le dimensioni delle feature (in questo caso 114) a uno spazio latente:

self.embedding = nn.Linear(canali * frequenze, d_model)

C) Permuta per il Transformer:

Dopo l'embedding, passa l'input alla forma (seq_length, batch, d_model):

x = x.permute(1, 0, 2)  # Ora: (tempo, batch, d_model)


2) Approccio B: usare i bin di frequenza come sequenza
In alternativa, se reputi più rilevante la risoluzione spettrale, puoi considerare i 38 bin come sequenza e combinare canali e tempo:


x = x.permute(0, 2, 1, 3)  # (batch, frequenze, canali, tempo)
batch, frequenze, canali, tempo = x.shape
x = x.reshape(batch, frequenze, canali * tempo)  # (batch, frequenze, 3*6 = 18)

E poi procedere con un embedding layer che mappa da 18 a d_model e permutare in (frequenze, batch, d_model).

Scelta dell'approccio:
Se l'aspetto temporale è più critico, probabilmente è meglio usare l’Approccio A (sequenza di lunghezza 6).
Se invece vuoi dare maggior rilievo alla struttura spettrale, l’Approccio B potrebbe essere più indicato.

Ricorda che la scelta dipende dalla natura del tuo problema e dalla rilevanza delle informazioni temporali rispetto a quelle spettrali.
'''

import torch
import torch.nn as nn

#Scelta: In questa implementazione abbiamo deciso di usare il tempo come sequenza.
#In alternativa, potresti scegliere i bin di frequenza come sequenza, ma ciò richiederebbe una diversa riorganizzazione delle dimensioni 
#(ad esempio, un permute diverso).



class ReadMYMind(nn.Module):

    def __init__(self, d_model, num_heads, num_layers, num_classes, channels=64, freqs=81):
        
        super(ReadMYMind, self).__init__()

        # Il layer di embedding mapperà la feature dimension (channels * freqs) a d_model
        self.embedding = nn.Linear(channels * freqs, d_model)
        
        # Transformer per l'attenzione spaziale (qui si applica direttamente alla sequenza temporale)
        self.spatial_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
        # Transformer per l'attenzione temporale (si potrebbe considerare un'iterazione successiva)
        self.temporal_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
        # Cross-attention per combinare le rappresentazioni
        self.cross_attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads)
        
        # Fusione e classificazione finale
        self.fc_fusion = nn.Linear(d_model, d_model)
        self.fc_classify = nn.Linear(d_model, num_classes)
    
    def forward(self, x):
        # x: (batch, canali, frequenze, tempo)
        
        # Utilizziamo il tempo come sequenza
        x = x.permute(0, 3, 1, 2)  # -> (batch, tempo, canali, frequenze)
        
        batch, time, channels, freqs = x.shape
        x = x.reshape(batch, time, channels * freqs)  # -> (batch, tempo, channels*frequencies)
        
        # Embedding: (batch, tempo, d_model)
        x = self.embedding(x)
        
        # Transformer richiede input di forma (seq_length, batch, embedding_dim)
        x = x.permute(1, 0, 2)  # -> (tempo, batch, d_model)
        
        # Applichiamo il Transformer per l'attenzione spaziale
        x_spatial = self.spatial_transformer(x)
        # Applichiamo il Transformer per l'attenzione temporale
        x_temporal = self.temporal_transformer(x_spatial)
        
        # Cross-attention: (tempo, batch, d_model)
        x_cross, _ = self.cross_attention(x_spatial, x_temporal, x_temporal)
        
        # Fusione: per esempio, facciamo una media sul tempo (dimensione 0)
        x_fused = self.fc_fusion((x_spatial + x_temporal).mean(dim=0))  # -> (batch, d_model)
        
        # Classificazione finale
        output = self.fc_classify(x_fused)  # -> (batch, num_classes)
        
        return output
    

In [ ]:
print(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist']['data'].shape)
print()
print(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist']['data'].shape)
print()
print(new_all_th_concat_spectrograms_coupled_exp_task_1['left_fist_vs_right_fist']['data'].shape)
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['left_fist_vs_right_fist']['labels'], return_counts = True))

In [ ]:
'''
Ecco un codice che fornisce dati di input fittizi a ciascuna rete neurale, 
stampa le dimensioni a ogni passaggio e verifica che gli output abbiano le forme attese.

Ho mantenuto le forme coerenti con i tuoi parametri:


Batch size: 8
Numero di canali EEG: 3
Numero di frequenze: 38
Numero di timepoints (campioni temporali): 100
Numero di classi: 2

'''


import torch
import torch.nn as nn
import torch.nn.functional as F

# Parametri
batch_size = 45
input_channels = 64   # Canali EEG
num_freqs = 81       # Numero di frequenze
num_timepoints = 9  # Numero di campioni temporali

num_classes = 2       # Numero di classi

dropout = 0.2

# Creazione di dati fittizi per il test
x = torch.randn(batch_size, input_channels, num_freqs, num_timepoints)  # (batch, canali, frequenze, tempo)
print(f"Input iniziale: {x.shape}\n")

# ---- CNN2D ----
#cnn_model = CNN2D(input_channels=input_channels, num_classes=num_classes)

cnn_model = CNN2D_LSTM_TF(input_channels = input_channels, num_classes =num_classes, dropout = dropout)
cnn_output = cnn_model(x)
print(f"Output CNN2D_LSTM_TF: {cnn_output.shape}\n")  # Atteso: (batch_size, num_classes)


# ---- ReadMEndYou (LSTM) ----
hidden_sizes = [24, 48, 62]
lstm_model = ReadMEndYou(input_size=input_channels * num_freqs, hidden_sizes=hidden_sizes, output_size=num_classes)
lstm_output = lstm_model(x)
print(f"Output ReadMEndYou (LSTM): {lstm_output.shape}\n")  # Atteso: (batch_size, num_classes)

# ---- ReadMYMind (Transformer) ----
d_model = 64   # Dimensione embedding
num_heads = 8   # Numero di teste di attenzione
num_layers = 3  # Numero di strati Transformer

transformer_model = ReadMYMind(d_model=d_model, num_heads=num_heads, num_layers=num_layers, num_classes=num_classes)
transformer_output = transformer_model(x)
print(f"Output ReadMYMind (Transformer): {transformer_output.shape}\n")  # Atteso: (batch_size, num_classes)


In [ ]:
'''
DEFINIZIONE EARLY STOPPING
'''

import io
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import pickle
import numpy as np


class EarlyStopping:
    def __init__(self, patience = 10, min_delta = 0.001, mode = 'max'):
        
        
        """
        :param patience: Numero di epoche da attendere prima di interrompere il training se non c'è miglioramento
        
        Esempio: il training si interromperà se non si osserva un miglioramento per (N = 5) epoche consecutive.
        
        :param min_delta: Variazione minima richiesta per considerare un miglioramento
        
        definisce il miglioramento minimo richiesto per essere considerato significativo. 
        Se il miglioramento è inferiore a min_delta, non viene considerato un vero miglioramento.
        
        Il parametro min_delta in una configurazione di early stopping indica 
        la minima variazione del valore di una metrica 
        (ad esempio, la perdita o l'accuratezza) 
        che deve verificarsi tra un'epoca e la successiva 
        per continuare l'allenamento. 
        
        In genere, il valore di min_delta dipende dal tipo di modello e dai dati specifici, 
        ma di solito si trova in un intervallo tra 0.001 e 0.01.
    
            - Se stai cercando di evitare che l'allenamento si fermi troppo presto,
            puoi impostare un valore più basso per min_delta (come 0.001), 
            - Se vuoi essere più conservativo e permettere fluttuazioni nei valori della metrica,
            un valore più alto (come 0.01) potrebbe essere appropriato.

        Un buon punto di partenza potrebbe essere 0.001, e poi fare dei test per capire quale valore funziona meglio
        nel tuo caso specifico!
        
        :param mode: 'min' per monitorare la loss (minimizzazione), 'max' per l'accuracy (massimizzazione)
        
        'max' → ottimizza metriche da massimizzare (es. accuracy, F1-score, AUC).
        'min' → ottimizza metriche da minimizzare (es. loss).
        
        """
            
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.best_score = None # Tiene traccia del miglior punteggio osservato
        self.counter = 0 # Conta quante epoche consecutive non migliorano
        self.early_stop = False # Flag che indica se attivare l'early stopping
        
        #Ogni volta che si chiama la classe con early_stopping(current_score), controlla se il modello sta migliorando o meno.

    def __call__(self, current_score):
        
        #Caso 1: Prima iterazione (best_score ancora None)
        #→ Se non esiste ancora un miglior punteggio, lo inizializza con il primo valore ricevuto.
        
        if self.best_score is None:
            self.best_score = current_score
            
        #Caso 2: Il modello migliora
        #→ Se il valore migliora di almeno min_delta, aggiorna best_score e resetta il contatore.

        elif (self.mode == 'min' and current_score < self.best_score - self.min_delta) or \
             (self.mode == 'max' and current_score > self.best_score + self.min_delta):
            self.best_score = current_score
            self.counter = 0  # Reset contatore se migliora
            
        #Caso 3: Il modello NON migliora
        
        #→ Se il valore non migliora, incrementa il contatore.
        #→ Se il contatore raggiunge patience, imposta early_stop = True, segnalando che il training deve essere interrotto.
        
        else:
            self.counter += 1  # Incrementa se non migliora
            if self.counter >= self.patience:
                print(f"🛑 Early stopping attivato! Nessun miglioramento per {self.patience} epoche consecutive.")
                self.early_stop = True


#### **Weight & Biases Login & REMOTE CHECKS - EEG Spectrograms - Time x Frequencies**

##### **Weight & Biases Login PRECEDURA CORRETTA ✅ (CANCELLAZIONE RUNS e NON SWEEPS!) - EEG Spectrograms - Time x Frequencies V1**

In [ ]:
'''

Puoi usare l’API Python di W&B per elencare da remoto tutti i progetti di una certa entity (tuo utente o organizzazione).

Ad esempio:

'''

import wandb

# 1) autenticati (se non l’hai già fatto con wandb.login())
# wandb.login()

# 2) crea un’istanza dell’API
api = wandb.Api()

# 3) scegli la tua entity (nome utente o org). 
#    Se omesso, prende la tua entity di default.

entity = 'stefano-bargione-universit-di-roma-tor-vergata'

# 4) prendi la lista dei progetti
projects = api.projects(entity=entity)

# 5) stampa i nomi
for proj in projects:
    print(proj.name)

In [ ]:
###TUTTE ASSIEME


import wandb

# Nomi dei progetti
project_names = [
    #'left_fist_vs_right_fist_spectrograms_channels_freqs_params_hyperparams',
    #'rest_vs_right_fist_spectrograms_channels_freqs_params_hyperparams',
    #'rest_vs_left_fist_spectrograms_channels_freqs_params_hyperparams'
    #'left_fist_vs_right_fist_spectrograms',
    #'rest_vs_right_fist_spectrograms',
    #'rest_vs_left_fist_spectrograms',
    'left_fist_vs_right_fist_spectrograms_new',
    'rest_vs_right_fist_spectrograms_new',
    'rest_vs_left_fist_spectrograms_new'
    
]

user_name = 'stefano-bargione-universit-di-roma-tor-vergata'

# Connessione all'API di W&B
api = wandb.Api()

# Itera su tutti i progetti
for project_name in project_names:
    print(f"\nSelezione delle run per il progetto: \033[1m{project_name}\033[0m")
    
    # Recupera tutte le run del progetto
    runs = api.runs(f"{user_name}/{project_name}")
    
    # Salva gli ID delle run da eliminare
    run_ids_to_delete = []
    
    for run in runs:
        #print(f"Estrazione run ID: {run.id} - Run Name: {run.name}")
        run_ids_to_delete.append(run.id)  # Aggiungi l'ID della run alla lista
    
    print(f"\nTotale runs da eliminare: \033[1m{len(run_ids_to_delete)}\033[0m")
    
    # Elimina le run
    for run_id in run_ids_to_delete:
        run = api.run(f"{user_name}/{project_name}/{run_id}")
        run.delete()  # Elimina la run
        #print(f"Eliminata la run con ID: {run_id}")
    print(f"Eliminazione runs completata")

        

##### **Weight & Biases Login PRECEDURA CORRETTA ✅ (CANCELLAZIONE RUNS e SWEEPS!) - EEG Spectrograms - Time x Frequencies V2** 

In [ ]:
'''UFFICIALE 19 LUGLIO

Spiegazione delle modifiche:
Comando per ottenere la lista degli sweep:


cmd_list = [
    sys.executable, "-m", "wandb", "sweep",
    "--project", path, "--list"
]
Qui usiamo --list per ottenere la lista di tutti gli sweep per il progetto.

Comando per cancellare uno sweep:


cmd_delete = [
    sys.executable, "-m", "wandb", "sweep",
    "--delete", f"{path}/{sweep_id}"
]
Qui usiamo --delete per eliminare ogni sweep individuato.

Verifica degli sweep dopo la cancellazione:

Una volta che abbiamo cancellato gli sweep, facciamo una seconda chiamata proj.sweeps() 
per verificare se ci sono ancora sweep attivi.

Come funziona:
Il codice esegue una lista di tutti i progetti che corrispondono ai pattern.

Per ogni progetto, le run vengono cancellate prima.

Poi, per ogni sweep associato al progetto, il comando wandb sweep --delete viene eseguito per eliminarlo.

Infine, viene effettuata una verifica se gli sweep sono stati effettivamente cancellati.

Con queste modifiche, dovresti essere in grado di eliminare completamente gli sweep oltre a fermarli.


'''

import sys
import subprocess
import wandb

# Configurazione API e entity
entity = "stefano-bargione-universit-di-roma-tor-vergata"
api = wandb.Api()

# 1) Pattern da cercare nel nome dei progetti
patterns = [
    "_spectrograms_time_freqs_new_3d_grid_multiband",
    #"_spectrograms_channels_freqs_new_imagery_3d_grid_multiband",
]

# 2) Prendi tutti i progetti e filtra
print("Recupero tutti i progetti dell'entity…")
all_projects = api.projects(entity=entity)
target_projects = [p for p in all_projects if any(pat in p.name for pat in patterns)]

if not target_projects:
    print("Nessun progetto corrispondente ai pattern trovati.")
    sys.exit(0)

# 3) Cicla sui progetti selezionati
for proj in target_projects:
    name = proj.name
    path = f"{entity}/{name}"
    print(f"\n→ Progetto: {path}")

    # 3.1) Elimina tutte le run (per_page=None fa paging automatico)
    try:
        runs = api.runs(path, per_page=None)
        runs = list(runs)  # forza il fetch di tutte le pagine
        if runs:
            print(f"   • Eliminando {len(runs)} run…")
            for run in runs:
                try:
                    run.delete()
                    print(f"     – Run {run.id} eliminata")
                except Exception as e:
                    print(f"     – Errore eliminando run {run.id}: {e}")
        else:
            print("   (nessuna run trovata)")
    except Exception as e:
        print(f"   ⚠️ Impossibile caricare le run: {e}")

    # 3.2) Cancella tutti gli sweep via CLI Python module
    # Evitiamo di chiamare un eseguibile esterno, usiamo `python -m wandb`
    # Lo stesso interprete che esegue questo script è in sys.executable
    
    cmd_list = [
        sys.executable, "-m", "wandb", "sweep",
        "--project", path, "--list"
    ]
    res = subprocess.run(cmd_list, capture_output=True, text=True)

    if res.returncode != 0 or not res.stdout.strip():
        print("   • Nessuno sweep trovato o progetto inesistente")
        continue

    # Ogni riga di res.stdout ha uno sweep_id come primo token
    for line in res.stdout.splitlines():
        sweep_id = line.split()[0]
        print(f"   • Cancello sweep {sweep_id}")
        cmd_delete = [
            sys.executable, "-m", "wandb", "sweep",
            "--delete", f"{path}/{sweep_id}"
        ]
        subprocess.run(cmd_delete, check=False)

    # Verifica che gli sweep siano stati cancellati
    print("Verifica sweep attivi dopo cancellazione...")
    sweeps = proj.sweeps()  # Riprova a ottenere gli sweep
    if not sweeps:
        print("   ✅ Nessuno sweep attivo trovato.")
    else:
        print(f"   ⚠️ Sweep ancora attivo: {sweeps}")

    print(f"  ✅ Run e sweep eliminati per {path}")


#### **Weight & Biases Login - EEG Spectrograms - Time x Frequencies**

In [ ]:
'''

WEIGHT AND BIASES LOGIN

Il messaggio che stai ricevendo indica 
che sei già connesso al tuo account Weights & Biases (wandb).

Se vuoi forzare il login, puoi usare il comando suggerito:

wandb login --relogin

Questo comando ti permetterà di reinserire le credenziali e riconnetterti al tuo account.
Se non hai bisogno di disconnetterti o di cambiare l'account,
puoi semplicemente continuare a usare wandb senza ulteriori passaggi. 
Hai bisogno di ulteriore assistenza con wandb o con il tuo progetto?
'''


import wandb
wandb.login()
print("✅ Weights & Biases login effettuato con successo!")

In [ ]:
'''
Per modificare il percorso in cui W&B salva i dati localmente,
puoi configurare la variabile di ambiente WANDB_DIR.

Questo ti permette di specificare una directory personalizzata in cui W&B salva tutti i file associati al tuo run, inclusi i dati e i log.
'''

import os

# Imposta la directory per i dati W&B:
# Questo cambierà la cartella in cui W&B salva i dati per quella sessione di esecuzione

# Definisci la cartella principale
WB_dir = "/home/stefano/Interrogait/WB_spectrograms_analyses_motor_task_1_new"
os.makedirs(WB_dir, exist_ok=True)


os.environ["WANDB_DIR"] = WB_dir


In [ ]:
import pickle

# Apri il file in modalità lettura binaria ('rb')

#path = '/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms/'
path = '/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/'

with open(f"{path}new_all_th_concat_spectrograms_coupled_exp_task_1.pkl", "rb") as f:
    data = pickle.load(f)

    

In [ ]:
# Itera sulle chiavi del dizionario principale
for condition, values in data.items():
    if isinstance(values, dict) and "data" in values and "labels" in values:
        X_shape = values["data"].shape
        y_length = len(values["labels"])
        print(f"🔹 Condizione: {condition}")
        print(f"   ➡ Shape dati: {X_shape}")
        print(f"   ➡ Lunghezza labels: {y_length}\n")


In [ ]:
new_all_th_concat_spectrograms_coupled_exp_task_1.keys()

In [ ]:
##################### CODICE UFFICIALE DEL 04/03/2025 ORE 9:30 #####################
                                 ##################### SENZA DETTAGLI SCRITTI V°3 #####################
        
'''ATTENZIONE: 

HO SOSTITUITO LE VARIABILI DI 

    1) DATASET_TRAIN_LOADER -->  TRAIN_LOADER
    2) DATASET_VAL_LOADER -->  VAL_LOADER

    VEDI FUNZIONE 'PREPARE_DATA_FOR_MODEL --> NOMI DELLE VARIABILI DEI TORCH TENSOR DATASET LOADER SON  'TRAIN_LOADER' E VAL_LOADER!!!  

'''
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import copy as cp
import numpy as np

import wandb
import random
import copy as cp


# Definisci le lista delle coppie di condizioni sperimentali
#experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

experimental_conditions = ["rest_vs_left_fist", "rest_vs_right_fist", "left_fist_vs_right_fist"]

# Inizializza il dizionario per caricare i dati
data_dict = {}

# Definisci la cartella principale
#base_dir = "/home/stefano/Interrogait/WB_spectrograms_best_results"

'''HO AGGIUNTO NEW QUI PER CREARE CARTELLA CON SWEEP NUOVI, CONFRONTANDO BILSTM & TRANSFORMER CON CNN2D_LSTM!'''

base_dir = "/home/stefano/Interrogait/WB_spectrograms_best_results_motor_task_1_new"
os.makedirs(base_dir, exist_ok=True)

'''LOOP DI CARICAMENTO DATI'''

for condition in experimental_conditions:
    # Crea la cartella per la condizione sperimentale
    condition_dir = os.path.join(base_dir, condition)
    os.makedirs(condition_dir, exist_ok=True)
    
    # Aggiungi un livello di annidamento per ogni condizione
    data_dict[condition] = {}
    
    for data_type in ["spectrograms"]:
        
        # Crea la cartella per il tipo di dato
        data_dir = os.path.join(condition_dir, data_type)
        os.makedirs(data_dir, exist_ok=True)
        
        #for category in ["familiar", "unfamiliar"]:
        
        for category in ["familiar"]:
            
            # Crea la cartella per la categoria
            #category_dir = os.path.join(data_dir, category)
            #os.makedirs(category_dir, exist_ok=True)
            
            #for subject_type in ["th", "pt"]:
            
            for subject_type in ["th"]:
                # Caricamento e suddivisione dei dati
                
                #if data_type == "spectrograms":
                    
                print(f"Caricamento dati per: {condition} - {data_type} - {category}_{subject_type}")
                X, y = load_data(data_type, category, subject_type, condition=condition)
                
                
                # Creazione della chiave per il dizionario annidato
                data_dict[condition][data_type] = data_dict[condition].get(data_type, {})
                data_dict[condition][data_type][f"{category}_{subject_type}"] = (X, y)
                
                # Stampa di conferma
                print(f"Dataset caricato: \033[1m{condition}\033[0m_\033[1m{data_type}\033[0m_\033[1m{category}_{subject_type}\033[0m - Shape X: \033[1m{X.shape}\033[0m, Shape y: \033[1m{len(y)}\033[0m\n")

In [ ]:
data_dict['rest_vs_left_fist'].keys()

#### **Sweep Configuration - EEG Spectrograms - Time x Frequencies ONLY HYPER-PARAMS**

#### **Sweep separati per ciascuno dei modelli CNN2D_LSTM_TF, BiLSTM e Transformer**

In [ ]:
'''
N.B. 

PER SAPERE A QUALE COMBINAZIONE DI FATTORI CORRISPONDONO I DATI (i.e, X_train, X_val, X_test, y_train, y_val, y_test)

MI CREO UN DIZIONARIO ULTERIORE, 'DATA_DICT_PREPROCESSED' CHE CONTIENE PER OGNI COMBINAZIONE DI FATTORI I DATI SPLITTATI

IN QUESTO MODO, QUANDO FORNISCO ALLA FUNZIONE 'TRAINING_SWEEP' LA TUPLA CON I VARI DATI ((TRAIN, VAL E TEST))
IO POSSO CAPIRE A QUALE COMBINAZIONI DI FATTORI CORRISPONDE QUELLA TUPLA DI DATI (TRAIN, VAL E TEST)


INOLTRE,
MI CREO ANCHE UNA LISTA DI TUPLE DI STRINGHE, DOVE OGNI TUPLA CONTIENE LE STRINGHE DELLE CHIAVI USATE 
PER LA GENERAZIONE DI DATA_DICT_PREPROCESSED.

IN QUESTO MODO, MI ASSICURO CHE SIA UNA COERENZA TRA LA CREAZIONE DEI 'NAME' E 'TAG' DELLA RUN
E
LA CORRETTA ESTRAZIONE DEI DATI (OSSIA I DATI DI QUALE CONDIZIONE SPERIMENTALE, QUALI EEG INPUT, E DA CHI PROVENGONO!)  


Questo approccio permette di garantire la corrispondenza tra 

1) le chiavi dei dati pre‐processati e 
2) la configurazione delle runs su W&B

andando a creare due strutture in parallelo:

- data_dict_preprocessed – che contiene, per ogni combinazione (condition, data_type, category_subject), 
                            la tupla dei dati già suddivisi (X_train, X_val, X_test, y_train, y_val, y_test);
                            
- sweeps_id – che contiene, per ogni combinazione (condition, data_type, category_subject), 
              sia la stringa univoca dello sweep ID, che l'insieme delle stringhe che formano la combinazione (condition, data_type, category_subject)



LOOP DI PREPARAZIONE DATI (FINO A DATASET SPLITTING)
'''



from sklearn.model_selection import train_test_split

#A QUESTO PUNTO PER OGNI DATASET, FACCIO STEP PRIMA DELLO SWEEP

# Set per tenere traccia dei dataset già elaborati
processed_datasets = set()

# Seleziona il dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Modelli che useremo nei sweep
MODEL_LIST = ["CNN2D_LSTM_TF", "BiLSTM", "Transformer"] #ReadMEndYou and #ReadMYMind



# Dizionario per salvare gli sweep ID associati a ogni condizione sperimentale

'''sweep_ids_for_models contiene la struttura che mi serve da copiare per best_models''' 
sweep_ids_for_models = {}

'''sweep_ids contiene la struttura che mi serve da copiare per iterare sui singoli swweps di ogni combinazione di fattori'''
sweep_ids = {}  

'''DIZIONARIO CHE VIENE FORNITO IN INGRESSO A TRAINING_SWEEP'''
# Dizionario per salvare la tupla di dati già preprocessati
data_dict_preprocessed = {}


# Loop di addestramento e test per ogni condizione sperimentale
for condition, data_types in data_dict.items():  # Itera sulle condizioni sperimentali
    
    data_dict_preprocessed[condition] = {}
    
    # Aggiungi al dizionario sweep_ids
    if condition not in sweep_ids:
        sweep_ids[condition] = {}
        
        '''sweep_ids_for_models'''
        sweep_ids_for_models[condition] = {}
        
    for data_type, categories in data_types.items():  # Itera sui tipi di dati (1_20, 1_45, wavelet)
        
        data_dict_preprocessed[condition][data_type] = {}
        
        if data_type not in sweep_ids[condition]:
            sweep_ids[condition][data_type] = {}
            
            '''sweep_ids_for_models'''
            sweep_ids_for_models[condition][data_type] = {}
            
        for category_subject, (X_data, y_data) in categories.items():  # Itera sulle coppie category_subject
            
            # 1. Prepara spazio nei dizionari: sotto category_subject, un dict per ogni modello
            
            data_dict_preprocessed[condition][data_type][category_subject] = None
            
            if category_subject not in sweep_ids[condition][data_type]:
                
                sweep_ids[condition][data_type][category_subject] = {}
                
                '''NUOVA MODIFICA'''
                sweep_ids[condition][data_type][category_subject] = {
                model: [] for model in MODEL_LIST
                }

                '''sweep_ids_for_models'''
                sweep_ids_for_models[condition][data_type][category_subject] = {}
                
                '''NUOVA MODIFICA'''
                sweep_ids_for_models[condition][data_type][category_subject] = {
                model: [] for model in MODEL_LIST
                }
                
            print(f"\n\n\033[1mEstrazione Dati\033[0m della Chiave \033[1m{condition}_{data_type}_{category_subject}\033[0m")
            
            # Controlla se il dataset è già stato elaborato (se la chiave è già nel set)
            if (condition, data_type, category_subject) in processed_datasets:
                print(f"⚠️ ATTENZIONE: Il dataset {condition} - {data_type} - {category_subject} è già stato elaborato! Salto iterazione...")
                continue  # Salta se il dataset è già stato processato

            # Aggiungi il dataset al set
            processed_datasets.add((condition, data_type, category_subject))

            X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
            
            data_dict_preprocessed[condition][data_type][category_subject] = (X_train, X_val, X_test, y_train, y_val, y_test)
            
            # Puoi anche aggiungere altri print per verificare la dimensione dei set
            print(f"\033[1mDataset Splitting\033[0m: Train Set Shape: {X_train.shape}, Validation Set Shape: {X_val.shape}, Test Set Shape: {X_test.shape}")

            
print(f"\nCreato \033[1mdata_dict_preprocessed\033[0m")


In [ ]:
data_dict['rest_vs_left_fist'].keys()

In [ ]:
print(data_dict_preprocessed.keys())
print(data_dict_preprocessed['rest_vs_left_fist'].keys())
print(data_dict_preprocessed['rest_vs_left_fist']['spectrograms'].keys())
print(type(data_dict_preprocessed['rest_vs_left_fist']['spectrograms'].keys()))

#All'interno, c'è una tupla, di 6 elementi!
print(type(data_dict_preprocessed['rest_vs_left_fist']['spectrograms']['familiar_th']))

#I 6 elementi della tupla sono X_train, X_val, X_test, y_train, y_val, y_test !
print(len(data_dict_preprocessed['rest_vs_left_fist']['spectrograms']['familiar_th']))

In [ ]:
print(sweep_ids_for_models)

In [ ]:
print(sweep_ids)

In [ ]:
print(sweep_ids_for_models)

In [ ]:
''' 

                                                                    AGGIORNATA AL 19 LUGLIO
                                                                    
                                                                    
#"learning rate : {"value"[1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]"}
#"n_epochs": {"value": 100},
# "patience": {"value": 12},
#"batch_size": {"values": [16, 24, 32, 48, 64, 72, 84, 96]}
#"standardization": {"values": [True, False]}, 
# "beta1": {"values": [0.8, 0.85, 0.9, 0.95]},
#  "beta2": {"values": [0.98, 0.99, 0.995, 0.999]},
#  "eps": {"value": [1e-8, 1e-7, 1e-6, 1e-5]}                                                                                                                            



sweep_config = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]}, # fissato al valore di default del paper

        "weight_decay":  {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        
        
        "model_name":{"values": ['CNN3D_LSTM_FC']},

        "batch_size": {"values": [32, 48, 64, 96]},

        "standardization":{"values": [True, False]},

        "beta1": {"values": [0.9, 0.95]},

        "beta2": {"values": [0.99, 0.995]},
        
        "eps": {"values": [1e-8, 1e-7]},
        
        #In questo modo:
        
        "use_lstm":      {"values":[True, False]},
        "lstm_hidden":   {"values":[32]},
        "dropout":       {"values":[0.5]},
        
    }
}


'''


#Tutti gli sweep saranno organizzati sotto lo stesso progetto,
#che corrisponde alla coppia di condizioni sperimentali corrente (i.e., exp_cond).

#Questo significa che tutte le runs che verranno lanciate con quello sweep, 
#saranno associate a quella specifica coppia di condizioni sperimentali corrente.

#Dato che sto iterando su ogni coppia di condizioni sperimentali, 
#ogni sweep verrà automaticamente salvato all'interno del progetto corrispondente 
#della specifica condizione sperimentale (exp_cond).

#In pratica, se hai più condizioni sperimentali 
#(ad esempio, "Condizione_A", "Condizione_B", ecc.),
#WandB creerà automaticamente sweep separati all'interno dei rispettivi progetti


#Creo la configurazione dello sweep e la eseguo:

#uno per il modello CNN3D_LSTM_FC, originariamente era così


#sweep_config_cnn3d= {
#    "method": "random",
#    "metric": {"name": "val_accuracy", "goal": "maximize"},
#    "parameters": {
#        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
#        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
#        "n_epochs": {"value": 100},
#       "patience": {"value": 12},
#        "model_name": {"values": ["CNN2D_LSTM_TF"]},
#        "batch_size": {"values": [32, 48, 64, 96]},
        
#        "standardization": {"values": [True,False]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
#        "beta1": {"values": [0.9, 0.95]},
#        "beta2": {"values": [0.99, 0.995]},
#        "eps": {"values": [1e-8, 1e-7]},
#        "use_lstm": {"values": [True, False]},
#        "lstm_hidden": {"values": [32]},
#        "dropout": {"values": [0.5]},
#    }
#}


#sweep_config_cnn_sep = {
#    "method": "random",
#    "metric": {"name": "val_accuracy", "goal": "maximize"},
#    "parameters": {
#        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
#        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
#        "n_epochs": {"value": 100},
#        "patience": {"value": 12},
#        "model_name": {"values": ["SeparableCNN2D_LSTM_FC"]},
#        "batch_size": {"values": [32, 48, 64, 96]},
#        "standardization": {"values": [True,False]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
#        "beta1": {"values": [0.9, 0.95]},
#        "beta2": {"values": [0.99, 0.995]},
#        "eps": {"values": [1e-8, 1e-7]},
#        "use_lstm": {"values": [True, False]},
#        "lstm_hidden": {"values": [32]},
#        "dropout": {"values": [0.5]},
#    }
#}


'''

# Comodo mapper per il tuo loop
sweep_config_dict_stft = {
    "CNN2D_LSTM_TF": sweep_config_cnn2d_lstm_tf,
    "ReadMEndYou": sweep_config_bilstm,
    "ReadMYMind": sweep_config_transformer,
}
Nota pratica (per l’integrazione nel training)
CNN2D_LSTM_TF: passa dropout=config.dropout.

ReadMEndYou: costruisci con
ReadMEndYou(input_size=channels*freqs, hidden_sizes=[config.hidden1, config.hidden2, config.hidden3],
output_size=2, dropout=config.dropout, bidirectional=config.bidirectional).

ReadMYMind: costruisci con
ReadMYMind(d_model=config.d_model, num_heads=config.num_heads, num_layers=config.num_layers, num_classes=2, channels=config.channels, freqs=config.freqs).

'''
# 2.1 – Sweep config per ciascun modello

#CNN2D_LSTM_TF
sweep_config_cnn2d_lstm_tf = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN2D_LSTM_TF"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
    
        "dropout": {"values": [0.5]},
    }
}


sweep_config_bilstm = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["BiLSTM"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
        "dropout": {"values": [0.5]},
        "bidirectional": {"values": [False, True]},
        
        #Soluzione 1 per mettere valori agli hidden sizes
        #"hidden1": {"values": [24, 32, 48, 64]},
        #"hidden2": {"values": [48, 64, 96, 128]},
        #"hidden3": {"values": [62, 96, 128, 160]}
        # in build del modello: hidden_sizes=[hidden1, hidden2, hidden3]
        
        #Soluzione 2 per mettere valori agli hidden sizes
        
        #hidden_sizes = [24, 48, 62]
        #lstm_model = ReadMEndYou(input_size=input_channels * num_freqs, hidden_sizes=hidden_sizes, output_size=num_classes)
    }
}


sweep_config_transformer = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["Transformer"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
        "d_model": {"values": [32]},
        "num_heads": {"values": [2]},
        "num_layers": {"values": [2]},
    }
}





    
'''SWEEP_IDS_FOR_MODELS

# 2) Popolo sweep_ids_for_models in base a MODEL_LIST (già inizializzato nella prima cella)
'''

#Preparazione del dizionario sweep_ids_for_models (lo aggiorno inserendo il livello delle chiavi dei modelli, per copiare poi la struttura per creare best_models)

#for condition in sweep_ids_for_models:
    #for data_type in sweep_ids_for_models[condition]:
        #for category_subject in sweep_ids_for_models[condition][data_type]:
            #for model_name in sweep_config["parameters"]["model_name"]["values"]:
                
                # Aggiungi il modello al dizionario, se non esiste già
                #if model_name not in sweep_ids_for_models[condition][data_type][category_subject]:
                    #sweep_ids_for_models[condition][data_type][category_subject][model_name] = []

                    
print(f"\nAggiornato \033[1msweep_ids_for_models\033[0m")


'''BEST_MODELS

# 3) Creo best_models da sweep_ids_for_models
'''

#Preparazione del dizionario best_models (facendo una copia della struttura di 'sweep_ids_for_models')

#In questo modo potrò, per ogni condizione sperimentale, tipo di dato EEG e combinazione di ruolo/gruppo,
#accedere facilmente al miglior modello (cioè ai suoi pesi e bias) e gestirlo in maniera separata!

import copy
best_models = copy.deepcopy(sweep_ids_for_models)

# Inizializzo il dizionario che contiene il migliori modello tra quelli degli sweep testati, 
# relativi ad una certa combinazione di fattori,
#per ogni condizione sperimentale
#tipo di dato EEG 
#combinazione di ruolo/gruppo

for condition in best_models:
    for data_type in best_models[condition]:
        for category_subject in best_models[condition][data_type]:
            for model_name in best_models[condition][data_type][category_subject]:
                best_models[condition][data_type][category_subject][model_name] = {
                    "model": None,
                    "max_val_acc": -float('inf'),
                    "best_epoch": None,
                    
                    #ATTENZIONE! CREATA ALTRA CHIAVE PER SALVARE 
                    #LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI OGNI MODELLO!
                    "config": None}
                
print(f"\nCreato \033[1mbest_models\033[0m")


#'''SWEEP_IDS'''

#Preparazione del dizionario sweep_ids (lo aggiorno inserendo solo una lista all'ultimo livello)

# Itera su sweep_ids e crea le chiavi per category_subject con liste vuote
#for condition in sweep_ids:
    #for data_type in sweep_ids[condition]:
        #for category_subject in sweep_ids[condition][data_type]:
            # Inizializza una lista vuota se non esiste già
            #if not isinstance(sweep_ids[condition][data_type][category_subject], list):
                #sweep_ids[condition][data_type][category_subject] = []
                    
#print(f"\nAggiornato \033[1msweep_ids\033[0m")




In [ ]:
#print(best_models)
#print(sweep_ids_for_models)
#print(sweep_ids)
#print(data_dict_preprocessed['th_resp_vs_pt_resp']['1_20']['familiar_th'][0].shape)

In [ ]:
print(best_models)

In [ ]:
print(sweep_ids_for_models)

In [ ]:
print(sweep_ids)

In [ ]:
#data_dict_preprocessed['th_resp_vs_pt_resp']['1_20']['familiar_th'][5].shape

**NOTA BENE**

Come output, io otterrò **quando crei gli sweeps** una cosa come questa, ad esempio:

        Create sweep with ID: y73iajvw
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/y73iajvw
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - CNN1D: n° sweep y73iajvw
        Create sweep with ID: 3b6o28jt
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/3b6o28jt
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - BiLSTM: n° sweep 3b6o28jt
        Create sweep with ID: q6yp4fas

        .....

Vedendole bene, per **ogni condizione sperimentale (3)**, **per ogni dato EEG (3)** e **per ogni provenienza del dato EEG (4)**, 
Io **DOVREI OTTENERE** in totale = **3x3x4 = 36 sweeps** per **OGNI CONDIZIONE SPERIMENTALE**


Per **ognuna di queste sweeps**, io se ho capito bene creerò **15 esperimenti** (le mie runs), che corrispondo alle **diverse configurazioni di iper-parametri testati per lo stesso specifico sweep**!

(ad esempio, solo questo 

<br> 

        Create sweep with ID: y73iajvw
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/y73iajvw
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - CNN1D: n° sweep y73iajvw)

Dove, le diverse configurazioni, son determinate randomicamente a partire dai valori dentro la variabile "**sweep_config**"  che è questa 


    #Creo la configurazione dello sweep e la eseguo
    sweep_config = {
        "method": "random",
        "metric": {"name": "val_accuracy", "goal": "maximize"},
        "parameters": {
            "lr": {"values": [0.01, 0.001, 0.0005, 0.0001]},
            "weight_decay": {"values": [0, 0.01, 0.001, 0.0001]},
            "n_epochs": {"value": 100},
            "patience": {"value": 10},
            "model_name":{"values": ['CNN1D', 'BiLSTM', 'Transformer']},
            "batch_size": {"values": [32, 48, 64, 96]},
            "standardization":{"values": [True, False]},
        }
    }
    
    



In [ ]:
'''
ATTENZIONE CHE A QUESTO PUNTO


1) sweep_ids[cond][dtype][cat][model_name] contiene le tuple (sweep_id, combo_key) per ciascun modello, che ancora non esistono perché devo esser create durante la creazione degli sweeps, ma ho solo una lista

{'rest_vs_left_fist': {'spectrograms': {'familiar_th': []}}, 
'rest_vs_right_fist': {'spectrograms': {'familiar_th': []}}, 
'left_fist_vs_right_fist': {'spectrograms': {'familiar_th': []}}}


2) sweep_ids_for_models e best_models sono paralleli a sweep_ids con lo stesso livello model_name

ossia 

sweep_ids_for_models come

{'rest_vs_left_fist': {'spectrograms': {'familiar_th': {'CNN2D_LSTM_FC': [], 'ReadMEndYou': [], 'ReadMYMind': []}}},
'rest_vs_right_fist': {'spectrograms': {'familiar_th': {'CNN2D_LSTM_FC': [], 'ReadMEndYou': [], 'ReadMYMind': []}}},
'left_fist_vs_right_fist': {'spectrograms': {'familiar_th': {'CNN2D_LSTM_FC': [], 'ReadMEndYou': [], 'ReadMYMind': []}}}}

best_models come

{'rest_vs_left_fist': {'spectrograms': {'familiar_th': 
{'CNN2D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}, 
'ReadMEndYou': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None},
'ReadMYMind': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}}}},

'rest_vs_right_fist': {'spectrograms': {'familiar_th': 
{'CNN2D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}, 
'ReadMEndYou': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None},
'ReadMYMind': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}}}},

'left_fist_vs_right_fist': {'spectrograms': {'familiar_th': 
{'CNN2D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}, 
'ReadMEndYou': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None},
'ReadMYMind': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}}}},


'''


In [ ]:
'''
Popolamento di sweep_ids e lancio degli agenti:

Obiettivo: 

Per ogni combinazione (condition, data_type, category_subject, model_name), 
Se la lista è vuota, crei uno sweep usando wandb.sweep(sweep_config, project=condition) e lo inserisci nella lista. 
In seguito, iteri su quella lista (che ora contiene IL TUO SPECIFICO sweep_id) e lanci wandb.agent() per eseguire il training.



Nota importante:
L'ID restituito da wandb.sweep() è una STRINGA UNIVOCA generata automaticamente da WandB.
Non puoi assegnargli direttamente una stringa personalizzata, ma puoi comunque usarlo per mappare nel tuo dizionario la combinazione di fattori! 

In questo ciclo, il fatto che la lista parta vuota è normale: il codice la popola se necessario e poi lancia l'agente per ogni sweep_id presente.


****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******
INOLTRE, BISOGNA CONTROLLARE CHE SI STIA ITERANDO CORRETTAMENTE SOLO SULLA COMBINAZIONE CORRENTE DI 

                CONDITION, DATA_TYPE, CATEGORY_SUBJECT E MODEL_NAME
                
QUESTO PERCHÉ SE UN CICLO SI RIPETE PER UNA CONDIZIONE IN PIÙ UNA COMBINAZIONE, POTREBBE GENERARE PIÙ  SWEEP IDS DI QUELLI CHE TI ASPETTI!
****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******



SOLUZIONE:

Un buon approccio per evitare la creazione ripetuta di Sweep ID 
per la stessa combinazione di condition, data_type, category_subject e model_name 
è quello di utilizzare un SET per tenere traccia delle combinazioni già processate.
Se una combinazione è già presente nel set, non dovresti creare un nuovo Sweep ID, ma semplicemente saltare quella parte del codice


Inoltre, ho avuto una idea ad un certo punto! 


****************************** ******************************
ILLUMINAZIONE DEL POMERIGGIO DEL 04/03/2025: 
****************************** ******************************


Quando creo ogni sweep singolarmente, si genera una stringa univoca di quello sweep, che si riferisce ad un dataset che è il prodotto di diversi fattori:

- una certa condizione sperimentale,  
- una certo preprocessing sui dati EEG (1_20, 1_45, wavelet)
- una certa provenienza del dato proprio (in termini di ruolo e gruppo --> th o pt, familiar o unfamiliar)


Di conseguenza, iterando su ogni sweep_ids (che ho fatto in modo avesse la STESSA struttura dei miei dati già splittati i.e, data_dict_preprocessed
io posso, 

1) da un lato eseguire la creazione della stringa univoca associata a quello sweep,
2) crearmi una 'combination_key', che sarebbe l'insieme delle stringhe che descrivono quel dataset specifico di data_dict_preprocessed

che sarà costituito da

- una certa condizione sperimentale,  
- una certo preprocessing sui dati EEG (1_20, 1_45, wavelet)
- una certa provenienza del dato proprio (in termini di ruolo e gruppo --> th o pt, familiar o unfamiliar)


Poiché quindi so già la corrispondenza tra ogni Sweep ID e la sua combinazione di fattori (condition, data_type, category_subject), 
posso creare un MAPPING, che associ, ad certo Sweep ID e la stringa che descrive i suoi fattori associati!


In questo modo, forse, si riesce a risolvere il PROBLEMA 2 NELLA CELLA DI CREAZIONE DELLA FUNZIONE DI TRAINING (VEDI SOTTO!)



                                                        ******IMPORTANTE MODIFICA*****
                                                        
Ora lo sweep_ids non si deve sdoppiare ora, perché sostanzialmente, 
per ogni modello si creano gli sweeps ids corrispondenti e salvati come valore
dentro la chiave del modello corrispondente, sotto forma di tupla...

cioè non più così

"sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))"

ma una cosa del genere

"sweep_ids[condition][data_type][category_subject][model_name].append((new_sweep_id, combination_key))




COME ERA PRIMA

#Inizializza un set per tenere traccia delle combinazioni già elaborate

created_combinations = set()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"

            # Controlla se la combinazione è già stata elaborata
            if combination_key not in created_combinations:

                if not sweep_ids[condition][data_type][category_subject]:
                    #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms")
                    new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband")

                    #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                    #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA 
                
                    sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))
                    
                    print(f"Sweep ID creato per \033[1m{combination_key}\033[0m: n° sweep \033[1m{new_sweep_id}\033[0m")

                # Aggiungi la combinazione al set per evitare duplicazioni
                created_combinations.add(combination_key)
            else:
                # Se la combinazione è già stata creata, salta
                print(f"Sweep ID per {combination_key} già esistente.")
                
'''


'''
ADESSO


Cosa fa questo snippet

Cicla su ogni (condition, data_type, category_subject) una volta sola grazie a created_combinations.

All’interno, fa un sottoloop su MODEL_LIST (i tuoi due modelli).

In base a model_name, sceglie sweep_config_cnn3d o sweep_config_cnn_sep.

Chiama wandb.sweep(...) con il config giusto e salva il risultato in


sweep_ids[condition][data_type][category_subject][model_name]
anziché nella lista “piatta” che avevi prima.


In questo modo:

sweep_ids[cond][dtype][cat] resta un dict con due chiavi ("CNN3D_LSTM_FC" e "SeparableCNN2D_LSTM_FC")

Ognuna di quelle chiavi punta a una propria lista di tuple (sweep_id, combo_key)

Non serve sdoppiare l’intero sweep_ids, perché tiene già separati gli sweep di ciascun modello

Più tardi, quando lancerai gli agent, ti basterà:


for model_name, sweeps in sweep_ids[cond][dtype][cat].items():
    for sweep_id, combo_key in sweeps:
        # qui scegli il train_fn in base a model_name
        wandb.agent(sweep_id, function=train_fn_map[model_name], count=200)
e ogni modello girerà solo i suoi sweep.



Alla fine, sweep_ids avrà la forma:

{
  'rest_vs_left_fist': {
    'spectrograms': {
      'familiar_th': {
         'CNN3D_LSTM_FC':       [(sweep_id_1, 'rest_vs_left_fist_spectrograms_familiar_th')],
         'SeparableCNN2D_LSTM_FC': [(sweep_id_2, 'rest_vs_left_fist_spectrograms_familiar_th')]
      }
    }
  },
  …
}
'''


#Ecco come puoi riscrivere solo la TERZA CELLA (quella in cui crei effettivamente gli sweep) 
#mantenendo la tua struttura “a celle” e usando per ognuno il sweep_config giusto in base al model_name.

#Creazione degli sweep (Terza cella)
#Ecco il solo snippet che devi usare per creare gli sweep ripartiti per modello, usando i due sweep_config_*:


'''
Per mantenere la stessa logica di prima ma tenendo conto che ora stai lavorando con modelli separati, 
dovresti modificare il controllo in modo che verifichi se una combinazione di condition, data_type, category_subject
è già stata processata per ciascun modello.

Quindi, il controllo dovrebbe essere fatto separatamente per ogni modello dentro il loop che itera sui modelli (MODEL_LIST).
Di seguito ti mostro la versione modificata che tiene conto di questo:



#Inizializza un set per tenere traccia delle combinazioni già elaborate

created_combinations = set()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"

            # Controlla se la combinazione è già stata elaborata
            if combination_key not in created_combinations:

                if not sweep_ids[condition][data_type][category_subject]:
                    #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms")
                    new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband")

                    #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                    #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA 
                
                    sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))
                    
                    print(f"Sweep ID creato per \033[1m{combination_key}\033[0m: n° sweep \033[1m{new_sweep_id}\033[0m")

                # Aggiungi la combinazione al set per evitare duplicazioni
                created_combinations.add(combination_key)
            else:
                # Se la combinazione è già stata creata, salta
                print(f"Sweep ID per {combination_key} già esistente.")
                
                
                
'''

                    
'''

Cosa è stato cambiato rispetto alla versione precedente?
Controllo della combinazione di modello:
La logica del controllo della combinazione (combination_key, model_name) nel set created_combinations è corretta, 
perché vogliamo evitare di creare più volte lo stesso sweep per una combinazione di condition, data_type, category_subject, e model_name.

Controllo e creazione dello sweep:
Il codice controlla prima se la combinazione con il modello non è stata già processata 
con il controllo if (combination_key, model_name) not in created_combinations. 

Se non è stata processata, procede a creare lo sweep corrispondente. 
Se la combinazione esiste già, salta la creazione dello sweep per quel modello.

Aggiunta del nuovo sweep ID:
Una volta creato il nuovo sweep per il modello, viene aggiunto correttamente 
alla lista del modello specifico sotto sweep_ids[condition][data_type][category_subject][model_name].

Aggiunta al set delle combinazioni:
Dopo aver creato lo sweep, aggiungiamo (combination_key, model_name) al set created_combinations
per tenere traccia delle combinazioni già elaborate.

Verifica della logica:
La combinazione (combination_key, model_name) deve essere unica per ciascun modello, 
e quindi il controllo che evita duplicazioni nel set è corretto.

La creazione dello sweep per ciascun modello separato è mantenuta, 
e viene applicata solo quando la combinazione specifica non è già stata elaborata per quel modello.

In questo modo, la logica funziona come nel codice precedente, ma ora si tiene conto anche dei modelli separati, 
creando un sweep per ciascuno di essi e mantenendo la traccia delle combinazioni in modo appropriato.

'''
created_combinations = set()



#CNN2D_LSTM_TF
#sweep_config_cnn2d_lstm_tf

#sweep_config_bilstm
#sweep_config_transformer
# Per semplicità, tieni MODEL_LIST a portata di mano
MODEL_LIST = ["CNN2D_LSTM_TF", "BiLSTM", "Transformer"]

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"
            
            # per ciascun modello, creo uno sweep separato
            for model_name in MODEL_LIST:

                # Controlla se la combinazione di condition, data_type, category_subject + modello è già stata elaborata
                if (combination_key, model_name) not in created_combinations:

                    # Scegli il config in base al model_name
                    if model_name == "CNN2D_LSTM_TF":
                        sweep_conf = sweep_config_cnn2d_lstm_tf
                        
                    elif model_name == "BiLSTM":  # ReadMEndYou
                        sweep_conf = sweep_config_bilstm
                        
                    elif model_name == "Transformer":  # ReadMYMind
                        sweep_conf = sweep_config_transformer
                    else:
                        raise ValueError(f"Modello non riconosciuto: {model_name}")
                    
                    # Controllo se la lista per il modello specifico è vuota
                    if not sweep_ids[condition][data_type][category_subject][model_name]:

                        # Crea lo sweep e lo appendo nella lista dedicata a quel modello
                        
                        '''
                        QUESTO COME ERA IMPOSTATO PRIMA NELLA VERSIONE SENZA SWEEP CONFIG MODEL-SPECIFIC
                        
                        #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms_new")
                        '''
                        
                        #new_sweep_id = wandb.sweep(sweep_conf, project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband")
                        
                        '''
                        QUESTO SAREBBE COME ORA LO IMPOSTO PER LA VERSIONE CON SWEEP CONFIG MODEL-SPECIFIC
                        
                        '''
                        
                        new_sweep_id = wandb.sweep(sweep_conf, project=f"{condition}_spectrograms_time_freqs_new_3d_grid_multiband")
                        
                        #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                        #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA 
                        
                        sweep_ids[condition][data_type][category_subject][model_name].append((new_sweep_id, combination_key))

                    print(f"▶ Sweep \033[1m{new_sweep_id}\033[0m creato per \033[1m{combination_key}\033[0m, modello \033[1m{model_name}\033[0m")
                    
                    # Aggiungi la combinazione al set per evitare duplicazioni
                    created_combinations.add((combination_key, model_name))  # Aggiungi la combinazione con il modello
                else:
                    # Se la combinazione è già stata creata, salta
                    print(f"⚠️ {combination_key} già processato per il modello {model_name}, skip.")
                    continue


In [ ]:
# Calcola e stampa il numero totale di combinazioni uniche (e quindi di sweep creati)

total_sweeps = len(created_combinations)
total_runs = total_sweeps * 200

print(f"Numero totale di sweep creati: {total_sweeps}")
print(f"Numero totale di runs da eseguire: {total_runs}")

In [ ]:
'''ESEGUI QUI QUESTA CELLA PER VEDERE COME SI STRUTTURA SWEEP_IDS'''

#sweep_ids

In [ ]:
#sweep_ids.keys()
#sweep_ids['th_resp_vs_pt_resp'].keys()
#sweep_ids['th_resp_vs_pt_resp'].keys()

**NOTA BENE**


I **numeri degli sweeps** tornano e son corretti! 
Tuttavia, avendo solo preparato l'inizializzazione degli sweeps dentro 'sweep_ids', 
Sul sito di weight and biases, io vedo le tre condizioni sperimentali, create ciascuna come un progetto separato, che è corretto, ma ancora le runs di ciascuna le vedo a 0

Deduco che questo comportamento, dovrebbe esser normale, dato che ancora non ho avviato l'agente appunto wandb.agent(), con cui gli fornisco lo sweep_id generato adesso in questo loop precedente.

In [ ]:
print(data_dict_preprocessed.keys())
print(sweep_ids.keys())

In [ ]:
data_dict_preprocessed.keys()

In [ ]:
data_dict_preprocessed['rest_vs_left_fist'].keys()

#### **VERSIONE DEL 6 MARZO (RISOLUZIONE DEFINITIVA)**

##### **Training Function Edits - EEG Spectrograms - Time x Frequencies ONLY HYPER-PARAMS**

#### **Sweep separati per ciascuno dei modelli CNN2D_LSTM_TF, BiLSTM e Transformer**

In [ ]:
'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

import re

def parse_combination_key(combination_key):
    """
    Estrae condition_experiment e subject_key da combination_key
    dove il data_type è fisso a "spectrograms".
    
    Esempio di chiave: 
    "pt_resp_vs_shared_resp_spectrograms_familiar_th"
    """
    match = re.match(
        r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        condition_experiment = match.group(1)
        subject_key = match.group(2)
        return condition_experiment, subject_key
    else:
        raise ValueError(f"Formato non valido: {combination_key}")

# Test
combination_key = "rest_vs_left_fist_spectrograms_familiar_th"
condition_experiment, subject_key = parse_combination_key(combination_key)

print("Condizione:", condition_experiment)
print("Soggetto:", subject_key)


In [ ]:
'''
                                                                ***** FUNZIONE DI TRAINING *****
                                                                ***** VERSIONE DEL 5 MARZO *****
                                                                
                                                                    **** SALVATAGGIO DI **** 
                                                        
                                                        1) PESI E BIAS DI UN CERTO MODELLO 
                                                        2) CONFIGURAZIONE IPER-PARAMETRI DI UN CERTO MODELLO
                                                                
Il punto critico è garantire che ogni configurazione di iperparametri estratta randomicamente da W&B per OGNI SWEEP sia coerente con:

Il dataset giusto (ossia la coppia di condizioni sperimentali corrispondente).
Il tipo di dato EEG usato (1_20, 1_45, wavelet ecc.).
L'origine dei dati tra le quattro tipologie di soggetti.


che io andrei a prelevare ogni volta da 'data_dict_preprocessed'!

Quindi, ad ogni iterazione del loop sui dati (i.e., data_dict_preprocessed?)
il codice dovrebbe assicurarsi/verificare che, 


1) la configurazione selezionata da W&B presa da uno SPECIFICO SWEEP,  
sia quella che effettivamente corrisponde ad un certo dataset in termini di combinazione di fattori 

- una specifica condizione sperimentale
- una specifico tipo di dato EEG 
- una specifica combinazione di ruolo/gruppo


2) che le run di quella sweep siano inserita nel progetto del dataset di quella specifica condizione sperimentale,


(3 PLUS OPZIONALE

e che il "name" e i "tag" (eventualmente, delle runs associate a quello sweep)
siano costruiti in maniera coerente con la combinazione di fattori associata allo sweep (e quindi alla condizione sperimentale corrente)



****************************** ******************************
CONCLUSIONE A CUI SON ARRIVATO LA MATTINA DEL 04/03/2025: 
****************************** ******************************

Dato che ogni sweep si applica per verificare, tra le 15 diversi set di iper-parametri diversi, 
quale sia la configurazione migliore, per uno specifico set di dati in termini di combinazione di fattori, che sono

- relativi ad una certa condizione sperimentale,  
- con un certo preprocessing
- con un certa provenienza del dato


Son arrivato ad un punto in cui credo che sia davvero molto complesso controllare la corrispondenza esatta tra 

1) di chi esegue lo sweep
2) la definizione del nome della sue 15 runs (cioè di quale dato si riferisca etc. in termini di combinazione di fattori) ...

Quindi l'unica cosa che ha senso è forse solo creare le runs in modo da inserirle tutte assieme in base al solo nome del progetto,
che però è prelevabile dalla prima chiave di 'data_dict_preprocessed'.. 

in questo modo, pur non avendo il controllo sul nome della run e del suo tag,
almeno dovrei esser sicuro che comunque le runs associate all'uso dei dati di ALMENO 
una certa condizione sperimentale vengano inserite nel relativo progetto su weight and biases...



TUTTAVIA, 

****************************** ******************************
ILLUMINAZIONE DEL POMERIGGIO DEL 04/03/2025: 
****************************** ******************************

MI HA PORTATO A PENSARE A PROVARE A CAPIRE ANCORA SE RIESCO A RISOLVERE IL PROBLEMA ...
'''


#VERSIONE NUOVA!

#Fase 2: Creazione della funzione di 'training_sweep' 
    
'''Questa funzione parse_combination_key serve per estrarre 
le varie stringhe che compongono la combinazioni di fattori (condizione sperimentale, tipo di dato EEG e provenienza del dato EEG) 
che si riferiscono allo sweep ID corrente.

Esempio:

Lo tupla sweep (sweep ID, combinazioni di fattori in stringa) è la seguente:

Inizio l'agent per sweep_id: ('4u94ovth', 'pt_resp_vs_shared_resp_wavelet_unfamiliar_pt') dove
- sweep ID: 4u94ovth
- combinazioni di fattori in stringa: pt_resp_vs_shared_resp_wavelet_unfamiliar_pt

Di conseguenza, quando avvio l'agent per quella condizione sperimentale nel loop, 
dentro la funzione di 'training_sweep' io prenderò in input la tupla


""" Esegue il training per uno specifico sweep """

def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 

sweep_id, combination_key = sweep_tuple
exp_cond, data_type, category_subject = parse_combination_key(combination_key)


E lui estrarrà la combinazione di fattori che la compongono, in questo caso è 

1) Condizione Sperimentale = pt_resp_vs_shared_resp
2) Tipo di Dato EEG = wavelet
3) Provenienza del Tipo di Dato EEG unfamiliar_pt

Successivamente, confronta se questa combinazione di stringhe si trova dentro la mia struttura dati e, se la trova

1) creerà il progetto con il nome della condizione sperimentale combaciante tra 
 
 - la combination_key associata allo Sweep ID corrente e
 - il sottodizionario di data_dict_preprocessed 
 
2) le relative run di quello specifico Sweep, verranno nominate con la combinazioni di fattori combaciante su W&B

3) Esegue e gestisce il salvataggio della migliore configurazione di iper-parametri del relativo modello preso in esame (CNN1D, BiLSTM e Transformer)
   tra le 15 runs di OGNI SWEEP
   

'''

import re

def parse_combination_key(combination_key):
    """
    Estrae condition_experiment e subject_key da combination_key
    dove il data_type è fisso a "spectrograms".
    
    Esempio di chiave: 
    "pt_resp_vs_shared_resp_spectrograms_familiar_th"
    """
    match = re.match(
        r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        condition_experiment = match.group(1)
        subject_key = match.group(2)
        return condition_experiment, subject_key
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        
        
def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 
    
    # Per ogni sweep, che viene iterato nel loop, io prendo 
    #1) la stringa univoca dello Sweep ID
    #2) la sua combinazione di fattori stringa (che mi serviranno per prelevare il dato corrispondente da 'data_dict_preprocessed'
    
    sweep_id, combination_key = sweep_tuple
    
    # Ora la funzione restituisce solo (exp_condition, subject_key)
    exp_cond, category_subject = parse_combination_key(combination_key)
    
    # Poiché ora i dati sono solo di tipo "spectrograms", li impostiamo in modo fisso:
    data_type = "spectrograms"

    if not (exp_cond in data_dict_preprocessed and category_subject in data_dict_preprocessed[exp_cond][data_type]):
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{category_subject}\033[0m")
    

    run_name = f"{exp_cond}_{data_type}_{category_subject}"
    
    tags = [exp_cond, data_type, category_subject]

    #Inizializza la run dello specifico Sweep dentro Weights & Biases (W&B) con

    #1) un nome del progetto pari alla condizione sperimentale corrente
    #2) il nome e tag della run in base alla combinazione di fattori corrispondente
    #3) la congiurazione di iper-parametri è pari a quella passata in input a 'training_sweep'

    #Vedi questo link su wandb.init() per vedere i suoi parametri --> #https://docs.wandb.ai/ref/python/init/
    
    # Inizializza la run in W&B nel progetto che termina con "_spectrograms"
    
    '''OCCHIO DA CAMBIARE CAMBIATO'''
        
    #wandb.init(project=f"{exp_cond}_spectrograms_channels_freqs_new_3d_grid_multiband", name=run_name, tags=tags)
    
    #PER TASK 1/3
    wandb.init(project=f"{exp_cond}_spectrograms_time_freqs_new_3d_grid_multiband", name=run_name, tags=tags)
    
    #PER TASK 2/4
    #wandb.init(project=f"{exp_cond}_spectrograms_time_freqs_new_imagery_3d_grid_multiband", name=run_name, tags=tags)

    print(f"\nCreo wandb project per: \033[1m{exp_cond}_spectrograms\033[0m")
    print(f"Lo sweep corrente è \033[1m{sweep_tuple}\033[0m")
    print(f"\nInizio addestramento sul dataset \033[1m{exp_cond}\033[0m con dati EEG \033[1m{data_type}\033[0m di \033[1m{category_subject}\033[0m")

    # Parametri dell'esperimento presi da wandb
    config = wandb.config

    # Recupera i dati pre-processati per la combinazione corrente una volta verificata l'esatta corrispondenza tra:
    #1)il combination_key dello sweep
    #2)l'esistenza di specifico dataset con le stesse 'combination_key' dentro data_dict_preprocessed

    try:
        X_train, X_val, X_test, y_train, y_val, y_test = data_dict_preprocessed[exp_cond][data_type][category_subject]
        print(f"\nCarico i dati di \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")
        print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
        print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
        print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")
    except KeyError:
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")


    if config.standardization:
        # Standardizzazione
        X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
        print(f"\nUso DATI \033[1mSTANDARDIZZATI\033[0m!")
    else:
        print(f"\nUso DATI \033[1mNON STANDARDIZZATI\033[0m!")

    # Preparazione dei dataloaders (N.B. prendo uno dei modelli considerati dentro config.model_name)
    train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
        X_train, X_val, X_test, y_train, y_val, y_test, model_type=config.model_name, batch_size = config.batch_size
    )

    #Qui estraggo il relativo modello su cui sto iterando al momento corrente e lo inizializzo

        
    # Inizializza il modello in base al valore scelto in config.model_name
    #if config.model_name == "CNN2D":
    #    model = CNN2D(input_channels=64, num_classes=2)
    #    print(f"\nInizializzazione Modello \033[1mCNN2D\033[0m")
    
    '''# ====== MODELLI TIME×FREQ ======
    
    cnn_model = CNN2D_LSTM_TF(input_channels = input_channels, num_classes =num_classes, dropout = dropout)
    hidden_sizes = [24, 48, 62]
    lstm_model = ReadMEndYou(input_size=input_channels * num_freqs, hidden_sizes=hidden_sizes, output_size=num_classes)
    transformer_model = ReadMYMind(d_model=d_model, num_heads=num_heads, num_layers=num_layers, num_classes=num_classes)
    
    # Creazione di dati fittizi per il test
    x = torch.randn(batch_size, input_channels, num_freqs, num_timepoints)  # (batch, canali, frequenze, tempo)
    
    '''
    
    # ricavo channels e freqs dai dati (shape attesa: N, C, F, T)
    channels, freqs = int(X_train.shape[1]), int(X_train.shape[2])
    
    if config.model_name == "CNN2D_LSTM_TF":
        
        '''OCCHIO QUI ADESSO SAREBBE TEMPO x FREQUENZA'''
    
        model = CNN2D_LSTM_TF(
            input_channels = channels, # qui 64
            num_classes = 2,
            dropout=config.dropout,
        )

        print(f"\nInizializzazione Modello \033[1mCNN2D_LSTM_TF\033[0m")
    
    
    elif config.model_name == "BiLSTM":
        hidden_sizes = [24, 48, 62]
        model = ReadMEndYou(
            input_size = channels * freqs, #  qui 64*81
            hidden_sizes = hidden_sizes,
            output_size = 2,
            dropout = config.dropout,
            bidirectional = config.bidirectional
        )
        print(f"\nInizializzazione Modello \033[1mReadMEndYou (BiLSTM)\033[0m")
    
    
    elif config.model_name == "Transformer":
        model = ReadMYMind(
            d_model=config.d_model,
            num_heads=config.num_heads,
            num_layers=config.num_layers,
            num_classes=2,
            channels = channels,
            freqs = freqs 
            
        )
        
        print(f"\nInizializzazione Modello \033[1mReadMYMind (Transformer)\033[0m")
    else:
        raise ValueError(f"Modello sconosciuto: {config.model_name}")
        
    #elif config.model_name == "BiLSTM":
        # Qui, input_size = canali * frequenze = 3 * 38 = 78
        #model = ReadMEndYou(input_size= 64 * 45, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
        #print(f"\nInizializzazione Modello \033[1mBiLSTM\033[0m")
        
    #elif config.model_name == "Transformer":
        # Per il Transformer, passiamo anche i parametri channels e freqs per adattare l'embedding
        #model = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=64, freqs=45)
        #print(f"\nInizializzazione Modello \033[1mTransformer\033[0m")

        
    #ORIGINAL VERSION OF TIME SERIES EEG DATA REPRESENTATION  
    #def initialize_models():
        #model_CNN = CNN1D(input_channels=3, num_classes=2)
        #model_LSTM = ReadMEndYou(input_size=3, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
        #model_Transformer = ReadMYMind(num_channels=3, seq_length=250, d_model=16, num_heads=4, num_layers=2, num_classes=2)
        
        #return model_CNN, model_LSTM, model_Transformer

    '''
    Cosa è cambiato rispetto alla tua versione
    Optimizer Adam ora prende betas=(0.9,0.999) e eps=1e-8.

    ReduceLROnPlateau posizionato subito dopo l’optimizer, chiamato su val_loss ogni epoca.

    EarlyStopping con patience=12, mode='min' su val_loss.

    Loop sulle epoche fino a config.n_epochs (100), senza limitare a 60.

    Tutti i parametri di sweep_config (lr, weight_decay, n_epochs, patience, batch_size, standardization…) rimangono esposti e loggati.

    In questo modo riproduci fedelmente il training descritto nel paper, senza stravolgere la tua pipeline di sweep.
    '''
    
    '''OLD VERSION'''
    #optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    
    '''NEW VERSION'''
    optimizer = optim.Adam(
        model.parameters(),
        lr = config.lr,               # da sweep: es. [0.01,0.001,...]
        betas = (config.beta1, config.beta2),          # paper
        eps = config.eps,                    # paper
        weight_decay=config.weight_decay
    )
    
    criterion = nn.CrossEntropyLoss(weight = class_weights_tensor)
    
    '''NEW VERSION'''
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode ='min',      # monitoriamo val_loss
        factor = 0.1,      # dimezza lr
        patience = 8,      # 4 epoche di plateau
        verbose = True
    )
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Parametri di training
    n_epochs = config.n_epochs
    patience = config.patience
    
    #early_stopping = EarlyStopping(patience=patience, mode='max')
    
    '''NEW VERSION'''
    early_stopping = EarlyStopping(patience=patience, mode='min')
    
    best_model = None
    max_val_acc = 0
    best_epoch = 0
        
    #'''AGGIORNAMENTI FINALI'''
    #from sklearn.metrics import roc_auc_score

    pbar = tqdm(range(n_epochs))

    for epoch in pbar:
        
        # ---------------------- TRAIN ----------------------
        #'''AGGIORNAMENTI FINALI'''
        #model.train()  
        
        train_loss_tmp = []
        correct_train = 0
        y_true_train_list, y_pred_train_list = [], []

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            y_pred = model(x)
            loss = criterion(y_pred, y.view(-1))
            loss.backward()
            optimizer.step()

            train_loss_tmp.append(loss.item())
            _, predicted_train = torch.max(y_pred, 1)
            correct_train += (predicted_train == y).sum().item()
            y_true_train_list.extend(y.cpu().numpy())
            y_pred_train_list.extend(predicted_train.cpu().numpy())
            
            #'''AGGIORNAMENTI FINALI'''
            
            # 👇 NOVITÀ: SCORE CONTINUO PER AUC TRAIN (usa la Softmax):
            # OPZIONE A: puoi usare la Softmax per avere le probabilità,
            # OPZIONE B: oppure direttamente CrossEntropy y_pred[:,1] (logit della classe 1).
            
            # Opzione A: usare le probabilità (softmax) 
            
            #DECOMMENTA QUESTE 2 RIGHE PER USARE SOFTMAX
            
            #probs_train = torch.softmax(y_pred, dim=1)
            #y_score_train_list.extend(probs_train[:, 1].detach().cpu().numpy())
            
            # Opzione B: usare direttamente i logits della classe 1 (consigliata, compatibile con CrossEntropy)
            
            #DECOMMENTA QUESTA RIGA PER USARE CROSSENTROPY
            
            # y_score_train_list.extend(y_pred[:, 1].detach().cpu().numpy())

        accuracy_train = correct_train / len(train_loader.dataset)
        loss_train = np.mean(train_loss_tmp)

        precision_train = precision_score(y_true_train_list, y_pred_train_list, average='weighted')
        recall_train = recall_score(y_true_train_list, y_pred_train_list, average='weighted')
        f1_train = f1_score(y_true_train_list, y_pred_train_list, average='weighted')
        
        '''come dovrebbe essere calcolato se non si dovesse passare al load_best_run_results'''
        #auc_train = roc_auc_score(y_true_train_list, y_pred_train_list)
        
        '''come è stato calcolato se si dovesse passare al load_best_run_results'''
        auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')
        
        
        
        #'''AGGIORNAMENTI FINALI'''
        #try:
            #auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')
        #except ValueError:
            #print("⚠️ AUC non calcolabile: nel train set c'è una sola classe.")
            #auc_val = np.nan
        
        # ---------------------- VALIDATION ----------------------
        #'''AGGIORNAMENTI FINALI'''
        #model.eval()
        
        loss_val_tmp = []
        correct_val = 0
        y_true_val_list, y_pred_val_list = [], []
        
                
        #'''AGGIORNAMENTI FINALI'''
        #y_score_val_list = []  # per AUC valida

        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                y_pred = model(x)

                loss = criterion(y_pred, y.view(-1))
                loss_val_tmp.append(loss.item())
                _, predicted_val = torch.max(y_pred, 1)

                correct_val += (predicted_val == y).sum().item()
                y_true_val_list.extend(y.cpu().numpy())
                y_pred_val_list.extend(predicted_val.cpu().numpy())
                
                #'''AGGIORNAMENTI FINALI'''
                
                # 👇 NOVITÀ: SCORE CONTINUO PER AUC TRAIN (usa la Softmax):
                
                # OPZIONE A: puoi usare la Softmax per avere le probabilità,
                # OPZIONE B: oppure direttamente CrossEntropy y_pred[:,1] (logit della classe 1).
                
                # Opzione A: usare le probabilità (softmax) 
                
                #DECOMMENTA QUESTE 2 RIGHE PER USARE SOFTMAX
                
                #probs_val = torch.softmax(y_pred, dim=1)
                #y_score_val_list.extend(probs_val[:, 1].detach().cpu().numpy())
                
                # Opzione B: usare direttamente i logits della classe 1 (consigliata, compatibile con CrossEntropy)
                
                #DECOMMENTA QUESTA RIGA PER USARE CROSSENTROPY
                # y_score_val_list.extend(y_pred[:, 1].detach().cpu().numpy())
                

        accuracy_val = correct_val / len(val_loader.dataset)
        loss_val = np.mean(loss_val_tmp)
        
        #'''AGGIORNAMENTI FINALI'''
        #precision_val = precision_score(y_true_val_list, y_pred_val_list, average='weighted')
        #recall_val    = recall_score(y_true_val_list, y_pred_val_list, average='weighted')
        #f1_val        = f1_score(y_true_val_list, y_pred_val_list, average='weighted')
        
        #try:
            # ATTENZIONE: qui usiamo gli score continui, NON le etichette
            #auc_val = roc_auc_score(y_true_val_list, y_score_val_list, average='weighted')
        #except ValueError:
            #print("⚠️ AUC non calcolabile: nel validation set c'è una sola classe.")
            #auc_val = np.nan

        wandb.log({
            "epoch": epoch,
            
            # TRAIN
            "train_loss": loss_train,
            "train_accuracy": accuracy_train,
            "train_precision": precision_train,
            "train_recall": recall_train,
            "train_f1": f1_train,
            "train_auc": auc_train,
            
            # VALIDATION
            
            "val_loss": loss_val,
            "val_accuracy": accuracy_val,
            
            # se vuoi loggare anche queste (consigliato):
            
            #"val_precision": precision_val,
            #"val_recall": recall_val,
            #"val_f1": f1_val,
            #"val_auc": auc_val,
        })
        
        #Nota: questa patch qua sopra (correzione su train e validation) rende corretto anche train_auc per le run future, 
        #quindi non avrai più bisogno della “correzione a posteriori” in load_best_run_results 
        #per i nuovi esperimenti (ma la puoi lasciare per compatibilità coi vecchi run).

        if accuracy_val > max_val_acc:
            max_val_acc = accuracy_val
            best_epoch = epoch
            best_model = cp.deepcopy(model)
            
        '''OLD VERSION'''
        #early_stopping(accuracy_val)
        #if early_stopping.early_stop:
            #print("🛑 Early stopping attivato!")
            #break

        '''NEW VERSION'''
        scheduler.step(loss_val)
        early_stopping(loss_val)
        if early_stopping.early_stop:
            print(f"🛑 Early stopping attivato!")
            break

        '''
        Qui, si usa config.model_name tra le chiavi di best_models, 
        così che gestisca automaticamente il salvataggio del best model estratto dalla configurazione randomica di iper-parametri
        della specifica run di un determinato sweep, che è relativa allo specifico modello correntemente estratto randomicamente dalla sweep_config!
        
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        IMPORTANTISSIMO: COME SALVARSI LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI UN CERTO MODELLO, DI UN DATO DI UNA CERTA COMBINAZIONE DI FATTORI
        (CONDIZIONE SPERIMENTALE, TIPO DI DATO, PROVENIENZA DEL DATO!)
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        
        CHATGPT:
        
        Nei run eseguiti con W&B ogni esecuzione registra automaticamente la configurazione degli iper-parametri (tramite wandb.config) 
        insieme alle metriche e ai log. 
        Quindi, a meno che tu non abbia modificato il comportamento predefinito, 
        ogni run con il tuo sweep ha già la configurazione associata registrata nei run logs di W&B.

        Tuttavia, per associare in modo “automatico” e diretto la migliore configurazione agli specifici modelli salvati in .pth, 
        potresti considerare di fare uno o più di questi aggiustamenti:

        Salvare la configurazione nel dizionario dei best_models:
        Quando aggiorni il dizionario best_models (cioè quando salvi il miglior modello per una determinata combinazione), 
        puoi salvare anche una copia della configurazione corrente. 
        
        Ad esempio, potresti modificare il blocco in cui aggiorni best_models in questo modo:
        
        best_models[exp_cond][data_type][category_subject][config.model_name] = {
            "model": cp.deepcopy(model),
            "max_val_acc": accuracy_val,
            "best_epoch": best_epoch,
            "config": dict(config)  # Salva la configurazione degli iper-parametri
        }
        
        In questo modo, ogni volta che un modello viene considerato il migliore per quella combinazione,
        la sua configurazione sarà salvata insieme ai pesi.
        Questo ti permetterà, in seguito, di sapere esattamente quali iper-parametri sono stati usati per ottenere quel modello.
        
        
        In sintesi, se hai già usato wandb.config e hai loggato le configurazioni durante le run,
        W&B le ha automaticamente salvate nei run logs. 
        
        Se vuoi rendere più esplicita l'associazione tra il modello salvato (.pth) e la sua configurazione, 
        è utile modificare il tuo codice di TRAINING per salvare ANCHE 
        
        1) il dizionario di configurazione insieme a 
        2) i pesi nel dizionario best_models oppure nei metadati del file salvato.
        
        Questo piccolo accorgimento ti consentirà di recuperare facilmente la configurazione ottimale per ogni modello salvato.
        
        OSSIA
        Aggiungendo la chiave "config": dict(config) nel dizionario che memorizza il best model,
        salvi anche la configurazione degli iper-parametri utilizzata in quella run.
        
        In questo modo, per ogni modello salvato (.pth) potrai recuperare facilmente sia i pesi che la configurazione ottimale che li ha generati.
        
        Questo approccio garantisce che ogni modello sia associato in modo esplicito al set di iper-parametri che ha prodotto le migliori performance, 
        rendendo più semplice il successivo confronto o la replica degli esperimenti.
        
        '''
        
        
        # ***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
        #1)Al posto di salvarmi solo i migliori pesi (i.e.,  model_file = f"{model_path}/{best_model_name}.pth")
        #  ora mi salvo anche la MIGLIORE configurazione di iper-parametri trovata rispetto alle 15 RUNS di un certo SWEEP
        #  di un certo MODELLO, applicato su un DATASET con una SPECIFICA COMBINAZIONE DI FATTORI
        #  condizione sperimentale, tipo di dato e provenienza del dato!
        
    

        if best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"] == -float('inf'):

            # Salvo il primo best_model per quella combinazione
            best_models[exp_cond][data_type][category_subject][config.model_name] = {
                "model": cp.deepcopy(model),
                "max_val_acc": accuracy_val,
                "best_epoch": epoch,
                
                #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
                #***** AGGIUNTA DELLA CHIAVE CONFIG CHE PRELEVA AUTOMATICAMENTE LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DENTRO 'BEST_MODELS'
                
                # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                # in relazione ad un certo modello applicato su un dataset costituito da 
                # una certa combinazione di fattori: 
                # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                "config": dict(config)  
            }

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"

            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)

            os.makedirs(model_path, exist_ok=True)
            
            #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
            #***** SALVATAGGIO DI UN FILE .PKL, CHE CONTIENE 
            
            # I PESI E BIAS DEL MODELLO DERIVATO DALLA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI OTTENUTA DALLA MIGLIORE RUN DI UN CERTO SWEEP
            # IN RELAZIONE AD UN CERTO DATASET COSTITUITO DA UNA CERTA COMBINAZIONE DI FATTORI
            
            '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
            #model_file = f"{model_path}/{best_model_name}.pth"
            
            model_file = f"{model_path}/{best_model_name}.pkl"
            
            '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
            #torch.save(best_model.state_dict(), model_file)
            
            # Salva un dizionario contenente sia i pesi che la configurazione
            torch.save({
                "state_dict": best_model.state_dict(),
                "config": dict(config)
            }, model_file)

            print(f"Il modello \n\033[1m{best_model_name}\033[0m verrà salvato in questa folder directory: \n\033[1m{model_file}\033[0m")

            #Condizione di aggiornamento:
            #Se l'accuracy corrente (accuracy_val) di quel modello di quello sweep supera il valore già salvato in best_models[...], 
            #allora aggiorniamo il dizionario e sovrascriviamo il file del best model, di quel modello, di quella combinazione di fattori.


            # Puoi confrontare e salvare il modello solo se il nuovo è migliore


            #Questo assicura che il salvataggio del modello avvenga solo se
            #il nuovo modello ha un'accuratezza di validazione (max_val_acc) migliore 
            #rispetto a quella già memorizzata per la condizione specifica (exp_cond).

            #In questo modo, si evita di sovrascrivere il modello salvato con uno peggiore


            # Nuovo modello migliore per questa combinazione: aggiorna e sovrascrivi il file


        elif accuracy_val > best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"]:
                best_models[exp_cond][data_type][category_subject][config.model_name] = {
                    "model": best_model,
                    "max_val_acc": accuracy_val,
                    "best_epoch": best_epoch,
                    
                    # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                    # in relazione ad un certo modello applicato su un dataset costituito da 
                    # una certa combinazione di fattori: 
                    # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                    "config": dict(config)  
                }
                best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
                model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
                os.makedirs(model_path, exist_ok=True)

                print(f"Il modello di questa folder directory:\n\033[1m{model_path}\033[0m")
                print(f"\nHa un MIGLIORAMENTO!")

                '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
                #model_file = f"{model_path}/{best_model_name}.pth"

                model_file = f"{model_path}/{best_model_name}.pkl"

                if os.path.exists(model_file):

                    # Se il file esiste, stampiamo un messaggio di aggiornamento
                    print(f"\n⚠️ ATTENZIONE: \nIl modello \033[1m{best_model_name}\033[0m verrà AGGIORNATO in \n\033[1m{model_path}\033[0m")

                    # Salva il miglior modello solo se è stato aggiornato
                    
                    '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
                    #torch.save(best_model.state_dict(), model_file)

                    # Salva un dizionario contenente sia i pesi che la configurazione
                    torch.save({
                        "state_dict": best_model.state_dict(),
                        "config": dict(config)
                    }, model_file)
                    
                    print(f"\nIl nome del modello AGGIORNATO è:\n\033[1m{best_model_name}\033[0m")

                else:
                    continue

                #Condizione "nessun miglioramento":
                #Se il modello corrente non migliora il best già salvato, viene semplicemente stampato un messaggio.

                #Questa logica garantisce che per ogni combinazione il file .pth contenga 
                #sempre i pesi del miglior modello (secondo la validation accuracy) fino a quel momento.
                #Adatta eventualmente i nomi delle variabili (es. accuracy_val vs max_val_acc) per essere coerente con il resto del tuo codice.
        else:
            ''''QUI VA RIDEFINITO LA MODEL_PATH (e anche se vuoi MODE_FILE) ALTRIMENTI IN QUESTO ELSE NON ESISTONO!'''

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
            model_file = f"{model_path}/{best_model_name}.pkl"
            print(f"Nessun miglioramento per il modello \033[1m{config.model_name}\033[0m in \n\033[1m{model_path}\033[0m, ossia \n\033[1m{model_file}\033[0m")

    wandb.finish()
    
    torch.cuda.empty_cache()
    
    return best_models


#### **Weight & Biases Procedure Final Edits - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS**
#### **Sweep separati per ciascuno dei modelli CNN2D_LSTM_TF, BiLSTM e Transformer**

In [ ]:
'''

                                        QUI IL LOOP LO ESEGUO SU OGNI SINGOLO SWEEP DI OGNI COMBINAZIONE DI FATTORI!!!
                                                            
                                                                    VERSIONE C 
                                                                    
                                                                    
                                                W&B SWEEPS AND TRAING LAUNCH WITH MULTIPLE GPUs MANAGEMENT
                                        
Questa volta, invece, andiamo ad iterare rispetto a 

- sweep_tuple, che la tuple che contiene

1) relativo codice stringa univoco dello Sweep ID
2  la sua combination_key, che ri-associa allo Sweep ID la combinazione di fattori della relativa condizione sperimentale


PRIMA FACEVO IN QUESTO MODO

for sweep_id in sweep_ids[condition][data_type][category_subject]:
    print(f"\033[1mInizio l'agent\033[0m per sweep_id: \033[1m{sweep_id}\033[0m")
    
ORA INVECE ITERO SULLA TUPLA!


for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            for sweep_tuple in sweep_ids[condition][data_type][data_tuples]:
        

VERSIONE C (SEMPLIFICATA!)


****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******

SPIEGAZIONE

GPU counter: Ho aggiunto un contatore (gpu_counter) che cicla tra le GPU disponibili. 

In questo modo, il primo sweep sarà eseguito sulla GPU 0, il secondo sulla GPU 1, e così via. 
Quando il contatore raggiunge il numero di GPU disponibili, torna a 0 per riusare la prima GPU.

Rotazione delle GPU: All'interno del loop, per ogni sweep, viene assegnata una GPU diversa. 
Se ci sono più di 1 GPU, il contatore incrementa, e la variabile CUDA_VISIBLE_DEVICES cambia automaticamente per assegnare la GPU corretta.

Esecuzione parallela: Ogni sweep viene eseguito su una GPU separata. Se ci sono 2 GPU, il primo sweep va su GPU 0, il secondo su GPU 1, il terzo su GPU 0, e così via.

Risposta alla tua domanda:
In questo modo, ogni sweep_id viene eseguito una sola volta, ma su GPU diverse (se disponibili). Non ci sono duplicati dello stesso sweep su entrambe le GPU.


DOMANDE SUL NUOVO CODICE

1) Gli sweep sono eseguiti già in parallelo giusto?
No, in questo caso gli sweep non sono eseguiti in parallelo in modo esplicito tramite il codice che hai scritto.

Anche se hai assegnato ciascun sweep a una GPU diversa, il codice esegue sequenzialmente ogni sweep, solo che li distribuisce su GPU differenti in modo rotazionale.
Ogni volta che il ciclo passa ad un nuovo sweep, assegna un ID GPU e poi esegue l'agent su quella GPU. Non vengono eseguiti in parallelo a livello di codice.

2) O semplicemente in questo modo faccio in modo di distribuire ogni sweep sull'altra GPU rispetto a quella usata dallo sweep precedente
per "ottimizzare" il carico computazionale di ogni GPU?

Esatto! Quello che stai facendo è distribuire i vari sweep su GPU diverse, assicurandoti che ogni sweep venga eseguito su una GPU separata (se ne hai di disponibili).
Questo permette di ottimizzare l'uso delle risorse, evitando che una GPU venga sovraccaricata da più sweep. Se il numero di GPU disponibili è maggiore di 1, 
allora i sweep sono distribuiti sulle diverse GPU, ma ogni sweep sarà ancora eseguito singolarmente.





Sì, con il codice che hai fornito, stai distribuendo gli sweep tra le diverse GPU, in modo da ottimizzare il carico computazionale e non sovraccaricare una sola GPU.

Dettaglio del funzionamento:
Distribuzione delle GPU (rotazionale):

Quando ci sono più di una GPU, il codice assegna a ciascun sweep una GPU diversa in modo rotazionale.

Per ogni ciclo del loop, la variabile gpu_counter determina a quale GPU assegnare il prossimo sweep.

Se ci sono 2 GPU, il primo sweep viene eseguito sulla GPU 0, il secondo sulla GPU 1, il terzo di nuovo sulla GPU 0, e così via.

Gestione della GPU:

Se hai più di una GPU, os.environ["CUDA_VISIBLE_DEVICES"] imposta il dispositivo GPU corrente su cui il codice deve girare (GPU 0 o GPU 1). Questo permette di gestire quale GPU eseguirà l'addestramento per ciascun sweep.

Quando num_gpus > 1, il codice alterna l'assegnazione della GPU per ogni sweep, evitando di sovraccaricare una singola GPU con troppe operazioni contemporaneamente.

Ottimizzazione del carico computazionale:

L'alternanza tra GPU successive per ciascun sweep aiuta a distribuire il carico in modo equilibrato, specialmente se il numero di sweep è alto.

In pratica, se hai 2 GPU e 10 sweep da eseguire, ogni GPU eseguirà 5 sweep, evitando di saturare una singola GPU.

Parallelizzazione effettiva:
Nel tuo codice non c'è parallelismo vero e proprio tra le GPU (come quello che otterresti con DataParallel o DistributedDataParallel), ma piuttosto distribuisci gli sweep su GPU diverse. Ogni sweep è eseguito sequenzialmente su una GPU diversa, ma non stai parallelizzando il training dello stesso sweep su più GPU.

Se volessi fare in modo che un singolo sweep fosse parallelizzato su più GPU (in modo che il lavoro venga diviso tra le GPU per uno stesso sweep), dovresti usare DataParallel o DistributedDataParallel, come descritto in precedenza. Ma in questo caso, ogni sweep è isolato e viene eseguito su una sola GPU, quindi non c'è un vero parallelismo dentro ogni singolo sweep.

Domande che potrebbero sorgere:
Gli sweep vengono eseguiti in parallelo?

No, i sweep sono eseguiti uno alla volta, ma su GPU diverse. Quindi, mentre il primo sweep usa la GPU 0, il secondo usa la GPU 1 e così via. Ogni sweep viene gestito separatamente, ma sfrutti più GPU per parallelizzare l'esecuzione di più sweep contemporaneamente.

La distribuzione delle GPU tra gli sweep è ottimizzata?

Sì, stai bilanciando il carico computazionale tra le GPU, assegnando a ogni GPU uno sweep alternato. Se hai molte GPU, puoi ottimizzare ulteriormente distribuendo i sweep su più dispositivi.

Se volessi parallelizzare più agenti W&B su diverse GPU, il codice che stai utilizzando sarebbe corretto, ma per ottimizzare ulteriormente i tempi di esecuzione, potresti prendere in considerazione anche l'utilizzo di tecniche come DataParallel o DistributedDataParallel per far sì che più GPU lavorino contemporaneamente sullo stesso sweep. Ma la logica che hai già implementato va bene per distribuire più sweep tra le GPU.

Se hai bisogno di ulteriori dettagli su come implementare il parallelismo vero e proprio (inclusi DataParallel o DistributedDataParallel), fammi sapere!



****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******



'''



'''

Per modificare il loop in modo che accetti i sweeps per ogni modello e gestisca correttamente
l'esecuzione del training per ciascun modello con il relativo sweep, dobbiamo fare alcune modifiche.


Modifiche principali:

1) Funzione make_train_wrapper:
La funzione dovrà essere adattata per passare correttamente la configurazione di sweep per ogni modello, 
invece di passare un'unica configurazione generica (sweep_config).

2) Identificazione corretta del modello: 
Nel loop, per ogni combinazione (condition, data_type, category_subject)
e per ogni modello (ad esempio, CNN3D_LSTM_FC e SeparableCNN2D_LSTM_FC), 

dobbiamo passare al wandb.agent il relativo sweep ID per il modello e la sua configurazione.

3) Modifica della funzione make_train_wrapper per gestire ogni modello separatamente: 
Ogni modello avrà il proprio sweep e la propria configurazione.


Spiegazione delle modifiche:

1) Funzione make_train_wrapper:

Adesso prende anche model_name per passare il relativo sweep_config dal dizionario sweep_config_dict.
Passa il sweep_config corretto per ogni modello, a seconda del model_name passato nel ciclo.

2) Dizionario sweep_config_dict:

Ho creato un dizionario sweep_config_dict che associa ciascun modello ("CNN3D_LSTM_FC" e "SeparableCNN2D_LSTM_FC")
alla sua configurazione di sweep (sweep_config_cnn3d e sweep_config_cnn_sep).
Questo permette di usare la corretta configurazione per ogni modello.

3) Modifica nel ciclo:

Il ciclo ora scorre su model_name (i.e., i modelli CNN3D_LSTM_FC e SeparableCNN2D_LSTM_FC) 
per ogni combinazione di condition, data_type, category_subject.

Per ogni modello, il relativo sweep viene creato ed eseguito.


Risultato:
Ora, per ogni combinazione di condition, data_type, e category_subject, 
il codice creerà e gestirà separatamente gli sweeps per ciascun modello,
e li eseguirà utilizzando la funzione training_sweep con la relativa configurazione specifica per ogni modello.

Questa modifica ti consente di avere il corretto flusso di lavoro per eseguire
il training separato per ogni modello con la sua configurazione.


'''


import time  # Importa il modulo time


# Definiamo una funzione wrapper che "cattura" lo sweep_id e le altre variabili

'''ATTENZIONE AGGIUNTO model_name tra i parametri di --> make_train_wrapper'''

def make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name):
    def train_wrapper():

        # Qui chiamiamo la funzione di training con i parametri appropriati
        #print(f"\nSetto il training per lo Sweep ID \033[1m{condition}_{data_type}_{category_subject}\033[0m con sweep_id {sweep_id}")
        #print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m")
        
            
        print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m, modello \033[1m{model_name}\033[0m")
        training_sweep(
            data_dict_preprocessed, 
            sweep_config_dict[model_name], # Prendi la configurazione per il modello specifico
            sweep_ids,
            sweep_id,
            sweep_tuple,
            best_models  # Best models viene aggiornato all'interno della funzione
        )
    return train_wrapper
                        

# Dizionari di configurazione per ogni modello

# Comodo mapper per il tuo loop

#sweep_config_dict = {
#    "CNN2D_LSTM_TF": sweep_config_cnn2d_lstm_tf,
#    "BiLSTM": sweep_config_bilstm,
#    "Transformer": sweep_config_transformer,
#}

'''AL PRIMO GIRO ABILITA SOLO QUESTO'''
#sweep_config_dict = {
#    "CNN2D_LSTM_TF": sweep_config_cnn2d_lstm_tf,
#    "BiLSTM": sweep_config_bilstm
#}


'''AL SECONDO GIRO ABILITA QUESTO'''
sweep_config_dict = {
    "Transformer": sweep_config_transformer
}

'''AL SECONDO GIRO ABILITA QUESTO'''
enabled_models = set(sweep_config_dict.keys())  # {'Transformer'}


# Verifica quante GPU sono disponibili
num_gpus = torch.cuda.device_count()


# Crea un contatore per assegnare un GPU diversa a ciascun sweep
gpu_counter = 0

# Registra il tempo di inizio
start_time = time.time()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            for model_name in sweep_ids[condition][data_type][category_subject]:  # Aggiunto loop per il modello
                
                '''AL SECONDO GIRO ABILITA QUESTO'''
                # ⬇️ SKIPPA tutti i modelli non abilitati (CNN2D_LSTM_TF, BiLSTM, ...)
                if model_name not in enabled_models:
                    print(f"Skip {model_name}: non abilitato in questo giro")
                    continue
                    
                #for sweep_tuple in sweep_ids[condition][data_type][category_subject]:
                
                for sweep_tuple in sweep_ids[condition][data_type][category_subject][model_name]:  # Itera sugli sweep per ciascun modello

                    # Esegui l'unpacking della tupla per ottenere solo il primo elemento della tupla (sweep_id, combination_key)
                    sweep_id, combination_key = sweep_tuple
                    
                    
                    combination_key = f"{condition}_{data_type}_{category_subject}"
                    
                    # Un modo efficace per "catturare" il contesto (come sweep_id e le altre variabili) 
                    # per ogni iterazione è definire una funzione wrapper locale all'interno del ciclo
                    # In questo modo, ogni volta che chiami l'agente, il wrapper avrà già i parametri specifici per quella combinazione


                    # Se ci sono più di 1 GPU, assegna a ciascuna GPU uno sweep diverso
                    if num_gpus > 1:

                        '''ATTENZIONE AGGIUNTO model_name tra i parametri di --> make_train_wrapper''' 
                        
                        # Assegna la GPU in modo rotazionale
                        os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_counter)
                        
                        agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name)
                    
                        #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband", count=200)
                        
                        #PER TASK 1/3
                        wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_time_freqs_new_3d_grid_multiband", count=200)
                        
                        #PER TASK 2/4
                        #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_time_freqs_new_imagery_3d_grid_multiband", count=200)
                        
                        # Passa alla prossima GPU per il prossimo sweep
                        gpu_counter = (gpu_counter + 1) % num_gpus

                    else:
                        
                        # Se c'è una sola GPU, esegui il sweep sulla GPU 0
                        os.environ["CUDA_VISIBLE_DEVICES"] = "0"
                        
                        agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name)
                        
                        #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband", count=200)
                        
                        #PER TASK 1/3
                        wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_time_freqs_new_3d_grid_multiband", count=200)
                        
                        #PER TASK 2/4
                        #wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_time_freqs_new_imagery_3d_grid_multiband", count=200)


                    # Crea la funzione wrapper per l'agent
                    '''COMMENTATO'''
                    #agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)


                    # NOTA: non assegno il valore di wandb.agent a best_models, lascio che training_sweep aggiorni best_models internamente!
                    '''DEVI INSERIRE PER L'AGENTE COME PARAMETRO IL NOME DELLA CONDIZIONE SPERIMENTALE DEL PROGETTO SU  W&B
                       ALTRIMENTI CERCA LO SWEEP NEL PROGETTO SBAGLIATO '''

                    print(f"Inizio l'\033[1magent\033[0m per \033[1msweep_id\033[0m \tN°: \033[1m{sweep_tuple}\033[0m")

                    '''COMMENTATO'''
                    #wandb.agent(sweep_id, function=agent_function, project = f"{condition}_spectrograms_channels_freqs_new_2d_grid_multiband_topomap", count=15)

                    print(f"\nLo sweep id corrente \033[1m{sweep_id}\033[0m ha la combinazione di fattori stringhe: \033[1m{condition}; {data_type}; {category_subject}\033[0m\n")

                    torch.cuda.empty_cache()

# Registra il tempo di fine
end_time = time.time()

# Calcola il tempo totale
total_time = end_time - start_time
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = int(total_time % 60)

# Stampa il tempo totale in formato leggibile
print(f"\nTempo totale impiegato: \033[1m{hours} ore, {minutes} minuti e {seconds} secondi\033[0m.\n")

## Impostazione **Recupero DL Optimized Models** con **Rappresentazione Tempo-Frequenza dei miei dati EEG** a seconda del Dataset del Task scelto


Notebook **"EEG Data Stats Descriptives & GradCam Checks"**

- Task 1: Motor Execution (Open and close right or left fist) - Unilateral - Runs 3, 7, 11 
- Task 2: Motor Imagery (Imagine opening and closing right and left fists) - Unilateral - Runs 4, 8, 12 
- Task 3: Motor Execution (Open and close both fists and feet) - Bilateral - Runs 5, 9, 13 
- Task 4: Motor Imagery (Imagine opening and closing both fists and feet) - Bilateral - Runs 6, 10, 14 

#### Implementazione dei Best Models Dopo W&B - EEG SPECTROGRAMS **+ GRADCAM FREQUENCY x TIME (ALL SUBJECTS)**! 

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm
from tqdm import tqdm

import random 

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la visualizzazione dei dati

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader

import os
import pickle

import random

import wandb

##### **NUOVE MODIFICHE SPECIFICHE PER I DATI NON HYPER POST W&B CON GRADCAM**

Allora le modifche che ho ultimato quindi sono:

- **1)Creazione della classe GradCAM**


    **GRADCAM CLASS**

        import torch
        import torch.nn.functional as F
        import cv2
        import numpy as np
        import matplotlib.pyplot as plt

        class GradCAM:
            def __init__(self, model, target_layer):
                self.model = model
                self.target_layer = target_layer
                self.activations = None
                self.gradients = None
                # Registra hook per catturare attivazioni e gradienti
                self.target_layer.register_forward_hook(self.save_activation)
                self.target_layer.register_backward_hook(self.save_gradient)

            def save_activation(self, module, input, output):
                self.activations = output.detach()

            def save_gradient(self, module, grad_input, grad_output):
                self.gradients = grad_output[0].detach()


- **2)** Creazione della funzione per generare delle immagini associate alla GradCAM compution**

    
    **FUNCTION FOR CREATING GRAD-CAM MAPS & FIGURES ASSOCIATED TO GRADCAM COMPUTATION**

        import cv2
        import numpy as np
        import matplotlib.pyplot as plt
        import io

        def compute_gradcam_figure(model, test_loader, exp_cond, data_type, category_subject, device):

            """
            Per il modello CNN2D, seleziona un campione per ciascuna classe (0 e 1),
            calcola la GradCAM e costruisce una figura con:
              - Riga 1: Heatmap per classe 0 e classe 1.
              - Riga 2: Sovrapposizione della heatmap sullo spettrogramma originale.
            I titoli della figura vengono personalizzati con exp_cond, data_type, category_subject.
            """

            # Assumiamo che il modello sia CNN2D e che il layer target sia model.conv3
            target_layer = model.conv3
            gradcam = GradCAM(model, target_layer)

            # Dizionari per salvare il campione per ogni classe
            samples = {}      # Salveremo il sample input per ogni classe
            labels_found = {} # Per tenere traccia delle etichette già trovate

            # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                for i, label in enumerate(labels):
                    label_int = int(label.item())
                    if label_int not in labels_found:
                        samples[label_int] = inputs[i].unsqueeze(0)  # salva come tensore 4D
                        labels_found[label_int] = True
                    if 0 in labels_found and 1 in labels_found:
                        break
                if 0 in labels_found and 1 in labels_found:
                    break

            # Se non troviamo entrambi gli esempi, esci con un messaggio
            if 0 not in samples or 1 not in samples:
                print("Non sono stati trovati esempi per entrambe le classi nel test_loader.")
                return None

            # Per ciascun campione, calcola GradCAM
            cams = {}
            overlays = {}
            for cls in [0, 1]:
                sample_input = samples[cls]
                sample_input.requires_grad = True  # Abilita gradiente per il campione
                cam = gradcam.generate_cam(sample_input)
                cams[cls] = cam

                # Converti il sample in immagine numpy per la visualizzazione
                img = sample_input.squeeze().cpu().detach().numpy().transpose(1, 2, 0)
                # Normalizza l'immagine in scala 0-255
                img_norm = np.uint8(255 * (img - img.min()) / (img.max() - img.min()))
                # Applica la heatmap
                heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
                heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
                # Sovrapponi la heatmap all'immagine originale
                overlay = cv2.addWeighted(img_norm, 0.6, heatmap, 0.4, 0)
                overlays[cls] = overlay

            # Crea la figura con due righe e due colonne
            fig, axs = plt.subplots(2, 2, figsize=(12, 10))

            # Titolo per la prima riga
            title_row1 = f"Grad-CAM mapping of experimental condition {exp_cond}, EEG {data_type}, Subject {category_subject}"
            # Titolo per la seconda riga
            title_row2 = f"Grad-CAM mapping superimposition over EEG Spectrogram of experimental condition {exp_cond}, Subject {category_subject}"

            # Prima riga: solo le heatmap
            for j, cls in enumerate([0, 1]):
                axs[0, j].imshow(cv2.cvtColor(cv2.applyColorMap(np.uint8(255 * cams[cls]), cv2.COLORMAP_JET), cv2.COLOR_BGR2RGB))
                axs[0, j].set_title(f"Class {cls} Heatmap")
                axs[0, j].axis('off')
            axs[0, 0].set_ylabel(title_row1, fontsize=10)

            # Seconda riga: overlay della heatmap sullo spettrogramma originale
            for j, cls in enumerate([0, 1]):
                axs[1, j].imshow(overlays[cls])
                axs[1, j].set_title(f"Class {cls} Overlay")
                axs[1, j].axis('off')
            axs[1, 0].set_ylabel(title_row2, fontsize=10)

            # Ottimizza la disposizione della figura
            plt.tight_layout()

            # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
            buf = io.BytesIO()
            plt.savefig(buf, format='png')
            buf.seek(0)
            fig_image = buf.getvalue()
            buf.close()
            plt.close(fig)

            return fig_image


- **3) Modifica delle funzioni per il salvataggio delle immagini create tramite la GradCAM compution**

    **FUNCTIONS FOR GRADCAM COMPUTATION & SAVING**
    
    Questa modifica consente di creare ed adattare le path di salvataggio ANCHE delle immagini calcolate dalla classe customizzata di GradCAM, 
    delle mappe di attivazione prodotte dalle feature maps e della sovrapposizione delle stesse aree decisionali
    rilevanti per la migliore classificazione dei dati di esempio di una certa classe,
    a partire da un certo dataset composto da una certa combinazione di fattori
    (i.e., exp_cond, data_type, category_subject)


#NEW VERSIONS FOR SPECTROGRAMS WITH GRADCAM COMPUTATION ON CNN2D!

    **Funzione per determinare a quale subfolder appartiene la chiave**
    def get_subfolder_from_key(key, model_standardization):

        #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
        if '_familiar_th' in key:
            return 'th_fam'
        elif '_unfamiliar_th' in key:
            return 'th_unfam'
        elif '_familiar_pt' in key:
            return 'pt_fam'
        elif '_unfamiliar_pt' in key:
            return 'pt_unfam'
        else:
            return None


    from PIL import Image
    import io
    import pickle
    import os

    **Funzione per salvare i risultati**
    def save_performance_results(model_name, 
                                 my_train_results,
                                 my_test_results, 
                                 key,
                                 exp_cond,
                                 model_standardization,
                                 base_folder,
                                 gradcam_image = None):
        """
        Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
        Se gradcam_image è fornita, la salva anche in formato PNG con un nome che inizia con 'GradCAM_results'.
        """

        # Identificazione del subfolder in base alla chiave
        subfolder = get_subfolder_from_key(key, model_standardization)

        # Debug: controllo sulla subfolder
        print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")

        if subfolder is None:
            print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
            return

        # Determinazione del tipo di dato direttamente dalla chiave
        if "spectrograms" in key:
            data_type_str = "spectrograms"
        else:
            print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
            return

        # Creazione del nome del file pickle con l'inclusione della combinazione key + model_name
        if model_standardization:
            file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}_std.pkl"
            folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
        else:
            file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}.pkl"
            folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)

        # Verifica se la cartella di destinazione esiste, altrimenti creala
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        file_path = os.path.join(folder_path, file_name)

        # Creazione del dizionario con i risultati
        results_dict = {
            'my_train_results': my_train_results,
            'my_test_results': my_test_results
        }

        # Salvataggio del dizionario con i risultati
        try:
            with open(file_path, 'wb') as f:
                pickle.dump(results_dict, f)
            print(f"\n🔬Risultati salvati con successo 👍 in: \n\033[1m{file_path}\033[0m\n")
        except Exception as e:
            print(f"❌Errore durante il salvataggio dei risultati: {e}")

        # Se è stata fornita l'immagine GradCAM, salvala come file PNG
        if gradcam_image is not None:
            if model_standardization:
                gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}_std.png"
            else:
                gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}.png"

            gradcam_file_path = os.path.join(folder_path, gradcam_file_name)

            #try:
            #    with open(gradcam_file_path, "wb") as f_img:
            #        f_img.write(gradcam_image)
            #    print(f"\n📸Immagine GradCAM salvata con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")

            try:

                '''
                Se gradcam_image è un oggetto BytesIO, allora rappresenta un flusso di dati binari in memoria.
                Quando si leggono dati da un BytesIO, il cursore interno avanza come in un file normale. 
                Se il cursore non è all'inizio, Image.open() potrebbe non leggere correttamente l'immagine.
                👉 seek(0) riporta il cursore all'inizio del buffer prima di leggerlo con Image.open()

                Per maggior info leggi cella successiva!
                '''

                # 🔄 Se gradcam_image è un buffer, convertirlo in immagine PIL
                if isinstance(gradcam_image, io.BytesIO):
                    gradcam_image.seek(0)  # 🔄 Reset puntatore del buffer
                    gradcam_image = Image.open(gradcam_image)

                print(f"\n📸Immagine GradCAM salvata con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")
                # 🔄 Salvare l'immagine nel percorso specificato
                gradcam_image.save(gradcam_file_path, format = "PNG")

            except Exception as e:
                print(f"❌Errore durante il salvataggio dell'immagine GradCAM: {e}")


- **4) Integrazione nel loop di training e test dei punti 1), 2) e 3)**

    **INTEGRATION OF GRADCAM COMPUTATION IN THE TRAINING E FOR LOOP**

    Nel loop che esegue il training ed il testing, integrazione della parte di inizializzazione della classe custom di GradCAM, con cui si esegue 

    il calcolo delle mappe di attivazione e della sovrapposizione delle mappe di attivazione stesse sullo spettogramma originale, 
    riportate poi in due immagini distinte create nella stessa figura che vengono salvate correttamente nella stessa directory path. 

    Le due immagini dovrebbero rappresentare l'heatmap activation e la sovrapposizione della mappa di attivazione sullo spettogramma originale,
    relativo ad un esempio rappresentativo per ciascuna delle due classi possibili presenti nello stesso dataset correntemente iterato.

    Il loro contributo è di descrivere se la CNN2D abbia identificato delle (possibili) differenti aree decisionali delle feature maps 
    (e dunque dello spettrogramma) maggiormente utili ai fini della discriminazione delle due condizioni sperimentali inserite all'interno del dataset correntemente iterato.


        ** Dizionario per tracciare la standardizzazione usata per ogni combinazione di dati**
        ** Dizionario per salvare informazioni sul modello (es. se i dati sono standardizzati)**

        models_info = {}

        ** Set per tenere traccia dei dataset già elaborati**
        processed_datasets = set()

        ** Set per tenere traccia delle combinazioni già elaborate**
        processed_models = set()

        ** Path delle performance dei modelli ottimizzati con weight and biases**
        ** Path per trovare le best performances di ogni modello per ogni combinazione dei dati**
        base_folder = "/home/stefano/Interrogait/WB_spectrograms_best_results"

        ** Path di salvataggio delle performance dei modelli dopo estrazione best models da base_folder**
        save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB"


        ** --- LOOP PRINCIPALE (con minime modifiche) ---**
        for key, (X_data, y_data) in data_dict.items():

            print(f"\n\nEstrazione Dati per il dataset: \033[1m{key}\033[0m, \tShape X: \033[1m{X_data.shape}\033[0m, Shape y: \033[1m{y_data.shape}\033[0m")

            if key in processed_datasets:
                print(f"ATTENZIONE: Il dataset {key} è già stato elaborato! Salto iterazione...")
                continue

            processed_datasets.add(key)

            X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
            print(f"Dataset Splitting: Train: \033[1m{X_train.shape}\033[0m, Val: \033[1m{X_val.shape}\033[0m, Test: \033[1m{X_test.shape}\033[0m")

            for model_name in ["CNN2D", "BiLSTM", "Transformer"]:

                model_key = f"{model_name}_{key}"
                if model_key in processed_models:
                    print(f"ATTENZIONE: Il modello {model_name} per il dataset {key} è già stato addestrato! Salto iterazione...")
                    continue
                processed_models.add(model_key)

                print(f"\nPreparazione dati per il dataset \033[1m{key}\033[0m e il modello \033[1m{model_name}\033[0m...")

                # Prova a caricare la configurazione e i pesi ottimali dal file .pkl

                '''
                load_config_if_available --> prende in input 'key' che è la chiave composita (i.e, th_resp_vs_pt_resp_1_20_familiar_th)
                parse_combination_key --> prende in input 'key' che suddivide la chiave composita in stringhe separate

                exp_cond, data_type, category_subject che sfrutto per crearmi la directory path che mi servirà per caricarmi 
                pesi del modello e i suoi iper-parametri

                Diciamo che in questo caso, sfrutto 'parse_combination_key per qualcosa che serve a 'load_config_if_available' in modo IMPLICITO..
                '''

                config, best_weights = load_config_if_available(key, model_name, base_folder)

                if config is None:
                    raise ValueError(f"\033[1mNessun file .pkl trovato per {model_name} su {key}\033[0m. Non posso procedere senza la configurazione ottimale.")

                '''
                Successivamente, queste variabili vengono invece create in maniera ESPLICITA per fasi successive del loop
                MA in questo caso, parsifica la chiave una VOLTA SOLA e memorizza i valori!
                '''

                # Parsifica la chiave una volta sola e memorizza i valori
                exp_cond, data_type, category_subject = parse_combination_key(key)

                '''
                Dpodiché, 

                1) si carica i vari valori degli iper-parametri,
                2) si esegue la standardizzazione se servisse,
                3) prepara il modello per la divisione in train_loader etc.,
                4) si carica la configurazione dei pesi del modello, 
                5) assegna i vari valori degli iper-parametri del modello corrente per la combinazione di dati correntemente iterata 

                6) esegue il training e il test e poi

                7) si salva il tutto nella path corrispondente...

                '''

                '''
                PER DARE UNIFORMITÀ AL CODICE, CAMBIO IL NOME DELLE VARIABILI, CHE CONTENGONO I VALORI OTTIMIZZATI 
                DA FORNIRE IN INPUT ALLE VARIE FUNZIONI CHE SONO RICHIAMATE NEL LOOP'''

                model_batch_size = config["batch_size"]
                model_n_epochs = config["n_epochs"]
                model_patience = config["patience"]
                model_lr = config["lr"]
                model_weight_decay = config["weight_decay"]
                model_standardization = config["standardization"]

                print(f"Parametri per \033[1m{model_name}\033[0m: batch_size= \033[1m{model_batch_size}\033[0m, n_epochs= \033[1m{model_n_epochs}\033[0m, patience= \033[1m{model_patience}\033[0m, lr= \033[1m{model_lr}\033[0m, weight_decay= \033[1m{model_weight_decay}\033[0m, standardization= \033[1m{model_standardization}\033[0m")

                # Salva nel dizionario se per quella combinazione è stata applicata la standardizzazione ai dati
                models_info[model_key] = {"standardization": model_standardization}


                '''PER MANTENERE LA STESSA LOGICA DEL CODICE (ANCHE SE POTREI INSERIRLA DENTRO PREPARE_DATA_FOR_MODEL MODIFICANDO LA FUNZIONE (SI VEDA IN CELLA SOPRA COME)
                IMPONGONO LA STANDARDIZZAZIONE PRIMA DI QUESTA FUNZIONE
                '''

                if model_standardization:
                    X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
                    print(f"\033[1mSÌ Standardizzazione Dati!\033[0m")
                else:
                    print(f"\033[1mNO Standardizzazione Dati!\033[0m")

                # Sposta il modello sulla GPU (se disponibile)
                device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


                # Preparazione dei dataloaders
                train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
                    X_train, X_val, X_test, y_train, y_val, y_test, model_type = model_name, batch_size = model_batch_size)

                # Inizializzazione del modello
                if model_name == "CNN2D":
                    model = CNN2D(input_channels=3, num_classes=2)
                elif model_name == "BiLSTM":
                    model = ReadMEndYou(input_size= 3 * 26, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
                elif model_name == "Transformer":
                    model = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=3, freqs=26)
                else:
                    raise ValueError(f"Modello {model_name} non riconosciuto.")

                # Se abbiamo caricato i pesi ottimali, li carichiamo nel modello
                if best_weights is not None:
                    try:
                        model.load_state_dict(best_weights)
                        print(f"📊 Modello \033[1m{model_name}\033[0m inizializzato con \033[01i pesi ottimizzati\033[0m tramite hyper-parameter tuning su \033[1mWeight & Biases\033[0m")
                    except Exception as e:
                        print(f"⚠️Errore nel caricamento dei pesi per {model_name} su {key}: {e}")
                        continue


                # Definizione del criterio di perdita
                criterion = nn.CrossEntropyLoss(weight = class_weights_tensor)

                # Definizione dell'ottimizzatore con i parametri aggiornati
                optimizer = torch.optim.Adam(model.parameters(), lr = model_lr, weight_decay = model_weight_decay)

                print(f"🏋️‍♂️Avvio del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
                my_train_results = training(model, train_loader, val_loader, optimizer, criterion, n_epochs = model_n_epochs, patience = model_patience)

                print(f"Avvio del testing per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
                my_test_results = testing(my_train_results, test_loader, criterion)

                '''
                GRADCAM COMPUTATION PER IL MODELLO CNN2D

                La funzione compute_gradcam_figure estrae due campioni (uno per ogni classe) e crea una figura con le due righe richieste.

                Il parametro gradcam_image (un buffer binario o un'immagine) viene passato alla funzione di salvataggio, 
                'save_performance_results', in modo da essere salvato nella path corretta. 

                La funzione 'save_performance_results' è stata modificata 
                per gestire ANCHE questo nuovo input dell'immagine 

                (ossia, per salvare il file con un nome che inizia con 'GradCAM_results_'
                seguito da tutte le altre stringhe corrispondenti alla combinazione di fattori che costituiscono il dataset corrente:

                - coppia di condizioni sperimentali da cui provengono i dati (i.e., th_resp_vs_pt_resp )
                - tipologia di dato EEG prelevato (i.e., spectrograms) 
                - provenienza del dato stesso (i.e., familiar_th)
                )

                '''

                # Se il modello è CNN2D, calcola anche GradCAM per la visualizzazione
                gradcam_image = None

                if model_name == "CNN2D":
                    gradcam_image = compute_gradcam_figure(model, test_loader, exp_cond, data_type, category_subject, device)
                    if gradcam_image is not None:
                        print(f"GradCAM image computed successfully for {model_name}.")

                print(f"Salvataggio dei risultati per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
                save_performance_results(model_name,
                                         my_train_results,
                                         my_test_results,
                                         key,
                                         exp_cond,
                                         model_standardization,
                                         base_folder = save_path_folder,
                                         gradcam_image = gradcam_image)

##### **UTILS DATI NON HYPER**

In [ ]:
import pickle
import numpy as np


def load_data(data_type, category, subject_type, condition = "th_resp_vs_pt_resp"):
    """
    Carica i dati EEG dalla directory appropriata, già salvati con la finestra temporale (0°-640° punto)

    Parameters:
    - data_type: str, "spectrograms",
    - category: str, "task_1"
    - subject_type: str, "th" (terapisti)
    - condition: str, condizione sperimentale da selezionare
    

    Returns:
    - X: Dati EEG sotto-selezionati (50°-300° punto e canali selezionati se applicabile)
    - y: Etichette corrispondenti
    """

    # Definizione dei percorsi base
    base_paths = {
        "spectrograms": {
            "familiar": "/home/stefano/Interrogait/all_datas/Spectrograms_motor_tasks/"
            #"unfamiliar": "/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms/"
        },
    }

    # Seleziona il path corretto
    base_path = base_paths[data_type][category]

    # Determina il nome del file corretto
    if data_type in ["spectrograms"]:
        filename = f"new_all_{subject_type}_concat_spectrograms_coupled_exp_task_1.pkl"
    else:
        raise ValueError("data_type non valido!")
        
    # Caricamento del file
    filepath = base_path + filename
    
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    
    '''
    Per i dati spectrogram, la funzione seleziona la condizione desiderata (i.e., condition = "th_resp_vs_pt_resp") 
    e preleva i dati e le etichette associati a quella condizione.
    '''
    
    # Selezione della finestra temporale e delle etichette
    X = data[condition]["data"]
    y = data[condition]["labels"]

    
    return X, y


def select_channels(data, channels=[12, 30, 48]):
    """
    Seleziona i canali EEG specificati SOLO per i dati 1-20 e 1-45.

    Parameters:
    - data: array NumPy, dati EEG con shape (n_trials, n_channels, n_timepoints)
    - channels: list, indici dei canali da selezionare

    Returns:
    - data filtrato sui canali specificati
    """
    return data[:, channels, :]


# Funzione per train-test split
def split_data(X, y, test_size=0.2, val_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test


'''ATTENZIONE CAMBIATA'''
# Funzione per standardizzare i dati
def standardize_data(X_train, X_val, X_test, eps = 1e-8):
    
    mean = X_train.mean(axis=0, keepdims=True)
    std = X_train.std(axis=0, keepdims=True)
    
    X_train = (X_train - mean) / (std + eps)
    X_val = (X_val - mean) / (std + eps)
    X_test = (X_test - mean) / (std + eps)
    
    return X_train, X_val, X_test

# Import modelli (definisci le classi CNN1D, ReadMEndYou, ReadMYMind)
#from models import CNN1D, ReadMEndYou, ReadMYMind  # Assicurati di avere i modelli definiti in 'models.py'

# Funzione per inizializzare i modelli
def initialize_models():
    #model = CNN1D(input_channels=3, num_classes=2)
    #model_CNN = CNN2D(input_channels=3, num_classes=2)
    
    model_CNN = CNN2D(input_channels=64, num_classes=2)
    
    #model_LSTM = ReadMEndYou(input_size=3, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    #model_LSTM = ReadMEndYou(input_size=3 * 26, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    
    model_LSTM = ReadMEndYou(input_size=64 * 81, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    
    #model_Transformer = ReadMYMind(num_channels=3, seq_length=250, d_model=16, num_heads=4, num_layers=2, num_classes=2)
    #model_Transformer = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=3, freqs=26)
    
    model_Transformer = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=64, freqs=81)
    
    
    return model_CNN, model_LSTM, model_Transformer


import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight


'''
Questa funzione prende in input i dati di training, validation e test, 
il tipo di modello scelto e la dimensione del batch. Si occupa di:

Calcolare i pesi delle classi.
Convertire i dati in tensori PyTorch, con le opportune trasformazioni per CNN, LSTM o Transformer.
Creare i dataset e i dataloader per il training.
'''


def prepare_data_for_model(X_train, X_val, X_test, y_train, y_val, y_test, model_type, batch_size=48):
    
    # Calcolo dei pesi delle classi
    class_weights = compute_class_weight(class_weight='balanced', 
                                         classes=np.unique(y_train), 
                                         y=y_train)
    
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
    class_weights_tensor = class_weights_tensor.to(dtype=torch.float32, device=device)
    
    # Conversione delle etichette in interi
    y_train = y_train.astype(int)
    y_val = y_val.astype(int)
    y_test = y_test.astype(int)
    
    # Conversione dei dati in tensori PyTorch con permutazione se necessario
    #if model_type == "CNN2D":
    
    if model_type == 'CNN2D_LSTM_TF':
        
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #BiLSTM (ReadMEndYou):
    #Ora il modello si aspetta l’input con shape (batch, canali, frequenze, tempo) 
    #e, al suo interno, 
    #esegue la permutazione per avere il tempo come dimensione sequenziale. 
    #Non serve quindi applicare una permutazione anche qui.
    
    elif model_type == "BiLSTM":
            
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #Transformer (ReadMYMind):
    #Analogamente, il modello gestisce internamente la riorganizzazione dell’input, quindi lasciamo i dati nella loro forma originale.
    elif model_type == "Transformer":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    else:
        raise ValueError("Modello non riconosciuto. Scegli tra 'CNN', 'BiLSTM' o 'Transformer'.")
    
    # Conversione delle etichette in tensori
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    # Creazione dei dataset
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    # Creazione dei dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, class_weights_tensor




'''
OLD VERSIONS BEFORE GRADCAM COMPUTATION ON CNN2D

# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, model_standardization):
    
    #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
    if '_familiar_th' in key:
        return 'th_fam'
    elif '_unfamiliar_th' in key:
        return 'th_unfam'
    elif '_familiar_pt' in key:
        return 'pt_fam'
    elif '_unfamiliar_pt' in key:
        return 'pt_unfam'
    else:
        return None
    
     
# Funzione per salvare i risultati
def save_performance_results(model_name, my_train_results, my_test_results, key, exp_cond, model_standardization, base_folder):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, model_standardization)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "spectrograms" in key:
        data_type_str = "spectrograms"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    if model_standardization:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}_std.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
        
    else:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo 👍 in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"❌Errore durante il salvataggio dei risultati: {e}")
'''


#NEW VERSIONS FOR SPECTROGRAMS WITH GRADCAM COMPUTATION ON CNN2D!

# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, model_standardization):
    
    #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
    if '_familiar_th' in key:
        return 'th_fam'
    elif '_unfamiliar_th' in key:
        return 'th_unfam'
    elif '_familiar_pt' in key:
        return 'pt_fam'
    elif '_unfamiliar_pt' in key:
        return 'pt_unfam'
    else:
        return None


from PIL import Image
import io
import pickle
import os
     
# Funzione per salvare i risultati
def save_performance_results(model_name, 
                             my_train_results,
                             my_test_results, 
                             key,
                             exp_cond,
                             model_standardization,
                             base_folder,
                             gradcam_image = None):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    Se gradcam_image è fornita, la salva anche in formato PNG con un nome che inizia con 'GradCAM_results'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, model_standardization)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "spectrograms" in key:
        data_type_str = "spectrograms"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file pickle con l'inclusione della combinazione key + model_name
    if model_standardization:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}_std.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
    else:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\n🔬Risultati salvati con successo 👍 in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"❌Errore durante il salvataggio dei risultati: {e}")
    
    # Se è stata fornita l'immagine GradCAM, salvala come file PNG
    if gradcam_image is not None:
        if model_standardization:
            gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}_std.png"
        else:
            gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}.png"
        
        gradcam_file_path = os.path.join(folder_path, gradcam_file_name)
        
        #try:
        #    with open(gradcam_file_path, "wb") as f_img:
        #        f_img.write(gradcam_image)
        #    print(f"\n📸Immagine GradCAM salvata con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")
        
        try:
            
            '''
            Se gradcam_image è un oggetto BytesIO, allora rappresenta un flusso di dati binari in memoria.
            Quando si leggono dati da un BytesIO, il cursore interno avanza come in un file normale. 
            Se il cursore non è all'inizio, Image.open() potrebbe non leggere correttamente l'immagine.
            👉 seek(0) riporta il cursore all'inizio del buffer prima di leggerlo con Image.open()
            
            Per maggior info leggi cella successiva!
            '''
            
            # 🔄 Se gradcam_image è un buffer, convertirlo in immagine PIL
            if isinstance(gradcam_image, io.BytesIO):
                gradcam_image.seek(0)  # 🔄 Reset puntatore del buffer
                gradcam_image = Image.open(gradcam_image)
            
            '''
            Il messaggio di errore indica che il tuo oggetto gradcam_image è di tipo bytes e non ha il metodo save(), 
            che è tipico di un oggetto PIL. 
            
            Per risolvere questo, devi convertire i byte in un'immagine PIL. 
            Per farlo, controlla se gradcam_image sia un oggetto di tipo bytes e,
            in tal caso, usa io.BytesIO per creare un buffer da passare a Image.open(). 
            
            Inserisci questa conversione all'interno del blocco che salva l'immagine, così da assicurarti che,
            indipendentemente dal tipo, gradcam_image diventi un oggetto PIL e possa chiamare il metodo save().
            '''
            
            if isinstance(gradcam_image, bytes):
                gradcam_image = io.BytesIO(gradcam_image)
                gradcam_image.seek(0)
                gradcam_image = Image.open(gradcam_image)
            
            
            print(f"\n📸Immagine \033[1mGradCAM salvata\033[0m con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")
            # 🔄 Salvare l'immagine nel percorso specificato
            gradcam_image.save(gradcam_file_path, format = "PNG")
            
        except Exception as e:
            print(f"❌Errore durante il salvataggio dell'immagine GradCAM: {e}")

##### **NUOVE UTILS DATI NON HYPER POST W&B**

###### **IMPLEMENTAZIONE ADOTTATA**

In [ ]:
'''
Parsing della chiave e costruzione del path:
Usando la funzione parse_combination_key si estraggono 

exp_cond, data_type e category_subject dalla chiave del dataset. 

Questi vengono usati per costruire il percorso in cui cercare i file .pkl.
'''
import re 

# Funzione per parsare la chiave
def parse_combination_key(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    Il formato atteso è:
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ 
    "1_20|1_45|wavelet" _ 
    "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    """
    match = re.match(
        #r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$", 
        #r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_(spectrograms)_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        

'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

# Test
combination_key = "rest_vs_left_fist_spectrograms_familiar_th"
condition_experiment, data_type, subject_key = parse_combination_key(combination_key)

print("Condizione:", condition_experiment)
print("Data Type:", data_type)
print("Soggetto:", subject_key)

In [ ]:
'''
Verifica del file .pkl:
La funzione load_config_if_available cerca, per ogni modello, il file con nome del tipo
"{model_name}_{exp_cond}_{data_type}_{category_subject}.pkl"
all’interno della struttura di cartelle basata su base_path. 

Se il file esiste, allora viene passata poi a load_model_config_and_weights, 
che carica il dizionario di partenza 
e da questo estrae i 2 sotto-dizionari 'config' e 'state_dict'.
'''

def load_config_if_available(dataset_key, model_name, base_path):
    """
    Data una chiave (es. "th_resp_vs_pt_resp_wavelet_familiar_th") e il nome del modello,
    cerca il file .pkl corrispondente e ritorna (config, state_dict).
    Se non esiste, restituisce (None, None).
    """
    try:
        exp_cond, data_type, category_subject = parse_combination_key(dataset_key)
        config, state_dict = load_model_config_and_weights(exp_cond, data_type, category_subject, model_name, base_path)
        print(f"✅ File .pkl trovato per \033[1m{model_name}\033[0m su \033[1m{dataset_key}\033[0m")
        
        return config, state_dict
    except Exception as e:
        print(f"⚠️ Nessun file .pkl per {model_name} su {dataset_key} - uso parametri di default. ({e})")
        return None, None

In [ ]:
'''
Caricamento del file .pkl:
La funzione load_model_config_and_weights cerca, per ogni modello, il file con nome del tipo
"{model_name}_{exp_cond}_{data_type}_{category_subject}.pkl"
all’interno della struttura di cartelle basata su base_path. Se il file esiste, vengono restituiti config e state_dict.
'''

# Funzione per caricare il file .pkl con la configurazione e i pesi ottimali
def load_model_config_and_weights(exp_cond, data_type, category_subject, model_name, base_path):
    """
    Costruisce il path usando:
        base_path / exp_cond / data_type / category_subject
    e il nome del file:
        {model_name}_{exp_cond}_{data_type}_{category_subject}.pkl
    Se il file esiste, lo carica e restituisce (config, state_dict).
    """
    
    file_name = f"{model_name}_{exp_cond}_{data_type}_{category_subject}.pkl"
    file_path = os.path.join(base_path, exp_cond, data_type, category_subject, file_name)
    
    if os.path.exists(file_path):
        print(f"🕵️‍♂️🔍Caricamento file .pkl: \033[1m{file_path}\033[0m")
        
        # Il file .pkl è stato salvato con torch.save() e contiene un dizionario con chiavi al suo interno che sono: "config" e "state_dict"
        with open(file_path, "rb") as f:
            data = torch.load(f)
        return data["config"], data["state_dict"]
    else:
        raise FileNotFoundError(f"File {file_path} non trovato.")

##### **TRAINING (NON DEVI ESEGUIRLA VAI DIRETTAMENTE AL TESTING!)**

###### **VERSIONE POST- WEIGHT AND BIASES (W&B)**

In [ ]:
'''UFFICIALE - VERSIONE POST- WEIGHT AND BIASES SENZA COMMENTI'''


import io
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


class EarlyStopping:
    def __init__(self, patience = 10, min_delta = 0.001, mode = 'max'):
        
            
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.best_score = None # Tiene traccia del miglior punteggio osservato
        self.counter = 0 # Conta quante epoche consecutive non migliorano
        self.early_stop = False # Flag che indica se attivare l'early stopping
        
        #Ogni volta che si chiama la classe con early_stopping(current_score), controlla se il modello sta migliorando o meno.

    def __call__(self, current_score):
        
        #Caso 1: Prima iterazione (best_score ancora None)
        #→ Se non esiste ancora un miglior punteggio, lo inizializza con il primo valore ricevuto.
        
        if self.best_score is None:
            self.best_score = current_score
            
        #Caso 2: Il modello migliora
        #→ Se il valore migliora di almeno min_delta, aggiorna best_score e resetta il contatore.

        elif (self.mode == 'min' and current_score < self.best_score - self.min_delta) or \
             (self.mode == 'max' and current_score > self.best_score + self.min_delta):
            self.best_score = current_score
            self.counter = 0  # Reset contatore se migliora
            
        #Caso 3: Il modello NON migliora
        
        #→ Se il valore non migliora, incrementa il contatore.
        #→ Se il contatore raggiunge patience, imposta early_stop = True, segnalando che il training deve essere interrotto.
        
        else:
            self.counter += 1  # Incrementa se non migliora
            if self.counter >= self.patience:
                print(f"🛑 Early stopping attivato! Nessun miglioramento per {self.patience} epoche consecutive.")
                self.early_stop = True
                

def plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history):
    
    '''
    # Creazione di una figura con 2 subplot
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))  # 2 righe, 1 colonna, dimensione figura

    # Plot della loss
    ax[0].plot(loss_train_history, label='Train Loss', color='blue')
    ax[0].plot(loss_val_history, label='Validation Loss', color='orange')
    #ax[0].set_title(f'Loss during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[0].set_title(f'Loss during Training: ', fontsize=12)  # Titolo più grande
    ax[0].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[0].set_ylabel('Loss', fontsize=12)    # Dimensione font asse y
    ax[0].legend(fontsize=12)  # Dimensione font legenda
    ax[0].grid(True)

    # Plot dell'accuracy
    ax[1].plot(accuracy_train_history, label='Train Accuracy', color='blue')
    ax[1].plot(accuracy_val_history, label='Validation Accuracy', color='orange')
    #ax[1].set_title(f'Accuracy during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[1].set_title(f'Accuracy during Training: ', fontsize=12)  # Titolo più grande
    ax[1].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[1].set_ylabel('Accuracy', fontsize=12)  # Dimensione font asse y
    ax[1].legend(fontsize=12)  # Dimensione font legenda
    ax[1].grid(True)
    
    # Regolare la spaziatura tra i subplot
    #plt.tight_layout()  # Alternativa: fig.subplots_adjust(hspace=0.3)
    '''
    
    # Salvare il plot in un buffer di memoria
    buf = io.BytesIO()
    plt.savefig(buf, format='png')  # Salviamo il plot in formato PNG
    buf.seek(0)  # Torniamo all'inizio del buffer

    # Convertire il buffer in un'immagine PIL (opzionale, per visualizzarla)
    img = Image.open(buf)

    # Aggiungere i dati dell'immagine nel dizionario
    plot_image_data = buf.getvalue()  # Otteniamo i dati binari dell'immagine
    buf.close()

    # Ritorniamo i dati dell'immagine da salvare nel dizionario
    return plot_image_data



def training(model, dataset_train_loader, dataset_val_loader, optimizer, criterion, n_epochs = 100, patience = 12):
    
    # Sposta il modello sulla GPU (se disponibile)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    model.to(device)
    
    #Setta il modello in fase di training
    model.train()
    
    # Storico delle metriche per ogni epoca
    loss_train_history = []  # History of Training loss
    loss_val_history = []    # History of Validation loss
    accuracy_train_history = []  # History of Training Accuracy
    accuracy_val_history = []    # History of Validation Accuracy
    
    '''OLD VERSION'''
    #early_stopping = EarlyStopping(patience=patience, mode='max')
    
    '''NEW VERSION'''
    # -- Early stopping su val_loss --
    early_stopping = EarlyStopping(patience=patience, mode='min')
    
    # Liste per le metriche di valutazione (precision, recall, F1, AUC)
    precision_train_history = []
    recall_train_history = []
    f1_train_history = []
    auc_train_history = []
    
    #Questa sarebbe la migliore accuratezza ottenuta sul validation set
    #in base alla quale viene preso il modello migliore!
    
    max_val_acc = 0
    best_model = None
    
    best_epoch = 0  # Epoca con la migliore validazione
    
    best_metrics = {} # Dizionario con le metriche del migliore modello nel set di validazione
    
    # Variabili per memorizzare le etichette vere e predette per l'intero training
    y_true_train_list = []
    y_pred_train_list = []
    
    
    pbar = tqdm(range(n_epochs))

    for epoch in pbar:
        
        #Create a list for temporary monitoring of train loss and accuracy at each epoch
        train_loss_tmp = [] 
        correct_train = 0 
        
        
        #'''STARTING OF THE TRAINING PHASE'''
        
        #Iterating for every batch inside dataset_train_loader
        for x, y in dataset_train_loader:
            
            x, y = x.to(device), y.to(device)
            
            #Run forward pass through my network and get a prediction
            y_pred = model(x)
            
            train_loss = criterion(y_pred, y.view(-1))
            optimizer.zero_grad() #so essentially finding where gradients is 0
                                  #we're looking for minimum's there

            train_loss.backward() #performing the backprop step
            optimizer.step() #update the model's hyperparameters based off of the step
        
            train_loss_tmp.append(train_loss.item()) #append the loss at each epoch in the temporary train loss list inside each epoch
            
            # Calculate the Accuracy Score during the Training Phase
                
            #qui il "_,"
            _, predicted_train = torch.max(y_pred, 1)
            correct_train += (predicted_train == y).sum().item()
            
            # Aggiungere le etichette vere e quelle predette alla lista
            y_true_train_list.extend(y.cpu().numpy())
            y_pred_train_list.extend(predicted_train.cpu().numpy())
        
        # Save the results of training set for every epoch
        
        #i.e., append the results in the whole train loss history list outside the cycle of each epoch 
        loss_train_history.append(np.mean(train_loss_tmp))
        accuracy_train = correct_train / len(dataset_train_loader.dataset)
        accuracy_train_history.append(accuracy_train)
        
        # Calcolare precision, recall, F1-score e AUC durante il training
        precision_train = precision_score(y_true_train_list, y_pred_train_list, average='weighted')
        recall_train = recall_score(y_true_train_list, y_pred_train_list, average='weighted')
        f1_train = f1_score(y_true_train_list, y_pred_train_list, average='weighted')
        auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')
        
        precision_train_history.append(precision_train)
        recall_train_history.append(recall_train)
        f1_train_history.append(f1_train)
        auc_train_history.append(auc_train)
        
        # '''STARTING OF THE VALIDATION PHASE'''
        
        #Setta il modello in fase di validation
        #model.eval() 
        
        loss_tmp_val = []  #create a list for temporary val list at each epoch
        correct_val = 0
        
        y_true_list = []
        y_pred_list = []
        
        #Here we disable gradient computation for the validation phase!
        with torch.no_grad():
            
            for x, y in dataset_val_loader:
                
                x, y = x.to(device), y.to(device)
                
                #Run forward pass through my network and get a prediction
                y_pred = model(x)

                #Calculate Validation Loss

                #remember: since we use CrossEntropyLoss we DO NOT need
                #to do any ONE HOT ENCODING between y_pred and y_train 
                
                #loss = criterion(y_pred.to(device), y.view(-1).to(device))
                
                val_loss = criterion(y_pred, y.view(-1))

                #Perform Backpropagation

                #HOW TO ADJUST THE VALUES (weights and biases)?
                #well, at every step the gradients will accumulate with every backprop,
                #so to prevent 'compounding', we need to reset the stored gradient for each new epoch!

                loss_tmp_val.append(val_loss.item()) #append the loss at each epoch in the temporary val loss list inside each epoch 
                
                # Calculate the Accuracy Score during the Validation Phase
                _, predicted_val = torch.max(y_pred, 1)
                correct_val += (predicted_val == y).sum().item()
                
                # Aggiungi le etichette e le predizioni per la confusion matrix
                y_true_list.extend(y.cpu().numpy())
                y_pred_list.extend(predicted_val.cpu().numpy())

                
        # Save the results of validation set for every epoch
        
        #i.e., append the results in the whole train loss history list outside the cycle of each epoch 
        
        loss_val_history.append(np.mean(loss_tmp_val)) 
        accuracy_val = correct_val / len(dataset_val_loader.dataset)
        accuracy_val_history.append(accuracy_val)
        
        #L'early stopping deve essere basato sulla val accuracy,
        #ma quando il training si interrompe, 
        #dobbiamo salvare le migliori performance ottenute sul training in corrispondenza dell'epoca in cui
        #la val accuracy era massima
        
        # Controllo della miglior validazione
        if accuracy_val > max_val_acc:
            max_val_acc = accuracy_val
            best_epoch = epoch
            
            best_metrics = {
                "train_loss": [round(loss_train_history[best_epoch], 4)],
                "train_accuracy": [round(accuracy_train_history[best_epoch], 4)],
                "train_precision": [round(precision_train, 4)],
                "train_recall": [round(recall_train, 4)],
                "train_f1_score": [round(f1_train, 4)],
                "train_auc": [round(auc_train, 4)]
            }
            best_model = cp.deepcopy(model)  # Salvo il miglior modello

        # Controllo Early Stopping
        early_stopping(accuracy_val)
        if early_stopping.early_stop:
            print(f"⚠️ Early stopping attivato all'epoca \033[1m{epoch}\033[0m, recupero il modello dell'epoca \033[1m{best_epoch}\033[0m")
            break

        # Update of the progress bar
        pbar.set_description(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {loss_train_history[-1]:.4f}, Val Loss: {loss_val_history[-1]:.4f}, Train Acc: {accuracy_train:.4f}, Val Acc: {accuracy_val:.4f}")

        # Calculate the confusion matrix and the classification report after all epochs in the Validation Phase
        conf_matrix = confusion_matrix(y_true_list, y_pred_list)
        class_report = classification_report(y_true_list, y_pred_list)

    # Salvataggio della configurazione del modello e iper-parametri
    model_config = {
        "model_architecture": str(model),
        "batch_size_train": train_loader.batch_size,
        "batch_size_val": val_loader.batch_size,
        "batch_size_test": test_loader.batch_size,
        "n_epochs": n_epochs
    }

    # Dizionario degli iper-parametri
    hyperparams = {
    "optimizer": str(optimizer),
    "loss_function": str(criterion),
    "learning_rate": optimizer.param_groups[0]['lr'],
   }

    
    # Plot dei risultati
    #plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history, exp_cond_1, exp_cond_2)
    training_plot = plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history)

    
    # Restituire tutti i risultati in un dizionario
    train_results = {
        "training_performances": best_metrics,  # Aggiungi il dizionario delle performance
        "loss_train_history": loss_train_history,
        "loss_val_history": loss_val_history,
        "accuracy_train_history": accuracy_train_history,
        "accuracy_val_history": accuracy_val_history,
        "best_model": best_model,
        "confusion_matrix_val": conf_matrix,
        "classification_report": class_report,
        "model_configuration": model_config,
        "hyperparameters": hyperparams,
        "training_plot": training_plot  # Salviamo il buffer con il plot
    }

    return train_results


##### **TESTING**

In [ ]:
'''
TESTING FUNCTION: CORRETTA ANCHE PER IL GRAD-CAM

SUCCESSIVAMENTE, DENTRO AL FOR LOOP DEL TRAINING E TESTING, 
SI RICHIAMA LA FUNZIONE DIRETTAMENTE DI 

1) compute_gradcam_figure, LA QUALE AL SUO INTERNO PRESENTA GIÀ 
TUTTO QUELLO CHE SERVE PER CALCOLARE IL GRADCAM, DI MODO CHE VADA A 

Selezionare esempi rappresentativi per ciascuna classe.
Calcolare le mappe GradCAM e gli overlay.
Creare una figura con le heatmap e le sovrapposizioni, completa di titoli esplicativi.
Restituire un'immagine (buffer) pronta per essere salvata

SUCCESSIVAMENTE, QUINDI, IL PROCEDIMENTO DIVENTA COME SEGUE:

1) Si esegue il TESTING, per ottenere le metriche e salvare i risultati (senza GradCAM)

2) Nel loop principale di TRAINING & TESTING, se il modello è CNN2D, allora 

 - richiama la funzione 'compute_gradcam_figure', la quale va a
    - calcolare le mappe di attivazione e successivamente creo le immagini che gli ho chiesto
    - passa l'immagine ottenuta da GradCAM alla funzione 'save_performance_results', la quale va a 
        - salvare i risultati di test ottenuti dalla funzione di 'testing'
        - salvare l'immagine risultatante del GradCAM e la sovrapposizione del GradCAM sullo spettrogramma originale della classe risultante
        
        
Questo approccio garantisce chiarezza e separa la parte di performance (testing) dalla parte di explainability (GradCAM).


'''

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

import io
from PIL import Image

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


def testing(results, test_loader, criterion):
    
    # Recupera il miglior modello ottenuto durante la validazione
    model = results['best_model']
    model.to(device)
    
    model.eval()  # Imposta il modello in modalità valutazione

    y_true_list = []  # Lista per salvare le etichette reali
    y_pred_list = []  # Lista per salvare le previsioni del modello
    
    '''AGGIUNTA NUOVA PER CALCOLO AUC-ROC'''
    y_score_list = []   # <— Lista per salvare gli score per le probabilità della classe positiva (per auc-roc!)
    
    total_loss = 0
    correct = 0
    
    test_performances = {
        "test_loss": [],
        "test_accuracy": [],
        "test_precision": [],
        "test_recall": [],
        "test_f1_score": [],
        "test_auc": []
    }
    

    with torch.no_grad():
        
        pbar = tqdm(test_loader, desc="Testing")
        
        for inputs, labels in pbar:
            
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Ottenere le predizioni del modello
            outputs = model(inputs)
            
            '''AGGIUNTA NUOVA PER CALCOLO AUC-ROC'''
            # aggiungi queste due righe
            probs = torch.softmax(outputs, dim=1)
            y_score_list.extend(probs[:,1].cpu().numpy())

            # Calcolare la loss
            test_loss = criterion(outputs, labels)
            total_loss += test_loss.item()

            # Memorizzare predizioni ed etichette vere
            _, predicted = torch.max(outputs, 1)
            y_pred_list.extend(predicted.cpu().numpy())
            y_true_list.extend(labels.cpu().numpy())

            # Aggiornare il numero di predizioni corrette
            correct += (predicted == labels).sum().item()

            pbar.set_description(f"Loss: {test_loss.item():.4f}")

    # Calcolare l'accuratezza complessiva
    accuracy = correct / len(test_loader.dataset)
    
    
    # Calcolare precision, recall, F1-score, AUC durante il testing
    precision_test = precision_score(y_true_list, y_pred_list, average='weighted')
    recall_test = recall_score(y_true_list, y_pred_list, average='weighted')
    f1_test = f1_score(y_true_list, y_pred_list, average='weighted')
    
    '''OLD VERSION'''
    #auc_test = roc_auc_score(y_true_list, y_pred_list, average='weighted')  # Assicurati che il problema sia binario o multi-class
    
    '''AGGIUNTA NUOVA PER CALCOLO AUC-ROC
    
    In questo modo l’roc_auc_score calcola l’area sotto tutta la curva ROC (tutte le soglie), 
    invece di valutare un solo punto corrispondente alla soglia 0.5
    '''
    
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    auc_test = roc_auc_score(y_true_list, y_score_list)


    # Aggiungere questi valori nel dizionario delle performance (arrotondando a 4 decimali)
    test_performances["test_loss"].append(round(total_loss / len(test_loader), 4))  # Media della loss
    test_performances["test_accuracy"].append(round(accuracy, 4))
    test_performances["test_precision"].append(round(precision_test, 4))
    test_performances["test_recall"].append(round(recall_test, 4))
    test_performances["test_f1_score"].append(round(f1_test, 4))
    test_performances["test_auc"].append(round(auc_test, 4))
    
    # Creare la confusion matrix
    conf_matrix = confusion_matrix(y_true_list, y_pred_list)
    
    # Stampare classification report
    class_report = classification_report(y_true_list, y_pred_list)

    print(f"\nTest Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", class_report)

    # Visualizzare la confusion matrix
    #plt.figure(figsize=(8, 6))
    #sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    #plt.title("Confusion Matrix")
    #plt.xlabel("Predicted")
    #plt.ylabel("True")
    #plt.show()
    
    # Salviamo l'immagine della confusion matrix in un buffer
    #buf = io.BytesIO()
    #plt.savefig(buf, format='png')
    #buf.seek(0)
    #conf_matrix_image_data = buf.getvalue()
    #buf.close()
    
    
    # Salviamo l'immagine della confusion matrix in un buffer
    buf = io.BytesIO()
    plt.figure(figsize=(8, 6))  # Nuova figura per evitare sovrapposizioni
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.savefig(buf, format='png')  # Salva l'immagine nel buffer
    buf.seek(0)  # Torna all'inizio del buffer
    conf_matrix_image_data = buf.getvalue()  # Ottieni l'immagine in formato binario
    buf.close()  # Chiudi il buffer

    # Mostra la confusion matrix (opzionale)
    #plt.show()
    
    # Salvataggio della configurazione del modello e iper-parametri
    '''COMMENTATO'''
    #model_config = {
        #"model_architecture": str(model),
        #"batch_size_test": test_loader.batch_size,
    #}
    
    '''COMMENTATO'''
    # Dizionario degli iper-parametri
    #hyperparams = {
        #"optimizer": str(optimizer),
        #"loss_function": str(criterion),
        #"learning_rate": optimizer.param_groups[0]['lr'],
    #}

    
    '''COMMENTATO'''
    # Restituisci i risultati come dizionario
    #test_results = {
        #"test_performances": test_performances,  # Aggiungi il dizionario delle performance
        #"confusion_matrix": conf_matrix,
        #"classification_report": class_report,
        #"model_configuration": model_config,
        #"hyperparameters": hyperparams,  # Aggiunti i due nuovi dizionari
        #"confusion_matrix_image": conf_matrix_image_data,  # Aggiunta l'immagine della confusion matrix
    #}
    
    
    # Restituisci i risultati come dizionario
    test_results = {
        "test_performances": test_performances,  # Aggiungi il dizionario delle performance
        "confusion_matrix": conf_matrix,
        "classification_report": class_report,
        "confusion_matrix_image": conf_matrix_image_data,  # Aggiunta l'immagine della confusion matrix
    }   
        
    return test_results


##### **CREAZIONE CLASSE GRADCAM**

In [ ]:
##### **CREAZIONE CLASSE GRADCAM**

'''
Creazione della classe GradCAM

-----1. Costruttore (init)-----

Cosa fa:

Salva il modello e il layer target (ad esempio, l'ultimo strato convoluzionale) su cui calcolare le mappe di attivazione.

A) Inizializza due variabili, 

1) self.activations e 2) self.gradients, che verranno usate per memorizzare rispettivamente 
1) le attivazioni (feature maps) e 2) i gradienti di quel layer

B) Registra due hook sul target_layer:

1) Forward Hook: Quando il modello effettua la forward pass, viene eseguito save_activation per salvare le attivazioni
2) Backward Hook: Durante la backward pass, save_gradient viene chiamato per salvare i gradienti


-----2. Hook per Salvare Attivazioni e Gradienti-----

B) Save Activation

def save_activation(self, module, input, output):
    self.activations = output.detach()

Cosa fa:

Quando viene eseguita la forward pass sul target_layer, questo hook cattura l'output (le attivazioni) del layer.
Usa detach() per ottenere una copia dei dati senza il tracking dei gradienti, in modo da non interferire con la retropropagazione.

C) Save Gradient

def save_gradient(self, module, grad_input, grad_output):
    self.gradients = grad_output[0].detach()


Cosa fa:

Durante la backward pass, questo hook cattura i gradienti che fluiscono attraverso il target_layer.
grad_output è una tupla; solitamente il primo elemento contiene i gradienti utili. 

Anche qui si usa detach() per isolare i dati dai grafi di calcolo.


'''

import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = None
        self.gradients = None
        
        # Registra hook per catturare attivazioni e gradienti
        self.target_layer.register_forward_hook(self.save_activation)
        self.target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output.detach()

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()

In [ ]:
'''
Creazione della funzione per generare le immagini associate alla GradCAM compution

FINAL VERSION WITH ULTIMATED EDITING PHASES


Spiegazione:

1) Selezione dei Campioni:
La funzione itera sul test_loader e salva il primo campione trovato per ciascuna delle due classi (0 e 1).

2) Calcolo GradCAM per ciascun campione:

Per ogni campione, si abilita il gradiente e si esegue la forward pass.
Viene scelto il target (se non specificato, quello predetto) e si esegue la backward pass per calcolare i gradienti.

- I pesi vengono calcolati come la media dei gradienti lungo le dimensioni spaziali (dim=(2,3)) e usati per eseguire una somma pesata sulle attivazioni.
- La mappa risultante viene passata attraverso una ReLU, normalizzata e upsampled per avere la stessa dimensione dell’input.

Creazione degli Overlay:
Viene normalizzata l’immagine originale e viene applicata una heatmap (usando OpenCV), quindi l’overlay viene ottenuto con cv2.addWeighted.

Costruzione della Figura:
Viene creata una figura con due righe e due colonne:

- La prima riga mostra le heatmap per ciascuna classe.
- La seconda riga mostra le sovrapposizioni (overlay) tra heatmap e spettrogramma originale.

I titoli sono personalizzati in base a exp_cond, data_type e category_subject.

Questa struttura mantiene tutta la logica necessaria (incluso il calcolo dei pesi) e la rende simile alla versione precedente,
con la differenza che il calcolo della CAM viene eseguito per campioni rappresentativi di entrambe le classi. 



Ciao ho bisogno che in questo calcolo del gradcam, tu qui nella funzione di calcolo del gradcam...
tu mi presenti poi le immagini (sia gradcam che per l'overlay) non solo un esempio, ma il valore medio di attivazione per la relativa condizione sperimentale...
ossia, anziché prendere un solo esempio e presentare il gradcam e l'overlay del solo esempio della classe 0, dovresti mostrarmi il valore medio, 
e allo stesso modo della classe 1.

quindi sia nel gradcam che nell'overlay dovrei avere una rappresentazione media sia delle regioni delle feature maps per la relativa condizione sperimentale,
sia dell'overlay risultante....

Ti è chiaro che cosa voglio?



Esatto, ho capito. Invece di estrarre un singolo campione per ciascuna classe, la funzione dovrebbe:
Accumulare le mappe GradCAM e gli overlay per tutti i campioni appartenenti a ciascuna classe
Calcolare la media delle mappe GradCAM ottenute per ciascuna condizione sperimentale.
Calcolare la media degli overlay corrispondenti, in modo da ottenere una rappresentazione aggregata della risposta del modello.
Quindi, anziché mostrare l'attivazione per un solo esempio, la figura finale presenterà:
Una heatmap media della GradCAM per la classe 0 e per la classe 1.
Un overlay medio (CAM + spettrogramma) per la classe 0 e per la classe 1.



Puoi implementare la media delle mappe GradCAM e degli overlay per ogni classe modificando la funzione nel modo seguente. 
In sostanza, invece di fermarti al primo campione trovato per ciascuna classe, raccogli tutti (o un sottoinsieme rappresentativo) e poi calcola la media.
Ecco i passaggi principali con qualche suggerimento su dove inserire il nuovo codice:


1. Raccogliere le mappe e gli overlay per ogni classe
Dove modificare:

Dopo il passaggio 3 (calcolo della GradCAM per ciascun campione), invece di elaborare un solo esempio per ciascuna classe, 
crea due liste (una per la classe 0 e una per la classe 1) in cui accumulare le mappe CAM e gli overlay per ogni campione processato.

Cosa fare:

Invece di usare un dizionario samples che salva solo il primo esempio trovato, modifica il ciclo sul test_loader per salvare tutti gli esempi 
(o un numero sufficiente di campioni) per ciascuna classe.

Per ogni campione, calcola la GradCAM e l'overlay e aggiungili alle rispettive liste, per esempio:

    cams_list[cls].append(cam)
    overlays_list[cls].append(overlay)


2. Calcolare la media delle mappe
Dove modificare:

Dopo aver processato tutti i campioni (o quelli desiderati) per ciascuna classe, 
aggiungi un nuovo passaggio per calcolare la media lungo l'asse delle mappe accumulate.

Cosa fare:

Per ogni classe, converti la lista in un array NumPy e calcola la media. Ad esempio:


    mean_cam = np.mean(np.array(cams_list[cls]), axis=0)
    mean_overlay = np.mean(np.array(overlays_list[cls]), axis=0).astype(np.uint8)
    
Salva questi valori medi in due nuovi dizionari (ad es. mean_cams e mean_overlays).


3. Utilizzare le mappe medie per la visualizzazione
Dove modificare:

Nel passaggio 7, durante la creazione della figura finale, utilizza mean_cams e mean_overlays anziché le mappe di un singolo esempio.

Cosa fare:

Nel ciclo che disegna le immagini, usa ad esempio:

    cam_img = cv2.applyColorMap(np.uint8(255 * mean_cams[cls]), cv2.COLORMAP_INFERNO)

E per l'overlay:

    overlay_img = np.flipud(mean_overlays[cls])

In questo modo la figura finale rappresenterà la media delle attivazioni per ogni condizione sperimentale.

Considerazioni finali
Selezione del campione: Se il dataset è molto grande, potresti voler limitare il numero di campioni per evitare calcoli troppo onerosi.
Ad esempio, puoi considerare i primi N campioni per ogni classe oppure usare un campionamento casuale.

Prestazioni: Calcolare la media su molti campioni potrebbe essere intensivo, quindi valuta se sia il caso di eseguire questo calcolo
su un sottoinsieme rappresentativo.

Visualizzazione: Assicurati che la normalizzazione sia coerente per la media.
                A volte, potrebbe essere utile normalizzare ogni mappa individualmente prima di calcolare la media 
                oppure normalizzare l'immagine media risultante.

Questi passaggi ti permetteranno di ottenere una rappresentazione aggregata (media) sia della GradCAM che dell'overlay, 
fornendo una visione più robusta della risposta del modello per ogni condizione sperimentale.

---- ---- ---- ---- ---- ----  ---- ---- ----  ---- ---- ----  ---- ---- ---- ---- ---- ----  ---- ---- ---- ---- ---- ----  ---- ---- ---- ---- ---- ----  ---- ---- ---- 
Riepilogo delle parti modificate
Selezione dei campioni:

Sostituisci la logica che si ferma al primo campione per ogni classe con la raccolta di tutti i campioni (o un campione rappresentativo) 
per ciascuna classe, salvandoli in un dizionario di liste.

Calcolo della Grad-CAM e Overlay:

Per ogni campione in ciascuna classe, calcola la mappa CAM e l'overlay e aggiungili a liste (cams_list e overlays_list).

Calcolo della media:

Dopo il ciclo, calcola la media per ogni classe usando np.mean.

Visualizzazione:

Nel passaggio di creazione della figura, utilizza le mappe medie (mean_cams e mean_overlays) al posto dei singoli campioni.

Queste modifiche ti permetteranno di ottenere, per ciascuna classe (0 e 1), una rappresentazione media sia della mappa Grad-CAM che dell'overlay, come richiesto.
---- ---- ---- ---- ---- ----  ---- ---- ----  ---- ---- ----  ---- ---- ---- ---- ---- ----  ---- ---- ---- ---- ---- ----  ---- ---- ---- ---- ---- ----  ---- ---- ---- 


'''

import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io


#La funzione compute_gradcam_figure serve a calcolare e visualizzare 
#le mappe di attivazione Grad-CAM per un modello CNN2D, applicandole a spettrogrammi EEG. 

#In particolare, seleziona un campione per ciascuna classe (0 e 1), calcola la Grad-CAM e costruisce una figura con:

#Prima riga → Heatmap della Grad-CAM per entrambe le classi.
#Seconda riga → Heatmap sovrapposta allo spettrogramma originale.
#Questa visualizzazione aiuta a interpretare su quali parti dell'immagine il modello si sta concentrando per prendere decisioni.



#Questa funzione aiuta a visualizzare le regioni attivate dalla rete CNN su immagini di spettrogrammi EEG,
#evidenziando le aree più importanti per la classificazione.

#🔹 Esempio finale:
#La figura risultante avrà due righe:

#Heatmap puro della Grad-CAM.
#Heatmap sovrapposta allo spettrogramma EEG originale.

def compute_gradcam_figure(model, test_loader, exp_cond, data_type, category_subject, device):
    """
    Per il modello CNN2D, seleziona un campione per ciascuna classe (0 e 1),
    calcola la GradCAM e costruisce una figura con:
    
      - Riga 1: Heatmap per classe 0 e classe 1.
      - Riga 2: Sovrapposizione della heatmap sullo spettrogramma originale.
      
    I titoli e le etichette degli assi sono personalizzati:
    
    - L'asse x rappresenta il tempo (ms) e l'asse y le frequenze (Hz) (solo per la riga overlay)    
    - I titoli dei subplot usano i nomi delle condizioni estratte automaticamente da 'exp_cond'
        (assumendo che exp_cond sia del tipo "th_resp_vs_pt_resp"), data_type e category_subject
    
    Il calcolo della CAM include il passaggio:
       weights = torch.mean(gradients, dim=(2, 3), keepdim=True)
       cam = torch.sum(weights * activations, dim=1)
       cam = F.relu(cam)
    """
    
    #Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
    
    #Qui si definisce quale layer convoluzionale sarà usato per la Grad-CAM.
    #In questo caso, conv3 è il terzo layer convoluzionale del modello model.
    
    #Grad-CAM calcola la mappa di attivazione basandosi sulle feature generate da questo livello.
    
    #🔹 Esempio:Se model.conv3 è un layer convoluzionale con 128 feature map,
    #la Grad-CAM genererà una mappa di attivazione basata su queste 128 feature.)


    # -------------------------------
    # Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
    # -------------------------------
    
    # Imposta il layer target (ad esempio conv3) e crea un'istanza di GradCAM
    target_layer = model.conv3
    gradcam = GradCAM(model, target_layer)
    
    # Estrai i nomi delle condizioni separando exp_cond (es: "th_resp_vs_pt_resp")
    condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    
    
    #Passaggio 2: Selezione di un campione per ogni classe
    
    #Qui la funzione cerca almeno un campione per ciascuna delle due classi (0 e 1) nel test_loader.
    
    #🔹 Esempio pratico:
    #Se il batch contiene:
        
    #labels = [1, 0, 1, 0, 1]  
    #inputs.shape = (5, 1, 64, 64)  # 5 immagini 64x64 in scala di grigi
    
    #Il codice estrae:

    #samples[0] = inputs[1] (il primo esempio della classe 0)
    #samples[1] = inputs[0] (il primo esempio della classe 1)
    #Se il test_loader non contiene entrambe le classi, la funzione stampa un messaggio di errore e termina.
    
    # -------------------------------
    # Passaggio 2: Selezione dei campioni per ciascuna classe
    # -------------------------------
    
    '''SOLO UN ESEMPIO'''
    # Dizionari per salvare un campione per ciascuna classe
    #samples = {}      # Qui salveremo il sample input per ogni classe 
    #labels_found = {} # Per tracciare se abbiamo già trovato un esempio per ciascuna classe di etichette
    
    '''CON MEDIA'''
    
    #Ora che ogni classe ha una sua chiave nel dizionario samples, non c'è più bisogno di usare labels_found 
    #per verificare la presenza di entrambe le classi.
    #In precedenza, stavi iterando nel test_loader e verificando la presenza di almeno un esempio per entrambe le classi (0 e 1),
    #ma ora i dati vengono direttamente organizzati nel dizionario in base alla loro classe. Quindi, se la classe non esiste nel dataset,
    #semplicemente non avrà una chiave nel dizionario samples.
    #Il controllo finale if 0 not in samples or 1 not in samples: è ancora necessario per assicurarsi che entrambe le classi siano presenti.
    #Se manca una classe, possiamo ancora uscire con un messaggio di errore.
    
    # Dizionari per salvare tutti i campioni per ciascuna classe
    samples = {0: [], 1: []}

    # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe (0 e 1)
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            
            # Aggiungi il campione alla lista della classe corrispondente
            if label_int in samples:  # Assumendo solo classi 0 e 1
                samples[label_int].append(inputs[i].unsqueeze(0))
                
            '''SOLO UN ESEMPIO'''
            #if label_int not in labels_found:
            #    samples[label_int] = inputs[i].unsqueeze(0)  # Salva come tensore 4D
                
                
                #labels_found[label_int] = True
            #if 0 in labels_found and 1 in labels_found:
            #    break
        #if 0 in labels_found and 1 in labels_found:
        #    break

    # Se non troviamo entrambi gli esempi, esci con un messaggio
    #if 0 not in samples or 1 not in samples:
    #    print("Non sono stati trovati esempi per entrambe le classi nel test_loader.")
    #    return None
    
    
    
    
    
    #Passaggio 3: Calcolo della Grad-CAM
    
    # Qui il codice:

    #Passa l'input al modello per ottenere le predizioni.
    #Identifica la classe predetta (target_class).
    #Fa il backpropagation per calcolare i gradienti rispetto alla classe target.

    #🔹 Esempio pratico:
    #Se output = [0.3, 0.7], il modello predice la classe 1, quindi target_class = 1 e il backpropagation calcola il gradiente rispetto a questa classe.
    
    
    # -------------------------------
    # Passaggio 3: Calcolo della Grad-CAM per ciascun campione
    # -------------------------------
    
    '''SOLO UN ESEMPIO'''
    # Per ciascun campione, calcola la GradCAM
    #cams = {} # Qui salveremo la mappa CAM per ogni classe
    #overlays = {} # Qui salveremo l'overlay (CAM + spettrogramma)
    
    '''
    L'errore si verifica perché ora la variabile samples[cls] è una lista di tensori (cioè, più campioni) e non un singolo tensore. 
    Di conseguenza, cercando di eseguire samples[cls].requires_grad ottieni l'errore (dato che la lista non ha l'attributo requires_grad).
    Per risolvere il problema devi iterare sui singoli campioni all'interno della lista per ciascuna classe. Ad esempio, sostituisci questo blocco:
    
    In questo modo, per ogni classe iteri su ciascun campione, calcoli la Grad-CAM e l'overlay, e li accumuli nelle rispettive liste 
    (cams_list e overlays_list). Successivamente potrai calcolare la media per ciascuna classe e utilizzarla per la visualizzazione.
    Con questa modifica non otterrai più l'errore e la logica sarà coerente con l'obiettivo di aggregare i risultati su più campioni.
    '''
    
    '''CON MEDIA'''
    cams_list = {0: [], 1: []}
    overlays_list = {0: [], 1: []}
    
    for cls in [0, 1]:
        for sample_input in samples[cls]:
        
        #sample_input = samples[cls]
        
            sample_input.requires_grad = True  # Abilita il gradiente per il campione

            # Esegui forward pass per ottenere l'output del modello
            output = model(sample_input)

            # Se non viene specificata una classe target, seleziona quella predetta
            target_class = output.argmax(dim=1).item()

            # Azzeramento dei gradienti e backward pass per la classe target
            # Azzera i gradienti e fai backpropagation rispetto al punteggio della target_class
            model.zero_grad()
            target = output[0, target_class]
            target.backward()

            #Passaggio 4: Computazione della mappa Grad-CAM

            #Qui si calcola la mappa CAM:

            #I pesi Grad-CAM sono la media dei gradienti lungo height & width.
            #La mappa CAM è la somma pesata delle attivazioni del layer target.
            #Si applica ReLU per eliminare i valori negativi.

            #🔹 Esempio pratico:
            #Se abbiamo 128 feature map in conv3, il calcolo sarà:

            #weights = torch.mean(gradcam.gradients, dim=(2, 3), keepdim=True)  # (batch, 128, 1, 1)
            #cam = torch.sum(weights * gradcam.activations, dim=1)  # (batch, height, width)

            # -------------------------------
            # Passaggio 4: Computazione della mappa Grad-CAM
            # -------------------------------

            # Calcola i pesi: media dei gradienti lungo le dimensioni spaziali (height e width)
            weights = torch.mean(gradcam.gradients, dim=(2, 3), keepdim=True)

            # Calcola la mappa CAM: somma pesata delle attivazioni
            cam = torch.sum(weights * gradcam.activations, dim=1)

            # Calcola la CAM: applica ReLU per eliminare i valori negativi
            cam = F.relu(cam)

            #Passaggio 5: Normalizzazione e upsampling

            #La mappa CAM viene normalizzata tra 0 e 1.
            #Viene ridimensionata (upsampling) per adattarsi alla dimensione originale dell'immagine

            #🔹 Esempio pratico:
            #Se cam ha dimensione 16x16 e l'immagine originale è 64x64, viene interpolata per adattarsi.

            # -------------------------------
            # Passaggio 5: Normalizzazione e upsampling della CAM
            # ---------------------------

            # Normalizza la mappa
            cam = cam - cam.min()
            cam = cam / (cam.max() + 1e-8)

            # Upsample alla dimensione dell'immagine di input
            cam = F.interpolate(cam.unsqueeze(1), size=sample_input.shape[2:], mode='bilinear', align_corners=False)
            cam = cam.squeeze().cpu().numpy()

            '''SOLO UN ESEMPIO'''
            #cams[cls] = cam


            '''CON MEDIA'''
            # Aggiungi la mappa alla lista per la classe
            cams_list[cls].append(cam)


            #Passaggio 6: Creazione dell’overlay Grad-CAM

            #L'immagine originale viene convertita in un array numpy.
            #La mappa CAM viene colorata con COLORMAP_JET.
            #Si sovrappone l'heatmap all'immagine originale.

            #🔹 Esempio pratico:
            #Se il CAM ha valori alti in alcune regioni, il colormap evidenzierà in rosso le aree più attivate.

            # -------------------------------
            # Passaggio 6: Creazione dell'Overlay
            # -------------------------------

            # Converte l'immagine originale in numpy; considerando che l'input è (batch, canali, frequenze, tempo)
            # dopo squeeze si ottiene (canali, frequenze, tempo). Per visualizzare come immagine color, trasformiamo in (frequenze, tempo, canali).

            # Prepara l'immagine originale per la visualizzazione
            img = sample_input.squeeze().cpu().detach().numpy().transpose(1, 2, 0)

            # Normalizza l'immagine in scala 0-255
            img_norm = np.uint8(255 * (img - img.min()) / (img.max() - img.min()))

            # Applica la heatmap usando OpenCV
            #Per l'Overlay possiamo scegliere un colormap alternativo,
            # ad esempio COLORMAP_HOT o COLORMAP_INFERNO, per contrastare lo spettrogramma originale

            '''
            Il processo è lo stesso di quello descritto per le cam:

            I valori del CAM (normalizzati) vengono scalati a 255 e convertiti in un'immagine in scala di grigi.
            Il colormap INFERNO viene applicato per ottenere una rappresentazione colorata (dove i valori elevati diventano in genere rossi/gialli).
            La conversione BGR→RGB assicura una visualizzazione corretta
            '''

            heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_VIRIDIS)
            heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

            # Sovrapponi la heatmap all'immagine originale
            # Crea l'overlay: scegliendo pesi diversi per ottenere un contrasto chiaro


            '''
            Overlay troppo sfocato e colori discordanti
            Il problema che descrivi (overlay con toni azzurri/turchesi anziché il rosso della heatmap) può derivare da:

            Differenza di colormap e blending:
            L'overlay viene creato con una combinazione di due immagini: 
                1)lo spettrogramma originale (che potrebbe avere un proprio mapping di colori) e
                2) la heatmap

            Se il bilanciamento (i pesi) è 0.5-0.5, l'influenza dello spettrogramma può "modificare" i colori della heatmap.

            Suggerimenti:

            a) Modifica i pesi in cv2.addWeighted:
            Ad esempio, prova con 0.3 per l'immagine originale e 0.7 per la heatmap, in modo che il colore della heatmap (ad es. il rosso) prevalga.

            b) Uniforma il formato dell'immagine originale:
            Se lo spettrogramma originale è in scala di grigi o usa un colormap diverso,
            considera di convertirlo in un'immagine in scala di grigi a 8 bit prima di creare l'overlay.

            c) Usa lo stesso colormap: 
            Se vuoi che l'overlay abbia colori simili a quelli della heatmap, 
            usa lo stesso colormap (qui COLORMAP_INFERNO) per entrambe e regola il blending.

            '''
            overlay = cv2.addWeighted(img_norm, 0.4, heatmap, 0.6, 0)
            #overlay = cv2.addWeighted(img_norm, 0.5, heatmap, 0.5, 0)

            '''SOLO UN ESEMPIO'''
            #overlays[cls] = overlay

            '''CON MEDIA'''
            # Aggiungi l'overlay alla lista per la classe
            overlays_list[cls].append(overlay)
    
    
    mean_cams = {}
    mean_overlays = {}
    
    for cls in [0, 1]:
        mean_cams[cls] = np.mean(np.array(cams_list[cls]), axis=0)
        mean_overlays[cls] = np.mean(np.array(overlays_list[cls]), axis=0).astype(np.uint8)
        
    #Passaggio 7: Creazione della figura finale
    
    #La prima riga mostra solo le heatmap Grad-CAM.
    #La seconda riga mostra le heatmap sovrapposte agli spettrogrammi.

    # Crea la figura con due righe e due colonne

    # -------------------------------
    # Passaggio 7: Creazione della figura finale
    # -------------------------------
    # Creiamo una figura con 2 righe e 2 colonne:
    # - Prima riga: le heatmap CAM (da 0 a 1) per ciascuna condizione.
    # - Seconda riga: l'overlay (CAM + spettrogramma) per ciascuna condizione, con etichette per gli assi.
    
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    
    # Imposta un titolo generale per la figura
    
    #plt.suptitle(f"Grad-CAM Mapping - Experimental Condition: {exp_cond} - Subject: {category_subject}", fontsize=12)
    
    #plt.suptitle(f"Grad-CAM Mapping and Resulting Overlay over EEG trial Spectrogram\nExperimental Condition: {exp_cond} - Subject: {category_subject}",
    #fontsize=10,
    #y=0.95  # Puoi regolare la posizione verticale se necessario
    #)
    
    plt.suptitle(f"Grad-CAM Mapping and Resulting Overlay over EEG Trial Spectrogram\nExperimental Conditions: {exp_cond} - Subject: {category_subject}", fontsize=15)
    
    
    # Prima riga: Visualizza solo le heatmap (CAM)
    for j, cls in enumerate([0, 1]):
        
        # Qui usiamo il colormap INFERNO per la CAM, ma puoi modificare se preferisci
        
        '''
        np.uint8(255 * cams[cls]):
        La mappa CAM (calcolata e normalizzata) ha valori compresi tra 0 e 1.
        Moltiplicando per 255 e convertendo in uint8, ottieni un'immagine in scala di grigi a 8 bit (0-255).
        
        cv2.applyColorMap(..., cv2.COLORMAP_INFERNO):
        Applica il colormap INFERNO che trasforma la scala di grigi in un'immagine a colori, 
        dove i valori bassi saranno scuri e quelli alti appariranno in toni caldi (ad es. giallo/rosso).
        
        cv2.cvtColor(..., cv2.COLOR_BGR2RGB):
        OpenCV usa il formato BGR per impostazione predefinita. 
        Convertire in RGB assicura che l'immagine venga visualizzata correttamente (matplotlib si aspetta RGB).
        
        '''
        
        '''SOLO UN ESEMPIO'''
        #cam_img = cv2.applyColorMap(np.uint8(255 * cams[cls]), cv2.COLORMAP_INFERNO)
        
        '''CON MEDIA'''
        cam_img = cv2.applyColorMap(np.uint8(255 * mean_cams[cls]), cv2.COLORMAP_INFERNO)
        
        cam_img = cv2.cvtColor(cam_img, cv2.COLOR_BGR2RGB)
        
        '''QUI AGGIUNGIAMO L'INVERSIONE DEGLI ASSI'''
        # Se necessario, inverti gli assi per ottenere la visualizzazione desiderata
        cam_img = np.flipud(cam_img)  # Inverte verticalmente
        
        '''COMMENTATO PER L'OVERLAY SOLO RAPPRESENTARE L'ASSE DEL TEMPO IN FORMATO DI MILLISECONDI E NON DI FINESTRE STFT'''
        #axs[0, j].imshow(cam_img)
        
        # Se conosci i limiti temporali e di frequenza, puoi usare l'argomento extent
        axs[0, j].imshow(cam_img, extent=[0, 1000, 0, 26], aspect='auto')
        
        axs[0, j].set_title(f"Grad-CAM Mean Heatmap for Class {condition_names[cls]}", fontsize=12)
        axs[0, j].axis('off')
    
    # Seconda riga: Visualizza gli overlay con etichette degli assi
    for j, cls in enumerate([0, 1]):
        
        '''COMMENTATO PER L'OVERLAY SOLO RAPPRESENTARE L'ASSE DEL TEMPO IN FORMATO DI MILLISECONDI E NON DI FINESTRE STFT'''
        #axs[1, j].imshow(overlays[cls])
        
        # Qui, se vuoi che l'asse y (frequenze) venga ordinato in modo crescente,
        # puoi anche invertire l'immagine verticalmente, se non è già corretto.
        
        '''SOLO UN ESEMPIO'''
        #overlay_img = np.flipud(overlays[cls])
        
        '''CON MEDIA'''
        overlay_img = np.flipud(mean_overlays[cls])
        
        
        # Se conosci i limiti temporali e di frequenza, puoi usare l'argomento extent
        axs[1, j].imshow(overlay_img, extent=[0, 1000, 0, 26], aspect='auto')
        
        axs[1, j].set_title(f"Overlay of Grad-CAM Heatmap for Class {condition_names[cls]}", fontsize=12)
        axs[1, j].set_xlabel("Time (mms)", fontsize=10)
        axs[1, j].set_ylabel("Frequency (Hz)", fontsize=10)
        axs[1, j].axis('on')
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    
    #Passaggio 8: Salvataggio della figura
    #Qui la figura viene salvata in un buffer di memoria, pronto per essere salvato o inviato altrove
    
    # -------------------------------
    # Passaggio 8: Salvataggio della figura in un buffer
    # -------------

    # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    fig_image = buf.getvalue()
    buf.close()
    plt.close(fig)
    
    return fig_image

##### **FINAL IMPLEMENTATION OF GRADCAM COMPUTATION PER EEG STATS**

In [ ]:
'''GRADCAM ALGORITHM PER RAPPRESENTAZIONE EEG TEMPO x FREQUENZA NUOVA CON 

1) gestione del cudrnn per modelli con layer LSTM per far sì che il contesto imposta train() per sbloccare CuDNN-RNN, 
congela BN/Dropout in eval(), abilita i gradienti e ripristina tutto alla fine.

2) integri la gestione del test loader in formato raw per i plot sullo spettogramma




                                                VERSIONE NUOVA PER RAPPRESENTAZIONE TEMPO x FREQUENZA
                                                
                                                            (POST 27/06 ULTIMA VERSIONE DATATA)
                                                                        
                                                                        17/09/2025
                                                                    
                                                                
'''




import torch.nn as nn

def model_has_cudnn_rnn(model):
    """Ritorna True se il modello usa LSTM/GRU/RNN supportati da CuDNN."""
    return any(isinstance(m, (nn.LSTM, nn.GRU, nn.RNN)) for m in model.modules())
    

'''RICORDATI: aggiunto parametro TEST_LOADER_RAW per i plots della POTENZA SPETTRALE MEDIA PER BANDA (i.e., test_loader_raw)'''
def compute_gradcam_figure(model, test_loader, test_loader_raw, exp_cond, data_type, category_subject, device):
    
    
    '''SOLO PER I MODELLI OTTIMIZZATI CON ANCHE LA LSTM'''
    
    #Solo i modelli con LSTM entrano in questo giro; gli altri non cambiano di stato.
    #Con questa sequenza:
    #non ottieni più l’errore “cudnn RNN backward…”;
    #la rete “si comporta” come in eval (Dropout off, BN congelato) mentre calcoli le CAM;
    #l’ambiente di chiamata (il tuo loop di testing) riceve il modello esattamente nello stato in cui l’aveva passato alla funzione compute_gradcam_figure
    

    ### Perché serve model.train() anche se la CAM è presa prima della LSTM
    
    #Il backward, per arrivare dal loss (o dal logit scelto) fino al tuo layer conv3, deve comunque attraversare l’LSTM che sta più avanti nella rete.
    #Le implementazioni CuDNN degli RNN (LSTM/GRU) alzano un’eccezione se provi a chiamare tensor.backward() mentre il modulo è in modalità eval().
    #RuntimeError: cudnn RNN backward can only be called in training mode
    #Quindi, anche se la CAM è calcolata su conv3, devi mettere l’intero modello in train() per il tempo del backward.
    #condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    
    ### Che cos’è model.training
    
    #model.training è un semplice flag booleano (impostato da nn.Module.train() / nn.Module.eval()), ereditato da tutti i sotto‑moduli.
    #Con was_training = model.training ricordi in che stato era il modello (quasi sempre False, cioè eval, nel tuo flusso)
    #per poterlo ripristinare dopo.
    
    #Facendo così
    
    #for m in model.modules():
    #if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d,
                      #nn.Dropout, nn.Dropout2d, nn.Dropout3d)):
        #if m.training:         # cioè erano in train
            #m.eval()
            #frozen_layers.append(m)
    
    #Li sposti in eval uno per uno, senza toccare il resto della rete che deve restare in train() per far funzionare CuDNN‑RNN.
    
    
    ### Perché, a fine blocco, servono due ripristini
    
    #1) Riattivo i BatchNorm / Dropout che avevo forzato in eval:
    
    #for m in frozen_layers:
        #m.train()              # torna come prima
    
    #2) Riporto l’intero modello nello stato in cui si trovava prima del Grad‑CAM:
    
    #model.train(was_training)  # se era eval() torna eval, altrimenti resta train
    
    #Se non facessi il punto 1, lasceresti quei moduli permanentemente in eval anche quando, più tardi, 
    #rientri in training (per esempio in un fine‑tuning).
    #Se non facessi il punto 2, lasceresti tutto il modello in train → dropout attivo, BN che accumula statistiche, ecc.

    
    
    # ------------------------------------------------------------ ------------------------------------------------------------
    # ❶ — se serve, abilito temporaneamente la modalità train per il modello ottimizzato che aveva ANCHE la LSTM... 
    # ------------------------------------------------------------ ------------------------------------------------------------
    
    needs_train_mode = model_has_cudnn_rnn(model)
    
    if needs_train_mode:
        was_training = model.training      # salvo lo stato
        model.train()                      # abilito backward su CuDNN‑RNN
        
        # ➊ salvo lo stato di OGNI BN/Dropout
        
        saved = [(m, m.training) for m in model.modules()
             if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d,
                               nn.Dropout, nn.Dropout2d, nn.Dropout3d))]
        
        model.train()                              # abilita backward su CuDNN‑RNN
        
        # ➋ congelo in ogni layer della rete gli strati di BatchNorm e Dropout
        for m, _ in saved:
            m.eval()
    
    # ------------------------------------------------------------
    # ❷ — QUI sotto metti tutto il tuo codice Grad‑CAM
    #      (forward, backward, costruzione delle mappe, plot, …)
    # ------------------------------------------------------------

    # … il tuo lunghissimo corpo della funzione rimane invariato …
    # → al momento di fare backward NON avrà più l’eccezione
    #   “cudnn RNN backward can only be called in training mode”
    
    
    
    #Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
    
    #Qui si definisce quale layer convoluzionale sarà usato per la Grad-CAM.
    #In questo caso, conv3 è il terzo layer convoluzionale del modello model.
    
    #Grad-CAM calcola la mappa di attivazione basandosi sulle feature generate da questo livello.
    
    #🔹 Esempio:Se model.conv3 è un layer convoluzionale con 128 feature map,
    #la Grad-CAM genererà una mappa di attivazione basata su queste 128 feature.)


    # -------------------------------
    # Passaggio 1: Impostazione del layer target e istanziazione di GradCAM
    # -------------------------------
    
    target_layer = model.conv3
    gradcam = GradCAM(model, target_layer)
    
    # Determina il target layer in base al tipo di modello
    #if isinstance(model, SeparableCNN2D_LSTM_FC):
        #target_layer = model.dw_conv1  # Per il modello separabile 2D
    #else:
        #target_layer = model.conv3  # Per il modello CNN3D
        

    condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    
    #Passaggio 2: Selezione di un campione per ogni classe
    
    #Qui la funzione cerca almeno un campione per ciascuna delle due classi (0 e 1) nel test_loader.
    
    #🔹 Esempio pratico:
    #Se il batch contiene:
        
    #labels = [1, 0, 1, 0, 1]  
    #inputs.shape = (5, 1, 64, 64)  # 5 immagini 64x64 in scala di grigi
    
    #Il codice estrae:

    #samples[0] = inputs[1] (il primo esempio della classe 0)
    #samples[1] = inputs[0] (il primo esempio della classe 1)
    #Se il test_loader non contiene entrambe le classi, la funzione stampa un messaggio di errore e termina.
    
    # -------------------------------
    # Passaggio 2: Selezione dei campioni per ciascuna classe
    # -------------------------------
    
    
    # ✅ Raccogli TUTTI i campioni per ciascuna classe
    # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe (0 e 1)
    
    '''DATI ORIGINALI DEL TEST LOADER'''
    samples = {0: [], 1: []}
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples:  # Assumendo solo classi 0 e 1
                
                samples[label_int].append(inputs[i].unsqueeze(0))
                
    
    '''TEST_LOADER RAW (DATI NON STANDARDIZZATI)'''
    samples_raw = {0: [], 1: []}
    for inputs, labels in test_loader_raw:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples_raw:  # Assumendo solo classi 0 e 1
                
                samples_raw[label_int].append(inputs[i].unsqueeze(0))
                
    
    #Passaggio 3: Calcolo della Grad-CAM
    
    # Qui il codice:

    #Passa l'input al modello per ottenere le predizioni.
    #Identifica la classe predetta (target_class).
    #Fa il backpropagation per calcolare i gradienti rispetto alla classe target.

    #🔹 Esempio pratico:
    #Se output = [0.3, 0.7], il modello predice la classe 1, quindi target_class = 1 e il backpropagation calcola il gradiente rispetto a questa classe.
    
    
    # -------------------------------
    # Passaggio 3: Calcolo della Grad-CAM per ciascun campione
    # -------------------------------
    
    
    '''
    L'errore si verifica perché ora la variabile samples[cls] è una lista di tensori (cioè, più campioni) e non un singolo tensore. 
    Di conseguenza, cercando di eseguire samples[cls].requires_grad ottieni l'errore (dato che la lista non ha l'attributo requires_grad).
    Per risolvere il problema devi iterare sui singoli campioni all'interno della lista per ciascuna classe. Ad esempio, sostituisci questo blocco:
    
    In questo modo, per ogni classe iteri su ciascun campione, calcoli la Grad-CAM e l'overlay, e li accumuli nelle rispettive liste 
    (cams_list e overlays_list). Successivamente potrai calcolare la media per ciascuna classe e utilizzarla per la visualizzazione.
    Con questa modifica non otterrai più l'errore e la logica sarà coerente con l'obiettivo di aggregare i risultati su più campioni.
    '''
    
    '''CON MEDIA'''
    cams_list = {0: [], 1: []}
    overlays_list = {0: [], 1: []}
    
    for cls in [0, 1]:
        for sample_input in samples[cls]:
        
        #sample_input = samples[cls]
        
            sample_input.requires_grad = True  # Abilita il gradiente per il campione

            # Esegui forward pass per ottenere l'output del modello
            output = model(sample_input)

            # Se non viene specificata una classe target, seleziona quella predetta
            target_class = output.argmax(dim=1).item()

            # Azzeramento dei gradienti e backward pass per la classe target
            # Azzera i gradienti e fai backpropagation rispetto al punteggio della target_class
            model.zero_grad()
            target = output[0, target_class]
            target.backward()

            #Passaggio 4: Computazione della mappa Grad-CAM

            #Qui si calcola la mappa CAM:

            #I pesi Grad-CAM sono la media dei gradienti lungo height & width.
            #La mappa CAM è la somma pesata delle attivazioni del layer target.
            #Si applica ReLU per eliminare i valori negativi.

            #🔹 Esempio pratico:
            #Se abbiamo 128 feature map in conv3, il calcolo sarà:

            #weights = torch.mean(gradcam.gradients, dim=(2, 3), keepdim=True)  # (batch, 128, 1, 1)
            #cam = torch.sum(weights * gradcam.activations, dim=1)  # (batch, height, width)

            # -------------------------------
            # Passaggio 4: Computazione della mappa Grad-CAM
            # -------------------------------

            # Calcola i pesi: media dei gradienti lungo le dimensioni spaziali (height e width)
            weights = torch.mean(gradcam.gradients, dim=(2, 3), keepdim=True)

            # Calcola la mappa CAM: somma pesata delle attivazioni
            cam = torch.sum(weights * gradcam.activations, dim=1)

            # Calcola la CAM: applica ReLU per eliminare i valori negativi
            cam = F.relu(cam)
            
            
            '''
            
            TUTTO IL PASSAGGIO DELLO STEP 5 
            
            OSSIA NORMALIZZAZIONE i.e.,  NEL SENSO DI RISCALATURA NEL RANGE 0-1 + UPSAMPLING 
            
            (CHE SERVIVA PER UNIFORMARE I VALORI E ADATTARSI ALLA DIMENSIONE DELLA IMMAGINE ORIGINALE 
            PER VEDERE UN SOLO ESEMPIO DELLA CLASSE RISPETTO ALLA MAPPA DI ATTIVAZIONE E ALL'OVERLAY
            DEL GRADCAM RISPETTO ALLA IMMAGINE ORIGINALE)

            #🔹 Esempio pratico:
            #Se cam ha dimensione 16x16 e l'immagine originale è 64x64, viene interpolata per adattarsi.

            
            NON SERVE PIU', AD ECCEZIONE DI QUESTE RIGHE CHE ORA TI RIMETTO QUI SOTTO!'''
            
            '''
            
            
            ✅ Cosa fa correttamente questo codice:
            
            Estrae i campioni da test_loader separandoli in samples[0] e samples[1].
            
            Per ogni campione di ogni classe:
            
            Calcola la Grad-CAM raw (senza riscaling),
            La interpola per adattarla alla dimensione originale (n_freq, n_time)
            Applica ReLU per tenere solo le attivazioni positive (come da standard Grad-CAM)
            La converte in NumPy e la salva in cams_list[cls].
            
            Alla fine, fa la media delle CAM raw per ciascuna classe:
            
            mean_cams[cls] = np.mean(np.array(cams_list[cls]), axis=0)
            
            🔍 Stato attuale del dato
            
            cams_list[cls] → lista di array cam 2D non normalizzati, uno per ogni trial.
            mean_cams[cls] → array 2D (frequenza × tempo), media dei trial per ciascuna classe.
            
            La normalizzazione Z-score congiunta la farai dopo, sulla base di mean_cams.
            
            '''
            
            target_size = (sample_input.shape[2:]) # -> (n_freq, n_time)
            
            cam = F.interpolate(cam.unsqueeze(1), size = target_size, mode='bilinear', align_corners=False)
            
            # squeeze
            cam = cam.squeeze()                 # tensor 2D
            
            # Infine sposti su CPU e passi a numpy
            cam = cam.cpu().numpy()


            '''CON MEDIA'''
            # Aggiungi la mappa del singolo esempio alla lista per la classe (per poi dopo farci la media dentro mean_cams!)
            cams_list[cls].append(cam)
            
    
    # ============================================================
    # Calcolo dello heatmap media dei valori (raw) per ciascuna classe
    # ============================================================
    
    mean_cams = {}
    
    for cls in [0, 1]:
        mean_cams[cls] = np.mean(np.array(cams_list[cls]), axis=0)
    
    
    # ============================================================
    # Calcolo dello spettrogramma medio (raw) per ciascuna classe
    # ============================================================
    
    mean_raw_spectrograms = {}
    for cls in [0, 1]:
        if len(samples_raw[cls]) > 0:
            
            mean_raw_spectrograms[cls] = torch.cat(samples_raw[cls], dim=0).mean(dim =(0, 1)).detach().cpu().numpy()
        else:
            mean_raw_spectrograms[cls] = None
            
            #mean_raw_spectrograms[cls] = torch.cat(samples[cls], dim=0).mean(dim=0).squeeze().cpu().numpy()
            '''
            🔍 Perché dovrebbe funzionare?
            .detach(): Disattiva il tracciamento del gradiente (rende il tensore statico, senza dipendenze dalla computational graph di PyTorch).
            .cpu(): Porta il tensore sulla CPU (necessario per numpy()).
            .numpy(): Converte il tensore in un array NumPy.
            '''
            
            
            '''
            
            1) Calcolo della media degli spettrogrammi (rimozione dei canali)
            L'errore "Invalid shape (3, 26, 11)" con questa riga commentata sopra ☝️
            
            #mean_raw_spectrograms[cls] = torch.cat(samples[cls], dim=0).mean(dim=0).squeeze().detach().cpu().numpy()
             
            indica che l'array finale ha 3 canali in più (la prima dimensione) che non ti aspetti. 
            I tuoi dati originali hanno la forma:

            (trials, canali, frequenze, tempo)

            Se vuoi ottenere una rappresentazione media dello spettrogramma per tutti i trial di una classe, mediando anche sui canali, 
            allora devi calcolare la media lungo la dimensione dei trial e quella dei canali.
            
            Dovresti fare:
            
            mean_raw_spectrograms[cls] = torch.cat(samples[cls], dim=0).mean(dim=(0, 1)).detach().cpu().numpy()
            
            ************ ************ ************ ************ ************ ************ ************ ************ ************ ************
            SPIEGAZIONE:

            torch.cat(samples[cls], dim=0)
            => Concatena tutti i trial per quella classe lungo la dimensione 0, ottenendo un tensore con forma:
            (num_trials, canali, frequenze, tempo).

            .mean(dim=(0, 1))
            => Calcola la media prima lungo la dimensione dei trial (dim=0) e poi lungo quella dei canali (dim=1) in un'unica operazione, 
            ottenendo un tensore di forma (frequenze, tempo).

            .detach().cpu().numpy()
            => Rimuove il tracking del gradiente, sposta il tensore sulla CPU e lo converte in un array NumPy, pronto per imshow.

            Questo ti darà l'array 2D (frequenze × tempo) che imshow si aspetta.
            
            
            CHAGPT:
            
            Nel contesto del tuo esempio:

            La forma iniziale dei dati EEG in un formato tempo-frequenza era (batch, canali, frequenze, tempo), che è una matrice 4D. 
            
            Qui, hai un batch di dati, dove ogni dato ha la dimensione dei canali, frequenze, e tempo.

            Usando il codice:

                mean_raw_spectrograms[cls] = torch.cat(samples[cls], dim=0).mean(dim=(0,1)).detach().cpu().numpy()
                
            Stai concatenando lungo la dimensione del batch (dim=0), quindi ottieni una matrice che somma tutte le informazioni sul batch. 
            Successivamente, con .mean(dim=(0,1)) stai calcolando la media lungo le dimensioni dei canali (0) e del batch (1), 
            riducendo il risultato a una matrice 2D con la forma (frequenze, tempo), che è quella che desideri, ovvero 
            
            --> la media delle frequenze e del tempo su tutto il batch e i canali.

            Quindi sì, la forma risultante di mean_raw_spectrograms[cls] sarà una matrice 2D che rappresenta 
                1) le frequenze sulle righe e 
                2) il tempo sulle colonne
            ************ ************ ************ ************ ************ ************ ************ ************ ************ ************

            '''
    
    
    '''
    # =======================================================
    # Passaggio Finale: Creazione della figura finale
    # Ora la figura ha 3 righe:
    
    #  - Riga 1: Istogramma della distribuzione dei valori della heatmap media per ciascuna classe 
    #            normalizzata rispetto alla distribuzione congiunta!
    
    #  - Riga 2: GradCAM medio della distribuzione dei valori della heatmap media per ogni classe, 
    #            a seguito della normalizzazione rispetto alla distribuzione congiunta!
    
    #  - Riga 3: Spettrogramma medio (raw) rispetto ai Trial della Stessa Classe, su range logaritmico 
    # =======================================================
    
    
    Quando devo plottare l'istogramma dei valori di ogni heatmap media solamente (riga 3), 
    devo plottarli in base alla normalizzazione rispetto alla distribuzione congiunta.
    
    Quindi, devo plottarli in base al range minimo e massimo della intera distribuzione congiunta, quando è stata normalizzata!
    Di conseguenza devo fare
    
    1) Prendere la Media delle CAM per ogni classe (già fatto)
    2) Costruzione distribuzione congiunta raw
    3) Calcolo Media e Deviazione Standard della Distribuzione Congiunta
    4) Normalizzazione Z-score della Distribuzione Congiunta
    
    5) Prendo il range minimo e massimo della Distribuzione Congiunta Normalizzata
    
    Ossia, il range minimo e massimo su cui plottare entrambe le heatmap medie normalizzate in base alla distribuzione congiunta,
    dovrà essere rispetto alla distribuzione congiunta a seguito della normalizzazione.
    
    Quindi, dovrei ricreare un'altra variabile che contiene i valori normalizzati di entrambe le distribuzioni assieme,
    ossia una cosa del tipo
    
    normalized_all_vals = np.concatenate([normalized_mean_cams[0].flatten(), normalized_mean_cams[1].flatten()])
    
    e da questa prendere il minimo ed il massimo!
    
    
    '''
   

    # Creiamo una figura con 4 righe e 2 colonne
    #fig, axs = plt.subplots(3, 2, figsize=(12, 15))
    #plt.suptitle(f"Grad-CAM Mapping and Overlay over EEG Spectrogram\nExperimental Conditions: {exp_cond} - Subject: {category_subject}", fontsize=15)
    
    #fig, axs = plt.subplots(4, 2, figsize=(12, 20))
    #plt.suptitle(f"Grad-CAM Mapping and Resulting Overlay over EEG Trial Spectrogram\nExperimental Conditions: {exp_cond} - Subject: {category_subject}", fontsize=15)
    
    # Creiamo una figura con 3 righe e 2 colonne
    fig, axs = plt.subplots(3, 2, figsize=(12, 15))
    plt.suptitle(f"Grad-CAM Mapping over EEG Trials\nExperimental Conditions: {exp_cond}", fontsize=15)
    
    plt.tight_layout()  # Regola automaticamente la spaziatura globale
    plt.subplots_adjust(hspace = 0.5, wspace = 0.4)  # Fine tuning della spaziatura tra subplot
    

    
        
    # PLOT RIGA 1: Visualizzazione degli istogrammi della distribuzione dei valori delle heatmap medie RAW
    # RISPETTO ALLA DISTRIBUZIONE CONGIUNTA
        
    '''
    Questi valori rappresentano la distribuzione delle attivazioni di entrambe le classi, 
    rispetto alla DISTRIBUZIONE CONGIUNTA
    
    Quindi si tratta di valori di attivazione della mappa Grad-CAM media rispetto alla distribuzione congiunta!

    Per chiarire meglio il processo:

        1) Valori di attivazione: Quando si calcola la Grad-CAM, ottieni una mappa di attivazione per ciascun pixel. 
                               Questa mappa mostra quanto ciascun pixel contribuisce alla decisione del modello.
                               Questi valori di attivazione sono pesati in base ai gradienti della classe di interesse.

        2) Mediati per classe: Nel tuo caso, stai calcolando la media di queste attivazioni per ogni classe (ad esempio, classe 0 e classe 1). 
                            Questo processo permette di ottenere una rappresentazione complessiva di come la rete percepisce l'importanza di ogni pixel 
                            rispetto alla classe.

        3) Istogramma dei valori raw medi di ogni classe (su distribuzione congiunta!): Stai visualizzando un istogramma di questi valori medi, 
                                                           sulla distribuzione congiunta, ossia
                                                           
                                                           - prendo i valori (raw)delle heatmap media di entrambe le classi
                                                           - calcolare la distribuzione congiunta dei valori (all_vals = ...)
                                                           - ottengo quindi la nuova distribuzione congiunta dalle heatmap medie di entrambe le classi
                                                          
                                                           - calcolo minimo e massimo a seguito della normalizzazione (?) e non prima
                                                           - faccio i plot di entrambe delle heatmap medie raw,
                                                             ma rispetto a distribuzione congiunta
                                                             
                                                           
                                                           Questo darebbe una visione della distribuzione delle attivazioni,
                                                           per capire come i valori siano distribuiti tra le 2 classi (che ora son confrontabili!)
                                                           a livello RAW!
                                                        
    
    N.B. PER IL NOME DEL TITOLO DEL PLOT
    
    Perché "Grad-CAM value" può creare confusione:
    Il termine "Grad-CAM value" potrebbe sembrare che faccia riferimento direttamente ai valori generati dalla mappa Grad-CAM finale. 
    Ma in realtà, i valori che stai trattando sono le attivazioni mediate e clippate, che formano la heatmap. 
    L'istogramma che stai tracciando rappresenta la distribuzione delle attivazioni prima della normalizzazione.

    Riepilogo
    Quindi, questi valori sono attivazioni pesate per ciascun pixel della mappa Grad-CAM, e mediati per classe. 
    Il processo di normalizzazione che segue (basato sui percentili) serve a enfatizzare il contrasto in modo da focalizzarsi sulle aree più significative 
    per la previsione.

    Per rispondere alla tua domanda: sì, è corretto dire che stai visualizzando la distribuzione delle attivazioni pesate prima della normalizzazione
    per migliorare il contrasto, ma è meglio riferirsi a questi valori come valori di attivazione della mappa Grad-CAM o valori della heatmap Grad-CAM, 
    piuttosto che "Grad-CAM value" che potrebbe risultare ambiguo.

    Se vuoi, puoi anche aggiungere una nota nella visualizzazione dell'istogramma che chiarisca il processo:
    
    axs[2, j].set_title(f"Histogram of Heatmap Activation Values (Raw, before Normalization) Class {condition_names[cls]}", fontsize=12)
    oppure
    axs[2, j].set_title(f"Histogram of Mean Heatmap Activation Values - Class {condition_names[cls]}", fontsize=12)
    
    è molto chiara e corretta!
    Indica perfettamente che stai visualizzando l'istogramma dei valori di attivazione medi della heatmap, 
    senza fare confusione sul fatto che si tratti di valori medi per ciascuna classe.
    
    In sintesi, questa frase comunica in modo preciso che stai mostrando la distribuzione delle attivazioni mediate dalla mappa Grad-CAM
    per una specifica classe. Quindi sì, va benissimo!
    
    '''
    
    #PER PLOT RIGA 1 
    
    # Creo la distribuzione congiunta dei valori di ogni heatmap media RAW delle due classi, srotolando i valori di entrambe
    
    '''PRE 45 Hz'''
    #all_vals_raw = np.concatenate([mean_cams[0].flatten(), mean_cams[1].flatten()])
    
    '''MODIFICA PER FREQUENZA FINO A 45Hz'''
    all_vals_raw = np.concatenate([mean_cams[0][:46, :].flatten(), mean_cams[1][:46, :].flatten()])
    
    # Il range minimo e massimo su cui plottare entrambe le heatmap medie raw in base alla distribuzione congiunta (riga 3)
    # dovrà essere rispetto alla distribuzione congiunta raw
    
    vmin_raw = all_vals_raw.min()
    vmax_raw = all_vals_raw.max()
    
    
    # Prima riga: Visualizza l'istogramma della heatmap media rispetto alla distribuzione congiunta!
    for j, cls in enumerate([0, 1]):
        
        # Calcola l'istogramma dei valori della heatmap media (prima della normalizzazione robusta)
        
        '''PRE 45 Hz'''
        #axs[0, j].hist(mean_cams[cls].flatten(), bins= 'auto', color='blue', edgecolor='black')
        
        '''MODIFICA PER FREQUENZA FINO A 45Hz'''
        axs[0, j].hist(mean_cams[cls][:46, :].flatten(), bins= 'auto', color='blue', edgecolor='black')
        
        
        #axs[0, j].set_title(f"Histogram of Mean Grad-CAM values (Raw) - Class {condition_names[cls]}", fontsize=12)
        axs[0, j].set_title(f"Histogram of Mean Heatmap Activation Values (Raw) - Class {condition_names[cls]}", fontsize=12)
        axs[0, j].set_xlabel("Grad-CAM value", fontsize=10)
        axs[0, j].set_ylabel("Frequency", fontsize=10)
        
        
    
    # PLOT RIGA 2: Visualizzazione dei valori delle heatmap medie delle due classi
    # RISPETTO ALLA DISTRIBUZIONE CONGIUNTA, SU CUI VIENE FATTA LA NORMALIZZAZIONE
    
    
    '''
    Questi valori rappresentano le heatmap medie delle attivazioni di entrambe le classi, 
    rispetto alla DISTRIBUZIONE CONGIUNTA, SU CUI VIENE FATTA LA NORMALIZZAZIONE
    
    Quindi si tratta di valori di attivazione della mappa Grad-CAM media rispetto alla distribuzione congiunta NORMALIZZATA

    Per chiarire meglio il processo:

        1) Valori di attivazione: Quando si calcola la Grad-CAM, ottieni una mappa di attivazione per ciascun pixel. 
                               Questa mappa mostra quanto ciascun pixel contribuisce alla decisione del modello.
                               Questi valori di attivazione sono pesati in base ai gradienti della classe di interesse.

        2) Mediati per classe: Nel tuo caso, stai calcolando la media di queste attivazioni per ogni classe (ad esempio, classe 0 e classe 1). 
                            Questo processo permette di ottenere una rappresentazione complessiva di come la rete percepisce l'importanza di ogni pixel 
                            rispetto alla classe.

        3) Calcolo la distribuzione congiunta dei valori raw medi di ogni classe (su distribuzione congiunta!): 
        Stai visualizzando un istogramma di questi valori medi, sulla DISTRIBUZIONE CONGIUNTA, ossia
                                                           
                                                           - prendo i valori (raw)delle heatmap media di entrambe le classi
                                                           - calcolare la distribuzione congiunta dei valori (all_vals = ...)
                                                           - ottengo quindi la nuova distribuzione congiunta dalle heatmap medie di entrambe le classi
                                                           
                                                           - calcolo media e deviazione standard delle distribuzione congiunta
                                                           - faccio la normalizzazione della distribuzione congiunta
                                                           
                                                           - calcolo minimo e massimo a seguito della normalizzazione e non prima
                                                             della distribuzione congiunta normalizzata
                                                           
                                                           - faccio i plot di entrambe delle heatmap medie normalizzate,
                                                             ma rispetto alla distribuzione congiunta
                                                             
                                                           
                                                           Questo darebbe una visione della distribuzione delle attivazioni,
                                                           per capire come i valori siano distribuiti tra le 2 classi (che ora son confrontabili!)
                                                           a livello NORMALIZZATO!
                                                        
    '''
    
    '''SOPRA ABBIAMO CREATO --> all_vals_raw'''
    
    # Creo la distribuzione congiunta dei valori di ogni heatmap media RAW delle due classi, srotolando i valori di entrambe
    #all_vals_raw = np.concatenate([mean_cams[0].flatten(), mean_cams[1].flatten()])
    
    #Calcolo media e deviazione standard della distribuzione congiunta dei valori (raw) delle heatmap medie di entrambe le classi 
    #joint_mean = np.mean(all_vals_raw)
    #joint_std = np.std(all_vals_raw)
    
    # Normalizzazione Z-score della distribuzione congiunta
    #normalized_mean_cams = {}
    
    #for cls in [0, 1]:
        #normalized_mean_cams[cls] = (mean_cams[cls] - joint_mean) / joint_std

    # Il range minimo e massimo su cui plottare entrambe le heatmap medie normalizzate in base alla distribuzione congiunta (riga 3)
    # dovrà essere rispetto alla distribuzione congiunta a seguito della normalizzazione
    
    #normalized_all_vals = np.concatenate([normalized_mean_cams[0].flatten(), normalized_mean_cams[1].flatten()])
    
    #vmin_normalized = normalized_all_vals.min()
    #vmax_normalized = normalized_all_vals.max()
    
    vmin_normalized = all_vals_raw.min()
    vmax_normalized = all_vals_raw.max()

    
    '''
    # Opzione: normalizzazione robusta con percentili
    vmin_normalized, vmax_normalized = np.percentile(all_vals_raw, [5, 95])
    '''
    
    # Seconda riga: Mean heatmap di ogni classe normalizzata a partire dalla distribuzione congiunta ( = di entrambe le classi)
    for j, cls in enumerate([0, 1]):
        
        
        im = axs[1, j].imshow(
            mean_cams[cls],
            #normalized_mean_cams[cls], #QUI LA RENDO IN 2D, NON IN 1D COME PRIMA
            #cmap='seismic',
            cmap='RdYlBu_r',
            vmin= vmin_normalized, vmax= vmax_normalized,
            extent=[0, 4000, 0, 81],
            aspect='auto',
            origin='lower'
        )
        
        
        # → calcola 6 tick equi-spaziati
        ticks = np.linspace(vmin_normalized, vmax_normalized, 6)  
        
        cbar = fig.colorbar(
            im,
            ax=axs[1, j],
            orientation='horizontal',
            pad=0.12,
            ticks=ticks)
        
        cbar.ax.set_xticklabels([f"{t:.4f}" for t in ticks])

        
        axs[1, j].set_title(f"Mean Grad-CAM Heatmap (Raw) - Class {condition_names[cls]}", fontsize=12)
        
        '''QUESTA NON CONSENTE DEFINIZIONE ASSI!'''
        #axs[1, j].axis('off')
        
        axs[1, j].axis('on') 
        axs[1,j].set_xlabel("Time (mms)")
        axs[1,j].set_ylabel("Frequency (Hz)")
        
        
        '''MODIFICA PER FREQUENZA FINO A 45Hz'''
        axs[1, j].set_ylim(0, 45)
        
        #fig.colorbar(im, ax=axs[3, j], orientation='horizontal', pad=0.05)
    
    print(f"\033[1mRange heatmap raw globale (vmin_raw, vmax_raw): {vmin_normalized}, {vmax_normalized}\033[0m")
    
    # PLOT RIGA 3: Spettrogramma medio (raw) per ciascuna classe log-scaled
    
    '''
    Spiegazione delle modifiche aggiunte:

    1) Calcolo dello spettrogramma medio raw:

    Dopo aver raccolto i campioni nel dizionario samples, viene creato il dizionario mean_raw_spectrograms.
    Per ogni classe, i tensori vengono concatenati lungo la dimensione batch e si calcola la media sul batch (dim=0).
    
    Poi, però, ogni spettogramma medio deve congiunto in una distribuzione in modo da plottare poi il valore dello spettrogramma  
    rispetto al minimo ed al massimo della distribuzione congiunta dello spettrogramma medio di entrambe le classi! 
    
    Il risultato viene convertito in un array NumPy per il plotting.
    '''
    
    # Calcolo della distribuzione congiunta degli spettrogrammi medi delle due classi! 
    #all_vals_raw_samples = np.concatenate([mean_raw_spectrograms[0].flatten(), mean_raw_spectrograms[1].flatten()])
    
    '''SE VOLESSI RESTRINGERE TRA 5° e 95° PERCENTILE'''
    #low_raw, high_raw = np.percentile(all_vals_raw, [5, 95])
    #half_width_raw = max(abs(low_raw), abs(high_raw))   
    #vmin_raw, vmax_raw = -half_width_raw, +half_width_raw
    
    '''ALTRIMENTI, TENGO TUTTO IL RANGE, DAL MINIMO AL MASSIMO'''
    
    #Ora qui prendo il miimo e massimo a partire dalla distribuzione congiunta!
    #vmin_raw_samples, vmax_raw_samples = all_vals_raw_samples.min(), all_vals_raw_samples.max()
    
    '''
    
    1) Qual è la differenza tra prima e ora?
    
    Prima calcolavo, dentro il for cls in [0,1], un nuovo vmin_raw_samples e vmax_raw_samples separatamente per ciascuna classe.
    Di conseguenza ogni subplot sulla riga 3 aveva la sua scala di colori, rendendo impossibile un confronto diretto visivo 
    fra le due condizioni.
    
    Ora invece calcolerai una sola volta il log-power medio di entrambe le classi, ne ricavi un unico array congiunto,
    quindi ne estrai un solo vmin e vmax. Questo ti garantisce che entrambi i subplot della riga 3 useranno la stessa scala di colori.


    Per far sì che tutte e due le condizioni usino lo stesso minimo e massimo, sposto la raccolta dei limiti fuori dal ciclo,
    usando la distribuzione congiunta dei log-power di entrambe le classi
    
    vmin_raw_samples e vmax_raw_samples li calcoli una volta sola, su tutti i valori logaritmici concatenati.
    Entrambe le mappe usano esattamente lo stesso range, così le barre dei colori saranno allineate.
    
    Con questa modifica:

    log_mean_power contiene già i valori in scala logaritmica.
    vmin_raw_samples e vmax_raw_samples sono condivisi fra entrambe le colonne.
    Ogni subplot userà la stessa “barretta” di colore, quindi potrai confrontare direttamente “deep blues” e “reds” delle due condizioni.
    '''
    
    # 1. Calcola i log-power medi per ciascuna classe
    log_mean_power = {
        cls: np.log1p(mean_raw_spectrograms[cls])
        for cls in [0,1]
    }

    # 2. Raccogli TUTTI i valori in un unico array
    
    '''PRE 45 Hz'''
    #all_log_vals = np.concatenate([
    #    log_mean_power[0].flatten(),
    #    log_mean_power[1].flatten()
    #])
    
    
    '''MODIFICA PER FREQUENZA FINO A 45Hz'''
    
    all_log_vals = np.concatenate([
        log_mean_power[0][:46, :].flatten(),
        log_mean_power[1][:46, :].flatten()
    ])
    

    # 3. Estrai un unico vmin/vmax condiviso
    vmin_raw_samples = all_log_vals.min()
    vmax_raw_samples = all_log_vals.max()

        
    # Se conosci i limiti temporali e di frequenza, puoi usare l'argomento extent
    for j, cls in enumerate([0, 1]):
        
        #if mean_raw_spectrograms[cls] is not None:
        if log_mean_power[cls] is not None:    
            
            #Trasformo in scala logaritmica i miei dati EEG sulla spettro medio di ogni classe
            #mean_raw_spectrograms[cls] = np.log1p(mean_raw_spectrograms[cls])
        
            
            # Calcolo della distribuzione congiunta degli spettrogrammi medi delle due classi! 
            #all_vals_raw_samples = np.concatenate([mean_raw_spectrograms[0].flatten(), mean_raw_spectrograms[1].flatten()])
            
            #Ora qui prendo il miimo e massimo a partire dalla distribuzione congiunta!
            #vmin_raw_samples, vmax_raw_samples = all_vals_raw_samples.min(), all_vals_raw_samples.max()
    
            im = axs[2, j].imshow(log_mean_power[cls], 
                                  #mean_raw_spectrograms[cls], 
                                  extent=[0, 4000, 0, 81],
                                  aspect='auto', 
                                  cmap='jet', 
                                  vmin = vmin_raw_samples, vmax = vmax_raw_samples,
                                  origin='lower')
            
            axs[2, j].set_title(f"Log-Scaled Mean Raw Spectrogram - Class {condition_names[cls]}", fontsize=12)
            axs[2, j].set_xlabel("Time (mms)", fontsize=10)
            axs[2, j].set_ylabel("Frequency (Hz)", fontsize=10)
            
        
            '''MODIFICA PER FREQUENZA FINO A 45Hz'''
            axs[2, j].set_ylim(0, 45)
            
            '''
            
            ATTENZIONE QUI CHE C'ERA UN GRAVE ERRORE
            
            --> fig.colorbar(im, ax=axs[3, j]) 
            
            #Qui la Color Bar Verticale sarebbe 
            #scala dello spettrogramma raw, finita per sbaglio sul Δ-GradCAM perché hai scritto ax=axs[3,j] invece di ax=axs[4,j].
            
            
            La barra VERTICALE (CHE DOVEVA STAR NELLA 5° RIGA!!!!) della color bar accanto alla heatmap ti sta mostrando
            
            i VALORI ASSOLUTI della Grad-CAM (nel tuo caso non normalizzati, quindi scala di milioni --> variabile hist_data
            ossia l'istogramma dei valori della heatmap media (prima della normalizzazione robusta)
            '''
            fig.colorbar(im, ax=axs[2, j])
            
            axs[2, j].axis('on')
        else:
            axs[2, j].axis("off")
            
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    
    # ------------------------------------------------------------ ------------------------------------------------------------
    # ❸ — Ripristino allo stato precedente il modello ottimizzato trovato migliore, che aveva incluso anche layer LSTM
    # ------------------------------------------------------------ ------------------------------------------------------------
    
    if needs_train_mode:
        # ➌ ripristino layer singoli (i.e., riporto BN/Dropout dove stavano in eval mode)
        for m, old_flag in saved:
            m.train(old_flag)
        # ➍ ripristino lo stato globale del modello (di nuovo ad .eval())
        # i.e.,  come era stato passato in input alla funzione compute_gradcam_figure a partire 'load_best_run_results'!
        
        #Così simuli l’eval (Dropout off, BN congelato) pur essendo in train() per soddisfare CuDNN‑RNN.
        model.train(was_training)
        
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    
    
    #Passaggio 8: Salvataggio della figura
    #Qui la figura viene salvata in un buffer di memoria, pronto per essere salvato o inviato altrove
    
    # -------------------------------
    # Passaggio 8: Salvataggio della figura in un buffer
    # -------------

    # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    fig_image = buf.getvalue()
    buf.close()
    plt.close(fig)
    
    return fig_image

##### **MODELLI CNN2D_LSTM_TF, BiLSTM e Transformer**

In [ ]:
'''
DEFINIZIONE DEI MODELLI NEW VERSION PER SPETTROGRAMMI
'''


'''CNN2D_LSTM_TF

Spiegazione dettagliata dei passaggi

1) Input

Shape Iniziale: (batch_size, 64, 81, 9)
64 canali EEG, 81 bin di frequenza, 9 finestre temporali

2) Block 1

BatchNorm2d(64) normalizza ciascun canale
Conv2d(64→32, 3×3, padding=1) produce 32 mappe (B,32,81,9)
ReLU mantiene la stessa shape
MaxPool2d(2,2) dimezza: frequenza 81→40, time 9→4 → (B,32,40,4)

3) Block 2 “residuo”

res = x salva skip (B,32,40,4)
res_conv 1×1: Conv2d(32→64,k=1) riallinea i canali → (B,64,40,4)
res_bn normalizza → (B,64,40,4)

    Main path:

        BatchNorm2d(32) → (B,32,40,4)
        Conv2d(32→64,3×3,pad=1) + ReLU → (B,64,40,4)
        BatchNorm2d(64) → (B,64,40,4)
        Conv2d(64→64,3×3,pad=1) → (B,64,40,4)

    Somma residua: (B,64,40,4) + (B,64,40,4) → (B,64,40,4)
    ReLU + Pool: dimezza → (B,64,20,2)

4) Block 3

    Conv2d(64→128,3×3,pad=1) + BatchNorm + ReLU → (B,128,20,2)
    MaxPool2d → frequenza 20→10, time 2→1 → (B,128,10,1)

5) Preparazione per LSTM

    permute(0,2,1,3): porta freq come sequenza → (B,10,128,1)
    reshape(b,10,128*1): collapse canali×time → (B,10,128)

6) LSTM + testata finale

    Dropout 0.2 su (B,10,128)
    LSTM unidirezionale 128→64: produce out = (B,10,64)
    Estrai ultimo timestep out[:, -1, :] → (B,64)
    Linear(64→2) → logits (B,2)

Così hai un modello che parte da 64 canali EEG, mantiene l’architettura a 3 blocchi residui + LSTM del paper, e termina con una classificazione binaria.

'''

import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN2D_LSTM_TF(nn.Module):

    def __init__(self, input_channels=64, num_classes=2, dropout=0.2):
        super().__init__()
        # --- Block 1 ---
        self.bn1   = nn.BatchNorm2d(input_channels)    # normalizza 64 canali
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool2d(2, 2)

        # --- Block 2 (residual) ---
        # Proiezione 1×1 per riallineare i canali di skip (32→64)
        self.res_conv = nn.Conv2d(32, 64, kernel_size=1, bias=False)
        self.res_bn   = nn.BatchNorm2d(64)

        self.bn2a   = nn.BatchNorm2d(32)
        self.conv2a = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2b   = nn.BatchNorm2d(64)
        self.conv2b = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # --- Block 3 ---
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(128)

        # --- Head: Dropout + LSTM + FC finale ---
        self.dropout     = nn.Dropout(dropout)
        self.hidden_size = 64
        
        # dopo 3 pool: freq da 81→10, time da 9→1 → feature per timestep = 128×1
        self.lstm = nn.LSTM(
            input_size=128,
            hidden_size=self.hidden_size,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )
        self.classifier = nn.Linear(self.hidden_size, num_classes)

    def forward(self, x):
        # x: (B,64,81,9)

        # --- Block 1 ---
        x = self.bn1(x)                   # → (B,64,81,9)
        x = F.relu(self.conv1(x))         # → (B,32,81,9)
        x = self.pool(x)                  # → (B,32,40,4)

        # --- Block 2 (residuo) ---
        res = x                           # skip: (B,32,40,4)
        res = self.res_conv(res)          # progetto: → (B,64,40,4)
        res = self.res_bn(res)            # → (B,64,40,4)

        # main path
        x = self.bn2a(x)                  # → (B,32,40,4)
        x = F.relu(self.conv2a(x))        # → (B,64,40,4)
        x = self.bn2b(x)                  # → (B,64,40,4)
        x = self.conv2b(x)                # → (B,64,40,4)

        x = x + res                       # somma residua valida → (B,64,40,4)
        x = F.relu(x)                     
        x = self.pool(x)                  # → (B,64,20,2)

        # --- Block 3 ---
        x = F.relu(self.bn3(self.conv3(x)))  # → (B,128,20,2)
        x = self.pool(x)                     # → (B,128,10,1)

        # --- Prepara per LSTM ---
        x = x.permute(0, 2, 1, 3)         # → (B,10,128,1)
        b, seq, ch, tw = x.size()        
        x = x.reshape(b, seq, ch * tw)    # → (B,10,128)

        # --- LSTM + classificazione ---
        out, _ = self.lstm(self.dropout(x))  # → out: (B,10,64)
        last = out[:, -1, :]                 # prendo l’ultima uscita → (B,64)
        logits = self.classifier(last)       # → (B,2)
        return logits

In [ ]:
'''
DEFINIZIONE DEI MODELLI NEW VERSION PER SPETTROGRAMMI
'''


'''CNN2D

Una CNN2D si aspetta input in forma (batch, canali, altezza, larghezza). 
Nel tuo caso, puoi interpretare l’"altezza" come i bin di frequenza (38)
e la "larghezza" come le finestre temporali (6).

Quindi, la tua CNN2D lavorerebbe direttamente con:
Shape: (batch, canali, frequenze, tempo)


'''

class CNN2D(nn.Module):
    
    def __init__(self, input_channels, num_classes):
        
        super(CNN2D, self).__init__()
        
        # Ipotizziamo kernel 3x3 con padding per mantenere le dimensioni (puoi adattare a tuo piacimento)
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2))
        
        self.conv3 = nn.Conv2d(32, 48, kernel_size=(2, 2), stride=(1, 1), padding=1)
        self.bn3 = nn.BatchNorm2d(48)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2))
        
        # Utilizzo LazyLinear per evitare di calcolare manualmente la dimensione piatta finale
        self.fc1 = nn.LazyLinear(8)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.LazyLinear(num_classes)
        
    def forward(self, x):
        
        # x: (batch, canali, frequenze, tempo)
        
        # Passaggio attraverso il primo strato convoluzionale, BatchNorm e pooling
        x = self.conv1(x)
        x = self.bn1(x)  # Batch Normalization
        x = F.elu(x)
        #x = torch.tanh(x)  # Sostituito ELU con tanh
        x = self.pool1(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv2(x)
        x = self.bn2(x)  # Batch Normalization
        x = F.elu(x)
        #x = torch.tanh(x)  # Sostituito ELU con tanh
        x = self.pool2(x)

        # Passaggio attraverso il secondo strato convoluzionale, BatchNorm e pooling
        x = self.conv3(x)
        x = self.bn3(x)  # Batch Normalization
        x = F.elu(x)
        x = torch.tanh(x)  # Sostituito ELU con tanh
        x = self.pool3(x)

        # Flatten per preparare i dati per gli strati fully connected
        x = torch.flatten(x, start_dim=1)

        # Passaggio attraverso il primo strato fully connected
        x = self.fc1(x)
        x = F.elu(x)
        #x = torch.tanh(x)  # Sostituito ELU con tanh

        # Dropout per evitare overfitting
        x = self.dropout(x)

        # Passaggio attraverso il secondo strato fully connected
        x = self.fc2(x)

        return x



'''
Gli LSTM si aspettano un input in forma (batch, lunghezza_sequenza, dimensione_feature). 
Dovrai quindi decidere qual è la dimensione sequenziale.

Opzione comune: usare il tempo come sequenza
Step 1: Trasponi i dati in modo da avere il tempo come dimensione sequenziale.

Dalla forma (batch, canali, frequenze, tempo) puoi fare:


x = x.permute(0, 3, 1, 2)  # Diventa (batch, tempo, canali, frequenze)

Step 2: Unisci le dimensioni dei canali e dei bin di frequenza in un’unica dimensione di feature:


batch, tempo, canali, frequenze = x.shape
x = x.reshape(batch, tempo, canali * frequenze)  # Ora: (batch, tempo, canali*frequenze)

Nel tuo caso, per 3 canali e 38 bin di frequenza: input_size = 3 * 38 = 114 e lunghezza sequenza = 6.

Nota: Se invece preferisci usare i bin di frequenza come sequenza, potresti fare:

x = x.permute(0, 2, 1, 3)  # (batch, frequenze, canali, tempo)
x = x.reshape(batch, frequenze, canali * tempo)  # Sequence length = 38, feature size = 3*6 = 18
La scelta dipende dal tipo di informazione temporale o spettrale che vuoi evidenziare.

'''

class ReadMEndYou(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, output_size, dropout=0.5, bidirectional=False):
        """
        input_size: dimensione delle feature per time-step (dovrà essere canali * frequenze)
        hidden_sizes: lista con le dimensioni degli hidden state, es. [24, 48, 62]
        output_size: numero di classi
        
        """
    
        super(ReadMEndYou, self).__init__()
        
        self.bidirectional = bidirectional # Impostazione della bidirezionalità    
        
        # Adattiamo hidden_size in base alla bidirezionalità
        self.hidden_sizes = [
            hidden_sizes[0] * 2 if bidirectional else hidden_sizes[0],
            hidden_sizes[1] * 2 if bidirectional else hidden_sizes[1],
            hidden_sizes[2] * 2 if bidirectional else hidden_sizes[2]
        ]
        
        self.lstm1 = nn.LSTM(input_size=input_size, 
                             hidden_size=self.hidden_sizes[0], 
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0, 
                             bidirectional=bidirectional)
        self.lstm2 = nn.LSTM(input_size=self.hidden_sizes[0] * 2 if bidirectional else self.hidden_sizes[0],
                             hidden_size=self.hidden_sizes[1], 
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0,
                             bidirectional=bidirectional)
        self.lstm3 = nn.LSTM(input_size=self.hidden_sizes[1] * 2 if bidirectional else self.hidden_sizes[1],
                             hidden_size=self.hidden_sizes[2],
                             num_layers=1, 
                             batch_first=True, 
                             dropout=0,
                             bidirectional=bidirectional)
        
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.hidden_sizes[2] * 2 if bidirectional else self.hidden_sizes[2], output_size)
    
    def forward(self, x):
        
        # x: (batch, canali, frequenze, tempo)
        
        # Trasponi per avere il tempo come dimensione sequenziale:
        x = x.permute(0, 3, 1, 2)  # -> (batch, tempo, canali, frequenze)
        
        batch, time, channels, freqs = x.shape
        
        x = x.reshape(batch, time, channels * freqs)  # -> (batch, tempo, channels*frequencies)
        # Ora input_size deve essere channels * freqs (es. 3 * 26 = 78)
        # Ora input_size deve essere channels * freqs (es. 64 * 81 = 5184)
        
        # LSTM 1
        out, _ = self.lstm1(x)
        out = self.dropout(out)
        
        # LSTM 2
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        
        # LSTM 3
        out, _ = self.lstm3(out)
        out = self.dropout(out)
        
        # Estraiamo l'output dell'ultimo time-step
        out = out[:, -1, :]
        
        # Dropout prima del layer fully connected    
        out = self.dropout(out)
        
        # Passaggio attraverso il layer finale per la previsione
        out = self.fc(out)
        return out
        


'''
Il modulo Transformer in PyTorch lavora tipicamente su input di forma (seq_length, batch, embedding_dim).

Nel codice attuale, si parte da una forma simile a (batch, canali, seq_length), ma dovrai adattarla alla nuova struttura.

Possibili approcci:

1) Approccio A: usare il tempo come sequenza

Se consideri il tempo (6 time windows) come la sequenza, puoi procedere come segue:

A) Unisci canali e frequenze in un’unica dimensione di feature:

# Dati originali: (batch, canali, frequenze, tempo)
x = x.permute(0, 3, 1, 2)  # (batch, tempo, canali, frequenze)
batch, tempo, canali, frequenze = x.shape
x = x.reshape(batch, tempo, canali * frequenze)  # (batch, tempo, 3*38 = 114)

B) Modifica il layer di embedding:

Nel codice attuale, l'embedding è definito come:

self.embedding = nn.Linear(seq_length, d_model)
Dovrai cambiarlo in modo che mappi le dimensioni delle feature (in questo caso 114) a uno spazio latente:

self.embedding = nn.Linear(canali * frequenze, d_model)

C) Permuta per il Transformer:

Dopo l'embedding, passa l'input alla forma (seq_length, batch, d_model):

x = x.permute(1, 0, 2)  # Ora: (tempo, batch, d_model)


2) Approccio B: usare i bin di frequenza come sequenza
In alternativa, se reputi più rilevante la risoluzione spettrale, puoi considerare i 38 bin come sequenza e combinare canali e tempo:


x = x.permute(0, 2, 1, 3)  # (batch, frequenze, canali, tempo)
batch, frequenze, canali, tempo = x.shape
x = x.reshape(batch, frequenze, canali * tempo)  # (batch, frequenze, 3*6 = 18)

E poi procedere con un embedding layer che mappa da 18 a d_model e permutare in (frequenze, batch, d_model).

Scelta dell'approccio:
Se l'aspetto temporale è più critico, probabilmente è meglio usare l’Approccio A (sequenza di lunghezza 6).
Se invece vuoi dare maggior rilievo alla struttura spettrale, l’Approccio B potrebbe essere più indicato.

Ricorda che la scelta dipende dalla natura del tuo problema e dalla rilevanza delle informazioni temporali rispetto a quelle spettrali.
'''

import torch
import torch.nn as nn

#Scelta: In questa implementazione abbiamo deciso di usare il tempo come sequenza.
#In alternativa, potresti scegliere i bin di frequenza come sequenza, ma ciò richiederebbe una diversa riorganizzazione delle dimensioni 
#(ad esempio, un permute diverso).

class ReadMYMind(nn.Module):

    def __init__(self, d_model, num_heads, num_layers, num_classes, channels=64, freqs=81):
        
        super(ReadMYMind, self).__init__()

        # Il layer di embedding mapperà la feature dimension (channels * freqs) a d_model
        self.embedding = nn.Linear(channels * freqs, d_model)
        
        # Transformer per l'attenzione spaziale (qui si applica direttamente alla sequenza temporale)
        self.spatial_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
        # Transformer per l'attenzione temporale (si potrebbe considerare un'iterazione successiva)
        self.temporal_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
        # Cross-attention per combinare le rappresentazioni
        self.cross_attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads)
        
        # Fusione e classificazione finale
        self.fc_fusion = nn.Linear(d_model, d_model)
        self.fc_classify = nn.Linear(d_model, num_classes)
    
    def forward(self, x):
        # x: (batch, canali, frequenze, tempo)
        
        # Utilizziamo il tempo come sequenza
        x = x.permute(0, 3, 1, 2)  # -> (batch, tempo, canali, frequenze)
        
        batch, time, channels, freqs = x.shape
        x = x.reshape(batch, time, channels * freqs)  # -> (batch, tempo, channels*frequencies)
        
        # Embedding: (batch, tempo, d_model)
        x = self.embedding(x)
        
        # Transformer richiede input di forma (seq_length, batch, embedding_dim)
        x = x.permute(1, 0, 2)  # -> (tempo, batch, d_model)
        
        # Applichiamo il Transformer per l'attenzione spaziale
        x_spatial = self.spatial_transformer(x)
        
        # Applichiamo il Transformer per l'attenzione temporale
        x_temporal = self.temporal_transformer(x_spatial)
        
        # Cross-attention: (tempo, batch, d_model)
        x_cross, _ = self.cross_attention(x_spatial, x_temporal, x_temporal)
        
        # Fusione: per esempio, facciamo una media sul tempo (dimensione 0)
        x_fused = self.fc_fusion((x_spatial + x_temporal).mean(dim=0))  # -> (batch, d_model)
        
        # Classificazione finale
        output = self.fc_classify(x_fused)  # -> (batch, num_classes)
        
        return output


##### **NUOVO LOOP PER DATI NON HYPER SU CNN2D_LSTM_TF, BiLSTM e Transformer**

In [ ]:
import os
import re

import random
#perché è importante numpy.random.seed()?
#https://www.analyticsvidhya.com/blog/2021/12/what-does-numpy-random-seed-do/#:~:text=The%20numpy%20random%20seed%20is,displays%20the%20same%20random%20numbers.
from numpy.random import seed

import numpy as np
import copy as cp

from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

#importing librerie pytorch
import torch 
import torch.nn as nn #neural network module
import torch.optim as optim #ottimizzatore
import torch.nn.functional as F 
from torch.utils.data import DataLoader, TensorDataset

#from sklearn.model_selection import KFold

#importing librerie numpy, pandas, scikit-learn e matplotlib
import numpy as np


import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

from tqdm import tqdm



In [ ]:
'''VERSIONE NUOVA UFFICIALE


Ecco come puoi correggere solo il calcolo dell’AUC–ROC sul training set a posteriori, 
lasciando invariato tutto il resto di load_best_run_results. 


L’idea è:

1) Estrarre la history normale da W&B (che contiene il vecchio train_auc)
2) Individuare best_epoch
3) Caricare il modello migliore da disco
4) Rifare un passaggio solo sullo train_loader per ottenere le vere probabilità e ricalcolare la ROC–AUC
5) Sovrascrivere il vecchio valore auc_train_history[best_epoch] e aggiornare best_metrics["train_auc"]



Cosa è cambiato

1) Ti ho inserito un passaggio 6) in cui ricalcoli l’AUC–ROC vero del train set, usando torch.softmax(…,dim=1)[:,1].
2) Sostituisci il vecchio auc_train_history[best_epoch] col valore corretto.
3) Ricomponi best_metrics["train_auc"] con true_auc_train.

Da qui in poi, puoi chiamare subito dopo la tua testing(...) per ottenere anche tutte le metriche sul test set e salvare la tabella finale in cui:

“Train” = best_metrics["train_*"] (ora con AUC corretta)

“Test” = test_results["test_performances"]

Ecco fatto: nessun re‑training, solo un passaggio aggiuntivo per correggere il calcolo dell’AUC–ROC sul train set.



Quindi il punto 6

# --- 6) Ricalcolo vero train AUC–ROC sul train_loader ---

serve per ri-calcolarsi correttamente l'auc roc al train set nell'epoca in cui sul val set ho ottenuto la migliore validation accuracy, 
che corrisponde quindi al modello salvato dentro il best_model che io ri-prelevo quando poi lo do in pasto al test set?


Esattamente: quel passaggio 6):

Riprende il modello caricato dal file .pkl (che è proprio il best_model scelto sull’epoca di miglior val_accuracy),

Lo mette in eval() e senza gradienti scorre tutto il train_loader,

Calcola le probabilità (softmax(:,1)) e da quelle ricava la vera ROC–AUC per il train set,

Infine sovrascrive auc_train_history[best_epoch] e aggiorna best_metrics["train_auc"] con questo valore corretto.

In questo modo la tua colonna “Train” nella tabella conterrà davvero l’AUC–ROC calcolata sulle probabilità del modello nella stessa epoca 
in cui hai ottenuto la migliore validazione, cioè esattamente quei pesi che poi passerai al test set.


'''

from wandb import Api
import torch
import numpy as np

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import io
import matplotlib.pyplot as plt
from PIL import Image


import re


    
'''
1) questa serve per plottare le metriche di loss e accuracy in ogni modello e condizione sperimentale
per salvarla dentro al dizionario 'training_plot' come buffer di memoria
'''


def plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history):
    
    '''
    # Creazione di una figura con 2 subplot
    '''
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))  # 2 righe, 1 colonna, dimensione figura

    #Plot della loss
    ax[0].plot(loss_train_history, label='Train Loss', color='blue')
    ax[0].plot(loss_val_history, label='Validation Loss', color='orange')
    #ax[0].set_title(f'Loss during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[0].set_title(f'Loss during Training: ', fontsize=12)  # Titolo più grande
    ax[0].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[0].set_ylabel('Loss', fontsize=12)    # Dimensione font asse y
    ax[0].legend(fontsize=12)  # Dimensione font legenda
    ax[0].grid(True)

    # Plot dell'accuracy
    ax[1].plot(accuracy_train_history, label='Train Accuracy', color='blue')
    ax[1].plot(accuracy_val_history, label='Validation Accuracy', color='orange')
    #ax[1].set_title(f'Accuracy during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[1].set_title(f'Accuracy during Training: ', fontsize=12)  # Titolo più grande
    ax[1].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[1].set_ylabel('Accuracy', fontsize=12)  # Dimensione font asse y
    ax[1].legend(fontsize=12)  # Dimensione font legenda
    ax[1].grid(True)
    
    # Regolare la spaziatura tra i subplot
    plt.tight_layout()  # Alternativa: fig.subplots_adjust(hspace=0.3)
    
    #plt.close(fig)
    
    '''
    # Salvare il plot in un buffer di memoria
    '''
    buf = io.BytesIO()
    plt.savefig(buf, format='png')  # Salviamo il plot in formato PNG
    buf.seek(0)  # Torniamo all'inizio del buffer

    # Convertire il buffer in un'immagine PIL (opzionale, per visualizzarla)
    img = Image.open(buf)

    # Aggiungere i dati dell'immagine nel dizionario
    plot_image_data = buf.getvalue()  # Otteniamo i dati binari dell'immagine
    buf.close()
    
    # Ritorniamo i dati dell'immagine da salvare nel dizionario
    return plot_image_data


'''
2) questa serve per estrarmi le stringhe per ricostruire il nome del progetto su W&B per 
poi estrarmi le metriche ottenute sul training e validation 
da salvare sempre dentro al dizionario 'training_plot' 
'''

# Funzione per parsare la chiave
def parse_combination_key(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    Il formato atteso è:
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ 
    "1_20|1_45|wavelet" _ 
    "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    """
    match = re.match(
        r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_(spectrograms)_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        

'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

# Test
#combination_key = "rest_vs_left_fist_spectrograms_familiar_th"
#condition_experiment, data_type, subject_key = parse_combination_key(combination_key)

#print("Condizione:", condition_experiment)
#print("Data Type:", data_type)
#print("Soggetto:", subject_key)



'''

Ecco come puoi correggere solo il calcolo dell’AUC–ROC sul training set a posteriori, lasciando invariato tutto il resto di load_best_run_results. 


L’idea è:

1) Estrarre la history normale da W&B (che contiene il vecchio train_auc)
2) Individuare best_epoch
3) Caricare il modello migliore da disco
4) Rifare un passaggio solo sullo train_loader per ottenere le vere probabilità e ricalcolare la ROC–AUC
5) Sovrascrivere il vecchio valore auc_train_history[best_epoch] e aggiornare best_metrics["train_auc"]



Cosa è cambiato

1) Ti ho inserito un passaggio 6) in cui ricalcoli l’AUC–ROC vero del train set, usando torch.softmax(…,dim=1)[:,1].
2) Sostituisci il vecchio auc_train_history[best_epoch] col valore corretto.
3) Ricomponi best_metrics["train_auc"] con true_auc_train.

Da qui in poi, puoi chiamare subito dopo la tua testing(...) per ottenere anche tutte le metriche sul test set e salvare la tabella finale in cui:

“Train” = best_metrics["train_*"] (ora con AUC corretta)

“Test” = test_results["test_performances"]

Ecco fatto: nessun re‑training, solo un passaggio aggiuntivo per correggere il calcolo dell’AUC–ROC sul train set.



Quindi il punto 6

# --- 6) Ricalcolo vero train AUC–ROC sul train_loader ---

serve per ri-calcolarsi correttamente l'auc roc al train set nell'epoca in cui sul val set ho ottenuto la migliore validation accuracy, 
che corrisponde quindi al modello salvato dentro il best_model che io ri-prelevo quando poi lo do in pasto al test set?


Esattamente: quel passaggio 6):

Riprende il modello caricato dal file .pkl (che è proprio il best_model scelto sull’epoca di miglior val_accuracy),

Lo mette in eval() e senza gradienti scorre tutto il train_loader,

Calcola le probabilità (softmax(:,1)) e da quelle ricava la vera ROC–AUC per il train set,

Infine sovrascrive auc_train_history[best_epoch] e aggiorna best_metrics["train_auc"] con questo valore corretto.

In questo modo la tua colonna “Train” nella tabella conterrà davvero l’AUC–ROC calcolata sulle probabilità del modello nella stessa epoca 
in cui hai ottenuto la migliore validazione, cioè esattamente quei pesi che poi passerai al test set.

'''


'''
3) Dopodiché, comincia la funzione di load_best_run_results che, 
per ogni progetto e sweep del relativo modello,

si va ad estrarre le metriche del train (corregge il calcolo del train_auc)
e si calcola anche per il validation phase la confusion matrix e classification report


4) dopodichè dovrebbe richiamare la funzione di 
"plot_training_results" in modo che poi si salvi i plot di training e validation (sia loss che accuracy)
in modo che si salvi tutto in una immagine come buffer che viene spuntato fuori da quella funzione 

e poi inserito come valore dentro al dizionario training_results che sarà l'output di "load_best_run_results" 


quindi qui sotto mi manca richiamare la funzione "plot_training_results" con una variabile tipo training_plot = plot_training_results che avrà come argomenti

queste liste qua salvate come colonne del df creato dentro a 'load_best_run_results!'


loss_train_history     = df["train_loss"].tolist()
loss_val_history       = df["val_loss"].tolist()
accuracy_train_history = df["train_accuracy"].tolist()
accuracy_val_history   = df["val_accuracy"].tolist()


5) dopodiché mi serve caricare tutte queste info dentro al dizionario train_results, che sarà l'output di load_best_run_results... 
e su questo ho dei dubbi su quali chiavi del dizionario tenere separate oppure se "unirne" qualcuna, aggregando tutte le info del sweep_config assieme, 
sia che siano veri iper-parametri (learning rate etc) o parametri architetturali della rete (anche se avevano valori fissi) il più delle volte se vedi



# 2.1 – Sweep config per ciascun modello
sweep_config_cnn3d = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN3D_LSTM_FC"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True, False]},
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "use_lstm": {"values": [True, False]},
        "lstm_hidden": {"values": [32]},
        "dropout": {"values": [0.5]},
    }
}


sweep_config_cnn_sep = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["SeparableCNN2D_LSTM_FC"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True, False]},
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "use_lstm": {"values": [True, False]},
        "lstm_hidden": {"values": [32]},
        "dropout": {"values": [0.5]},
    }
}


'''

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def load_best_run_results(
    key, # es. "rest_vs_left_fist_spectrograms_familiar_th"
    model, # # <-- istanza PyTorch già caricata con i pesi best es. "CNN3D_LSTM_FC"
    
    sweep_config,      # <— qui richiamo lo sweep config del modello corrispondente
    
    data_loaders, # dict con DataLoader per "train" e "val"
    entity = "my_wb_entity"): # entity = "stefano‑bargione‑universit‑di‑roma‑tor‑vergata"
    
    
    # --- 1) Parse key e ricava project name ---
    exp_cond, data_type, category_subject = parse_combination_key(key)
    
    
    '''CAMBIATA PER DATI MOTORI (MOVIMENTO REALE!) IN RAPPRESENTAZIONE TIME X FREQS'''
    #project = f"{exp_cond}_{data_type}_channels_freqs_new_3d_grid_multiband"
    
    project = f"{exp_cond}_{data_type}_time_freqs_new_3d_grid_multiband"
    
    '''OLD APPROACH'''
    #model_name = type(model).__name__
    

    '''SE ESTRAGGO SWEEP ID A POSTERIORI DAL PROGETTO

    1) Prendo tutte le run del progetto e modello corrispondente
    2) Filtro solo quelle con config["model_name"] == model_name.
    3) Controllo che ce ne sia almeno una (altrimenti errore).
    4) Costruisce un set di tutti gli r.sweep e verifica che sia esattamente uno (altrimenti errore).
    5) Estrae quello unico (.pop()) e lo stampa insieme al numero di run.
    6) Infine, seleziona la singola best_run sulla base di val_accuracy.

    '''
    
    # 2) Recupero tutte le run del progetto
    api  = Api()
    runs = api.runs(f"{entity}/{project}")
    
    
    '''NEW APPROACH'''
    # === PATCH: accetta alias tra nome classe PyTorch e nome usato nello sweep ===
    def _get_param_list(cfg, key):
        p = cfg["parameters"][key]
        vals = p.get("values", p.get("value"))
        if isinstance(vals, str): 
            return [vals]
        return list(vals) if isinstance(vals, (list, tuple)) else [vals]

    model_class = type(model).__name__          # es. "ReadMEndYou"
    cfg_names   = _get_param_list(sweep_config, "model_name")   # es. ["BiLSTM"]
    aliases     = set([model_class, *cfg_names])                # {"ReadMEndYou","BiLSTM"}
    
    def matches_aliases(r):
        return (
            r.config.get("model_name") in aliases or
            r.config.get("model_class") in aliases or
            bool(set(r.tags or []) & aliases)
        )
    
    # 3) filtro solo quelle del modello giusto
    
    '''OLD APPROACH'''
    #runs_filtered = [r for r in runs if r.config.get("model_name", "") == model_name]
    #n_runs = len(runs_filtered)
    
    '''NEW APPROACH'''
    runs_filtered = [r for r in runs if matches_aliases(r)]
    n_runs = len(runs_filtered)
    

    if n_runs == 0:
        raise RuntimeError(f"Nessuna run trovata per progetto `{project}` e modello `{model_name}`")

    # 4) controllo che le run filtrate appartengano tutte allo stesso sweep
    unique_sweeps = {r.sweep for r in runs_filtered}
    if len(unique_sweeps) != 1:
        raise RuntimeError(
            f"Trovati più sweep per progetto `{project}` e modello `{model_name}`: {unique_sweeps}"
        )

    # 5) estraggo lo sweep_id
    sweep_id_unico = unique_sweeps.pop()
    #print(f"✓ Trovate \033[1m{n_runs}\033[0m runs in progetto `{project}` e modello `{model_name}`, sweep: `{sweep_id_unico}`")
    print(f"✓ Trovate \033[1m{n_runs}\033[0m runs\n")
    print(f"✓ Progetto \033[1m`{project}`\033[0m\n")
    print(f"✓ Modello \033[1m`{model_name}`\033[0m\n")
    print(f"✓ Sweep \033[1m`{sweep_id_unico}`\033[0m\n\n")

    # 6) scelgo la run con val_accuracy massima
    best_run = max(runs_filtered, key=lambda r: r.summary.get("val_accuracy", 0.0))

    # --- 7) Estraggo tutta la history (compresi i train_auc sbagliati) ---
    df = best_run.history(
        keys=[
          "train_loss","train_accuracy","train_precision",
          "train_recall","train_f1","train_auc",
          "val_loss","val_accuracy"
        ],
        pandas=True
    )
    # converto in liste
    loss_train_history     = df["train_loss"].tolist()
    loss_val_history       = df["val_loss"].tolist()
    accuracy_train_history = df["train_accuracy"].tolist()
    accuracy_val_history   = df["val_accuracy"].tolist()
    precision_train_history= df["train_precision"].tolist()
    recall_train_history   = df["train_recall"].tolist()
    f1_train_history       = df["train_f1"].tolist()
    auc_train_history      = df["train_auc"].tolist()

    # best_epoch (su val_accuracy)
    best_epoch = int(df["val_accuracy"].idxmax())

    # --- 8) Prendo il modello ottimizzato .pkl corrispondente passato in input ---
    device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device).eval()

    # --- 9) Ricalcolo vero train AUC–ROC sul train_loader ---
    y_t_train, y_s_train = [], []
    with torch.no_grad():
        for x,y in data_loaders["train"]:
            x = x.to(device)
            logits = model(x)
            probs  = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
            y_s_train.extend(probs)
            y_t_train.extend(y.numpy())
            
    true_auc_train = roc_auc_score(np.array(y_t_train), np.array(y_s_train))

    # Sovrascrivo il vecchio valore sbagliato
    auc_train_history[best_epoch] = true_auc_train

    # Ricostruisco best_metrics
    best_metrics = {
      "train_loss":       [round(loss_train_history[best_epoch],4)],
      "train_accuracy":   [round(accuracy_train_history[best_epoch],4)],
      "train_precision":  [round(precision_train_history[best_epoch],4)],
      "train_recall":     [round(recall_train_history[best_epoch],4)],
      "train_f1_score":   [round(f1_train_history[best_epoch],4)],
      "train_auc":        [round(true_auc_train,4)]
    }
    
    #Solo una nota: qui non serve per training che l'auc abbia l'average='weighted' 
    #perché è binario e stai usando score continui.
    #anche se sopra lo avevi messo in "training_sweep".
    
    #Per le altre metriche (precision, recall, f1_score invece) l'average andava bene!
    #Anche in binario: average='weighted' = fai la media pesata per supporto delle metriche per ciascuna classe (0 e 1). 
    #È sensato se hai sbilanciamento e vuoi che le metriche riflettano anche quanto è frequente ciascuna classe. 
    
    #L’unica cosa da essere consapevoli è che non stai riportando “F1 della classe positiva”, 
    #ma una F1 complessiva pesata sulle due classi. 
    #Ma va bene, basta essere coerenti e chiari nel testo della tesi/paper."
    

    # --- 10) Ricreo confusion matrix e classification report su val set ---
    
    #Per il validation set, invece, rifai il calcolo:
    
    #y_t_val = true labels (0/1).
    #y_p_val = predizioni binarie (0/1), usate per accuracy / precision / recall / f1.
    #y_s_val = score continui (probabilità o logit della classe 1), usati per il calcolo dell'AUC-ROC:
    
    #Quindi diventerà ---> val_auc = roc_auc_score(y_t_val, y_s_val)
    #Quindi qui "y_s_val" è semplicemente la lista di p(y=1) per ogni campione di validation.
    
    y_t_val, y_p_val, y_s_val = [], [], []
    with torch.no_grad():
        for x,y in data_loaders["val"]:
            x = x.to(device)
            logits = model(x)
            probs  = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
            preds  = (probs >= 0.5).astype(int)
            
            
            y_p_val.extend(preds) # predizioni 0/1
            y_s_val.extend(probs) # score continui per AUC
            y_t_val.extend(y.numpy()) 
            
    confusion_matrix_val = confusion_matrix(y_t_val, y_p_val)
    classification_report_val = classification_report(y_t_val, y_p_val, output_dict=False)
    
    # Metriche Validation
    val_accuracy = accuracy_score(y_t_val, y_p_val)
    val_precision = precision_score(y_t_val, y_p_val, average='weighted')
    val_recall    = recall_score(y_t_val, y_p_val, average='weighted')
    val_f1        = f1_score(y_t_val, y_p_val, average='weighted')
    
    try:
        val_auc = roc_auc_score(y_t_val, y_s_val)   # <-- NOTHING average=... qui
    except ValueError:
        print("⚠️ AUC non calcolabile: nel val set c'è una sola classe.")
        val_auc = np.nan
    
    # Val performances alla best_epoch
    validation_performances = {
        # dalla history di W&B (loss/acc per quella epoch)
        "val_loss":       [round(loss_val_history[best_epoch],4)],
        "val_accuracy":   [round(accuracy_val_history[best_epoch],4)],
        
        # dalle metriche ricalcolate con il best_model
        "val_precision":  [round(val_precision,4)],
        "val_recall":     [round(val_recall,4)],
        "val_f1_score":   [round(val_f1,4)],
        "val_auc":        [round(val_auc,4)],
    }
    
        
    # --- 10) Plot delle curve loss/accuracy tra train e test ---
    training_plot = plot_training_results(
        loss_train_history,
        loss_val_history,
        accuracy_train_history,
        accuracy_val_history
    )

    # --- 11) Composizione del dict finale identico a `training()` ---
    
    # Restituire tutti i risultati in un dizionario
    train_results = {
        "training_performances": best_metrics,  # Aggiungi il dizionario delle performance
        
        "loss_train_history": loss_train_history,
        "loss_val_history": loss_val_history,
        
        "accuracy_train_history": accuracy_train_history,
        "accuracy_val_history": accuracy_val_history,
        
        "best_model": model,
        
        # VALIDATION
        "validation_performances": validation_performances,
        
        "confusion_matrix": confusion_matrix_val,
        "classification_report": classification_report_val,
    
        "hyperparams" : {k: best_run.config[k] for k in best_run.config.keys() if k in sweep_config["parameters"]},
            
        "training_plot": training_plot  # Salviamo il buffer con il plot
    }
    
    '''
    Ho questo errore "Errore “cudnn RNN backward can only be called in training mode”" solo con i dati di 
    left_fist_vs_right_fist, per il modello SeparableCNN2D_LSTM_FC, 
    mentre con i dati delle altre condizioni sperimentali, ossia:
    
    rest_vs_left_fist o rest_vs_right_fist, sempre per il modello SeparableCNN2D_LSTM_FC,non succede ... come mai solo con l'ultimo succede? 
    
    cioè dove dovrei aver lasciato il modello caricato in eval.() ?
    
    probabilmente qui nella funzione load_best_train_results!?
    
    quindi qui poi alla fine dovrei rimettere il modello in un'altra modalità alla fine della funzione? 
    perché in sostanza, dovrebbe succedere che in sostanza... non succede nulla per lo stesso modello per  gli altri dati, 
    perché ogni volta che ne prendo uno lo porto in eval e vabbè.. ma poi il problema succede solo per l'ultimo caso solo, 
    perché forse l'ultimo proprio, ossia solo SeparableCNN2D_LSTM_FC usa proprio il layer LSTM e quindi da errore là,
    perché dentro a load_best_train_results è rimasto in .eval() ed ha il layer LSTM e quindi dà errore?
    
    
    
    Perché l’errore appare “solo” con l’ultima combinazione

    1. load_best_run_results() termina con:

    model.to(device).eval()   # ← il modello rimane in eval()
    
    2. In compute_gradcam_figure() tu usi il best model che hai messo in train_results["best_model"] (quello appena impostato in eval()), poi esegui:

   
    output = model(sample_input)
    ...
    target.backward()         # <-- gradiente attraverso l’LSTM
    3. Il kernel CuDNN per gli RNN (LSTM/GRU) rifiuta il backward quando il modulo è in modalità inference (eval()), e solleva:

    
    RuntimeError: cudnn RNN backward can only be called in training mode
    
    4. Per le combinazioni precedenti con lo stesso modello “SeparableCNN2D_LSTM_FC” non è esploso perché, con ogni probabilità, 
    use_lstm=False nelle relative run migliori (quindi l’LSTM non c’è e CuDNN non interviene).
    
    Nell’ultima combinazione invece la best‑run ha use_lstm=True, quindi compare l’LSTM e l’errore salta fuori.
    
    '''
    
    return train_results

In [ ]:
#LOOP PER CARICARE I DATI NON HYPER
data_dict = {}

# Condizioni sperimentali
#experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

experimental_conditions = ["rest_vs_left_fist", "rest_vs_right_fist", "left_fist_vs_right_fist"]

for condition in experimental_conditions:

    for data_type in ["spectrograms"]:
        
        #for category in ["familiar", "unfamiliar"]:
        for category in ["familiar"]:
            
            #for subject_type in ["th", "pt"]:
            for subject_type in ["th"]:
            
                # Caricamento e suddivisione dei dati
                if data_type == "wavelet":
                    X, y = load_data(data_type, category, subject_type, wavelet_level="delta", condition = condition)
                else:
                    X, y = load_data(data_type, category, subject_type, condition = condition)

                #key = f"{condition}/{data_type}_{category}_{subject_type}"
                key = f"{condition}_{data_type}_{category}_{subject_type}"
                data_dict[key] = (X, y)

                # Stampa di conferma
                print(f"Dataset caricato: \033[1m{key}\033[0m - Forma X: {X.shape}, Lunghezza y: {len(y)}")

In [ ]:
data_dict.keys()

In [ ]:
#data_dict['rest_vs_left_fist_spectrograms_familiar_th'][0].shape

In [ ]:
'''
perfetto ora, siccome ho creato data_dict nel modo di cui sopra, 
ora dentro ogni chiave, 
ci sono già tutte le chiavi associate correttamente, per estrarmi i dati e labels corrispondenti di quella combinazione di fattori lì.

infatti dentro ogni chiave c'è una tupla, con 2 elementi, il primo è l'array dei dati, il secondo è l'array delle labels
'''

#data_dict['rest_vs_left_fist']['spectrograms'][0].shape

data_dict['rest_vs_left_fist_spectrograms_familiar_th'][0].shape

In [ ]:
'''NEW VERSION'''

# Percorso base per il salvataggio
base_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB_GradCAM_Checks_motor_task_1_new"

os.makedirs(base_folder, exist_ok=True)

# Condizioni sperimentali
#experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

experimental_conditions = ["rest_vs_left_fist", "rest_vs_right_fist", "left_fist_vs_right_fist"]

# Tipologie di dati
data_types = ["spectrograms"]

# Subfolders per tipologia di soggetto
#subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]
subfolders = ["th_fam"]

# Creazione della struttura delle cartelle
for condition in experimental_conditions:
    for data_type in data_types:
        for subfolder in subfolders:
            
            path = os.path.join(base_folder, condition, data_type, subfolder)
            
            if not os.path.exists(path):
                os.makedirs(path, exist_ok=True)
            
            print(f"Cartella creata: \033[1m{path}\033[0m")

In [ ]:
'''


# 2.1 – Sweep config per ciascun modello
sweep_config_cnn2d_lstm_tf = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN2D_LSTM_TF"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
    
        "dropout": {"values": [0.5]},
    }
}


sweep_config_bilstm = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["BiLSTM"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True]}, #        ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
        "dropout": {"values": [0.5]},
        "bidirectional": {"values": [False, True]},
        
        #Soluzione 1 per mettere valori agli hidden sizes
        #"hidden1": {"values": [24, 32, 48, 64]},
        #"hidden2": {"values": [48, 64, 96, 128]},
        #"hidden3": {"values": [62, 96, 128, 160]}
        # in build del modello: hidden_sizes=[hidden1, hidden2, hidden3]
        
        #Soluzione 2 per mettere valori agli hidden sizes
        
        #hidden_sizes = [24, 48, 62]
        #lstm_model = ReadMEndYou(input_size=input_channels * num_freqs, hidden_sizes=hidden_sizes, output_size=num_classes)
    }
}


sweep_config_transformer = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["Transformer"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True]}, #        ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
        "d_model": {"values": [32]},
        "num_heads": {"values": [2]},
        "num_layers": {"values": [2]},
    }
}




  ....
  ....
  ....
  
  
  
        # ====== MODELLI TIME×FREQ ======
    
        cnn_model = CNN2D_LSTM_TF(input_channels = input_channels, num_classes =num_classes, dropout = dropout)
        hidden_sizes = [24, 48, 62]
        lstm_model = ReadMEndYou(input_size=input_channels * num_freqs, hidden_sizes=hidden_sizes, output_size=num_classes)
        transformer_model = ReadMYMind(d_model=d_model, num_heads=num_heads, num_layers=num_layers, num_classes=num_classes)

        # Creazione di dati fittizi per il test
        x = torch.randn(batch_size, input_channels, num_freqs, num_timepoints)  # (batch, canali, frequenze, tempo)
        

        # ricavo channels e freqs dai dati (shape attesa: N, C, F, T)
        channels, freqs = int(X_train.shape[1]), int(X_train.shape[2])

        if config.model_name == "CNN2D_LSTM_TF":

            #OCCHIO QUI ADESSO SAREBBE TEMPO x FREQUENZA

            model = CNN2D_LSTM_TF(
                input_channels = channels, # qui 64
                num_classes = 2,
                dropout=config.dropout,
            )

            print(f"\nInizializzazione Modello \033[1mCNN2D_LSTM_TF\033[0m")


        elif config.model_name == "BiLSTM":
            hidden_sizes = [24, 48, 62]
            model = ReadMEndYou(
                input_size = channels * freqs, #  qui 64*81
                hidden_sizes = hidden_sizes,
                output_size = 2,
                dropout = config.dropout,
                bidirectional = config.bidirectional
            )
            print(f"\nInizializzazione Modello \033[1mReadMEndYou (BiLSTM)\033[0m")

        elif config.model_name == "Transformer":
            model = ReadMYMind(
                d_model=config.d_model,
                num_heads=config.num_heads,
                num_layers=config.num_layers,
                num_classes=2,
                channels = channels,
                freqs = freqs
            )

            print(f"\nInizializzazione Modello \033[1mReadMYMind (Transformer)\033[0m")
        else:
            raise ValueError(f"Modello sconosciuto: {config.model_name}")
            
            
'''

In [ ]:
# 2.1 – Sweep config per ciascun modello
sweep_config_cnn2d_lstm_tf = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN2D_LSTM_TF"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
    
        "dropout": {"values": [0.5]},
    }
}


sweep_config_bilstm = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["BiLSTM"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True]}, #        ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
        "dropout": {"values": [0.5]},
        "bidirectional": {"values": [False, True]},
        
        #Soluzione 1 per mettere valori agli hidden sizes
        #"hidden1": {"values": [24, 32, 48, 64]},
        #"hidden2": {"values": [48, 64, 96, 128]},
        #"hidden3": {"values": [62, 96, 128, 160]}
        # in build del modello: hidden_sizes=[hidden1, hidden2, hidden3]
        
        #Soluzione 2 per mettere valori agli hidden sizes
        
        #hidden_sizes = [24, 48, 62]
        #lstm_model = ReadMEndYou(input_size=input_channels * num_freqs, hidden_sizes=hidden_sizes, output_size=num_classes)
    }
}


sweep_config_transformer = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["Transformer"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True]}, #        ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        
        # --- specifici del modello ---
        "d_model": {"values": [32]},
        "num_heads": {"values": [2]},
        "num_layers": {"values": [2]},
    }
}


In [ ]:
'''                                              *************** CON NUOVE MODIFICHE DEL PAPER ***************


Cosa cambia rispetto a prima

1) load_config_if_available:

    estrae la config (batch_size, lr, β₁, β₂, ε, ecc.)
    carica i best_weights salvati in .pkl da W&B

2) processed_datasets / processed_models

    evitano duplicate runs

3) Optimizer & Scheduler & EarlyStopping

    sono configurati con i valori estratti da config
    scheduler con patience=4 (dimezza lr dopo 4 epoche senza miglioramento di val_loss)
    early‐stopping con patience=12 sul val_loss

4) Training/testing

    la tua funzione training(...) dovrebbe accettare ora anche scheduler e early_stopping

5) GradCAM

    calcolato e salvato solo per il tuo CNN2D+LSTM

Con queste modifiche, il tuo loop:

    rispetta fedelmente il setup del paper
    mantiene la pipeline di W&B per la logica di sweep e salvataggio dei migliori modelli
    aggiunge minimi cambiamenti, lasciando intatte le tue funzioni training(), testing() e save_performance_results()

'''


# Imposta il seme per la riproducibilità

#Imposta il seme per i generatori casuali di PyTorch (per operazioni sui tensori e inizializzazione dei pesi dei modelli).
#Importante se vuoi garantire che l'addestramento del modello produca gli stessi risultati in diverse esecuzioni.
torch.manual_seed(32)

#Imposta il seme per NumPy, utile se NumPy viene usato per operazioni casuali (ad es. shuffling dei dati, inizializzazione di matrici, ecc.).
#Importante se usi NumPy per il preprocessing dei dati e vuoi riproducibilità.

np.random.seed(32)

#mposta il seme per il modulo random di Python (utile se si usano funzioni di randomizzazione di Python puro).
#Importante solo se usi random per operazioni come mescolamento di liste.
random.seed(32)

#Imposta il seme per i generatori casuali su GPU, se disponibile.
#Utile se stai eseguendo il codice su una GPU per garantire riproducibilità anche in quel contesto.

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(32)

       
'''

In questo caso, 

il set processed_datasets traccia i dataset già elaborati, 
e il set processed_models tiene traccia delle combinazioni già effettuate (modello + dataset). 

In questo modo, puoi escludere un dataset dal training se è già stato utilizzato in precedenza, 
anche se usato con un modello differente.
'''

# Dizionario per tracciare la standardizzazione usata per ogni combinazione d
# Dizionario per salvare informazioni sul modello (es. se i dati sono standardizzati)
models_info = {}


# Set per tenere traccia dei dataset già elaborati
processed_datasets = set()

# Set per tenere traccia delle combinazioni già elaborate
processed_models = set()


# Path delle performance dei modelli ottimizzati con weight and biases
# Path per trovare le best performances di ogni modello per ogni combinazione dei dati
base_folder = "/home/stefano/Interrogait/WB_spectrograms_best_results_motor_task_1_new"
                                        #WB_spectrograms_best_results_motor_task_1

# Path di salvataggio delle performance dei modelli dopo estrazione best models da base_folder
#save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB"


save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB_GradCAM_Checks_motor_task_1_new"
                                              #spectrograms_best_models_post_WB_GradCAM_Checks_motor_task_1_new


# --- LOOP PRINCIPALE (con minime modifiche) ---
for key, (X_data, y_data) in data_dict.items():
    
    print(f"\n\nEstrazione Dati per il dataset: \033[1m{key}\033[0m, \tShape X: \033[1m{X_data.shape}\033[0m, Shape y: \033[1m{y_data.shape}\033[0m")
    
    if key in processed_datasets:
        print(f"ATTENZIONE: Il dataset {key} è già stato elaborato! Salto iterazione...")
        continue
        
    processed_datasets.add(key)
    
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
    print(f"Dataset Splitting: Train: \033[1m{X_train.shape}\033[0m, Val: \033[1m{X_val.shape}\033[0m, Test: \033[1m{X_test.shape}\033[0m")
    
    
    '''
    CREO COPIA TEST_LOADER_RAW PER I PLOT DEL POWER RAW PER BANDA E CLASSE
    '''
    # 1) salva una copia RAW dei soli dati di test PRIMA di standardizzare
    X_test_raw = X_test.copy()
    y_test_raw = y_test.copy()
    
    # 2) tensori
    X_raw_tensor = torch.tensor(X_test_raw, dtype=torch.float32)
    y_raw_tensor = torch.tensor(y_test_raw, dtype=torch.long)
    
    
    
    
    #for model_name in ["CNN2D", "BiLSTM", "Transformer"]:
    
    '''ATTENZIONE MODIFICA QUI'''
    
    for model_name in ["CNN2D_LSTM_TF", "BiLSTM", "Transformer"]:

        model_key = f"{model_name}_{key}"
        if model_key in processed_models:
            print(f"ATTENZIONE: Il modello {model_name} per il dataset {key} è già stato addestrato! Salto iterazione...")
            continue
        processed_models.add(model_key)
        
        print(f"\nPreparazione dati per il dataset \033[1m{key}\033[0m e il modello \033[1m{model_name}\033[0m...")
        
        # Prova a caricare la configurazione e i pesi ottimali dal file .pkl
        
        '''
        load_config_if_available --> prende in input 'key' che è la chiave composita (i.e, th_resp_vs_pt_resp_1_20_familiar_th)
        parse_combination_key --> prende in input 'key' che suddivide la chiave composita in stringhe separate
        
        exp_cond, data_type, category_subject che sfrutto per crearmi la directory path che mi servirà per caricarmi 
        pesi del modello e i suoi iper-parametri
        
        Diciamo che in questo caso, sfrutto 'parse_combination_key per qualcosa che serve a 'load_config_if_available' in modo IMPLICITO..
        '''
        
        config, best_weights = load_config_if_available(key, model_name, base_folder)
        
        if config is None:
            raise ValueError(f"\033[1mNessun file .pkl trovato per {model_name} su {key}\033[0m. Non posso procedere senza la configurazione ottimale.")
        
        '''
        Successivamente, queste variabili vengono invece create in maniera ESPLICITA per fasi successive del loop
        MA in questo caso, parsifica la chiave una VOLTA SOLA e memorizza i valori!
        '''
        
        # Parsifica la chiave una volta sola e memorizza i valori
        exp_cond, data_type, category_subject = parse_combination_key(key)
        
        '''
        Dpodiché, 
        
        1) si carica i vari valori degli iper-parametri,
        2) si esegue la standardizzazione se servisse,
        3) prepara il modello per la divisione in train_loader etc.,
        4) si carica la configurazione dei pesi del modello, 
        5) assegna i vari valori degli iper-parametri del modello corrente per la combinazione di dati correntemente iterata 
        
        6) esegue il training e il test e poi
        
        7) si salva il tutto nella path corrispondente...
        
        
        
        
        Ricordati di aggiungere le varie variabili associate alla definizione dinamica dei valori dei parametri model-specific qui!
    
        
        
        '''
        
        '''
        PER DARE UNIFORMITÀ AL CODICE, CAMBIO IL NOME DELLE VARIABILI, CHE CONTENGONO I VALORI OTTIMIZZATI 
        DA FORNIRE IN INPUT ALLE VARIE FUNZIONI CHE SONO RICHIAMATE NEL LOOP'''
        
        
        model_lr = config["lr"]
        model_weight_decay = config["weight_decay"]
        model_n_epochs = config["n_epochs"]
        model_patience = config["patience"]
        
        
        model_batch_size = config["batch_size"]
        model_standardization = config["standardization"]
        
        #model_n_epochs = config["n_epochs"]
        #model_patience = config["patience"]
        
        #model_lr = config["lr"]
        
        '''NUOVE MODIFICHE'''
        model_beta1 =  config["beta1"]
        model_beta2 =  config["beta2"]
        model_eps = config["eps"]
        
        
        
        #model_weight_decay = config["weight_decay"]
        #model_standardization = config["standardization"]
        
        #print(f"Parametri per \033[1m{model_name}\033[0m: batch_size= \033[1m{model_batch_size}\033[0m, n_epochs= \033[1m{model_n_epochs}\033[0m, patience= \033[1m{model_patience}\033[0m, lr= \033[1m{model_lr}\033[0m, weight_decay= \033[1m{model_weight_decay}\033[0m, standardization= \033[1m{model_standardization}\033[0m")
        print(f"Parametri per \033[1m{model_name}\033[0m: batch_size= \033[1m{model_batch_size}\033[0m, n_epochs= \033[1m{model_n_epochs}\033[0m, patience= \033[1m{model_patience}\033[0m, lr= \033[1m{model_lr}\033[0m, model_beta1= \033[1m{model_beta1}\033[0m,  model_beta2= \033[1m{model_beta2}\033[0m,  model_eps= \033[1m{model_eps}\033[0m, standardization= \033[1m{model_standardization}\033[0m")
        
        # Salva nel dizionario se per quella combinazione è stata applicata la standardizzazione ai dati
        models_info[model_key] = {"standardization": model_standardization}
        
        
        # 3) dataset & loader per test set (per plots power raw) –‑  IMPORTANTISSIMO: shuffle=False
        raw_dataset = TensorDataset(X_raw_tensor, y_raw_tensor)
        test_loader_raw = DataLoader(raw_dataset,
                             batch_size=model_batch_size,
                             shuffle=False)
        
        
        
        '''PER MANTENERE LA STESSA LOGICA DEL CODICE (ANCHE SE POTREI INSERIRLA DENTRO PREPARE_DATA_FOR_MODEL MODIFICANDO LA FUNZIONE (SI VEDA IN CELLA SOPRA COME)
        IMPONGONO LA STANDARDIZZAZIONE PRIMA DI QUESTA FUNZIONE
        '''

        if model_standardization:
            X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
            print(f"\033[1mSÌ Standardizzazione Dati!\033[0m")
        else:
            print(f"\033[1mNO Standardizzazione Dati!\033[0m")
        
        # Sposta il modello sulla GPU (se disponibile)
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

        
        # Preparazione dei dataloaders
        train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
            X_train, X_val, X_test, y_train, y_val, y_test, model_type = model_name, batch_size = model_batch_size)
        
        # Inizializzazione del modello
        #if model_name == "CNN2D":
        #    model = CNN2D(input_channels=64, num_classes=2)
        
        '''
        # ====== MODELLI TIME×FREQ ======
    
        cnn_model = CNN2D_LSTM_TF(input_channels = input_channels, num_classes =num_classes, dropout = dropout)
        hidden_sizes = [24, 48, 62]
        lstm_model = ReadMEndYou(input_size=input_channels * num_freqs, hidden_sizes=hidden_sizes, output_size=num_classes)
        transformer_model = ReadMYMind(d_model=d_model, num_heads=num_heads, num_layers=num_layers, num_classes=num_classes)

        # Creazione di dati fittizi per il test
        x = torch.randn(batch_size, input_channels, num_freqs, num_timepoints)  # (batch, canali, frequenze, tempo)
        

        # ricavo channels e freqs dai dati (shape attesa: N, C, F, T)
        channels, freqs = int(X_train.shape[1]), int(X_train.shape[2])

        if config.model_name == "CNN2D_LSTM_TF":

            #OCCHIO QUI ADESSO SAREBBE TEMPO x FREQUENZA

            model = CNN2D_LSTM_TF(
                input_channels = channels, # qui 64
                num_classes = 2,
                dropout=config.dropout,
            )

            print(f"\nInizializzazione Modello \033[1mCNN2D_LSTM_TF\033[0m")


        elif config.model_name == "BiLSTM":
            hidden_sizes = [24, 48, 62]
            model = ReadMEndYou(
                input_size = channels * freqs, #  qui 64*81
                hidden_sizes = hidden_sizes,
                output_size = 2,
                dropout = config.dropout,
                bidirectional = config.bidirectional
            )
            print(f"\nInizializzazione Modello \033[1mReadMEndYou (BiLSTM)\033[0m")

        elif config.model_name == "Transformer":
            model = ReadMYMind(
                d_model=config.d_model,
                num_heads=config.num_heads,
                num_layers=config.num_layers,
                num_classes=2,
                channels = channels,
                freqs = freqs
            )

            print(f"\nInizializzazione Modello \033[1mReadMYMind (Transformer)\033[0m")
        else:
            raise ValueError(f"Modello sconosciuto: {config.model_name}")
            
        
        '''
        
        #channels, freqs = int(X_train.shape[1]), int(X_train.shape[2])
        
        #sweep_config_cnn2d_lstm_tf
        #sweep_config_bilstm
        #sweep_config_transformer
        
        '''PARAMETRI MODEL-SPECIFIC DI CNN2D_LSTM_TF, BiLSTM e Transformer, ma richiamati al momenti della inizializzazione deii relativi sweep_config! '''
        
        #model_dropout = config['dropout']
        #model_bidirectional = config['bidirectional']
        #model_d_model = config['d_model']
        #model_num_heads = config['num_heads']
        #model_num_layers  = config['num_layers']
        
        
        channels, freqs = int(X_train.shape[1]), int(X_train.shape[2])
        
        if model_name == "CNN2D_LSTM_TF":
            
            sweep_config = sweep_config_cnn2d_lstm_tf
            
            model_dropout = float(config.get('dropout', 0.5))
            
            
            model = CNN2D_LSTM_TF(input_channels = channels, num_classes = 2, dropout = model_dropout) #input_channels = 64
            
        elif model_name == "BiLSTM":
            
            sweep_config = sweep_config_bilstm
            
            model_dropout = float(config.get('dropout', 0.5))
            model_bidirectional = bool(config.get('bidirectional', False))
            
            #input_size= 64 * 81
            model = ReadMEndYou(input_size= channels * freqs, hidden_sizes=[24, 48, 62], output_size=2, dropout = model_dropout, bidirectional = model_bidirectional)
            
        elif model_name == "Transformer":
            
            sweep_config = sweep_config_transformer
            
            model_d_model = int(config.get('d_model', 32))
            model_num_heads = int(config.get('num_heads', 2))
            model_num_layers  = int(config.get('num_layers', 2))
            
            model = ReadMYMind(d_model = model_d_model, num_heads = model_num_heads, num_layers = model_num_layers, num_classes=2, channels = channels, freqs= freqs) #channels = 64, freqs=81
        else:
            raise ValueError(f"Modello {model_name} non riconosciuto.")
        
        # Se abbiamo caricato i pesi ottimali, li carichiamo nel modello
        if best_weights is not None:
            try:
                model.load_state_dict(best_weights)
                print(f"📊 Modello \033[1m{model_name}\033[0m inizializzato con \033[01i pesi ottimizzati\033[0m tramite hyper-parameter tuning su \033[1mWeight & Biases\033[0m")
            except Exception as e:
                print(f"⚠️Errore nel caricamento dei pesi per {model_name} su {key}: {e}")
                continue
        
        
        '''NUOVE MODIFICHE'''
        # Definizione del criterio di perdita
        criterion = nn.CrossEntropyLoss(weight = class_weights_tensor)
        
        '''OLD VERSION'''
        # Definizione dell'ottimizzatore con i parametri aggiornati
        #optimizer = torch.optim.Adam(model.parameters(), lr = model_lr, weight_decay = model_weight_decay)
        
        '''NUOVE MODIFICHE'''
        
         # 10) ottimizzatore + scheduler + early stopping
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr    = model_lr,
            betas = (model_beta1, model_beta2),
            eps   = model_eps,
            weight_decay = model_weight_decay
            
        )
            
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode     = "min",   # monitoriamo val_loss
            factor   = 0.1,
            patience = 8,
            verbose  = True
        )
        early_stopping = EarlyStopping(patience=model_patience, mode="min")
        
        
        #criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
        
        
        '''OLD VERSION'''
        #print(f"🏋️‍♂️Avvio del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        #my_train_results = training(model, train_loader, val_loader, optimizer, criterion, n_epochs = model_n_epochs, patience = model_patience)
    
        #print(f"Avvio del testing per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        #my_test_results = testing(my_train_results, test_loader, criterion)
        
        '''NEW VERSION'''
        # --- dopo model.load_state_dict(best_weights) e criterion = nn.CrossEntropyLoss() ---

        # 1) prepara i data_loaders per train/val
        data_loaders = {
            "train": train_loader,
            "val":   val_loader
        }
        
        print(f"🏋️‍♂️Salvo le metriche del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m a seguito della ottimizzazione su W&B ...")
        # 2) richiama la funzione che pesca da W&B la best‐run e corregge la train AUC
        
        #ATTENZIONE al potenziale problema di stringa, non di API: 

        #i due esempi che hai postato in realtà usano diversi caratteri “‑” (uno è il classico ASCII U+002D, l’altro è un non‑breaking hyphen U+2011 o simili), quindi quando chiami
        
        #entity = "stefano‑bargione‑universit‑di‑roma‑tor‑vergata"
        #stai passando un nome che W&B non riconosce (e quindi api.projects(entity=…) torna vuoto), mentre con

        #entity = "stefano-bargione-universit-di-roma-tor-vergata"
        #funziona perché lì usi i semplici - ASCII.

        my_train_results = load_best_run_results(
            key=key,
            model = model,
            sweep_config = sweep_config,
            data_loaders = data_loaders,
            entity = "stefano-bargione-universit-di-roma-tor-vergata"
        )
        
        
        '''
        L’entity che passi a Api().runs(f"{entity}/{project}") è semplicemente il tuo account (o l’organizzazione) su W&B,
        cioè la parte che compare subito prima del nome del progetto nell’URL.

        Per esempio, se quando apri il tuo progetto su W&B vedi un indirizzo del tipo
        
        -> https://wandb.ai/steclab/some_project_name, allora entity = "steclab".
        
        Se invece lavori sotto un’organizzazione 
        
        -> “cool‑team”, e l’URL è https://wandb.ai/cool-team/some_project_name, allora userai entity = "cool-team".

        Puoi verificarlo:

        Accedi a wandb.ai e vai sul progetto.
        Leggi la prima parte dell’URL (tra wandb.ai/ e il /project_name).
        Copiala esattamente come stringa in entity.

        Così il tuo Api().runs(f"{entity}/{project}") andrà a pescare proprio le run che hai lanciato tu.

        my_train_results = load_best_run_results(
            key= key,
            model = model,
            sweep_config = sweep_config,
            data_loaders = data_loaders,
            entity= "mio-entity"
        )
        
        '''
        
        print(f"Avvio del testing per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        # 3) usa il best_model caricato dentro `train_results` e chiama il testing
        my_test_results = testing(my_train_results, test_loader, criterion)
        
        
        
        '''
        GRADCAM COMPUTATION PER IL MODELLO CNN2D_LSTM_TF
        
        La funzione compute_gradcam_figure estrae due campioni (uno per ogni classe) e crea una figura con le due righe richieste.
        
        Il parametro gradcam_image (un buffer binario o un'immagine) viene passato alla funzione di salvataggio, 
        'save_performance_results', in modo da essere salvato nella path corretta. 
        
        La funzione 'save_performance_results' è stata modificata 
        per gestire ANCHE questo nuovo input dell'immagine 
        
        (ossia, per salvare il file con un nome che inizia con 'GradCAM_results_'
        seguito da tutte le altre stringhe corrispondenti alla combinazione di fattori che costituiscono il dataset corrente:
        
        - coppia di condizioni sperimentali da cui provengono i dati (i.e., th_resp_vs_pt_resp )
        - tipologia di dato EEG prelevato (i.e., spectrograms) 
        - provenienza del dato stesso (i.e., familiar_th)
        )
        
        Spiegazione:
        
        La funzione compute_gradcam_figure eseguire il calcolo di GradCAM (vedi dettagli nella sua funzione)
        e alla fine ritornerà in output una variabile 
        
        'fig_image' che sarà poi assegnata alla variabile 'gradcam_image',
        che è un oggetto buffer, che contiene i dati binari dell'immagine in formato PNG
        (poiché abbiamo usato plt.savefig con format='png'). 
        
        Quindi, quando passi gradcam_image (cioè fig_image) alla funzione 'save_performance_results',
        viene scritto direttamente su disco come file PNG.
        
        Non c'è bisogno di ri-aprire o convertire ulteriormente, a meno che tu non voglia manipolare l'immagine in seguito.
        Quindi, la soluzione è corretta così com'è:
        il buffer viene salvato come file PNG nella directory specificata, 
        e successivamente potrai aprirlo con una libreria come cv2 o PIL se necessario.        
        
        Quindi, gradcam_image (i.e., fig_image) viene quindi passato correttamente dentro al loop di training e test, 
        tramite 'save_performance_results', come input, 
        che salverà quindi poi l'immagine nella path corrispondente 

        '''
        
        # Se il modello è CNN2D, calcola anche GradCAM per la visualizzazione
        gradcam_image = None
        
        #if model_name == "CNN2D":
        
        '''ATTENZIONE MODIFICA QUI'''
        
        if model_name == "CNN2D_LSTM_TF":
            
            '''
            ATTENZIONE! Qui ho aggiunto alla nuova versione di "compute_gradcam_figure" (versione del 17/09/2025)
            il test_loader_raw tra gli argomenti della funzione!
            '''
            gradcam_image = compute_gradcam_figure(model, test_loader, test_loader_raw, exp_cond, data_type, category_subject, device)
            if gradcam_image is not None:
                print(f"Creazione di \033[1mGradCAM Image\033[0m per il modello \033[1m{model_name}\033[0m.")
                
        print(f"Salvataggio dei risultati per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        save_performance_results(model_name,
                                 my_train_results,
                                 my_test_results,
                                 key,
                                 exp_cond,
                                 model_standardization,
                                 base_folder = save_path_folder,
                                 gradcam_image = gradcam_image)
        
        
        '''
        N.B
        
        gradcam_image = None avverrà solo all'inizio cioè per il primo modello CNN2D, che verrà testato con una certa combinazione di dati mi sa.. 
        ma servirebbe tracciare in qualche modo 

        1) o che la gradcam_image di ogni combinazione venga ri-azzerata alla fine loop
        2) o che venga monitorato che gradcam_image di una combinazione di dati già analizzata venga esclusa poi
        (o messa in un set) in modo che rivenga per errore sovrascritta più volte.. 
        
        Forse la strada più veloce potrebbe essere la soluzione 1)
        
        La soluzione più veloce e semplice è reimpostare la variabile gradcam_image a None alla fine dell'iterazione per ogni combinazione di dati
        (cioè, all'interno del ciclo esterno che itera su key). 
         
        In questo modo, per ogni nuovo dataset la variabile viene "azzera" e viene calcolata l'immagine GradCAM solo per quella combinazione, 
        evitando di sovrascrivere accidentalmente i risultati già calcolati per combinazioni precedenti.
         
        Un'altra possibilità sarebbe tenere traccia delle chiavi (o combinazioni) per cui hai già calcolato la GradCAM,
        ad esempio usando un set, e saltare il calcolo se la combinazione è già presente. 
        
        Tuttavia, se ogni combinazione deve avere la sua immagine, 
        la soluzione più semplice è quella di reimpostare gradcam_image = None alla fine dell'iterazione.
        
        Quindi, per esempio, alla fine del ciclo per ogni dataset (key) potresti fare:
        (VEDI SOTTO)
        
        In questo modo, ti assicuri che per ogni nuova combinazione la variabile sia pulita e pronta per essere ricalcolata, 
        senza rischio di sovrascrivere o confondere i risultati
        '''
        
        # Reimposta gradcam_image a None per la prossima combinazione di dati
        gradcam_image = None
    

In [ ]:
print("finito")

In [ ]:
print(models_info.keys())

In [ ]:
with open('/home/stefano/Interrogait/spectrograms_best_models_post_WB_GradCAM_Checks_motor_task_1_new.pkl', 'wb') as f:
    pickle.dump(models_info, f)

#### **CREAZIONE DELLE TABLES CON INTEGRAZIONE DELLE PERFORMANCE TRAINING & TEST DEI MODELLI DENTRO DATAFRAME**

#### **Integrazioni in Tabella delle Performance Training e Test del Modello dentro DataFrame - NEW APPROACH**

#### Spiegazione

Ok in questo modo, model_standardization_dict dovrebbe andare a salvarsi se, i dati per quella combinazione di fattori, rispetto ad uno specifico modello, siano stati standardizzati o meno.

Di conseguenza, dentro questo loop

    import os
    import pickle
    import pandas as pd
    import matplotlib.pyplot as plt
    from pandas.plotting import table

    # Base folder
    base_folder = "/home/stefano/Interrogait/time_domain_best_models_post_WB"

    # Condizioni sperimentali
    experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

    # Tipologie di dati
    data_types = ["1_20", "1_45", "wavelet_delta"]

    # Subfolders per tipologia di soggetto
    subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

    # Dizionario per salvare tutti i modelli
    all_models = {}

    # Caricamento dei modelli
    for condition in experimental_conditions:
        for data_type in data_types:
            for subfolder in subfolders:

                path = os.path.join(base_folder, condition, data_type, subfolder)

                if not os.path.exists(path):
                    print(f"Directory non trovata: {path}")
                    continue

                # Creiamo la chiave per questa combinazione
                key = f"{condition}_{data_type}_{subfolder}"
                all_models[key] = {}

                # Otteniamo la lista di file nella directory
                files = os.listdir(path)

                # Filtriamo e carichiamo i file .pkl
                for file in files:
                    if file.endswith(".pkl"):  # Controlliamo se è un file modello
                        file_path = os.path.join(path, file)
                        try:
                            with open(file_path, "rb") as f:
                                all_models[key][file] = pickle.load(f)
                        except Exception as e:
                            print(f"Errore nel caricamento di {file}: {e}")

    # Creazione delle tabelle di performance
    for key, models_dict in all_models.items():

        # Otteniamo le informazioni dalla chiave
        #condition, data_type, subfolder = key.split("_", 2)
        condition, data_type, subfolder = parse_combination_models_keys(key)

        print(f"\nProcessing: \033[1m{condition}\033[0m - \033[1m{data_type}\033[0m - \033[1m{subfolder}\033[0m\n")

        # Creazione della tabella
        df_data = {"Metriche": ["Accuracy", "Loss", "Precision", "Recall", "F1-Score", "AUC-ROC"]}

        # Iteriamo sui modelli caricati
        for model_name, model_data in models_dict.items():
            name_model = model_name.split("_")[0]  # Nome modello
            print(f"    Processing model: \033[1m{name_model}\033[0m")

            try:
                # Recupera i risultati di training e testing
                train_scores = model_data.get('my_train_results', {}).get('training_performances', {})
                test_scores = model_data.get('my_test_results', {}).get('test_performances', {})

                # Converti i valori in float
                train_scores = {key: float(value[0]) for key, value in train_scores.items()}
                test_scores = {key: float(value[0]) for key, value in test_scores.items()}


                # Aggiungi le metriche di training
                df_data[f"{name_model} (Training)"] = [
                    train_scores["train_accuracy"],
                    train_scores["train_loss"],
                    train_scores["train_precision"],
                    train_scores["train_recall"],
                    train_scores["train_f1_score"],
                    train_scores["train_auc"],
                ]

                # Aggiungi le metriche di test
                df_data[f"{name_model} (Testing)"] = [
                    test_scores["test_accuracy"],
                    test_scores["test_loss"],
                    test_scores["test_precision"],
                    test_scores["test_recall"],
                    test_scores["test_f1_score"],
                    test_scores["test_auc"],
                ]


            except Exception as e:
                print(f"    Errore nell'elaborazione di {model_name}: {e}")

        # Creazione del DataFrame
        #df_performances = pd.DataFrame(df_data)

        # Crea un'immagine della tabella
        #fig, ax = plt.subplots(figsize=(10, 6))
        #ax.axis('off')
        #tabla = table(ax, df_performances, loc='center', colWidths=[0.2] * len(df_performances.columns))
        #tabla.auto_set_font_size(True)
        #tabla.set_fontsize(10)
        #tabla.scale(2, 2)

        # Evidenzia i nomi delle colonne
        #for key, cell in tabla.get_celld().items():
        #    if key[0] == 0:
        #        cell.set_text_props(weight='bold')

        # Salva l'immagine della tabella
        path = os.path.join(base_folder, condition, data_type, subfolder)
        file_name = f"models_performances_{condition}_{data_type}_{subfolder}.png"
        img_file_path = os.path.join(path, file_name)
        #fig.savefig(img_file_path, bbox_inches='tight', dpi=300)
        #plt.close(fig)

        print(f"\nTabella dei dati di \033[1m{key}\033[0m salvati in: \n\033[1m{img_file_path}\033[0m")


vorrei provare ad iterare con "zip", sia all_models che su model_standardization_dict ...? (che forse dovrebbero avere la stessa struttura, che renderebbe possibile questa cosa...?)

E, nel momento in cui si aggiungono le metriche del training e test del relativo modello, controllare rispetto a model_standardization_dict (di cui si ha la chiave per accedere all' informazione su se quel modello, per quella combinazioni di fattori che compongono quel dato) se il dato sia stato standardizzato... 

Se questo è VERO, allora nella colonna del dataframe che si riferisce al modello... vorrei che ci mettessi accanto, alla stringa che si riferisce al nome del modello (name_model) un asterisco, SOLO SE, per quel modello, allenato con quella combinazioni di fattori che compongono quel dato, i dati siano stati standardizzati...

chiaro?

#### Implementazione 

In [ ]:
import pickle 
path = '/home/stefano/Interrogait/'

with open(f"{path}spectrograms_best_models_post_WB_GradCAM_Checks_motor_task_1_new.pkl", "rb") as f:
    models_info = pickle.load(f)

In [ ]:
'''
In questo codice:

model_info.get('standardization', False) cerca la chiave 'standardization' all'interno di ogni sottodizionario. 
Se non esiste, restituirà False come valore di default.
Se standardization è True, stampa la chiave associata.
'''

# Ciclo attraverso le chiavi di 'models_info'
for key, model_info in models_info.items():
    # Controllo se 'standardization' è True
    if model_info.get('standardization', False):  # Default a False nel caso in cui non esista la chiave
        print(key)  # Stampa la chiave



In [ ]:
models_info

In [ ]:
#for key, model_info in all_models.items():
#    print(key)

In [ ]:
'''
Siccome la stringa associata alla category subject è diversa tra i due.. 

familiar_th  familiar_pt unfamiliar_pt unfamiliar_pt  da un lato (models_info)
th_fam, pt_fam, th_unfam, pt_unfam  dall'altro (all_models)

la corrispondenza non avverrà mai... per cui, si deve fare il mapping corrispondente tra 
le stringhe di uno e dell'altro, in modo che models_info cambi come parte della stringa della sua chiave da queste 

familiar_th  familiar_pt unfamiliar_pt unfamiliar_pt
a queste
th_fam, pt_fam, th_unfam, pt_unfam 

'''

mapping_subject = {
    "familiar_th": "th_fam",
    "familiar_pt": "pt_fam",
    "unfamiliar_th": "th_unfam",
    "unfamiliar_pt": "pt_unfam"
}

# Creiamo un nuovo dizionario con le chiavi corrette
updated_models_info = {}

for key, value in models_info.items():
    for old_suffix, new_suffix in mapping_subject.items():
        if key.endswith(old_suffix):
            new_key = key.replace(old_suffix, new_suffix)
            updated_models_info[new_key] = value
            break  # Evita sostituzioni multiple se una è già stata fatta
    else:
        # Se nessuna sostituzione è stata fatta, mantieni la chiave originale
        updated_models_info[key] = value

# Sostituisci il vecchio dizionario con quello aggiornato
models_info = updated_models_info


In [ ]:
models_info.keys()

In [ ]:
''' Ciclo attraverso le chiavi di 'models_info' AGGIORNATO!'''

for key, model_info in models_info.items():
    # Controllo se 'standardization' è True
    if model_info.get('standardization', False):  # Default a False nel caso in cui non esista la chiave
        print(key)  # Stampa la chiavi

In [ ]:
'''
Parsing della chiave e costruzione del path:
Usando la funzione parse_combination_key si estraggono 

exp_cond, data_type e category_subject dalla chiave del dataset. 

Questi vengono usati per costruire il percorso in cui cercare i file .pkl.
'''

# Funzione per parsare la chiave
def parse_combination_models_keys(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    
    Il formato atteso PRIMA è:
    
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ spectrograms" _ "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    
    Il formato atteso ORA è:
    
     "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ spectrograms" _ "th_fam|th_unfam|pt_fam|pt_unfam"
     
    """
    #r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(th_fam|th_unfam|pt_fam|pt_unfam)$", 
    
    match = re.match(
        r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_(spectrograms)_(th_fam|th_unfam|pt_fam|pt_unfam)$",
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        
    return exp_cond, data_type, category_subject

In [ ]:
'''
NEW APPROACH 

Adesso replichiamo l'approccio usato prima, ma stavolta integrado tutte le combinazioni di dati. 
Andiamo a

1) iterare sulla struttura delle directory a partire da base_folder, 
2) caricare i modelli .pkl per ogni combinazione di fattori che compongono i dati
3) creare un DataFrame che raccolga le metriche di tutti i modelli relativi alla stessa combinazione di dati. 

Infine, salviamo questa tabella come immagine all'interno della cartella corrispondente
'''


import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table



# Base folder
#base_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB"

base_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB_GradCAM_Checks_motor_task_1_new"
                                        #spectrograms_best_models_post_WB_GradCAM_Checks_motor_task_1_new

# Condizioni sperimentali
#experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

experimental_conditions = ["rest_vs_left_fist", "rest_vs_right_fist", "left_fist_vs_right_fist"]


# Tipologie di dati
data_types = ["spectrograms"]

# Subfolders per tipologia di soggetto
#subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

subfolders = ["th_fam"]

# Dizionario per salvare tutti i modelli
all_models = {}

# Caricamento dei modelli
for condition in experimental_conditions:
    for data_type in data_types:
        for subfolder in subfolders:
            
            path = os.path.join(base_folder, condition, data_type, subfolder)
            
            if not os.path.exists(path):
                print(f"Directory non trovata: {path}")
                continue
            
            # Creiamo la chiave per questa combinazione
            key = f"{condition}_{data_type}_{subfolder}"
            all_models[key] = {}

            # Otteniamo la lista di file nella directory
            files = os.listdir(path)
            
            # Filtriamo e carichiamo i file .pkl
            for file in files:
                if file.endswith(".pkl"):  # Controlliamo se è un file modello
                    file_path = os.path.join(path, file)
                    try:
                        with open(file_path, "rb") as f:
                            all_models[key][file] = pickle.load(f)
                    except Exception as e:
                        print(f"Errore nel caricamento di {file}: {e}")

# Creazione delle tabelle di performance
for key, models_dict in all_models.items():
    
    # Otteniamo le informazioni dalla chiave
    condition, data_type, subfolder = parse_combination_models_keys(key)
    
    print(f"\nProcessing: \033[1m{condition}\033[0m - \033[1m{data_type}\033[0m - \033[1m{subfolder}\033[0m\n")
    
    # Creazione della tabella
    df_data = {"Metriche": ["Accuracy", "Loss", "Precision", "Recall", "F1-Score", "AUC-ROC"]}

    # Iteriamo sui modelli caricati
    for model_name, model_data in models_dict.items():
        
        # Estrai il nome del modello dal file (ad esempio, "CNN1D" da "CNN1D_performances_...pkl")
        #name_model = model_name.split("_")[0]
        
        if model_name.startswith(("CNN2D_LSTM")):
            
            # es. “CNN3D_LSTM_FC”  →  split → [“CNN3D”, “LSTM”, “FC”] → prendi i primi 2 elementi
            parts = model_name.split("_")
            name_model = "_".join(parts[:2])      # “CNN3D_LSTM” o “SeparableCNN2D_LSTM”
        else:
            name_model = model_name.split("_")[0]  # Prende solo CNN1D, CNN2D
        
        print(f"    Processing model: \033[1m{name_model}\033[0m")
        
        # Costruisci la chiave utilizzata nel dizionario models_info
        
        '''
        Nota: occorrerà che il formato della chiave sia consistente tra i due loop.
        
        Ad esempio, se nel primo loop era f"{key}_{model_name}", qui potresti dover fare:
        model_key = f"{key}_{name_model}"
        
        Oppure, se nel primo loop era f"{model_name}_{key}", qui potresti dover fare:
        model_key = f"{name_model}_{key}"
        
        '''
        model_key = f"{name_model}_{key}"
        
        # Controlla se i dati sono stati standardizzati per questo modello
        standardization_flag = models_info.get(model_key, {}).get("standardization", False)
        
        if standardization_flag:
            #suffix = "*" 
            suffix = "" 
        else:
            suffix = "" 
        
        try:
            # Recupera i risultati di training e testing
            train_scores = model_data.get('my_train_results', {}).get('training_performances', {})
            
            val_scores = model_data.get('my_train_results', {}).get('validation_performances', {})
            
            test_scores = model_data.get('my_test_results', {}).get('test_performances', {})
            
            # Converti i valori in float
            train_scores = {key: float(value[0]) for key, value in train_scores.items()}
            
            val_scores = {key: float(value[0]) for key, value in val_scores.items()}
            
            
            test_scores = {key: float(value[0]) for key, value in test_scores.items()}
            
            
            # Aggiunge le metriche di training, modificando il nome della colonna se è vera la condizione
            col_train = f"{name_model} (Training){suffix}"  # Usa suffix qui per il nome
            
            df_data[f"{col_train}"] = [
                train_scores["train_accuracy"],
                train_scores["train_loss"],
                train_scores["train_precision"],
                train_scores["train_recall"],
                train_scores["train_f1_score"],
                train_scores["train_auc"],
            ]
            
            col_val = f"{name_model} (Validation){suffix}"  # Usa suffix qui per il nome
            
            
            df_data[f"{col_val}"] = [
                val_scores["val_accuracy"],
                val_scores["val_loss"],
                val_scores["val_precision"],
                val_scores["val_recall"],
                val_scores["val_f1_score"],
                val_scores["val_auc"],
            ]

            # Aggiunge le metriche di training, modificando il nome della colonna se è vera la condizione
            col_test = f"{name_model} (Test){suffix}"  # Usa suffix qui per il nome
            
            df_data[f"{col_test}"] = [
                test_scores["test_accuracy"],
                test_scores["test_loss"],
                test_scores["test_precision"],
                test_scores["test_recall"],
                test_scores["test_f1_score"],
                test_scores["test_auc"],
            ]
        
        except Exception as e:
            print(f"    Errore nell'elaborazione di {model_name}: {e}")

    # Creazione del DataFrame
    df_performances = pd.DataFrame(df_data)

    # Crea un'immagine della tabella
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.axis('off')
    
    # Aggiunta del titolo
    #title = f"DL Models performances for Exp Conditions: {condition}, EEG data: {data_type}, Subject: {subfolder}"
    title = f"DL Models performances for Exp Conditions: {condition}, EEG data: {data_type}"
    ax.set_title(title, fontsize=12, fontweight="bold", pad=20)

    tabla = table(ax, df_performances, loc='center', colWidths=[0.2] * len(df_performances.columns))
    tabla.auto_set_font_size(True)
    tabla.set_fontsize(10)
    tabla.scale(2, 2)

    # Evidenzia i nomi delle colonne
    for key, cell in tabla.get_celld().items():
        if key[0] == 0:
            cell.set_text_props(weight='bold')

    # Salva l'immagine della tabella
    path = os.path.join(base_folder, condition, data_type, subfolder)
    file_name = f"models_performances_{condition}_{data_type}_{subfolder}.png"
    img_file_path = os.path.join(path, file_name)
    fig.savefig(img_file_path, bbox_inches='tight', dpi=300)
    plt.close(fig)

    print(f"\nTabella dei dati di \033[1m{condition}_{data_type}_{subfolder}\033[0m salvati in: \n\033[1m{img_file_path}\033[0m")

## Impostazione **Weight & Biases DL Training** con **Rappresentazione Frequenza-Elettrodi dei miei dati EEG** a seconda del Dataset del Task scelto


Notebook **"Optimization Weight and Biases Time Domain EEG Spectrograms"**

- Task 1: Motor Execution (Open and close right or left fist) - Unilateral - Runs 3, 7, 11 
- Task 2: Motor Imagery (Imagine opening and closing right and left fists) - Unilateral - Runs 4, 8, 12 
- Task 3: Motor Execution (Open and close both fists and feet) - Bilateral - Runs 5, 9, 13 
- Task 4: Motor Imagery (Imagine opening and closing both fists and feet) - Bilateral - Runs 6, 10, 14 

### Optimization Weight and Biases - EEG Spectrograms - Electrodes x Frequencies - CNN2D_LSTM

### (**Solo Iper-parametri Dinamici**)

#### **Weight & Biases Login & REMOTE CHECKS - EEG Spectrograms - Electrodes x Frequencies**

##### **Weight & Biases Login PRECEDURA CORRETTA ✅ (CANCELLAZIONE RUNS e NON SWEEPS!) - EEG Spectrograms - Electrodes x Frequencies V1**

In [ ]:
'''

Puoi usare l’API Python di W&B per elencare da remoto tutti i progetti di una certa entity (tuo utente o organizzazione).

Ad esempio:

'''

import wandb

# 1) autenticati (se non l’hai già fatto con wandb.login())
# wandb.login()

# 2) crea un’istanza dell’API
api = wandb.Api()

# 3) scegli la tua entity (nome utente o org). 
#    Se omesso, prende la tua entity di default.

entity = 'stefano-bargione-universit-di-roma-tor-vergata'

# 4) prendi la lista dei progetti
projects = api.projects(entity=entity)

# 5) stampa i nomi
for proj in projects:
    print(proj.name)

In [ ]:
###TUTTE ASSIEME


import wandb

# Nomi dei progetti
project_names = [
    #'left_fist_vs_right_fist_spectrograms_channels_freqs_new_2d_grid_multiband',
    #'rest_vs_right_fist_spectrograms_channels_freqs_new_2d_grid_multiband',
    #'rest_vs_left_fist_spectrograms_channels_freqs_new_2d_grid_multiband',
    
    #'left_fist_vs_right_fist_spectrograms_channels_freqs_new_imagery_2d_grid_multiband'
    #'rest_vs_right_fist_spectrograms_channels_freqs_new_imagery_2d_grid_multiband',
    #'rest_vs_left_fist_spectrograms_channels_freqs_new_imagery_2d_grid_multiband',
    
    #'both_feet_vs_both_fists_spectrograms_channels_freqs_new_imagery_2d_grid_multiband',
    #'rest_vs_both_fists_spectrograms_channels_freqs_new_imagery_2d_grid_multiband',
    #'rest_vs_both_feet_spectrograms_channels_freqs_new_imagery_2d_grid_multiband',
    
    #'both_feet_vs_both_fists_spectrograms_channels_freqs_new_2d_grid_multiband',
    #'rest_vs_both_fists_spectrograms_channels_freqs_new_2d_grid_multiband',
    #'rest_vs_both_feet_spectrograms_channels_freqs_new_2d_grid_multiband'

]

user_name = 'stefano-bargione-universit-di-roma-tor-vergata'

# Connessione all'API di W&B
api = wandb.Api()

# Itera su tutti i progetti
for project_name in project_names:
    print(f"\nSelezione delle run per il progetto: \033[1m{project_name}\033[0m")
    
    # Recupera tutte le run del progetto
    runs = api.runs(f"{user_name}/{project_name}")
    
    # Salva gli ID delle run da eliminare
    run_ids_to_delete = []
    
    for run in runs:
        #print(f"Estrazione run ID: {run.id} - Run Name: {run.name}")
        run_ids_to_delete.append(run.id)  # Aggiungi l'ID della run alla lista
    
    print(f"\nTotale runs da eliminare: \033[1m{len(run_ids_to_delete)}\033[0m")
    
    # Elimina le run
    for run_id in run_ids_to_delete:
        run = api.run(f"{user_name}/{project_name}/{run_id}")
        run.delete()  # Elimina la run
        #print(f"Eliminata la run con ID: {run_id}")
    print(f"Eliminazione runs completata")

        

##### **Weight & Biases Login PRECEDURA CORRETTA ✅ (CANCELLAZIONE RUNS e SWEEPS!) - EEG Spectrograms - Electrodes x Frequencies** V2

In [ ]:
'''UFFICIALE 19 LUGLIO

Spiegazione delle modifiche:
Comando per ottenere la lista degli sweep:


cmd_list = [
    sys.executable, "-m", "wandb", "sweep",
    "--project", path, "--list"
]
Qui usiamo --list per ottenere la lista di tutti gli sweep per il progetto.

Comando per cancellare uno sweep:


cmd_delete = [
    sys.executable, "-m", "wandb", "sweep",
    "--delete", f"{path}/{sweep_id}"
]
Qui usiamo --delete per eliminare ogni sweep individuato.

Verifica degli sweep dopo la cancellazione:

Una volta che abbiamo cancellato gli sweep, facciamo una seconda chiamata proj.sweeps() 
per verificare se ci sono ancora sweep attivi.

Come funziona:
Il codice esegue una lista di tutti i progetti che corrispondono ai pattern.

Per ogni progetto, le run vengono cancellate prima.

Poi, per ogni sweep associato al progetto, il comando wandb sweep --delete viene eseguito per eliminarlo.

Infine, viene effettuata una verifica se gli sweep sono stati effettivamente cancellati.

Con queste modifiche, dovresti essere in grado di eliminare completamente gli sweep oltre a fermarli.


'''

import sys
import subprocess
import wandb

# Configurazione API e entity
entity = "stefano-bargione-universit-di-roma-tor-vergata"
api = wandb.Api()

# 1) Pattern da cercare nel nome dei progetti
patterns = [
    "_spectrograms_channels_freqs_new_3d_grid_multiband",
    "_spectrograms_channels_freqs_new_imagery_3d_grid_multiband",
]

# 2) Prendi tutti i progetti e filtra
print("Recupero tutti i progetti dell'entity…")
all_projects = api.projects(entity=entity)
target_projects = [p for p in all_projects if any(pat in p.name for pat in patterns)]

if not target_projects:
    print("Nessun progetto corrispondente ai pattern trovati.")
    sys.exit(0)

# 3) Cicla sui progetti selezionati
for proj in target_projects:
    name = proj.name
    path = f"{entity}/{name}"
    print(f"\n→ Progetto: {path}")

    # 3.1) Elimina tutte le run (per_page=None fa paging automatico)
    try:
        runs = api.runs(path, per_page=None)
        runs = list(runs)  # forza il fetch di tutte le pagine
        if runs:
            print(f"   • Eliminando {len(runs)} run…")
            for run in runs:
                try:
                    run.delete()
                    print(f"     – Run {run.id} eliminata")
                except Exception as e:
                    print(f"     – Errore eliminando run {run.id}: {e}")
        else:
            print("   (nessuna run trovata)")
    except Exception as e:
        print(f"   ⚠️ Impossibile caricare le run: {e}")

    # 3.2) Cancella tutti gli sweep via CLI Python module
    # Evitiamo di chiamare un eseguibile esterno, usiamo `python -m wandb`
    # Lo stesso interprete che esegue questo script è in sys.executable
    
    cmd_list = [
        sys.executable, "-m", "wandb", "sweep",
        "--project", path, "--list"
    ]
    res = subprocess.run(cmd_list, capture_output=True, text=True)

    if res.returncode != 0 or not res.stdout.strip():
        print("   • Nessuno sweep trovato o progetto inesistente")
        continue

    # Ogni riga di res.stdout ha uno sweep_id come primo token
    for line in res.stdout.splitlines():
        sweep_id = line.split()[0]
        print(f"   • Cancello sweep {sweep_id}")
        cmd_delete = [
            sys.executable, "-m", "wandb", "sweep",
            "--delete", f"{path}/{sweep_id}"
        ]
        subprocess.run(cmd_delete, check=False)

    # Verifica che gli sweep siano stati cancellati
    print("Verifica sweep attivi dopo cancellazione...")
    sweeps = proj.sweeps()  # Riprova a ottenere gli sweep
    if not sweeps:
        print("   ✅ Nessuno sweep attivo trovato.")
    else:
        print(f"   ⚠️ Sweep ancora attivo: {sweeps}")

    print(f"  ✅ Run e sweep eliminati per {path}")


In [ ]:
print('finito')

#### **IO**

Prima avevo avviata una ricerca di iper-parametri con API da jupyter e avevo impostato la creazione di cartelle per ogni combinazione di dati...

tuttavia, ora quando andava ho notato che le cartelle non so state create alle fine.. come mai? è dovuto ad un overload di sovraccarica o è semplicemente un ritardo dovuto all'esecuzione di grande mole di dati da remoto tra l'altro con l'api che ritarda la sincronizzazione delle operazioni del codice su quello che dovrei vedere su jupyter? 

è fondamentale che le cartelle ed i vari files .pkl del mio codice si salvino, perché poi io le richiamo successivamente per estrarmi gli iper-parametri migliori di ogni ottimizzazione fatta.. devo gestire le risorse di calcolo in modo più equilibrato, affinché le operazioni di salvataggio vengano eseguite correttamente?

#### **CHATGPT**

È molto più probabile che la cartella ci sia già, ma:

Non si trovi sotto la radice che Jupyter ti mostra (di solito /mnt/data), bensì in /home/stefano/..., per cui il file‑browser non la “vede” fino a quando non navighi esplicitamente lì.

O che tu abbia settato WANDB_DIR dopo l’import wandb, quindi wandb abbia già inizializzato il suo percorso di default.

In ogni caso non è un problema di “troppa mole di dati che rallenta” o di W&B che perde file, bensì di:

percorso sbagliato in cui guardi

sincronizzazione asincrona di wandb (che salva localmente in run.dir e lo invia al cloud in background)

Controlla davvero se la cartella esiste
Nella cella del notebook, esegui:

bash
Copia
Modifica
!ls -l /home/stefano/Interrogait/WB_spectrograms_analyses_motor_task_1
o più genericamente

bash
Copia
Modifica
!find /home/stefano/Interrogait -maxdepth 2 -type d
Se la cartella esiste veramente, basta che la navighi nel file‑browser (“…” in alto a sinistra → “Change directory” → incolli quel path) o che tu salvi direttamente in /mnt/data/... così la vedi subito.

Come forzare wandb a usare la cartella che vuoi
Metti prima di ogni import wandb queste due righe:

python
Copia
Modifica
import os
WB_dir = "/mnt/data/WB_spectrograms_analyses_motor_task_1"
os.makedirs(WB_dir, exist_ok=True)
os.environ["WANDB_DIR"] = WB_dir
Poi:

python
Copia
Modifica
import wandb
wandb.login()
run = wandb.init(
    project="spectrograms-analyses-motor-task-1",
    dir=WB_dir,         # <–– anche qui puoi sovrascrivere il path
    resume="allow"      # <–– se fai sweep/runs multipli
)
In questo modo:

Wandb salverà localmente tutti i log e i file di run in WB_dir.

Il file‑browser di Jupyter (che punta su /mnt/data) li vedrà immediatamente.

Wandb poi sincronizzerà tutto in cloud, ma tu puoi lavorare “in locale” senza perdere nulla.

Salvare i .pkl
Quando salvi i tuoi .pkl all’interno della loop di hyperopt fai:

python
Copia
Modifica
out_path = os.path.join(WB_dir, f"{run.id}__{combo_name}.pkl")
with open(out_path, "wb") as f:
    pickle.dump(my_results, f)
wandb.save(out_path)   # <–– forza wandb ad aggiungere questo file allo snapshot del run
In questo modo:

Il file viene scritto subito in WB_dir

Viene collegato al run W&B (utile se poi fai wandb.restore)

Verifica “a freddo”
Dopo aver fatto partire i tuoi esperimenti, in un’altra cella puoi lanciare:

bash
Copia
Modifica
!ls -R /mnt/data/WB_spectrograms_analyses_motor_task_1
oppure – da CLI –

bash
Copia
Modifica
wandb files <ENTITY>/<PROJECT>/<RUN_ID>
per vedere l’elenco dei file che sono stati salvati/sincronizzati.

Se invece proprio non trovi nulla, controlla:

che non ci siano errori silenziosi (un try/except: pass che assorbe i problemi di I/O)

che tu stia eseguendo la cella di creazione delle cartelle prima di qualunque wandb.init

che non stia scrivendo in un container o in un filesystem ephemeral diverso da quello mostrato dal file‑browser

Con questa configurazione dovresti avere:

Cartelle create sotto /mnt/data/WB_spectrograms_analyses_motor_task_1

I tuoi .pkl visibili subito in Jupyter

Wandb che continua a fare upload in background senza perderli

Fammi sapere se così riesci a trovarli ed evitare di perdere i risultati dei tuoi sweep!

#### **Weight & Biases Procedure FINAL SEQUENCE OF STEPS - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS**

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm
from tqdm import tqdm

import random

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la visualizzazione dei dati

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader

import os
import pickle

import random

import wandb

In [ ]:
base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
with open(f'{base_path}new_all_th_concat_spectrograms_coupled_exp_task_1_2d_grid_multiband.pkl', 'rb') as f:
    new_all_th_concat_spectrograms_coupled_exp_task_1 = pickle.load(f)

In [ ]:
new_all_th_concat_spectrograms_coupled_exp_task_1.keys() 

In [ ]:
new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist'].keys()

In [ ]:
print(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist']['data'].shape)
print()
print(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist']['data'].shape)
print()
print(new_all_th_concat_spectrograms_coupled_exp_task_1['left_fist_vs_right_fist']['data'].shape)
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['left_fist_vs_right_fist']['labels'], return_counts = True))

#### **Utils Functions - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
'''
QUI DENTRO HO CONFIGURATO 
LE FUNZIONI DI CONTROLLO DELLE STRINGHE 
PER IL SALVATAGGIO DELLE PERFORMANCE DEL MODELLO
NELLE RELATIVE SUBFOLDERS

(I.E., get_subfolder_from_key, get_subfolder_from_key_hyper)

IN MODO CHE SI LEGHINO ALLA CHIAVE 'STANDARDIZATION' DELL'OGGETTO SWEEP_CONFIG
'''


import pickle
import numpy as np


def load_data(data_type, category, subject_type, condition = "th_resp_vs_pt_resp"):
    """
    Carica i dati EEG dalla directory appropriata, già salvati con la finestra temporale (0°-640° punto)

    Parameters:
    - data_type: str, "spectrograms",
    - category: str, "task_1"
    - subject_type: str, "th" (terapisti)
    - condition: str, condizione sperimentale da selezionare
    

    Returns:
    - X: Dati EEG sotto-selezionati (50°-300° punto e canali selezionati se applicabile)
    - y: Etichette corrispondenti
    """

    # Definizione dei percorsi base
    base_paths = {
        "spectrograms": {
            "familiar": "/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/"
            #"unfamiliar": "/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms/"
        },
    }

    # Seleziona il path corretto
    base_path = base_paths[data_type][category]

    # Determina il nome del file corretto
    if data_type in ["spectrograms"]:
        filename = f"new_all_{subject_type}_concat_spectrograms_coupled_exp_task_1_2d_grid_multiband.pkl"
    else:
        raise ValueError("data_type non valido!")
        
    # Caricamento del file
    filepath = base_path + filename
    
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    
    '''
    Per i dati spectrogram, la funzione seleziona la condizione desiderata (i.e., condition = "th_resp_vs_pt_resp") 
    e preleva i dati e le etichette associati a quella condizione.
    '''
    
    # Selezione della finestra temporale e delle etichette
    X = data[condition]["data"]
    y = data[condition]["labels"]

    
    return X, y


def select_channels(data, channels=[12, 30, 48]):
    """
    Seleziona i canali EEG specificati SOLO per i dati 1-20 e 1-45.

    Parameters:
    - data: array NumPy, dati EEG con shape (n_trials, n_channels, n_timepoints)
    - channels: list, indici dei canali da selezionare

    Returns:
    - data filtrato sui canali specificati
    """
    return data[:, channels, :]


# Funzione per train-test split
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

def split_data(X, y, test_size=0.2, val_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test


'''ATTENZIONE MODIFICATA FUNZIONE DI STANDARDIZZAZIONE'''
# Funzione per standardizzare i dati
# Con questa modifica eviti che std==0 produca NaN e i tuoi loss torneranno numeri sensati.
def standardize_data(X_train, X_val, X_test, eps = 1e-8):
    
    mean = X_train.mean(axis=0, keepdims=True)
    std = X_train.std(axis=0, keepdims=True)
    
    #aggiungo eps per evitare divisione per zero
    X_train = (X_train - mean) / (std + eps)
    X_val = (X_val - mean) / (std + eps)
    X_test = (X_test - mean) / (std + eps)
    
    return X_train, X_val, X_test


# Import modelli (definisci le classi CNN1D, ReadMEndYou, ReadMYMind)
#from models import CNN1D, ReadMEndYou, ReadMYMind  # Assicurati di avere i modelli definiti in 'models.py'

# Funzione per inizializzare i modelli
def initialize_models():
    #model = CNN1D(input_channels=3, num_classes=2)
    model_CNN = CNN2D(input_channels = 61, num_classes=2)
    #model_LSTM = ReadMEndYou(input_size=3, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    model_LSTM = ReadMEndYou(input_size=61 * 26, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    #model_Transformer = ReadMYMind(num_channels=3, seq_length=250, d_model=16, num_heads=4, num_layers=2, num_classes=2)
    model_Transformer = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=61, freqs=26)
    
    return model_CNN, model_LSTM, model_Transformer


import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight


'''
Questa funzione prende in input i dati di training, validation e test, 
il tipo di modello scelto e la dimensione del batch. Si occupa di:

Calcolare i pesi delle classi.
Convertire i dati in tensori PyTorch, con le opportune trasformazioni per CNN, LSTM o Transformer.
Creare i dataset e i dataloader per il training.
'''


def prepare_data_for_model(X_train, X_val, X_test, y_train, y_val, y_test, model_type, batch_size=48):
    
    # Calcolo dei pesi delle classi
    class_weights = compute_class_weight(class_weight='balanced', 
                                         classes=np.unique(y_train), 
                                         y=y_train)
    
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
    class_weights_tensor = class_weights_tensor.to(dtype=torch.float32, device=device)
    
    # Conversione delle etichette in interi
    y_train = y_train.astype(int)
    y_val = y_val.astype(int)
    y_test = y_test.astype(int)
    
    # Conversione dei dati in tensori PyTorch con permutazione se necessario
    
    '''OCCHIO QUI CAMBIATO'''
    #if model_type == "CNN2D":
    
    if model_type == "CNN2D_LSTM_FC":
        
    #if model_type == "CNN2D":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #BiLSTM (ReadMEndYou):
    #Ora il modello si aspetta l’input con shape (batch, canali, frequenze, tempo) 
    #e, al suo interno, 
    #esegue la permutazione per avere il tempo come dimensione sequenziale. 
    #Non serve quindi applicare una permutazione anche qui.
    
    elif model_type == "CNN3D_LSTM_FC":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    elif model_type == "SeparableCNN2D_LSTM_FC":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        
    elif model_type == "BiLSTM":
            
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #Transformer (ReadMYMind):
    #Analogamente, il modello gestisce internamente la riorganizzazione dell’input, quindi lasciamo i dati nella loro forma originale.
    elif model_type == "Transformer":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    else:
        raise ValueError("Modello non riconosciuto. Scegli tra 'CNN', 'LSTM' o 'Transformer'.")
    
    # Conversione delle etichette in tensori
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    # Creazione dei dataset
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    # Creazione dei dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, class_weights_tensor


'''
# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, sweep_config):
    
    
    #Mi richiamo la chiave 'standardization' che ho impostato nella configurazione dell'oggetto weight and biases
    #(i.e., sweep_config['standardization']) e eseguo una procedura condizionale 
    
    ossia che, se risulta o True o False, lui cambi le condizioni di gestione 
    della costruzione delle path di salvataggio 
    
    
    # Controlla se i dati sono standardizzati
    if sweep_config['standardization']:
    
        #PER I DATI SCALED
        if '_familiar_th' in key:
            return 'TH_FAM'
        elif '_unfamiliar_th' in key:
            return 'TH_UNFAM'
        elif '_familiar_pt' in key:
            return 'PT_FAM'
        elif '_unfamiliar_pt' in key:
            return 'PT_UNFAM'
        else:
            return None
    else: 
        #PER I DATI UNSCALED

        if '_familiar_th' in key:
            return 'TH_FAM_UNSCALED'
        elif '_unfamiliar_th' in key:
            return 'TH_UNFAM_UNSCALED'
        elif '_familiar_pt' in key:
            return 'PT_FAM_UNSCALED'
        elif '_unfamiliar_pt' in key:
            return 'PT_UNFAM_UNSCALED'
        else:
            return None
    
     
   
    
# Funzione per salvare i risultati
def save_performance_results(model_name, my_train_results, my_test_results, key, data_type, sweep_config, condition = "th_resp_vs_pt_resp"):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, sweep_config)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "wavelet" in key:
        data_type_str = "wavelet_delta"
    elif "1_20" in key:
        data_type_str = "1_20"
    elif "1_45" in key:
        data_type_str = "1_45"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    file_name = f"{model_name}_performances_{condition}_{subfolder}_{data_type_str}.pkl"
    folder_path = os.path.join(base_folder, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo ✅ in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"Errore durante il salvataggio dei risultati: {e}")
        
'''


# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, model_standardization):
    
    #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
    if '_familiar_th' in key:
        return 'th_fam'
    elif '_unfamiliar_th' in key:
        return 'th_unfam'
    elif '_familiar_pt' in key:
        return 'pt_fam'
    elif '_unfamiliar_pt' in key:
        return 'pt_unfam'
    else:
        return None
     
   
    
# Funzione per salvare i risultati
def save_performance_results(model_name, my_train_results, my_test_results, key, data_type, sweep_config, condition = "th_resp_vs_pt_resp"):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, sweep_config)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
     # Determinazione del tipo di dato direttamente dalla chiave
    if "spectrograms" in key:
        data_type_str = "spectrograms"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    file_name = f"{model_name}_performances_{condition}_{subfolder}_{data_type_str}.pkl"
    folder_path = os.path.join(base_folder, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo ✅ in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"Errore durante il salvataggio dei risultati: {e}")

#### **DL Models - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
'''

VERSIONE CONVOLUZIONE 3D PURA e CONVOLUZIONI SEPARABILI 19 LUGLIO 2025


Due versioni dell’architettura:

CNN3D_LSTM_FC: usa nn.Conv3d per eseguire una vera convoluzione 3D sui cinque depth (bande di frequenza), 
mantenendo il resto del flusso identico.

SeparableCNN2D_LSTM_FC: applica in sequenza una convoluzione depthwise (gruppi = canali) e una pointwise (1×1) 
per fondere i cinque canali in modo efficiente.

Entrambe le classi si integrano con il tuo blocco LSTM e il classificatore come nella versione originale.



******** PROBLEMA SUL GRADCAM NELLE ANALISI PRE 19 LUGLIO ********


Per ottenere un Grad‑CAM “3D” su ciascuna delle 5 bande (cioè un volume 9×9×5) 
invece di schiacciare tutto in una mappa 9×9, bisogna:

Non appiattire la dimensione di profondità (“depth” = bande) con cam.mean(dim=1).

Calcolare i pesi medi dei gradienti solo su altezza e larghezza, non su depth, in modo da preservare D=5.

Upsample (solo) le due dimensioni spaziali H×W, lasciando inalterata la profondità D.

(Opzionale) 

Se il tuo primo Conv3d usa un kernel di profondità pari all’intera profondità d’ingresso, 
quella informazione viene compressa in D=1!





******** PROBLEMA SUL CONV3D NELLE ANALISI PRE 19 LUGLIO ********


Se il tuo primo Conv3d usa un kernel di profondità pari all’intera profondità d’ingresso, 
quella informazione viene compressa in D=1!


Se vuoi davvero avere D=5 in uscita, devi cambiare conv1 in:


# ❌ kernel_size=(5,3,3), padding=(0,1,1) → D_out = 1

self.conv1 = nn.Conv3d(1, 32, kernel_size=(3,3,3), padding=(1,1,1))
così la profondità si conserva da 5→5.



1) Perché in conv1 useremo padding=(1,1,1) e negli altri layer padding=(0,1,1)
Obiettivo: mantenere la profondità (numero di bande, D = 5) costante lungo tutta la rete.

In conv1, abbiamo scelto kernel_size=(3,3,3) perché vogliamo che il filtro “scorra” su tutti e tre gli assi (D,H,W).

Con kernel_depth=3, per avere

𝐷out = (𝐷in + 2⋅𝑃 depth − 𝐾 depth)/ 𝑆 + 1 = 5

Da qui (1,1,1) per (depth, height, width).

Negli altri layer 3D (conv2a, conv2b, conv3) il kernel depth = 1 (kernel_size=(1,3,3)), 
quindi la profondità non cambia se mettiamo padding_depth=0 con padding (0,1,1) nel layer conv2 e conv3

In altre parole, su quell’asse non serve alcun padding:

se P dept = 0 allora diventa infatti

𝐷out = (𝐷in + 2⋅0 − 𝐾 depth)/ 𝑆 + 1 = 5

2⋅0


Non è che la tua rete “CNN3D_LSTM_FC” sia sbagliata in senso assoluto, 
ma — proprio a causa di quel primo Conv3d con kernel_size=(5,3,3) e padding=(0,1,1) — 

stai automaticamente comprimendo tutte e 5 le bande nella singola fetta di profondità:


self.conv1 = nn.Conv3d(
    in_channels=1, out_channels=32,
    kernel_size=(5, 3, 3),  # → D_out = (5 − 5 + 2·0)/1 + 1 = 1
    padding=(0, 1, 1)
)
Quindi il tuo tensore (B, 1, 5, 9, 9) diventa (B, 32, 1, 9, 9): la dimensione depth (5) si riduce a 1 subito.

Se invece vuoi davvero preservare le 5 “fette” come vera terza dimensione spaziale, hai due possibili correzioni:

Usare un kernel 3×3×3 (o 1×3×3) in conv1, in modo da non “abbracciare” tutta la profondità d’ingresso:


- self.conv1 = nn.Conv3d(1, 32, kernel_size=(5, 3, 3), padding=(0, 1, 1))
+ # preserva D: depth out = depth in = 5
+ self.conv1 = nn.Conv3d(1, 32, kernel_size=(3, 3, 3), padding=(1, 1, 1))

Oppure, se vuoi mantenere le bande completamente indipendenti in questo layer,


self.conv1 = nn.Conv3d(1, 32, kernel_size=(1, 3, 3), padding=(0, 1, 1))
che trasforma (B,1,5,9,9) → (B,32,5,9,9).

Lasciare com’è, sapendo però che la rete “fonderà” le 5 bande in un’unica mappa di profondità: 
non è un bug, è una scelta architetturale.



La SeparableCNN2D_LSTM_FC
Quella architettura non comprime mai le bande all’interno di un’unica fetta, perché:

Le bande diventano canali di un Conv2d depth‑wise:


x = x.permute(0, 3, 1, 2)  # (B,5,9,9)
self.dw_conv1 = nn.Conv2d(5, 5, kernel_size=3, padding=1, groups=5)
Ogni “fetta” (banda) resta separata fino al pointwise e agli strati successivi.

Quindi se il tuo obiettivo è avere un’uscita per banda (e poi poter plottare un Grad‑CAM 2D per ciascuna),
la SeparableCNN2D è già configurata correttamente.

Se invece vuoi un Grad‑CAM “volumetrico” 3D (5×9×9) direttamente dal modello 3D puro,
la vera modifica necessaria è solo sul primo Conv3d, come mostrato sopra.

Fammi sapere quale dei due setup stai usando e ti aiuto a integrare il Grad‑CAM 3D di conseguenza!

'''



'''
Ecco l’implementazione completa di CNN3D_LSTM_FC (“approccio sequenza di profondità”) in cui:

mantieni il tuo primo blocco 3D con kernel (3,3,3) e padding (1,1,1), quindi D rimane 5 fino alla fine;

riduci le spatial singleton dims (H=1,W=1) e trasformi la depth D=5 in una sequenza di lunghezza 5;

imposti l’input_size=128 nell’LSTM (feature per time‑step = 128);


Con questa versione:

la sequenza per l’LSTM ha lunghezza D=5;

ogni passo ha 128 feature, esattamente input_size=128;

non servono trucchi di reshape su scala globale.


'''


import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN3D_LSTM_FC(nn.Module):
    """
    Version with pure 3D convolutions treating the 5 frequency bands
    as a sequence (depth) for the LSTM.
    Input: Tensor of shape (B, 9, 9, 5) --> reshaped to (B, 1, 5, 9, 9)
    """
    def __init__(self, num_classes=2, dropout=0.5, hidden_size=64, use_lstm=True):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.use_lstm = use_lstm

        # --- Block 1 (3D) ---
        self.conv1   = nn.Conv3d(1,  32, kernel_size=(3,3,3), padding=(1,1,1))
        self.bn1     = nn.BatchNorm3d(32)
        self.pool3d  = nn.MaxPool3d((1,2,2))  # non tocca D

        # --- Block 2 (3D Residual) ---
        self.res_conv3d = nn.Conv3d(32, 64, kernel_size=1, bias=False)
        self.res_bn3d   = nn.BatchNorm3d(64)
        self.conv2a     = nn.Conv3d(32, 64, kernel_size=(1,3,3), padding=(0,1,1))
        self.bn2a       = nn.BatchNorm3d(64)
        self.conv2b     = nn.Conv3d(64, 64, kernel_size=(1,3,3), padding=(0,1,1))
        self.bn2b       = nn.BatchNorm3d(64)

        # --- Block 3 (3D) ---
        self.conv3 = nn.Conv3d(64, 128, kernel_size=(1,3,3), padding=(0,1,1))
        self.bn3   = nn.BatchNorm3d(128)

        # LSTM o FC finale
        if self.use_lstm:
            # input_size = feature_dim per time‑step = 128
            self.lstm       = nn.LSTM(input_size=128,
                                      hidden_size=self.hidden_size,
                                      num_layers=1,
                                      batch_first=True)
            self.classifier = nn.LazyLinear(num_classes)
        else:
            self.classifier = nn.LazyLinear(num_classes)

    def forward(self, x):
        # x: (B, 9, 9, 5)
        if x.ndim == 4:
            # -> (B,1,D=5,H=9,W=9)
            x = x.permute(0, 3, 1, 2).unsqueeze(1)

        # --- Block 1 ---
        x = F.relu(self.bn1(self.conv1(x)))  # (B,32,5,9,9)
        x = self.pool3d(x)                   # (B,32,5,4,4)

        # --- Block 2 (Residual) ---
        res = self.res_bn3d(self.res_conv3d(x))  # (B,64,5,4,4)
        x   = F.relu(self.conv2a(x))             # (B,64,5,4,4)
        x   = self.bn2b(self.conv2b(x))          # (B,64,5,4,4)
        x   = F.relu(x + res)                    # (B,64,5,4,4)
        x   = self.pool3d(x)                     # (B,64,5,2,2)

        # --- Block 3 ---
        x = F.relu(self.bn3(self.conv3(x)))      # (B,128,5,2,2)
        x = self.pool3d(x)                       # (B,128,5,1,1)

        # Stampa delle dimensioni prima di passare al classifier
        #print(f"Dimensioni prima del classifier: {x.shape}")

        if self.use_lstm:
            # x: (B,128,5,1,1)
            # -> squeeze spatial dims → (B,128,5)
            x = x.squeeze(-1).squeeze(-1)
            # -> permute per batch_first → (B, seq_len=5, feat=128)
            x = x.permute(0, 2, 1)
            x = self.dropout(x)
            out, _ = self.lstm(x)               # out: (B,5,hidden_size)
            last    = out[:, -1, :]             # prendo l’ultimo time-step
            logits  = self.classifier(last)     # (B, num_classes)
        else:
            # x: (B,128,5,1,1) → flatten → (B,128)
            x = x.view(x.size(0), -1)
            logits = self.classifier(self.dropout(x))

        return logits

    

class SeparableCNN2D_LSTM_FC(nn.Module):
    """
    Version with depthwise + pointwise separable convolutions
    across the 5 channels.
    Input: Tensor of shape (B, 9, 9, 5) -> (B,5,9,9)
    """
    def __init__(self, num_classes=2, dropout=0.5, hidden_size=64, use_lstm=True):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.use_lstm = use_lstm

        # --- Block 1 separabile ---
        self.dw_conv1 = nn.Conv2d(5, 5, kernel_size=3, padding=1, groups=5)
        self.pw_conv1 = nn.Conv2d(5, 32, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)

        # --- Block 2 (residuo) ---
        self.res_conv = nn.Conv2d(32, 64, kernel_size=1, bias=False)
        self.res_bn = nn.BatchNorm2d(64)
        self.bn2a = nn.BatchNorm2d(32)
        self.conv2a = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2b = nn.BatchNorm2d(64)
        self.conv2b = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # --- Block 3 ---
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.lstm = nn.LSTM(input_size=128 * 5, hidden_size=self.hidden_size, num_layers=1, batch_first=True)

        if self.use_lstm:
            self.lstm = nn.LSTM(
                input_size=128 * 1,
                hidden_size=self.hidden_size,
                num_layers=1,
                batch_first=True
            )
            self.classifier = nn.Linear(self.hidden_size, num_classes)
        else:
            self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)  # -> (B,5,9,9)

        x = F.relu(self.dw_conv1(x))
        x = F.relu(self.bn1(self.pw_conv1(x)))
        x = self.pool(x)

        res = self.res_bn(self.res_conv(x))
        x = F.relu(self.conv2a(self.bn2a(x)))
        x = self.bn2b(self.conv2b(x))
        x = F.relu(x + res)
        x = self.pool(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)  # (B,128,1,1)

        if self.use_lstm:
            x = x.permute(0, 2, 1, 3).reshape(x.size(0), 1, -1)  # (B,1,128)
            out, _ = self.lstm(self.dropout(x))
            last = out[:, -1, :]
            logits = self.classifier(last)
        else:
            x = x.view(x.size(0), -1)
            logits = self.classifier(self.dropout(x))

        return logits

In [ ]:
# Quick test
if __name__ == "__main__":
    cnn = CNN3D_LSTM_FC()
    sep_conv = SeparableCNN2D_LSTM_FC()
    
    # Test both input orders
    x2 = torch.randn(8, 9, 9, 5)
    print(cnn(x2).shape)   # -> (8,2)
    print(sep_conv(x2).shape)  # -> (8,2)

In [ ]:
print(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist']['data'].shape)
print()
print(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist']['data'].shape)
print()
print(new_all_th_concat_spectrograms_coupled_exp_task_1['left_fist_vs_right_fist']['data'].shape)
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['left_fist_vs_right_fist']['labels'], return_counts = True))

#### **Early Stopping - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
'''
DEFINIZIONE EARLY STOPPING
'''

import io
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import pickle
import numpy as np


class EarlyStopping:
    def __init__(self, patience = 10, min_delta = 0.001, mode = 'max'):
        
        
        """
        :param patience: Numero di epoche da attendere prima di interrompere il training se non c'è miglioramento
        
        Esempio: il training si interromperà se non si osserva un miglioramento per (N = 5) epoche consecutive.
        
        :param min_delta: Variazione minima richiesta per considerare un miglioramento
        
        definisce il miglioramento minimo richiesto per essere considerato significativo. 
        Se il miglioramento è inferiore a min_delta, non viene considerato un vero miglioramento.
        
        Il parametro min_delta in una configurazione di early stopping indica 
        la minima variazione del valore di una metrica 
        (ad esempio, la perdita o l'accuratezza) 
        che deve verificarsi tra un'epoca e la successiva 
        per continuare l'allenamento. 
        
        In genere, il valore di min_delta dipende dal tipo di modello e dai dati specifici, 
        ma di solito si trova in un intervallo tra 0.001 e 0.01.
    
            - Se stai cercando di evitare che l'allenamento si fermi troppo presto,
            puoi impostare un valore più basso per min_delta (come 0.001), 
            - Se vuoi essere più conservativo e permettere fluttuazioni nei valori della metrica,
            un valore più alto (come 0.01) potrebbe essere appropriato.

        Un buon punto di partenza potrebbe essere 0.001, e poi fare dei test per capire quale valore funziona meglio
        nel tuo caso specifico!
        
        :param mode: 'min' per monitorare la loss (minimizzazione), 'max' per l'accuracy (massimizzazione)
        
        'max' → ottimizza metriche da massimizzare (es. accuracy, F1-score, AUC).
        'min' → ottimizza metriche da minimizzare (es. loss).
        
        """
            
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.best_score = None # Tiene traccia del miglior punteggio osservato
        self.counter = 0 # Conta quante epoche consecutive non migliorano
        self.early_stop = False # Flag che indica se attivare l'early stopping
        
        #Ogni volta che si chiama la classe con early_stopping(current_score), controlla se il modello sta migliorando o meno.

    def __call__(self, current_score):
        
        #Caso 1: Prima iterazione (best_score ancora None)
        #→ Se non esiste ancora un miglior punteggio, lo inizializza con il primo valore ricevuto.
        
        if self.best_score is None:
            self.best_score = current_score
            
        #Caso 2: Il modello migliora
        #→ Se il valore migliora di almeno min_delta, aggiorna best_score e resetta il contatore.

        elif (self.mode == 'min' and current_score < self.best_score - self.min_delta) or \
             (self.mode == 'max' and current_score > self.best_score + self.min_delta):
            self.best_score = current_score
            self.counter = 0  # Reset contatore se migliora
            
        #Caso 3: Il modello NON migliora
        
        #→ Se il valore non migliora, incrementa il contatore.
        #→ Se il contatore raggiunge patience, imposta early_stop = True, segnalando che il training deve essere interrotto.
        
        else:
            self.counter += 1  # Incrementa se non migliora
            if self.counter >= self.patience:
                print(f"🛑 Early stopping attivato! Nessun miglioramento per {self.patience} epoche consecutive.")
                self.early_stop = True


#### **Weight & Biases Login - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
'''

WEIGHT AND BIASES LOGIN

Il messaggio che stai ricevendo indica 
che sei già connesso al tuo account Weights & Biases (wandb).

Se vuoi forzare il login, puoi usare il comando suggerito:

wandb login --relogin

Questo comando ti permetterà di reinserire le credenziali e riconnetterti al tuo account.
Se non hai bisogno di disconnetterti o di cambiare l'account,
puoi semplicemente continuare a usare wandb senza ulteriori passaggi. 
Hai bisogno di ulteriore assistenza con wandb o con il tuo progetto?
'''


import wandb
wandb.login()
print("✅ Weights & Biases login effettuato con successo!")

In [ ]:
'''
Per modificare il percorso in cui W&B salva i dati localmente,
puoi configurare la variabile di ambiente WANDB_DIR.

Questo ti permette di specificare una directory personalizzata in cui W&B salva tutti i file associati al tuo run, inclusi i dati e i log.
'''

import os

# Imposta la directory per i dati W&B:
# Questo cambierà la cartella in cui W&B salva i dati per quella sessione di esecuzione

# Definisci la cartella principale
#WB_dir = "/home/stefano/Interrogait/WB_spectrograms_analyses_channels_frequencies_params_hyperparams_motor_task_1"

WB_dir = "/home/stefano/Interrogait/WB_spectrograms_analyses_channels_frequencies_params_motor_task_1_new_3d_grid_multiband"
os.makedirs(WB_dir, exist_ok=True)


os.environ["WANDB_DIR"] = WB_dir


In [ ]:
import pickle
import numpy

# Apri il file in modalità lettura binaria ('rb')

#path = '/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms/'

#path = '/home/stefano/Interrogait/all_datas/Familiar_Spectrograms_channels_frequencies/'

path = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

with open(f"{path}new_all_th_concat_spectrograms_coupled_exp_task_1_2d_grid_multiband.pkl", "rb") as f:
    data = pickle.load(f)


In [ ]:
# Itera sulle chiavi del dizionario principale
for condition, values in data.items():
    if isinstance(values, dict) and "data" in values and "labels" in values:
        X_shape = values["data"].shape
        y_length = len(values["labels"])
        print(f"🔹 Condizione: {condition}")
        print(f"   ➡ Shape dati: {X_shape}")
        print(f"   ➡ Lunghezza labels: {y_length}\n")


In [ ]:
new_all_th_concat_spectrograms_coupled_exp_task_1.keys()

#### **Datasets Loading - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
##################### CODICE UFFICIALE DEL 04/03/2025 ORE 9:30 #####################
                                 ##################### SENZA DETTAGLI SCRITTI V°3 #####################
        
'''ATTENZIONE: 

HO SOSTITUITO LE VARIABILI DI 

    1) DATASET_TRAIN_LOADER -->  TRAIN_LOADER
    2) DATASET_VAL_LOADER -->  VAL_LOADER

    VEDI FUNZIONE 'PREPARE_DATA_FOR_MODEL --> NOMI DELLE VARIABILI DEI TORCH TENSOR DATASET LOADER SON  'TRAIN_LOADER' E VAL_LOADER!!!  

'''

import torch
import torch.nn as nn
import torch.optim as optim
import wandb
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import copy as cp
import numpy as np

import wandb
import random
import copy as cp


# Definisci le lista delle coppie di condizioni sperimentali
#experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

experimental_conditions = ["rest_vs_left_fist", "rest_vs_right_fist", "left_fist_vs_right_fist"]


# Inizializza il dizionario per caricare i dati
data_dict = {}

#Definisci la cartella principale

#base_dir = "/home/stefano/Interrogait/WB_spectrograms_best_results"

#base_dir = "/home/stefano/Interrogait/WB_spectrograms_best_results_channels_frequencies"

#base_dir = "/home/stefano/Interrogait/WB_spectrograms_best_results_channels_frequencies_params_hyperparams"


'''HO AGGIUNTO NEW QUI PER CREARE CARTELLA CON SWEEP NUOVI, CONFRONTANDO BILSTM & TRANSFORMER CON CNN2D_LSTM!'''

base_dir = "/home/stefano/Interrogait/WB_spectrograms_best_results_channels_frequencies_params_motor_task_1_new_3d_grid_multiband"


os.makedirs(base_dir, exist_ok = True)

'''LOOP DI CARICAMENTO DATI'''

for condition in experimental_conditions:
    # Crea la cartella per la condizione sperimentale
    condition_dir = os.path.join(base_dir, condition)
    os.makedirs(condition_dir, exist_ok=True)
    
    # Aggiungi un livello di annidamento per ogni condizione
    data_dict[condition] = {}
    
    for data_type in ["spectrograms"]:
        
        # Crea la cartella per il tipo di dato
        data_dir = os.path.join(condition_dir, data_type)
        os.makedirs(data_dir, exist_ok=True)
        
        #for category in ["familiar", "unfamiliar"]:
        for category in ["familiar"]:
            
            # Crea la cartella per la categoria
            #category_dir = os.path.join(data_dir, category)
            #os.makedirs(category_dir, exist_ok=True)
            
            #for subject_type in ["th", "pt"]:
            for subject_type in ["th"]:
                # Caricamento e suddivisione dei dati
                
                #if data_type == "spectrograms":
                    
                print(f"Caricamento dati per: {condition} - {data_type} - {category}_{subject_type}")
                X, y = load_data(data_type, category, subject_type, condition=condition)
                
                
                # Creazione della chiave per il dizionario annidato
                data_dict[condition][data_type] = data_dict[condition].get(data_type, {})
                data_dict[condition][data_type][f"{category}_{subject_type}"] = (X, y)
                
                # Stampa di conferma
                print(f"Dataset caricato: \033[1m{condition}\033[0m_\033[1m{data_type}\033[0m_\033[1m{category}_{subject_type}\033[0m - Shape X: \033[1m{X.shape}\033[0m, Shape y: \033[1m{len(y)}\033[0m\n")

#### **Datasets Loading - EEG Spectrograms - Electrodes x Frequencies** 

#### **CHECK PER ASSENZA DI VALORI NEGATIVI NELLA POTENZA SPETTRALE DEI DATI EEG** 

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/'

# Caricare l'intero dizionario annidato con pickle
#with open(f'{path_channels}ordered_electrodes.pkl', 'rb') as f:

with open(f'{path}electrode_grid_map.pkl', 'rb') as f:
    EEG_channels_names = pickle.load(f)

In [ ]:
for condition in data_dict.keys():
    for data_type in data_dict[condition].keys():
        for group_key in data_dict[condition][data_type].keys():
            
            # Estrai i dati
            X, y = data_dict[condition][data_type][group_key]

            # Controlli di validità
            min_val = X.min()
            has_neg = (X < 0).any()
            has_nan = np.isnan(X).any()
            has_inf = np.isinf(X).any()

            # Stampa risultato
            print(f"\n🧪 Check: {condition} → {data_type} → {group_key}")
            print(f"  ➤ Shape X: {X.shape}, Shape y: {y.shape}")
            print(f"  ➤ Valore minimo assoluto: {min_val}")
            print(f"  ➤ Contiene negativi? {'✅ SÌ' if has_neg else '❌ NO'}")
            print(f"  ➤ Contiene NaN? {'✅ SÌ' if has_nan else '❌ NO'}")
            print(f"  ➤ Contiene Inf? {'✅ SÌ' if has_inf else '❌ NO'}")

#### **Datasets Loading - EEG Spectrograms - Electrodes x Frequencies**

#### **CHECK VALORI POTENZA SPETTRALE PER BANDA** 
#### **VISUALIZZAZIONE POTENZA MEDIA PER BANDA A LIVELLO TOPOGRAFICO, TRA LE DUE CONDIZIONI SPERIMENTALI**

| Oggetto              | Contenuto                      | Uso nello script                                                                          |
| -------------------- | ------------------------------ | ----------------------------------------------------------------------------------------- |
| `electrode_grid_map` | `{ 'Fz': (y,x), … }`           | serve a posizionare il testo con `ax.text(x,y,label,…)`.                                  |
| `grand_X`            | tutti i trial → `(N, 9, 9, 5)` | da cui ricavi: <br>• `mean_power_per_band` (bar plot) <br>• `mean_maps` (heat‑map 9×9×5). |


Così vedrai:

Bar‑plot delle bande dominate (Delta, Theta, Alpha, …).

Heat‑map della distribuzione spaziale media per ciascuna banda, con il nome di ogni elettrodo sovra‑impresso nella sua cella 9 × 9.

In [ ]:
# funzione d’aiuto: dalla stringa condition → lista [name0, name1]
def split_condition(cond_str: str):
    if "_vs_" in cond_str:
        return cond_str.split("_vs_", maxsplit=1)
    else:
        return [f"{cond_str}_0", f"{cond_str}_1"]     # fallback

# ------------------------------------------------------------------
# 1.  Riepilogo min/max (dataset  ➜ banda  ➜ label 0/1 con nome)
# ------------------------------------------------------------------
band_names = ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']

for condition in data_dict:                      # es. 'rest_vs_left_fist'
    cls_names = split_condition(condition)       # ['rest', 'left_fist']
    for data_type in data_dict[condition]:
        for group_key in data_dict[condition][data_type]:
            
            X, y = data_dict[condition][data_type][group_key]

            print(f"\n🧪  \033[1m{condition} → {data_type} → {group_key}\033[0m\n")
            print(f"    Shape X: {X.shape},  Shape y: {y.shape}")
            print(f"    Min assoluto dataset : {X.min():.3e}")
            print(f"    Negativi presenti?   {'✅ SÌ' if (X < 0).any() else '❌ NO'}\n")
            
            print(f" \033[1mValori di potenza spettrale per banda (tra i dati di tutte e due le classi)\033[0m")
            # range banda – tutti i trial insieme
            for b, name in enumerate(band_names):
                mm = X[..., b]
                print(f"       • {name:<6} tot : min={mm.min():.3e}   max={mm.max():.3e}")

            # range banda – separato per label 0 / 1  (con nome condizione)
            for lbl in [0, 1]:
                mask = (y == lbl)
                if not mask.any():
                    continue
                X_lbl = X[mask]
                print(f"\n       ▼ Label \033[1m{lbl}\033[0m  (\033[1m{cls_names[lbl]}\033[0m)")
                for b, name in enumerate(band_names):
                    mm = X_lbl[..., b]
                    print(f"         {name:<6}: min={mm.min():.3e}   max={mm.max():.3e}")


In [ ]:
'''

                                                    UNITA' MISURA IN VOLT, CON SCALA COLORI GLOBALE PER TUTTE LE BANDE


Ho preparato uno snippet che, per ogni dataset:

1) Prende tutti i trial di ognuna delle 2 condizioni sperimentali per dataset e calcola:

2) la potenza media per banda (bar‑plot) e la visualizza rispetto a quella per le altre 

3) la mappa 9 × 9 della potenza media per ogni banda (un’heat‑map per banda), mettendo i relativi plot della relativa banda uno accanto all'altro 
per le 2 condizioni sperimentali 

'''

import pickle, numpy as np, matplotlib.pyplot as plt

# ---------- mappa elettrodo → (y,x) ----------
with open('/home/stefano/EEGMMIDB_Analyses/electrode_grid_map.pkl', 'rb') as f:
    electrode_grid_map = pickle.load(f)          # {'Fz':(0,4), 'Cz':(4,4), ...}

band_names = ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']

def split_condition(cond_str):
    return cond_str.split("_vs_", 1) if "_vs_" in cond_str else [f"{cond_str}_0",
                                                                 f"{cond_str}_1"]

# ============================================================
# LOOP su ogni dataset (= coppia di condizioni sperimentali)
# ============================================================
for condition in data_dict:                             # es. 'rest_vs_left_fist'
    cls_names = split_condition(condition)              # ['rest', 'left_fist']
    
    for dtype in data_dict[condition]:
        for gk in data_dict[condition][dtype]:
            
            X, y = data_dict[condition][dtype][gk]      # X:(N,9,9,5)
            print(f"\n👀  Figura   {condition}  /  {gk}   (shape X={X.shape})")

            # ---------- pre‑calcola medie ----------
            mean_bar   = []      # (2, 5)
            mean_maps  = []      # (2, 9,9,5)
            for lbl in (0,1):
                mask = (y == lbl)
                if not mask.any():               # se manca una classe, riempi di NaN
                    mean_bar.append(np.full(5, np.nan))
                    mean_maps.append(np.full((9,9,5), np.nan))
                else:
                    Xlbl = X[mask]
                    mean_bar.append(Xlbl.mean(axis=(0,1,2)))
                    mean_maps.append(Xlbl.mean(axis=0))
            
            
            '''MODIFICA PER TROVARE MINIMO E MASSIMO PER OGNI SCALA'''
            # ---------- calcola scala cromatica comune per le heatmap ----------
            vmin = np.nanmin([np.nanmin(mean_maps[0]), np.nanmin(mean_maps[1])])
            vmax = np.nanmax([np.nanmax(mean_maps[0]), np.nanmax(mean_maps[1])])

            # ---------- layout figure ----------
            n_rows, n_cols = 1 + len(band_names), 2       # 6 × 2
            
            fig, axs = plt.subplots(n_rows, n_cols,
                                    figsize=(10, 18),
                                    constrained_layout=False)
            
            fig.suptitle(f"{condition}  –  {dtype} / {gk}",
                         fontsize=10, 
                         y=0.97)      # ⬅︎ 0.97 invece di 0.995
            
            '''più vai verso 0.99 e più sovrapposizione ci sarà tra il fig.suptitle e i titoli delle varie righe'''
            fig.subplots_adjust(top=0.95)          # ⬅︎ spinge giù l’intera griglia

            # --- riga‑0: bar‑plot ---
            for col, lbl in enumerate((0,1)):
                ax = axs[0, col]
                ax.bar(band_names, mean_bar[lbl])
                ax.set_title(f"{cls_names[lbl]}  (label {lbl})")
                ax.set_ylabel("Mean power")
                ax.set_ylim(0, np.nanmax(mean_bar))       # scala comune
                if col == 1: ax.set_ylabel("")            # evita doppio testo

            # --- righe 1‑5: heat‑map per banda ---
            for r, band in enumerate(band_names, start=1):
                for col, lbl in enumerate((0,1)):
                    ax = axs[r, col]
                    
                    '''Tutte le heatmap ora useranno la stessa scala di colori, 
                    rendendo comparabili le potenze tra bande e tra classi.'''
                    
                    im = ax.imshow(mean_maps[lbl][:,:,r-1],
                       cmap='jet', origin='upper', aspect='equal',
                       vmin=vmin, vmax=vmax)  # ⬅️ scala colori comune
                    
                    #im = ax.imshow(mean_maps[lbl][:,:,r-1],
                                   #cmap='jet', origin='upper', aspect='equal')
                    # etichette elettrodo
                    for lab,(yy,xx) in electrode_grid_map.items():
                        ax.text(xx, yy, lab,
                                ha='center', va='center',
                                fontsize=5.5, color='black', weight='bold')
                    ax.set_xticks([]); ax.set_yticks([])
                    if col == 0:
                        ax.set_ylabel(band, rotation=90, labelpad=6,
                                      fontsize=9, weight='bold')
                    # color‑bar SOLO a destra
                    if col == 1:
                        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

            fig.suptitle(f"{condition}  –  {dtype} / {gk}", fontsize=14, y=0.995)
            plt.show()


In [ ]:
'''

Due punti veloci prima del codice:

Ampiezza vs Potenza. L’EEG “grezzo” è in ampiezza (Volt). Quando fai FFT e sommi i bin di una banda, stai calcolando (a fattori di scala vicini)
la potenza nella banda: la potenza è proporzionale al quadrato dell’ampiezza → unità V² (oppure V²/Hz se usi PSD e poi integri).

Unità nei plot. Se lasci in Volt, la potenza risulta in V²; per resa “da neuroscienze” conviene convertire a µV² (moltiplicando per 1e12) 
prima di plottare e scrivere le unità su assi e colorbar.

Quando usare scala colore “globale” vs “per banda”

Globale (tutte le bande insieme): mantiene il confronto assoluto tra bande (es. Delta ≫ Gamma). Rischia di “appiattire” le bande più deboli.

Per banda (comune alle due condizioni): ogni riga/banda ha la stessa scala tra le 2 condizioni, così vedi meglio le differenze topografiche all’interno della banda,
senza che Delta domini tutto.




Come provarlo subito

1) Potenza in µV² + scala globale

plot_band_power(data_dict, scale_mode="global", unit_mode="uV2", use_sem=False)


2) Potenza in µV² + scala per banda (comune alle 2 condizioni)

plot_band_power(data_dict, scale_mode="per_band", unit_mode="uV2", use_sem=False)


3) dB re: 1 µV² + scala per banda

plot_band_power(data_dict, scale_mode="per_band", unit_mode="dB", use_sem=False)


4) Potenza relativa per banda (%), scala per banda con range [0,100]

plot_band_power(data_dict, scale_mode="per_band", unit_mode="relative",
                use_sem=False, relative_pct=True)



Quando preferire ciascuna

a) µV²: valori assoluti; utile per magnitudini reali (attenzione: Delta tende a dominare).

b) dB: confronti più “compressi” e leggibili, differenze proporzionali diventano additive.

c) Relativa % (per elettrodo): evidenzia come si ridistribuisce lo spettro tra bande nello stesso elettrodo, indipendente dall’ampiezza assoluta
(ottimo per topografie comparative).

Se vuoi aggiungere un’opzione “per Hz” (µV²/Hz, cioè normalizzare per larghezza di banda) o “delta vs baseline” in dB, dimmelo e la integro con 2–3 righe.

                                                            
                                                            
                                                        UNITA' MISURA IN MICROVOLT, CON OPZIONE PER SCALA COLORI
                                                        
                                                        GLOBALE PER TUTTE LE BANDE (in MICROVOLT PERO'!)
                                                        PER BANDA ((in MICROVOLT PERO'!)
                                                        LOGARITMICA (dB)
                                                        

'''




import pickle, numpy as np, matplotlib.pyplot as plt

# ---------- mappa elettrodo → (y,x) ----------
with open('/home/stefano/EEGMMIDB_Analyses/electrode_grid_map.pkl', 'rb') as f:
    electrode_grid_map = pickle.load(f)  # {'Fz':(0,4), 'Cz':(4,4), ...}

band_names = ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']

def split_condition(cond_str):
    return cond_str.split("_vs_", 1) if "_vs_" in cond_str else [f"{cond_str}_0", f"{cond_str}_1"]

def _to_units(X_v2, unit_mode="uV2", relative_pct=True, eps=1e-30):
    """
    X_v2: array (N, 9, 9, 5) in V² (potenza per banda).
    Ritorna (X_plot, unit_label).
    """
    if unit_mode == "uV2":
        return X_v2 * 1e12, "µV²"
    elif unit_mode == "dB":
        return 10 * np.log10(np.maximum(X_v2 * 1e12, eps)), "dB (re: 1 µV²)"
    elif unit_mode == "relative":
        # per trial & elettrodo: normalizza per somma sulle 5 bande
        denom = np.sum(X_v2, axis=-1, keepdims=True)  # (N,9,9,1) in V²
        frac = np.divide(X_v2, np.maximum(denom, eps), where=denom>0)  # 0..1
        if relative_pct:
            return frac * 100.0, "%"
        else:
            return frac, "rel"
    else:
        raise ValueError("unit_mode deve essere 'uV2', 'dB' o 'relative'.")

def plot_band_power(
    data_dict,
    scale_mode="global",      # "global" oppure "per_band"
    unit_mode="uV2",          # "uV2", "dB", "relative"
    use_sem=False,            # True per barre d'errore
    cmap="viridis",
    relative_pct=True
):
    for condition in data_dict:                             # es. 'rest_vs_left_fist'
        cls_names = split_condition(condition)              # ['rest', 'left_fist']

        for dtype in data_dict[condition]:
            for gk in data_dict[condition][dtype]:

                X_v2, y = data_dict[condition][dtype][gk]   # X_v2:(N,9,9,5) in V²
                print(f"\n👀  Figura   {condition}  /  {gk}   (shape X={X_v2.shape})")

                # ---------- CONVERSIONE UNITÀ ----------
                X_plot, unit_label = _to_units(X_v2, unit_mode=unit_mode, relative_pct=relative_pct)

                # ---------- pre-calcola medie ----------
                mean_bar   = []      # (2, 5): media sui trial della media spaziale
                sem_bar    = []      # (2, 5): SEM sui trial (se richiesto)
                mean_maps  = []      # (2, 9,9,5): media su trial (per heatmap)

                for lbl in (0,1):
                    mask = (y == lbl)
                    if not mask.any():
                        mean_bar.append(np.full(5, np.nan))
                        if use_sem:
                            sem_bar.append(np.full(5, np.nan))
                        mean_maps.append(np.full((9,9,5), np.nan))
                    else:
                        Xlbl = X_plot[mask]                     # (n_lbl, 9,9,5)
                        mean_maps.append(Xlbl.mean(axis=0))     # (9,9,5)

                        # bar: media per banda dopo media spaziale per trial
                        trial_means = Xlbl.mean(axis=(1,2))     # (n_lbl, 5)
                        mean_bar.append(trial_means.mean(axis=0))
                        if use_sem:
                            sem_bar.append(trial_means.std(axis=0, ddof=1) / np.sqrt(trial_means.shape[0]))

                # ---------- scale cromatiche ----------
                if scale_mode == "global":
                    # un’unica scala per tutte le bande/condizioni
                    both = np.stack([mean_maps[0], mean_maps[1]], axis=0)   # (2,9,9,5)
                    vmin_global = np.nanmin(both)
                    vmax_global = np.nanmax(both)
                    band_vmin = [vmin_global]*len(band_names)
                    band_vmax = [vmax_global]*len(band_names)
                else:
                    # "per_band": una scala per riga/banda, comune alle due condizioni
                    band_vmin, band_vmax = [], []
                    for b in range(len(band_names)):
                        both_b = np.stack([mean_maps[0][:,:,b], mean_maps[1][:,:,b]], axis=0)  # (2,9,9)
                        band_vmin.append(np.nanmin(both_b))
                        band_vmax.append(np.nanmax(both_b))

                    # Facoltativo: per "relative %" puoi forzare [0,100] per confronti immediati
                    if unit_mode == "relative" and relative_pct:
                        band_vmin = [0.0]*len(band_names)
                        band_vmax = [100.0]*len(band_names)

                # ---------- layout figure ----------
                n_rows, n_cols = 1 + len(band_names), 2   # 6 × 2
                fig, axs = plt.subplots(n_rows, n_cols, figsize=(10, 18), constrained_layout=False)
                fig.suptitle(f"{condition}  –  {dtype} / {gk}", fontsize=12, y=0.97)
                
                #fig.subplots_adjust(top=0.95, hspace=0.35)
                fig.subplots_adjust(top=0.92) 
                
                # --- riga-0: bar-plot (con o senza SEM) ---
                if use_sem:
                    ymax = np.nanmax(np.vstack([
                        (mean_bar[0] + sem_bar[0]),
                        (mean_bar[1] + sem_bar[1])
                    ]))
                else:
                    ymax = np.nanmax(np.vstack([mean_bar[0], mean_bar[1]]))
                ymax = (ymax * 1.05) if np.isfinite(ymax) else 1.0

                for col, lbl in enumerate((0,1)):
                    ax = axs[0, col]
                    if use_sem:
                        ax.bar(band_names, mean_bar[lbl], yerr=sem_bar[lbl], capsize=3)
                    else:
                        ax.bar(band_names, mean_bar[lbl])
                    ax.set_title(f"{cls_names[lbl]}  (label {lbl})")
                    ax.set_ylabel(f"Mean power [{unit_label}]")
                    ax.set_ylim(0, ymax)
                    if col == 1:
                        ax.set_ylabel("")

                # --- righe 1-5: heat-map per banda ---
                for r, band in enumerate(band_names, start=1):
                    for col, lbl in enumerate((0,1)):
                        ax = axs[r, col]
                        im = ax.imshow(
                            mean_maps[lbl][:,:,r-1],
                            cmap=cmap, origin='upper', aspect='equal',
                            vmin=band_vmin[r-1], vmax=band_vmax[r-1]
                        )
                        # etichette elettrodo
                        for lab,(yy,xx) in electrode_grid_map.items():
                            ax.text(xx, yy, lab, ha='center', va='center',
                                    fontsize=5.5, color='black', weight='bold')
                        ax.set_xticks([]); ax.set_yticks([])
                        if col == 0:
                            ax.set_ylabel(band, rotation=90, labelpad=6,
                                          fontsize=9, weight='bold')
                    # colorbar UNA per banda (colonna destra)
                    cbar = fig.colorbar(im, ax=axs[r, 1], fraction=0.046, pad=0.04)
                    cbar.set_label(unit_label)

                plt.show()
                plt.close(fig)

                

In [ ]:
plot_band_power(data_dict, scale_mode="global", unit_mode="uV2", use_sem=False)

In [ ]:
'''

-- IO:
Qui sopra la potenza media in alpha tra le due condizioni ti sembra sensato con l'attivaizone motoria per il movimento di mano sn rispetto a baseline (fermo)?



-- CHATGPT:
Sì—se in alpha (8–12 Hz) vedi potenza più alta a rest e più bassa in left_fist, è coerente: durante l’attivazione motoria (o immaginazione)
ci si aspetta desincronizzazione mu/alpha (ERD) rispetto alla baseline. Per mano sinistra l’ERD dovrebbe essere più marcata controlateralmente, 
quindi su elettrodi centrali destro-emisferici (≈ C4/FC4/CP4), mentre l’emisfero sinistro (C3…) funge da confronto.

Detto questo, due accortezze per capire se l’effetto è davvero “motorio” e non solo attenzione/arousal:

Topografia
Controlla che il calo in alpha sia focalizzato su area sensori-motoria destra più che occipitale. Se vedi un calo molto diffuso/occipitale,
può riflettere attenzione/occhi/vision più che motricità.

Metodica di misura

Usa un indice ERD% = (power_task − power_rest) / power_rest × 100 per banda/elettrodo.

Focalizzati su una ROI: ROI_contra = {FC4, C4, CP4} vs ROI_ipsi = {FC3, C3, CP3}; atteso ERD% più negativo nella ROI controlaterale.

Prova anche beta (13–30 Hz): in movimento/MI tipicamente ERD beta e, se c’è movimento reale, rebound post-movimento.

Se vuoi calcolarlo col tuo schema dati (N,9,9,5):

# alpha = indice 2 nelle tue band
alpha = 2
mask_rest = (y==0)
mask_left = (y==1)
rest_map  = X_v2[mask_rest].mean(axis=0)[:,:,alpha]
left_map  = X_v2[mask_left].mean(axis=0)[:,:,alpha]
erd_alpha = (left_map - rest_map) / (rest_map + 1e-30) * 100  # ERD% (negativo = desync)

# media su ROI (coordinate da electrode_grid_map)
roi_contra = {'FC4','C4','CP4'}
roi_ipsi   = {'FC3','C3','CP3'}
def roi_mean(m):
    idx = [electrode_grid_map[ch] for ch in roi_contra]  # (y,x)
    contra = np.mean([m[y,x] for y,x in idx])
    idx = [electrode_grid_map[ch] for ch in roi_ipsi]
    ipsi = np.mean([m[y,x] for y,x in idx])
    return contra, ipsi

contra_erd, ipsi_erd = roi_mean(erd_alpha)
print('ERD% alpha – contra:', contra_erd, 'ipsi:', ipsi_erd)


Interpretazione attesa: ERD% alpha più negativo su C4-FC4-CP4 rispetto a C3-FC3-CP3. Se questo pattern c’è, 
i tuoi plot sono perfettamente sensati per “left_fist vs rest”. Se non c’è, prova: unità relative (%) nel tuo plot_band_power,
montaggio Laplacian/CSD, e controlla eventuale alpha occipitale (occhi/attenzione).




-- IO:

ok tu mi hai detto infatti "Controlla che il calo in alpha sia focalizzato su area sensori-motoria destra più che occipitale" 
con: Usa un indice ERD% = (power_task − power_rest) / power_rest × 100 per banda/elettrodo. Focalizzati su una ROI: ROI_contra = {FC4, C4, CP4} 
vs ROI_ipsi = {FC3, C3, CP3}; atteso ERD% più negativo nella ROI controlaterale allora mi concentrei su tutta la corteccia motoria per la mia ROI, 
ossia ossia tra FC5 fino a scendere a CP5 e tra FC6 fino a scendere a CP6, tutti quegli elettrodi lì se vedi. 

riesci a correggermi allora il codice per fare questa verifica, e dirmi che stiamo facendo e cosa ci aspettiamo? "


Se vuoi calcolarlo col tuo schema dati (N,9,9,5):

# alpha = indice 2 nelle tue band
alpha = 2
mask_rest = (y==0)
mask_left = (y==1)
rest_map  = X_v2[mask_rest].mean(axis=0)[:,:,alpha]
left_map  = X_v2[mask_left].mean(axis=0)[:,:,alpha]
erd_alpha = (left_map - rest_map) / (rest_map + 1e-30) * 100  # ERD% (negativo = desync)

# media su ROI (coordinate da electrode_grid_map)
roi_contra = {'FC4','C4','CP4'}
roi_ipsi   = {'FC3','C3','CP3'}
def roi_mean(m):
    idx = [electrode_grid_map[ch] for ch in roi_contra]  # (y,x)
    contra = np.mean([m[y,x] for y,x in idx])
    idx = [electrode_grid_map[ch] for ch in roi_ipsi]
    ipsi = np.mean([m[y,x] for y,x in idx])
    return contra, ipsi

contra_erd, ipsi_erd = roi_mean(erd_alpha)
print('ERD% alpha – contra:', contra_erd, 'ipsi:', ipsi_erd)


-- CHATGPT:

Assolutamente sì — se vuoi testare l’ipotesi “ERD α (8–12 Hz) controlaterale più forte sulla cintura motoria”, prendiamo proprio le strisce FC6–C6–CP6 

(controlaterale per mano sinistra) vs FC5–C5–CP5 (ipsilaterale) e calcoliamo l’ERD%:

Cosa facciamo:

stimiamo la mappa media di potenza in alpha per rest (label 0) e left_fist (label 1);

calcoliamo l’ERD% per elettrodo: 
(task−rest)/rest×100  → più negativo = maggiore desincronizzazione;

mediamo l’ERD% sulla ROI controlaterale {FC6, C6, CP6} e sulla ROI ipsilaterale {FC5, C5, CP5};

verifichiamo che la ROI controlaterale sia più negativa.

Cosa ci aspettiamo (left_fist):
ERD%_contra (FC6–C6–CP6) < ERD%_ipsi (FC5–C5–CP5) ≤ 0.
Se l’effetto è motorio, la differenza contra - ipsi dovrebbe essere < 0 (più desync a destra).



Codice pronto da incollare

Usa questo “drop-in” sul tuo (X_v2, y) e la tua electrode_grid_map (nessun altro cambiamento necessario). È robusto a etichette mancanti.


import numpy as np

# --- parametri ---
BAND_NAMES = ['Delta','Theta','Alpha','Beta','Gamma']
ALPHA_IDX = BAND_NAMES.index('Alpha')     # = 2
REST_LABEL = 0
LEFTFIST_LABEL = 1
EPS = 1e-30

# ROI per test "left_fist vs rest" (controlaterale = emisfero destro)
ROI_CONTRA = {'FC6','C6','CP6'}  # destra
ROI_IPSI   = {'FC5','C5','CP5'}  # sinistra

def compute_erd_map(X_v2, y, band_idx=ALPHA_IDX, task_label=LEFTFIST_LABEL, rest_label=REST_LABEL):
    """
    X_v2: (N, 9, 9, 5) in V^2
    y   : (N,) con 0=rest, 1=left_fist (qui)
    ritorna: (erd_map, rest_mean, task_mean) per la banda richiesta
    """
    rest = X_v2[y == rest_label, :, :, band_idx]   # (n_r, 9, 9)
    task = X_v2[y == task_label, :, :, band_idx]   # (n_t, 9, 9)
    if rest.size == 0 or task.size == 0:
        raise ValueError("Mancano trial per rest o task nella selezione corrente.")
    rest_mean = rest.mean(axis=0)                  # (9, 9)
    task_mean = task.mean(axis=0)                  # (9, 9)
    erd_map = (task_mean - rest_mean) / (rest_mean + EPS) * 100.0
    return erd_map, rest_mean, task_mean

def roi_mean_from_map(m2d, roi_set, electrode_grid_map):
    """
    m2d: (9,9) mappa (es. ERD%)
    roi_set: set di etichette canali (es. {'FC6','C6','CP6'})
    electrode_grid_map: dict {'Cz': (y,x), ...}
    """
    coords = [(lab, electrode_grid_map[lab]) for lab in roi_set if lab in electrode_grid_map]
    if not coords:
        raise ValueError(f"Nessun canale della ROI trovato nella mappa: {sorted(list(roi_set))}")
    vals = [m2d[y, x] for _, (y, x) in coords]
    # info su eventuali canali mancanti (non blocca)
    missing = sorted(list(roi_set - set(lab for lab, _ in coords)))
    if missing:
        print(f"[WARN] Canali ROI assenti nella mappa: {missing}")
    return float(np.mean(vals))

# ----------------- ESECUZIONE -----------------
# supponendo che tu abbia: X_v2, y, electrode_grid_map già in scope
erd_alpha, rest_alpha, task_alpha = compute_erd_map(X_v2, y, band_idx=ALPHA_IDX)

contra_erd = roi_mean_from_map(erd_alpha, ROI_CONTRA, electrode_grid_map)
ipsi_erd   = roi_mean_from_map(erd_alpha, ROI_IPSI,   electrode_grid_map)
delta_roi  = contra_erd - ipsi_erd

print(f"ERD% α (left_fist vs rest):")
print(f"  ROI controlaterale (FC6,C6,CP6): {contra_erd:.2f} %")
print(f"  ROI ipsilaterale   (FC5,C5,CP5): {ipsi_erd:.2f} %")
print(f"  Differenza contra - ipsi        : {delta_roi:.2f} %  (atteso < 0)")

# opzionale: verifica che l'ERD occipitale non domini (attenzione/occhi)
ROI_OCCIP = {'O1','Oz','O2'}
try:
    occip_erd = roi_mean_from_map(erd_alpha, ROI_OCCIP, electrode_grid_map)
    print(f"  Controllo occipitale (O1,Oz,O2): {occip_erd:.2f} %  (atteso vicino a 0 per effetto motorio puro)")
except ValueError:
    pass



Nota: il calcolo è fatto direttamente su V²; il rapporto con la baseline annulla qualsiasi fattore di scala (µV², dB, ecc.), 
quindi non serve convertire unità prima dell’ERD%.

Se vuoi integrarlo nel tuo data_dict, basta infilare le ultime 10 righe dentro il loop dove già stampi shape X=..., usando l’X_v2, y di quel blocco.



IO: ok quindi come dovrei integrare questo pezzo di codice qui sopra a quellod del plotting?
'''




In [ ]:
'''

Sotto trovi il patch pronto-incolla che fa esattamente tre cose:

capisce in automatico quale mano è il task (dalla coppia di classi nella condizione);

inverte le ROI (controlaterale/ipsilaterale) in base alla mano;

salta l’ERD quando la condizione non contiene rest (es. left_fist_vs_right_fist).


Con queste tre modifiche:

rest_vs_left_fist → ROI controlaterale = destra;

rest_vs_right_fist → ROI controlaterale = sinistra;

left_fist_vs_right_fist → nessuna riga ERD (niente baseline), oppure Lateralization Index!

1) posso mettere un parametro aggiuntivo 'lateralization_index' come booleno per cui: 

se è false allora mantiene ERD disabilitata per confronto 'mano vs mano', 
mentre se è true allora mi mostra il lateralization index normalizzato di cui mi hai accennato


il valore Δ è sempre “contra − ipsi” e ci aspettiamo Δ < 0 quando la lateralizzazione è corretta.


'''

# -*- coding: utf-8 -*-
# ============================================================
# Plot potenza per banda + ERD% Alpha con ROI auto-invertite
# ============================================================

import pickle, numpy as np, matplotlib.pyplot as plt

# ---------- mappa elettrodo → (y,x) ----------
with open('/home/stefano/EEGMMIDB_Analyses/electrode_grid_map.pkl', 'rb') as f:
    electrode_grid_map = pickle.load(f)  # {'Fz':(0,4), 'Cz':(4,4), ...}

# ---------- bande ----------
BAND_NAMES = ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']
band_names = BAND_NAMES[:]  # alias usato nei plot
ALPHA_IDX = BAND_NAMES.index('Alpha')     # = 2

# ---------- utility ----------
def split_condition(cond_str):
    """'rest_vs_left_fist' -> ['rest','left_fist']."""
    return cond_str.split("_vs_", 1) if "_vs_" in cond_str else [f"{cond_str}_0", f"{cond_str}_1"]

def _to_units(X_v2, unit_mode="uV2", relative_pct=True, eps=1e-30):
    """
    X_v2: array (N, 9, 9, 5) in V² (potenza per banda).
    Ritorna (X_plot, unit_label).
    """
    if unit_mode == "uV2":
        return X_v2 * 1e12, "µV²"
    
    elif unit_mode == "dB":
        return 10 * np.log10(np.maximum(X_v2 * 1e12, eps)), "dB (re: 1 µV²)"
    
    elif unit_mode == "relative":
        # per trial & elettrodo: normalizza per somma sulle 5 bande
        denom = np.sum(X_v2, axis=-1, keepdims=True)  # (N,9,9,1) in V²
        frac = np.divide(X_v2, np.maximum(denom, eps), where=denom>0)  # 0..1
        return (frac * 100.0, "%") if relative_pct else (frac, "rel")
    
    else:
        raise ValueError("unit_mode deve essere 'uV2', 'dB' o 'relative'.")

# ---------- costanti ERD ----------
EPS = 1e-30


#1) Aggiungere il “lateralization_index” (anche per mano vs mano)

#Di seguito ti passo patch minimali da copiare nel tuo file.
#A) metti queste funzioni helper vicino a compute_erd_map:


'''

1) Cosa significano “LI_power α (left/right)” e la riga ERD?

ERD% α – ROI contra/ipsi, Δ
Misura la desincronizzazione in alpha/mu (8–12 Hz) del task rispetto a rest:

ERD % = (task − rest/rest) × 100


più negativa ⇒ maggiore desincronizzazione (effetto motorio).
• Δ = contra − ipsi: atteso < 0 nel movimento (controlateralità).

LI_power α (left/right)
Quando non c’è rest (mano vs mano) mostriamo l’indice di lateralizzazione su potenza:


LI= (Pow contra + Pow ipsi)/ (Pow contra − Pow ipsi)

→ range [−1,1]

• LI > 0: la potenza alpha è maggiore nell’emisfero controlaterale (per quella mano).
• LI < 0: maggiore nell’ipsilaterale.

È utile per mano vs mano; con il rest conviene usare l’ERD (più interpretabile fisiologicamente).


Posso farne un plot?

Sì. Sotto trovi una patch che:

in mano vs mano disegna una barra/marker orizzontale su [−1,1] con il valore LI_power α per left e right;
mantiene la mappa ERD% quando è presente rest.



2) Letteratura su “lateralization index” (EEG mu/alpha)

Sì, l’indice di lateralizzazione è usato sia su potenza (contra vs ipsi) sia su ERD:

Forme su potenza (tipico α somatosensoriale/attenzione): molti lavori calcolano contra − ipsi o 
(contra−ipsi)/(contra+ipsi) per la banda α --> ad esempio vedi 

Haegens et al., J Neurosci 2011 (α-lateralization index definito su canali selezionati) 
https://www.jneurosci.org/content/31/14/5197?utm_source=chatgpt.com

Bacigalupo & Luck 2019 (lateralized α = contra − ipsi). 
https://pmc.ncbi.nlm.nih.gov/articles/PMC6382983/?utm_source=chatgpt.com



Forme su ERD (mu/β motorie): nelle SMR motorie è comune calcolare l’ERD controlaterale e confrontarla con l’ipsilaterale; 
alcuni studi e applicazioni di neurofeedback definiscono un LI basato su ERD

(es. LI= (ERD contra − ERD ipsi) /(∣ERD contra∣ + ∣ERD ipsi∣). 

Vedi Hayashi et al., NeuroImage 2020 (formula esplicita del LI), insieme alla letteratura classica di Pfurtschellera su ERD/ERS e dominanza controlaterale. 

https://www.sciencedirect.com/science/article/pii/S1053811920307849?utm_source=chatgpt.com
https://doc.ml.tu-berlin.de/bbci/teaching/PfuSil99.pdf?utm_source=chatgpt.com
https://pubmed.ncbi.nlm.nih.gov/11322970/


Evidenza di dominanza controlaterale in mu-ERD durante esecuzione/MI: review e case-studies storici (Pfurtscheller et al.) 
e lavori più recenti confermano il pattern atteso. 


https://doc.ml.tu-berlin.de/bbci/teaching/PfuSil99.pdf?utm_source=chatgpt.com
https://pmc.ncbi.nlm.nih.gov/articles/PMC6261430/?utm_source=chatgpt.com
https://pub.ista.ac.at/~schloegl/publications/pfurtscheller2006.pdf?utm_source=chatgpt.com



In breve: il tuo LI normalizzato è sensato; su “mano vs mano” puoi usare la versione su potenza, mentre su “mano vs rest” preferisci la versione su ERD.


Forse non conviene estendere ad altri elettrodi per estendere la ROI su anche

FC3, FC1,C5,C3,C1 e CP3 e CP1 per lato sinistro e 
FC2, FC4,C6,C2,C4 e CP4 e CP2 per lato destro?


2) ROI “wide motor strip”

Aggiungo un parametro roi_variant:

"narrow" (attuale: {FC6,C6,CP6} vs {FC5,C5,CP5})

"wide" (estesa come richiesto):

sinistra: {FC5,FC3,FC1, C5,C3,C1, CP5,CP3,CP1}

destra: {FC6,FC4,FC2, C6,C4,C2, CP6,CP4,CP2}


'''



#Aggiunge:

#import per gli inset
#funzioni per bootstrap CI e permutation test
#calcolo trial-wise per ERD e LI
#rendering degli inset con barre/CI e p-value


'''
Qui sotto c’è una patch diff minimale che:

allarga la figura quando c’è la riga ERD/LI;

invece di sovrapporre l’inset lo mette a destra della mappa ERD con make_axes_locatable;

sposta anche la colorbar ancora più a destra, così non si schiaccia nulla

'''

import pickle, numpy as np, matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
#from mpl_toolkits.axes_grid1 import make_axes_locatable



# ---------- statistiche & CI (no SciPy) ----------
def _bootstrap_ci_mean(x, n_boot=5000, alpha=0.05, rng=None):
    x = np.asarray(x, float)
    if rng is None:
        rng = np.random.default_rng()
    idx = rng.integers(0, len(x), size=(n_boot, len(x)))
    m = x[idx].mean(axis=1)
    lo, hi = np.percentile(m, [100*alpha/2, 100*(1-alpha/2)])
    return lo, hi


def _perm_test_signed(d, n_perm=10000, rng=None):
    """
    Test a 0 media per differenze appaiate: d = contra - ipsi (per trial).
    Ritorna p-value a due code.
    """
    d = np.asarray(d, float)
    obs = np.mean(d)
    if rng is None:
        rng = np.random.default_rng()
    signs = rng.choice([-1.0, 1.0], size=(n_perm, len(d)))
    perm_means = (signs * d).mean(axis=1)
    p = (np.sum(np.abs(perm_means) >= np.abs(obs)) + 1) / (n_perm + 1)
    return float(p), obs

def _perm_test_onesample(x, n_perm=10000, rng=None):
    """
    One-sample vs 0 (per LI): flip di segno. Ritorna p-value a due code.
    """
    x = np.asarray(x, float)
    if rng is None:
        rng = np.random.default_rng()
    obs = np.mean(x)
    signs = rng.choice([-1.0, 1.0], size=(n_perm, len(x)))
    perm_means = (signs * x).mean(axis=1)
    p = (np.sum(np.abs(perm_means) >= np.abs(obs)) + 1) / (n_perm + 1)
    return float(p), obs

def _roi_mean_trial(m2d, roi_set, electrode_grid_map):
    coords = [electrode_grid_map[lab] for lab in roi_set if lab in electrode_grid_map]
    return float(np.mean([m2d[y, x] for (y, x) in coords]))



def compute_erd_trials(X_v2, y, band_idx, task_label, rest_label, roi_contra, roi_ipsi):
    """
    ERD% per trial di TASK rispetto alla media REST, poi media ROI -> due vettori trial-wise.
    """
    rest_mean = X_v2[y==rest_label, :, :, band_idx].mean(axis=0)  # (9,9)
    task_trials = X_v2[y==task_label, :, :, band_idx]             # (T,9,9)
    erd_trials_maps = (task_trials - rest_mean) / (rest_mean + EPS) * 100.0
    contra = [ _roi_mean_trial(m, roi_contra, electrode_grid_map) for m in erd_trials_maps ]
    ipsi   = [ _roi_mean_trial(m, roi_ipsi,   electrode_grid_map) for m in erd_trials_maps ]
    return np.array(contra), np.array(ipsi)




def compute_li_erd(erd_map, roi_contra, roi_ipsi, eps=EPS):
    ec = roi_mean_from_map(erd_map, roi_contra, electrode_grid_map)
    ei = roi_mean_from_map(erd_map, roi_ipsi,   electrode_grid_map)
    li = (ec - ei) / (abs(ec) + abs(ei) + eps)   # LI normalizzato su ERD
    return li, ec, ei

def compute_li_power(power_map, roi_contra, roi_ipsi, eps=EPS):
    pc = roi_mean_from_map(power_map, roi_contra, electrode_grid_map)
    pi = roi_mean_from_map(power_map, roi_ipsi,   electrode_grid_map)
    li = (pc - pi) / (pc + pi + eps)             # LI normalizzato su potenza
    return li, pc, pi


def compute_erd_map(X_v2, y, band_idx=ALPHA_IDX, task_label=None, rest_label=None):
    """
    Calcola ERD% = (task - rest) / rest * 100 per la banda richiesta.
    X_v2: (N, 9, 9, 5) in V^2; y: (N,)
    Ritorna: (erd_map, rest_mean, task_mean) con shape (9,9) per mappa.
    """
    if task_label is None or rest_label is None:
        raise ValueError("task_label e rest_label devono essere forniti.")
    rest = X_v2[y == rest_label, :, :, band_idx]   # (n_r, 9, 9)
    task = X_v2[y == task_label, :, :, band_idx]   # (n_t, 9, 9)
    if rest.size == 0 or task.size == 0:
        raise ValueError("Mancano trial per rest o task nella selezione corrente.")
    rest_mean = rest.mean(axis=0)                  # (9, 9)
    task_mean = task.mean(axis=0)                  # (9, 9)
    erd_map = (task_mean - rest_mean) / (rest_mean + EPS) * 100.0
    return erd_map, rest_mean, task_mean

def roi_mean_from_map(m2d, roi_set, electrode_grid_map):
    """
    Media della mappa 'm2d' su una ROI definita da etichette canali (es. {'FC6','C6','CP6'}).
    """
    coords = [(lab, electrode_grid_map[lab]) for lab in roi_set if lab in electrode_grid_map]
    if not coords:
        raise ValueError(f"Nessun canale della ROI trovato nella mappa: {sorted(list(roi_set))}")
    vals = [m2d[y, x] for _, (y, x) in coords]
    missing = sorted(list(roi_set - set(lab for lab, _ in coords)))
    if missing:
        print(f"[WARN] Canali ROI assenti nella mappa: {missing}")
    return float(np.mean(vals))

def _label_of(y_label_names, target):
    """Ricava l'id (0/1) dato il nome classe; y_label_names: {0:'rest',1:'left_fist'}."""
    for k, v in y_label_names.items():
        if v == target:
            return k
    return None

# ---------- funzione principale ----------
def plot_band_power(
    data_dict,
    scale_mode="global",
    unit_mode="uV2",
    use_sem=False,
    cmap="viridis",
    relative_pct=True,
    show_alpha_erd=True,
    cmap_erd="RdBu_r",
    lateralization_index=False,
    roi_variant="narrow",
    erd_stats_layout="side"   # "side" | "center"
):
    for condition in data_dict:
        cls_names = split_condition(condition)

        for dtype in data_dict[condition]:
            for gk in data_dict[condition][dtype]:

                X_v2, y = data_dict[condition][dtype][gk]
                print(f"\n👀  Figura   {condition}  /  {gk}   (shape X={X_v2.shape})")

                # ---------- ERD% Alpha (solo se c'è 'rest' + una mano) ----------
                y_label_names = {0: cls_names[0], 1: cls_names[1]}
                has_rest = ('rest' in cls_names)
                erd_alpha = None
                roi_title = ""

                li_power_left = None
                li_power_right = None

                erd_contra_trials = erd_ipsi_trials = None
                p_erd = None
                ci_contra = ci_ipsi = None
                li_left_trials = li_right_trials = None
                p_li_left = p_li_right = None

                if has_rest:
                    if 'left_fist' in cls_names:
                        hand = 'left_fist'
                        task_label = _label_of(y_label_names, 'left_fist')
                        rest_label = _label_of(y_label_names, 'rest')
                        if roi_variant == "wide":
                            ROI_contra = {'FC6','FC4','FC2','C6','C4','C2','CP6','CP4','CP2'}
                            ROI_ipsi   = {'FC5','FC3','FC1','C5','C3','C1','CP5','CP3','CP1'}
                        else:
                            ROI_contra = {'FC6','C6','CP6'}
                            ROI_ipsi   = {'FC5','C5','CP5'}
                    elif 'right_fist' in cls_names:
                        hand = 'right_fist'
                        task_label = _label_of(y_label_names, 'right_fist')
                        rest_label = _label_of(y_label_names, 'rest')
                        if roi_variant == "wide":
                            ROI_contra = {'FC5','FC3','FC1','C5','C3','C1','CP5','CP3','CP1'}
                            ROI_ipsi   = {'FC6','FC4','FC2','C6','C4','C2','CP6','CP4','CP2'}
                        else:
                            ROI_contra = {'FC5','C5','CP5'}
                            ROI_ipsi   = {'FC6','C6','CP6'}
                    else:
                        hand = None
                        task_label = None
                        rest_label = _label_of(y_label_names, 'rest')

                    if hand is not None and task_label is not None:
                        erd_alpha, rest_alpha, task_alpha = compute_erd_map(
                            X_v2, y, band_idx=ALPHA_IDX, task_label=task_label, rest_label=rest_label
                        )
                        erd_contra_trials, erd_ipsi_trials = compute_erd_trials(
                            X_v2, y, ALPHA_IDX, task_label, rest_label, ROI_contra, ROI_ipsi
                        )
                        ci_contra = _bootstrap_ci_mean(erd_contra_trials)
                        ci_ipsi   = _bootstrap_ci_mean(erd_ipsi_trials)
                        p_erd, _  = _perm_test_signed(erd_contra_trials - erd_ipsi_trials)

                        contra_erd = roi_mean_from_map(erd_alpha, ROI_contra, electrode_grid_map)
                        ipsi_erd   = roi_mean_from_map(erd_alpha, ROI_ipsi,   electrode_grid_map)
                        delta_roi  = contra_erd - ipsi_erd

                        print(f"ERD% α ({hand} vs rest): contra={contra_erd:.2f}%  ipsi={ipsi_erd:.2f}%  "
                              f"Δ={delta_roi:.2f}%  [atteso Δ<0]  p_paired={p_erd:.4f}")

                        vmax_erd = float(np.nanmax(np.abs(erd_alpha)))
                        vmin_erd = -vmax_erd
                        roi_title = (f"ROI: contra={contra_erd:.1f}%  ipsi={ipsi_erd:.1f}%  "
                                     f"Δ={delta_roi:.1f}% (atteso<0)")
                else:
                    print("ERD% α non mostrata: condizione senza 'rest'.")
                    if lateralization_index and ('left_fist' in cls_names) and ('right_fist' in cls_names):
                        lbl_left  = 0 if y_label_names[0]=='left_fist'  else 1
                        lbl_right = 0 if y_label_names[0]=='right_fist' else 1
                        ROI_R = {'FC6','FC4','FC2','C6','C4','C2','CP6','CP4','CP2'} if roi_variant=="wide" else {'FC6','C6','CP6'}
                        ROI_L = {'FC5','FC3','FC1','C5','C3','C1','CP5','CP3','CP1'} if roi_variant=="wide" else {'FC5','C5','CP5'}
                        left_trials  = X_v2[y==lbl_left , :, :, ALPHA_IDX]
                        right_trials = X_v2[y==lbl_right, :, :, ALPHA_IDX]
                        li_left_trials = []
                        for m in left_trials:
                            pc = _roi_mean_trial(m, ROI_R, electrode_grid_map)
                            pi = _roi_mean_trial(m, ROI_L, electrode_grid_map)
                            li_left_trials.append( (pc - pi) / (pc + pi + EPS) )
                        li_right_trials = []
                        for m in right_trials:
                            pc = _roi_mean_trial(m, ROI_L, electrode_grid_map)
                            pi = _roi_mean_trial(m, ROI_R, electrode_grid_map)
                            li_right_trials.append( (pc - pi) / (pc + pi + EPS) )
                        li_left_trials  = np.asarray(li_left_trials)
                        li_right_trials = np.asarray(li_right_trials)
                        li_power_left  = float(np.mean(li_left_trials))
                        li_power_right = float(np.mean(li_right_trials))
                        p_li_left,  _ = _perm_test_onesample(li_left_trials)
                        p_li_right, _ = _perm_test_onesample(li_right_trials)
                        print(f"LI_power α: left={li_power_left:+.2f} (p={p_li_left:.4f})  "
                              f"right={li_power_right:+.2f} (p={p_li_right:.4f})")

                # ---------- CONVERSIONE UNITÀ ----------
                X_plot, unit_label = _to_units(X_v2, unit_mode=unit_mode, relative_pct=relative_pct)

                # ---------- pre-calcola medie ----------
                mean_bar, sem_bar, mean_maps = [], [], []
                for lbl in (0,1):
                    mask = (y == lbl)
                    if not mask.any():
                        mean_bar.append(np.full(5, np.nan))
                        if use_sem: sem_bar.append(np.full(5, np.nan))
                        mean_maps.append(np.full((9,9,5), np.nan))
                    else:
                        Xlbl = X_plot[mask]
                        mean_maps.append(Xlbl.mean(axis=0))
                        trial_means = Xlbl.mean(axis=(1,2))
                        mean_bar.append(trial_means.mean(axis=0))
                        if use_sem:
                            sem_bar.append(trial_means.std(axis=0, ddof=1) / np.sqrt(trial_means.shape[0]))

                # ---------- scale cromatiche ----------
                if scale_mode == "global":
                    both = np.stack([mean_maps[0], mean_maps[1]], axis=0)
                    vmin_global = np.nanmin(both); vmax_global = np.nanmax(both)
                    band_vmin = [vmin_global]*len(band_names)
                    band_vmax = [vmax_global]*len(band_names)
                else:
                    band_vmin, band_vmax = [], []
                    for b in range(len(band_names)):
                        both_b = np.stack([mean_maps[0][:,:,b], mean_maps[1][:,:,b]], axis=0)
                        band_vmin.append(np.nanmin(both_b))
                        band_vmax.append(np.nanmax(both_b))
                    if unit_mode == "relative" and relative_pct:
                        band_vmin = [0.0]*len(band_names)
                        band_vmax = [100.0]*len(band_names)

                # ---------- layout figure ----------
                extra_rows = 1 if ((show_alpha_erd and (erd_alpha is not None)) \
                                   or (lateralization_index and (li_power_left is not None))) else 0

                n_rows, n_cols = 1 + len(band_names) + extra_rows, 2
                fig_w = 16 if extra_rows else 10
                fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_w, 20 if extra_rows else 18),
                                        constrained_layout=False)

                fig.suptitle(f"{condition}  –  {dtype} / {gk}", fontsize=12, y=0.97)
                # quando il pannello ERD è a destra, lasciamo più aria sul bordo destro
                if extra_rows and erd_stats_layout == "side":
                    fig.subplots_adjust(top=0.94, right=0.72)
                else:
                    fig.subplots_adjust(top=0.94, right=0.94)

                # --- riga-0: bar-plot ---
                if use_sem:
                    ymax = np.nanmax(np.vstack([(mean_bar[0] + sem_bar[0]), (mean_bar[1] + sem_bar[1])]))
                else:
                    ymax = np.nanmax(np.vstack([mean_bar[0], mean_bar[1]]))
                ymax = (ymax * 1.05) if np.isfinite(ymax) else 1.0

                for col, lbl in enumerate((0,1)):
                    ax = axs[0, col]
                    if use_sem:
                        ax.bar(band_names, mean_bar[lbl], yerr=sem_bar[lbl], capsize=3)
                    else:
                        ax.bar(band_names, mean_bar[lbl])
                    ax.set_title(f"{cls_names[lbl]}  (label {lbl})")
                    ax.set_ylabel(f"Mean power [{unit_label}]")
                    ax.set_ylim(0, ymax)
                    if col == 1:
                        ax.set_ylabel("")

                # --- righe 1-5: heat-map per banda ---
                for r, band in enumerate(band_names, start=1):
                    for col, lbl in enumerate((0,1)):
                        ax = axs[r, col]
                        im = ax.imshow(
                            mean_maps[lbl][:,:,r-1],
                            cmap=cmap, origin='upper', aspect='equal',
                            vmin=band_vmin[r-1], vmax=band_vmax[r-1]
                        )
                        for lab,(yy,xx) in electrode_grid_map.items():
                            ax.text(xx, yy, lab, ha='center', va='center',
                                    fontsize=5.5, color='black', weight='bold')
                        ax.set_xticks([]); ax.set_yticks([])
                        if col == 0:
                            ax.set_ylabel(band, rotation=90, labelpad=6, fontsize=9, weight='bold')
                    # colorbar UNA per banda (sottile)
                    cbar = fig.colorbar(im, ax=axs[r, 1], fraction=0.030, pad=0.018)
                    cbar.set_label(unit_label)

                # --- riga finale opzionale: ERD% Alpha ---
                if show_alpha_erd and (erd_alpha is not None):
                    r = 1 + len(band_names)
                    for col in (0, 1):
                        ax = axs[r, col]
                        im_erd = ax.imshow(
                            erd_alpha, cmap=cmap_erd, origin='upper', aspect='equal',
                            vmin=vmin_erd, vmax=vmax_erd
                        )
                        for lab,(yy,xx) in electrode_grid_map.items():
                            ax.text(xx, yy, lab, ha='center', va='center',
                                    fontsize=5.5, color='black', weight='bold')
                        ax.set_xticks([]); ax.set_yticks([])

                    # etichetta sinistra
                    axs[r, 0].set_ylabel("Alpha ERD% (task−rest)/rest", rotation=90,
                                         labelpad=6, fontsize=9, weight='bold')

                    # titolo centrato leggermente più basso
                    fig.canvas.draw()
                    bboxL = axs[r, 0].get_position()
                    bboxR = axs[r, 1].get_position()
                    x_center = 0.5 * (bboxL.x0 + bboxR.x1)
                    y_top    = max(bboxL.y1, bboxR.y1) + 0.008
                    fig.text(x_center, y_top, roi_title, ha='center', va='bottom',
                             fontsize=12, weight='bold')

                    # pannello statistico SOLO qui (ERD Alpha)
                    y0     = bboxR.y0
                    h      = bboxR.height
                    means  = [np.mean(erd_contra_trials), np.mean(erd_ipsi_trials)]
                    ci_con = ci_contra; ci_ips = ci_ipsi
                    cis    = np.array([[means[0]-ci_con[0], ci_con[1]-means[0]],
                                       [means[1]-ci_ips[0],  ci_ips[1]-means[1]]])

                    if erd_stats_layout == "side":
                        # ancoriamo la colorbar al bordo destro della figura (più a destra di così non si può)
                        right_edge = 0.010   # margine dal bordo destro della figura
                        gap        = 0.012   # spazio tra bar e colorbar
                        cb_w       = 0.022   # colorbar sottile ma leggibile
                        # bar_w occupa il resto utile ma tenuto sottile
                        bar_w_max  = 0.12
                        bar_w_min  = 0.06
                        # calcoliamo una larghezza che non sfori
                        avail_for_bar = 1.0 - right_edge - cb_w - gap - (bboxR.x1 + gap)
                        bar_w = max(bar_w_min, min(bar_w_max, avail_for_bar))
                        # posizioni: cb tutto a destra, bar subito a sinistra
                        x_cb  = 1.0 - right_edge - cb_w
                        x_bar = x_cb - gap - bar_w
                        ax_bar = fig.add_axes([x_bar, y0 + 0.12*h, bar_w, 0.76*h])
                        ax_cb  = fig.add_axes([x_cb,  y0 + 0.06*h, cb_w,  0.88*h])
                        cbar   = fig.colorbar(im_erd, cax=ax_cb)
                        cbar.set_label("ERD% (Alpha)")
                        ax_bar.bar([0,1], means, yerr=cis.T, capsize=3)
                        ax_bar.set_xticks([0,1]); ax_bar.set_xticklabels(['contra','ipsi'])
                        ax_bar.set_ylabel('ERD% α')
                        ax_bar.set_title(f"paired p={p_erd:.3f}", fontsize=9)

                    else:
                        # pannello in mezzo (sempre e solo in ultima riga)
                        gap   = max(0.02, bboxR.x0 - bboxL.x1)
                        mid_w = gap * 0.68
                        x_mid = bboxL.x1 + (gap - mid_w)/2
                        ax_mid = fig.add_axes([x_mid, y0 + 0.10*h, mid_w, 0.80*h])
                        ax_mid.bar([0,1], means, yerr=cis.T, capsize=3)
                        ax_mid.set_xticks([0,1]); ax_mid.set_xticklabels(['contra','ipsi'])
                        ax_mid.set_ylabel('ERD% α')
                        ax_mid.set_title(f"paired p={p_erd:.3f}", fontsize=9)
                        cax = inset_axes(ax_mid, width="8%", height="78%", loc="center right", borderpad=1.0)
                        cbar = fig.colorbar(im_erd, cax=cax)
                        cbar.set_label("ERD% (Alpha)")

                elif lateralization_index and (li_power_left is not None):
                    r = 1 + len(band_names)
                    axL, axR = axs[r, 0], axs[r, 1]
                    axL.axis('off'); axR.axis('off')
                    axL.text(0.5, 0.55, f"LI_power α (left)  = {li_power_left:+.2f}",
                             ha='center', va='center', fontsize=11, weight='bold')
                    axR.text(0.5, 0.55, f"LI_power α (right) = {li_power_right:+.2f}",
                             ha='center', va='center', fontsize=11, weight='bold')

                plt.show()
                plt.close(fig)



In [ ]:
'''
La riga ERD% Alpha apparirà per le condizioni con rest (p.es. rest_vs_left_fist, rest_vs_right_fist),
con ROI auto-invertite in base alla mano. Per condizioni “mano vs mano” non verrà mostrata.


Con condizioni rest_vs_left_fist / rest_vs_right_fist → vedrai la mappa ERD% α e, nel titolo della colonna destra, anche i valori ROI (come già fai).

Con left_fist_vs_right_fist (senza rest) → niente ERD; in fondo comparirà la riga con LI_power α (left/right).


# ---------- funzione principale ----------
def plot_band_power(
    data_dict,
    scale_mode="global",      # "global" oppure "per_band"
    unit_mode="uV2",          # "uV2", "dB", "relative"
    use_sem=False,            # True per barre d'errore
    cmap="viridis",
    relative_pct=True,
    show_alpha_erd=True,      # aggiunge riga con ERD% Alpha se presente 'rest'
    cmap_erd="RdBu_r",         # colormap divergente per ERD (negativo/positivo)
    lateralization_index=False,
    roi_variant="narrow"      # "narrow" | "wide"
):

'''


plot_band_power(
    data_dict,
    scale_mode="per_band",
    unit_mode="uV2",
    use_sem=False,
    relative_pct=True,
    show_alpha_erd=True,
    cmap_erd="RdBu_r",
    lateralization_index=False,   # abilita LI
    roi_variant="narrow",
    erd_stats_layout = "center"
)



In [ ]:
data_dict.keys()

In [ ]:
data_dict['rest_vs_left_fist'].keys()

In [ ]:
data_dict['rest_vs_left_fist']['spectrograms'].keys()

In [ ]:
len(data_dict['rest_vs_left_fist']['spectrograms']['familiar_th'])

In [ ]:
data_dict['rest_vs_left_fist']['spectrograms']['familiar_th'][0].shape

In [ ]:
data_dict['rest_vs_left_fist']['spectrograms']['familiar_th'][1].shape

#### **Sweep Configuration - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS**


#### **Sweep separati per ciascuno dei modelli CNN3D e CNN Sep**

In [ ]:
'''
N.B. 

PER SAPERE A QUALE COMBINAZIONE DI FATTORI CORRISPONDONO I DATI (i.e, X_train, X_val, X_test, y_train, y_val, y_test)

MI CREO UN DIZIONARIO ULTERIORE, 'DATA_DICT_PREPROCESSED' CHE CONTIENE PER OGNI COMBINAZIONE DI FATTORI I DATI SPLITTATI

IN QUESTO MODO, QUANDO FORNISCO ALLA FUNZIONE 'TRAINING_SWEEP' LA TUPLA CON I VARI DATI ((TRAIN, VAL E TEST))
IO POSSO CAPIRE A QUALE COMBINAZIONI DI FATTORI CORRISPONDE QUELLA TUPLA DI DATI (TRAIN, VAL E TEST)


INOLTRE,
MI CREO ANCHE UNA LISTA DI TUPLE DI STRINGHE, DOVE OGNI TUPLA CONTIENE LE STRINGHE DELLE CHIAVI USATE 
PER LA GENERAZIONE DI DATA_DICT_PREPROCESSED.

IN QUESTO MODO, MI ASSICURO CHE SIA UNA COERENZA TRA LA CREAZIONE DEI 'NAME' E 'TAG' DELLA RUN
E
LA CORRETTA ESTRAZIONE DEI DATI (OSSIA I DATI DI QUALE CONDIZIONE SPERIMENTALE, QUALI EEG INPUT, E DA CHI PROVENGONO!)  


Questo approccio permette di garantire la corrispondenza tra 

1) le chiavi dei dati pre‐processati e 
2) la configurazione delle runs su W&B

andando a creare due strutture in parallelo:

- data_dict_preprocessed – che contiene, per ogni combinazione (condition, data_type, category_subject), 
                            la tupla dei dati già suddivisi (X_train, X_val, X_test, y_train, y_val, y_test);
                            
- sweeps_id – che contiene, per ogni combinazione (condition, data_type, category_subject), 
              sia la stringa univoca dello sweep ID, che l'insieme delle stringhe che formano la combinazione (condition, data_type, category_subject)



LOOP DI PREPARAZIONE DATI (FINO A DATASET SPLITTING)
'''



from sklearn.model_selection import train_test_split

#A QUESTO PUNTO PER OGNI DATASET, FACCIO STEP PRIMA DELLO SWEEP

# Set per tenere traccia dei dataset già elaborati
processed_datasets = set()

# Seleziona il dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Modelli che useremo nei sweep
MODEL_LIST = ["CNN3D_LSTM_FC", "SeparableCNN2D_LSTM_FC"]


# Dizionario per salvare gli sweep ID associati a ogni condizione sperimentale

'''sweep_ids_for_models contiene la struttura che mi serve da copiare per best_models''' 
sweep_ids_for_models = {}

'''sweep_ids contiene la struttura che mi serve da copiare per iterare sui singoli swweps di ogni combinazione di fattori'''
sweep_ids = {}  

'''DIZIONARIO CHE VIENE FORNITO IN INGRESSO A TRAINING_SWEEP'''
# Dizionario per salvare la tupla di dati già preprocessati
data_dict_preprocessed = {}


# Loop di addestramento e test per ogni condizione sperimentale
for condition, data_types in data_dict.items():  # Itera sulle condizioni sperimentali
    
    data_dict_preprocessed[condition] = {}
    
    # Aggiungi al dizionario sweep_ids
    if condition not in sweep_ids:
        sweep_ids[condition] = {}
        
        '''sweep_ids_for_models'''
        sweep_ids_for_models[condition] = {}
        
    for data_type, categories in data_types.items():  # Itera sui tipi di dati (1_20, 1_45, wavelet)
        
        data_dict_preprocessed[condition][data_type] = {}
        
        if data_type not in sweep_ids[condition]:
            sweep_ids[condition][data_type] = {}
            
            '''sweep_ids_for_models'''
            sweep_ids_for_models[condition][data_type] = {}
            
        for category_subject, (X_data, y_data) in categories.items():  # Itera sulle coppie category_subject
            
            # 1. Prepara spazio nei dizionari: sotto category_subject, un dict per ogni modello
            
            data_dict_preprocessed[condition][data_type][category_subject] = None
            
            if category_subject not in sweep_ids[condition][data_type]:
                
                sweep_ids[condition][data_type][category_subject] = {}
                
                '''NUOVA MODIFICA'''
                sweep_ids[condition][data_type][category_subject] = {
                model: [] for model in MODEL_LIST
                }

                '''sweep_ids_for_models'''
                sweep_ids_for_models[condition][data_type][category_subject] = {}
                
                '''NUOVA MODIFICA'''
                sweep_ids_for_models[condition][data_type][category_subject] = {
                model: [] for model in MODEL_LIST
                }
                
            print(f"\n\n\033[1mEstrazione Dati\033[0m della Chiave \033[1m{condition}_{data_type}_{category_subject}\033[0m")
            
            # Controlla se il dataset è già stato elaborato (se la chiave è già nel set)
            if (condition, data_type, category_subject) in processed_datasets:
                print(f"⚠️ ATTENZIONE: Il dataset {condition} - {data_type} - {category_subject} è già stato elaborato! Salto iterazione...")
                continue  # Salta se il dataset è già stato processato

            # Aggiungi il dataset al set
            processed_datasets.add((condition, data_type, category_subject))

            X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
            
            data_dict_preprocessed[condition][data_type][category_subject] = (X_train, X_val, X_test, y_train, y_val, y_test)
            
            # Puoi anche aggiungere altri print per verificare la dimensione dei set
            print(f"\033[1mDataset Splitting\033[0m: Train Set Shape: {X_train.shape}, Validation Set Shape: {X_val.shape}, Test Set Shape: {X_test.shape}")

            
print(f"\nCreato \033[1mdata_dict_preprocessed\033[0m")


In [ ]:
data_dict['rest_vs_left_fist'].keys()

In [ ]:
print(data_dict_preprocessed.keys())
print(data_dict_preprocessed['rest_vs_left_fist'].keys())
print(data_dict_preprocessed['rest_vs_left_fist']['spectrograms'].keys())
print(type(data_dict_preprocessed['rest_vs_left_fist']['spectrograms'].keys()))

#All'interno, c'è una tupla, di 6 elementi!
print(type(data_dict_preprocessed['rest_vs_left_fist']['spectrograms']['familiar_th']))

#I 6 elementi della tupla sono X_train, X_val, X_test, y_train, y_val, y_test !
print(len(data_dict_preprocessed['rest_vs_left_fist']['spectrograms']['familiar_th']))

In [ ]:
print(sweep_ids_for_models)

In [ ]:
print(sweep_ids)

In [ ]:
print(sweep_ids_for_models)

In [ ]:
''' 

                                                                    AGGIORNATA AL 19 LUGLIO
                                                                    
                                                                    
#"learning rate : {"value"[1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]"}
#"n_epochs": {"value": 100},
# "patience": {"value": 12},
#"batch_size": {"values": [16, 24, 32, 48, 64, 72, 84, 96]}
#"standardization": {"values": [True, False]}, 
# "beta1": {"values": [0.8, 0.85, 0.9, 0.95]},
#  "beta2": {"values": [0.98, 0.99, 0.995, 0.999]},
#  "eps": {"value": [1e-8, 1e-7, 1e-6, 1e-5]}                                                                                                                            



sweep_config = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]}, # fissato al valore di default del paper

        "weight_decay":  {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        
        
        "model_name":{"values": ['CNN3D_LSTM_FC']},

        "batch_size": {"values": [32, 48, 64, 96]},

        "standardization":{"values": [True, False]},

        "beta1": {"values": [0.9, 0.95]},

        "beta2": {"values": [0.99, 0.995]},
        
        "eps": {"values": [1e-8, 1e-7]},
        
        #In questo modo:
        
        "use_lstm":      {"values":[True, False]},
        "lstm_hidden":   {"values":[32]},
        "dropout":       {"values":[0.5]},
        
    }
}


'''


#Tutti gli sweep saranno organizzati sotto lo stesso progetto,
#che corrisponde alla coppia di condizioni sperimentali corrente (i.e., exp_cond).

#Questo significa che tutte le runs che verranno lanciate con quello sweep, 
#saranno associate a quella specifica coppia di condizioni sperimentali corrente.

#Dato che sto iterando su ogni coppia di condizioni sperimentali, 
#ogni sweep verrà automaticamente salvato all'interno del progetto corrispondente 
#della specifica condizione sperimentale (exp_cond).

#In pratica, se hai più condizioni sperimentali 
#(ad esempio, "Condizione_A", "Condizione_B", ecc.),
#WandB creerà automaticamente sweep separati all'interno dei rispettivi progetti


#Creo la configurazione dello sweep e la eseguo:

#uno per il modello CNN3D_LSTM_FC, uno oer 


# 2.1 – Sweep config per ciascun modello
sweep_config_cnn3d = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN3D_LSTM_FC"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "use_lstm": {"values": [True, False]},
        "lstm_hidden": {"values": [32]},
        "dropout": {"values": [0.5]},
    }
}


sweep_config_cnn_sep = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["SeparableCNN2D_LSTM_FC"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        
        "standardization": {"values": [True]}, #        '''ATTENZIONE QUI IMPOSTIAMO SEMPRE A TRUE'''
        
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "use_lstm": {"values": [True, False]},
        "lstm_hidden": {"values": [32]},
        "dropout": {"values": [0.5]},
    }
}





    
'''SWEEP_IDS_FOR_MODELS

# 2) Popolo sweep_ids_for_models in base a MODEL_LIST (già inizializzato nella prima cella)
'''

#Preparazione del dizionario sweep_ids_for_models (lo aggiorno inserendo il livello delle chiavi dei modelli, per copiare poi la struttura per creare best_models)

#for condition in sweep_ids_for_models:
    #for data_type in sweep_ids_for_models[condition]:
        #for category_subject in sweep_ids_for_models[condition][data_type]:
            #for model_name in sweep_config["parameters"]["model_name"]["values"]:
                
                # Aggiungi il modello al dizionario, se non esiste già
                #if model_name not in sweep_ids_for_models[condition][data_type][category_subject]:
                    #sweep_ids_for_models[condition][data_type][category_subject][model_name] = []

                    
print(f"\nAggiornato \033[1msweep_ids_for_models\033[0m")


'''BEST_MODELS

# 3) Creo best_models da sweep_ids_for_models
'''

#Preparazione del dizionario best_models (facendo una copia della struttura di 'sweep_ids_for_models')

#In questo modo potrò, per ogni condizione sperimentale, tipo di dato EEG e combinazione di ruolo/gruppo,
#accedere facilmente al miglior modello (cioè ai suoi pesi e bias) e gestirlo in maniera separata!

import copy
best_models = copy.deepcopy(sweep_ids_for_models)

# Inizializzo il dizionario che contiene il migliori modello tra quelli degli sweep testati, 
# relativi ad una certa combinazione di fattori,
#per ogni condizione sperimentale
#tipo di dato EEG 
#combinazione di ruolo/gruppo

for condition in best_models:
    for data_type in best_models[condition]:
        for category_subject in best_models[condition][data_type]:
            for model_name in best_models[condition][data_type][category_subject]:
                best_models[condition][data_type][category_subject][model_name] = {
                    "model": None,
                    "max_val_acc": -float('inf'),
                    "best_epoch": None,
                    
                    #ATTENZIONE! CREATA ALTRA CHIAVE PER SALVARE 
                    #LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI OGNI MODELLO!
                    "config": None}
                
print(f"\nCreato \033[1mbest_models\033[0m")


#'''SWEEP_IDS'''

#Preparazione del dizionario sweep_ids (lo aggiorno inserendo solo una lista all'ultimo livello)

# Itera su sweep_ids e crea le chiavi per category_subject con liste vuote
#for condition in sweep_ids:
    #for data_type in sweep_ids[condition]:
        #for category_subject in sweep_ids[condition][data_type]:
            # Inizializza una lista vuota se non esiste già
            #if not isinstance(sweep_ids[condition][data_type][category_subject], list):
                #sweep_ids[condition][data_type][category_subject] = []
                    
#print(f"\nAggiornato \033[1msweep_ids\033[0m")


In [ ]:
#print(best_models)
#print(sweep_ids_for_models)
#print(sweep_ids)
#print(data_dict_preprocessed['th_resp_vs_pt_resp']['1_20']['familiar_th'][0].shape)

In [ ]:
print(best_models)

In [ ]:
print(sweep_ids_for_models)

In [ ]:
print(sweep_ids)

In [ ]:
#data_dict_preprocessed['th_resp_vs_pt_resp']['1_20']['familiar_th'][5].shape

**NOTA BENE**

Come output, io otterrò **quando crei gli sweeps** una cosa come questa, ad esempio:

        Create sweep with ID: y73iajvw
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/y73iajvw
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - CNN1D: n° sweep y73iajvw
        Create sweep with ID: 3b6o28jt
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/3b6o28jt
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - BiLSTM: n° sweep 3b6o28jt
        Create sweep with ID: q6yp4fas

        .....

Vedendole bene, per **ogni condizione sperimentale (3)**, **per ogni dato EEG (3)** e **per ogni provenienza del dato EEG (4)**, 
Io **DOVREI OTTENERE** in totale = **3x3x4 = 36 sweeps** per **OGNI CONDIZIONE SPERIMENTALE**


Per **ognuna di queste sweeps**, io se ho capito bene creerò **15 esperimenti** (le mie runs), che corrispondo alle **diverse configurazioni di iper-parametri testati per lo stesso specifico sweep**!

(ad esempio, solo questo 

<br> 

        Create sweep with ID: y73iajvw
        Sweep URL: https://wandb.ai/stefano-bargione-universit-di-roma-tor-vergata/th_resp_vs_pt_resp/sweeps/y73iajvw
        Sweep ID creato per th_resp_vs_pt_resp - 1_20 - familiar_th - CNN1D: n° sweep y73iajvw)

Dove, le diverse configurazioni, son determinate randomicamente a partire dai valori dentro la variabile "**sweep_config**"  che è questa 


    #Creo la configurazione dello sweep e la eseguo
    sweep_config = {
        "method": "random",
        "metric": {"name": "val_accuracy", "goal": "maximize"},
        "parameters": {
            "lr": {"values": [0.01, 0.001, 0.0005, 0.0001]},
            "weight_decay": {"values": [0, 0.01, 0.001, 0.0001]},
            "n_epochs": {"value": 100},
            "patience": {"value": 10},
            "model_name":{"values": ['CNN1D', 'BiLSTM', 'Transformer']},
            "batch_size": {"values": [32, 48, 64, 96]},
            "standardization":{"values": [True, False]},
        }
    }
    
    



In [ ]:
'''
ATTENZIONE CHE A QUESTO PUNTO


1) sweep_ids[cond][dtype][cat][model_name] contiene le tuple (sweep_id, combo_key) per ciascun modello, che ancora non esistono perché devo esser create durante la creazione degli sweeps, ma ho solo una lista

{'rest_vs_left_fist': {'spectrograms': {'familiar_th': []}}, 
'rest_vs_right_fist': {'spectrograms': {'familiar_th': []}}, 
'left_fist_vs_right_fist': {'spectrograms': {'familiar_th': []}}}


2) sweep_ids_for_models e best_models sono paralleli a sweep_ids con lo stesso livello model_name

ossia 

sweep_ids_for_models come

{'rest_vs_left_fist': {'spectrograms': {'familiar_th': {'CNN3D_LSTM_FC': [], 'SeparableCNN2D_LSTM_FC': []}}},
'rest_vs_right_fist': {'spectrograms': {'familiar_th': {'CNN3D_LSTM_FC': [], 'SeparableCNN2D_LSTM_FC': []}}},
'left_fist_vs_right_fist': {'spectrograms': {'familiar_th': {'CNN3D_LSTM_FC': [], 'SeparableCNN2D_LSTM_FC': []}}}}

best_models come

{'rest_vs_left_fist': {'spectrograms': {'familiar_th': 
{'CNN3D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}, 
'SeparableCNN2D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}}}},

'rest_vs_right_fist': {'spectrograms': {'familiar_th': 
{'CNN3D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}, 
'SeparableCNN2D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}}}}, 

'left_fist_vs_right_fist': {'spectrograms': {'familiar_th': 
{'CNN3D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None},
'SeparableCNN2D_LSTM_FC': {'model': None, 'max_val_acc': -inf, 'best_epoch': None, 'config': None}}}}}


'''


In [ ]:
'''
Popolamento di sweep_ids e lancio degli agenti:

Obiettivo: 

Per ogni combinazione (condition, data_type, category_subject, model_name), 
Se la lista è vuota, crei uno sweep usando wandb.sweep(sweep_config, project=condition) e lo inserisci nella lista. 
In seguito, iteri su quella lista (che ora contiene IL TUO SPECIFICO sweep_id) e lanci wandb.agent() per eseguire il training.



Nota importante:
L'ID restituito da wandb.sweep() è una STRINGA UNIVOCA generata automaticamente da WandB.
Non puoi assegnargli direttamente una stringa personalizzata, ma puoi comunque usarlo per mappare nel tuo dizionario la combinazione di fattori! 

In questo ciclo, il fatto che la lista parta vuota è normale: il codice la popola se necessario e poi lancia l'agente per ogni sweep_id presente.


****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******
INOLTRE, BISOGNA CONTROLLARE CHE SI STIA ITERANDO CORRETTAMENTE SOLO SULLA COMBINAZIONE CORRENTE DI 

                CONDITION, DATA_TYPE, CATEGORY_SUBJECT E MODEL_NAME
                
QUESTO PERCHÉ SE UN CICLO SI RIPETE PER UNA CONDIZIONE IN PIÙ UNA COMBINAZIONE, POTREBBE GENERARE PIÙ  SWEEP IDS DI QUELLI CHE TI ASPETTI!
****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******  ****** ****** ******



SOLUZIONE:

Un buon approccio per evitare la creazione ripetuta di Sweep ID 
per la stessa combinazione di condition, data_type, category_subject e model_name 
è quello di utilizzare un SET per tenere traccia delle combinazioni già processate.
Se una combinazione è già presente nel set, non dovresti creare un nuovo Sweep ID, ma semplicemente saltare quella parte del codice


Inoltre, ho avuto una idea ad un certo punto! 


****************************** ******************************
ILLUMINAZIONE DEL POMERIGGIO DEL 04/03/2025: 
****************************** ******************************


Quando creo ogni sweep singolarmente, si genera una stringa univoca di quello sweep, che si riferisce ad un dataset che è il prodotto di diversi fattori:

- una certa condizione sperimentale,  
- una certo preprocessing sui dati EEG (1_20, 1_45, wavelet)
- una certa provenienza del dato proprio (in termini di ruolo e gruppo --> th o pt, familiar o unfamiliar)


Di conseguenza, iterando su ogni sweep_ids (che ho fatto in modo avesse la STESSA struttura dei miei dati già splittati i.e, data_dict_preprocessed
io posso, 

1) da un lato eseguire la creazione della stringa univoca associata a quello sweep,
2) crearmi una 'combination_key', che sarebbe l'insieme delle stringhe che descrivono quel dataset specifico di data_dict_preprocessed

che sarà costituito da

- una certa condizione sperimentale,  
- una certo preprocessing sui dati EEG (1_20, 1_45, wavelet)
- una certa provenienza del dato proprio (in termini di ruolo e gruppo --> th o pt, familiar o unfamiliar)


Poiché quindi so già la corrispondenza tra ogni Sweep ID e la sua combinazione di fattori (condition, data_type, category_subject), 
posso creare un MAPPING, che associ, ad certo Sweep ID e la stringa che descrive i suoi fattori associati!


In questo modo, forse, si riesce a risolvere il PROBLEMA 2 NELLA CELLA DI CREAZIONE DELLA FUNZIONE DI TRAINING (VEDI SOTTO!)



                                                        ******IMPORTANTE MODIFICA*****
                                                        
Ora lo sweep_ids non si deve sdoppiare ora, perché sostanzialmente, 
per ogni modello si creano gli sweeps ids corrispondenti e salvati come valore
dentro la chiave del modello corrispondente, sotto forma di tupla...

cioè non più così

"sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))"

ma una cosa del genere

"sweep_ids[condition][data_type][category_subject][model_name].append((new_sweep_id, combination_key))




COME ERA PRIMA

#Inizializza un set per tenere traccia delle combinazioni già elaborate

created_combinations = set()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"

            # Controlla se la combinazione è già stata elaborata
            if combination_key not in created_combinations:

                if not sweep_ids[condition][data_type][category_subject]:
                    #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms")
                    new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband")

                    #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                    #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA 
                
                    sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))
                    
                    print(f"Sweep ID creato per \033[1m{combination_key}\033[0m: n° sweep \033[1m{new_sweep_id}\033[0m")

                # Aggiungi la combinazione al set per evitare duplicazioni
                created_combinations.add(combination_key)
            else:
                # Se la combinazione è già stata creata, salta
                print(f"Sweep ID per {combination_key} già esistente.")
                
'''


'''
ADESSO


Cosa fa questo snippet

Cicla su ogni (condition, data_type, category_subject) una volta sola grazie a created_combinations.

All’interno, fa un sottoloop su MODEL_LIST (i tuoi due modelli).

In base a model_name, sceglie sweep_config_cnn3d o sweep_config_cnn_sep.

Chiama wandb.sweep(...) con il config giusto e salva il risultato in


sweep_ids[condition][data_type][category_subject][model_name]
anziché nella lista “piatta” che avevi prima.


In questo modo:

sweep_ids[cond][dtype][cat] resta un dict con due chiavi ("CNN3D_LSTM_FC" e "SeparableCNN2D_LSTM_FC")

Ognuna di quelle chiavi punta a una propria lista di tuple (sweep_id, combo_key)

Non serve sdoppiare l’intero sweep_ids, perché tiene già separati gli sweep di ciascun modello

Più tardi, quando lancerai gli agent, ti basterà:


for model_name, sweeps in sweep_ids[cond][dtype][cat].items():
    for sweep_id, combo_key in sweeps:
        # qui scegli il train_fn in base a model_name
        wandb.agent(sweep_id, function=train_fn_map[model_name], count=200)
e ogni modello girerà solo i suoi sweep.



Alla fine, sweep_ids avrà la forma:

{
  'rest_vs_left_fist': {
    'spectrograms': {
      'familiar_th': {
         'CNN3D_LSTM_FC':       [(sweep_id_1, 'rest_vs_left_fist_spectrograms_familiar_th')],
         'SeparableCNN2D_LSTM_FC': [(sweep_id_2, 'rest_vs_left_fist_spectrograms_familiar_th')]
      }
    }
  },
  …
}
'''


#Ecco come puoi riscrivere solo la TERZA CELLA (quella in cui crei effettivamente gli sweep) 
#mantenendo la tua struttura “a celle” e usando per ognuno il sweep_config giusto in base al model_name.

#Creazione degli sweep (Terza cella)
#Ecco il solo snippet che devi usare per creare gli sweep ripartiti per modello, usando i due sweep_config_*:


'''
Per mantenere la stessa logica di prima ma tenendo conto che ora stai lavorando con modelli separati, 
dovresti modificare il controllo in modo che verifichi se una combinazione di condition, data_type, category_subject
è già stata processata per ciascun modello.

Quindi, il controllo dovrebbe essere fatto separatamente per ogni modello dentro il loop che itera sui modelli (MODEL_LIST).
Di seguito ti mostro la versione modificata che tiene conto di questo:



#Inizializza un set per tenere traccia delle combinazioni già elaborate

created_combinations = set()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"

            # Controlla se la combinazione è già stata elaborata
            if combination_key not in created_combinations:

                if not sweep_ids[condition][data_type][category_subject]:
                    #new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms")
                    new_sweep_id = wandb.sweep(sweep_config, project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband")

                    #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                    #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA 
                
                    sweep_ids[condition][data_type][category_subject].append((new_sweep_id, combination_key))
                    
                    print(f"Sweep ID creato per \033[1m{combination_key}\033[0m: n° sweep \033[1m{new_sweep_id}\033[0m")

                # Aggiungi la combinazione al set per evitare duplicazioni
                created_combinations.add(combination_key)
            else:
                # Se la combinazione è già stata creata, salta
                print(f"Sweep ID per {combination_key} già esistente.")
                
                
                
'''

                    
'''

Cosa è stato cambiato rispetto alla versione precedente?
Controllo della combinazione di modello:
La logica del controllo della combinazione (combination_key, model_name) nel set created_combinations è corretta, 
perché vogliamo evitare di creare più volte lo stesso sweep per una combinazione di condition, data_type, category_subject, e model_name.

Controllo e creazione dello sweep:
Il codice controlla prima se la combinazione con il modello non è stata già processata 
con il controllo if (combination_key, model_name) not in created_combinations. 

Se non è stata processata, procede a creare lo sweep corrispondente. 
Se la combinazione esiste già, salta la creazione dello sweep per quel modello.

Aggiunta del nuovo sweep ID:
Una volta creato il nuovo sweep per il modello, viene aggiunto correttamente 
alla lista del modello specifico sotto sweep_ids[condition][data_type][category_subject][model_name].

Aggiunta al set delle combinazioni:
Dopo aver creato lo sweep, aggiungiamo (combination_key, model_name) al set created_combinations
per tenere traccia delle combinazioni già elaborate.

Verifica della logica:
La combinazione (combination_key, model_name) deve essere unica per ciascun modello, 
e quindi il controllo che evita duplicazioni nel set è corretto.

La creazione dello sweep per ciascun modello separato è mantenuta, 
e viene applicata solo quando la combinazione specifica non è già stata elaborata per quel modello.

In questo modo, la logica funziona come nel codice precedente, ma ora si tiene conto anche dei modelli separati, 
creando un sweep per ciascuno di essi e mantenendo la traccia delle combinazioni in modo appropriato.

'''
created_combinations = set()

# Per semplicità, tieni MODEL_LIST a portata di mano
MODEL_LIST = ["CNN3D_LSTM_FC", "SeparableCNN2D_LSTM_FC"]

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            combination_key = f"{condition}_{data_type}_{category_subject}"
            
            # per ciascun modello, creo uno sweep separato
            for model_name in MODEL_LIST:

                # Controlla se la combinazione di condition, data_type, category_subject + modello è già stata elaborata
                if (combination_key, model_name) not in created_combinations:

                    # Scegli il config in base al model_name
                    if model_name == "CNN3D_LSTM_FC":
                        sweep_conf = sweep_config_cnn3d
                        
                    else:  # SeparableCNN2D_LSTM_FC
                        sweep_conf = sweep_config_cnn_sep
                    
                    # Controllo se la lista per il modello specifico è vuota
                    if not sweep_ids[condition][data_type][category_subject][model_name]:

                        # Crea lo sweep e lo appendo nella lista dedicata a quel modello
                        new_sweep_id = wandb.sweep(sweep_conf, project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband")
                        
                        #QUI, viene creata la mappatura tra Sweep ID e la descrizione della combinazione (in formato di stringhe)
                        #CON LA CREAZIONE DI UNA TUPLA, DENTRO LA LISTA 
                        
                        sweep_ids[condition][data_type][category_subject][model_name].append((new_sweep_id, combination_key))

                    print(f"▶ Sweep \033[1m{new_sweep_id}\033[0m creato per \033[1m{combination_key}\033[0m, modello \033[1m{model_name}\033[0m")
                    
                    # Aggiungi la combinazione al set per evitare duplicazioni
                    created_combinations.add((combination_key, model_name))  # Aggiungi la combinazione con il modello
                else:
                    # Se la combinazione è già stata creata, salta
                    print(f"⚠️ {combination_key} già processato per il modello {model_name}, skip.")
                    continue


In [ ]:
# Calcola e stampa il numero totale di combinazioni uniche (e quindi di sweep creati)

total_sweeps = len(created_combinations)
total_runs = total_sweeps * 200

print(f"Numero totale di sweep creati: {total_sweeps}")
print(f"Numero totale di runs da eseguire: {total_runs}")

In [ ]:
'''ESEGUI QUI QUESTA CELLA PER VEDERE COME SI STRUTTURA SWEEP_IDS'''

#sweep_ids

In [ ]:
#sweep_ids.keys()
#sweep_ids['th_resp_vs_pt_resp'].keys()
#sweep_ids['th_resp_vs_pt_resp'].keys()

**NOTA BENE**


I **numeri degli sweeps** tornano e son corretti! 
Tuttavia, avendo solo preparato l'inizializzazione degli sweeps dentro 'sweep_ids', 
Sul sito di weight and biases, io vedo le tre condizioni sperimentali, create ciascuna come un progetto separato, che è corretto, ma ancora le runs di ciascuna le vedo a 0

Deduco che questo comportamento, dovrebbe esser normale, dato che ancora non ho avviato l'agente appunto wandb.agent(), con cui gli fornisco lo sweep_id generato adesso in questo loop precedente.

In [ ]:
print(data_dict_preprocessed.keys())
print(sweep_ids.keys())

In [ ]:
data_dict_preprocessed.keys()

In [ ]:
data_dict_preprocessed['rest_vs_left_fist'].keys()

**IO**:

però non capisco perché aggiornamento del modello e scheduler & early stopping monitorano cose diverse...

tu mi hai suggerito di fare così...

    "
    # -- Aggiorna miglior modello --
            if val_acc > max_val_acc:
                max_val_acc = val_acc
                best_epoch  = epoch
                best_model  = copy.deepcopy(model)

            # -- Scheduler & Early Stopping --
            scheduler.step(val_loss)
            early_stopping(val_loss)
            if early_stopping.early_stop:
                print(f"🛑 Early stopping (val_loss) dopo {epoch} epoche.")
                break
    "

non dovrebbero vedere o la val loss o val accuracy entrambe? spiegami sta cosa

<br>

**CHATGPT**:


È perfettamente normale — anzi, consigliabile — usare metriche diverse per:

Selezionare il “best model” in base all’obiettivo finale (accuracy, F1, AUC…)

Gestire l’ottimizzazione del training (scheduler ed early stopping) in base a una funzione di ottimizzazione continua (tipicamente la loss)

1. Perché salvo il modello sul miglior val_accuracy
Obiettivo finale del tuo task di classificazione è massimizzare l’accuratezza (o qualche altra metrica di performance).

Quindi, quando decidi “questo è il modello migliore finora”, lo scegli in base alla metrica che ti interessa davvero in produzione: val_acc.

if val_acc > max_val_acc:
    max_val_acc = val_acc
    best_model  = copy.deepcopy(model)
    
2. Perché scheduler e early-stopping monitorano la val_loss
Scheduler (ReduceLROnPlateau): lavora sulla loss, perché è una curva continua e più liscia.

La loss tende a fluttuare meno drasticamente di accuracy (che salta tra 0 e 1), e quindi il plateau su loss è un indicatore più affidabile di “sto smettendo di imparare”.

EarlyStopping: idem, se monitori la loss puoi fermarti non appena il modello smette di migliorare nella sua funzione di ottimizzazione primaria, 
evitando di aspettare 12 epoche solo perché l’accuracy potrebbe oscillare o rimanere piatta un po’ più a lungo.


scheduler.step(val_loss)      # dimezza lr se val_loss non cala
early_stopping(val_loss)     # ferma se val_loss non cala per 12 epoche


3. Riassunto

val_loss → serve per ottimizzare il training (quando aggiornare lr, quando fermarsi).
val_acc → serve per selezionare quale modello salvare come “migliore”.

Questo approccio misto ti dà sia:

Un training più stabile e sensibile (via loss),

Una scelta di modello finale più allineata all’obiettivo (via accuracy).


<br>


**IO**:

ok quindi qui nello sweep config

sweep_config = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [0.01, 0.001, 0.0005, 0.0001]},
        #"weight_decay": {"values": [0, 0.01, 0.001, 0.0001]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        #"model_name":{"values": ['CNN2D', 'BiLSTM', 'Transformer']},
        "model_name":{"values": ['CNN2D_LSTM_TF', 'BiLSTM', 'Transformer']},
        #"batch_size": {"values": [32, 48, 64, 96]},
        "batch_size": {"values": [32, 48, 64, 72, 84, 96]},
        "standardization":{"values": [True, False]},
        
    }
}

commentando le righe con # ho fatto in modo che il training abbia sia l'obiettivo di massimizzare l'accuracy e di usare anche lo scheduler sotto e early stopping su val loss?


<br>


**CHATGPT**:

Sì, esattamente: con quella configurazione dello sweep


"metric": {"name": "val_accuracy", "goal": "maximize"}
stai dicendo a W&B di cercare di massimizzare l’accuracy di validazione.
E togliendo weight_decay dallo sweep (o commentandolo) e forzandolo a zero nell’Adam, aderisci al setup “puro” del paper.

Lo scheduler (ReduceLROnPlateau) e l’early stopping su val_loss non sono parametri di sweep, ma comportamenti che hai inserito dentro training_sweep. In pratica:

Sweep esplora solo

lr
n_epochs (fissato a 100)
patience (ora 12, anche se è usato solo per W&B, l’early-stopping vero lo hai forzato a 12 su loss)
model_name
batch_size
standardization

Dentro training_sweep

Optimizer Adam è costruito con weight_decay=0, betas=(0.9, 0.999), eps=1e-8, e lr=config.lr.
Hai creato un ReduceLROnPlateau su val_loss (mode='min', factor=0.5, patience=4).
Hai un EarlyStopping su val_loss con patience=12.

Metriche loggate a W&B includono sia val_loss sia val_accuracy, quindi il tuo sweep valuta e sceglie la config migliore in base a val_accuracy ma gli aggiustamenti di lr e l’interruzione anticipata sono decisi internamente dal comportamento di scheduler+early stopping sulla curva di loss.

Questa separazione ti dà:

Una ricerca di iper-parametri incentrata sull’accuracy (quello che desideri massimizzare).
Un allenamento stabile e fedele al paper grazie a scheduler e early stopping su loss.

Puoi andare avanti così!


#### **VERSIONE DEL 6 MARZO (RISOLUZIONE DEFINITIVA)**

##### **Training Function Edits - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS**

#### **Sweep separati per ciascuno dei modelli CNN3D e CNN Sep**

In [ ]:
'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

import re

def parse_combination_key(combination_key):
    """
    Estrae condition_experiment e subject_key da combination_key
    dove il data_type è fisso a "spectrograms".
    
    Esempio di chiave: 
    "pt_resp_vs_shared_resp_spectrograms_familiar_th"
    """
    match = re.match(
        r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        condition_experiment = match.group(1)
        subject_key = match.group(2)
        return condition_experiment, subject_key
    else:
        raise ValueError(f"Formato non valido: {combination_key}")

# Test
combination_key = "rest_vs_left_fist_spectrograms_familiar_th"
condition_experiment, subject_key = parse_combination_key(combination_key)

print("Condizione:", condition_experiment)
print("Soggetto:", subject_key)


In [ ]:
'''
                                                                ***** FUNZIONE DI TRAINING *****
                                                                ***** VERSIONE DEL 5 MARZO *****
                                                                
                                                                    **** SALVATAGGIO DI **** 
                                                        
                                                        1) PESI E BIAS DI UN CERTO MODELLO 
                                                        2) CONFIGURAZIONE IPER-PARAMETRI DI UN CERTO MODELLO
                                                                
Il punto critico è garantire che ogni configurazione di iperparametri estratta randomicamente da W&B per OGNI SWEEP sia coerente con:

Il dataset giusto (ossia la coppia di condizioni sperimentali corrispondente).
Il tipo di dato EEG usato (1_20, 1_45, wavelet ecc.).
L'origine dei dati tra le quattro tipologie di soggetti.


che io andrei a prelevare ogni volta da 'data_dict_preprocessed'!

Quindi, ad ogni iterazione del loop sui dati (i.e., data_dict_preprocessed?)
il codice dovrebbe assicurarsi/verificare che, 


1) la configurazione selezionata da W&B presa da uno SPECIFICO SWEEP,  
sia quella che effettivamente corrisponde ad un certo dataset in termini di combinazione di fattori 

- una specifica condizione sperimentale
- una specifico tipo di dato EEG 
- una specifica combinazione di ruolo/gruppo


2) che le run di quella sweep siano inserita nel progetto del dataset di quella specifica condizione sperimentale,


(3 PLUS OPZIONALE

e che il "name" e i "tag" (eventualmente, delle runs associate a quello sweep)
siano costruiti in maniera coerente con la combinazione di fattori associata allo sweep (e quindi alla condizione sperimentale corrente)



****************************** ******************************
CONCLUSIONE A CUI SON ARRIVATO LA MATTINA DEL 04/03/2025: 
****************************** ******************************

Dato che ogni sweep si applica per verificare, tra le 15 diversi set di iper-parametri diversi, 
quale sia la configurazione migliore, per uno specifico set di dati in termini di combinazione di fattori, che sono

- relativi ad una certa condizione sperimentale,  
- con un certo preprocessing
- con un certa provenienza del dato


Son arrivato ad un punto in cui credo che sia davvero molto complesso controllare la corrispondenza esatta tra 

1) di chi esegue lo sweep
2) la definizione del nome della sue 15 runs (cioè di quale dato si riferisca etc. in termini di combinazione di fattori) ...

Quindi l'unica cosa che ha senso è forse solo creare le runs in modo da inserirle tutte assieme in base al solo nome del progetto,
che però è prelevabile dalla prima chiave di 'data_dict_preprocessed'.. 

in questo modo, pur non avendo il controllo sul nome della run e del suo tag,
almeno dovrei esser sicuro che comunque le runs associate all'uso dei dati di ALMENO 
una certa condizione sperimentale vengano inserite nel relativo progetto su weight and biases...



TUTTAVIA, 

****************************** ******************************
ILLUMINAZIONE DEL POMERIGGIO DEL 04/03/2025: 
****************************** ******************************

MI HA PORTATO A PENSARE A PROVARE A CAPIRE ANCORA SE RIESCO A RISOLVERE IL PROBLEMA ...
'''


#VERSIONE NUOVA!

#Fase 2: Creazione della funzione di 'training_sweep' 
    
'''Questa funzione parse_combination_key serve per estrarre 
le varie stringhe che compongono la combinazioni di fattori (condizione sperimentale, tipo di dato EEG e provenienza del dato EEG) 
che si riferiscono allo sweep ID corrente.

Esempio:

Lo tupla sweep (sweep ID, combinazioni di fattori in stringa) è la seguente:

Inizio l'agent per sweep_id: ('4u94ovth', 'pt_resp_vs_shared_resp_wavelet_unfamiliar_pt') dove
- sweep ID: 4u94ovth
- combinazioni di fattori in stringa: pt_resp_vs_shared_resp_wavelet_unfamiliar_pt

Di conseguenza, quando avvio l'agent per quella condizione sperimentale nel loop, 
dentro la funzione di 'training_sweep' io prenderò in input la tupla


""" Esegue il training per uno specifico sweep """

def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 

sweep_id, combination_key = sweep_tuple
exp_cond, data_type, category_subject = parse_combination_key(combination_key)


E lui estrarrà la combinazione di fattori che la compongono, in questo caso è 

1) Condizione Sperimentale = pt_resp_vs_shared_resp
2) Tipo di Dato EEG = wavelet
3) Provenienza del Tipo di Dato EEG unfamiliar_pt

Successivamente, confronta se questa combinazione di stringhe si trova dentro la mia struttura dati e, se la trova

1) creerà il progetto con il nome della condizione sperimentale combaciante tra 
 
 - la combination_key associata allo Sweep ID corrente e
 - il sottodizionario di data_dict_preprocessed 
 
2) le relative run di quello specifico Sweep, verranno nominate con la combinazioni di fattori combaciante su W&B

3) Esegue e gestisce il salvataggio della migliore configurazione di iper-parametri del relativo modello preso in esame (CNN1D, BiLSTM e Transformer)
   tra le 15 runs di OGNI SWEEP
   

'''

import re

def parse_combination_key(combination_key):
    """
    Estrae condition_experiment e subject_key da combination_key
    dove il data_type è fisso a "spectrograms".
    
    Esempio di chiave: 
    "pt_resp_vs_shared_resp_spectrograms_familiar_th"
    """
    match = re.match(
        r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        condition_experiment = match.group(1)
        subject_key = match.group(2)
        return condition_experiment, subject_key
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        
        
def training_sweep(data_dict_preprocessed, sweep_config, sweep_ids, sweep_id, sweep_tuple, best_models): 
    
    # Per ogni sweep, che viene iterato nel loop, io prendo 
    #1) la stringa univoca dello Sweep ID
    #2) la sua combinazione di fattori stringa (che mi serviranno per prelevare il dato corrispondente da 'data_dict_preprocessed'
    
    sweep_id, combination_key = sweep_tuple
    
    # Ora la funzione restituisce solo (exp_condition, subject_key)
    exp_cond, category_subject = parse_combination_key(combination_key)
    
    # Poiché ora i dati sono solo di tipo "spectrograms", li impostiamo in modo fisso:
    data_type = "spectrograms"

    if not (exp_cond in data_dict_preprocessed and category_subject in data_dict_preprocessed[exp_cond][data_type]):
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{category_subject}\033[0m")
    

    run_name = f"{exp_cond}_{data_type}_{category_subject}"
    
    tags = [exp_cond, data_type, category_subject]

    #Inizializza la run dello specifico Sweep dentro Weights & Biases (W&B) con

    #1) un nome del progetto pari alla condizione sperimentale corrente
    #2) il nome e tag della run in base alla combinazione di fattori corrispondente
    #3) la congiurazione di iper-parametri è pari a quella passata in input a 'training_sweep'

    #Vedi questo link su wandb.init() per vedere i suoi parametri --> #https://docs.wandb.ai/ref/python/init/
    
    # Inizializza la run in W&B nel progetto che termina con "_spectrograms"
    
    '''OCCHIO QUI CAMBIATO'''
        
    wandb.init(project=f"{exp_cond}_spectrograms_channels_freqs_new_3d_grid_multiband", name=run_name, tags=tags)

    print(f"\nCreo wandb project per: \033[1m{exp_cond}_spectrograms\033[0m")
    print(f"Lo sweep corrente è \033[1m{sweep_tuple}\033[0m")
    print(f"\nInizio addestramento sul dataset \033[1m{exp_cond}\033[0m con dati EEG \033[1m{data_type}\033[0m di \033[1m{category_subject}\033[0m")

    # Parametri dell'esperimento presi da wandb
    config = wandb.config

    # Recupera i dati pre-processati per la combinazione corrente una volta verificata l'esatta corrispondenza tra:
    #1)il combination_key dello sweep
    #2)l'esistenza di specifico dataset con le stesse 'combination_key' dentro data_dict_preprocessed

    try:
        X_train, X_val, X_test, y_train, y_val, y_test = data_dict_preprocessed[exp_cond][data_type][category_subject]
        print(f"\nCarico i dati di \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")
        print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
        print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
        print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")
    except KeyError:
        raise ValueError(f"❌ ERRORE - Combinazione \033[1mNON TROVATA\033[0m in data_dict_preprocessed: \033[1m{exp_cond}\033[0m, \033[1m{data_type}\033[0m, \033[1m{category_subject}\033[0m")


    if config.standardization:
        # Standardizzazione
        X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
        print(f"\nUso DATI \033[1mSTANDARDIZZATI\033[0m!")
    else:
        print(f"\nUso DATI \033[1mNON STANDARDIZZATI\033[0m!")

    # Preparazione dei dataloaders (N.B. prendo uno dei modelli considerati dentro config.model_name)
    train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
        X_train, X_val, X_test, y_train, y_val, y_test, model_type=config.model_name, batch_size = config.batch_size
    )

    #Qui estraggo il relativo modello su cui sto iterando al momento corrente e lo inizializzo

        
    # Inizializza il modello in base al valore scelto in config.model_name
    #if config.model_name == "CNN2D":
    #    model = CNN2D(input_channels=64, num_classes=2)
    #    print(f"\nInizializzazione Modello \033[1mCNN2D\033[0m")
    
    '''OCCHIO QUI CAMBIATO'''
    #class CNN2D_LSTM_FC(nn.Module):
    #def __init__(self, n_freq=45, input_channels=64, num_classes = 2, dropout=0.2):
    
    if config.model_name == "CNN3D_LSTM_FC":
        #model = CNN2D_LSTM_FC(n_freq =45, input_channels=64, num_classes=2, dropout=0.2)
        
        '''OCCHIO QUI CAMBIATO PER GRIGLIA 3D'''
    
        model = CNN3D_LSTM_FC(
            num_classes=2,
            dropout=config.dropout,
            hidden_size=config.lstm_hidden,
            use_lstm=config.use_lstm
        )

        print(f"\nInizializzazione Modello \033[1mCNN3D_LSTM_FC\033[0m")
    
    
    elif config.model_name == "SeparableCNN2D_LSTM_FC":
        model = SeparableCNN2D_LSTM_FC(
            num_classes=2,
            dropout=config.dropout,
            hidden_size=config.lstm_hidden,
            use_lstm=config.use_lstm
        )
        print(f"\nInizializzazione Modello \033[1mSeparableCNN2D_LSTM_FC\033[0m")
    
    #elif config.model_name == "TopomapNet":
        #model = TopomapNet(
            #input_channels=5,
            #num_classes=2,
            #base_channels=config.base_channels,
            #use_lstm=config.use_lstm,
            #lstm_hidden=config.lstm_hidden,
            #dropout=config.dropout
        #)
        #print(f"\nInizializzazione Modello \033[1mTopomapNet\033[0m")
    else:
        raise ValueError(f"Modello sconosciuto: {config.model_name}")
        
    #elif config.model_name == "BiLSTM":
        # Qui, input_size = canali * frequenze = 3 * 38 = 78
        #model = ReadMEndYou(input_size= 64 * 45, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
        #print(f"\nInizializzazione Modello \033[1mBiLSTM\033[0m")
        
    #elif config.model_name == "Transformer":
        # Per il Transformer, passiamo anche i parametri channels e freqs per adattare l'embedding
        #model = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=64, freqs=45)
        #print(f"\nInizializzazione Modello \033[1mTransformer\033[0m")

        
    #ORIGINAL VERSION OF TIME SERIES EEG DATA REPRESENTATION  
    #def initialize_models():
        #model_CNN = CNN1D(input_channels=3, num_classes=2)
        #model_LSTM = ReadMEndYou(input_size=3, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
        #model_Transformer = ReadMYMind(num_channels=3, seq_length=250, d_model=16, num_heads=4, num_layers=2, num_classes=2)
        
        #return model_CNN, model_LSTM, model_Transformer

    '''
    Cosa è cambiato rispetto alla tua versione
    Optimizer Adam ora prende betas=(0.9,0.999) e eps=1e-8.

    ReduceLROnPlateau posizionato subito dopo l’optimizer, chiamato su val_loss ogni epoca.

    EarlyStopping con patience=12, mode='min' su val_loss.

    Loop sulle epoche fino a config.n_epochs (100), senza limitare a 60.

    Tutti i parametri di sweep_config (lr, weight_decay, n_epochs, patience, batch_size, standardization…) rimangono esposti e loggati.

    In questo modo riproduci fedelmente il training descritto nel paper, senza stravolgere la tua pipeline di sweep.
    '''
    
    '''OLD VERSION'''
    #optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    
    '''NEW VERSION'''
    optimizer = optim.Adam(
        model.parameters(),
        lr = config.lr,               # da sweep: es. [0.01,0.001,...]
        betas = (config.beta1, config.beta2),          # paper
        eps = config.eps,                    # paper
        weight_decay=config.weight_decay
    )
    
    criterion = nn.CrossEntropyLoss(weight = class_weights_tensor)
    
    '''NEW VERSION'''
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode ='min',      # monitoriamo val_loss
        factor = 0.1,      # dimezza lr
        patience = 8,      # 4 epoche di plateau
        verbose = True
    )
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Parametri di training
    n_epochs = config.n_epochs
    patience = config.patience
    
    #early_stopping = EarlyStopping(patience=patience, mode='max')
    
    '''NEW VERSION'''
    early_stopping = EarlyStopping(patience=patience, mode='min')
    
    best_model = None
    max_val_acc = 0
    best_epoch = 0

    pbar = tqdm(range(n_epochs))

    for epoch in pbar:
        train_loss_tmp = []
        correct_train = 0
        y_true_train_list, y_pred_train_list = [], []

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            y_pred = model(x)
            loss = criterion(y_pred, y.view(-1))
            loss.backward()
            optimizer.step()

            train_loss_tmp.append(loss.item())
            _, predicted_train = torch.max(y_pred, 1)
            correct_train += (predicted_train == y).sum().item()
            y_true_train_list.extend(y.cpu().numpy())
            y_pred_train_list.extend(predicted_train.cpu().numpy())

        accuracy_train = correct_train / len(train_loader.dataset)
        loss_train = np.mean(train_loss_tmp)

        precision_train = precision_score(y_true_train_list, y_pred_train_list, average='weighted')
        recall_train = recall_score(y_true_train_list, y_pred_train_list, average='weighted')
        f1_train = f1_score(y_true_train_list, y_pred_train_list, average='weighted')
        auc_train = roc_auc_score(y_true_train_list, y_pred_train_list, average='weighted')

        loss_val_tmp = []
        correct_val = 0
        y_true_val_list, y_pred_val_list = [], []

        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                y_pred = model(x)

                loss = criterion(y_pred, y.view(-1))
                loss_val_tmp.append(loss.item())
                _, predicted_val = torch.max(y_pred, 1)

                correct_val += (predicted_val == y).sum().item()
                y_true_val_list.extend(y.cpu().numpy())
                y_pred_val_list.extend(predicted_val.cpu().numpy())

        accuracy_val = correct_val / len(val_loader.dataset)
        loss_val = np.mean(loss_val_tmp)

        wandb.log({
            "epoch": epoch,
            "train_loss": loss_train,
            "train_accuracy": accuracy_train,
            "train_precision": precision_train,
            "train_recall": recall_train,
            "train_f1": f1_train,
            "train_auc": auc_train,
            "val_loss": loss_val,
            "val_accuracy": accuracy_val
        })

        if accuracy_val > max_val_acc:
            max_val_acc = accuracy_val
            best_epoch = epoch
            best_model = cp.deepcopy(model)
            
        '''OLD VERSION'''
        #early_stopping(accuracy_val)
        #if early_stopping.early_stop:
            #print("🛑 Early stopping attivato!")
            #break

        '''NEW VERSION'''
        scheduler.step(loss_val)
        early_stopping(loss_val)
        if early_stopping.early_stop:
            print(f"🛑 Early stopping attivato!")
            break

        '''
        Qui, si usa config.model_name tra le chiavi di best_models, 
        così che gestisca automaticamente il salvataggio del best model estratto dalla configurazione randomica di iper-parametri
        della specifica run di un determinato sweep, che è relativa allo specifico modello correntemente estratto randomicamente dalla sweep_config!
        
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        IMPORTANTISSIMO: COME SALVARSI LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DI UN CERTO MODELLO, DI UN DATO DI UNA CERTA COMBINAZIONE DI FATTORI
        (CONDIZIONE SPERIMENTALE, TIPO DI DATO, PROVENIENZA DEL DATO!)
        ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
        
        CHATGPT:
        
        Nei run eseguiti con W&B ogni esecuzione registra automaticamente la configurazione degli iper-parametri (tramite wandb.config) 
        insieme alle metriche e ai log. 
        Quindi, a meno che tu non abbia modificato il comportamento predefinito, 
        ogni run con il tuo sweep ha già la configurazione associata registrata nei run logs di W&B.

        Tuttavia, per associare in modo “automatico” e diretto la migliore configurazione agli specifici modelli salvati in .pth, 
        potresti considerare di fare uno o più di questi aggiustamenti:

        Salvare la configurazione nel dizionario dei best_models:
        Quando aggiorni il dizionario best_models (cioè quando salvi il miglior modello per una determinata combinazione), 
        puoi salvare anche una copia della configurazione corrente. 
        
        Ad esempio, potresti modificare il blocco in cui aggiorni best_models in questo modo:
        
        best_models[exp_cond][data_type][category_subject][config.model_name] = {
            "model": cp.deepcopy(model),
            "max_val_acc": accuracy_val,
            "best_epoch": best_epoch,
            "config": dict(config)  # Salva la configurazione degli iper-parametri
        }
        
        In questo modo, ogni volta che un modello viene considerato il migliore per quella combinazione,
        la sua configurazione sarà salvata insieme ai pesi.
        Questo ti permetterà, in seguito, di sapere esattamente quali iper-parametri sono stati usati per ottenere quel modello.
        
        
        In sintesi, se hai già usato wandb.config e hai loggato le configurazioni durante le run,
        W&B le ha automaticamente salvate nei run logs. 
        
        Se vuoi rendere più esplicita l'associazione tra il modello salvato (.pth) e la sua configurazione, 
        è utile modificare il tuo codice di TRAINING per salvare ANCHE 
        
        1) il dizionario di configurazione insieme a 
        2) i pesi nel dizionario best_models oppure nei metadati del file salvato.
        
        Questo piccolo accorgimento ti consentirà di recuperare facilmente la configurazione ottimale per ogni modello salvato.
        
        OSSIA
        Aggiungendo la chiave "config": dict(config) nel dizionario che memorizza il best model,
        salvi anche la configurazione degli iper-parametri utilizzata in quella run.
        
        In questo modo, per ogni modello salvato (.pth) potrai recuperare facilmente sia i pesi che la configurazione ottimale che li ha generati.
        
        Questo approccio garantisce che ogni modello sia associato in modo esplicito al set di iper-parametri che ha prodotto le migliori performance, 
        rendendo più semplice il successivo confronto o la replica degli esperimenti.
        
        '''
        
        
        # ***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
        #1)Al posto di salvarmi solo i migliori pesi (i.e.,  model_file = f"{model_path}/{best_model_name}.pth")
        #  ora mi salvo anche la MIGLIORE configurazione di iper-parametri trovata rispetto alle 15 RUNS di un certo SWEEP
        #  di un certo MODELLO, applicato su un DATASET con una SPECIFICA COMBINAZIONE DI FATTORI
        #  condizione sperimentale, tipo di dato e provenienza del dato!
        
    

        if best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"] == -float('inf'):

            # Salvo il primo best_model per quella combinazione
            best_models[exp_cond][data_type][category_subject][config.model_name] = {
                "model": cp.deepcopy(model),
                "max_val_acc": accuracy_val,
                "best_epoch": epoch,
                
                #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
                #***** AGGIUNTA DELLA CHIAVE CONFIG CHE PRELEVA AUTOMATICAMENTE LA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI DENTRO 'BEST_MODELS'
                
                # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                # in relazione ad un certo modello applicato su un dataset costituito da 
                # una certa combinazione di fattori: 
                # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                "config": dict(config)  
            }

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"

            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)

            os.makedirs(model_path, exist_ok=True)
            
            #***** ATTENZIONE: CAMBIAMENTI ESEGUITI RISPETTO A PRIMA *****
            #***** SALVATAGGIO DI UN FILE .PKL, CHE CONTIENE 
            
            # I PESI E BIAS DEL MODELLO DERIVATO DALLA MIGLIORE CONFIGURAZIONE DI IPER-PARAMETRI OTTENUTA DALLA MIGLIORE RUN DI UN CERTO SWEEP
            # IN RELAZIONE AD UN CERTO DATASET COSTITUITO DA UNA CERTA COMBINAZIONE DI FATTORI
            
            '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
            #model_file = f"{model_path}/{best_model_name}.pth"
            
            model_file = f"{model_path}/{best_model_name}.pkl"
            
            '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
            #torch.save(best_model.state_dict(), model_file)
            
            # Salva un dizionario contenente sia i pesi che la configurazione
            torch.save({
                "state_dict": best_model.state_dict(),
                "config": dict(config)
            }, model_file)

            print(f"Il modello \n\033[1m{best_model_name}\033[0m verrà salvato in questa folder directory: \n\033[1m{model_file}\033[0m")

            #Condizione di aggiornamento:
            #Se l'accuracy corrente (accuracy_val) di quel modello di quello sweep supera il valore già salvato in best_models[...], 
            #allora aggiorniamo il dizionario e sovrascriviamo il file del best model, di quel modello, di quella combinazione di fattori.


            # Puoi confrontare e salvare il modello solo se il nuovo è migliore


            #Questo assicura che il salvataggio del modello avvenga solo se
            #il nuovo modello ha un'accuratezza di validazione (max_val_acc) migliore 
            #rispetto a quella già memorizzata per la condizione specifica (exp_cond).

            #In questo modo, si evita di sovrascrivere il modello salvato con uno peggiore


            # Nuovo modello migliore per questa combinazione: aggiorna e sovrascrivi il file


        elif accuracy_val > best_models[exp_cond][data_type][category_subject][config.model_name]["max_val_acc"]:
                best_models[exp_cond][data_type][category_subject][config.model_name] = {
                    "model": best_model,
                    "max_val_acc": accuracy_val,
                    "best_epoch": best_epoch,
                    
                    # Salva la configurazione degli iper-parametri della migliore run di uno sweep 
                    # in relazione ad un certo modello applicato su un dataset costituito da 
                    # una certa combinazione di fattori: 
                    # condizione sperimentale, tipo di dato EEG usato, provenienza del dato usato
                    "config": dict(config)  
                }
                best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
                model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
                os.makedirs(model_path, exist_ok=True)

                print(f"Il modello di questa folder directory:\n\033[1m{model_path}\033[0m")
                print(f"\nHa un MIGLIORAMENTO!")

                '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
                #model_file = f"{model_path}/{best_model_name}.pth"

                model_file = f"{model_path}/{best_model_name}.pkl"

                if os.path.exists(model_file):

                    # Se il file esiste, stampiamo un messaggio di aggiornamento
                    print(f"\n⚠️ ATTENZIONE: \nIl modello \033[1m{best_model_name}\033[0m verrà AGGIORNATO in \n\033[1m{model_path}\033[0m")

                    # Salva il miglior modello solo se è stato aggiornato
                    
                    '''OLD VERSION (SOLO SALVATAGGIO PESI E BIAS DEL MODELLO!'''
                    #torch.save(best_model.state_dict(), model_file)

                    # Salva un dizionario contenente sia i pesi che la configurazione
                    torch.save({
                        "state_dict": best_model.state_dict(),
                        "config": dict(config)
                    }, model_file)
                    
                    print(f"\nIl nome del modello AGGIORNATO è:\n\033[1m{best_model_name}\033[0m")

                else:
                    continue

                #Condizione "nessun miglioramento":
                #Se il modello corrente non migliora il best già salvato, viene semplicemente stampato un messaggio.

                #Questa logica garantisce che per ogni combinazione il file .pth contenga 
                #sempre i pesi del miglior modello (secondo la validation accuracy) fino a quel momento.
                #Adatta eventualmente i nomi delle variabili (es. accuracy_val vs max_val_acc) per essere coerente con il resto del tuo codice.
        else:
            ''''QUI VA RIDEFINITO LA MODEL_PATH (e anche se vuoi MODE_FILE) ALTRIMENTI IN QUESTO ELSE NON ESISTONO!'''

            best_model_name = f"{config.model_name}_{exp_cond}_{data_type}_{category_subject}"
            model_path = os.path.join(base_dir, exp_cond, data_type, category_subject)
            model_file = f"{model_path}/{best_model_name}.pth"
            print(f"Nessun miglioramento per il modello \033[1m{config.model_name}\033[0m in \n\033[1m{model_path}\033[0m, ossia \n\033[1m{model_file}\033[0m")

    wandb.finish()
    
    torch.cuda.empty_cache()
    
    return best_models


#### **Weight & Biases Procedure Final Edits - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS**
#### **Sweep separati per ciascuno dei modelli CNN3D e CNN Sep**

In [ ]:
'''

                                        QUI IL LOOP LO ESEGUO SU OGNI SINGOLO SWEEP DI OGNI COMBINAZIONE DI FATTORI!!!
                                                            
                                                                    VERSIONE C 
                                                                    
                                                                    
                                                W&B SWEEPS AND TRAING LAUNCH WITH MULTIPLE GPUs MANAGEMENT
                                        
Questa volta, invece, andiamo ad iterare rispetto a 

- sweep_tuple, che la tuple che contiene

1) relativo codice stringa univoco dello Sweep ID
2  la sua combination_key, che ri-associa allo Sweep ID la combinazione di fattori della relativa condizione sperimentale


PRIMA FACEVO IN QUESTO MODO

for sweep_id in sweep_ids[condition][data_type][category_subject]:
    print(f"\033[1mInizio l'agent\033[0m per sweep_id: \033[1m{sweep_id}\033[0m")
    
ORA INVECE ITERO SULLA TUPLA!


for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            for sweep_tuple in sweep_ids[condition][data_type][data_tuples]:
        

VERSIONE C (SEMPLIFICATA!)


****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******

SPIEGAZIONE

GPU counter: Ho aggiunto un contatore (gpu_counter) che cicla tra le GPU disponibili. 

In questo modo, il primo sweep sarà eseguito sulla GPU 0, il secondo sulla GPU 1, e così via. 
Quando il contatore raggiunge il numero di GPU disponibili, torna a 0 per riusare la prima GPU.

Rotazione delle GPU: All'interno del loop, per ogni sweep, viene assegnata una GPU diversa. 
Se ci sono più di 1 GPU, il contatore incrementa, e la variabile CUDA_VISIBLE_DEVICES cambia automaticamente per assegnare la GPU corretta.

Esecuzione parallela: Ogni sweep viene eseguito su una GPU separata. Se ci sono 2 GPU, il primo sweep va su GPU 0, il secondo su GPU 1, il terzo su GPU 0, e così via.

Risposta alla tua domanda:
In questo modo, ogni sweep_id viene eseguito una sola volta, ma su GPU diverse (se disponibili). Non ci sono duplicati dello stesso sweep su entrambe le GPU.


DOMANDE SUL NUOVO CODICE

1) Gli sweep sono eseguiti già in parallelo giusto?
No, in questo caso gli sweep non sono eseguiti in parallelo in modo esplicito tramite il codice che hai scritto.

Anche se hai assegnato ciascun sweep a una GPU diversa, il codice esegue sequenzialmente ogni sweep, solo che li distribuisce su GPU differenti in modo rotazionale.
Ogni volta che il ciclo passa ad un nuovo sweep, assegna un ID GPU e poi esegue l'agent su quella GPU. Non vengono eseguiti in parallelo a livello di codice.

2) O semplicemente in questo modo faccio in modo di distribuire ogni sweep sull'altra GPU rispetto a quella usata dallo sweep precedente
per "ottimizzare" il carico computazionale di ogni GPU?

Esatto! Quello che stai facendo è distribuire i vari sweep su GPU diverse, assicurandoti che ogni sweep venga eseguito su una GPU separata (se ne hai di disponibili).
Questo permette di ottimizzare l'uso delle risorse, evitando che una GPU venga sovraccaricata da più sweep. Se il numero di GPU disponibili è maggiore di 1, 
allora i sweep sono distribuiti sulle diverse GPU, ma ogni sweep sarà ancora eseguito singolarmente.





Sì, con il codice che hai fornito, stai distribuendo gli sweep tra le diverse GPU, in modo da ottimizzare il carico computazionale e non sovraccaricare una sola GPU.

Dettaglio del funzionamento:
Distribuzione delle GPU (rotazionale):

Quando ci sono più di una GPU, il codice assegna a ciascun sweep una GPU diversa in modo rotazionale.

Per ogni ciclo del loop, la variabile gpu_counter determina a quale GPU assegnare il prossimo sweep.

Se ci sono 2 GPU, il primo sweep viene eseguito sulla GPU 0, il secondo sulla GPU 1, il terzo di nuovo sulla GPU 0, e così via.

Gestione della GPU:

Se hai più di una GPU, os.environ["CUDA_VISIBLE_DEVICES"] imposta il dispositivo GPU corrente su cui il codice deve girare (GPU 0 o GPU 1). Questo permette di gestire quale GPU eseguirà l'addestramento per ciascun sweep.

Quando num_gpus > 1, il codice alterna l'assegnazione della GPU per ogni sweep, evitando di sovraccaricare una singola GPU con troppe operazioni contemporaneamente.

Ottimizzazione del carico computazionale:

L'alternanza tra GPU successive per ciascun sweep aiuta a distribuire il carico in modo equilibrato, specialmente se il numero di sweep è alto.

In pratica, se hai 2 GPU e 10 sweep da eseguire, ogni GPU eseguirà 5 sweep, evitando di saturare una singola GPU.

Parallelizzazione effettiva:
Nel tuo codice non c'è parallelismo vero e proprio tra le GPU (come quello che otterresti con DataParallel o DistributedDataParallel), ma piuttosto distribuisci gli sweep su GPU diverse. Ogni sweep è eseguito sequenzialmente su una GPU diversa, ma non stai parallelizzando il training dello stesso sweep su più GPU.

Se volessi fare in modo che un singolo sweep fosse parallelizzato su più GPU (in modo che il lavoro venga diviso tra le GPU per uno stesso sweep), dovresti usare DataParallel o DistributedDataParallel, come descritto in precedenza. Ma in questo caso, ogni sweep è isolato e viene eseguito su una sola GPU, quindi non c'è un vero parallelismo dentro ogni singolo sweep.

Domande che potrebbero sorgere:
Gli sweep vengono eseguiti in parallelo?

No, i sweep sono eseguiti uno alla volta, ma su GPU diverse. Quindi, mentre il primo sweep usa la GPU 0, il secondo usa la GPU 1 e così via. Ogni sweep viene gestito separatamente, ma sfrutti più GPU per parallelizzare l'esecuzione di più sweep contemporaneamente.

La distribuzione delle GPU tra gli sweep è ottimizzata?

Sì, stai bilanciando il carico computazionale tra le GPU, assegnando a ogni GPU uno sweep alternato. Se hai molte GPU, puoi ottimizzare ulteriormente distribuendo i sweep su più dispositivi.

Se volessi parallelizzare più agenti W&B su diverse GPU, il codice che stai utilizzando sarebbe corretto, ma per ottimizzare ulteriormente i tempi di esecuzione, potresti prendere in considerazione anche l'utilizzo di tecniche come DataParallel o DistributedDataParallel per far sì che più GPU lavorino contemporaneamente sullo stesso sweep. Ma la logica che hai già implementato va bene per distribuire più sweep tra le GPU.

Se hai bisogno di ulteriori dettagli su come implementare il parallelismo vero e proprio (inclusi DataParallel o DistributedDataParallel), fammi sapere!



****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******



'''



'''

Per modificare il loop in modo che accetti i sweeps per ogni modello e gestisca correttamente
l'esecuzione del training per ciascun modello con il relativo sweep, dobbiamo fare alcune modifiche.


Modifiche principali:

1) Funzione make_train_wrapper:
La funzione dovrà essere adattata per passare correttamente la configurazione di sweep per ogni modello, 
invece di passare un'unica configurazione generica (sweep_config).

2) Identificazione corretta del modello: 
Nel loop, per ogni combinazione (condition, data_type, category_subject)
e per ogni modello (ad esempio, CNN3D_LSTM_FC e SeparableCNN2D_LSTM_FC), 

dobbiamo passare al wandb.agent il relativo sweep ID per il modello e la sua configurazione.

3) Modifica della funzione make_train_wrapper per gestire ogni modello separatamente: 
Ogni modello avrà il proprio sweep e la propria configurazione.


Spiegazione delle modifiche:

1) Funzione make_train_wrapper:

Adesso prende anche model_name per passare il relativo sweep_config dal dizionario sweep_config_dict.
Passa il sweep_config corretto per ogni modello, a seconda del model_name passato nel ciclo.

2) Dizionario sweep_config_dict:

Ho creato un dizionario sweep_config_dict che associa ciascun modello ("CNN3D_LSTM_FC" e "SeparableCNN2D_LSTM_FC")
alla sua configurazione di sweep (sweep_config_cnn3d e sweep_config_cnn_sep).
Questo permette di usare la corretta configurazione per ogni modello.

3) Modifica nel ciclo:

Il ciclo ora scorre su model_name (i.e., i modelli CNN3D_LSTM_FC e SeparableCNN2D_LSTM_FC) 
per ogni combinazione di condition, data_type, category_subject.

Per ogni modello, il relativo sweep viene creato ed eseguito.


Risultato:
Ora, per ogni combinazione di condition, data_type, e category_subject, 
il codice creerà e gestirà separatamente gli sweeps per ciascun modello,
e li eseguirà utilizzando la funzione training_sweep con la relativa configurazione specifica per ogni modello.

Questa modifica ti consente di avere il corretto flusso di lavoro per eseguire
il training separato per ogni modello con la sua configurazione.


'''


import time  # Importa il modulo time


# Definiamo una funzione wrapper che "cattura" lo sweep_id e le altre variabili

'''ATTENZIONE AGGIUNTO model_name tra i parametri di --> make_train_wrapper'''

def make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name):
    def train_wrapper():

        # Qui chiamiamo la funzione di training con i parametri appropriati
        #print(f"\nSetto il training per lo Sweep ID \033[1m{condition}_{data_type}_{category_subject}\033[0m con sweep_id {sweep_id}")
        #print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m")
        
        print(f"\nSetting Up Training per lo Sweep ID \033[1m{sweep_id}\033[0m --> \033[1m{combination_key}\033[0m, modello \033[1m{model_name}\033[0m")
        training_sweep(
            data_dict_preprocessed, 
            sweep_config_dict[model_name], # Prendi la configurazione per il modello specifico
            sweep_ids,
            sweep_id,
            sweep_tuple,
            best_models  # Best models viene aggiornato all'interno della funzione
        )
    return train_wrapper
                        

# Dizionari di configurazione per ogni modello
sweep_config_dict = {
    "CNN3D_LSTM_FC": sweep_config_cnn3d,
    "SeparableCNN2D_LSTM_FC": sweep_config_cnn_sep
}

# Verifica quante GPU sono disponibili
num_gpus = torch.cuda.device_count()


# Crea un contatore per assegnare un GPU diversa a ciascun sweep
gpu_counter = 0

# Registra il tempo di inizio
start_time = time.time()

for condition in sweep_ids:
    for data_type in sweep_ids[condition]:
        for category_subject in sweep_ids[condition][data_type]:
            
            for model_name in sweep_ids[condition][data_type][category_subject]:  # Aggiunto loop per il modello
                
                #for sweep_tuple in sweep_ids[condition][data_type][category_subject]:
                
                 for sweep_tuple in sweep_ids[condition][data_type][category_subject][model_name]:  # Itera sugli sweep per ciascun modello

                    # Esegui l'unpacking della tupla per ottenere solo il primo elemento della tupla (sweep_id, combination_key)
                    sweep_id, combination_key = sweep_tuple
                    
                    
                    combination_key = f"{condition}_{data_type}_{category_subject}"
                    
                    # Un modo efficace per "catturare" il contesto (come sweep_id e le altre variabili) 
                    # per ogni iterazione è definire una funzione wrapper locale all'interno del ciclo
                    # In questo modo, ogni volta che chiami l'agente, il wrapper avrà già i parametri specifici per quella combinazione


                    # Se ci sono più di 1 GPU, assegna a ciascuna GPU uno sweep diverso
                    if num_gpus > 1:

                        '''ATTENZIONE AGGIUNTO model_name tra i parametri di --> make_train_wrapper''' 
                        
                        # Assegna la GPU in modo rotazionale
                        os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_counter)
                        
                        agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name)
                    
                        wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband", count=200)
                        
                        # Passa alla prossima GPU per il prossimo sweep
                        gpu_counter = (gpu_counter + 1) % num_gpus

                    else:
                        
                        # Se c'è una sola GPU, esegui il sweep sulla GPU 0
                        os.environ["CUDA_VISIBLE_DEVICES"] = "0"
                        
                        agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name)
                        
                        wandb.agent(sweep_id, function=make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject, model_name), project=f"{condition}_spectrograms_channels_freqs_new_3d_grid_multiband", count=200)


                    # Crea la funzione wrapper per l'agent
                    '''COMMENTATO'''
                    #agent_function = make_train_wrapper(sweep_id, sweep_tuple, condition, data_type, category_subject)


                    # NOTA: non assegno il valore di wandb.agent a best_models, lascio che training_sweep aggiorni best_models internamente!
                    '''DEVI INSERIRE PER L'AGENTE COME PARAMETRO IL NOME DELLA CONDIZIONE SPERIMENTALE DEL PROGETTO SU  W&B
                       ALTRIMENTI CERCA LO SWEEP NEL PROGETTO SBAGLIATO '''

                    print(f"Inizio l'\033[1magent\033[0m per \033[1msweep_id\033[0m \tN°: \033[1m{sweep_tuple}\033[0m")

                    '''COMMENTATO'''
                    #wandb.agent(sweep_id, function=agent_function, project = f"{condition}_spectrograms_channels_freqs_new_2d_grid_multiband_topomap", count=15)

                    print(f"\nLo sweep id corrente \033[1m{sweep_id}\033[0m ha la combinazione di fattori stringhe: \033[1m{condition}; {data_type}; {category_subject}\033[0m\n")

                    torch.cuda.empty_cache()

# Registra il tempo di fine
end_time = time.time()

# Calcola il tempo totale
total_time = end_time - start_time
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = int(total_time % 60)

# Stampa il tempo totale in formato leggibile
print(f"\nTempo totale impiegato: \033[1m{hours} ore, {minutes} minuti e {seconds} secondi\033[0m.\n")

In [ ]:
print('Finito Training su W&B !')

In [ ]:
# Stampa il numero totale di sweeps
print(f"Numero totale di sweeps che verranno eseguiti: {total_sweeps}")

## Impostazione **Recupero DL Optimized Models** con **Rappresentazione Frequenza-Elettrodi dei miei dati EEG** a seconda del Dataset del Task scelto


Notebook **"EEG Data Stats Descriptives & GradCam Checks"**

- Task 1: Motor Execution (Open and close right or left fist) - Unilateral - Runs 3, 7, 11 
- Task 2: Motor Imagery (Imagine opening and closing right and left fists) - Unilateral - Runs 4, 8, 12 
- Task 3: Motor Execution (Open and close both fists and feet) - Bilateral - Runs 5, 9, 13 
- Task 4: Motor Imagery (Imagine opening and closing both fists and feet) - Bilateral - Runs 6, 10, 14 

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
path = '/home/stefano/EEGMMIDB_Analyses/'

# Caricare l'intero dizionario annidato con pickle
#with open(f'{path_channels}ordered_electrodes.pkl', 'rb') as f:

with open(f'{path}electrode_grid_map.pkl', 'rb') as f:
    EEG_channels_names = pickle.load(f)

In [ ]:
type(EEG_channels_names)

In [ ]:
# supponendo che EEG_channels_names sia la tua lista originale
#EEG_channels_names = [ch.rstrip('.') for ch in EEG_channels_names]
#print(EEG_channels_names, end = '')

In [ ]:
#len(EEG_channels_names)

In [ ]:
len(EEG_channels_names)

In [ ]:
EEG_channels_names

#### **Weight & Biases Login & REMOTE CHECKS - EEG Spectrograms - Electrodes x Frequencies**

##### **Weight & Biases Login PRECEDURA CORRETTA ✅ (CANCELLAZIONE RUNS e NON SWEEPS!) - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
'''

Puoi usare l’API Python di W&B per elencare da remoto tutti i progetti di una certa entity (tuo utente o organizzazione).

Ad esempio:

'''

import wandb

# 1) autenticati (se non l’hai già fatto con wandb.login())
# wandb.login()

# 2) crea un’istanza dell’API
api = wandb.Api()

# 3) scegli la tua entity (nome utente o org). 
#    Se omesso, prende la tua entity di default.

entity = 'stefano-bargione-universit-di-roma-tor-vergata'

# 4) prendi la lista dei progetti
projects = api.projects(entity=entity)

# 5) stampa i nomi
for proj in projects:
    print(proj.name)

#### **Weight & Biases Procedure FINAL SEQUENCE OF STEPS - EEG Spectrograms - Electrodes x Frequencies ONLY HYPER-PARAMS**

In [ ]:
#Library Importing 
    
import os
import math
import copy as cp 

import tqdm
from tqdm import tqdm

import random 

#import mne 
import scipy

import numpy as np  # NumPy per operazioni numeriche
import matplotlib.pyplot as plt  # Matplotlib per la visualizzazione dei dati

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader

import os
import pickle

import random

import wandb

In [ ]:
base_path = '/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/'

# Controlla se la cartella esiste, altrimenti la crea
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
import pickle

# Caricare l'intero dizionario annidato con pickle
with open(f'{base_path}new_all_th_concat_spectrograms_coupled_exp_task_1_2d_grid_multiband.pkl', 'rb') as f:
    new_all_th_concat_spectrograms_coupled_exp_task_1 = pickle.load(f)

In [ ]:
new_all_th_concat_spectrograms_coupled_exp_task_1.keys() 

In [ ]:
new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist'].keys()

In [ ]:
print(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist']['data'].shape)
print()
print(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist']['data'].shape)
print()
print(new_all_th_concat_spectrograms_coupled_exp_task_1['left_fist_vs_right_fist']['data'].shape)
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_left_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['rest_vs_right_fist']['labels'], return_counts = True))
print()
print(np.unique(new_all_th_concat_spectrograms_coupled_exp_task_1['left_fist_vs_right_fist']['labels'], return_counts = True))

#### **Utils Functions - EEG Spectrograms - Electrodes x Frequencies**

In [ ]:
'''
QUI DENTRO HO CONFIGURATO 
LE FUNZIONI DI CONTROLLO DELLE STRINGHE 
PER IL SALVATAGGIO DELLE PERFORMANCE DEL MODELLO
NELLE RELATIVE SUBFOLDERS

(I.E., get_subfolder_from_key, get_subfolder_from_key_hyper)

IN MODO CHE SI LEGHINO ALLA CHIAVE 'STANDARDIZATION' DELL'OGGETTO SWEEP_CONFIG

'''


import pickle
import numpy as np


def load_data(data_type, category, subject_type, condition = "th_resp_vs_pt_resp"):
    """
    Carica i dati EEG dalla directory appropriata, già salvati con la finestra temporale (0°-640° punto)

    Parameters:
    - data_type: str, "spectrograms",
    - category: str, "task_1"
    - subject_type: str, "th" (terapisti)
    - condition: str, condizione sperimentale da selezionare
    

    Returns:
    - X: Dati EEG sotto-selezionati (50°-300° punto e canali selezionati se applicabile)
    - y: Etichette corrispondenti
    """

    # Definizione dei percorsi base
    base_paths = {
        "spectrograms": {
            "familiar": "/home/stefano/Interrogait/all_datas/Spectrograms_channels_frequencies_motor_tasks/"
            #"unfamiliar": "/home/stefano/Interrogait/all_datas/Unfamiliar_Spectrograms/"
        },
    }

    # Seleziona il path corretto
    base_path = base_paths[data_type][category]

    # Determina il nome del file corretto
    if data_type in ["spectrograms"]:
        filename = f"new_all_{subject_type}_concat_spectrograms_coupled_exp_task_1_2d_grid_multiband.pkl"
    else:
        raise ValueError("data_type non valido!")
        
    # Caricamento del file
    filepath = base_path + filename
    
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    
    '''
    Per i dati spectrogram, la funzione seleziona la condizione desiderata (i.e., condition = "th_resp_vs_pt_resp") 
    e preleva i dati e le etichette associati a quella condizione.
    '''
    
    # Selezione della finestra temporale e delle etichette
    X = data[condition]["data"]
    y = data[condition]["labels"]

    
    return X, y


def select_channels(data, channels=[12, 30, 48]):
    """
    Seleziona i canali EEG specificati SOLO per i dati 1-20 e 1-45.

    Parameters:
    - data: array NumPy, dati EEG con shape (n_trials, n_channels, n_timepoints)
    - channels: list, indici dei canali da selezionare

    Returns:
    - data filtrato sui canali specificati
    """
    return data[:, channels, :]


# Funzione per train-test split
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

def split_data(X, y, test_size=0.2, val_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test

'''ATTENZIONE CAMBIATA'''
# Funzione per standardizzare i dati
def standardize_data(X_train, X_val, X_test, eps = 1e-8):
    
    mean = X_train.mean(axis=0, keepdims=True)
    std = X_train.std(axis=0, keepdims=True)
    
    X_train = (X_train - mean) / (std + eps)
    X_val = (X_val - mean) / (std + eps)
    X_test = (X_test - mean) / (std + eps)
    
    return X_train, X_val, X_test


# Import modelli (definisci le classi CNN1D, ReadMEndYou, ReadMYMind)
#from models import CNN1D, ReadMEndYou, ReadMYMind  # Assicurati di avere i modelli definiti in 'models.py'

# Funzione per inizializzare i modelli
def initialize_models():
    #model = CNN1D(input_channels=3, num_classes=2)
    model_CNN = CNN2D(input_channels = 61, num_classes=2)
    #model_LSTM = ReadMEndYou(input_size=3, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    model_LSTM = ReadMEndYou(input_size=61 * 26, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
    #model_Transformer = ReadMYMind(num_channels=3, seq_length=250, d_model=16, num_heads=4, num_layers=2, num_classes=2)
    model_Transformer = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=61, freqs=26)
    
    return model_CNN, model_LSTM, model_Transformer


import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight


'''
Questa funzione prende in input i dati di training, validation e test, 
il tipo di modello scelto e la dimensione del batch. Si occupa di:

Calcolare i pesi delle classi.
Convertire i dati in tensori PyTorch, con le opportune trasformazioni per CNN, LSTM o Transformer.
Creare i dataset e i dataloader per il training.
'''


def prepare_data_for_model(X_train, X_val, X_test, y_train, y_val, y_test, model_type, batch_size=48):
    
    # Calcolo dei pesi delle classi
    class_weights = compute_class_weight(class_weight='balanced', 
                                         classes=np.unique(y_train), 
                                         y=y_train)
    
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
    class_weights_tensor = class_weights_tensor.to(dtype=torch.float32, device=device)
    
    # Conversione delle etichette in interi
    y_train = y_train.astype(int)
    y_val = y_val.astype(int)
    y_test = y_test.astype(int)
    
    # Conversione dei dati in tensori PyTorch con permutazione se necessario
    
    '''ATTENZIONE CAMBIATO QUI!'''
    #if model_type == "CNN2D":
    
    if model_type == "CNN2D_LSTM_FC":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    elif model_type == "TopomapNet":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    elif model_type == "CNN3D_LSTM_FC":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    elif model_type == "SeparableCNN2D_LSTM_FC":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

    #BiLSTM (ReadMEndYou):
    #Ora il modello si aspetta l’input con shape (batch, canali, frequenze, tempo) 
    #e, al suo interno, 
    #esegue la permutazione per avere il tempo come dimensione sequenziale. 
    #Non serve quindi applicare una permutazione anche qui.
    
    elif model_type == "BiLSTM":
            
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    #Transformer (ReadMYMind):
    #Analogamente, il modello gestisce internamente la riorganizzazione dell’input, quindi lasciamo i dati nella loro forma originale.
    elif model_type == "Transformer":
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    
    else:
        raise ValueError("Modello non riconosciuto. Scegli tra 'CNN', 'LSTM' o 'Transformer'.")
    
    # Conversione delle etichette in tensori
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    # Creazione dei dataset
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    # Creazione dei dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, class_weights_tensor


'''
OLD VERSIONS BEFORE GRADCAM COMPUTATION ON CNN2D

# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, model_standardization):
    
    #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
    if '_familiar_th' in key:
        return 'th_fam'
    elif '_unfamiliar_th' in key:
        return 'th_unfam'
    elif '_familiar_pt' in key:
        return 'pt_fam'
    elif '_unfamiliar_pt' in key:
        return 'pt_unfam'
    else:
        return None
    
     
# Funzione per salvare i risultati
def save_performance_results(model_name, my_train_results, my_test_results, key, exp_cond, model_standardization, base_folder):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, model_standardization)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "spectrograms" in key:
        data_type_str = "spectrograms"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file con l'inclusione della combinazione key + model_name
    if model_standardization:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}_std.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
        
    else:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\nRisultati salvati con successo 👍 in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"❌Errore durante il salvataggio dei risultati: {e}")
'''


#NEW VERSIONS FOR SPECTROGRAMS WITH GRADCAM COMPUTATION ON CNN2D!

# Funzione per determinare a quale subfolder appartiene la chiave
def get_subfolder_from_key(key, model_standardization):
    
    #DEFINIZIONE DELLA PATH DOVE VIENE SALVATO IL FILE
    if '_familiar_th' in key:
        return 'th_fam'
    elif '_unfamiliar_th' in key:
        return 'th_unfam'
    elif '_familiar_pt' in key:
        return 'pt_fam'
    elif '_unfamiliar_pt' in key:
        return 'pt_unfam'
    else:
        return None


from PIL import Image
import io
import pickle
import os
     
# Funzione per salvare i risultati
def save_performance_results(model_name, 
                             my_train_results,
                             my_test_results, 
                             key,
                             exp_cond,
                             model_standardization,
                             base_folder,
                             gradcam_image = None):
    """
    Funzione che salva i risultati del modello in base alla combinazione di 'key' e 'model_name'.
    Se gradcam_image è fornita, la salva anche in formato PNG con un nome che inizia con 'GradCAM_results'.
    """
    
    # Identificazione del subfolder in base alla chiave
    subfolder = get_subfolder_from_key(key, model_standardization)
    
    # Debug: controllo sulla subfolder
    print(f"\nDEBUG - Chiave: \033[1m{key}\033[0m, Subfolder ottenuto: \033[1m{subfolder}\033[0m")
    
    if subfolder is None:
        print(f"Errore: La chiave \033[1m{key}\033[0m non corrisponde a nessun subfolder valido.\n")
        return
    
    # Determinazione del tipo di dato direttamente dalla chiave
    if "spectrograms" in key:
        data_type_str = "spectrograms"
    else:
        print(f"Errore: Tipo di dato non riconosciuto nella chiave '{key}'.")
        return

    # Creazione del nome del file pickle con l'inclusione della combinazione key + model_name
    if model_standardization:
        
        '''AGGIUNGO _multiband ALLA FINE PER DISTINGUERE DA QUELLI TOTAL POWER'''
        
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}_std_3d_multiband.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
    else:
        file_name = f"{model_name}_performances_{exp_cond}_{data_type_str}_{subfolder}_3d_multiband.pkl"
        folder_path = os.path.join(base_folder, exp_cond, data_type_str, subfolder)
    
    # Verifica se la cartella di destinazione esiste, altrimenti creala
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, file_name)

    # Creazione del dizionario con i risultati
    results_dict = {
        'my_train_results': my_train_results,
        'my_test_results': my_test_results
    }

    # Salvataggio del dizionario con i risultati
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"\n🔬Risultati salvati con successo 👍 in: \n\033[1m{file_path}\033[0m\n")
    except Exception as e:
        print(f"❌Errore durante il salvataggio dei risultati: {e}")
    
    # Se è stata fornita l'immagine GradCAM, salvala come file PNG
    if gradcam_image is not None:
        if model_standardization:
            
            '''AGGIUNGO _multiband ALLA FINE PER DISTINGUERE DA QUELLI TOTAL POWER'''
            
            gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}_std_3d_multiband.png"
        else:
            gradcam_file_name = f"GradCAM_results_{model_name}_{exp_cond}_{data_type_str}_{subfolder}_3d_multiband.png"
        
        gradcam_file_path = os.path.join(folder_path, gradcam_file_name)
        
        #try:
        #    with open(gradcam_file_path, "wb") as f_img:
        #        f_img.write(gradcam_image)
        #    print(f"\n📸Immagine GradCAM salvata con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")
        
        try:
            
            '''
            Se gradcam_image è un oggetto BytesIO, allora rappresenta un flusso di dati binari in memoria.
            Quando si leggono dati da un BytesIO, il cursore interno avanza come in un file normale. 
            Se il cursore non è all'inizio, Image.open() potrebbe non leggere correttamente l'immagine.
            👉 seek(0) riporta il cursore all'inizio del buffer prima di leggerlo con Image.open()
            
            Per maggior info leggi cella successiva!
            '''
            
            # 🔄 Se gradcam_image è un buffer, convertirlo in immagine PIL
            if isinstance(gradcam_image, io.BytesIO):
                gradcam_image.seek(0)  # 🔄 Reset puntatore del buffer
                gradcam_image = Image.open(gradcam_image)
            
            '''
            Il messaggio di errore indica che il tuo oggetto gradcam_image è di tipo bytes e non ha il metodo save(), 
            che è tipico di un oggetto PIL. 
            
            Per risolvere questo, devi convertire i byte in un'immagine PIL. 
            Per farlo, controlla se gradcam_image sia un oggetto di tipo bytes e,
            in tal caso, usa io.BytesIO per creare un buffer da passare a Image.open(). 
            
            Inserisci questa conversione all'interno del blocco che salva l'immagine, così da assicurarti che,
            indipendentemente dal tipo, gradcam_image diventi un oggetto PIL e possa chiamare il metodo save().
            '''
            
            if isinstance(gradcam_image, bytes):
                gradcam_image = io.BytesIO(gradcam_image)
                gradcam_image.seek(0)
                gradcam_image = Image.open(gradcam_image)
            
            
            print(f"\n📸Immagine \033[1mGradCAM salvata\033[0m con successo 👍 in: \n\033[1m{gradcam_file_path}\033[0m\n")
            # 🔄 Salvare l'immagine nel percorso specificato
            gradcam_image.save(gradcam_file_path, format = "PNG")
            
        except Exception as e:
            print(f"❌Errore durante il salvataggio dell'immagine GradCAM: {e}")

#### **NUOVE UTILS DATI POST W&B (PER RICHIAMARE MODELLI OTTIMIZZATI)**

In [ ]:
'''
Parsing della chiave e costruzione del path:
Usando la funzione parse_combination_key si estraggono 

exp_cond, data_type e category_subject dalla chiave del dataset. 

Questi vengono usati per costruire il percorso in cui cercare i file .pkl.
'''
import re 

# Funzione per parsare la chiave
def parse_combination_key(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    Il formato atteso è:
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ 
    "1_20|1_45|wavelet" _ 
    "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    """
    match = re.match(
        #r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$", 
        #r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_spectrograms_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_(spectrograms)_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        

'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

# Test
combination_key = "rest_vs_left_fist_spectrograms_familiar_th"
condition_experiment, data_type, subject_key = parse_combination_key(combination_key)

print("Condizione:", condition_experiment)
print("Data Type:", data_type)
print("Soggetto:", subject_key)

In [ ]:
'''
Verifica del file .pkl:
La funzione load_config_if_available cerca, per ogni modello, il file con nome del tipo
"{model_name}_{exp_cond}_{data_type}_{category_subject}.pkl"
all’interno della struttura di cartelle basata su base_path. 

Se il file esiste, allora viene passata poi a load_model_config_and_weights, 
che carica il dizionario di partenza 
e da questo estrae i 2 sotto-dizionari 'config' e 'state_dict'.
'''

def load_config_if_available(dataset_key, model_name, base_path):
    """
    Data una chiave (es. "th_resp_vs_pt_resp_wavelet_familiar_th") e il nome del modello,
    cerca il file .pkl corrispondente e ritorna (config, state_dict).
    Se non esiste, restituisce (None, None).
    """
    try:
        exp_cond, data_type, category_subject = parse_combination_key(dataset_key)
        config, state_dict = load_model_config_and_weights(exp_cond, data_type, category_subject, model_name, base_path)
        print(f"✅ File .pkl trovato per \033[1m{model_name}\033[0m su \033[1m{dataset_key}\033[0m")
        
        return config, state_dict
    except Exception as e:
        print(f"⚠️ Nessun file .pkl per {model_name} su {dataset_key} - uso parametri di default. ({e})")
        return None, None

In [ ]:
'''
Caricamento del file .pkl:
La funzione load_model_config_and_weights cerca, per ogni modello, il file con nome del tipo
"{model_name}_{exp_cond}_{data_type}_{category_subject}.pkl"
all’interno della struttura di cartelle basata su base_path. Se il file esiste, vengono restituiti config e state_dict.
'''

# Funzione per caricare il file .pkl con la configurazione e i pesi ottimali
def load_model_config_and_weights(exp_cond, data_type, category_subject, model_name, base_path):
    """
    Costruisce il path usando:
        base_path / exp_cond / data_type / category_subject
    e il nome del file:
        {model_name}_{exp_cond}_{data_type}_{category_subject}.pkl
    Se il file esiste, lo carica e restituisce (config, state_dict).
    """
    
    file_name = f"{model_name}_{exp_cond}_{data_type}_{category_subject}.pkl"
    file_path = os.path.join(base_path, exp_cond, data_type, category_subject, file_name)
    
    if os.path.exists(file_path):
        print(f"🕵️‍♂️🔍Caricamento file .pkl: \033[1m{file_path}\033[0m")
        
        # Il file .pkl è stato salvato con torch.save() e contiene un dizionario con chiavi al suo interno che sono: "config" e "state_dict"
        with open(file_path, "rb") as f:
            data = torch.load(f)
        return data["config"], data["state_dict"]
    else:
        raise FileNotFoundError(f"File {file_path} non trovato.")

#### **TESTING**

In [ ]:
'''
TESTING FUNCTION: CORRETTA ANCHE PER IL GRAD-CAM

SUCCESSIVAMENTE, DENTRO AL FOR LOOP DEL TRAINING E TESTING, 
SI RICHIAMA LA FUNZIONE DIRETTAMENTE DI 

1) compute_gradcam_figure, LA QUALE AL SUO INTERNO PRESENTA GIÀ 
TUTTO QUELLO CHE SERVE PER CALCOLARE IL GRADCAM, DI MODO CHE VADA A 

Selezionare esempi rappresentativi per ciascuna classe.
Calcolare le mappe GradCAM e gli overlay.
Creare una figura con le heatmap e le sovrapposizioni, completa di titoli esplicativi.
Restituire un'immagine (buffer) pronta per essere salvata

SUCCESSIVAMENTE, QUINDI, IL PROCEDIMENTO DIVENTA COME SEGUE:

1) Si esegue il TESTING, per ottenere le metriche e salvare i risultati (senza GradCAM)

2) Nel loop principale di TRAINING & TESTING, se il modello è CNN2D, allora 

 - richiama la funzione 'compute_gradcam_figure', la quale va a
    - calcolare le mappe di attivazione e successivamente creo le immagini che gli ho chiesto
    - passa l'immagine ottenuta da GradCAM alla funzione 'save_performance_results', la quale va a 
        - salvare i risultati di test ottenuti dalla funzione di 'testing'
        - salvare l'immagine risultatante del GradCAM e la sovrapposizione del GradCAM sullo spettrogramma originale della classe risultante
        
        
Questo approccio garantisce chiarezza e separa la parte di performance (testing) dalla parte di explainability (GradCAM).


'''

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

import io
from PIL import Image

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


def testing(results, test_loader, criterion):
    
    # Recupera il miglior modello ottenuto durante la validazione
    model = results['best_model']
    model.to(device)
    
    model.eval()  # Imposta il modello in modalità valutazione

    y_true_list = []  # Lista per salvare le etichette reali
    y_pred_list = []  # Lista per salvare le previsioni del modello
    
    '''AGGIUNTA NUOVA PER CALCOLO AUC-ROC'''
    y_score_list = []   # <— Lista per salvare gli score per le probabilità della classe positiva (per auc-roc!)
    
    total_loss = 0
    correct = 0
    
    test_performances = {
        "test_loss": [],
        "test_accuracy": [],
        "test_precision": [],
        "test_recall": [],
        "test_f1_score": [],
        "test_auc": []
    }
    

    with torch.no_grad():
        
        pbar = tqdm(test_loader, desc="Testing")
        
        for inputs, labels in pbar:
            
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Ottenere le predizioni del modello
            outputs = model(inputs)
            
            '''AGGIUNTA NUOVA PER CALCOLO AUC-ROC'''
            # aggiungi queste due righe
            probs = torch.softmax(outputs, dim=1)
            y_score_list.extend(probs[:,1].cpu().numpy())

            # Calcolare la loss
            test_loss = criterion(outputs, labels)
            total_loss += test_loss.item()

            # Memorizzare predizioni ed etichette vere
            _, predicted = torch.max(outputs, 1)
            y_pred_list.extend(predicted.cpu().numpy())
            y_true_list.extend(labels.cpu().numpy())

            # Aggiornare il numero di predizioni corrette
            correct += (predicted == labels).sum().item()

            pbar.set_description(f"Loss: {test_loss.item():.4f}")

    # Calcolare l'accuratezza complessiva
    accuracy = correct / len(test_loader.dataset)
    
    
    # Calcolare precision, recall, F1-score, AUC durante il testing
    precision_test = precision_score(y_true_list, y_pred_list, average='weighted')
    recall_test = recall_score(y_true_list, y_pred_list, average='weighted')
    f1_test = f1_score(y_true_list, y_pred_list, average='weighted')
    
    '''OLD VERSION'''
    #auc_test = roc_auc_score(y_true_list, y_pred_list, average='weighted')  # Assicurati che il problema sia binario o multi-class
    
    '''AGGIUNTA NUOVA PER CALCOLO AUC-ROC
    
    In questo modo l’roc_auc_score calcola l’area sotto tutta la curva ROC (tutte le soglie), invece di valutare un solo punto corrispondente alla soglia 0.5
    '''
    
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    auc_test = roc_auc_score(y_true_list, y_score_list)


    # Aggiungere questi valori nel dizionario delle performance (arrotondando a 4 decimali)
    test_performances["test_loss"].append(round(total_loss / len(test_loader), 4))  # Media della loss
    test_performances["test_accuracy"].append(round(accuracy, 4))
    test_performances["test_precision"].append(round(precision_test, 4))
    test_performances["test_recall"].append(round(recall_test, 4))
    test_performances["test_f1_score"].append(round(f1_test, 4))
    test_performances["test_auc"].append(round(auc_test, 4))
    
    # Creare la confusion matrix
    conf_matrix = confusion_matrix(y_true_list, y_pred_list)
    
    # Stampare classification report
    class_report = classification_report(y_true_list, y_pred_list)

    print(f"\nTest Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", class_report)

    # Visualizzare la confusion matrix
    #plt.figure(figsize=(8, 6))
    #sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    #plt.title("Confusion Matrix")
    #plt.xlabel("Predicted")
    #plt.ylabel("True")
    #plt.show()
    
    # Salviamo l'immagine della confusion matrix in un buffer
    #buf = io.BytesIO()
    #plt.savefig(buf, format='png')
    #buf.seek(0)
    #conf_matrix_image_data = buf.getvalue()
    #buf.close()
    
    
    # Salviamo l'immagine della confusion matrix in un buffer
    buf = io.BytesIO()
    plt.figure(figsize=(8, 6))  # Nuova figura per evitare sovrapposizioni
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.savefig(buf, format='png')  # Salva l'immagine nel buffer
    buf.seek(0)  # Torna all'inizio del buffer
    conf_matrix_image_data = buf.getvalue()  # Ottieni l'immagine in formato binario
    buf.close()  # Chiudi il buffer

    # Mostra la confusion matrix (opzionale)
    #plt.show()
    
    # Salvataggio della configurazione del modello e iper-parametri
    '''COMMENTATO'''
    #model_config = {
        #"model_architecture": str(model),
        #"batch_size_test": test_loader.batch_size,
    #}
    
    '''COMMENTATO'''
    # Dizionario degli iper-parametri
    #hyperparams = {
        #"optimizer": str(optimizer),
        #"loss_function": str(criterion),
        #"learning_rate": optimizer.param_groups[0]['lr'],
    #}

    
    '''COMMENTATO'''
    # Restituisci i risultati come dizionario
    #test_results = {
        #"test_performances": test_performances,  # Aggiungi il dizionario delle performance
        #"confusion_matrix": conf_matrix,
        #"classification_report": class_report,
        #"model_configuration": model_config,
        #"hyperparameters": hyperparams,  # Aggiunti i due nuovi dizionari
        #"confusion_matrix_image": conf_matrix_image_data,  # Aggiunta l'immagine della confusion matrix
    #}
    
    
    # Restituisci i risultati come dizionario
    test_results = {
        "test_performances": test_performances,  # Aggiungi il dizionario delle performance
        "confusion_matrix": conf_matrix,
        "classification_report": class_report,
        "confusion_matrix_image": conf_matrix_image_data,  # Aggiunta l'immagine della confusion matrix
    }
    
        
    return test_results


#### **CREAZIONE CLASSE GRADCAM**

In [ ]:
##### **CREAZIONE CLASSE GRADCAM**

'''
Creazione della classe GradCAM

-----1. Costruttore (init)-----

Cosa fa:

Salva il modello e il layer target (ad esempio, l'ultimo strato convoluzionale) su cui calcolare le mappe di attivazione.

A) Inizializza due variabili, 

1) self.activations e 2) self.gradients, che verranno usate per memorizzare rispettivamente 
1) le attivazioni (feature maps) e 2) i gradienti di quel layer

B) Registra due hook sul target_layer:

1) Forward Hook: Quando il modello effettua la forward pass, viene eseguito save_activation per salvare le attivazioni
2) Backward Hook: Durante la backward pass, save_gradient viene chiamato per salvare i gradienti


-----2. Hook per Salvare Attivazioni e Gradienti-----

B) Save Activation

def save_activation(self, module, input, output):
    self.activations = output.detach()

Cosa fa:

Quando viene eseguita la forward pass sul target_layer, questo hook cattura l'output (le attivazioni) del layer.
Usa detach() per ottenere una copia dei dati senza il tracking dei gradienti, 
in modo da non interferire con la retropropagazione.

C) Save Gradient

def save_gradient(self, module, grad_input, grad_output):
    self.gradients = grad_output[0].detach()


Cosa fa:

Durante la backward pass, questo hook cattura i gradienti che fluiscono attraverso il target_layer.
grad_output è una tupla; solitamente il primo elemento contiene i gradienti utili. 

Anche qui si usa detach() per isolare i dati dai grafi di calcolo.


'''

import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = None
        self.gradients = None
        
        # Registra hook per catturare attivazioni e gradienti
        self.target_layer.register_forward_hook(self.save_activation)
        self.target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output.detach()

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()

#### **FINAL IMPLEMENTATION OF GRADCAM COMPUTATION: FREQUENCIES X ELECTRODES PER EEG STATS**

##### **CONV3D & CONV SEPARABLE**

#### **CONV3D & CONV SEPARABLE: Codice per il Calcolo del GradCAM**


In [ ]:
'''

                                                                        NEW VERSION 17/11/2025
                                                                        
                                                                
                                                                        
                                                                  
                                                                    VERSION FREQUENCY x CHANNELS
                                                                            
                                                                     ****** PER GRID 2D! ******
                                                                      ****** MULTI BAND******
                                                                      
                                                                      PER CONVOLUZIONE 3D (PURA)
                                                                              +
                                                                      PER CONVOLUZIONI SEPARABILI
                                                                      
                                                              
                                                                CON VALORI MEAN GRADCAM e MEAN RAW POWER
                                                                            SU STESSA SCALA
                                                                PER OGNI CLASSE E BANDA DI FREQUENZA!
                                                                
                                                                        ^^^^^SENZA COMMENTI^^^^^
                                                                        ^^^^^            ^^^^^
                                                                        
                                                                    SENZA ADOZIONE DELLA MASCHERA 
                                                                PER INDICARE LE POSIZIONI DELLA GRIGLIA REALI    
                                                            
                                                            1) STIMOLA A VERIFICARE CHE LA RETE DISTINGUA TRA 
                                                        
                                                        COORDINATE ELETTRODICHE REALI VS FITTIZIE (SPAZI VUOTI GRIGLIA)
                                                            
                                                            2) CONFERMA IN MODO DATA-DRIVEN LA RILEVANZA NEUROFISIOLOGICA
                                                            DEL FENOMENO IPOTIZZATO
                                                            

OLTRETUTTO

1) senza mask vedevi hotspot che escono fuori dalla “sagoma” degli elettrodi;

2) con la mask, l’informazione si appiattisce / cambia abbastanza 
→ questo ti dice che il modello 2D sta effettivamente usando anche le celle fittizie / bordi / padding come feature.
Mascherando a posteriori, tu forzi la visualizzazione dentro la sagoma, 
ma non stai più mostrando fedelmente dove il modello guarda.


Quindi la tua intuizione:

“non imporre una maschera aiuta a capire se davvero il modello discrimina tra posizioni reali e non reali”

è giusta al 100%. La mask è solo un filtro di visualizzazione, non cambia il modello: se la applichi 
puoi rendere la mappa “più neuro-plausibile”, ma rischi di nascondere il fatto che
la Separable CNN2D sta facendo cose un po’ spurie nella parte fittizia della griglia.


'''

'''
SINTESI DELLA FUNZIONE check_negative_residuals


| Blocco                          | Scopo                                                                                                                                                                                                                                                                                                                                                                                                                                         | Perché serve                                                                                                                                                                                                                                                                                                                                                                                   |
| ------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **1. eps / “dynamic tol”**      | Calcola una soglia *dinamica* sotto la quale i valori negativi sono quasi certamente puro rumore numerico.<br><br>`python<br>eps32 = np.finfo(np.float32).eps  # ≈ 1.19e‑7<br>dynamic_tol = -eps32 * max(arr.max(), 1.0)`                                                                                                                                                                                                                     | *machine‑epsilon* (ε) è l’errore di arrotondamento massimo relativo per `float32` vicino a 1.<br>Moltiplicandolo per il massimo **positivo** trovato nella mappa definiamo una “fascia di tolleranza” proporzionale alla scala reale del dato. Tutto ciò che cade **sotto** `dynamic_tol` è troppo piccolo perché sia fisico — è, con ottima probabilità, soltanto rumore di rappresentazione. |
| **2. Negativi “significativi”** | Conta e, se ce ne sono, stampa quante celle sono < `dynamic_tol`, quanto è il minimo osservato e la tolleranza stessa.                                                                                                                                                                                                                                                                                                                        | Ti permette di capire a colpo d’occhio se il preprocessing ha generato valori negativi (che non dovrebbero esistere in potenza) che **superano** il rumore ammesso.                                                                                                                                                                                                                            |
| **3. Log‑hint**                 | Se *non* ci sono negativi significativi, calcola il **dynamic range** della mappa (max/min **> 0**) e decide se suggerire la scala log.<br><br>`python<br>positive = arr[arr > 0]<br>if positive.size == 0:<br>    ratio = np.inf          # tutto zero<br>else:<br>    min_pos = positive.min()<br>    max_pos = positive.max()<br>    ratio   = max_pos / max(min_pos, 1e‑12)<br>note = "LOG consigliato" if ratio > 1e3 else "lineare ok"` | – Ignoriamo gli zeri: la scala log non li supporta.<br>– Aggiungiamo un “cuscinetto” di `1e‑12` per evitare div/0.<br>– Se il range è > 10³ (tre ordini di grandezza ≃ “la potenza massima è **> 1000 ×** la minima”), la lettura lineare diventa poco informativa → meglio log10.                                                                                                             |



In sintesi

Prima parte → caccia ai negativi “numeric‑noise” tramite ε.

Seconda parte → valuta solo i valori positivi e suggerisce log‑scale quando il dynamic‑range supera ~3 decadi (≈ 10³).


Quando il codice passa in log‑scale?
Raccogli tutte le mappe (di entrambe le classi e di tutte le bande)


all_mean_pow = np.concatenate([...])

Filtra i positivi e trova vmin_pow (con un 10 % di margine per non “appiattire” il minimo nella color‑bar)

positive_vals = all_mean_pow[all_mean_pow > 0]
vmin_pow = positive_vals.min() * 0.9 if positive_vals.size else 1e‑12
vmax_pow = all_mean_pow.max()

Decidi

use_log = vmax_pow / max(vmin_pow, 1e‑12) > 1e3
Se la potenza massima è > 1000 × la minima positiva, usare LogNorm.

Come si spiega “tre ordini di grandezza”?

“Il massimo è mille volte il minimo”.
“Dynamic‑range di 3 decadi”.
Oppure “max/min > 10³”.



Riassunto finale
ε: misura il rumore di quantizzazione, ti dice quando un (piccolo) negativo è solo un effetto di arrotondamento.

Dynamic‑range: se la banda ha valori reali che variano più di 10³ ×, la scala log10 rende le differenze leggibili senza “schiacciare” i dettagli bassi.

La funzione: un’unica utility per

diagnosticare residui numerici,

suggerire in automatico la rappresentazione (lineare / log) più sensata per i tuoi plot di potenza.

Così il flusso diventa:


check_negative_residuals(...)   # → log “SCALA LOG consigliata”
↓
use_log = True                  # ratio > 1e3
↓
plot con LogNorm + LogLocator   # color‑bar pulita, dettagli visibili




Perché vedi ratio = inf
* Nelle bande Beta e Gamma la tua mappa media è tutta a zero (o comunque tutti i valori ≤ ε).
* Con soli zeri il vettore positive = arr[arr > 0] è vuoto, quindi lo tratto come “dynamic‑range infinito” per evitare la divisione per 0.
* In realtà non hai un’«escursione infinita»; semplicemente non hai segnale in quelle bande → la scala log non aggiungerebbe nulla.

Se vuoi evitare quel suggerimento “falso‑positivo”, basta cambiare la logica così (l’ho già indicato ma lo riscrivo compatto):

positive = arr[arr > 0]
if positive.size < 2:          # 0 o 1 valore positivo → niente dinamica utile
    ratio = 0                  # forza il consiglio a “lineare”
else:
    ratio = positive.max() / max(positive.min(), 1e-12)
    
    
Linear vs log: quale usare davvero?
Banda	Dynamic‑range (≈ max/min)	Scala consigliabile
Delta ‑ Theta	3‑11 ×	Lineare: già leggibile.
Alpha	12 ×	Ancora lineare (o log, ma non cambia molto).
Beta – Gamma	0 (tutti zeri)	Log inutile: non c’è potenza da mostrare.

Di conseguenza:

Mantieni la scala lineare globale come nel tuo blocco finale.

Se in altri dataset vedrai rapporti > 1 000 con almeno 2 valori positivi, allora attiva la parte LogNorm.



'''

def check_negative_residuals(band_names, tensor_dict, tag, log_hint=True):
    
    """
    • band_names   : lista di stringhe  (lunghezza = n_bands)
    • tensor_dict  : {cls: [np.ndarray(H,W), … n_bands]} --> dict  {cls: [np.ndarray(H,W), … 5 bande]}
    • tag          : prefisso stampato nel log --> string visualizzato nel log
    • log_hint     : se True mostra il rapporto max/min ⇒ aiuta a decidere
                     se usare la scala log nei plot.
    """
    
    
    #https://numpy.org/doc/2.1/reference/generated/numpy.finfo.html
    eps32 = np.finfo(np.float32).eps        # 1.19e‑7
    for cls in [0, 1]:
        for b, b_name in enumerate(band_names):
            arr   = tensor_dict[cls][b]
            
            # Soglia dinamica = −eps * valore_massimo_della_mappa
            
            #dynamic_tol = -eps32 * arr.max()      # tolleranza dinamica
            
            dynamic_tol = -eps32 * max(arr.max(), 1.0)   # evita max==0
            
            neg_mask  = arr < dynamic_tol # “negativi significativi”
            
            min_v, max_v = float(arr.min()), float(arr.max())
            
            if np.any(neg_mask):
                print(f"Valori sotto la soglia per classe e banda:\n")
                
                #forma	cosa fa	quando differisce
                #np.count_nonzero(neg_mask)	converte il bool‑array in int (True→1, False→0) e somma gli 1	è sempre un intero Python
                
                #neg_mask.sum()	chiama il metodo .sum() dell’ndarray; 
                #per tipo bool fa esattamente la stessa somma di sopra	restituisce uno numpy.int_ (stesso valore, differente tipo)
                
                n_neg   = np.count_nonzero(neg_mask)
                
                min_val = arr.min()
                
                print(f"[{tag} {b_name}] class={cls}  band={b_name:<6}  "
                      f"neg={n_neg}  min={min_val:.3e}  tol={dynamic_tol:.3e}")
            else:
                print(f"Nessun valore sotto la soglia per classe {cls} e banda {b_name}\n")
                print(f"Definisco il range minimo e massimo per classe {cls} e banda{b_name} :\n")
                
                # ‑‑ opzionale: suggerimento scala log
                if log_hint:                    
                    #if max_v == 0 or min_v == 0:
                    positive = arr[arr > 0]           # considera solo i valori > 0
                    
                    #Se il primo valore della potenza è proprio 0:
                    #per evitare problemi di NaN lo impongo ad infinito
                    if positive.size == 0:
                        ratio = float('inf')          # tutto zero → range “infinito”
                    
                    #Se il primo valore della potenza è proprio 0:
                    #per evitare problemi di NaN lo impongo ad infinito
                    else:
                        min_pos = positive.min()
                        max_pos = positive.max()
                        ratio   = max_pos / max(min_pos, 1e-12)
                    note = f"\033[1mSCALA LOG per plots consigliata\033[0m" if ratio > 1e3 else f"\033[1mSCALA LINEARE per plots consigliata ok\033[0m"
                    print(f"        dynamic‑range ≈ {ratio:8.1f}  → {note}")
            print()
                
                
                
import torch.nn as nn

def model_has_cudnn_rnn(model):
    """Ritorna True se il modello usa LSTM/GRU/RNN supportati da CuDNN."""
    return any(isinstance(m, (nn.LSTM, nn.GRU, nn.RNN)) for m in model.modules())


from matplotlib.ticker import FixedLocator

from matplotlib.colors import LogNorm
from matplotlib.ticker import (LogLocator, LogFormatterMathtext,
                               ScalarFormatter)


'''RICORDATI: aggiunto parametro TEST_LOADER_RAW per i plots della POTENZA SPETTRALE MEDIA PER BANDA (i.e., test_loader_raw)'''
def compute_gradcam_figure(model, test_loader, test_loader_raw, exp_cond, data_type, category_subject, device, channel_names=None, debug = False):
    
    
    '''SOLO PER I MODELLI OTTIMIZZATI CON ANCHE LA LSTM'''
    
    #Solo i modelli con LSTM entrano in questo giro; gli altri non cambiano di stato.
    #Con questa sequenza:
    #non ottieni più l’errore “cudnn RNN backward…”;
    #la rete “si comporta” come in eval (Dropout off, BN congelato) mentre calcoli le CAM;
    #l’ambiente di chiamata (il tuo loop di testing) riceve il modello esattamente nello stato in cui l’aveva passato alla funzione compute_gradcam_figure
    

    ### Perché serve model.train() anche se la CAM è presa prima della LSTM
    
    #Il backward, per arrivare dal loss (o dal logit scelto) fino al tuo layer conv3, deve comunque attraversare l’LSTM che sta più avanti nella rete.
    #Le implementazioni CuDNN degli RNN (LSTM/GRU) alzano un’eccezione se provi a chiamare tensor.backward() mentre il modulo è in modalità eval().
    #RuntimeError: cudnn RNN backward can only be called in training mode
    #Quindi, anche se la CAM è calcolata su conv3, devi mettere l’intero modello in train() per il tempo del backward.
    #condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    
    ### Che cos’è model.training
    
    #model.training è un semplice flag booleano (impostato da nn.Module.train() / nn.Module.eval()), ereditato da tutti i sotto‑moduli.
    #Con was_training = model.training ricordi in che stato era il modello (quasi sempre False, cioè eval, nel tuo flusso)
    #per poterlo ripristinare dopo.
    
    #Facendo così
    
    #for m in model.modules():
    #if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d,
                      #nn.Dropout, nn.Dropout2d, nn.Dropout3d)):
        #if m.training:         # cioè erano in train
            #m.eval()
            #frozen_layers.append(m)
    
    #Li sposti in eval uno per uno, senza toccare il resto della rete che deve restare in train() per far funzionare CuDNN‑RNN.
    
    
    ### Perché, a fine blocco, servono due ripristini
    
    #1) Riattivo i BatchNorm / Dropout che avevo forzato in eval:
    
    #for m in frozen_layers:
        #m.train()              # torna come prima
    
    #2) Riporto l’intero modello nello stato in cui si trovava prima del Grad‑CAM:
    
    #model.train(was_training)  # se era eval() torna eval, altrimenti resta train
    
    #Se non facessi il punto 1, lasceresti quei moduli permanentemente in eval anche quando, più tardi, 
    #rientri in training (per esempio in un fine‑tuning).
    #Se non facessi il punto 2, lasceresti tutto il modello in train → dropout attivo, BN che accumula statistiche, ecc.

    
    
    # ------------------------------------------------------------ ------------------------------------------------------------
    # ❶ — se serve, abilito temporaneamente la modalità train per il modello ottimizzato che aveva ANCHE la LSTM... 
    # ------------------------------------------------------------ ------------------------------------------------------------
    
    needs_train_mode = model_has_cudnn_rnn(model)
    
    if needs_train_mode:
        was_training = model.training      # salvo lo stato
        model.train()                      # abilito backward su CuDNN‑RNN
        
        # ➊ salvo lo stato di OGNI BN/Dropout
        
        saved = [(m, m.training) for m in model.modules()
             if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d,
                               nn.Dropout, nn.Dropout2d, nn.Dropout3d))]
        
        model.train()                              # abilita backward su CuDNN‑RNN
        
        # ➋ congelo in ogni layer della rete gli strati di BatchNorm e Dropout
        for m, _ in saved:
            m.eval()
    
    # ------------------------------------------------------------
    # ❷ — QUI sotto metti tutto il tuo codice Grad‑CAM
    #      (forward, backward, costruzione delle mappe, plot, …)
    # ------------------------------------------------------------

    # … il tuo lunghissimo corpo della funzione rimane invariato …
    # → al momento di fare backward NON avrà più l’eccezione
    #   “cudnn RNN backward can only be called in training mode”

    
    '''SE VUOI USARE STESSO LAYER PER GRADCAM IN ENTRAMBE ARCHITETTURE'''
    target_layer = model.conv2b #model.conv3
    
    
    '''SE VUOI USARE DIVERSI LAYER PER GRADCAM NELLE 2 ARCHITETTURE'''
    #if isinstance(model, CNN3D_LSTM_FC):
        #target_layer = model.conv2b
        
    #elif isinstance(model, SeparableCNN2D_LSTM_FC):
        #target_layer = model.pw_conv1  # feature map 9x9, allineata 1:1 alla griglia
    
    gradcam = GradCAM(model, target_layer)
    
    # Determina il target layer in base al tipo di modello
    #if isinstance(model, SeparableCNN2D_LSTM_FC):
        #target_layer = model.dw_conv1  # Per il modello separabile 2D
    #else:
        #target_layer = model.conv3  # Per il modello CNN3D
        

    condition_names = exp_cond.split("_vs_") if "_vs_" in exp_cond else ["Class 0", "Class 1"]
    
    
    # ✅ Raccogli TUTTI i campioni per ciascuna classe
    # Itera sul test_loader fino a trovare almeno un esempio per ciascuna classe (0 e 1)
    
    
    #PER IL CASO CONV 3D
    
    # Ogni mio sample è 3D, perché infatti è fatto per convoluzione 3d pura o convoluzioni separabili, 
    # Quindi ha shape  (B, C, D, H, W), dove:  
    
    #B = batch (in questo caso, per ogni singolo esempio quindi sarà 1 -> singolo esempio alla volta)
    #C = feature maps/canali (numero di feature maps estratte dalla convoluzione, o meglio anche noti come  canali convoluzionali)
    #D = depth (la dimensione di profondità del mio tensore --> 5 ossia, la potenza spettrale ad ogni banda di frequenza - i.e.,  delta, theta, alfa, beta e gamma)
    #H = height (altezza, prima dimensione SPAZIALE del mio tensore i.e., altezza griglia, canali EEG) 
    #W = width (larghezza, seconda dimensione SPAZIALE del mio tensore i.e., larghezza griglia, canali EEG)
    
    
    '''SHAPE DEI DATI ORIGINALE SAREBBE (B, 9, 9, 5)'''
    samples = {0: [], 1: []}
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples:  # Assumendo solo classi 0 e 1
                
                '''OSSIA QUI DIVENTA (1, 9, 9, 5)'''
                samples[label_int].append(inputs[i].unsqueeze(0))
                
    
    '''TEST_LOADER RAW (B, 9, 9, 5)'''
    samples_raw = {0: [], 1: []}
    for inputs, labels in test_loader_raw:
        inputs, labels = inputs.to(device), labels.to(device)
        for i, label in enumerate(labels):
            label_int = int(label.item())
            if label_int in samples_raw:  # Assumendo solo classi 0 e 1
                
                '''OSSIA QUI DIVENTA (1, 9, 9, 5)'''
                samples_raw[label_int].append(inputs[i].unsqueeze(0))
                
    # ============================================================
    # Calcolo delle Grad-CAM per ogni singola banda di frequenza
    # ============================================================
    
    n_bands = 5  # numero di canali/bande di frequenza
    band_names = ['Delta (δ)', 'Theta (Θ)', 'Alpha (α)', 'Beta (β) ', 'Gamma (γ)']  
    
    
    '''STRUTTURE DATI PER CONV3D'''
    #✅ Struttura per il GradCAM 3D, ossia qui raccolgo il GradCAM 3D di ogni esempio
    # quindi la mappa di attivazione che identifica l'attivazioni più rilevanti per la classificazione di una esemplare di una certa classe
    # sia spazialmente (height and width, ossia le dimensioni spaziali del mio tensore)
    # sia frequenzialmente (depth), ossia le attivazioni più rilevanti in base alla banda di frequenza
    
    global_cams_3d = {0: [], 1: []} # shape (D, 9, 9)
    
    #Poi qui abbiamo: 

    # ✅ Struttura: classe → banda → immagini raw di input filtrate per singola banda (senza passare dal modello)
    #tutte le mappe di potenza per la classe cls nella banda b-esima
    
    #La struttura per salvare invece lo potenza spettrale media per ogni relativa banda 
    raw_power_per_band_3d = {0: [[] for _ in range(n_bands)], 1: [[] for _ in range(n_bands)]}
    
    #La struttura che salverà la "fetta" del gradcam3D, ossia dove plotto solo la fetta della banda a partire dal global_cam_3d
    #Ossia per ogni esempio di una specifica classe, prenderò la mappa di attivazione spazialmente più rilevante, in base alla specifica banda di frequenza indagata

    #✅ Struttura: classe → banda → lista CAM
    #cams_per_band_3d[cls][banda]: la slice D-esima (ossia la slice frequenziale) della mappa GradCAM per ogni campione di classe cls.
    
    cams_per_band_3d = {0: [[] for _ in range(n_bands)], 1: [[] for _ in range(n_bands)]}
    
    '''STRUTTURE DATI PER CONV SEPARABLE'''

    
    global_cams_2d = {0: [], 1: []} 
    
    raw_power_per_band_2d = {0: [[] for _ in range(n_bands)], 1: [[] for _ in range(n_bands)]}
    
    cams_per_band_2d = {0: [[] for _ in range(n_bands)], 1: [[] for _ in range(n_bands)]}
    
    '''
    
    Da qui in giù, vado a eseguire i passaggi essenziali per:
    
    1) Calcolare il GradCAM 3D per ogni esempio,
    2) Isolare le fette (slice) per banda,
    3) Raccogliere le immagini di potenza raw per ogni banda,
    4) Calcolare le medie per classe e banda.
    
    '''


    for cls in [0, 1]:
        
        for sample_input, sample_input_raw in zip(samples[cls], samples_raw[cls]):
            
            # 1) Preparo il sample per il calcolo
            sample_input = sample_input.clone().detach().requires_grad_(True)
            
            '''
            
            Devo far squeeze perché essendo prelevati dal test loader, li avevo già resi con un 1 davanti ossia qui sopra

            samples[label_int].append(inputs[i].unsqueeze(0))
            
            perché io, è vero che prendo i dati dal test_loader che son in formato batch (batch, D, H, W)
            ma siccome poi prelevo ogni singolo esempio per metterli dentro a "samples", allora quella dimensione di batch si leva nel dizionario "samples".
            
            Per cui, quando voglio prendermi il singolo esempio per salvarmelo dentro a "raw_vol" (ossia la potenza spettrale del volume proprio di ogni esempio),
            allora devo ri-assegnare ad ogni esempio la dimensione del batch (ossia 1, ossia il singolo esempio)
            quando li salvo dentro samples. 
            
            Ed infatti, quindi quell' ".unsqueeze(0)" --> samples[label_int].append(inputs[i].unsqueeze(0)) serve proprio a questo...

            Quando poi, devo prendere il singolo esempio da salvare, nei termini di potenza spettrale di ogni esempio (come volume 3D), 
            allora devo rifare .squeeze(), per togliere nuovamente la dimensione del batch (ossia 1, ossia il singolo esempio)
            perché la rappresentazione del singolo esempio è, appunto, composta da, le sole dimensioni che costituiscono proprio il singolo esempio,
            ossia 9x9x5 -->  ossia la griglia 3D !

            E quindi, per ogni esempio, mi salvo la griglia 3d, ossia
            "raw_vol = sample_input.detach().cpu().numpy().squeeze()   # → (9, 9, 5)"
            
            E poi, la divido però per banda 
            
            "for b in range(n_bands):
                raw_power_per_band_3d[cls][b].append(raw_vol[:, :, b])
            "
            '''
            
            # 2) Subito qui prendo la potenza raw del volume 9×9×5, senza passare dal modello!
            #raw_vol[:, :, b] è una mappa 2D (9, 9) della potenza spettrale per la banda b per ogni singolo esempio
            
            raw_vol = sample_input_raw.detach().cpu().numpy().squeeze()     # → (9, 9, 5)
            
            '''CASTING IN FLOAT64'''
            #raw_vol = sample_input.detach().cpu().numpy().squeeze().astype(np.float64)         # 🔹 cast a float64   # → (9, 9, 5)
            
            for b in range(n_bands):
                
                
                # Determina in base al tipo di modello i dati e le shape da salvare:
                
                #if isinstance(model, CNN3D_LSTM_FC):
                #if isinstance(model, SeparableCNN2D_LSTM_FC):
                
                    #target_layer = model.dw_conv1  # Per il modello separabile 2D
                #else:
                    #target_layer = model.conv3  # Per il modello CNN3D
                    
                if isinstance(model, CNN3D_LSTM_FC):
                    
                    #raw_power_per_band_3d invece raccoglie TUTTE le mappe 2D (9, 9) di ogni singolo esempio che conterrà la potenza spettrale alla stessa banda b 
                    #Lista di mappe di potenza 2D (una per trial) per la relativa banda

                    raw_power_per_band_3d[cls][b].append(raw_vol[:, :, b])
                    
                elif isinstance(model, SeparableCNN2D_LSTM_FC):
                    raw_power_per_band_2d[cls][b].append(raw_vol[:,:,b])

            
            # 2) Esegui il forward pass
            output = model(sample_input)
            target_class = output.argmax(dim=1).item()
            
            # 3) Esegui il backward pass
            model.zero_grad()
            target = output[0, target_class]
            target.backward()
            
            # 4) Preleva attivazioni e gradienti
            activ = gradcam.activations   # shape può essere 5D (B,C,D,H,W) per CNN3D o 4D (B, C, H, W) per CNN Separable
            grads = gradcam.gradients # shape può essere 5D (B,C,D,H,W) per CNN3D o 4D (B, C, H, W) per CNN Separable
            
            #Nel caso 3D dovrebbe essere

            #Media dei gradienti solo su H,W → (B,C,D,1,1)
            #w3d = torch.mean(grads, dim=(3, 4))

            # b) Sommo sui canali → (B,D,H,W)
            #cam3d = F.relu(torch.sum(w3d * activ, dim=1))
            
            #e così la shape finale sarebbe 3D con (B,C,D)
            
            #✔️ w3d è correttamente calcolato per ogni (B, C, D, 1, 1)
            #✔️ La somma su dim=1 aggrega le feature maps con pesi per ogni banda
            #✔️ ReLU rimuove componenti negative
            
            '''
            Nel caso della CNN3D calcoli una mappa Grad-CAM 3D "globale" direttamente dal layer convoluzionale 3D, ottenendo attivazioni di shape 
            (B,C,D,H,W) e quindi una CAM volumetrica per ogni trial
            
            CNN3D → calcoli una CAM 3D da attivazioni (B,C,D,H,W), una volta sola per ogni esempio.

            '''
            
            if activ.ndim == 5:  # Caso per modello CNN3D pura 
                
                # 3D Volumetric Grad-CAM
                
                # a) Media dei gradienti solo su H,W → (B,C,D,1,1)
                w3d = torch.mean(grads, dim=(3, 4), keepdim=True)

                # b) Sommo sui canali (feature maps) → (B,D,H,W)
                cam3d = F.relu(torch.sum(w3d * activ, dim=1))
                
                # c) Upsample H×W, mantenendo D intatto
                B, D, H, W = cam3d.shape
                cam_flat = cam3d.view(B*D, 1, H, W)
                cam_up   = F.interpolate(cam_flat,
                                         size=(9, 9),
                                         mode='bilinear',
                                         align_corners=False)
                
                cam_vol  = cam_up.view(B, D, 9, 9).cpu().numpy()
                
                # d) Prendi ogni batch-item
                
                '''
                Quindi qui ottengo che:
            
                1) appendo a global_cams_3d che cosa qui? il gradcam 3D ossia la mappa di attivazione di volume,
                ossia OGNI esempio (volumetrico) per ogni classe (ossia 9x9x5 ancora, di OGNI esempio)
                
                Quindi semplicemente anziché rendere il dato come 'batch, D, H, W'.. siccome prendiamo ogni esempio UNO ALLA VOLTA
                è inutile mantenere la dimensione batch (che sarebbe sempre 1, perché parliamo di ogni esempio, uno alla volta)
                
                ossia anziché fare 
                
                global_cams_3d[cls].append(cam_vol)
                
                faccio
                
                global_cams_3d[cls].append(cam_vol[0])
                
                
                E quindi, mi salvo per OGNI esempio direttamente la mappa cam 3d, per ogni banda, direttamente
                ossia ogni esempio sarà costituito da 3 dimensioni (D, H, W) anziché dire 
                
                "Ogni dato (ossia ogni esempio) è composto da (batch, D, H, W) 
                se tanto il batch = 1 (perché il batch è il singolo esempio ogni volta)
                
                e quindi significherebbe aggiungere una dimensione (quella del batch) che in realtà è inutile, 
                perché si riferisce all'esempio stesso di già!
                
                Quindi:
                👉 cam_vol[0] estrae la CAM 3D senza la dimensione "batch", che è inutile in quel contesto
                👉 Serve per poter fare medie e slicing banda per banda correttamente dopo lo stack
                👉 Questo rende compatibile il risultato finale con imshow (che accetta solo 2D o 3D RGB)
       
                2) appendo anche l'esempio volumetrico a cams_per_band_3d, MA GIA' suddiviso per banda! (per cui diventa 2d là dentro! 9x9)

                '''
                
                global_cams_3d[cls].append(cam_vol[0])
                
                for b in range(n_bands):
                    cams_per_band_3d[cls][b].append(cam_vol[0,b])
            
                '''
                Nel caso della SeparableCNN2D, invece, il layer convoluzionale è 2D e riceve in input 
                (B,5,9,9), cioè con le bande di frequenza come canali (non come profondità). 

                Questo significa che non puoi ottenere direttamente una CAM 3D nello stesso modo, 
                ma si può ottenere una CAM 2D per ogni banda, 
                "mascherando" l’input attivando una banda alla volta

                SeparableCNN2D → non hai accesso diretto a una "profondità" come in GradCAM 3D, quindi:

                Simuli la profondità attivando una banda alla volta.

                Ottieni una CAM 2D per ogni slice (banda), iterando sulle bande.

                Questo approccio ti consente di costruire comunque strutture 3D:
                cams_per_band_2d[cls][b] con b = 0...4 contiene 
                le CAM 2D relative alla banda b, ricostruendo idealmente la distribuzione tridimensionale
                '''
            
            elif activ.ndim == 4: #Caso per il modello Conv Separabili 
                
                # 1) Preparo il sample per il calcolo
                sample_input = sample_input.clone().detach().requires_grad_(True) # sample_input: (1, 9, 9, 5)
                
                
                '''
                il mio sample input ora è sempre 4D, ma a differenza di prima, io sto trattando in questo caso 
                le bande come CANALI, e non come DEPTH (della convoluzione 3d pura!)
                
                Come prima, devo togliere la dimensione del batch ...
                '''
                
                
                # 2) Subito qui prendo la potenza raw del volume 9×9×5, senza passare dal modello!
                #raw_vol[:, :, b] è una mappa 2D (9, 9) della potenza spettrale per la banda b per ogni singolo esempio
                
                raw_vol = sample_input_raw.detach().cpu().numpy().squeeze()     # → (9, 9, 5)
                
                '''CASTING IN FLOAT64'''
                #raw_vol = sample_input.detach().cpu().numpy().squeeze().astype(np.float64)         # 🔹 cast a float64   # → (9, 9, 5)
                
                
                '''
                cams_per_band_2d e raw_power_per_band_2d son dentro al loop di MASKING, 
                perché entrambe si riferiscono alla banda e quindi devo essere inserite dentro al loop di masking...
                
                --> raw_power_per_band_2d[cls][b] e cams_per_band_2d[cls][b] sono dentro il loop (for b in ...) 


                
                mentre global_cams_2d è FUORI da quel loop, perché raccoglie tutti gli esempi delle gradcam, 
                considerando però le mappe di attivazione di ogni banda singolarmente 
                e le aggrega per avere una visualizzazione dell'impatto complessivo di ogni singola banda sulla decisione del modello,
                facendo vedere dove son maggiormente concentrate le attivazioni a livello spaziale TRA le bande ( = considerando TUTTE le bande assieme!)
                
                --> global_cams_2d[cls] sta dopo quel for b, raccogliendo una sola mappa 2D “complessiva” per trial
                
                In questo modo:

                raw_power_per_band_2d e cams_per_band_2d catturano tutti i trial per banda.

                global_cams_2d cattura un’unica mappa per trial, che poi aggregherò in global_mean_cams_2d per ottenere la heatmap 2D “globale”
                che comprende tutte le bande insieme

                '''
                
                
                per_band_cams = []  # 👈 CAM di questo trial, una per banda
                
                
                for b in range(n_bands):
                    
                    #raw_power_per_band_2d invece raccoglie TUTTE le mappe 2D (9, 9) di ogni singolo esempio che conterrà la potenza spettrale alla stessa banda b 
                    #Lista di mappe di potenza 2D (una per trial) per la relativa banda
                    raw_power_per_band_2d[cls][b].append(raw_vol[:, :, b])
                    
                    # ✅ Creo un input mascherato con **solo** la banda b attiva
                    masked = np.zeros_like(raw_vol)  # (9, 9, 5)
                    
                    masked[:, :, b] = raw_vol[:, :, b]  # attiva solo la banda b
                    
                    #Qui lo prepari in formato 4D, come vorrebbe il modello Conv Separable
                    masked_tensor = torch.tensor(masked).unsqueeze(0).to(device)  # (1, 9, 9, 5)
                    
                    # Preparo il sample
                    masked_tensor.requires_grad_(True)
                    
                    # Forward + backward
                    output = model(masked_tensor)
                    target_class = output.argmax(dim=1).item()
                    model.zero_grad()
                    
                    target = output[0, target_class]
                    target.backward()

                    activ = gradcam.activations   # (B, C, H, W)
                    grads = gradcam.gradients     # (B, C, H, W)
                    
                    # Calcolo CAM 2D (come standard GradCAM)
                    w2d = torch.mean(grads, dim=(2, 3), keepdim=True)  # (B, C, 1, 1)
                    cam = F.relu(torch.sum(w2d * activ, dim=1))        # (B, H, W) --> # (1, H, W)
                    
                    # ---- stesso nome in entrambi i casi ----
                    
                    B, H, W = cam.shape
                    
                    '''
                    NEL CASO LAYER SIA DIVERSO DA pw_conv1, ALLORA FACCIO UPSAMPLING
                    Cosa cambia in pratica:

                    Se il target_layer è pw_conv1 (feature map 32×9×9):

                    H, W = 9, 9 → salta l’interpolate, quindi nessuna distorsione spaziale.

                    Se un domani cambi target_layer a un layer dopo un pool (es. conv2b con mappe 4×4):

                    H, W != 9 → scatta l’upsampling e tutto continua a funzionare come prima.

                    Il ramo 3D (activ.ndim == 5) non lo tocchi, lì l’upsampling è ancora necessario (4×4 → 9×9).

                    Con questa modifica il caso SeparableCNN2D è “pulito” e allineato alla griglia elettrodica 1:1
                    '''
                    
                    if (H, W) != (9, 9):
                    
                        #Riporto la shape con .unsqueeze(1) a 4D per fare interpolation e alla fine di nuovo in 3d
                        cam = F.interpolate(cam.unsqueeze(1), size=(9, 9), mode='bilinear', align_corners=False).squeeze(1)
                    
                    
                    # caso pw_conv1: la mappa è già 9x9 → nessun upsampling
                    
                    #Riporto la shape con .squeeze(1) a 3D per salvare i dati
                    #cam_2d = cam_up.squeeze(1).cpu().numpy()  # (B, 9, 9)
                    
                    cam_2d = cam.detach().cpu().numpy()      # (B,9,9)

                    # ✅ Aggiungo la mappa CAM alla banda corrispondente
                    cams_per_band_2d[cls][b].append(cam_2d[0])  # prende il CAM per il sample corrente (9x9)
                    
                    # Conteggio della CAM corrente, per questo trial
                    per_band_cams.append(cam_2d[0])                   
                    
                #Qui “ricompatti” i 5 CAM 2D in un volume e poi medii, per ottenere una mappa 2D complessiva che tenga insieme l’informazione su tutte le bande.
                #Durante il loop, dopo il masking e il calcolo di cam_2d, fai:
                # cam_2d ha shape (1,9,9) → [0] è la matrice 9×9
                
                # 👇 qui, fuori dal for b
                
                all_cams_2d_per_band = np.mean(np.stack(per_band_cams), axis=0)  # (9,9)
                
                global_cams_2d[cls].append(all_cams_2d_per_band)
            else:
                raise RuntimeError(f"activ.ndim inatteso: {activ.ndim}")
                
                
    '''
    CASO MODELLO CON 3D PURO, dovrei fare: 

    1) per global_cams_3d vado ad ottenere una media, ossia una global_mean_cams_3d, che riassume il contributo GLOBALE della gradcam 3D aggregata
    all'interno dell'intero volume 3D (che poi al massimo si può scorporare vedendo per ogni banda successivamante)

    '''
    
    if isinstance(model, CNN3D_LSTM_FC):
        
        #1) global_cams_3d → media “globale” del volume 3D Grad‑CAM
        # media sul numero di esempi per ogni classe → ottieni un array (D, H, W)
        global_mean_cams_3d = {
            cls: np.mean(np.stack(global_cams_3d[cls]), axis=0)  # da [ (1,D,H,W), … ] a (D,H,W)
            for cls in [0,1]
        }
        
        # ➜ CAM 2D globale per classe (media su TUTTE le bande dal VOLUME 3D)
        global_2d_from3d = {
            cls: np.mean(global_mean_cams_3d[cls], axis=0)  # (9,9)
            for cls in [0, 1]
        }
        
        


        '''
        2) poi, dentro a raw_power_per_band_3d (siccome è già suddivsa ogni potenza spettrale in 2D di ogni esempio, per ogni classe e per ogni banda !)
        ottenere una media sulla potenza spettrale dei gradcam di ogni trial della relativa classe, sulla relativa banda... (i.e., mean_raw_power_per_band_3d)
        '''

        #2) raw_power_per_band_3d → media della potenza raw per banda
        #Hai già raccolto, per ogni cls e per ogni banda b, tutte le mappe 2D raw_power_per_band_3d[cls][b] (una per trial).
        #La media diventa:

        mean_raw_power_per_band_3d = {
            cls: [ np.mean(np.stack(raw_power_per_band_3d[cls][b]), axis=0)
                   for b in range(n_bands) ]
            for cls in [0,1]
        }

        # risultato: mean_raw_power_per_band_3d[cls][b] è (H,W)

        '''
        3) dentro a cams_per_band_3d, (siccome è già suddivsa ogni gradcam in 2D di ogni esempio, per ogni classe e per ogni banda !) 
        ottenere una media sulle gradcam di ogni trial della relativa classe, sulla relativa banda... (i.e., mean_cam_3d_per_band) 

        '''

        #3) cams_per_band_3d → media della Grad‑CAM per banda
        #Analogamente hai raccolto tutte le slice 2D di Grad‑CAM in cams_per_band_3d[cls][b].
        #La media diventa:

        mean_cams_per_band_3d = {
            cls: [ np.mean(np.stack(cams_per_band_3d[cls][b]), axis=0)
                   for b in range(n_bands) ]
            for cls in [0,1]
        }
        # mean_cam_3d_per_band[cls][b] ha shape (H,W)


        '''
        Con queste tre strutture (global_mean_cams_3d, mean_raw_power_per_band_3d, mean_cam_3d_per_band) puoi

        riga 1–2: istogrammi di mean_cam_3d_per_band[cls][b]

        riga 3–4: heatmap di mean_cam_3d_per_band[cls][b]

        riga 5–6: heatmap di mean_raw_power_per_band_3d[cls][b]

        riga 7–8: slice di global_mean_cams_3d[cls] per ogni b

        '''
    
    elif isinstance(model, SeparableCNN2D_LSTM_FC):
    
        '''
        CASO MODELLO CONV SEPARABLE, dovrei fare: 

        1) per global_cams_2d vado ad ottenere una media, ossia una global_mean_cams_2d, che riassume il contributo GLOBALE della gradcam 2D aggregata
        all'interno di TUTTE LE BANDE ASSIME (che mi dovrebbe dare quindi per OGNI CLASSE un plot unico, 
        e non come il global_mean_cams_3d, dove dovrei vedere in quel caso, invece, la stessa mappa di “rilevanza complessiva”, MA distribuita lungo la profondità,
        ossia tra le bande e quindi potrei vedere se effettivamente io abbia una banda che è specificatamente più attiva di altre COMPLESSIVAMENTE...

        Per la SeparableCNN2D ricostruisci un Grad‑CAM “3D” artificiale facendo 5 Grad‑CAM 2D una per ogni banda
        '''

        #global_cams_2d
        #Qui “ricompatti” i 5 CAM 2D in un volume e poi medii, per ottenere una mappa 2D complessiva che tenga insieme l’informazione su tutte le bande.
        #Durante il loop, dopo il masking e il calcolo di cam_2d, fai:

        # cam_2d ha shape (1,9,9) → [0] è la matrice 9×9
        #global_cams_2d[cls].append(cam_2d[0])

        global_mean_cams_2d = {
            cls: np.mean(np.stack(global_cams_2d[cls]), axis=0)
            for cls in [0,1]
        }
        # global_mean_cams_2d[cls] shape = (9,9)

        '''
        2) poi, dentro a raw_power_per_band_2d (siccome è già suddivsa ogni potenza spettrale in 2D di ogni esempio, per ogni classe e per ogni banda !)
        ottenere una media sulla potenza spettrale dei gradcam di ogni trial della relativa classe, sulla relativa banda... (i.e., mean_raw_power_per_band_2d)
        '''

        #raw_power_per_band_2d
        #Hai già in raw_power_per_band_2d[cls][b] tutte le mappe 2D di potenza (9×9) per trial, per ciascuna banda b.
        #La media finale:


        mean_raw_power_per_band_2d = {
            cls: [ np.mean(np.stack(raw_power_per_band_2d[cls][b]), axis=0)
                  for b in range(n_bands) ]
            for cls in [0,1]
        }

        # mean_raw_power_per_band_2d[cls][b] shape = (9,9)


        '''
        3) dentro a cams_per_band_2d, (siccome è già suddivsa ogni gradcam in 2D di ogni esempio, per ogni classe e per ogni banda !) 
        ottenere una media sulle gradcam di ogni trial della relativa classe, sulla relativa banda... (i.e., mean_cam_2d_per_band) 

        '''

        #cams_per_band_2d
        #Durante il masking loop appendi in cams_per_band_2d[cls][b] il CAM 2D (9×9) di ogni trial.
        #La media finale:

        mean_cams_per_band_2d = {
            cls: [ np.mean(np.stack(cams_per_band_2d[cls][b]), axis=0)
                   for b in range(n_bands) ]
            for cls in [0,1]
        }
        # mean_cam_2d_per_band[cls][b] shape = (9,9)

        '''
        Con queste tre strutture —

        mean_raw_power_per_band_2d (5 mappe 9×9),

        mean_cam_2d_per_band (5 mappe 9×9),

        global_mean_cams_2d (1 mappa 9×9) —

        puoi costruire esattamente le stesse righe di plot che avevi per il caso 3D, solo che al posto di “slice” del volume userai le CAM 2D mascherate.


        '''
    
    # Preleva la struttura corretta in base al modello
    
    if isinstance(model, SeparableCNN2D_LSTM_FC):
        mean_cams_per_band = mean_cams_per_band_2d
        mean_raw_power_per_band = mean_raw_power_per_band_2d
        global_mean_cams = global_mean_cams_2d
    
    else:  # Caso per modello CNN3D
        mean_cams_per_band = mean_cams_per_band_3d
        mean_raw_power_per_band = mean_raw_power_per_band_3d
        
        #global_mean_cams = global_mean_cams_3d
        
        
        
        '''NUOVA MODIFICA''' 
        global_mean_cams = global_2d_from3d
        
    
    
    
    
    
    # prima di salvare la figura, solo se richiesto vedi i valori delle potenze medie per banda e condizione sperimentale...
    
    #"tag_names=[f"{model.__class__.__name__} power"]")
    
    if debug:
        if isinstance(model, CNN3D_LSTM_FC):
            model_tag = f"{model.__class__.__name__} power"
            check_negative_residuals(band_names,
                                     mean_raw_power_per_band_3d,
                                     model_tag)
        else:
            model_tag = f"{model.__class__.__name__} power"
            check_negative_residuals(band_names,
                                     mean_raw_power_per_band_2d,
                                     model_tag)
            
    
    
    
    # Crea la figura dinamicamente in base al modello
    if isinstance(model, SeparableCNN2D_LSTM_FC):
        fig, axs = plt.subplots(8, 5, figsize=(24, 30))  # 2 righe per 5 colonne per modello 2D
    else:
        fig, axs = plt.subplots(8, 5, figsize=(24, 30))  # 5 righe per 2 colonne per modello 3D

    
    
            
    title = (
        f"Grad-CAM Mapping over EEG Trials – Experimental Conditions: {exp_cond}\n\n"
        "Row 1-2: Histogram of Mean Grad-CAM raw values for each class and frequency band\n"
        "Row 3-4: Normalized Mean Grad-CAM heatmaps for Class 0 (top) and Class 1 (bottom)\n"
        "Row 5-6: Log Mean Raw power maps for Class 0 (top) and Class 1 (bottom)\n"
        "Row 7-8: Global CAM per Class"
        )
    
    plt.suptitle(title, fontsize=15)

    # Spaziatura verticale per evitare sovrapposizione
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    plt.subplots_adjust(hspace = 0.7, wspace = 0.4)  # Fine tuning della spaziatura tra subplot
    
    #PER PLOT RIGA 1-2 
    
    from matplotlib.ticker import ScalarFormatter, MaxNLocator

    # Crea un formatter per notazione scientifica
    sci_formatter = ScalarFormatter(useMathText=True)
    
    #Questa chiamata serve a forzare il range entro cui usare la notazione scientifica
    #Se non metti questo limite, il comportamento può variare leggermente a seconda della scala dei dati —
    #a volte sarà decimale (0.0001), altre volte esponenziale (1e-4), e potrebbe non essere uniforme tra subplot.
    
    sci_formatter.set_powerlimits((-3, 3))  # usa 1e-xxx se valori sono piccoli

    for b, b_name in enumerate(band_names):
    
        for j, cls in enumerate([0, 1]):
            
            # Calcola l'istogramma dei valori della heatmap media
            # rispetto alle 2 classi in base alla banda di frequenza isolata
            
            ax = axs[0, b] if cls == 0 else axs[1, b]
            
            ax.hist(mean_cams_per_band[cls][b].flatten(), bins='auto', color='blue', edgecolor='black')
            ax.set_title(f"{b_name} - Class {condition_names[cls]}", fontsize=10)
            ax.set_xlabel("Grad-CAM Value")
            ax.set_ylabel("Count")
            
            # ✅ Format tick con notazione scientifica
            ax.xaxis.set_major_formatter(sci_formatter)
            ax.xaxis.set_major_locator(MaxNLocator(4))      # max ~4 tick
            
            #ax.tick_params(axis='x', labelrotation=45, labelsize=6)
    
    
    
    #PER PLOT RIGA 3-4 
    
    '''
    Concateno tutte le mean-CAM (cls 0+1, tutte le bande) in un unico array
    in modo da confrontare le Gradcam tra classi e bande tra di loro! 
    '''
    
    all_mean_cams = np.concatenate([
        mean_cams_per_band[0][b].flatten()
        for b in range(n_bands)
    ] + [
        mean_cams_per_band[1][b].flatten()
        for b in range(n_bands)
    ])
    vmin_cam = all_mean_cams.min()
    vmax_cam = all_mean_cams.max()
    
    
    for b, band in enumerate(band_names):                                 
        
        for j, cls in enumerate([0, 1]):
            
            ax = axs[2, b] if cls == 0 else axs[3, b]
            
            cam = mean_cams_per_band[cls][b] 
            
            # Controlla se la forma è corretta per l'input di imshow
            assert cam.ndim == 2, f"Expected 2D array, got {cam.ndim}D array"
            
            im = ax.imshow(
                cam,
                cmap = 'RdYlBu_r',
                vmin = vmin_cam, 
                vmax = vmax_cam,
                aspect = 'equal',
                origin = 'upper'
            )
            
            ticks = np.linspace(vmin_cam, vmax_cam, 6)

            cbar = fig.colorbar(
                im, ax=ax, orientation='horizontal', pad=0.12, ticks=ticks, format='%.1e')
            
            cbar.set_ticks(ticks)
            cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
            cbar.set_ticklabels([f"{t:.2f}" for t in ticks])

            ax.set_title(
                f"{band} - Class {condition_names[cls]}",
                fontsize=10
            )

            if channel_names is not None:
                ax.set_xticks([])
                ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(
                        x, y, name,
                        ha='center', va='center',
                        fontsize=6, color='black', weight='bold'
                    )
            else:
                ax.axis("off")
    
    
    
    #PER PLOT RIGA 5-6 
    
    '''
    Concateno tutte lo spettogrammam medio logaritmico
    da confrontare tra le classi per ogni banda tra di loro! 
    '''

    
    # Log-trasform delle mappe di potenza per banda
    log_mean_power_per_band = {
        cls: [np.log1p(mean_raw_power_per_band[cls][b])
              for b in range(n_bands)]
        for cls in [0, 1]
    }
    
    #Concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    
    
    # concateno tutte le log mean-power (cls 0+1, tutte le bande) in un unico array
    all_mean_pow = np.concatenate([
        log_mean_power_per_band[0][b].flatten()
        for b in range(n_bands)
    ] + [
        log_mean_power_per_band[1][b].flatten()
        for b in range(n_bands)
    ])
    vmin_pow = all_mean_pow.min()
    vmax_pow = all_mean_pow.max()
    
    ticks = np.linspace(vmin_pow, vmax_pow, 6)
    
    from matplotlib.ticker import ScalarFormatter

    sci = ScalarFormatter(useMathText=True)
    sci.set_powerlimits((-2, 2))          # forza 1eX fuori dall’intervallo 1e‑2 … 1e2

    for b, band in enumerate(band_names):
        
        for cls in [0, 1]:
            ax = axs[4, b] if cls == 0 else axs[5, b]

            log_power = log_mean_power_per_band[cls][b]

            im = ax.imshow(
                log_power,
                cmap='jet',
                vmin=vmin_pow,
                vmax=vmax_pow,
                aspect='equal',
                origin='upper'
            )
            
            cbar = fig.colorbar(im, ax=ax, orientation='horizontal', pad=0.12, ticks = ticks, format= sci)#format='%.1e')

            cbar.ax.xaxis.set_major_formatter(sci) # <-- solo formatter
            cbar.ax.tick_params(labelsize=6)

            ax.set_title(f"{band} Log Mean Power - Class {condition_names[cls]}", fontsize=10)
            
            if channel_names is not None:
                ax.set_xticks([])
                ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name, ha='center', va='center', fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")
                
                
    '''
    OLD PLOTS SPETTROGRAMMA RAW 
    
    
    #****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******
     
    #PER PLOT RIGA 5-6 (SCALA LINEARE)
    
    from matplotlib.ticker import ScalarFormatter

    sci = ScalarFormatter(useMathText=True)
    sci.set_powerlimits((-2, 2))          # forza 1eX fuori dall’intervallo 1e‑2 … 1e2

    
    #Concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    
    
    # concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    all_mean_pow = np.concatenate([
        mean_raw_power_per_band[0][b].flatten()
        for b in range(n_bands)
    ] + [
        mean_raw_power_per_band[1][b].flatten()
        for b in range(n_bands)
    ])
    vmin_pow = all_mean_pow.min()
    vmax_pow = all_mean_pow.max()
    
    
    ticks = np.linspace(vmin_pow, vmax_pow, 6)
    
    # Riga 3: Mappa della potenza media rispetto a distribuzione congiunta (su ciascuna banda e classe)
    for b, band in enumerate(band_names):
        
        for cls in [0, 1]:
            ax = axs[4, b] if cls == 0 else axs[5, b]
            
            power = mean_raw_power_per_band[cls][b] 
            
            im = ax.imshow(
                power, 
                cmap='jet',
                vmin= vmin_pow,
                vmax= vmax_pow,
                aspect='equal',
                origin='upper'
            )
            
            #ticks = np.linspace(vmin_pow, vmax_pow, 6)
            
            cbar = fig.colorbar(im, ax=ax, orientation='horizontal', pad=0.12, ticks = ticks, format= sci)#format='%.1e')
            
            #cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
            #cbar.set_ticklabels([f"{t:.2f}" for t in ticks])
            
            cbar.ax.xaxis.set_major_formatter(sci)   # <-- solo formatter
            cbar.ax.tick_params(labelsize=6)
            
            ax.set_title(f"{band} Power - Class {condition_names[cls]}", fontsize=10)
            
            if channel_names is not None:
                ax.set_xticks([])
                ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name, ha='center', va='center', fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")
                
                
    
    #****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ****** ******
    
    #PER PLOT RIGA 5-6 (SCALA LOGARITMICA)
        
    
    # ----- 1. calcola vmin_pow / vmax_pow -----
    
    #Concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    
    
    # concateno tutte le mean-power (cls 0+1, tutte le bande) in un unico array
    all_mean_pow = np.concatenate([
        mean_raw_power_per_band[0][b].flatten()
        for b in range(n_bands)
    ] + [
        mean_raw_power_per_band[1][b].flatten()
        for b in range(n_bands)
    ])
    vmin_pow = all_mean_pow.min()
    vmax_pow = all_mean_pow.max()
    
    #Filtra solo i valori strettamente > 0 (la scala log non accetta zeri o negativi).
    #Perché? Se l’intero array fosse ≤ 0 (caso patologico) avremmo positive.size == 0.
    positive = all_mean_pow[all_mean_pow > 0]
    
    
    
    #In pratica: stiamo abbassando il bordo inferiore del colormap di un 10 % rispetto al minimo positivo reale, 
    #così quei pixel non finiscono “incollati” al limite della color‑bar. Se preferisci usare un altro margine (5 %, 1 %) basta cambiare 0.9 in 0.95, 0.99, ecc.
    #Se invece vuoi proprio che vada esattamente sul minimo, puoi togliere *0.9 (ma occhio ai warning di Matplotlib)
    
    #Il resto del blocco:

    #Calcola vmax_pow dal massimo globale.
    #Decide automaticamente use_log se il dynamic‑range supera 10³.
    #Imposta una sola logica di plotting: quando use_log è True usa LogNorm, LogLocator e LogFormatterMathtext; altrimenti scala lineare + ScalarFormatter.

    #I titoli aggiungono “(log10)” solo quando serve.
    #Nota: quando use_log è True, passiamo vmin/vmax tramite LogNorm; quando è False, li passiamo direttamente a imshow con i parametri vmin=…, vmax=….
    #Così la stessa funzione disegna correttamente entrambe le situazioni senza dover duplicare codice.
    
    
    #1. Se esistono valori positivi, prende il più piccolo e lo moltiplica per 0.9 (−10 %).
    # Obiettivo: Creare un piccolo margine: il vero minimo non cade esattamente sul bordo inferiore della scala log, evitando clip / warning.
    
    #2. Se non esistono, imposta un fallback sicur0
    # Obiettivo: Garantire che vmin_pow > 0 in ogni caso (requisito di LogNorm).
    
    #vmin_pow = positive.min()*0.9 if positive.size else 1e-12
    #vmin_pow = positive.min() if positive.size else 1e-12
    
    vmin_pow = positive.min()
    
    vmax_pow = all_mean_pow.max()

    use_log  = vmax_pow / max(vmin_pow, 1e-12) > 1e3   # o il flag suggest_log

    if use_log:
        norm      = LogNorm(vmin=vmin_pow, vmax=vmax_pow)
        locator   = LogLocator(base=10.0)
        formatter = LogFormatterMathtext(base=10.0)
    else:
        norm      = None
        locator   = None
        formatter = ScalarFormatter(useMathText=True)
        formatter.set_powerlimits((-2, 2))         # 1e‑2 – 1e2 lineare

    # ----- 2. plot -----
    for b, band in enumerate(band_names):
        for cls in (0, 1):
            ax   = axs[4, b] if cls == 0 else axs[5, b]
            pow_ = mean_raw_power_per_band[cls][b]

            im = ax.imshow(pow_, cmap='jet', norm=norm,
                           vmin=None if use_log else vmin_pow,
                           vmax=None if use_log else vmax_pow,
                           aspect='equal', origin='upper')

            cbar = fig.colorbar(im, ax=ax, orientation='horizontal', pad=0.12)
            
            if locator is not None:
                cbar.locator   = locator
            cbar.formatter = formatter
            cbar.update_ticks()
            cbar.ax.tick_params(labelsize=8)

            scale = "(log10)" if use_log else ""
            ax.set_title(f"{band} Power {scale} – Class {condition_names[cls]}",
                         fontsize=10)
            
            if channel_names is not None:
                ax.set_xticks([])
                ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name, ha='center', va='center', fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")
    
    '''
    
    
    
    #PER PLOT RIGA 7-8
    
    '''
    Vorrei solo verificare allora l'ultima riga 7-8 per la differenza tra i due modelli, perchè: 
    
    
    1) Nel caso del modello 3d puro, ho ancora una fetta rappresentata, ossia

    per global_cams_3d vado ad ottenere una media, ossia una global_mean_cams_3d, che riassume il contributo GLOBALE della gradcam 3D aggregata
    all'interno dell'intero volume 3D (che poi al massimo si può scorporare vedendo per ogni banda successivamante) 

    ed è quello che vorrei fare per il modello Conv3D puro...

    2) per il modello Conv Separabili invece, 

    per global_cams_2d vado ad ottenere una media, ossia una global_mean_cams_2d, che riassume il contributo GLOBALE della gradcam 2D aggregata
    all'interno di TUTTE LE BANDE ASSIEME (che mi dovrebbe dare quindi per OGNI CLASSE un plot unico, 
    
    e non come il global_mean_cams_3d, dove dovrei vedere in quel caso, invece, la stessa mappa di “rilevanza complessiva”, MA distribuita lungo la profondità,
    ossia tra le bande e quindi potrei vedere se effettivamente io abbia una banda che è specificatamente più attiva di altre COMPLESSIVAMENTE ...

    devo verificare che per queste righe 7-8, a seconda del modello, il codice sia corretto, in base a come so che 

    global_mean_cams_3d e global_mean_cams_2d sono in realtà adesso ossia 
    
    global_mean_cams_3d[cls]	(5, 9, 9)	volume medio 3D
    global_mean_cams_2d[cls]	(9, 9)	heatmap 2D “globale” su tutte le bande

    '''
    
    
    '''
    Costruiamo la distribuzione congiunta della media del singolo input multi-canale per ogni classe
    '''
    
    all_global_mean_cams = np.concatenate([global_mean_cams[0].flatten(), global_mean_cams[1].flatten()])
    
    global_vmin_cam = all_global_mean_cams.min()
    global_vmax_cam = all_global_mean_cams.max()
    
    
    '''
    In sintesi:

    CNN3D: global_mean_cams_3d[cls] è già shape (5,9,9), quindi fai subito mat2d = global_mean_cams_3d[cls][b]
    SeparableCNN2D: global_mean_cams_2d[cls] è shape (9,9), e la metti in axs[6, cls]
    
    '''
    
    
    #'''Global CAM 2D: una mappa per classe, entrambe su riga 6'''
    
    if isinstance(model, SeparableCNN2D_LSTM_FC):
        
        #mean_cams_per_band = mean_cams_per_band_2d
        #mean_raw_power_per_band = mean_raw_power_per_band_2d
        global_mean_cams = global_mean_cams_2d
        
        # Global 2D: una sola heatmap per classe
        for cls in [0, 1]:
            ax = axs[6, cls]
            mat2d = global_mean_cams[cls]  # (9,9)
            im = ax.imshow(mat2d,
                           cmap='RdYlBu_r',
                           vmin=global_vmin_cam,
                           vmax=global_vmax_cam,
                           aspect='equal',
                           origin='upper')
            ticks = np.linspace(global_vmin_cam, global_vmax_cam, 6)
            cbar = fig.colorbar(im, ax=ax,
                                orientation='horizontal',
                                pad=0.12,
                                ticks=ticks,
                                format='%.1e')
            cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
            cbar.set_ticklabels([f"{t:.2f}" for t in ticks])
            ax.set_title(f"Global CAM 2D (across bands) – Class {condition_names[cls]}", fontsize=10)

            if channel_names is not None:
                ax.set_xticks([]);  ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name,
                            ha='center', va='center',
                            fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")

        # spegni i subplot vuoti
        for col in range(2, n_bands):
            axs[6, col].axis("off")
        for col in range(n_bands):
            axs[7, col].axis("off")

    else:
        
        '''
        # Global 3D: una heatmap per banda e per classe
        #mean_cams_per_band = mean_cams_per_band_3d
        #mean_raw_power_per_band = mean_raw_power_per_band_3d
        
        global_mean_cams = global_mean_cams_3d
        
        for b, band in enumerate(band_names):
            
            #for cls in [0, 1]:
            for j, cls in enumerate([0, 1]):
                
                #ax = axs[6 + cls, b]  # cls==0→riga6, cls==1→riga7
                
                ax = axs[6, b] if cls == 0 else axs[7, b]
                
                vol3d = global_mean_cams[cls]     # (5,9,9) --> perché? 
                                                  # Perché sopra è stato fatto 'global_cams_3d[cls].append(cam_vol[0])'
                                                  # Quindi ogni dato non era più fatto da (B, D, W, H) dove B = 1 (ossia l'esempio stesso)
                                                  # Per cui dopo in global_mean_cams_3d quando ho fatto la media, ho ottenuto una rappresentazione MEDIA
                                                  # del gradcam 3D, PER OGNI BANDA. Quindi, quando prelevo la SINGOLA BANDA, basta che faccio lo 'slicing' ossia
                                                  # mat2d = vol3d[b]  --> da (5,9,9) diventa --> (9,9)
                
                mat2d = vol3d[b]                  # slice b → (9,9)
                
                im = ax.imshow(mat2d,
                               cmap='RdYlBu_r',
                               vmin=global_vmin_cam,
                               vmax=global_vmax_cam,
                               aspect='equal',
                               origin='upper')
                ticks = np.linspace(global_vmin_cam, global_vmax_cam, 6)
                cbar = fig.colorbar(im, ax=ax,
                                    orientation='horizontal',
                                    pad=0.12,
                                    ticks=ticks,
                                    format='%.1e')
                cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
                cbar.set_ticklabels([f"{t:.2f}" for t in ticks])
                ax.set_title(f"{band} Global CAM 3D – Class {condition_names[cls]}", fontsize=10)

                if channel_names is not None:
                    ax.set_xticks([]);  ax.set_yticks([])
                    for name, (y, x) in channel_names.items():
                        ax.text(x, y, name,
                                ha='center', va='center',
                                fontsize=6, color='black', weight='bold')
                else:
                    ax.axis("off")
        '''
        
        
        '''
        # Global 3D: una heatmap per classe TRA le bande
        '''
        
        # ➜ CNN3D: una sola mappa 2D globale per classe (media sulle bande)
        #   global_2d_from3d[cls] ha shape (9,9)
        
        global_mean_cams = global_2d_from3d
        
        for cls in [0, 1]:
            ax = axs[6, cls]   # uso riga 7, colonne 0 e 1
            mat2d = global_2d_from3d[cls]  # (9,9)

            im = ax.imshow(
                mat2d,
                cmap='RdYlBu_r',
                vmin=global_vmin_cam,
                vmax=global_vmax_cam,
                aspect='equal',
                origin='upper'
            )
            ticks = np.linspace(global_vmin_cam, global_vmax_cam, 6)
            cbar = fig.colorbar(
                im, ax=ax,
                orientation='horizontal',
                pad=0.12,
                ticks=ticks,
                format='%.1e'
            )
            cbar.ax.xaxis.set_major_locator(FixedLocator(ticks))
            cbar.set_ticklabels([f"{t:.2f}" for t in ticks])
            ax.set_title(f"Global CAM 3D (across bands) – Class {condition_names[cls]}", fontsize=10)

            if channel_names is not None:
                ax.set_xticks([]); ax.set_yticks([])
                for name, (y, x) in channel_names.items():
                    ax.text(x, y, name,
                            ha='center', va='center',
                            fontsize=6, color='black', weight='bold')
            else:
                ax.axis("off")

        # spegni gli altri subplot della riga 7–8
        for col in range(2, n_bands):
            axs[6, col].axis("off")
        for col in range(n_bands):
            axs[7, col].axis("off")
    
    
    # ------------------------------------------------------------ ------------------------------------------------------------
    # ❸ — Ripristino allo stato precedente il modello ottimizzato trovato migliore, che aveva incluso anche layer LSTM
    # ------------------------------------------------------------ ------------------------------------------------------------
    
    if needs_train_mode:
        # ➌ ripristino layer singoli (i.e., riporto BN/Dropout dove stavano in eval mode)
        for m, old_flag in saved:
            m.train(old_flag)
        # ➍ ripristino lo stato globale del modello (di nuovo ad .eval())
        # i.e.,  come era stato passato in input alla funzione compute_gradcam_figure a partire 'load_best_run_results'!
        
        #Così simuli l’eval (Dropout off, BN congelato) pur essendo in train() per soddisfare CuDNN‑RNN.
        model.train(was_training)
        
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
                                
    
    #Passaggio 8: Salvataggio della figura
    #Qui la figura viene salvata in un buffer di memoria, pronto per essere salvato o inviato altrove
    
    # -------------------------------
    # Passaggio 8: Salvataggio della figura in un buffer
    # -------------

    # Salva la figura in un buffer (che potrai poi passare a save_performance_results)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    fig_image = buf.getvalue()
    buf.close()
    plt.close(fig)
    
    return fig_image


#### **MODELLI CNN3D e Conv Separable**

In [ ]:
'''

VERSIONE CONVOLUZIONE 3D PURA e CONVOLUZIONI SEPARABILI 19 LUGLIO 2025


Due versioni dell’architettura:

CNN3D_LSTM_FC: usa nn.Conv3d per eseguire una vera convoluzione 3D sui cinque depth (bande di frequenza), 
mantenendo il resto del flusso identico.

SeparableCNN2D_LSTM_FC: applica in sequenza una convoluzione depthwise (gruppi = canali) e una pointwise (1×1) 
per fondere i cinque canali in modo efficiente.

Entrambe le classi si integrano con il tuo blocco LSTM e il classificatore come nella versione originale.



Per ottenere un Grad‑CAM “3D” su ciascuna delle 5 bande (cioè un volume 9×9×5) 
invece di schiacciare tutto in una mappa 9×9, bisogna:

Non appiattire la dimensione di profondità (“depth” = bande) con cam.mean(dim=1).

Calcolare i pesi medi dei gradienti solo su altezza e larghezza, non su depth, in modo da preservare D=5.

Upsample (solo) le due dimensioni spaziali H×W, lasciando inalterata la profondità D.

(Opzionale) 

Se il tuo primo Conv3d usa un kernel di profondità pari all’intera profondità d’ingresso, 
quella informazione viene compressa in D=1!

Se vuoi davvero avere D=5 in uscita, devi cambiare conv1 in:


# ❌ kernel_size=(5,3,3), padding=(0,1,1) → D_out = 1
self.conv1 = nn.Conv3d(1, 32, kernel_size=(3,3,3), padding=(1,1,1))
così la profondità si conserva da 5→5.



1) Perché in conv1 useremo padding=(1,1,1) e negli altri layer padding=(0,1,1)
Obiettivo: mantenere la profondità (numero di bande, D = 5) costante lungo tutta la rete.

In conv1, abbiamo scelto kernel_size=(3,3,3) perché vogliamo che il filtro “scorra” su tutti e tre gli assi (D,H,W).

Con kernel_depth=3, per avere

𝐷out = (𝐷in + 2⋅𝑃 depth − 𝐾 depth)/ 𝑆 + 1 = 5

Da qui (1,1,1) per (depth, height, width).

Negli altri layer 3D (conv2a, conv2b, conv3) il kernel depth = 1 (kernel_size=(1,3,3)), 
quindi la profondità non cambia se mettiamo padding_depth=0 con padding (0,1,1) nel layer conv2 e conv3

In altre parole, su quell’asse non serve alcun padding:

se P dept = 0 allora diventa infatti

𝐷out = (𝐷in + 2⋅0 − 𝐾 depth)/ 𝑆 + 1 = 5

2⋅0


Non è che la tua rete “CNN3D_LSTM_FC” sia sbagliata in senso assoluto, 
ma — proprio a causa di quel primo Conv3d con kernel_size=(5,3,3) e padding=(0,1,1) — 

stai automaticamente comprimendo tutte e 5 le bande nella singola fetta di profondità:


self.conv1 = nn.Conv3d(
    in_channels=1, out_channels=32,
    kernel_size=(5, 3, 3),  # → D_out = (5 − 5 + 2·0)/1 + 1 = 1
    padding=(0, 1, 1)
)
Quindi il tuo tensore (B, 1, 5, 9, 9) diventa (B, 32, 1, 9, 9): la dimensione depth (5) si riduce a 1 subito.

Se invece vuoi davvero preservare le 5 “fette” come vera terza dimensione spaziale, hai due possibili correzioni:

Usare un kernel 3×3×3 (o 1×3×3) in conv1, in modo da non “abbracciare” tutta la profondità d’ingresso:


- self.conv1 = nn.Conv3d(1, 32, kernel_size=(5, 3, 3), padding=(0, 1, 1))
+ # preserva D: depth out = depth in = 5
+ self.conv1 = nn.Conv3d(1, 32, kernel_size=(3, 3, 3), padding=(1, 1, 1))

Oppure, se vuoi mantenere le bande completamente indipendenti in questo layer,


self.conv1 = nn.Conv3d(1, 32, kernel_size=(1, 3, 3), padding=(0, 1, 1))
che trasforma (B,1,5,9,9) → (B,32,5,9,9).

Lasciare com’è, sapendo però che la rete “fonderà” le 5 bande in un’unica mappa di profondità: 
non è un bug, è una scelta architetturale.

La SeparableCNN2D_LSTM_FC
Quella architettura non comprime mai le bande all’interno di un’unica fetta, perché:

Le bande diventano canali di un Conv2d depth‑wise:


x = x.permute(0, 3, 1, 2)  # (B,5,9,9)
self.dw_conv1 = nn.Conv2d(5, 5, kernel_size=3, padding=1, groups=5)
Ogni “fetta” (banda) resta separata fino al pointwise e agli strati successivi.

Quindi se il tuo obiettivo è avere un’uscita per banda (e poi poter plottare un Grad‑CAM 2D per ciascuna),
la SeparableCNN2D è già configurata correttamente.

Se invece vuoi un Grad‑CAM “volumetrico” 3D (5×9×9) direttamente dal modello 3D puro,
la vera modifica necessaria è solo sul primo Conv3d, come mostrato sopra.

Fammi sapere quale dei due setup stai usando e ti aiuto a integrare il Grad‑CAM 3D di conseguenza!

'''



'''
Ecco l’implementazione completa di CNN3D_LSTM_FC (“approccio sequenza di profondità”) in cui:

mantieni il tuo primo blocco 3D con kernel (3,3,3) e padding (1,1,1), quindi D rimane 5 fino alla fine;

riduci le spatial singleton dims (H=1,W=1) e trasformi la depth D=5 in una sequenza di lunghezza 5;

imposti l’input_size=128 nell’LSTM (feature per time‑step = 128);


Con questa versione:

la sequenza per l’LSTM ha lunghezza D=5;

ogni passo ha 128 feature, esattamente input_size=128;

non servono trucchi di reshape su scala globale.


'''


import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN3D_LSTM_FC(nn.Module):
    """
    Version with pure 3D convolutions treating the 5 frequency bands
    as a sequence (depth) for the LSTM.
    Input: Tensor of shape (B, 9, 9, 5) --> reshaped to (B, 1, 5, 9, 9)
    """
    def __init__(self, num_classes=2, dropout=0.5, hidden_size=64, use_lstm=True):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.use_lstm = use_lstm

        # --- Block 1 (3D) ---
        self.conv1   = nn.Conv3d(1,  32, kernel_size=(3,3,3), padding=(1,1,1))
        self.bn1     = nn.BatchNorm3d(32)
        self.pool3d  = nn.MaxPool3d((1,2,2))  # non tocca D

        # --- Block 2 (3D Residual) ---
        self.res_conv3d = nn.Conv3d(32, 64, kernel_size=1, bias=False)
        self.res_bn3d   = nn.BatchNorm3d(64)
        self.conv2a     = nn.Conv3d(32, 64, kernel_size=(1,3,3), padding=(0,1,1))
        self.bn2a       = nn.BatchNorm3d(64)
        self.conv2b     = nn.Conv3d(64, 64, kernel_size=(1,3,3), padding=(0,1,1))
        self.bn2b       = nn.BatchNorm3d(64)

        # --- Block 3 (3D) ---
        self.conv3 = nn.Conv3d(64, 128, kernel_size=(1,3,3), padding=(0,1,1))
        self.bn3   = nn.BatchNorm3d(128)

        # LSTM o FC finale
        if self.use_lstm:
            # input_size = feature_dim per time‑step = 128
            self.lstm       = nn.LSTM(input_size=128,
                                      hidden_size=self.hidden_size,
                                      num_layers=1,
                                      batch_first=True)
            self.classifier = nn.LazyLinear(num_classes)
        else:
            self.classifier = nn.LazyLinear(num_classes)

    def forward(self, x):
        # x: (B, 9, 9, 5)
        if x.ndim == 4:
            # -> (B,1,D=5,H=9,W=9)
            x = x.permute(0, 3, 1, 2).unsqueeze(1)

        # --- Block 1 ---
        x = F.relu(self.bn1(self.conv1(x)))  # (B,32,5,9,9)
        x = self.pool3d(x)                   # (B,32,5,4,4)

        # --- Block 2 (Residual) ---
        res = self.res_bn3d(self.res_conv3d(x))  # (B,64,5,4,4)
        x   = F.relu(self.conv2a(x))             # (B,64,5,4,4)
        x   = self.bn2b(self.conv2b(x))          # (B,64,5,4,4)
        x   = F.relu(x + res)                    # (B,64,5,4,4)
        x   = self.pool3d(x)                     # (B,64,5,2,2)

        # --- Block 3 ---
        x = F.relu(self.bn3(self.conv3(x)))      # (B,128,5,2,2)
        x = self.pool3d(x)                       # (B,128,5,1,1)

        # Stampa delle dimensioni prima di passare al classifier
        #print(f"Dimensioni prima del classifier: {x.shape}")

        if self.use_lstm:
            # x: (B,128,5,1,1)
            # -> squeeze spatial dims → (B,128,5)
            x = x.squeeze(-1).squeeze(-1)
            # -> permute per batch_first → (B, seq_len=5, feat=128)
            x = x.permute(0, 2, 1)
            x = self.dropout(x)
            out, _ = self.lstm(x)               # out: (B,5,hidden_size)
            last    = out[:, -1, :]             # prendo l’ultimo time-step
            logits  = self.classifier(last)     # (B, num_classes)
        else:
            # x: (B,128,5,1,1) → flatten → (B,128)
            x = x.view(x.size(0), -1)
            logits = self.classifier(self.dropout(x))

        return logits

    

class SeparableCNN2D_LSTM_FC(nn.Module):
    """
    Version with depthwise + pointwise separable convolutions
    across the 5 channels.
    Input: Tensor of shape (B, 9, 9, 5) -> (B,5,9,9)
    
    
    groups=5 → impone che ogni canale venga convoluto indipendentemente dagli altri → depthwise ✅

    kernel_size=1 → combina i 5 canali in un nuovo spazio di 32 feature maps → pointwise ✅


    """
    def __init__(self, num_classes=2, dropout=0.5, hidden_size=64, use_lstm=True):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.use_lstm = use_lstm

        # --- Block 1 separabile ---
        self.dw_conv1 = nn.Conv2d(5, 5, kernel_size=3, padding=1, groups=5)
        self.pw_conv1 = nn.Conv2d(5, 32, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)

        # --- Block 2 (residuo) ---
        self.res_conv = nn.Conv2d(32, 64, kernel_size=1, bias=False)
        self.res_bn = nn.BatchNorm2d(64)
        self.bn2a = nn.BatchNorm2d(32)
        self.conv2a = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2b = nn.BatchNorm2d(64)
        self.conv2b = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # --- Block 3 ---
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.lstm = nn.LSTM(input_size=128 * 5, hidden_size=self.hidden_size, num_layers=1, batch_first=True)

        if self.use_lstm:
            self.lstm = nn.LSTM(
                input_size=128 * 1,
                hidden_size=self.hidden_size,
                num_layers=1,
                batch_first=True
            )
            self.classifier = nn.Linear(self.hidden_size, num_classes)
        else:
            self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)  # -> (B,5,9,9)

        x = F.relu(self.dw_conv1(x))
        x = F.relu(self.bn1(self.pw_conv1(x)))
        x = self.pool(x)

        res = self.res_bn(self.res_conv(x))
        x = F.relu(self.conv2a(self.bn2a(x)))
        x = self.bn2b(self.conv2b(x))
        x = F.relu(x + res)
        x = self.pool(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)  # (B,128,1,1)

        if self.use_lstm:
            x = x.permute(0, 2, 1, 3).reshape(x.size(0), 1, -1)  # (B,1,128)
            out, _ = self.lstm(self.dropout(x))
            last = out[:, -1, :]
            logits = self.classifier(last)
        else:
            x = x.view(x.size(0), -1)
            logits = self.classifier(self.dropout(x))

        return logits

##### **NUOVO LOOP PER DATI NON HYPER SU CNN2D, BiLSTM e Transformer**

##### **CON RICHIAMO DEI VALORI DA WAND LOG**

Qui vado a creare una funzione che al posto di "training" (che ri-eseguiva il training, sbagliando), 

1) mi prenda fornisca in output i risultati delle metriche estratte dal wand.log ...
PERO' QUESTA VOLTA della run con la migliore accuratezza di validazione ottenuta per quel modello e progetto corrispondente..

2) prova a suggerirmi se in qualche modo ci sia dentro questa ultima funzione da creare, di ricostruirmi il modo di ricomputare
l'auc roc per il training allora (senza dover far ripartire l'ottimizzazione degli iper-parametri...)

3) a quel punto in output a questa funzione da creare, vorrei ottenere una stessa struttura che ottenevo in output della funzione di training ... 
la differenza, sostanzialmente è che, mentre prima si ri-eseguiva il training sul modello già ottimizzato, sbagliando, 
qui invece dovrebbe solo estrarsi le metriche ottenute proprio quel modello migliore che è quello con la val accuracy migliore 
(ossia il mio modello migliore già salvato in best model), 

ossia dalla run di quel progetto e modello corrispondente con la run che possiede appunto la val accuracy migliore (ossia il mio modello migliore già salvato in best model)


#### **Implementazione: Carico Dati** 

In [ ]:
'''VERSIONE NUOVA UFFICIALE


Ecco come puoi correggere solo il calcolo dell’AUC–ROC sul training set a posteriori, 
lasciando invariato tutto il resto di load_best_run_results. 


L’idea è:

1) Estrarre la history normale da W&B (che contiene il vecchio train_auc)
2) Individuare best_epoch
3) Caricare il modello migliore da disco
4) Rifare un passaggio solo sullo train_loader per ottenere le vere probabilità e ricalcolare la ROC–AUC
5) Sovrascrivere il vecchio valore auc_train_history[best_epoch] e aggiornare best_metrics["train_auc"]



Cosa è cambiato

1) Ti ho inserito un passaggio 6) in cui ricalcoli l’AUC–ROC vero del train set, usando torch.softmax(…,dim=1)[:,1].
2) Sostituisci il vecchio auc_train_history[best_epoch] col valore corretto.
3) Ricomponi best_metrics["train_auc"] con true_auc_train.

Da qui in poi, puoi chiamare subito dopo la tua testing(...) per ottenere anche tutte le metriche sul test set e salvare la tabella finale in cui:

“Train” = best_metrics["train_*"] (ora con AUC corretta)

“Test” = test_results["test_performances"]

Ecco fatto: nessun re‑training, solo un passaggio aggiuntivo per correggere il calcolo dell’AUC–ROC sul train set.



Quindi il punto 6

# --- 6) Ricalcolo vero train AUC–ROC sul train_loader ---

serve per ri-calcolarsi correttamente l'auc roc al train set nell'epoca in cui sul val set ho ottenuto la migliore validation accuracy, 
che corrisponde quindi al modello salvato dentro il best_model che io ri-prelevo quando poi lo do in pasto al test set?


Esattamente: quel passaggio 6):

Riprende il modello caricato dal file .pkl (che è proprio il best_model scelto sull’epoca di miglior val_accuracy),

Lo mette in eval() e senza gradienti scorre tutto il train_loader,

Calcola le probabilità (softmax(:,1)) e da quelle ricava la vera ROC–AUC per il train set,

Infine sovrascrive auc_train_history[best_epoch] e aggiorna best_metrics["train_auc"] con questo valore corretto.

In questo modo la tua colonna “Train” nella tabella conterrà davvero l’AUC–ROC calcolata sulle probabilità del modello nella stessa epoca 
in cui hai ottenuto la migliore validazione, cioè esattamente quei pesi che poi passerai al test set.


'''

from wandb import Api
import torch
import numpy as np

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import io
import matplotlib.pyplot as plt
from PIL import Image


import re


    
'''
1) questa serve per plottare le metriche di loss e accuracy in ogni modello e condizione sperimentale
per salvarla dentro al dizionario 'training_plot' come buffer di memoria
'''


def plot_training_results(loss_train_history, loss_val_history, accuracy_train_history, accuracy_val_history):
    
    '''
    # Creazione di una figura con 2 subplot
    '''
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))  # 2 righe, 1 colonna, dimensione figura

    #Plot della loss
    ax[0].plot(loss_train_history, label='Train Loss', color='blue')
    ax[0].plot(loss_val_history, label='Validation Loss', color='orange')
    #ax[0].set_title(f'Loss during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[0].set_title(f'Loss during Training: ', fontsize=12)  # Titolo più grande
    ax[0].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[0].set_ylabel('Loss', fontsize=12)    # Dimensione font asse y
    ax[0].legend(fontsize=12)  # Dimensione font legenda
    ax[0].grid(True)

    # Plot dell'accuracy
    ax[1].plot(accuracy_train_history, label='Train Accuracy', color='blue')
    ax[1].plot(accuracy_val_history, label='Validation Accuracy', color='orange')
    #ax[1].set_title(f'Accuracy during Training: {exp_cond_1} vs {exp_cond_2}', fontsize=16)  # Titolo più grande
    ax[1].set_title(f'Accuracy during Training: ', fontsize=12)  # Titolo più grande
    ax[1].set_xlabel('Epochs', fontsize=12)  # Dimensione font asse x
    ax[1].set_ylabel('Accuracy', fontsize=12)  # Dimensione font asse y
    ax[1].legend(fontsize=12)  # Dimensione font legenda
    ax[1].grid(True)
    
    # Regolare la spaziatura tra i subplot
    plt.tight_layout()  # Alternativa: fig.subplots_adjust(hspace=0.3)
    
    #plt.close(fig)
    
    '''
    # Salvare il plot in un buffer di memoria
    '''
    buf = io.BytesIO()
    plt.savefig(buf, format='png')  # Salviamo il plot in formato PNG
    buf.seek(0)  # Torniamo all'inizio del buffer

    # Convertire il buffer in un'immagine PIL (opzionale, per visualizzarla)
    img = Image.open(buf)

    # Aggiungere i dati dell'immagine nel dizionario
    plot_image_data = buf.getvalue()  # Otteniamo i dati binari dell'immagine
    buf.close()
    
    # Ritorniamo i dati dell'immagine da salvare nel dizionario
    return plot_image_data


'''
2) questa serve per estrarmi le stringhe per ricostruire il nome del progetto su W&B per 
poi estrarmi le metriche ottenute sul training e validation 
da salvare sempre dentro al dizionario 'training_plot' 
'''

# Funzione per parsare la chiave
def parse_combination_key(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    Il formato atteso è:
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ 
    "1_20|1_45|wavelet" _ 
    "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    """
    match = re.match(
        r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_(spectrograms)_(familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt)$",
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        

'''CELLA DI ESEMPIO PER VERIFICARE SE QUESTA FUNZIONE FACESSE IL PARSING DELLE STRINGHE DELLE COMBINAZIONI DI FATTORI CORRETTAMENTE'''

# Test
#combination_key = "rest_vs_left_fist_spectrograms_familiar_th"
#condition_experiment, data_type, subject_key = parse_combination_key(combination_key)

#print("Condizione:", condition_experiment)
#print("Data Type:", data_type)
#print("Soggetto:", subject_key)



'''

Ecco come puoi correggere solo il calcolo dell’AUC–ROC sul training set a posteriori, lasciando invariato tutto il resto di load_best_run_results. 


L’idea è:

1) Estrarre la history normale da W&B (che contiene il vecchio train_auc)
2) Individuare best_epoch
3) Caricare il modello migliore da disco
4) Rifare un passaggio solo sullo train_loader per ottenere le vere probabilità e ricalcolare la ROC–AUC
5) Sovrascrivere il vecchio valore auc_train_history[best_epoch] e aggiornare best_metrics["train_auc"]



Cosa è cambiato

1) Ti ho inserito un passaggio 6) in cui ricalcoli l’AUC–ROC vero del train set, usando torch.softmax(…,dim=1)[:,1].
2) Sostituisci il vecchio auc_train_history[best_epoch] col valore corretto.
3) Ricomponi best_metrics["train_auc"] con true_auc_train.

Da qui in poi, puoi chiamare subito dopo la tua testing(...) per ottenere anche tutte le metriche sul test set e salvare la tabella finale in cui:

“Train” = best_metrics["train_*"] (ora con AUC corretta)

“Test” = test_results["test_performances"]

Ecco fatto: nessun re‑training, solo un passaggio aggiuntivo per correggere il calcolo dell’AUC–ROC sul train set.



Quindi il punto 6

# --- 6) Ricalcolo vero train AUC–ROC sul train_loader ---

serve per ri-calcolarsi correttamente l'auc roc al train set nell'epoca in cui sul val set ho ottenuto la migliore validation accuracy, 
che corrisponde quindi al modello salvato dentro il best_model che io ri-prelevo quando poi lo do in pasto al test set?


Esattamente: quel passaggio 6):

Riprende il modello caricato dal file .pkl (che è proprio il best_model scelto sull’epoca di miglior val_accuracy),

Lo mette in eval() e senza gradienti scorre tutto il train_loader,

Calcola le probabilità (softmax(:,1)) e da quelle ricava la vera ROC–AUC per il train set,

Infine sovrascrive auc_train_history[best_epoch] e aggiorna best_metrics["train_auc"] con questo valore corretto.

In questo modo la tua colonna “Train” nella tabella conterrà davvero l’AUC–ROC calcolata sulle probabilità del modello nella stessa epoca 
in cui hai ottenuto la migliore validazione, cioè esattamente quei pesi che poi passerai al test set.

'''


'''
3) Dopodiché, comincia la funzione di load_best_run_results che, 
per ogni progetto e sweep del relativo modello,

si va ad estrarre le metriche del train (corregge il calcolo del train_auc)
e si calcola anche per il validation phase la confusion matrix e classification report


4) dopodichè dovrebbe richiamare la funzione di 
"plot_training_results" in modo che poi si salvi i plot di training e validation (sia loss che accuracy)
in modo che si salvi tutto in una immagine come buffer che viene spuntato fuori da quella funzione 

e poi inserito come valore dentro al dizionario training_results che sarà l'output di "load_best_run_results" 


quindi qui sotto mi manca richiamare la funzione "plot_training_results" con una variabile tipo training_plot = plot_training_results che avrà come argomenti

queste liste qua salvate come colonne del df creato dentro a 'load_best_run_results!'


loss_train_history     = df["train_loss"].tolist()
loss_val_history       = df["val_loss"].tolist()
accuracy_train_history = df["train_accuracy"].tolist()
accuracy_val_history   = df["val_accuracy"].tolist()


5) dopodiché mi serve caricare tutte queste info dentro al dizionario train_results, che sarà l'output di load_best_run_results... 
e su questo ho dei dubbi su quali chiavi del dizionario tenere separate oppure se "unirne" qualcuna, aggregando tutte le info del sweep_config assieme, 
sia che siano veri iper-parametri (learning rate etc) o parametri architetturali della rete (anche se avevano valori fissi) il più delle volte se vedi



# 2.1 – Sweep config per ciascun modello
sweep_config_cnn3d = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN3D_LSTM_FC"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True, False]},
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "use_lstm": {"values": [True, False]},
        "lstm_hidden": {"values": [32]},
        "dropout": {"values": [0.5]},
    }
}


sweep_config_cnn_sep = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["SeparableCNN2D_LSTM_FC"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True, False]},
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "use_lstm": {"values": [True, False]},
        "lstm_hidden": {"values": [32]},
        "dropout": {"values": [0.5]},
    }
}


'''

def load_best_run_results(
    key, # es. "rest_vs_left_fist_spectrograms_familiar_th"
    model, # # <-- istanza PyTorch già caricata con i pesi best es. "CNN3D_LSTM_FC"
    
    sweep_config,      # <— qui richiamo lo sweep config del modello corrispondente
    
    data_loaders, # dict con DataLoader per "train" e "val"
    entity= "my_wb_entity"): # entity = "stefano‑bargione‑universit‑di‑roma‑tor‑vergata"
    
    
    # --- 1) Parse key e ricava project name ---
    exp_cond, data_type, category_subject = parse_combination_key(key)
    
    project = f"{exp_cond}_{data_type}_channels_freqs_new_3d_grid_multiband"
    
    model_name = type(model).__name__
    

    '''SE ESTRAGGO SWEEP ID A POSTERIORI DAL PROGETTO

    1) Prendo tutte le run del progetto e modello corrispondente
    2) Filtro solo quelle con config["model_name"] == model_name.
    3) Controllo che ce ne sia almeno una (altrimenti errore).
    4) Costruisce un set di tutti gli r.sweep e verifica che sia esattamente uno (altrimenti errore).
    5) Estrae quello unico (.pop()) e lo stampa insieme al numero di run.
    6) Infine, seleziona la singola best_run sulla base di val_accuracy.

    '''
    
    # 2) Recupero tutte le run del progetto
    api  = Api()
    runs = api.runs(f"{entity}/{project}")

    # 3) filtro solo quelle del modello giusto
    runs_filtered = [r for r in runs if r.config.get("model_name", "") == model_name]
    n_runs = len(runs_filtered)

    if n_runs == 0:
        raise RuntimeError(f"Nessuna run trovata per progetto `{project}` e modello `{model_name}`")

    # 4) controllo che le run filtrate appartengano tutte allo stesso sweep
    unique_sweeps = {r.sweep for r in runs_filtered}
    if len(unique_sweeps) != 1:
        raise RuntimeError(
            f"Trovati più sweep per progetto `{project}` e modello `{model_name}`: {unique_sweeps}"
        )

    # 5) estraggo lo sweep_id
    sweep_id_unico = unique_sweeps.pop()
    #print(f"✓ Trovate \033[1m{n_runs}\033[0m runs in progetto `{project}` e modello `{model_name}`, sweep: `{sweep_id_unico}`")
    print(f"✓ Trovate \033[1m{n_runs}\033[0m runs\n")
    print(f"✓ Progetto \033[1m`{project}`\033[0m\n")
    print(f"✓ Modello \033[1m`{model_name}`\033[0m\n")
    print(f"✓ Sweep \033[1m`{sweep_id_unico}`\033[0m\n\n")

    # 6) scelgo la run con val_accuracy massima
    best_run = max(runs_filtered, key=lambda r: r.summary.get("val_accuracy", 0.0))

    # --- 7) Estraggo tutta la history (compresi i train_auc sbagliati) ---
    df = best_run.history(
        keys=[
          "train_loss","train_accuracy","train_precision",
          "train_recall","train_f1","train_auc",
          "val_loss","val_accuracy"
        ],
        pandas=True
    )
    # converto in liste
    loss_train_history     = df["train_loss"].tolist()
    loss_val_history       = df["val_loss"].tolist()
    accuracy_train_history = df["train_accuracy"].tolist()
    accuracy_val_history   = df["val_accuracy"].tolist()
    precision_train_history= df["train_precision"].tolist()
    recall_train_history   = df["train_recall"].tolist()
    f1_train_history       = df["train_f1"].tolist()
    auc_train_history      = df["train_auc"].tolist()

    # best_epoch (su val_accuracy)
    best_epoch = int(df["val_accuracy"].idxmax())

    # --- 8) Prendo il modello ottimizzato .pkl corrispondente passato in input ---
    device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device).eval()

    # --- 9) Ricalcolo vero train AUC–ROC sul train_loader ---
    y_t_train, y_s_train = [], []
    with torch.no_grad():
        for x,y in data_loaders["train"]:
            x = x.to(device)
            logits = model(x)
            probs  = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
            y_s_train.extend(probs)
            y_t_train.extend(y.numpy())
            
    true_auc_train = roc_auc_score(np.array(y_t_train), np.array(y_s_train))

    # Sovrascrivo il vecchio valore sbagliato
    auc_train_history[best_epoch] = true_auc_train

    # Ricostruisco best_metrics
    best_metrics = {
      "train_loss":       [round(loss_train_history[best_epoch],4)],
      "train_accuracy":   [round(accuracy_train_history[best_epoch],4)],
      "train_precision":  [round(precision_train_history[best_epoch],4)],
      "train_recall":     [round(recall_train_history[best_epoch],4)],
      "train_f1_score":   [round(f1_train_history[best_epoch],4)],
      "train_auc":        [round(true_auc_train,4)]
    }

    # --- 10) Ricreo confusion matrix e classification report su val set ---
    y_t_val, y_p_val = [], []
    with torch.no_grad():
        for x,y in data_loaders["val"]:
            x = x.to(device)
            logits = model(x)
            probs  = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
            preds  = (probs >= 0.5).astype(int)
            y_p_val.extend(preds)
            y_t_val.extend(y.numpy())
            
    confusion_matrix_val = confusion_matrix(y_t_val, y_p_val)
    classification_report_val = classification_report(y_t_val, y_p_val, output_dict=False)

    # --- 10) Plot delle curve loss/accuracy tra train e test ---
    training_plot = plot_training_results(
        loss_train_history,
        loss_val_history,
        accuracy_train_history,
        accuracy_val_history
    )

    # --- 11) Composizione del dict finale identico a `training()` ---
    
    # Restituire tutti i risultati in un dizionario
    train_results = {
        "training_performances": best_metrics,  # Aggiungi il dizionario delle performance
        
        "loss_train_history": loss_train_history,
        "loss_val_history": loss_val_history,
        
        "accuracy_train_history": accuracy_train_history,
        "accuracy_val_history": accuracy_val_history,
        
        "best_model": model,
        
        "confusion_matrix_val": confusion_matrix_val,
        "classification_report": classification_report_val,
    
        "hyperparams" : {k: best_run.config[k] for k in best_run.config.keys() if k in sweep_config["parameters"]},
            
        "training_plot": training_plot  # Salviamo il buffer con il plot
    }
    
    '''
    Ho questo errore "Errore “cudnn RNN backward can only be called in training mode”" solo con i dati di 
    left_fist_vs_right_fist, per il modello SeparableCNN2D_LSTM_FC, 
    mentre con i dati delle altre condizioni sperimentali, ossia:
    
    rest_vs_left_fist o rest_vs_right_fist, sempre per il modello SeparableCNN2D_LSTM_FC,non succede ... come mai solo con l'ultimo succede? 
    
    cioè dove dovrei aver lasciato il modello caricato in eval.() ?
    
    probabilmente qui nella funzione load_best_train_results!?
    
    quindi qui poi alla fine dovrei rimettere il modello in un'altra modalità alla fine della funzione? 
    perché in sostanza, dovrebbe succedere che in sostanza... non succede nulla per lo stesso modello per  gli altri dati, 
    perché ogni volta che ne prendo uno lo porto in eval e vabbè.. ma poi il problema succede solo per l'ultimo caso solo, 
    perché forse l'ultimo proprio, ossia solo SeparableCNN2D_LSTM_FC usa proprio il layer LSTM e quindi da errore là,
    perché dentro a load_best_train_results è rimasto in .eval() ed ha il layer LSTM e quindi dà errore?
    
    
    
    Perché l’errore appare “solo” con l’ultima combinazione

    1. load_best_run_results() termina con:

    model.to(device).eval()   # ← il modello rimane in eval()
    
    2. In compute_gradcam_figure() tu usi il best model che hai messo in train_results["best_model"] (quello appena impostato in eval()), poi esegui:

   
    output = model(sample_input)
    ...
    target.backward()         # <-- gradiente attraverso l’LSTM
    3. Il kernel CuDNN per gli RNN (LSTM/GRU) rifiuta il backward quando il modulo è in modalità inference (eval()), e solleva:

    
    RuntimeError: cudnn RNN backward can only be called in training mode
    
    4. Per le combinazioni precedenti con lo stesso modello “SeparableCNN2D_LSTM_FC” non è esploso perché, con ogni probabilità, 
    use_lstm=False nelle relative run migliori (quindi l’LSTM non c’è e CuDNN non interviene).
    
    Nell’ultima combinazione invece la best‑run ha use_lstm=True, quindi compare l’LSTM e l’errore salta fuori.
    
    '''
    

    
    return train_results

In [ ]:
#LOOP PER CARICARE I DATI NON HYPER
data_dict = {}

# Condizioni sperimentali
#experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

experimental_conditions = ["rest_vs_left_fist", "rest_vs_right_fist", "left_fist_vs_right_fist"]

for condition in experimental_conditions:

    for data_type in ["spectrograms"]:
        
        #for category in ["familiar", "unfamiliar"]:
        for category in ["familiar"]:
            
            #for subject_type in ["th", "pt"]:
            for subject_type in ["th"]:
            
                # Caricamento e suddivisione dei dati
                if data_type == "wavelet":
                    X, y = load_data(data_type, category, subject_type, wavelet_level="delta", condition = condition)
                else:
                    X, y = load_data(data_type, category, subject_type, condition = condition)

                #key = f"{condition}/{data_type}_{category}_{subject_type}"
                key = f"{condition}_{data_type}_{category}_{subject_type}"
                data_dict[key] = (X, y)

                # Stampa di conferma
                print(f"Dataset caricato: \033[1m{key}\033[0m - Forma X: {X.shape}, Lunghezza y: {len(y)}")

In [ ]:
data_dict.keys()

In [ ]:
#data_dict['rest_vs_left_fist_spectrograms_familiar_th'][0].shape

In [ ]:
'''
perfetto ora, siccome ho creato data_dict nel modo di cui sopra, 
ora dentro ogni chiave, 
ci sono già tutte le chiavi associate correttamente, per estrarmi i dati e labels corrispondenti di quella combinazione di fattori lì.

infatti dentro ogni chiave c'è una tupla, con 2 elementi, il primo è l'array dei dati, il secondo è l'array delle labels
'''

#data_dict['rest_vs_left_fist']['spectrograms'][0].shape

data_dict['rest_vs_left_fist_spectrograms_familiar_th'][0].shape

In [ ]:
'''NEW VERSION'''

# Percorso base per il salvataggio
#base_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB_GradCAM_Checks_motor_task_1_new"

base_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks_motor_task_1_new_3d_grid_multiband"

os.makedirs(base_folder, exist_ok=True)

# Condizioni sperimentali
#experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

experimental_conditions = ["rest_vs_left_fist", "rest_vs_right_fist", "left_fist_vs_right_fist"]

# Tipologie di dati
data_types = ["spectrograms"]

# Subfolders per tipologia di soggetto
#subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]
subfolders = ["th_fam"]

# Creazione della struttura delle cartelle
for condition in experimental_conditions:
    for data_type in data_types:
        for subfolder in subfolders:
            
            path = os.path.join(base_folder, condition, data_type, subfolder)
            
            if not os.path.exists(path):
                os.makedirs(path, exist_ok=True)
            
            print(f"Cartella creata: \033[1m{path}\033[0m")

In [ ]:
# 2.1 – Sweep config per ciascun modello
sweep_config_cnn3d = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["CNN3D_LSTM_FC"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True, False]},
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "use_lstm": {"values": [True, False]},
        "lstm_hidden": {"values": [32]},
        "dropout": {"values": [0.5]},
    }
}


sweep_config_cnn_sep = {
    "method": "random",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        "weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        "n_epochs": {"value": 100},
        "patience": {"value": 12},
        "model_name": {"values": ["SeparableCNN2D_LSTM_FC"]},
        "batch_size": {"values": [32, 48, 64, 96]},
        "standardization": {"values": [True, False]},
        "beta1": {"values": [0.9, 0.95]},
        "beta2": {"values": [0.99, 0.995]},
        "eps": {"values": [1e-8, 1e-7]},
        "use_lstm": {"values": [True, False]},
        "lstm_hidden": {"values": [32]},
        "dropout": {"values": [0.5]},
    }
}

#### **Implementazione: Richiamo Reti Ottimizzate dopo W&B** 

In [ ]:
'''                                              *************** CON NUOVE MODIFICHE DEL PAPER ***************
                                                                    
                                                                    NEW VERSION CON
                                                            
                                                                   load_best_run_results


Cosa cambia rispetto a prima

1) load_config_if_available:

    estrae la config (batch_size, lr, β₁, β₂, ε, ecc.)
    carica i best_weights salvati in .pkl da W&B

2) processed_datasets / processed_models

    evitano duplicate runs

3) Optimizer & Scheduler & EarlyStopping

    sono configurati con i valori estratti da config
    scheduler con patience=4 (dimezza lr dopo 4 epoche senza miglioramento di val_loss)
    early‐stopping con patience=12 sul val_loss

4) Training/testing

    la tua funzione training(...) dovrebbe accettare ora anche scheduler e early_stopping

5) GradCAM

    calcolato e salvato solo per il tuo CNN2D+LSTM

Con queste modifiche, il tuo loop:

    rispetta fedelmente il setup del paper
    mantiene la pipeline di W&B per la logica di sweep e salvataggio dei migliori modelli
    aggiunge minimi cambiamenti, lasciando intatte le tue funzioni training(), testing() e save_performance_results()

'''


# Imposta il seme per la riproducibilità

#Imposta il seme per i generatori casuali di PyTorch (per operazioni sui tensori e inizializzazione dei pesi dei modelli).
#Importante se vuoi garantire che l'addestramento del modello produca gli stessi risultati in diverse esecuzioni.
torch.manual_seed(32)

#Imposta il seme per NumPy, utile se NumPy viene usato per operazioni casuali (ad es. shuffling dei dati, inizializzazione di matrici, ecc.).
#Importante se usi NumPy per il preprocessing dei dati e vuoi riproducibilità.

np.random.seed(32)

#mposta il seme per il modulo random di Python (utile se si usano funzioni di randomizzazione di Python puro).
#Importante solo se usi random per operazioni come mescolamento di liste.
random.seed(32)

#Imposta il seme per i generatori casuali su GPU, se disponibile.
#Utile se stai eseguendo il codice su una GPU per garantire riproducibilità anche in quel contesto.

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(32)

       
'''

In questo caso, 

il set processed_datasets traccia i dataset già elaborati, 
e il set processed_models tiene traccia delle combinazioni già effettuate (modello + dataset). 

In questo modo, puoi escludere un dataset dal training se è già stato utilizzato in precedenza, 
anche se usato con un modello differente.
'''

# Dizionario per tracciare la standardizzazione usata per ogni combinazione d
# Dizionario per salvare informazioni sul modello (es. se i dati sono standardizzati)
models_info = {}


# Set per tenere traccia dei dataset già elaborati
processed_datasets = set()

# Set per tenere traccia delle combinazioni già elaborate
processed_models = set()


# Path delle performance dei modelli ottimizzati con weight and biases
# Path per trovare le best performances di ogni modello per ogni combinazione dei dati
base_folder = "/home/stefano/Interrogait/WB_spectrograms_best_results_channels_frequencies_params_motor_task_1_new_3d_grid_multiband"
                                        #WB_spectrograms_best_results_motor_task_1

# Path di salvataggio delle performance dei modelli dopo estrazione best models da base_folder
#save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB"


save_path_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks_motor_task_1_new_3d_grid_multiband"
                    
                                              #spectrograms_best_models_post_WB_GradCAM_Checks_motor_task_1_new
                                    #spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks_motor_task_1_new


# --- LOOP PRINCIPALE (con minime modifiche) ---
for key, (X_data, y_data) in data_dict.items():
    
    print(f"\n\nEstrazione Dati per il dataset: \033[1m{key}\033[0m, \tShape X: \033[1m{X_data.shape}\033[0m, Shape y: \033[1m{y_data.shape}\033[0m")
    
    if key in processed_datasets:
        print(f"ATTENZIONE: Il dataset {key} è già stato elaborato! Salto iterazione...")
        continue
        
    processed_datasets.add(key)
    
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_data, y_data)
    print(f"Dataset Splitting: Train: \033[1m{X_train.shape}\033[0m, Val: \033[1m{X_val.shape}\033[0m, Test: \033[1m{X_test.shape}\033[0m")
    
    
    '''
    CREO COPIA TEST_LOADER_RAW PER I PLOT DEL POWER RAW PER BANDA E CLASSE
    '''
    # 1) salva una copia RAW dei soli dati di test PRIMA di standardizzare
    X_test_raw = X_test.copy()
    y_test_raw = y_test.copy()
    
    # 2) tensori
    X_raw_tensor = torch.tensor(X_test_raw, dtype=torch.float32)
    y_raw_tensor = torch.tensor(y_test_raw, dtype=torch.long)
    
    
    #for model_name in ["CNN2D", "BiLSTM", "Transformer"]:
    
    '''ATTENZIONE MODIFICA QUI'''
    
    #for model_name in ["CNN2D_LSTM_TF", "BiLSTM", "Transformer"]:
    for model_name in ["CNN3D_LSTM_FC", "SeparableCNN2D_LSTM_FC"]:

        model_key = f"{model_name}_{key}"
        if model_key in processed_models:
            print(f"ATTENZIONE: Il modello {model_name} per il dataset {key} è già stato addestrato! Salto iterazione...")
            continue
        processed_models.add(model_key)
        
        print(f"\nPreparazione dati per il dataset \033[1m{key}\033[0m e il modello \033[1m{model_name}\033[0m...")
        
        # Prova a caricare la configurazione e i pesi ottimali dal file .pkl
        
        '''
        load_config_if_available --> prende in input 'key' che è la chiave composita (i.e, th_resp_vs_pt_resp_1_20_familiar_th)
        parse_combination_key --> prende in input 'key' che suddivide la chiave composita in stringhe separate
        
        exp_cond, data_type, category_subject che sfrutto per crearmi la directory path che mi servirà per caricarmi 
        pesi del modello e i suoi iper-parametri
        
        Diciamo che in questo caso, sfrutto 'parse_combination_key per qualcosa che serve a 'load_config_if_available' in modo IMPLICITO..
        '''
        
        config, best_weights = load_config_if_available(key, model_name, base_folder)
        
        if config is None:
            raise ValueError(f"\033[1mNessun file .pkl trovato per {model_name} su {key}\033[0m. Non posso procedere senza la configurazione ottimale.")
        
        '''
        Successivamente, queste variabili vengono invece create in maniera ESPLICITA per fasi successive del loop
        MA in questo caso, parsifica la chiave una VOLTA SOLA e memorizza i valori!
        '''
        
        # Parsifica la chiave una volta sola e memorizza i valori
        exp_cond, data_type, category_subject = parse_combination_key(key)
        
        '''
        Dpodiché, 
        
        1) si carica i vari valori degli iper-parametri,
        2) si esegue la standardizzazione se servisse,
        3) prepara il modello per la divisione in train_loader etc.,
        4) si carica la configurazione dei pesi del modello, 
        5) assegna i vari valori degli iper-parametri del modello corrente per la combinazione di dati correntemente iterata 
        
        6) esegue il training e il test e poi
        
        7) si salva il tutto nella path corrispondente...
        
        '''
        
        '''
        PER DARE UNIFORMITÀ AL CODICE, CAMBIO IL NOME DELLE VARIABILI, CHE CONTENGONO I VALORI OTTIMIZZATI 
        DA FORNIRE IN INPUT ALLE VARIE FUNZIONI CHE SONO RICHIAMATE NEL LOOP'''
        
        
        #"lr": {"values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]},
        #"weight_decay": {"values": [0, 1e-4, 1e-3, 1e-2, 1e-1]},
        #"n_epochs": {"value": 100},
        #"patience": {"value": 12},
        
        #"model_name": {"values": ["SeparableCNN2D_LSTM_FC"]},
        #"batch_size": {"values": [32, 48, 64, 96]},
        #"standardization": {"values": [True, False]},
        #"beta1": {"values": [0.9, 0.95]},
        #"beta2": {"values": [0.99, 0.995]},
        #"eps": {"values": [1e-8, 1e-7]},
        
        #"use_lstm": {"values": [True, False]},
        #"lstm_hidden": {"values": [32]},
        #"dropout": {"values": [0.5]},
        
        
        model_lr = config["lr"]
        model_weight_decay = config["weight_decay"]
        model_n_epochs = config["n_epochs"]
        model_patience = config["patience"]
        
        
        model_batch_size = config["batch_size"]
        model_standardization = config["standardization"]
    
        
        '''NUOVE MODIFICHE'''
        model_beta1 =  config["beta1"]
        model_beta2 =  config["beta2"]
        model_eps = config["eps"]
        

        '''Per CNN3D_LSTM_FC e SeparableCNN2D_LSTM_FC'''
        model_use_lstm      = config["use_lstm"]
        model_lstm_hidden   = config["lstm_hidden"]
        model_dropout       = config["dropout"]
    
            
        #print(f"Parametri per \033[1m{model_name}\033[0m: batch_size= \033[1m{model_batch_size}\033[0m, n_epochs= \033[1m{model_n_epochs}\033[0m, patience= \033[1m{model_patience}\033[0m, lr= \033[1m{model_lr}\033[0m, weight_decay= \033[1m{model_weight_decay}\033[0m, standardization= \033[1m{model_standardization}\033[0m")
        print(f"Parametri per \033[1m{model_name}\033[0m: batch_size= \033[1m{model_batch_size}\033[0m, n_epochs= \033[1m{model_n_epochs}\033[0m, patience= \033[1m{model_patience}\033[0m, lr= \033[1m{model_lr}\033[0m, model_beta1= \033[1m{model_beta1}\033[0m,  model_beta2= \033[1m{model_beta2}\033[0m,  model_eps= \033[1m{model_eps}\033[0m, standardization= \033[1m{model_standardization}\033[0m")
        
        # Salva nel dizionario se per quella combinazione è stata applicata la standardizzazione ai dati
        models_info[model_key] = {"standardization": model_standardization}
        
        
        
        # 3) dataset & loader per test set (per plots power raw) –‑  IMPORTANTISSIMO: shuffle=False
        raw_dataset = TensorDataset(X_raw_tensor, y_raw_tensor)
        test_loader_raw = DataLoader(raw_dataset,
                             batch_size=model_batch_size,
                             shuffle=False)
        
        '''PER MANTENERE LA STESSA LOGICA DEL CODICE (ANCHE SE POTREI INSERIRLA DENTRO PREPARE_DATA_FOR_MODEL MODIFICANDO LA FUNZIONE (SI VEDA IN CELLA SOPRA COME)
        IMPONGONO LA STANDARDIZZAZIONE PRIMA DI QUESTA FUNZIONE
        '''

        if model_standardization:
            X_train, X_val, X_test = standardize_data(X_train, X_val, X_test)
            print(f"\033[1mSÌ Standardizzazione Dati!\033[0m")
        else:
            print(f"\033[1mNO Standardizzazione Dati!\033[0m")
        
        # Sposta il modello sulla GPU (se disponibile)
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

        
        # Preparazione dei dataloaders
        train_loader, val_loader, test_loader, class_weights_tensor = prepare_data_for_model(
            X_train, X_val, X_test, y_train, y_val, y_test, model_type = model_name, batch_size = model_batch_size)
        

        # Inizializzazione del modello
        #if model_name == "CNN2D":
        #    model = CNN2D(input_channels=64, num_classes=2)
        
        #if model_name == "CNN2D_LSTM_FC":
            
            #model = CNN2D_LSTM_FC(n_freq = 45, input_channels=64, num_classes=2, dropout = 0.2)
            #model = CNN2D_LSTM_FC(input_channels = 5, num_classes=2, dropout=0.2)
            
        #elif model_name == "BiLSTM":
        #    model = ReadMEndYou(input_size= 64 * 81, hidden_sizes=[24, 48, 62], output_size=2, bidirectional=True)
        #elif model_name == "Transformer":
        #    model = ReadMYMind(d_model=16, num_heads=4, num_layers=2, num_classes=2, channels=64, freqs=81)
        
        #elif model_name == "TopomapNet":
            #model = TopomapNet(
                #input_channels=5,
                #num_classes=2,
                #base_channels=model_base_channels,
                #use_lstm=model_use_lstm,
                #lstm_hidden=model_lstm_hidden,
                #dropout=model_dropout
            #)
        
        '''OCCHIO QUI CAMBIATO PER GRIGLIA 3D'''
        if model_name == "CNN3D_LSTM_FC":
            
            sweep_config = sweep_config_cnn3d
            
            model = CNN3D_LSTM_FC(
                num_classes=2,
                dropout=model_dropout,
                hidden_size=model_lstm_hidden,
                use_lstm=model_use_lstm)

            print(f"\nInizializzazione Modello \033[1mCNN3D_LSTM_FC\033[0m")
        
        elif model_name == "SeparableCNN2D_LSTM_FC":
            
            sweep_config = sweep_config_cnn_sep
            
            model = SeparableCNN2D_LSTM_FC(
                num_classes=2,
                dropout=model_dropout,
                hidden_size=model_lstm_hidden,
                use_lstm=model_use_lstm
            )
            print(f"\nInizializzazione Modello \033[1mSeparableCNN2D_LSTM_FC\033[0m")
            
        else:
            raise ValueError(f"Modello {model_name} non riconosciuto.")
        
        
        # Se abbiamo caricato i pesi ottimali, li carichiamo nel modello
        if best_weights is not None:
            try:
                model.load_state_dict(best_weights)
                print(f"📊 Modello \033[1m{model_name}\033[0m inizializzato con \033[01i pesi ottimizzati\033[0m tramite hyper-parameter tuning su \033[1mWeight & Biases\033[0m")
            except Exception as e:
                print(f"⚠️Errore nel caricamento dei pesi per {model_name} su {key}: {e}")
                continue
        
        '''OLD VERSION'''
        # Definizione del criterio di perdita
        #criterion = nn.CrossEntropyLoss(weight = class_weights_tensor)
        
        '''NUOVE MODIFICHE'''
        criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
        
        '''OLD VERSION'''
        # Definizione dell'ottimizzatore con i parametri aggiornati
        #optimizer = torch.optim.Adam(model.parameters(), lr = model_lr, weight_decay = model_weight_decay)
        
        '''NUOVE MODIFICHE'''
        
         # 10) ottimizzatore + scheduler + early stopping
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr    = model_lr,
            betas = (model_beta1, model_beta2),
            eps   = model_eps, 
            weight_decay = model_weight_decay
        )
            
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode     = "min",   # monitoriamo val_loss
            factor   = 0.1,
            patience = 8,
            verbose  = True
        )
        
        
        #early_stopping = EarlyStopping(patience=model_patience, mode="min")
        
        '''OLD VERSION'''
        #print(f"🏋️‍♂️Avvio del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        #my_train_results = training(model, train_loader, val_loader, optimizer, criterion, n_epochs = model_n_epochs, patience = model_patience)
        
        '''NEW VERSION'''
        # --- dopo model.load_state_dict(best_weights) e criterion = nn.CrossEntropyLoss() ---

        # 1) prepara i data_loaders per train/val
        data_loaders = {
            "train": train_loader,
            "val":   val_loader
        }
        
        print(f"🏋️‍♂️Salvo le metriche del training per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m a seguito della ottimizzazione su W&B ...")
        # 2) richiama la funzione che pesca da W&B la best‐run e corregge la train AUC
        
        #ATTENZIONE al potenziale problema di stringa, non di API: 

        #i due esempi che hai postato in realtà usano diversi caratteri “‑” (uno è il classico ASCII U+002D, l’altro è un non‑breaking hyphen U+2011 o simili), quindi quando chiami
        
        #entity = "stefano‑bargione‑universit‑di‑roma‑tor‑vergata"
        #stai passando un nome che W&B non riconosce (e quindi api.projects(entity=…) torna vuoto), mentre con

        #entity = "stefano-bargione-universit-di-roma-tor-vergata"
        #funziona perché lì usi i semplici - ASCII.

        my_train_results = load_best_run_results(
            key=key,
            model = model,
            sweep_config = sweep_config,
            data_loaders = data_loaders,
            entity = "stefano-bargione-universit-di-roma-tor-vergata"
        )
        
        
        
        '''
        L’entity che passi a Api().runs(f"{entity}/{project}") è semplicemente il tuo account (o l’organizzazione) su W&B,
        cioè la parte che compare subito prima del nome del progetto nell’URL.

        Per esempio, se quando apri il tuo progetto su W&B vedi un indirizzo del tipo
        
        -> https://wandb.ai/steclab/some_project_name, allora entity = "steclab".
        
        Se invece lavori sotto un’organizzazione 
        
        -> “cool‑team”, e l’URL è https://wandb.ai/cool-team/some_project_name, allora userai entity = "cool-team".

        Puoi verificarlo:

        Accedi a wandb.ai e vai sul progetto.
        Leggi la prima parte dell’URL (tra wandb.ai/ e il /project_name).
        Copiala esattamente come stringa in entity.

        Così il tuo Api().runs(f"{entity}/{project}") andrà a pescare proprio le run che hai lanciato tu.

        my_train_results = load_best_run_results(
            key= key,
            model = model,
            sweep_config = sweep_config,
            data_loaders = data_loaders,
            entity= "mio-entity"
        )
        
        '''
        
        print(f"Avvio del testing per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        # 3) usa il best_model caricato dentro `train_results` e chiama il testing
        my_test_results = testing(my_train_results, test_loader, criterion)
        
        '''
        GRADCAM COMPUTATION PER IL MODELLO CNN3D e ConvSep
        
        La funzione compute_gradcam_figure estrae i campioni (per ogni classe) e crea una figura con le due righe richieste.
        
        Il parametro gradcam_image (un buffer binario o un'immagine) viene passato alla funzione di salvataggio, 
        'save_performance_results', in modo da essere salvato nella path corretta. 
        
        La funzione 'save_performance_results' è stata modificata 
        per gestire ANCHE questo nuovo input dell'immagine 
        
        (ossia, per salvare il file con un nome che inizia con 'GradCAM_results_'
        seguito da tutte le altre stringhe corrispondenti alla combinazione di fattori che costituiscono il dataset corrente:
        
        - coppia di condizioni sperimentali da cui provengono i dati (i.e., th_resp_vs_pt_resp )
        - tipologia di dato EEG prelevato (i.e., spectrograms) 
        - provenienza del dato stesso (i.e., familiar_th)
        )
        
        Spiegazione:
        
        La funzione compute_gradcam_figure eseguire il calcolo di GradCAM (vedi dettagli nella sua funzione)
        e alla fine ritornerà in output una variabile 
        
        'fig_image' che sarà poi assegnata alla variabile 'gradcam_image',
        che è un oggetto buffer, che contiene i dati binari dell'immagine in formato PNG
        (poiché abbiamo usato plt.savefig con format='png'). 
        
        Quindi, quando passi gradcam_image (cioè fig_image) alla funzione 'save_performance_results',
        viene scritto direttamente su disco come file PNG.
        
        Non c'è bisogno di ri-aprire o convertire ulteriormente, a meno che tu non voglia manipolare l'immagine in seguito.
        Quindi, la soluzione è corretta così com'è:
        il buffer viene salvato come file PNG nella directory specificata, 
        e successivamente potrai aprirlo con una libreria come cv2 o PIL se necessario.        
        
        Quindi, gradcam_image (i.e., fig_image) viene quindi passato correttamente dentro al loop di training e test, 
        tramite 'save_performance_results', come input, 
        che salverà quindi poi l'immagine nella path corrispondente 

        '''
        
        # Se il modello è CNN2D, calcola anche GradCAM per la visualizzazione
        gradcam_image = None
        
        #if model_name == "CNN2D":
        
        '''ATTENZIONE MODIFICA QUI'''
        
        #if model_name == "CNN2D_LSTM_FC":
        
        if model_name in ("CNN3D_LSTM_FC", "SeparableCNN2D_LSTM_FC"):
            
            gradcam_image = compute_gradcam_figure(model, test_loader, test_loader_raw, exp_cond, data_type, category_subject, device, EEG_channels_names, debug = False)
            if gradcam_image is not None:
                print(f"Creazione di \033[1mGradCAM Image\033[0m per il modello \033[1m{model_name}\033[0m.")
                
        print(f"Salvataggio dei risultati per \033[1m{model_name}\033[0m sul dataset \033[1m{key}\033[0m...")
        save_performance_results(model_name,
                                 my_train_results,
                                 my_test_results,
                                 key,
                                 exp_cond,
                                 model_standardization,
                                 base_folder = save_path_folder,
                                 gradcam_image = gradcam_image)
        
        
        '''
        N.B
        
        gradcam_image = None avverrà solo all'inizio cioè per il primo modello, che verrà testato con una certa combinazione di dati.. 
        ma servirebbe tracciare in qualche modo 

        1) o che la gradcam_image di ogni combinazione venga ri-azzerata alla fine loop
        2) o che venga monitorato che gradcam_image di una combinazione di dati già analizzata venga esclusa poi
        (o messa in un set) in modo che rivenga per errore sovrascritta più volte.. 
        
        Forse la strada più veloce potrebbe essere la soluzione 1)
        
        La soluzione più veloce e semplice è re-impostare la variabile gradcam_image a None alla fine dell'iterazione per ogni combinazione di dati
        (cioè, all'interno del ciclo esterno che itera su key). 
         
        In questo modo, per ogni nuovo dataset la variabile viene "azzera" e viene calcolata l'immagine GradCAM solo per quella combinazione, 
        evitando di sovrascrivere accidentalmente i risultati già calcolati per combinazioni precedenti.
         
        Un'altra possibilità sarebbe tenere traccia delle chiavi (o combinazioni) per cui hai già calcolato la GradCAM,
        ad esempio usando un set, e saltare il calcolo se la combinazione è già presente. 
        
        Tuttavia, se ogni combinazione deve avere la sua immagine, 
        la soluzione più semplice è quella di reimpostare gradcam_image = None alla fine dell'iterazione.
        
        Quindi, per esempio, alla fine del ciclo per ogni dataset (key) potresti fare:
        (VEDI SOTTO)
        
        In questo modo, ti assicuri che per ogni nuova combinazione la variabile sia pulita e pronta per essere ricalcolata, 
        senza rischio di sovrascrivere o confondere i risultati
        '''
        
        # Reimposta gradcam_image a None per la prossima combinazione di dati
        gradcam_image = None
    

In [ ]:
#axs[0, j].hist(mean_cams_per_band[cls][b].flatten(), bins='auto', color='blue', edgecolor='black')
            #axs[0, j].set_title(f"\nClass {condition_names[cls]} - Band {b_name}", fontsize=9)
            #axs[0, j].set_xlabel("Grad-CAM value")
            #axs[0, j].set_ylabel("Count")
    

In [ ]:
print("finito")

In [ ]:
print(models_info.keys())

In [ ]:
with open('/home/stefano/Interrogait/spectrograms_best_models_channels_freqs_params_post_WB_GradCAM_Checks_motor_task_1_new_3d_grid_multiband.pkl', 'wb') as f:
    pickle.dump(models_info, f)

#### **CREAZIONE DELLE TABLES CON INTEGRAZIONE DELLE PERFORMANCE TRAINING & TEST DEI MODELLI DENTRO DATAFRAME**

#### **Integrazioni in Tabella delle Performance Training e Test del Modello dentro DataFrame - NEW APPROACH**

In [ ]:
import pickle 
path = '/home/stefano/Interrogait/'

#with open(f"{path}spectrograms_best_models_post_WB_GradCAM_Checks_motor_task_1_new.pkl", "rb") as f: 
with open(f"{path}spectrograms_best_models_channels_freqs_params_post_WB_GradCAM_Checks_motor_task_1_new_3d_grid_multiband.pkl", "rb") as f:
    models_info = pickle.load(f)

In [ ]:
'''
In questo codice:

model_info.get('standardization', False) cerca la chiave 'standardization' all'interno di ogni sottodizionario. 
Se non esiste, restituirà False come valore di default.
Se standardization è True, stampa la chiave associata.
'''

# Ciclo attraverso le chiavi di 'models_info'
for key, model_info in models_info.items():
    # Controllo se 'standardization' è True
    if model_info.get('standardization', False):  # Default a False nel caso in cui non esista la chiave
        print(key)  # Stampa la chiave

In [ ]:
models_info

In [ ]:
#for key, model_info in all_models.items():
#    print(key)

In [ ]:
'''
Siccome la stringa associata alla category subject è diversa tra i due.. 

familiar_th  familiar_pt unfamiliar_pt unfamiliar_pt  da un lato (models_info)
th_fam, pt_fam, th_unfam, pt_unfam  dall'altro (all_models)

la corrispondenza non avverrà mai... per cui, si deve fare il mapping corrispondente tra 
le stringhe di uno e dell'altro, in modo che models_info cambi come parte della stringa della sua chiave da queste 

familiar_th  familiar_pt unfamiliar_pt unfamiliar_pt
a queste
th_fam, pt_fam, th_unfam, pt_unfam 

'''

mapping_subject = {
    "familiar_th": "th_fam",
    "familiar_pt": "pt_fam",
    "unfamiliar_th": "th_unfam",
    "unfamiliar_pt": "pt_unfam"
}

# Creiamo un nuovo dizionario con le chiavi corrette
updated_models_info = {}

for key, value in models_info.items():
    for old_suffix, new_suffix in mapping_subject.items():
        if key.endswith(old_suffix):
            new_key = key.replace(old_suffix, new_suffix)
            updated_models_info[new_key] = value
            break  # Evita sostituzioni multiple se una è già stata fatta
    else:
        # Se nessuna sostituzione è stata fatta, mantieni la chiave originale
        updated_models_info[key] = value

# Sostituisci il vecchio dizionario con quello aggiornato
models_info = updated_models_info


In [ ]:
models_info.keys()

In [ ]:
''' Ciclo attraverso le chiavi di 'models_info' AGGIORNATO!'''

for key, model_info in models_info.items():
    # Controllo se 'standardization' è True
    if model_info.get('standardization', False):  # Default a False nel caso in cui non esista la chiave
        print(key)  # Stampa la chiavi

In [ ]:
'''
Parsing della chiave e costruzione del path:
Usando la funzione parse_combination_key si estraggono 

exp_cond, data_type e category_subject dalla chiave del dataset. 

Questi vengono usati per costruire il percorso in cui cercare i file .pkl.
'''

# Funzione per parsare la chiave
def parse_combination_models_keys(combination_key):
    """
    Estrae (exp_cond, data_type, category_subject) da combination_key.
    
    Il formato atteso PRIMA è:
    
    "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ spectrograms" _ "familiar_th|familiar_pt|unfamiliar_th|unfamiliar_pt"
    
    Il formato atteso ORA è:
    
     "th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp" _ spectrograms" _ "th_fam|th_unfam|pt_fam|pt_unfam"
     
    """
    #r"^(th_resp_vs_pt_resp|pt_resp_vs_shared_resp|th_resp_vs_shared_resp)_(spectrograms)_(th_fam|th_unfam|pt_fam|pt_unfam)$", 
    
    match = re.match(
        r"^(rest_vs_left_fist|rest_vs_right_fist|left_fist_vs_right_fist)_(spectrograms)_(th_fam|th_unfam|pt_fam|pt_unfam)$",
        combination_key
    )
    if match:
        return match.groups()  # (exp_cond, data_type, category_subject)
    else:
        raise ValueError(f"Formato non valido: {combination_key}")
        
    return exp_cond, data_type, category_subject

In [ ]:
'''
NEW APPROACH 

Adesso replichiamo l'approccio usato prima, ma stavolta integrado tutte le combinazioni di dati. 
Andiamo a

1) iterare sulla struttura delle directory a partire da base_folder, 
2) caricare i modelli .pkl per ogni combinazione di fattori che compongono i dati
3) creare un DataFrame che raccolga le metriche di tutti i modelli relativi alla stessa combinazione di dati. 

Infine, salviamo questa tabella come immagine all'interno della cartella corrispondente
'''


import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table



# Base folder
#base_folder = "/home/stefano/Interrogait/spectrograms_best_models_post_WB"

base_folder = "/home/stefano/Interrogait/spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks_motor_task_1_new_3d_grid_multiband"
                                        #spectrograms_best_models_channels_frequencies_params_post_WB_GradCAM_Checks_motor_task_1_new
                                            
# Condizioni sperimentali
#experimental_conditions = ["th_resp_vs_pt_resp", "th_resp_vs_shared_resp", "pt_resp_vs_shared_resp"]

experimental_conditions = ["rest_vs_left_fist", "rest_vs_right_fist", "left_fist_vs_right_fist"]


# Tipologie di dati
data_types = ["spectrograms"]

# Subfolders per tipologia di soggetto
#subfolders = ["th_fam", "th_unfam", "pt_fam", "pt_unfam"]

subfolders = ["th_fam"]

# Dizionario per salvare tutti i modelli
all_models = {}

# Caricamento dei modelli
for condition in experimental_conditions:
    for data_type in data_types:
        for subfolder in subfolders:
            
            path = os.path.join(base_folder, condition, data_type, subfolder)
            
            if not os.path.exists(path):
                print(f"Directory non trovata: {path}")
                continue
            
            # Creiamo la chiave per questa combinazione
            key = f"{condition}_{data_type}_{subfolder}"
            all_models[key] = {}

            # Otteniamo la lista di file nella directory
            files = os.listdir(path)
            
            # Filtriamo e carichiamo i file .pkl
            for file in files:
                if file.endswith(".pkl"):  # Controlliamo se è un file modello
                    file_path = os.path.join(path, file)
                    try:
                        with open(file_path, "rb") as f:
                            all_models[key][file] = pickle.load(f)
                    except Exception as e:
                        print(f"Errore nel caricamento di {file}: {e}")

# Creazione delle tabelle di performance
for key, models_dict in all_models.items():
    
    # Otteniamo le informazioni dalla chiave
    condition, data_type, subfolder = parse_combination_models_keys(key)
    
    print(f"\nProcessing: \033[1m{condition}\033[0m - \033[1m{data_type}\033[0m - \033[1m{subfolder}\033[0m\n")
    
    # Creazione della tabella
    df_data = {"Metriche": ["Accuracy", "Loss", "Precision", "Recall", "F1-Score", "AUC-ROC"]}

    # Iteriamo sui modelli caricati
    for model_name, model_data in models_dict.items():
        
        # Estrai il nome del modello dal file (ad esempio, "CNN1D" da "CNN1D_performances_...pkl")
        #name_model = model_name.split("_")[0]
        
        # Estrai il nome del modello, gestendo il caso speciale di CNN2D_LSTM
        #if model_name.startswith("CNN2D_LSTM"):
            #name_model = "_".join(model_name.split("_")[:2])  # Prende CNN2D_LSTM
            
        # estraiamo sempre la porzione fino a “_LSTM
        #if model_name.startswith(("CNN2D_LSTM", "CNN3D_LSTM", "SeparableCNN2D_LSTM")):
        if model_name.startswith(("CNN3D_LSTM", "SeparableCNN2D_LSTM")):
            
            # es. “CNN3D_LSTM_FC”  →  split → [“CNN3D”, “LSTM”, “FC”] → prendi i primi 2 elementi
            parts = model_name.split("_")
            name_model = "_".join(parts[:2])      # “CNN3D_LSTM” o “SeparableCNN2D_LSTM”
        else:
            name_model = model_name.split("_")[0]  # Prende solo CNN1D, CNN2D
        
        print(f"    Processing model: \033[1m{name_model}\033[0m")
        
        # Costruisci la chiave utilizzata nel dizionario models_info
        
        '''
        Nota: occorrerà che il formato della chiave sia consistente tra i due loop.
        
        Ad esempio, se nel primo loop era f"{key}_{model_name}", qui potresti dover fare:
        model_key = f"{key}_{name_model}"
        
        Oppure, se nel primo loop era f"{model_name}_{key}", qui potresti dover fare:
        model_key = f"{name_model}_{key}"
        
        '''
        model_key = f"{name_model}_{key}"
        
        # Controlla se i dati sono stati standardizzati per questo modello
        standardization_flag = models_info.get(model_key, {}).get("standardization", False)
        
        if standardization_flag:
            suffix = "" 
        else:
            suffix = "" 
        
        try:
            # Recupera i risultati di training e testing
            train_scores = model_data.get('my_train_results', {}).get('training_performances', {})
            test_scores = model_data.get('my_test_results', {}).get('test_performances', {})
            
            # Converti i valori in float
            train_scores = {key: float(value[0]) for key, value in train_scores.items()}
            test_scores = {key: float(value[0]) for key, value in test_scores.items()}
            
            
            # Aggiunge le metriche di training, modificando il nome della colonna se è vera la condizione
            col_train = f"{name_model} (Training){suffix}"  # Usa suffix qui per il nome
            
            df_data[f"{col_train}"] = [
                train_scores["train_accuracy"],
                train_scores["train_loss"],
                train_scores["train_precision"],
                train_scores["train_recall"],
                train_scores["train_f1_score"],
                train_scores["train_auc"],
            ]

            # Aggiunge le metriche di training, modificando il nome della colonna se è vera la condizione
            col_test = f"{name_model} (Test){suffix}"  # Usa suffix qui per il nome
            
            df_data[f"{col_test}"] = [
                test_scores["test_accuracy"],
                test_scores["test_loss"],
                test_scores["test_precision"],
                test_scores["test_recall"],
                test_scores["test_f1_score"],
                test_scores["test_auc"],
            ]
        
        except Exception as e:
            print(f"    Errore nell'elaborazione di {model_name}: {e}")

    # Creazione del DataFrame
    df_performances = pd.DataFrame(df_data)

    # Crea un'immagine della tabella
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.axis('off')
    
    # Aggiunta del titolo
    #title = f"DL Models performances for Exp Conditions: {condition}, EEG data: {data_type}, Subject: {subfolder}"
    title = f"DL Models performances for Exp Conditions: {condition}, EEG data: {data_type}"
    ax.set_title(title, fontsize=12, fontweight="bold", pad=20)

    tabla = table(ax, df_performances, loc='center', colWidths=[0.2] * len(df_performances.columns))
    tabla.auto_set_font_size(True)
    tabla.set_fontsize(10)
    tabla.scale(2, 2)

    # Evidenzia i nomi delle colonne
    for key, cell in tabla.get_celld().items():
        if key[0] == 0:
            cell.set_text_props(weight='bold')

    # Salva l'immagine della tabella
    path = os.path.join(base_folder, condition, data_type, subfolder)
    file_name = f"models_performances_{condition}_{data_type}_{subfolder}.png"
    img_file_path = os.path.join(path, file_name)
    fig.savefig(img_file_path, bbox_inches='tight', dpi=300)
    plt.close(fig)

    print(f"\nTabella dei dati di \033[1m{condition}_{data_type}_{subfolder}\033[0m salvati in: \n\033[1m{img_file_path}\033[0m")